#### Load packages

In [ ]:
import os
import io
import re
import gzip
import random
import warnings

import pandas as pd

import numpy as np

from varname import nameof

import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
from matplotlib.cm import ScalarMappable

import statsmodels.stats.multitest as smm

import matplotlib as matpl
matpl.rcParams['svg.fonttype'] = 'none'

import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import fisher_exact
from scipy.stats import chi2_contingency
from scipy.stats import binomtest

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio


import seaborn as sns

from venn import venn


rcParams['font.family'] = 'Arial'

%matplotlib inline

#### General stuff and fuctions for later

In [ ]:
enseble_ids_to_genes = {"ENSGWIG00000012019":"znf236", 
                        "ENSGWIG00000023771":"il15ra",
                       "ENSGWIG00000021087":"ENSGWIG00000021087",
                       "ENSGWIG00000022958":"LOC114465680",
                       "ENSGWIG00000006263":"SYNM",
                       "ENSGWIG00000024862":"ENSGWIG00000024862",
                       "ENSGWIG00000018693": '"adam12-like" nc-RNA',
                        "ENSGWIG00000018697":"IDO"}

In [ ]:
def get_5percent_outliers(input_column):
    dA_outliers_threshold = dA_windows[input_column].quantile(0.975)
    #dA_hofrichteri_orientalis_outliers_threshold
    dA_outliers = dA_windows.loc[dA_windows[input_column] >= dA_outliers_threshold]
    return(dA_outliers) 

In [ ]:
def extract_deduplicated_gene_from_snpeff_list(data_frame_column, outfilename):
    
    extracted_genes = []

    for i in data_frame_column.drop_duplicates(): # This regex extracts this "si:dkey" genes that have a "-" symbol
        if  re.match(r"\Asi:.+?-.+?(?=-)", i): 
            i1 = re.search(r"\Asi:.+?-.+?(?=-)", i).group(0)
            i0 = re.split(r"\Asi:.+?-.+?(?=-)", i)[0]
            extracted_genes.append(i0)
            extracted_genes.append(i1)

        elif "-si:" in i:
            i1 = re.search(r"-si:(.*)", i).group(0)[1:]
            i0 = re.split(r"-si:(.*)", i)[0]
            extracted_genes.append(i0)
            extracted_genes.append(i1)

        elif "si:" not in i and len(i.split("-")) > 1:
            i1 = i.split("-")[1]
            i0 = i.split("-")[0]
            extracted_genes.append(i0)
            extracted_genes.append(i1)

        else:
            extracted_genes.append(i)
    
    extracted_genes_deduplicated = pd.DataFrame(extracted_genes).drop_duplicates().stack().tolist()
    
    extracted_genes_deduplicated_updated = [x if x not in enseble_ids_to_genes else enseble_ids_to_genes[x] for x in extracted_genes_deduplicated]
    
    fhout = str(outfilename)
    
    with open(fhout, "w") as outfh:
        for i in extracted_genes_deduplicated_updated:
            outfh.write(i+"\n")
            
    print("There are private fixed alleles associated with {} genes in {}.".format(len(extracted_genes_deduplicated_updated), fhout))
    
    return(extracted_genes_deduplicated_updated)

In [ ]:
def extract_MK(input_df, AF_colname_species1, AF_colname_species2, num_bootstraps=1000):
    import numpy as np
    import pandas as pd
    from scipy.stats import fisher_exact, chi2_contingency

    # Subset to the two species
    AF_subset = input_df[["CHROM", "POS", "cat", "type", "gene_snpeff", str(AF_colname_species1), str(AF_colname_species2)]]
    AF_subset[str(AF_colname_species1)] = AF_subset[str(AF_colname_species1)].astype(float)
    AF_subset[str(AF_colname_species2)] = AF_subset[str(AF_colname_species2)].astype(float)

    # Polymorphisms (0 < AF < 1)
    polymorphisms_df = AF_subset[((AF_subset[str(AF_colname_species1)] > 0) & (AF_subset[str(AF_colname_species1)] < 1)) |
                                 ((AF_subset[str(AF_colname_species2)] > 0) & (AF_subset[str(AF_colname_species2)] < 1))]

    polymorphisms = polymorphisms_df.groupby(["gene_snpeff", "type"]).size().unstack(fill_value=0).reset_index()
    polymorphisms.columns = ['gene_snpeff', 'Pn_nonsynonymous_polymorphisms', 'Ps_synonymous_polymorphisms']

    # Fixed variants (AF == 0 or 1)
    fixed_variants = AF_subset[((AF_subset[str(AF_colname_species1)] == 0) & (AF_subset[str(AF_colname_species2)] == 1)) |
                               ((AF_subset[str(AF_colname_species1)] == 1) & (AF_subset[str(AF_colname_species2)] == 0))]

    fixed_nonsyn_syn = fixed_variants.groupby(["gene_snpeff", "type"]).size().unstack(fill_value=0).reset_index()
    fixed_nonsyn_syn.columns = ['gene_snpeff', 'Dn_nonsynonymous_fixed', 'Ds_synonymous_fixed']

    # Merge and compute MK statistics
    mk_table_gene_wise = polymorphisms.merge(fixed_nonsyn_syn, on='gene_snpeff')

    mk_table_gene_wise["alpha"] = 1 - (mk_table_gene_wise["Ds_synonymous_fixed"] * mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"]) / \
                                  (mk_table_gene_wise["Dn_nonsynonymous_fixed"] * mk_table_gene_wise["Ps_synonymous_polymorphisms"])

    mk_table_gene_wise["NI"] = (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] / mk_table_gene_wise["Ps_synonymous_polymorphisms"]) / \
                               (mk_table_gene_wise["Dn_nonsynonymous_fixed"] / mk_table_gene_wise["Ds_synonymous_fixed"])

    mk_table_gene_wise["DoS"] = (mk_table_gene_wise["Dn_nonsynonymous_fixed"] / (mk_table_gene_wise["Dn_nonsynonymous_fixed"] + mk_table_gene_wise["Ds_synonymous_fixed"])) - \
                                (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] / (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] + mk_table_gene_wise["Ps_synonymous_polymorphisms"]))

    # Fisher's exact test
    pvalues_exact = [fisher_exact([[row['Dn_nonsynonymous_fixed'], row['Ds_synonymous_fixed']],
                                   [row['Pn_nonsynonymous_polymorphisms'], row['Ps_synonymous_polymorphisms']]])[1]
                     for _, row in mk_table_gene_wise.iterrows()]
    mk_table_gene_wise['pvalue_exact'] = pvalues_exact

    # Chi-squared test
    pvalues_chi2 = []
    for _, row in mk_table_gene_wise.iterrows():
        obs = np.array([[row['Dn_nonsynonymous_fixed']+0.1, row['Ds_synonymous_fixed']+0.1],
                        [row['Pn_nonsynonymous_polymorphisms']+0.1, row['Ps_synonymous_polymorphisms']+0.1]])
        chi2, p, dof, expected = chi2_contingency(obs)
        pvalues_chi2.append(p)
    mk_table_gene_wise['pvalue_chi2'] = pvalues_chi2

    # Genome-wide values
    alpha_genome = 1 - (mk_table_gene_wise["Ds_synonymous_fixed"].sum() * mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"].sum()) / \
                          (mk_table_gene_wise["Dn_nonsynonymous_fixed"].sum() * mk_table_gene_wise["Ps_synonymous_polymorphisms"].sum())

    NI_genome = (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"].sum() / mk_table_gene_wise["Ps_synonymous_polymorphisms"].sum()) / \
                (mk_table_gene_wise["Dn_nonsynonymous_fixed"].sum() / mk_table_gene_wise["Ds_synonymous_fixed"].sum())

    NITG_genome = (mk_table_gene_wise["Ds_synonymous_fixed"] * mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] / \
                   (mk_table_gene_wise["Ps_synonymous_polymorphisms"] + mk_table_gene_wise["Ds_synonymous_fixed"])).sum() / \
                  (mk_table_gene_wise["Ps_synonymous_polymorphisms"] * mk_table_gene_wise["Dn_nonsynonymous_fixed"] / \
                   (mk_table_gene_wise["Ps_synonymous_polymorphisms"] + mk_table_gene_wise["Ds_synonymous_fixed"])).sum()

    
    NI_genome_log_ratio = np.log2(NI_genome)
    NI_genome_pval = 2 * min(np.mean(NI_genome_log_ratio > 0), np.mean(NI_genome_log_ratio < 0))
    
    # Parametric bootstrap for NITG under neutrality (null: NITG = 1)
    bootstrapped_nitg = []
    for _ in range(num_bootstraps):
        shuffled = mk_table_gene_wise.copy()
        shuffled["Pn_nonsynonymous_polymorphisms"] = np.random.permutation(shuffled["Pn_nonsynonymous_polymorphisms"].values)
        shuffled["Dn_nonsynonymous_fixed"] = np.random.permutation(shuffled["Dn_nonsynonymous_fixed"].values)
        numerator = (shuffled["Ds_synonymous_fixed"] * shuffled["Pn_nonsynonymous_polymorphisms"] /
                     (shuffled["Ps_synonymous_polymorphisms"] + shuffled["Ds_synonymous_fixed"])).sum()
        denominator = (shuffled["Ps_synonymous_polymorphisms"] * shuffled["Dn_nonsynonymous_fixed"] /
                       (shuffled["Ps_synonymous_polymorphisms"] + shuffled["Ds_synonymous_fixed"])).sum()
        nitg_boot = numerator / denominator if denominator != 0 else np.nan
        bootstrapped_nitg.append(nitg_boot)

    bootstrapped_nitg = np.array(bootstrapped_nitg)
    bootstrapped_nitg = bootstrapped_nitg[~np.isnan(bootstrapped_nitg)]
    p_value_nitg_parametric = np.mean(np.abs(bootstrapped_nitg - 1) >= np.abs(NITG_genome - 1))


    # Calculate 95% CI using percentiles
    #nitg_CI_lower = np.percentile(bootstrapped_nitg, 2.5)
    #nitg_CI_upper = np.percentile(bootstrapped_nitg, 97.5)
    
    #nitg_distribution, nitg_ci_low, nitg_ci_high, nitg_pval = bootstrap_nitg(mk_table_gene_wise)

    whole_genome_values = {
        "alpha": alpha_genome,
        "NI": NI_genome,
        "NI_pval": NI_genome_pval,
        "NITG": NITG_genome,
        #"NITG_CI": (nitg_ci_low, nitg_ci_high),
        #"NITG_pval": nitg_pval,
        "pvalue_NITG_parametric": p_value_nitg_parametric,
        #"NITG_CI_parametric": (nitg_CI_lower, nitg_CI_upper),
    }


    return mk_table_gene_wise, whole_genome_values

In [ ]:
def calculate_p_value_permutation(row):#, num_permutations=10):
    # Create an empty array to store the dos values
    dos_permuted = np.empty(num_permutations)
    
    # Generate permutations of the indices
    idx_fixed_perm = np.random.permutation(["Dn_nonsynonymous_fixed", "Ds_synonymous_fixed"])
    idx_poly_perm = np.random.permutation(["Pn_nonsynonymous_polymorphisms", "Ps_synonymous_polymorphisms"])
    
    permutations = 100
    for i in range(permutations):
        # Calculate the permuted dos value
        dos_permuted[i] = (row[idx_fixed_perm].loc["Dn_nonsynonymous_fixed"] / (row[idx_fixed_perm].loc["Dn_nonsynonymous_fixed"] + row[idx_fixed_perm].loc["Ds_synonymous_fixed"])) - \
                          (row[idx_poly_perm].loc["Pn_nonsynonymous_polymorphisms"] / (row[idx_poly_perm].loc["Pn_nonsynonymous_polymorphisms"] + row[idx_poly_perm].loc["Ps_synonymous_polymorphisms"]))
    
    
        
    # Calculate the p-value as the proportion of permuted dos values that are greater than the observed dos value
    p_value = (dos_permuted > row["DoS"]).sum() / num_permutations
    
    #print("{} has a pvalue of {}".format(row, p_value))
    
    return p_value

In [ ]:
def read_vcf(path):
    with gzip.open(path, 'rt') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t').rename(columns={'#CHROM': 'CHROM'})

In [ ]:
def extract_depth(cell):
    values = cell.split(':')
    if len(values) == 3:
        return values[1]  # Extract the second value
    elif len(values) > 3:
        return values[2]  # Extract the third value
    else:
        return None  # Return None for other cases

In [ ]:
def replace_values_iupac(cell_value):
    IUPAC_code = {"A": ["A/A", "A|A"], "T": ["T/T", "T|T"],
              "C": ["C/C", "C|C"], "G": ["G/G", "G|G"],
              "M": ["A/C", "A|C", "C/A", "C|A"],
              "R": ["A/G", "A|G", "G/A", "G|A"],
              "W": ["A/T", "A|T", "T/A", "T|A"],
              "S": ["C/G", "C|G", "G/C", "G|C"],
              "Y": ["C/T", "C|T", "T/C", "T|C"],
              "K": ["G/T", "G|T", "T/G", "T|G"]}
    for key, value in IUPAC_code.items():
        if cell_value in value:
            return key
    return cell_value

In [ ]:
def replace_values(cell_value, ref, alt):
    if cell_value == '0|0':
        return f'{ref}|{ref}'
    elif cell_value == '0|1' or cell_value == '1|0':
        return f'{ref}|{alt}' if cell_value == '0|1' else f'{alt}|{ref}'
    elif cell_value == '1|1':
        return f'{alt}|{alt}'
    else:
        return cell_value

In [ ]:
def order_columns_by_prefix(dataframe, prefixes):
    """
    Orders DataFrame columns based on prefixes.

    Parameters:
    - dataframe (pd.DataFrame): The DataFrame whose columns need to be ordered.
    - prefixes (list): A list of prefixes. Columns starting with these prefixes will be ordered first.

    Returns:
    - pd.DataFrame: DataFrame with columns ordered based on prefixes.
    """
    # Get column names starting with each prefix and sort them alphabetically
    ordered_columns = []
    for prefix in prefixes:
        prefix_columns = [col for col in dataframe.columns if col.startswith(prefix)]
        prefix_columns.sort()
        ordered_columns.extend(prefix_columns)

    # Get the remaining columns that do not start with any prefix
    remaining_columns = [col for col in dataframe.columns if not any(col.startswith(prefix) for prefix in prefixes)]
    remaining_columns.sort()

    # Concatenate ordered columns and remaining columns
    ordered_columns.extend(remaining_columns)

    # Reindex the DataFrame columns based on the ordered list
    ordered_dataframe = dataframe.reindex(columns=ordered_columns)

    return ordered_dataframe

In [ ]:
def extract_significant_genes_exact(df):
    df = df[(df["pvalue_exact"] <= 0.05) & (df["NI"] < 1)]
    return(df)

def extract_significant_genes_exact_list(df):
    df = df[(df["pvalue_exact"] <= 0.05) & (df["NI"] < 1)]
    return(df["gene_snpeff"].to_list())

def extract_significant_genes_chi2(df):
    df = df[df["pvalue_chi2"] <= 0.05 & (df["NI"] < 1)]
    return(list(df["gene_snpeff"]))

def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [ ]:
def run_permutation_variants_test(subset_df, full_df, column='type', n_permutations=1000, seed=42):
    np.random.seed(seed)
    
    subset_size = len(subset_df)
    observed_counts = subset_df[column].value_counts()
    unique_types = full_df[column].unique()

    # Store permuted counts
    permuted_counts = {snp_type: [] for snp_type in unique_types}

    full_types = full_df[column].values

    for _ in range(n_permutations):
        sampled = np.random.choice(full_types, size=subset_size, replace=False)
        sampled_counts = pd.Series(sampled).value_counts()
        for snp_type in unique_types:
            permuted_counts[snp_type].append(sampled_counts.get(snp_type, 0))

    # Calculate empirical p-values
    pvals = {}
    for snp_type in unique_types:
        obs = observed_counts.get(snp_type, 0)
        perm = np.array(permuted_counts[snp_type])
        # One-sided p-value (enrichment)
        pval = np.mean(perm >= obs)
        pvals[snp_type] = pval

    # Assemble results
    results = pd.DataFrame({
        'type': list(pvals.keys()),
        'observed_count': [observed_counts.get(t, 0) for t in pvals.keys()],
        'mean_permuted': [np.mean(permuted_counts[t]) for t in pvals.keys()],
        'pval_perm': list(pvals.values())
    })

    # Adjust for multiple testing
    results['pval_perm_adj'] = smm.multipletests(results['pval_perm'], method='fdr_bh')[1]
    return results.sort_values('pval_perm_adj')


In [ ]:
def enrichment_test_variants(subset_counts, background_counts, all_types):
    results = []
    total_subset = subset_counts.sum()
    total_background = background_counts.sum()

    for t in all_types:
        a = subset_counts.get(t, 0)  # Count in subset
        b = background_counts.get(t, 0) - a  # In background but not subset
        c = total_subset - a  # Other types in subset
        d = total_background - a - b - c  # Everything else

        contingency = [[a, b], [c, d]]

        # Fisher's exact test (use 'greater' if you expect enrichment)
        oddsratio, p = fisher_exact(contingency, alternative='greater')

        results.append((t, a, b, c, d, oddsratio, p))

    # Convert to DataFrame
    res_df = pd.DataFrame(results, columns=['type', 'a', 'b', 'c', 'd', 'oddsratio', 'pval'])

    # Adjust p-values for multiple testing (FDR)
    res_df['pval_adj'] = smm.multipletests(res_df['pval'], method='fdr_bh')[1]

    return res_df.sort_values('pval_adj')


# 1) Window based Scans

## 1.1 Nucleotide divergence (pi) and Net-between group mean divergence (Da)

Windows were calculated along the genome in sliding windows of 20,000 bp.

#### load dataframe

In [ ]:
dA_windows = pd.read_csv("raw_data/all_sites.tif1.sft1.all_sites.pass.snps_indels.biallelic.w20000.m1000.s10000.Fst.Dxy.pi.Da.tsv.gz",
                        sep = "\t")
dA_windows

### 1.1.1 Nucleotide divergence (pi) plot

In [ ]:

# Select the columns you want to plot
columns_to_plot = ["pi_willdenowi","pi_hofrichteri","pi_orientalis","pi_pigra","pi_adriatica"]


# Create a box plot of the selected columns
sns.set_style('ticks')

fig, ax = plt.subplots(figsize=(8, 8))

dA_windows[columns_to_plot].boxplot(ax=ax, whis=1.5)
ax.set_title('Pi in 20.000 bp windows for each species', fontsize=12)
ax.set_xlabel('Species', fontsize=12)
ax.set_ylabel('pi', fontsize=12)
ax.tick_params(axis='both', labelsize=8)
# Turns off grid on the left Axis.
ax.grid(False)

plt.tight_layout()
plt.savefig('results/window_pi.svg', format='svg')


### 1.1.2 Net-between group mean divergence (Da)

#### extract upper 5% thresholds for each comparison

In [ ]:
column_names = list(dA_windows.columns.values)
column_names_dA = [i for i in column_names if i.startswith("dA")]

dA_dict = {}

for i in column_names_dA:
    dA_dict[i] = get_5percent_outliers(i)
    
for k,v in dA_dict.items():
    print("The pair {} has {} upper five percent outlier regions.".format(k, len(v)))

#### pair 1: pigra (slender1) vs. willdenowi (stout1)

In [ ]:
dA_pigra_willdenowi_outliers_threshold = dA_windows.dA_pigra_willdenowi.quantile(0.975)

dA_pigra_willdenowi_outliers = dA_windows.loc[dA_windows['dA_pigra_willdenowi'] >= dA_pigra_willdenowi_outliers_threshold]
dA_pigra_willdenowi_outliers

#### pair 2: hofrichteri (slender2) vs. orientalis (stout2)

In [ ]:
dA_hofrichteri_orientalis_outliers_threshold = dA_windows["dA_hofrichteri_orientalis"].quantile(0.975)
#dA_hofrichteri_orientalis_outliers_threshold
dA_hofrichteri_orientalis_outliers = dA_windows.loc[dA_windows['dA_hofrichteri_orientalis'] >= dA_hofrichteri_orientalis_outliers_threshold]
dA_hofrichteri_orientalis_outliers

In [ ]:
### delete later
#subhead = ["scaffold", "start", "end"]
#dA_hofrichteri_orientalis_outliers.to_csv('../../../Gouania_crete_pop_genomics/analysis_local/results/Genome_scans_from_other_paper/windows_old_VCF/DA_outliers_hofrichteri_orientalis.tsv', 
#                                  columns = subhead,
#                                  sep = "\t", index=False, header=False)


#### plot

In [ ]:
dA_windows['mid_new'] = 10000 * pd.Series(range(len(dA_windows)))

dA_outliers_vis = dA_windows[(dA_windows['dA_hofrichteri_orientalis'] >= dA_hofrichteri_orientalis_outliers_threshold) &
                            (dA_windows['dA_pigra_willdenowi'] >= dA_pigra_willdenowi_outliers_threshold)]

print("There are {} common parallel outliers in the dataset for window wise dA calculations.".format(len(dA_outliers_vis)))

# Create a new figure and axis object with figsize
fig, ax = plt.subplots(figsize=(20, 5))
#fig.subplots_adjust(left=0.01, right=0.99)

# Group the DataFrame by scaffold and iterate over the groups
for i, (scaffold, group) in enumerate(dA_windows.groupby('scaffold')):
    # Plot the mid vs dA_hofrichteri_pigra for this scaffold group
    group.plot(x='mid_new', y='dA_hofrichteri_orientalis', kind='scatter', alpha = 0.3,ax=ax, 
               label=None, color='lightgrey' if i % 2 == 0 else 'grey')

    
    
threshold = dA_hofrichteri_orientalis_outliers_threshold
ax.axhline(threshold, linestyle='--', color='red')

ax.scatter(dA_outliers_vis['mid_new'], 
           dA_outliers_vis['dA_hofrichteri_orientalis'], 
           color='orange')#, alpha = 0.2)


# Calculate the midpoint for each scaffold and store in a dictionary
scaffold_midpoints = dA_windows.groupby('scaffold')['mid_new'].mean().to_dict()

xtick_loc = []
for k,v in scaffold_midpoints.items():
    xtick_loc.append(v)
ax.set_xticks(xtick_loc)

xtick_labels = []
for k,v in scaffold_midpoints.items():
    xtick_labels.append(k)

ax.set_xticklabels(xtick_labels)

ax.set_xlabel('chromosome', fontsize=20, fontweight='bold', fontname='Arial')
ax.set_ylabel('dA hofrichteri vs. orientalis', fontsize=20, fontweight='bold',fontname='Arial')
ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=6)

plt.savefig("results/dA_scans_pigraVSwilldenowi.pdf")


# Create a new figure and axis object with figsize
fig, ax = plt.subplots(figsize=(20, 5))
#fig.subplots_adjust(left=0.01, right=0.99)

# Group the DataFrame by scaffold and iterate over the groups
for i, (scaffold, group) in enumerate(dA_windows.groupby('scaffold')):
    # Plot the mid vs dA_hofrichteri_pigra for this scaffold group
    group.plot(x='mid_new', y='dA_pigra_willdenowi', kind='scatter', alpha = 0.3,ax=ax, 
               label=None, color='lightgrey' if i % 2 == 0 else 'grey')

    
    
threshold = dA_pigra_willdenowi_outliers_threshold
ax.axhline(threshold, linestyle='--', color='red')

# color points above the threshold blue
ax.scatter(dA_outliers_vis['mid_new'], 
           dA_outliers_vis['dA_pigra_willdenowi'], 
           color='orange')#, #alpha = )


    
# Calculate the midpoint for each scaffold and store in a dictionary
scaffold_midpoints = dA_windows.groupby('scaffold')['mid_new'].mean().to_dict()

xtick_loc = []
for k,v in scaffold_midpoints.items():
    xtick_loc.append(v)
ax.set_xticks(xtick_loc)

xtick_labels = []
for k,v in scaffold_midpoints.items():
    xtick_labels.append(k)

ax.set_xticklabels(xtick_labels)

ax.set_xlabel('chromosome', fontsize=20, fontweight='bold',fontname='Arial')
ax.set_ylabel('dA pigra vs. willdenowi', fontsize=20, fontweight='bold',fontname='Arial')
ax.tick_params(axis='both', which='major', labelsize=15, width=2, length=6)

# save plot as pdf

plt.savefig("results/dA_scans_hofrichteriVSorientalis.pdf")

# Show the plot
plt.show()

subhead = ["scaffold", "start", "end"]
dA_outliers_vis.to_csv('results/OUTLIERS.tif1.sft1.all_sites.pass.snps_indels.biallelic.w20000.m1000.s10000.Fst.Dxy.pi.Da.tsv', 
                                  columns = subhead,
                                  sep = "\t", index=False, header=False)

dA_outliers_vis

NOTE: Gene infos extracted with snpsift can be found in this Snakefile: /scratch/antwerpen/grp/asvardal/projects/clingfishes/projects/GouaniaPhylogenomics/snakemake/outlier_detection_extraction/DA_window_outliers

## 1.2 Composite selection signals (CSS)

#### load css results df

In [ ]:
css_windows = pd.read_csv("raw_data/css_step20000_CSSm.100000perm.txt",sep = "\t")
css_windows

#### extract significant values

In [ ]:
css_windows_significant = css_windows[css_windows["qvalue"] <= 0.05]
print("There are {} significant outliers obtained from 10000 permutations for css.".format(len(css_windows_significant)))

# 2) Site wise scans (allele frequency based scans)

## 2.1 Allele frequency scans

In [ ]:
AF_df = pd.read_csv("raw_data/AF_Gouania.tsv.gz", sep = "\t")#, compression='gzip')
#AF_df

##### Overview

In [ ]:
number_snps_indels =  AF_df.groupby(['cat'])["cat"].count()
number_snps_indels.plot(kind='pie', autopct='%1.1f%%')
plt.rcParams['font.family'] = 'Arial'
plt.axis('equal')
plt.show()

In [ ]:
number_types_all =  AF_df.groupby(['type'])["type"].count()
number_types_all = pd.DataFrame(number_types_all)
number_types_all.columns = ["counts"]
number_types_all = number_types_all.reset_index()

# Set the threshold value for the count
threshold = 100000

# Identify the rows to merge as "others"
merged_rows = number_types_all[number_types_all['counts'] < threshold]
merged_count = merged_rows['counts'].sum()

# Create a new DataFrame where the rows are merged
number_types_subset = pd.concat([number_types_all[number_types_all['counts'] >= threshold],
                    pd.DataFrame({'type': ['others'], 'counts': [merged_count]})])


number_types_subset['type'] = number_types_subset['type'].replace(['3_prime_UTR_variant', '5_prime_UTR_variant'], "3' & 5'  UTR variant")
number_types_subset['type'] = number_types_subset['type'].str.replace("_", " ") 


# Group the dataframe by the new category names
number_types_subset = number_types_subset.groupby('type')['counts'].sum().reset_index()


# Sort the dataframe by counts
number_types_subset_sorted = number_types_subset.sort_values(by='counts', ascending=False)

# Plot the counts in a pie chart
fig, ax = plt.subplots()
wedges, texts, autotexts = ax.pie(number_types_subset_sorted['counts'], 
                                  labels=number_types_subset_sorted['type'], 
                                  autopct='%1.1f%%')
ax.set_title('Count by Type')

# Add percentage labels
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(10)

# Position the legend outside of the plot
ax.legend(wedges, number_types_subset_sorted['type'], title='Type', loc='center left', bbox_to_anchor=(1, 0.5))

# Set the font to Arial
plt.rcParams['font.family'] = 'Arial'

# Display the plot
plt.savefig('results/All_variants_overview.pdf')
plt.show()


### 2.1.1 Fixed alleles in genes for species and groups (slender, stout) 

##### fixed for both slender (1) and stout (0)

In [ ]:
fixed_slender_1_stout_0 = AF_df[#(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 1)]
fixed_slender_1_stout_0_genes = extract_deduplicated_gene_from_snpeff_list(fixed_slender_1_stout_0["gene_snpeff"], "results/outliers/AF_outliers/AF_fixed_slender_1_stout_0_snpeff_two_comp.txt")
fixed_slender_1_stout_0

##### fixed for both stout (1) and slender (0)

In [ ]:
fixed_stout_1_stout_0 = AF_df[#(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 0)]
fixed_stout_1_stout_0_genes = extract_deduplicated_gene_from_snpeff_list(fixed_stout_1_stout_0["gene_snpeff"], "results/outliers/AF_outliers/AF_fixed_stout_1_slender_0_snpeff_two_comp.txt")
fixed_stout_1_stout_0

##### independently fixed variants for pigra

In [ ]:
fixed_pigra_1 = AF_df[#(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 0)]


fixed_pigra_0 = AF_df[#(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 1)]

fixed_pigra = pd.concat([fixed_pigra_0, fixed_pigra_1])

fixed_pigra_genes = extract_deduplicated_gene_from_snpeff_list(fixed_pigra["gene_snpeff"], 
                                                               "results/outliers/AF_outliers/AF_pigra_fixed_snpeff_two_comp.txt")

print("There are {} fixed snps/indels with genotype 1|1 for pigra.".format(len(fixed_pigra_1)))
print("There are {} fixed snps/indels with genotype 0|0 for pigra.".format(len(fixed_pigra_0)))

#fixed_pigra

In [ ]:
background_pigra = fixed_pigra 
#fixed_pigra_permut = run_permutation_variants_test(fixed_pigra, AF_df)
fixed_pigra_permut

In [ ]:
fixed_pigra_permut.to_excel('results/divergently_fixed_variants_permut_pigra.xlsx', index=False)

##### independently fixed variants for hofrichteri

In [ ]:
fixed_hofrichteri_1 = AF_df[#(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 1)]

fixed_hofrichteri_0 = AF_df[#(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 0)]

fixed_hofrichteri = pd.concat([fixed_hofrichteri_0, fixed_hofrichteri_1])                            
fixed_hofrichteri_genes = extract_deduplicated_gene_from_snpeff_list(fixed_hofrichteri["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/AF_hofrichteri_fixed_snpeff_two_comp.txt")

print("There are {} fixed snps/indels with genotype 1|1 for hofrichteri.".format(len(fixed_hofrichteri_1)))
print("There are {} fixed snps/indels with genotype 0|0 for hofrichteri.".format(len(fixed_hofrichteri_0)))
                            
#fixed_hofrichteri

In [ ]:
#fixed_hofrichteri_permut = run_permutation_variants_test(fixed_hofrichteri, AF_df)
fixed_hofrichteri_permut

In [ ]:
fixed_hofrichteri_permut.to_excel('results/divergently_fixed_variants_permut_hofrichteri.xlsx', index=False)

##### independently fixed variants for willdenowi

In [ ]:
fixed_willdenowi_1 = AF_df[#(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 0)]

fixed_willdenowi_0 = AF_df[#(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 1)]

fixed_willdenowi = pd.concat([fixed_willdenowi_1, fixed_willdenowi_0])                            
fixed_willdenowi_genes = extract_deduplicated_gene_from_snpeff_list(fixed_willdenowi["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/AF_willdenowi_fixed_snpeff_two_comp.txt")

print("There are {} fixed snps/indels with genotype 1|1 for willdenowi.".format(len(fixed_willdenowi_1)))
print("There are {} fixed snps/indels with genotype 0|0 for willdenowi.".format(len(fixed_willdenowi_0)))
                            
#fixed_willdenowi

In [ ]:
#fixed_willdenowi_permut = run_permutation_variants_test(fixed_willdenowi, AF_df)
fixed_willdenowi_permut

In [ ]:
fixed_willdenowi_permut.to_excel('results/divergently_fixed_variants_permut_willdenowi.xlsx', index=False)

##### independently fixed variants for orientalis

In [ ]:
fixed_orientalis_1 = AF_df[#(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 0)]

fixed_orientalis_0 = AF_df[#(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 1)]


fixed_orientalis = pd.concat([fixed_orientalis_1, fixed_orientalis_0])                            
fixed_orientalis_genes = extract_deduplicated_gene_from_snpeff_list(fixed_orientalis["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/AF_orientalis_fixed_snpeff_two_comp.txt")

print("There are {} fixed snps/indels with genotype 1|1 for orientalis.".format(len(fixed_orientalis_1)))
print("There are {} fixed snps/indels with genotype 0|0 for orienatlis.".format(len(fixed_orientalis_0)))

#fixed_orientalis

In [ ]:
#fixed_orientalis_permut = run_permutation_variants_test(fixed_orientalis, AF_df)
fixed_orientalis_permut

In [ ]:
fixed_orientalis_permut.to_excel('results/divergently_fixed_variants_permut_orientalis.xlsx', index=False)

##### missense variants overview

In [ ]:
def extract_missense_variants(df):
    missense_df = df[df["type"] == "missense_variant"]
    return missense_df

fixed_orientalis_missense = extract_missense_variants(fixed_orientalis)
fixed_hofrichteri_missense = extract_missense_variants(fixed_hofrichteri)
fixed_pigra_missense = extract_missense_variants(fixed_pigra)
fixed_willdenowi_missense = extract_missense_variants(fixed_willdenowi)

fixed_orientalis_missense_genes = extract_deduplicated_gene_from_snpeff_list(fixed_orientalis_missense["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/Missense_AF_orientalis_fixed_snpeff_two_comp.txt")

fixed_hofrichteri_missense_genes = extract_deduplicated_gene_from_snpeff_list(fixed_hofrichteri_missense["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/Missense_AF_hofrichteri_fixed_snpeff_two_comp.txt")

fixed_pigra_missense_genes = extract_deduplicated_gene_from_snpeff_list(fixed_pigra_missense["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/Missense_AF_pigra_fixed_snpeff_two_comp.txt")

fixed_willdenowi_missense_genes = extract_deduplicated_gene_from_snpeff_list(fixed_willdenowi_missense["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/Missense_AF_willdenowi_fixed_snpeff_two_comp.txt")



In [ ]:
missense_count = (fixed_hofrichteri['type'] == 'missense_variant').sum()
synonymous_count = (fixed_hofrichteri['type'] == 'synonymous_variant').sum()

# Avoid division by zero
if synonymous_count == 0:
    ratio = np.inf  # or np.nan, depending on how you want to handle it
else:
    ratio = missense_count / synonymous_count

print("Missense:", missense_count)
print("Synonymous:", synonymous_count)
print("Missense/Synonymous ratio:", ratio)

In [ ]:
missense_count = (fixed_pigra['type'] == 'missense_variant').sum()
synonymous_count = (fixed_pigra['type'] == 'synonymous_variant').sum()

# Avoid division by zero
if synonymous_count == 0:
    ratio = np.inf  # or np.nan, depending on how you want to handle it
else:
    ratio = missense_count / synonymous_count

print("Missense:", missense_count)
print("Synonymous:", synonymous_count)
print("Missense/Synonymous ratio:", ratio)

In [ ]:
missense_count = (fixed_orientalis['type'] == 'missense_variant').sum()
synonymous_count = (fixed_orientalis['type'] == 'synonymous_variant').sum()

# Avoid division by zero
if synonymous_count == 0:
    ratio = np.inf  # or np.nan, depending on how you want to handle it
else:
    ratio = missense_count / synonymous_count

print("Missense:", missense_count)
print("Synonymous:", synonymous_count)
print("Missense/Synonymous ratio:", ratio)

In [ ]:
missense_count = (fixed_willdenowi['type'] == 'missense_variant').sum()
synonymous_count = (fixed_willdenowi['type'] == 'synonymous_variant').sum()

# Avoid division by zero
if synonymous_count == 0:
    ratio = np.inf  # or np.nan, depending on how you want to handle it
else:
    ratio = missense_count / synonymous_count

print("Missense:", missense_count)
print("Synonymous:", synonymous_count)
print("Missense/Synonymous ratio:", ratio)

In [ ]:
fixed_missense_genes_venn = {#'fixed adriatica':set(fixed_adriatica_genes), 
               'fixed missense orientalis': set(fixed_orientalis_missense_genes),
               'fixed willdenowi': set(fixed_willdenowi_missense_genes),
               'fixed hofrichteri': set(fixed_hofrichteri_missense_genes),
               'fixed pigra': set(fixed_pigra_missense_genes)
                    }

ax3 = venn(fixed_missense_genes_venn)
ax3.set_title('Fixed missense genes in all species')

plt.savefig('results/MIssense_venn_all-in-all_two_comp.svg', format='svg')

In [ ]:
# Extract sets
orientalis = fixed_missense_genes_venn['fixed missense orientalis']
willdenowi = fixed_missense_genes_venn['fixed willdenowi']
hofrichteri = fixed_missense_genes_venn['fixed hofrichteri']
pigra = fixed_missense_genes_venn['fixed pigra']

# All sets for exclusion
all_sets = [orientalis, willdenowi, hofrichteri, pigra]

# 1. Only shared between hofrichteri and pigra
hof_pigra_shared = hofrichteri & pigra
hof_pigra_exclusive = hof_pigra_shared - (orientalis | willdenowi)

# 2. Only shared between orientalis and willdenowi
ori_will_shared = orientalis & willdenowi
ori_will_exclusive = ori_will_shared - (hofrichteri | pigra)

# Union of the two exclusive sets
final_result = hof_pigra_exclusive | ori_will_exclusive

# Optional: print or save

#final_result


##### summary plot independently fixed snps

In [ ]:
# Assuming fixed_pigra, fixed_willdenowi, fixed_orientalis, and fixed_hofrichteri are defined lists
# Also assuming fixed_pigra_genes, fixed_willdenowi_genes, fixed_orientalis_genes, and fixed_hofrichteri_genes are defined lists
nsites_pigra = len(fixed_pigra)
nsites_willdenowi = len(fixed_willdenowi)
nsites_orientalis = len(fixed_orientalis)
nsites_hofrichteri = len(fixed_hofrichteri)

ngenes_pigra = len(fixed_pigra_genes)
ngenes_willdenowi = len(fixed_willdenowi_genes)
ngenes_orientalis = len(fixed_orientalis_genes)
ngenes_hofrichteri = len(fixed_hofrichteri_genes)

total_sites = nsites_pigra + nsites_willdenowi + nsites_orientalis + nsites_hofrichteri

# Sort the categories and counts
sorted_categories = ['orientalis', 'willdenowi', 'hofrichteri', 'pigra']
sorted_counts_sites = [nsites_orientalis, nsites_willdenowi, nsites_hofrichteri, nsites_pigra]
sorted_counts_genes = [ngenes_orientalis, ngenes_willdenowi, ngenes_hofrichteri, ngenes_pigra]

x = np.arange(len(sorted_categories))  # the label locations
width = 0.25  # the width of the bars

fig, ax1 = plt.subplots(figsize=(8, 5))

# Plot number of sites on primary y-axis
bars_sites = ax1.bar(x - width/2, sorted_counts_sites, width, color='blue', label='Number of Sites')

# Create a second y-axis for number of genes
ax2 = ax1.twinx()

# Plot number of genes on secondary y-axis
bars_genes = ax2.bar(x + width/2, sorted_counts_genes, width, color='orange', label='Number of Genes')

# Add numbers on top of each bar for sites
for bar in bars_sites:
    yval = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width() / 2.0, yval, int(yval), ha='center', va='bottom', color='blue')

# Add numbers on top of each bar for genes
for bar in bars_genes:
    yval = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width() / 2.0, yval, int(yval), ha='center', va='bottom', color='orange')

# Add labels, title, and legend
ax1.set_xlabel('Species')
ax1.set_ylabel('Number of Sites', color='blue')
ax2.set_ylabel('Number of Genes', color='orange')
ax1.set_title('Number of independently fixed sites and genes for each species; total sites {}'.format(total_sites))
ax1.set_xticks(x)
ax1.set_xticklabels(sorted_categories)

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.savefig('results/independentl_fixed_overview.svg', format='svg')

# Show the plot
plt.show()


##### overlapping genes slender (fixed hofrichteri ∩ pigra)

In [ ]:
slender_overlap = list(set(fixed_hofrichteri_genes) & set(fixed_pigra_genes))
print("There are fixed snps/indels in {} genes that overlap in the two slender species (fixed genes hofrichteri ∩ pigra).".format(len(slender_overlap)))

with open("results/outliers/AF_outliers/AF_slender_fixed_snpeff_two_comp.txt", "w") as outfile:
    for i in slender_overlap:
        outfile.write(i + "\n")

#### overlapping stout (fixed orientalis ∩ willdenowi)

In [ ]:
stout_overlap = list(set(fixed_orientalis_genes) & set(fixed_willdenowi_genes))
print("There are fixed snps/indels in {} genes that overlap in the two slender species (fixed genes orientalis ∩ willdenowi).".format(len(stout_overlap)))

with open("results/outliers/AF_outliers/AF_stout_fixed_snpeff_two_comp.txt", "w") as outfile:
    for i in stout_overlap:
        outfile.write(i + "\n")

##### unique overlapping genes (fixed SLENDER)
(fixed_pigra ∩ fixed_hofrichteri) - fixed_orientalis - fixed_willdenowi

In [ ]:
slender_unique = list(set(slender_overlap) - set(fixed_orientalis_genes) - set(fixed_willdenowi_genes))
print("There are {} unique genes shared between the two slender types only ((fixed_pigra ∩ fixed_hofrichteri) - fixed_orientalis - fixed_willdenowi).".format(len(slender_unique)))

with open("results/outliers/AF_outliers/AF_slender_unique_snpeff_two_comp.txt", "w") as outfile:
    for i in slender_unique:
        outfile.write(i + "\n")

##### overlapping genes (fixed STOUT)
(fixed_orientalis ∩ fixed_willdenowi) - fixed_pigra - fixed_hofrichteri) 

In [ ]:
stout_unique = list(set(stout_overlap) - set(fixed_pigra_genes) - set(fixed_hofrichteri_genes))
print("There are {} unique genes shared between the two stout types only ((fixed_orientalis ∩ fixed_willdenowi) - fixed_hofrichteri - fixed_pigra).".format(len(stout_unique)))

with open("results/outliers/AF_outliers/AF_stout_unique_snpeff_two_comp.txt", "w") as outfile:
    for i in stout_unique:
        outfile.write(i + "\n")

##### fixed parallel slender and stout all in all

In [ ]:
AF_unique_all_in_all = stout_unique + slender_unique
with open("results/outliers/AF_outliers/AF_unique_all_in_all_snpeff_two_comp.txt", "w") as outfile:
    for i in AF_unique_all_in_all:
        outfile.write(i + "\n")
print(len(AF_unique_all_in_all))

In [ ]:
fixed_genes_slender_venn = {#'fixed adriatica (0)':set(fixed_adriatica_0_genes), 
               #'fixed orientalis (1)': set(fixed_orientalis_genes),
               #'fixed willdenowi (1)': set(fixed_willdenowi_genes),
               'fixed hofrichteri': set(fixed_hofrichteri_genes),
               'fixed pigra': set(fixed_pigra_genes)}

ax1 = venn(fixed_genes_slender_venn)
ax1.set_title('Fixed genes in Slender')

fixed_genes_stout_venn = {#'fixed adriatica':set(fixed_adriatica_genes), 
               'fixed orientalis': set(fixed_orientalis_genes),
               'fixed willdenowi': set(fixed_willdenowi_genes),
               #'fixed hofrichteri (1)': set(fixed_hofrichteri_genes),
               #'fixed pigra (1)': set(fixed_pigra_genes)
                    }

ax2 = venn(fixed_genes_stout_venn)
ax2.set_title('Fixed genes in Stout')

fixed_genes_venn = {#'fixed adriatica':set(fixed_adriatica_genes), 
               'fixed orientalis': set(fixed_orientalis_genes),
               'fixed willdenowi': set(fixed_willdenowi_genes),
               'fixed hofrichteri': set(fixed_hofrichteri_genes),
               'fixed pigra': set(fixed_pigra_genes)
                    }

ax3 = venn(fixed_genes_venn)
ax3.set_title('Fixed genes in all species')

plt.savefig('results/venn_all-in-all_two_comp.svg', format='svg')

#### Permutation test for over-representation of genes shared between within-morphotype comparisons compared to between morphotype comparison (Hannes' Code Snipped)

In [ ]:
#willdenowi = ['ENSGWIG00000000092', 'pdgfba', 'mgat3b', 'tab1', 'cacng2b', 'ENSGWIG00000000124', 'ENSGWIG00000000244', 'ENSGWIG00000000252', 'rac2', 'ENSGWIG00000000466', 'ENSGWIG00000000498', 'ENSGWIG00000000549', 'ENSGWIG00000000656', 'ENSGWIG00000000731', 'ENSGWIG00000000735', 'elfn2a', 'ENSGWIG00000000743', 'ENSGWIG00000000748', 'nacc1b', 'cacna1ab', '5S_rRNA', 'ENSGWIG00000002175', 'gtf3aa', 'ENSGWIG00000002246', 'ENSGWIG00000002299', 'hoga1', 'ENSGWIG00000002940', 'ENSGWIG00000002962', 'lcor', 'ENSGWIG00000003015', 'slit1b', 'arhgap19', 'dla', 'ENSGWIG00000003452', 'ENSGWIG00000003838', 'myom2a', 'si:dkey-23f9.4', 'abcc10', 'sp2', 'socs7', 'ENSGWIG00000016913', 'ENSGWIG00000016912', 'wipf2a', 'nup133', 'taf5l', 'ENSGWIG00000020683', 'ltv1', 'ENSGWIG00000020686', 'pus10', 'cct4', 'USP34', 'tomm20b', 'nid1b', 'ero1b', 'ENSGWIG00000023941', 'vps54', 'ENSGWIG00000023995', 'akap8l', 'ENSGWIG00000024289', 'lpar2b', 'ENSGWIG00000024304', 'soul5l', 'ENSGWIG00000024313', 'ENSGWIG00000024314', 'uncx4.1', 'ENSGWIG00000024316', 'ENSGWIG00000024318', 'ENSGWIG00000024322', 'ENSGWIG00000024323', 'si:ch73-12o23.1', 'ENSGWIG00000024434', 'ENSGWIG00000024441', 'ENSGWIG00000024490', 'ENSGWIG00000000123', 'ENSGWIG00000000128', 'ENSGWIG00000000132', 'ENSGWIG00000000222', 'ENSGWIG00000000237', 'ENSGWIG00000000286', 'ENSGWIG00000000303', 'ENSGWIG00000000319', 'ENSGWIG00000000323', 'mad1l1', 'ENSGWIG00000000557', 'ENSGWIG00000000562', 'ENSGWIG00000000568', 'ENSGWIG00000000666', 'RDH8', 'ENSGWIG00000000738', 'ENSGWIG00000000752', 'TRIP10', 'ENSGWIG00000001813', 'VAV1', 'UBL5', 'alkbh5', 'ENSGWIG00000002139', 'zgc:77849', 'arpc1b', 'radil', 'foxk1', 'cyp3a65', 'sdk1a', 'gna12a', 'ENSGWIG00000009632', 'ern2', 'si:dkeyp-9d4.4', 'ENSGWIG00000003229', 'ENSGWIG00000003289', 'prkcba', 'ENSGWIG00000007106', 'ENSGWIG00000007152', 'ENSGWIG00000007160', 'pdia2', 'mnd1', 'fam193a', 'sfrp2', 'ENSGWIG00000007598', 'plrg1', 'sorbs2a', 'ENSGWIG00000008298', 'tlr3', 'cyp4v8', 'mtnr1aa', 'fat1a', 'ENSGWIG00000008671', 'ENSGWIG00000008678', 'ENSGWIG00000008685', 'ENSGWIG00000008687', 'frg1', 'asah1b', 'osbp', 'casp3a', 'stox2a', 'lap3', 'tmem192', 'ENSGWIG00000021114', 'fbxw7', 'fam160a1a', 'SNORD73', 'lrba', 'DCLK2', 'arhgap10', 'ednraa', 'pou4f2', 'ENSGWIG00000004291', 'lsm6', 'znf827', 'smad1', 'hhip', 'ENSGWIG00000004411', 'frem3', 'usp38', 'gab1', 'inpp4b', 'ENSGWIG00000009843', 'fam13a', 'ppm1k', 'coro2a', 'anp32b', 'ENSGWIG00000012586', 'foxe1', 'sh3gl2a', 'cntln', 'bnc2', 'ENSGWIG00000012921', 'dnajb14', 'MCUB', 'lef1', 'papss1', 'dkk2', 'tbck', 'npnta', 'gstcd', 'arhgef38', 'tet2', 'cxxc4', 'ENSGWIG00000000135', 'toporsa', 'ENSGWIG00000000232', 'ENSGWIG00000000369', 'fbxo41', 'rnf24', 'ENSGWIG00000002690', 'primpol', 'acsl1a', 'SYNPO2', 'ENSGWIG00000012969', 'ENSGWIG00000012970', 'slc22a7b.1', 'DHX15', 'ENSGWIG00000013200', 'mttp', 'acer2', 'rps6', 'BPTF', 'C17orf58', 'SMURF2', 'ENSGWIG00000018107', 'srp68', 'evpla', 'RHBDF2', 'UBE2O', 'SPHK1', 'cygb2', 'myh10', 'slc25a38b', 'myh9a', 'pkn1a', 'ENSGWIG00000025534', 'mcoln1a', 'ENSGWIG00000002933', 'ENSGWIG00000002960', 'rasd4', 'ENSGWIG00000004054', 'PIK3R6', 'pik3r5', 'NTN1', 'usp43a', 'gga3a', 'sept9b', 'ENSGWIG00000004234', 'rhot1b', 'dand5', 'NFIX', 'ENSGWIG00000011749', 's1pr5b', 'si:zfos-323e3.4', 'ldlrb', 'ENSGWIG00000021086', 'rnaseh2a', 'serhl', 'sept3', 'atf7ip', 'grin2bb', 'rbfox2', 'ENSGWIG00000000374', 'si:dkey-245p14.4', 'kdm4c', 'PTPRD', 'ENSGWIG00000023378', 'tyrp1b', 'adgrl3.1', 'gtf2e2', 'ENSGWIG00000024646', 'pcdh7a', 'crmp1', 'ENSGWIG00000025703', 'pax5', 'zcchc7', 'tbc1d1', 'klf3', 'wdr19', 'lias', 'ube2kb', 'ENSGWIG00000026762', 'slit2', 'ENSGWIG00000026772', 'ENSGWIG00000026783', 'ENSGWIG00000026784', 'rbm47', 'doc2d', 'dlc1', 'slc44a1a', 'ugt8', 'ndst3', 'prss12', 'adra1d', 'ctnna2', 'ENSGWIG00000000861', 'suclg1', 'si:dkey-162b23.4', 'TACR3', 'dnaja1', 'ENSGWIG00000002294', 'dnm2b', 'ENSGWIG00000002495', 'ENSGWIG00000003619', 'ENSGWIG00000019350', 'crebbpa', 'ENSGWIG00000021759', 'ENSGWIG00000021760', 'cxcl12b', 'ENSGWIG00000021771', 'xpnpep1', 'add3a', 'mxi1', 'dusp5', 'smc3', 'RBM20', 'pdcd4b', 'bbip1', 'shoc2', 'adra2a', 'ENSGWIG00000022224', 'micu3a', 'fgf20a', 'ENSGWIG00000003494', 'nansa', 'clta', 'gne', 'stra6l', 'abca1a', 'SGCZ', 'tusc3', 'tppp2', 'ENSGWIG00000021035', 'PITPNC1', 'psmd12', 'helz', 'cacng5a', 'PRKCA', 'ENSGWIG00000023243', 'slc25a10', 'ENSGWIG00000000125', 'mycbpap', 'si:cabz01100188.1', 'ENSGWIG00000001024', 'pkd1a', 'TNPO2', 'ENSGWIG00000007891', 'ENSGWIG00000009727', 'ENSGWIG00000009734', 'sgsh', 'pcyt2', 'MAFG', 'ENSGWIG00000010146', 'TMC6', 'ldb1b', 'jupa', 'si:dkey-28e7.3', 'SMAP1', 'dpcd', 'inab', 'ENSGWIG00000016330', 'ENSGWIG00000016334', 'card14', 'ENSGWIG00000016352', 'ENSGWIG00000016354', 'cbx8b', 'rasal3', 'si:dkey-237i9.1', 'NPTX1', 'ENSGWIG00000000297', 'rptor', 'p4hb', 'ppp1r27b', 'ENSGWIG00000001105', 'SMARCE1', 'ENSGWIG00000001120', 'krt97', 'pcdh18a', 'ENSGWIG00000001127', 'pcdh10a', 'ENSGWIG00000001131', 'zgc:77739', 'chrna6', 'chrnb5a', 'htra3a', 'gpr78a', 'hmx4', 'hmx1', 'ENSGWIG00000001929', 'adra2c', 'ENSGWIG00000001944', 'ENSGWIG00000002056', 'ENSGWIG00000002353', 'grk4', 'ENSGWIG00000003004', 'ENSGWIG00000003051', 'ENSGWIG00000003063', '', 'si:dkey-217d24.6', 'acp5a', 'ENSGWIG00000005241', 'ENSGWIG00000005681', 'MARF1', 'fopnl', 'coq7', 'rps15a', 'xylt1', 'nomo', 'TMC7', 'ENSGWIG00000021987', 'pdcd11', 'elf2b', 'ENSGWIG00000022053', 'ENSGWIG00000022054', 'CLASP1', 'clcn4', 'ENSGWIG00000002112', 'ENSGWIG00000003141', 'ENSGWIG00000004456', 'GABRG3', 'DBI', 'en1b', 'ENSGWIG00000004740', 'ENSGWIG00000004910', 'ptger2a', 'cpo', 'ENSGWIG00000006139', 'zgc:92335', 'rnd3a', 'ENSGWIG00000006832', 'zranb3', 'ENSGWIG00000007209', 'ENSGWIG00000007253', 'ENSGWIG00000007255', 'ENSGWIG00000007505', 'arhgap1', 'ENSGWIG00000010135', 'chrm4a', 'mdka', 'ENSGWIG00000010163', 'pudp', 'nlgn4xa', 'prkx', 'ENSGWIG00000006322', 'plcd4a', 'zgc:152904', 'ENSGWIG00000010099', 'jagn1b', 'ENSGWIG00000011140', 'ENSGWIG00000011144', 'bmpr2a', 'dock9b', 'slc4a10a', 'GCG', 'col28a2b', 'klhl41b', 'si:busm1-52i16.2', 'APP', 'ADAMTS5', 'ENSGWIG00000026227', 'ENSGWIG00000026230', 'caskb', 'ENSGWIG00000003135', 'eif5b', 'ENSGWIG00000003631', 'gpd1c', 'march7', 'ENSGWIG00000003671', 'ENSGWIG00000003764', 'ENSGWIG00000003918', 'ENSGWIG00000003921', 'ENSGWIG00000003952', 'gpd2', 'pwp2h', 'pcbp3', 'DMD', 'ENSGWIG00000012104', 'ENSGWIG00000012123', 'abi2b', 'mpp4l', 'inpp4ab', 'zgc:113142', 'ENSGWIG00000016131', 'ENSGWIG00000016162', 'ENSGWIG00000016164', 'ENSGWIG00000016169', 'orc4', 'aox6', 'bzw1b', 'si:ch211-246m6.5', 'ENSGWIG00000020881', 'mbnl2', 'ENSGWIG00000020921', 'ENSGWIG00000020935', 'nrp2a', 'cnppd1', 'raph1b', 'ENSGWIG00000023509', 'ENSGWIG00000023511', 'ENSGWIG00000023512', 'atp11a', 'tubgcp3', 'ENSGWIG00000024127', 'mcf2la', 'ENSGWIG00000024500', 'ENSGWIG00000025058', 'GPR180', 'ENSGWIG00000007747', 'si:ch211-132g1.7', 'zgc:85722', 'ENSGWIG00000007783', 'ENSGWIG00000007787', 'ENSGWIG00000007811', 'chst10', 'arhgef7b', 'ENSGWIG00000010286', 'ENSGWIG00000010319', 'sema5bb', 'ENSGWIG00000010464', 'zgc:110699', 'TCEANC', 'ENSGWIG00000013907', 'CBS', 'ENSGWIG00000014784', 'ENSGWIG00000014791', 'ENSGWIG00000014931', 'FILIP1L', 'ENSGWIG00000015009', 'ccdc28a', 'ENSGWIG00000015013', 'ENSGWIG00000015093', 'ENSGWIG00000021651', 'stk11ip', 'ENSGWIG00000021677', 'ENSGWIG00000021830', 'CREB1', 'klf7a', 'ENSGWIG00000021835', 'erbb4a', 'acadl', 'ENSGWIG00000000243', 'kcnh7', 'ENSGWIG00000000472', 'scn1lab', 'ENSGWIG00000001172', 'ENSGWIG00000001175', 'b3galt1a', 'spry2', 'SLITRK1', 'ENSGWIG00000001223', 'ENSGWIG00000001228', 'slitrk6', 'ENSGWIG00000001235', 'ENSGWIG00000001237', 'MID1', 'ARHGAP6', 'ENSGWIG00000001324', 'ENSGWIG00000001332', 'ENSGWIG00000001336', 'ENSGWIG00000001338', 'ENSGWIG00000001340', 'zeb2b', 'ENSGWIG00000001348', 'si:ch211-186j3.6', 'dpy19l3', 'znf507', 'ENSGWIG00000019291', 'ENSGWIG00000019293', 'ENSGWIG00000019294', 'tshz3b', 'ENSGWIG00000019298', 'znf536', 'ENSGWIG00000019303', 'ccne1', 'ENSGWIG00000019341', 'cd81a', 'celf1', 'ENSGWIG00000020512', 'ENSGWIG00000020513', 'snrpa1', 'slc7a10a', 'rhpn2', 'zgc:165481', 'abcc12', 'mctp2b', 'ENSGWIG00000021896', 'GLO1', 'ENSGWIG00000000532', 'ENSGWIG00000000540', 'fam219b', 'igf1rb', 'ap3s2', 'si:ch211-106j24.1', 'ENSGWIG00000000995', 'si:dkey-172h23.2', 'ntrk3b', 'ENSGWIG00000001155', 'ENSGWIG00000001158', 'mrps11', 'si:cabz01113374.3', 'pnoca', 'kif13ba', 'tdp1', 'eprs1', 'ENSGWIG00000011283', 'ENSGWIG00000011339', 'cemip', 'abhd17c', 'mthfs', 'si:cabz01029535.1', 'si:cabz01029535.1-adamts17', 'adamts17', 'asb7', 'aldh1a3', 'ENSGWIG00000017036', 'mesd', 'ENSGWIG00000017049', 'cyp1a', 'ENSGWIG00000017054', 'CSK', 'cplx3b', 'ENSGWIG00000017213', 'ulk3', 'tars3', 'tm2d3', 'larp6a', 'ENSGWIG00000020999', 'THSD4', 'pepd', 'ENSGWIG00000021087', 'si:ch211-269c21.2', 'ENSGWIG00000003030', 'gpib', 'wtip', 'si:dkey-78a14.4', 'cep83', 'heatr3', 'adcy7', 'brd7', 'nkd1', 'nod2', 'cylda', 'sall1a', 'tox3', 'ENSGWIG00000007870', 'ENSGWIG00000007873', 'chd9', 'aktip', 'ENSGWIG00000012624', 'fto', 'irx3a', 'ENSGWIG00000012637', 'irx5a', 'ENSGWIG00000012647', 'irx6a', 'mmp2', 'lpcat2', 'slc6a2', 'slc12a4', 'dpep2', 'ENSGWIG00000020407', 'ENSGWIG00000020499', 'rec114', 'chrna5', 'ENSGWIG00000022177', 'ENSGWIG00000022198', 'ube2q1', 'SHF', 'cd276', 'ENSGWIG00000022257', 'insyn1', 'ENSGWIG00000022259', 'loxl1', 'igdcc4', 'UNC13C', 'onecut1', 'fam214a', 'myo5aa', 'gnb5b', 'trpm7', 'ENSGWIG00000026633', 'ENSGWIG00000026637', 'si:ch211-39k3.2', 'ENSGWIG00000026643', 'ENSGWIG00000026646', 'znf710b', 'ENSGWIG00000026687', 'frmd5', 'cry5', 'ENSGWIG00000000177', 'tspan3b', 'pstpip1b', 'ENSGWIG00000000305', 'tln2a', 'nfatc3a', 'amfra', 'styx', 'ENSGWIG00000010283', 'mdga1', 'ENSGWIG00000010428', 'slc7a9', 'ENSGWIG00000011615', 'greb1', 'spata17', 'ENSGWIG00000011713', 'ENSGWIG00000011725', 'ahctf1', 'ENSGWIG00000018497', 'cgref1', 'ENSGWIG00000018529', 'ENSGWIG00000018532', 'galnt2', 'DNA2', 'ENSGWIG00000020102', 'tspan15', 'tacr2', 'chs1', 'ENSGWIG00000000552', 'ENSGWIG00000000887', 'rps6ka2', 'immp1l', 'elp4', 'pax6b', 'rcn1', 'qser1', 'depdc7a', 'spon1a', 'ppfibp2a', 'MAF', 'ENSGWIG00000006287', 'chchd2', 'CDYL2', 'atmin', 'ENSGWIG00000006300', 'gse1', 'irf8', 'ENSGWIG00000006319', 'mcee', 'bdnf', 'KIF18A', 'mettl15', 'lgr4', 'mybpc3', 'igdcc3', 'ENSGWIG00000017487', 'cilp', 'FAM189A1', 'tjp1a', 'terb2', 'sord', 'apba2b', 'syt7b', 'sdhaf2', 'polr2l', 'dagla', 'ext2', 'sox6', 'ENSGWIG00000024774', 'pde3b', 'rras2', 'tub', 'ENSGWIG00000024802', 'lmo1', 'stk33', 'dennd2b', 'tmem9b', 'sinhcafl', 'dennd5a', 'tmem41b', 'sbf2', 'mrvi1', 'galnt18b', 'ENSGWIG00000018698', 'ENSGWIG00000018706', 'TEAD1', 'mical2a', 'ENSGWIG00000019008', 'CLEC3A', 'pkd1l2a', 'nfat5b', 'kars1', 'zgc:162592', 'dhx38', 'ENSGWIG00000023881', 'zfhx3', 'crispld2', 'marveld3', 'PHLPP2', 'ENSGWIG00000024682', 'myo1ea', 'rnf111', 'sltm', 'mindy2', 'aqp9b', 'aldh1a2', 'polr2m', 'cgnl1', 'tcf12', 'znf280d', 'nedd4a', 'ENSGWIG00000026030', 'PRTG', 'slc38a8b', 'furina', 'fes', 'rhcgl1', 'kif7', 'ENSGWIG00000026288', 'ENSGWIG00000003199', 'ENSGWIG00000003206', 'pdcd5', 'dpep1', 'ankrd11', 'spg7', 'ACSF3', 'cbfa2t3', 'trappc2l', 'galns', 'nae1', 'ENSGWIG00000010908', 'epha6', 'kctd12.2', 'gpc6a', 'gpc5a', 'ENSGWIG00000014069', 'tpp2', 'rasa3', 'tmco3', 'lamp1a', 'cul4a', 'ENSGWIG00000018742', 'lims1', 'nkpd1', 'ENSGWIG00000018771', 'ENSGWIG00000018774', 'pcdh17', 'diaph3', 'tdrd3', 'PCDH20', 'pcdh9', 'pou4f1', 'ENSGWIG00000018942', 'rbm26', 'ENSGWIG00000018987', 'ENSGWIG00000019046', 'baz2ba', 'ly75', 'pla2r1', 'itgb6', 'fn1b', 'atic', 'ppef1', 'RS1', 'cdkl5', 'scml2', 'nhsb', 'ENSGWIG00000002084', 'ENSGWIG00000002402', 'nnt2', 'fgf7', 'cops2', 'SHC4', 'slc24a5', 'si:dkeyp-86f7.4', 'sema6dl', 'dmxl2', 'mapk6', 'pde8a', 'ENSGWIG00000024287', 'ENSGWIG00000024299', 'ENSGWIG00000024354', 'rxfp3.3a2', 'itga11a', 'coro2ba', 'glceb', 'KIF23', 'ENSGWIG00000024494', 'TLE3', 'uacab', 'pias1a', 'map2k5', 'SMAD3', 'smad6a', 'mfge8a', 'slc30a4', 'arfgap2', 'MEGF11', 'rab11a', 'ENSGWIG00000006340', 'ENSGWIG00000006907', 'cd82a', 'rpl12', 'kcng4b', 'ENSGWIG00000006930', 'ENSGWIG00000007011', 'ENSGWIG00000007025', 'ENSGWIG00000007029', 'katnb1', 'wdr59', 'cebpa', 'cebpg', 'ENSGWIG00000010036', 'fhod1', 'hsd11b2', 'slc22a18', 'NR2E3', 'ENSGWIG00000011312', 'mpped2a', 'zdhhc13', 'nav2a', 'slc6a5', 'ENSGWIG00000012438', 'NELL1', 'ppfia1', 'rbpms2a', 'psma4', 'crabp1a', 'si:ch211-288d18.1', 'brsk2b', 'ENSGWIG00000000463', 'shank2a', 'slc1a2a', 'kcnq1.1', 'luzp2', 'kcna4', 'cd151l', 'ddb1', 'tspan4a', 'large2', 'ENSGWIG00000004595', 'ctsd', 'ENSGWIG00000007133', 'ENSGWIG00000007148', 'ENSGWIG00000007140', 'map1aa', 'ENSGWIG00000011101', 'MAPK8IP1', 'ENSGWIG00000011404', 'ENSGWIG00000011425', 'ENSGWIG00000007221', 'gmds', 'foxf2b', 'exoc2', 'SLC22A23', 'adcy1a', 'ENSGWIG00000007342', 'igfbp3', 'ENSGWIG00000007345', 'ENSGWIG00000007346', 'ENSGWIG00000007371', 'si:ch211-191a24.4', 'naprt', 'ENSGWIG00000007719', 'rgs16', 'ENSGWIG00000007743', 'rgs8', 'brinp3a.1', 'ENSGWIG00000007754', 'brinp3a.2', 'ENSGWIG00000007798', 'ENSGWIG00000007815', 'RGS1', 'uchl5', 'ENSGWIG00000007901', 'b3galt2', 'ENSGWIG00000007906', 'ENSGWIG00000007910', 'bcar3', 'ENSGWIG00000004600', 'ENSGWIG00000004617', 'ttc14', 'ENSGWIG00000005481', 'si:dkey-211e20.10', 'mccc1', 'ENSGWIG00000013264', 'ENSGWIG00000013266', 'fam102bb', 'ENSGWIG00000013277', 'ENSGWIG00000013283', 'FNDC7', 'fam183a', 'ENSGWIG00000013835', 'lrp8', 'ENSGWIG00000013932', 'ap1m3', 'ENSGWIG00000016363', 'ENSGWIG00000016365', 'ENSGWIG00000016370', 'si:ch211-262h13.5', 'ENSGWIG00000017092', 'ENSGWIG00000017096', 'ENSGWIG00000017636', 'meltf', 'ENSGWIG00000024449', 'ENSGWIG00000024451', 'ENSGWIG00000024483', 'cp', 'ttc22', 'lurap1', 'dnajc6', 'ENSGWIG00000009044', 'ENSGWIG00000009047', 'raver2', 'cachd1', 'ror1', 'pgm1', 'atg4c', 'dock7', 'usp1', 'pde4ba', 'ENSGWIG00000023958', 'si:ch73-61d6.3', 'rexo1', 'zap70', 'adamts10', 'ENSGWIG00000025106', 'myo1f', 'ENSGWIG00000025111', 'rfx2', 'fcho1', 'tprb', 'prg4b', 'pdcb', 'ENSGWIG00000025735', 'ivns1abpa', 'swt1', 'ENSGWIG00000025740', 'nek7', 'lhx9', 'si:dkey-121a9.3', 'dennd1b', 'RABGAP1L', 'ENSGWIG00000026054', 'ENSGWIG00000026146', 'ndufs7', 'r3hdm4', 'si:ch73-40i7.5', 'mast3b', 'ENSGWIG00000002130', 'uhrf1', 'kdm4b', 'ptprsa', 'si:zfos-943e10.1', 'camsap2a', 'ENSGWIG00000005771', 'nr5a2', 'ENSGWIG00000005850', 'alpi.1', 'ENSGWIG00000006451', 'b3gnt7', 'armc9', 'ENSGWIG00000006753', 'ENSGWIG00000006862', 'ENSGWIG00000010056', 'ENSGWIG00000010080', 'ENSGWIG00000010081', 'psmd1', 'ENSGWIG00000017009', 'her6', 'opa1', 'ankrd13c', 'cthl', 'zranb2', 'negr1', 'vtg3', 'adgrl4', 'adgrl2a', 'ttll7', 'ENSGWIG00000022445', 'dnase2b', 'samd13', 'uox', 'ENSGWIG00000022806', 'ENSGWIG00000025904', 'ENSGWIG00000026088', 'lpar3', 'mcoln2', 'plaat1', 'gnb4b', 'usp13', 'pex5la', 'ccdc50', 'cpox', 'prrg1', 'ENSGWIG00000026426', 'ENSGWIG00000026429', 'ENSGWIG00000026430', 'guk1b', 'rnf220b', 'dmap1', 'timm44', 'ctxn1', 'fbn2b', 'cers4a', 'lsm7', 'enc3', 'arhgef18b', 'insrb', 'ENSGWIG00000026491', 'cers1', 'ENSGWIG00000026516', 'zfr2', 'sin3b', 'qsox1', 'cep350', 'cacna1ea', 'ENSGWIG00000008408', 'osbpl9', 'calr', 'ttc39a', 'ENSGWIG00000010909', 'faf1', 'dmrta2', 'elavl4', 'agbl4', 'bend5', 'ENSGWIG00000011366', 'safb', 'ap3d1', 'C2CD4C', 'gna15.4', 'ENSGWIG00000013242', 'tle5', 'tle2b', 'tle2a', 'ENSGWIG00000014339', 'shc2', 'odf3l2b', 'ENSGWIG00000014353', 'FOXD2', 'trabd2b', 'ENSGWIG00000014387', 'ENSGWIG00000000474', 'atf6', 'fyco1a', 'GHRHR', 'faah', 'st6galnac5b', 'slc44a5b', 'erich3', 'ssbp3b', 'glis1b', 'rorca', 'sema6bb', 'abhd17ab', 'scfd2', 'lnx1', 'gsx2', 'kita', 'kdr', 'CLOCK', 'exoc1', 'cep135', 'chst14', 'ENSGWIG00000012507', 'tgm1l2', 'si:ch211-121a2.4', 'ENSGWIG00000018960', 'ENSGWIG00000018962', 'usp24', 'bloc1s2', 'prkaa2', 'ENSGWIG00000000220', 'dab1a', 'ENSGWIG00000000364', 'oma1', 'mysm1', 'suco', 'ENSGWIG00000000631', 'vamp4', 'myoc', 'prdx6', 'caiap', 'angptl1a', 'ralgps2', 'pfas', 'cyp2ad2', 'il12rb2', 'hsd17b7', 'serbp1a', 'scamp4', 'si:ch211-129c21.1', 'fsd1', 'march2', 'pip5k1ca', 'tjp3', 'si:ch73-63e15.2', 'tm6sf2', 'hapln4', 'ENSGWIG00000008539', 'nr2c2ap', 'si:ch211-125m10.6', 'tmem161a', 'ENSGWIG00000008568', 'slc25a42', 'gatad2ab', 'yjefn3', 'si:ch211-212d10.2', 'ENSGWIG00000008632', 'ENSGWIG00000000162', 'bsg', 'ENSGWIG00000000249', 'polrmt', 'ENSGWIG00000000484', 'hsd11b1la', 'rgmd', 'celf5b', 'ENSGWIG00000000527', 'EEF2', 'mbd3b', 'mbd3a', 'tcf3b', 'zbtb7a', 'pias4a', 'foxq2', 'onecut3a', 'atp8b3', 'lingo3a', 'ENSGWIG00000019368', 'org', 'ENSGWIG00000019375', 'oaz1a', 'ell', 'SSBP4', 'tmem59l', 'notch2', 'sec22bb', 'edem3', 'zgc:92140', 'ENSGWIG00000020098', 'fryl', 'slain2', 'ENSGWIG00000020556', 'cpeb2', 'prdm5', 'ENSGWIG00000020613', 'slc34a2b', 'cdh24a', 'ENSGWIG00000004555', 'si:dkey-194e6.1', 'reps2', 'ENSGWIG00000004586', 'nhsa', 'RAI2', 'ENSGWIG00000004614', 'CDKL5', 'zgc:153311', 'ENSGWIG00000006245', 'chd1l', 'nit2', 'tbc1d23', 'ENSGWIG00000010722', 'jade3', 'slc9a7', 'tubgcp5', 'ENSGWIG00000013295', 'cyfip1', 'ENSGWIG00000013341', 'oca2', 'gabra5', 'gabrb3', 'atp10a', 'cnga3a', 'tmem131', 'bivm', 'zgc:172121', 'nme7', 'atp1b1a', 'dpt', 'zbtb11', 'tp63', 'tprg1', 'ENSGWIG00000015238', 'lpp', 'podn', 'ENSGWIG00000015291', 'ENSGWIG00000005999', 'GFI1', 'glmnb', 'tgfbr3', 'barhl2', 'ENSGWIG00000009438', 'ENSGWIG00000009439', 'lrrc8c', 'kyat3', 'ENSGWIG00000010721', 'lmo4b', 'SOX14', 'hdlbpa', 'farp2', 'ENSGWIG00000016759', 'ENSGWIG00000018465', 'espnla', 'klhl30', 'sned1', 'ryk', 'ENSGWIG00000018802', 'ENSGWIG00000018808', 'ENSGWIG00000018811', 'ptprfa', 'kdm4aa', 'artna', 'ENSGWIG00000025213', 'smx5', 'inpp5d', 'ENSGWIG00000000442', 'sap130a', 'amotl2a', 'cep63', 'im:7152348', 'arhgap39', 'ENSGWIG00000001006', 'EPS15L1', 'axdnd1', 'ENSGWIG00000001008', 'ENSGWIG00000001044', 'ENSGWIG00000001048', 'NAALADL2', 'ENSGWIG00000001057', 'nlgn1', 'ENSGWIG00000001097', 'ENSGWIG00000001098', 'ect2', 'mcf2l2', 'b3gnt5a', 'ENSGWIG00000007448', 'ENSGWIG00000007472', 'ENSGWIG00000008391', 'ccndbp1', 'rims4', 'lsm3', 'ENSGWIG00000008453', 'ENSGWIG00000008844', 'grip2b', 'zgc:171566', 'PTPRT', 'chd6', 'lmcd1', 'ENSGWIG00000000153', 'blcap', 'mtss1', 'ENSGWIG00000000636', 'ENSGWIG00000000692', 'ENSGWIG00000000694', 'ENSGWIG00000000830', 'pxk', 'si:dkeyp-87e7.4', 'stab1', 'hgh1', 'rbbp5', 'ENSGWIG00000001196', 'gpr37l1a', 'atp2b4', 'zgc:162255', 'nfs1', 'rbm12', 'cpne1', 'unm_sa1614', 'ENSGWIG00000007565', 'pnp4a', 'ENSGWIG00000000134', 'draxin', 'agtrap', 'chd5', 'arhgef10la', 'ENSGWIG00000001413', 'igsf21a', 'dhrs3b', 'ENSGWIG00000001428', 'ephb2b', 'lactbl1b', 'nmur3', 'ENSGWIG00000005019', 'alpl', 'eif4g3a', 'hp1bp3', 'yod1', 'PLXNA2', 'si:ch211-286o17.1', 'ENSGWIG00000022344', 'ENSGWIG00000022350', 'magi3b', 'padi2', 'si:dkey-166d12.2', 'slc2a1b', 'ENSGWIG00000023209', 'ENSGWIG00000023210', 'ENSGWIG00000023211', 'ilrun', 'rps10', 'nudt3b', 'grm4', 'ENSGWIG00000023400', 'ENSGWIG00000023402', 'igfn1.2', 'ENSGWIG00000023406', 'nav1b', 'tead3b', 'rplp2', 'KCNA10', 'kcna2a', 'KCNA3', 'ngfb', 'ENSGWIG00000023961', 'slc16a1a', 'si:dkey-238f9.1', 'ncoa5', 'ENSGWIG00000007499', 'cyp24a1', 'manbal', 'SRC', 'VAPB', 'ENSGWIG00000018806', 'ENSGWIG00000019433', 'tfap2c', 'cdh22', 'znfx1', 'ENSGWIG00000025447', 'tox2', 'wnt2ba', 'KCND3', 'lmod1b', 'ENSGWIG00000025685', 'ppp1r16b', 'shmt2', 'NXPH4', 'STAC3', 'r3hdm2', 'INHBC', 'gli1', 'ENSGWIG00000025812', 'dctn2', 'KIF5A', 'ENSGWIG00000025843', 'sp1', 'myg1', 'ENSGWIG00000025870', 'zgc:172352', 'POC1A', 'dusp7', 'iars1', 'atp2b2', 'HRH1', 'atg7', 'ENSGWIG00000021449', 'PPARG', 'ENSGWIG00000022362', 'megf6b', 'kcnab2a', 'pax7b', 'mrps16', 'trim62.1', 'utp3', 'FOXJ3', 'kaznb', 'skib', 'ENSGWIG00000022764', 'wdr82', 'hyal1', 'cacna2d2a', 'mst1', 'ENSGWIG00000023061', 'sema3fa', 'rbm6', 'TEX264', 'rrp9', 'oxtrl', 'ENSGWIG00000013292', 'rdh20', 'stau1', 'camk1a', 'si:dkeyp-67f1.2', 'ENSGWIG00000020816', 'ca16b', 'cadpsb', 'slc6a22.1', 'fkbp5', 'trub2', 'ENSGWIG00000023027', 'cacna2d2b', 'acy1', 'cyb561d2', 'bap1', 'cacna1da', 'fblim1', 'slc25a34', 'fbxo42', 'ENSGWIG00000005898', 'MFAP2', 'ENSGWIG00000005952', 'celsr2', 'dstyk', 'SLC6A17', 'ENSGWIG00000010228', 'ENSGWIG00000010257', 'eps8l3a', 'ampd2a', 'GPR61', 'rad18', 'oxtr', 'slc38a3a', 'ENSGWIG00000020426', 'eefsec', 'ENSGWIG00000000717', 'ccdc174', 'b4galnt1b', 'ENSGWIG00000012987', 'c1ql4b', 'pfkmb', 'dip2bb', 'scn8ab', 'BHLHE40', 'itpr1a', 'sumf1', 'ENSGWIG00000022032', 'si:dkey-28n18.9', 'ENSGWIG00000022231', 'ENSGWIG00000022303', 'rbms2a', 'rargb', 'ENSGWIG00000022426', 'SPRYD3', 'ENSGWIG00000022455', 'osgn1', 'mettl1', 'ENSGWIG00000004733', 'ENSGWIG00000005525', 'elmo2', 'ENSGWIG00000008157', 'il19l', 'dyrk3', 'si:dkey-85p17.3', 'srgap2', 'zgc:101564', 'foxp4', 'ppfia4', 'mdm4', 'ENSGWIG00000014195', 'ENSGWIG00000014230', 'dnajc14', 'nab2', 'ENSGWIG00000014238', 'inavab', 'gdf11', 'avil', 'cd8b', 'ENSGWIG00000025510', 'pcsk1', 'ell2', 'glrx', 'si:dkey-6b12.5', 'ube2r2', 'ubap1', 'dcaf12', 'ubap2a', 'celf4', 'myorg', 'ENSGWIG00000005204', 'ENSGWIG00000005517', 'crybb3', 'cita', 'pxna', 'golga7', 'ENSGWIG00000008154', 'loxl2a', 'aggf1', 'tacc1', 'stx18', 'msx1a', 'CYTL1', 'ENSGWIG00000009966', 'ENSGWIG00000009988', 'ENSGWIG00000009989', 'ENSGWIG00000010154', 'ENSGWIG00000016858', 'chchd6b', 'tpra1', 'zgc:123305', 'myl9a', 'ENSGWIG00000016927', 'quo', 'ENSGWIG00000018578', 'tns2b', 'vdrb', 'twist3', 'dnajc11b', 'ENSGWIG00000018772', 'acap3b', 'kif21b', 'cpne5b', 'ENSGWIG00000026100', 'gnl3', 'glt8d1', 'spcs1', 'camk1b', 'nek4', 'ccdc3b', 'ENSGWIG00000014029', 'WDR1', 'setd5', 'mtmr14', 'si:dkey-166c18.1', 'ENSGWIG00000015683', 'klhdc8a', 'slc41a1', 'ENSGWIG00000015796', 'etnk2', 'sox13', 'snrpe', 'ENSGWIG00000015807', 'wnt4', 'ENSGWIG00000015810', 'ENSGWIG00000015821', 'ENSGWIG00000015836', 'txnrd3', 'CHST13', 'ENSGWIG00000016035', 'phactr3b', 'acot8', 'ENSGWIG00000016635', 'ift52', 'cdk5rap1', 'ENSGWIG00000018882', 'ENSGWIG00000018997', 'cntn4', 'ENSGWIG00000021155', 'ENSGWIG00000021257', 'chl1b', 'ENSGWIG00000024538', 'ENSGWIG00000024539', 'ENSGWIG00000024541', 'cntn3b', 'pdzrn3b', 'ENSGWIG00000024727', 'ppp4r2b', 'gxylt2', 'shq1', 'rybpb', 'ENSGWIG00000024745', 'ENSGWIG00000024783', 'ENSGWIG00000024786', 'foxp1b', 'ENSGWIG00000024794', 'ENSGWIG00000024795', 'mitfa', 'frmd4ba', 'arl6ip5a', 'trnt1', 'lrrn1', 'ENSGWIG00000024820', 'NAAA', 'prickle2b', 'adamts9', 'magi1b', 'ENSGWIG00000001164', 'ENSGWIG00000001166', 'lrig1', 'ENSGWIG00000001396', 'kbtbd8', 'ENSGWIG00000001421', 'suclg2', 'ENSGWIG00000001685', 'TAFA1', 'tafa4b', 'fgd5a', 'gata2a', 'ENSGWIG00000009779', 'ENSGWIG00000009786', 'ENSGWIG00000009788', 'chchd4a', 'si:dkey-202e22.2', 'BMP7', 'ENSGWIG00000011704', 'lsm14b', 'cdh4', 'ENSGWIG00000011987', 'ENSGWIG00000011988', 'ENSGWIG00000011990', 'ENSGWIG00000011991', 'ENSGWIG00000012000', 'zmynd8', 'ncoa3', 'id1', 'sulf2a', 'ENSGWIG00000015392', 'zgc:92107', 'ndrg3a', 'top1l', 'fam83d', 'ENSGWIG00000000808', 'gata1a', 'hcfc1a', 'tfe3b', 'cxxc1b', 'mbd1b', 'ccdc120', 'chrm2a', 'mtpn', 'ENSGWIG00000005265', 'sema3c', 'cd36', 'ENSGWIG00000009035', 'si:dkey-97m3.1', 'kdm7ab', 'ndufb2', 'lhfpl3', 'ccnd2b', 'ada2b', 'spic', 'spi2', 'ENSGWIG00000007577', 'mcat', 'def6c', 'ttll1', 'pacsin2', 'prkcq', 'sfmbt2', 'ENSGWIG00000010793', 'CELF2', 'ENSGWIG00000011036', 'kcnj8', 'ECHDC3', 'plxnb2a', 'creb3l2', 'ENSGWIG00000012678', 'rassf3', 'uhrf1bp1l', 'anks1b', 'tcp11l2', 'si:ch211-244b2.4', 'dcn', 'ENSGWIG00000019166', 'ENSGWIG00000019193', 'btg1', 'si:dkey-159a18.1', 'ENSGWIG00000019457', 'snd1', 'pax4', 'cacna2d1a', 'sema3aa', 'tspan11', 'cracr2ab', 'tmtc2a', 'ENSGWIG00000009162', 'PARP11', 'ENSGWIG00000009168', 'iqsec3a', 'ENSGWIG00000009384', 'st8sia1', 'ENSGWIG00000009387', 'ENSGWIG00000009481', 'copg2', 'ENSGWIG00000012298', 'ENSGWIG00000012300', 'ENSGWIG00000012302', 'lrguk', 'exoc4', 'chchd3a', 'ENSGWIG00000013938', 'plxna4', 'nrf1', 'ube2h', 'cax1', 'smo', 'cacna1c', 'ENSGWIG00000022294', 'ENSGWIG00000022295', 'lrtm2a', 'ENSGWIG00000022321', 'ENSGWIG00000022322', 'ninj2', 'wnt5b', 'ENSGWIG00000022948', 'ENSGWIG00000022950', 'ENSGWIG00000022956', 'LOC114465680', 'ENSGWIG00000022959', 'pkp2', 'ube2na', 'ints13', 'ENSGWIG00000023452', 'waslb', 'SPAM1', 'TMEM229A', 'gpr37b', 'ENSGWIG00000023487', 'ENSGWIG00000023489', 'ENSGWIG00000023491', 'grm8a', 'ZNF800', 'ATXN7L1', 'sypl1', 'si:dkey-145c18.3', 'gsap', 'ENSGWIG00000002637', 'lrrc17', 'pus7l', 'ENSGWIG00000002659', 'zcrb1', 'yaf2', 'gxylt1b', 'pdzrn4', 'nrcama', 'pnpla8', 'dnajb9b', 'akr1b1.1', 'tfec', 'si:ch211-262n1.4', 'ENSGWIG00000017781', 'gpr85', 'tmem168a', 'ifrd1', 'dock4b', 'ENSGWIG00000021023', 'lrrn3b', 'ENSGWIG00000021027', 'ahcyl2b', 'prdm4', 'ENSGWIG00000024632', 'ENSGWIG00000024636', 'pparaa', 'wnt7ba', 'atxn10', 'ENSGWIG00000024656', 'kdm5a', 'USP15', 'ENSGWIG00000025161', 'pim3', 'creld2', 'brd1a', 'ENSGWIG00000025180', 'ENSGWIG00000025181', 'TAFA5', 'tbc1d22a', 'cerk', 'gramd4a', 'celsr1a', 'ENSGWIG00000025584', 'trmu', 'si:ch73-263o4.3', 'tprkb', 'CPNE8', 'ENSGWIG00000025598', 'SLC2A13', 'ENSGWIG00000001346', 'pyroxd1', 'etnk1', 'sox5', 'ENSGWIG00000007382', 'ENSGWIG00000007384', 'atp23', 'lrig3', 'zgc:165507', 'ENSGWIG00000010012', 'ETV6', 'SYN3', 'ENSGWIG00000011303', 'ENSGWIG00000013444', 'ccdc3a', 'camk1da', 'nudt5', 'ENSGWIG00000014410', 'slc38a4', 'slc38a2', 'arid2', 'stk38l', 'ENSGWIG00000021240', 'coa6', 'ENSGWIG00000021419', 'pik3cg', 'ano2b', 'ENSGWIG00000021810', 'ntf3', 'pfkfb3', 'rbm17', 'ANO4', 'nav3', 'ENSGWIG00000007590', 'e2f7', 'zdhhc17', 'OSBPL8', 'nap1l1', 'slc26a3.2', 'ppp6r2a', 'ENSGWIG00000013312', 'ENSGWIG00000014338', 'gdi2', 'ENSGWIG00000014375', 'net1', 'mkln1', 'chst11', 'slc41a2b', 'samm50l', 'scube1', 'mpped1', 'ttll12', 'ENSGWIG00000020263', 'dennd5b', 'ipo8', 'si:dkey-8e10.3', 'crebl2', 'eps8a', 'ptpro', 'sco2', 'tspan12', 'ENSGWIG00000023149', 'cped1', 'zgc:55888', 'tmtc1', 'ptprr', 'ENSGWIG00000023801', 'plekha5', 'kcnc2', 'psmc2', 'ENSGWIG00000006723', 'ENSGWIG00000007395', 'pthlhb', 'tmpob', 'cnot4a', 'ENSGWIG00000007512', 'ENSGWIG00000007918', 'ENSGWIG00000008994', 'ENSGWIG00000008997', 'nup160', 'ENSGWIG00000009139', 'lamb1a', 'ENSGWIG00000009173', 'ENSGWIG00000011091', 'ENSGWIG00000011095', 'ENSGWIG00000011225', 'ENSGWIG00000011227', 'commd9', 'ENSGWIG00000011290', 'dus2', 'tln2b', 'TPM1', 'TMEM266', 'scaper', 'ENSGWIG00000017744', 'znf592', 'si:ch211-69m14.1', 'zgc:162879', 'si:ch211-13k12.2', 'pex11a', 'GNRHR', 'ENSGWIG00000005387', 'bcar1', 'ENSGWIG00000005412', 'lrp5', 'ENSGWIG00000010382', 'ric8a', 'ENSGWIG00000010487', 'ripor1', 'ENSGWIG00000010680', 'ENSGWIG00000010681', 'ENSGWIG00000010682', 'SLCO1C1', 'ENSGWIG00000010691', 'gpr22b', 'ENSGWIG00000010700', 'ENSGWIG00000011619', 'ENSGWIG00000011621', 'bmt2', 'ENSGWIG00000012198', 'ENSGWIG00000012199', 'si:ch211-272n13.3', 'AVPR1A', 'ppm1h', 'mon2', 'SLC6A13', 'AKR1D1', 'ENSGWIG00000016104', 'fbln1', 'wnt7bb', 'PPARA', 'yars2', 'cdkn1bb', 'ENSGWIG00000016320', 'ENSGWIG00000016323', 'mrps35', 'ENSGWIG00000016332', 'DENND2A', 'ENSGWIG00000017110', 'ENSGWIG00000018807', 'parvb', 'shisal1b', 'U4', 'plxnb2b', 'appl2', 'ENSGWIG00000001325', 'nuak1b', 'ENSGWIG00000001331', 'ENSGWIG00000001343', 'nell2a', 'ENSGWIG00000007749', 'nptna', 'ENSGWIG00000007792', 'pkmb', 'ENSGWIG00000008968', 'arid3b', 'ENSGWIG00000008979', 'adal', 'caprin1b', 'lmo2', 'alx4b', 'cd82b', 'tp53i11b', 'si:dkey-29p10.4', 'ENSGWIG00000012340', 'ENSGWIG00000012346', 'ENSGWIG00000012349', 'RELN', 'PLEKHA5', 'ENSGWIG00000012460', 'ENSGWIG00000012466', 'ENSGWIG00000012467', 'ENSGWIG00000012469', 'ENSGWIG00000012538', 'scamp5a', 'galk2', 'cep152', 'tjp1b', 'ENSGWIG00000013038', 'pard6a', 'ENSGWIG00000013049', 'galn', 'ENSGWIG00000013068', 'ENSGWIG00000013080', 'fam174b', 'ENSGWIG00000005028', 'ENSGWIG00000006097', 'ENSGWIG00000006100', 'lysmd4', 'ENSGWIG00000006104', 'nr2f2', 'mctp2a', 'mef2aa', 'SYNM', 'igf1ra', 'ENSGWIG00000006275', 'ENSGWIG00000006737', 'ENSGWIG00000006747', 'ENSGWIG00000006803', 'iqsec3b', 'ddx11', 'fkbp4', 'tnni1c', 'ppp6r2b', 'ENSGWIG00000007114', 'dnaja2b', 'ENSGWIG00000007150', 'ENSGWIG00000007154', 'ENSGWIG00000007173', 'grm3', 'ENSGWIG00000007363', 'sema3d', 'sema3ab', 'sema3e', 'ENSGWIG00000018601', 'PCLO', 'si:dkey-266j7.2', 'lrrc4.2', 'ENSGWIG00000019034', 'impdh1a', 'si:ch211-220f12.1', 'neto2b', 'zgc:77752', 'odf3b', 'ENSGWIG00000000563', 'ENSGWIG00000000567', 'mapk8ip2', 'chkb', 'wwox', 'ENSGWIG00000002511', 'wnk1a', 'prr5a', 'RSBN1L', 'ENSGWIG00000004396', 'ENSGWIG00000007230', 'ENSGWIG00000009092', 'ENSGWIG00000009093', 'ENSGWIG00000009338', 'polr3b', 'ppfibp1b', 'arntl2', 'sass6', 'eea1', 'itpr2', 'rassf8a', 'ENSGWIG00000025522', 'ENSGWIG00000025523', 'ENSGWIG00000025524', 'ENSGWIG00000025525', 'ENSGWIG00000025526', 'banp', 'fanci', 'rhcgb', 'furinb', 'ENSGWIG00000009357', 'rag2', 'ENSGWIG00000009364', 'ENSGWIG00000009367', 'ENSGWIG00000009369', 'lrrc4ca', 'EHF', 'pdhx', 'cd44b', 'slc1a2b', 'pamr1', 'fjx1', 'trim44', 'LDLRAD3', 'ENSGWIG00000010933', 'rassf7a', 'ENSGWIG00000010941', 'ENSGWIG00000010947', 'hrasb', 'ldha', 'ENSGWIG00000011006', 'abtb2b', 'ENSGWIG00000011064', 'uevld', 'cpt1ab', 'si:ch211-236l14.4', 'mrpl23', 'ENSGWIG00000011871', 'igf2b', 'th', 'nap1l4a', 'phlda2', 'osbpl5', 'ENSGWIG00000014839', 'dusp8a', 'mob2a', 'abcc8', 'ush1c', 'ENSGWIG00000015713', 'agbl2', 'washc4', 'ENSGWIG00000018757', 'si:ch73-62b13.1', 'btbd11b', 'mtss1la', 'ENSGWIG00000000603', 'ENSGWIG00000000612', 'rxylt1', 'srgap1a', 'parp12a', 'tbxas1', 'hipk2', 'si:ch211-1e14.1', 'tspan9a', 'ENSGWIG00000007046', 'tmtc2b', 'ENSGWIG00000007281', 'bicd1a', 'fgd4a', 'cecr2', 'si:dkey-246g23.4', 'kcng4a', 'si:dkey-246g23.2', 'mlycd', 'cdh13', 'ENSGWIG00000014979', 'ccnd2a', 'FGF6', 'zgc:77158', 'zgc:154077', 'elmo3', 'zgc:103697', 'rspry1', 'pllp', 'ENSGWIG00000019828', 'hsf4', 'gnao1a', 'si:dkey-102f14.5', 'bbs2', 'cnot1', 'si:ch211-216l23.1', 'si:ch211-212o1.2', 'cbln1', 'ENSGWIG00000024329', 'syt9b', 'spon1b', 'ENSGWIG00000000878', 'wt1a', 'ano10b', 'fa2h', 'znf319b', 'chst6', 'tmem231', 'wwp2', 'si:dkey-30c15.10', 'smpd3', 'sept15', 'ENSGWIG00000009957', 'btbd10a', 'galnt18a', 'slc12a3', 'herpud1', 'cetp', 'chka', 'anpepa', 'fam169b', 'ldhd', 'znrf1', 'ndrg4', 'si:ch211-194m7.5', 'si:ch211-194m7.4', 'mical2b', 'parvaa', 'RASSF10', 'si:ch211-266k8.4', 'ENSGWIG00000013239', 'sf3b3', 'ENSGWIG00000014026', 'hydin', 'necab2', 'pskh1', 'nrn1la', 'edc4', 'ranbp10', 'ENSGWIG00000000245', 'det1', 'zcchc14', 'ca5a', 'spire2', 'cdk10', 'ENSGWIG00000006315', 'flnca', 'ENSGWIG00000014682', 'ENSGWIG00000014709', 'ENSGWIG00000015150', 'gnb5a', 'ano3', 'ENSGWIG00000016726', 'ENSGWIG00000016737', 'ano9a', 'PKP3', 'kcnc1b', 'sergef', 'saal1', 'ENSGWIG00000019582', 'ENSGWIG00000019658', 'hps5', 'ENSGWIG00000019909', 'pnpla2', 'syt7a', 'ENSGWIG00000019996', 'ENSGWIG00000019998', 'EPS8L2', 'myrf', 'cracr2b', 'ENSGWIG00000001202', 'kif23', 'tle3a', 'morf4l1', 'smad3b', 'smad6b', 'map2k1', 'megf11', 'piwil2', 'dennd4a', 'slc24a1', 'ints14', 'ENSGWIG00000014618', 'ca12', 'fbxl22', 'ENSGWIG00000014684', 'dapk2b', 'tbc1d2b', 'idh3a', 'ENSGWIG00000015833', 'ENSGWIG00000015888', 'ENSGWIG00000015939', 'slc7a10b', 'ENSGWIG00000016868', 'roraa', 'bnip2', 'si:dkey-37f18.2', 'trpm1b', 'dhtkd1', 'ENSGWIG00000020356', 'cdk17', 'ELK3', 'met', 'CAPZA2', 'st7', 'wnt2', 'ENSGWIG00000004755', 'cftr', 'cttnbp2', 'ENSGWIG00000012932', 'ENSGWIG00000012943', 'CERK', 'ENSGWIG00000012952', 'aass', 'CADPS2', 'asb15a', 'hyal6', 'gpr37a', 'ENSGWIG00000021571', 'grm8b', 'ftsj1', 'ENSGWIG00000022722', 'dldh', 'ENSGWIG00000023301', 'tnni2a.2', 'ENSGWIG00000023340', 'ENSGWIG00000023345', 'ENSGWIG00000023347', 'rfwd3', 'ENSGWIG00000024024', 'zgc:101566', 'ENSGWIG00000024147', 'ckap5', 'ENSGWIG00000025909', 'cry2', 'zc3h18', 'trhr2', 'zfpm1', 'ENSGWIG00000000628', 'cmip', 'ENSGWIG00000001381', 'ENSGWIG00000001406', 'acanb', 'mfge8b', 'hsd17b12a', 'api5', 'tdg.2', 'tdg.1', 'ppp1r15b', 'PLEKHA7', 'ENSGWIG00000008312', 'ENSGWIG00000009265', 'INSC', 'ENSGWIG00000009346', 'znf143b', 'ireb2', 'wdr61', 'cspg4', 'lingo1a', 'ENSGWIG00000015323', 'ENSGWIG00000000671', 'hmg20a', 'sall4', 'zfp64', 'tshz2', 'ENSGWIG00000023668', 'ENSGWIG00000023671', 'ENSGWIG00000023674', 'ENSGWIG00000023677', 'cbln4', 'ENSGWIG00000023691', 'plekhg5b', 'zbtb48', 'klhl21', 'syt6a', 'olfml3a', 'c1qtnf12', 'hipk1a', 'acap3a', 'fam43b', 'zmp:0000001088', 'ENSGWIG00000010182', 'pink1', 'agmat', 'hspg2', 'ENSGWIG00000011718', 'wu:fi36a10', 'taf8', 'camk1gb', 'itih6', 'ENSGWIG00000015146', 'ccnt1', 'ENSGWIG00000015192', 'ENSGWIG00000015412', 'rbm39b', 'slc12a5b', 'magi1a', 'rho', 'prickle2a', 'ENSGWIG00000021090', 'ENSGWIG00000000805', 'ENSGWIG00000000897', 'ENSGWIG00000001060', 'FBLN2', 'nup210', 'IQSEC1', 'raf1a', 'vgll4a', 'ENSGWIG00000001814', 'spo11', 'ENSGWIG00000002092', 'cdh26.1', 'ENSGWIG00000002102', 'ENSGWIG00000002239', 'kcnk15', 'ogfr', 'ENSGWIG00000005077', 'ENSGWIG00000005099', 'prkcbp1l', 'zgc:113278', 'ENSGWIG00000008592', 'rbm38', 'mipb', 'ENSGWIG00000008609', 'baz2a', 'ptges3b', 'prim1', 'ENSGWIG00000009030', 'adcy6a', 'g6pd', 'ENSGWIG00000015585', 'asic1a', 'ENSGWIG00000025215', 'scube3', 'ENSGWIG00000025998', 'ANKS1A', 'uhrf1bp1', 'ENSGWIG00000026007', 'ENSGWIG00000026008', 'ENSGWIG00000026009', 'syt2a', 'guca1b', 'frmd4bb', 'TAFA3', 'tafa1a', 'ENSGWIG00000000800', 'pm20d1.2', 'mfsd4ab', 'ENSGWIG00000001289', 'camta1a', 'dvl1a', 'rap1gap', 'YTHDF1', 'bhlhe23', 'slc17a9b', 'tasorb', 'ENSGWIG00000001385', 'GRM7', 'DAG1', 'ENSGWIG00000001402', 'DOCK3', 'DCAF1', 'grip2a', 'slc6a6b', 'ENSGWIG00000012098', 'nol4la', 'si:dkey-70p6.1', 'rad21l1', 'ENSGWIG00000012245', 'ENSGWIG00000014730', 'ENSGWIG00000014731', 'iqsec2b', 'kdm5c', 'si:ch73-162j3.4', 'ENSGWIG00000022017', 'sema3ga', 'nkain4', 'ENSGWIG00000007976', 'zbtb46', 'uckl1b', 'si:dkey-256k13.2', 'ENSGWIG00000008094', 'prpf6', 'aar2', 'EMILIN3', 'b4galt5', 'opn7b', 'rnf114', 'ENSGWIG00000009702', 'MAFB', 'zgc:158828', 'plcg1', 'ncoa6', 'GGT7', 'ENSGWIG00000012373', 'snai1b', 'samhd1', 'ENSGWIG00000017108', 'ENSGWIG00000017118', 'ENSGWIG00000017301', 'kcnd3', 'ENSGWIG00000020050', 'cttnbp2nlb', 'ERC2', 'CACNA2D3', 'atp6ap1lb', 'b4galt3', 'bgnb', 'uba1', 'camkvl', 'ENSGWIG00000022972', 'ngfa', 'LGR6', 'tead3a', 'PKP1', 'ENSGWIG00000004858', 'vwa1', 'kcnq2a', 'chrna4b', 'ENSGWIG00000007637', 'ctnnbl1', 'ENSGWIG00000008608', 'itchb', 'ENSGWIG00000019384', 'VDR', 'hdac7a', 'ENSGWIG00000019405', 'chchd6a', 'ENSGWIG00000019458', 'ENSGWIG00000019679', 'ENSGWIG00000019680', 'ENSGWIG00000019681', 'si:ch211-236h17.3', 'ENSGWIG00000019692', 'klf15', 'cpne9', 'lhfpl4a', 'plxnb1a', 'tnnc1b', 'ENSGWIG00000021321', 'myt1a', 'si:dkey-16n15.6', 'suv39h1b', 'ENSGWIG00000021498', 'ENSGWIG00000021499', 'ENSGWIG00000021501', 'ccdc22', 'ENSGWIG00000021528', 'rcc2', 'ENSGWIG00000021593', 'tpx2', 'pdyn', 'stk35', 'MTCH1', 'mthfr', 'clcn6', 'igsf21b', 'stk38a', 'dennd2da', 'ENSGWIG00000008729', 'si:ch211-207e14.4', 'tns2a', 'ENSGWIG00000013591', 'ENSGWIG00000013596', 'krt8', 'krt5', 'klhdc8b', 'ENSGWIG00000013947', 'ENSGWIG00000013953', 'ENSGWIG00000014060', 'ENSGWIG00000014132', 'ENSGWIG00000014754', 'ENSGWIG00000014758', 'kif5aa', 'ENSGWIG00000014801', 'stac3', 'ndufa4l2a', 'actr5', 'ENSGWIG00000014814', 'ENSGWIG00000014821', 'ENSGWIG00000014822', 'ENSGWIG00000014873', 'plagl2', 'hck', 'rbbp8l', 'ENSGWIG00000016710', 'ENSGWIG00000016712', 'gata5', 'ENSGWIG00000016715', 'ENSGWIG00000016742', 'ENSGWIG00000016895', 'ppdpfb', 'ENSGWIG00000016900', 'myt1b', 'tbc1d22b', 'snrpg', 'atp2b3b', 'crebzf', 'ENSGWIG00000006835', 'ENSGWIG00000006940', 'skia', 'ENSGWIG00000012684', 'FAM110A', 'cox4i2', 'bcl2l1', 'ENSGWIG00000012784', 'pard6b', 'ENSGWIG00000012885', 'lamb2l', 'lamb2', 'HES2', 'acot7', 'gpr153', 'ENSGWIG00000019299', 'ENSGWIG00000019724', 'trim33', 'nr1h5', 'mapk13', 'tafa5l', 'ENSGWIG00000020486', 'mtor', 'angptl7', 'agrn', 'ptpn11b', 'CSDE1', 'ENSGWIG00000001403', 'MOV10', 'ENSGWIG00000002458', 'ENSGWIG00000002459', 'zgc:174906', 'cbx5', 'ENSGWIG00000002574', 'RARG', 'si:dkey-21c19.3', 'rbms2b', 'lrp1ab', 'fmnl3', 'ENSGWIG00000010858', 'erbb3b', 'ENSGWIG00000018289', 'ACVR1B', 'acvrl1', 'ankrd33aa', 'znf385a', 'cacna1sb', 'l1cama', 'ENSGWIG00000020200', 'phf8', 'ENSGWIG00000020544', 'avpr2ab', 'galnt6', 'adcy6b', 'rnd1b', 'zgc:66440', 'FIGNL2', 'ENSGWIG00000000333', 'scn8aa', 'dip2ba', 'atf7a', 'larp4aa', 'def6a', 'fance', 'mapk14a', 'cpne5a', 'dclre1b', 'adora1b', 'si:ch211-269m15.3', 'lamtor5', 'slc16a4', 'ENSGWIG00000024559', 'slc6a17', 'sypl2b', 'ENSGWIG00000024734', 'atxn7l2b', 'ca6', 'eno1a', 'rerea', 'samd11', 'noc2l', 'plekhn1', 'ENSGWIG00000025661', 'her9', 'ctnnbip1', 'clstn1', 'lzic', 'kif1b', 'tardbpl', 'ENSGWIG00000026182', 'casz1', 'pex14', 'cntn3a.2', 'ENSGWIG00000000896', 'cntn3a.1', 'ENSGWIG00000001766', 'ENSGWIG00000001782', 'ENSGWIG00000002288', 'ENSGWIG00000002290', 'ttc34', 'rimkla', 'ENSGWIG00000002446', 'ENSGWIG00000002447', 'ENSGWIG00000002448', 'KAZN', 'ela2', 'ENSGWIG00000003715', 'si:ch211-263k4.2', 'mib2', 'ttll10', 'dnali1', 'clcnk', 'epha2b', 'spen', 'nadka', 'gnb1a', 'CFAP74', 'gabrd', 'ENSGWIG00000007375', 'ajap1', 'ENSGWIG00000007390', 'dffb', 'ENSGWIG00000007477', 'dnajc16', 'casp9', 'ENSGWIG00000007514', 'ep300b', 'xpnpep3', 'st13', 'chadla', 'RANGAP1', 'ZC3H7B', 'tefb', 'ENSGWIG00000010671', 'plppr2a', 'tlcd4b', 'cnn1b', 'ELOF1', 'epor', 'znf653', 'gadd45gip1', 'nfixb', 'ENSGWIG00000006129', 'rad23aa', 'gng13b', 'gucy2c', 'ENSGWIG00000006177', 'ldlra', 'ENSGWIG00000006479', 'kri1', 's1pr5a', 'ENSGWIG00000006491', 'ENSGWIG00000006492', 'ankrd54', 'foxred2', 'si:ch211-237i5.4', 'hdac5', 'cyb561', 'pvalb6', 'ENSGWIG00000006617', 'ENSGWIG00000006693', 'ENSGWIG00000006695', 'adsl', 'atf4b', 'mief1', 'ENSGWIG00000007439', 'ENSGWIG00000007451', 'arhgap17b', 'grid2ipa', 'ENSGWIG00000014739', 'fahd1', 'ENSGWIG00000015181', 'ENSGWIG00000015222', 'RPL3L', 'tbl3', 'sept9a', 'jpt1b', 'nup85', 'gga3b', 'mrps7', 'ENSGWIG00000004771', 'srebf1', 'ENSGWIG00000005723', 'ENSGWIG00000005728', 'rhot1a', 'adap2', 'fam20ca', 'nptx2a', 'baiap2l1b', 'lmtk2', 'cacng1b', 'ENSGWIG00000008129', 'ENSGWIG00000008131', 'rfx1a', 'ENSGWIG00000008281', 'fbxw9', 'ENSGWIG00000009616', 'spag9a', 'tob1a', 'wfikkn2a', 'cacna1ia', 'ENSGWIG00000011777', 'ELFN2', 'ENSGWIG00000011810', 'ENSGWIG00000011811', 'ENSGWIG00000011820', 'alkbh7', 'nacc1a', 'cabp5b', 'slc6a16a', 'slc17a7a', 'zgc:55558', 'hspbp1', 'mapk8ip3', 'mapk3', 'pecam1', 'ENSGWIG00000012880', 'nptx1l', 'ENSGWIG00000012882', 'MGAT5B', 'ENSGWIG00000012901', 'ENSGWIG00000012904', 'ENSGWIG00000013116', 'ENSGWIG00000013158', 'cacna1ha', 'grin2aa', 'ENSGWIG00000016217', 'emp2', 'clec16a', 'PRKCB', 'cacng3b', 'dctn5', 'usp7', 'slc2a6', 'clcn7', 'coro7', 'glis2b', 'tfap4', 'ADCY9', 'crebbpb', 'prkar1b', 'pdgfab', 'ENSGWIG00000023060', 'sdr42e2', 'MOSMO', 'prkcab', 'cep112', 'axin2', 'tbc1d16', 'cbx4', 'ENSGWIG00000007697', 'ENSGWIG00000007698', 'desi1a', 'LLGL1', 'ENSGWIG00000010482', 'abca3b', 'ENSGWIG00000017223', 'sox8b', 'usp22', 'sstr2b', 'ENSGWIG00000017298', 'sox9b', 'ENSGWIG00000017310', 'ENSGWIG00000017322', 'ENSGWIG00000017336', 'cyth1a', 'bahcc1a', 'ENSGWIG00000019003', 'ENSGWIG00000005817', 'aatka', 'baiap2a', 'ttyh2l', 'sdk2a', 'si:ch211-114m9.1', 'tbcd', 'epn3a', 'hgs', 'mrpl12', 'gcgra', 'ENSGWIG00000012473', 'trim35', '12', 'ENSGWIG00000013183', 'adgrl1a', 'ENSGWIG00000019888', 'slc27a1a', 'pgls', 'ENSGWIG00000020357', 'lmx1al', 'CHCHD5', 'pbx4', 'brd4', 'stxbp2', 'ENSGWIG00000020732', 'ENSGWIG00000020755', 'xab2', 'notch3', 'rdh8a', 'col5a3a', 'ENSGWIG00000021530', 'olfm2a', 'trip10a', 'ppan', 's1pr2', 'tyk2', 'prkcsh', 'rhbdf1a', 'shisa9b', 'baiap3', 'hs3st2', 'hs3st4', 'ENSGWIG00000011673', 'kdm8', 'axin1', 'micall2a', 'ENSGWIG00000012276', 'psmg3', 'ELFN1', 'mrm2', 'si:dkey-159f12.2', 'si:ch211-278a6.1', 'ENSGWIG00000013032', 'wnt9b', 'map3k14a', 'ENSGWIG00000013093', 'EZH1', 'ENSGWIG00000000295', 'c1ql3b', 'ccdc43', 'mylk5', 'atp6v0a1a', 'stat5a', 'si:ch211-210g13.5', 'rab5c', 'dhx58', 'znf385c', 'cnp', 'ENSGWIG00000000911', 'fbxl20', 'cdk12', 'NEUROD2', 'ppp1r1b', 'stard3', 'itgb3a', 'mrc2', 'vat1', 'eftud2', 'GJC1', 'smarce1', 'ENSGWIG00000013282', 'tspan4b', 'nosip', 'aldh3b1', 'ppp4cb', 'rab11fip4a', 'glyr1', 'ENSGWIG00000017515', 'pdxdc1', 'slc25a23b', 'gps1', 'ENSGWIG00000000845', 'ap2a1', 'ENSGWIG00000004420', 'rbfox1', 'hoxb13a', 'hoxb9a', 'igf2bp1', 'ENSGWIG00000005423', 'st8sia6', 'PLXDC1', 'ENSGWIG00000020344', 'arhgap23a', 'plcd3a', 'kcnj19a', 'si:ch211-198m17.1', 'kcnh6a', 'ddx42', 'rundc3aa', 'fam117aa', 'ENSGWIG00000022820', 'nxph3', 'ngfra', 'ENSGWIG00000024239', 'ENSGWIG00000024241', 'ENSGWIG00000024243', 'mpp3a', 'ENSGWIG00000024267', 'ENSGWIG00000024268', 'ENSGWIG00000004806', 'si:dkey-167k11.5', 'ENSGWIG00000005760', 'tcf20', 'ENSGWIG00000006169', 'ENSGWIG00000006178', 'GPRC5D', 'ENSGWIG00000006183', 'ENSGWIG00000006185', 'ENSGWIG00000006190', 'ENSGWIG00000006261', 'evplb', 'smurf2', 'ENSGWIG00000008340', 'usp43b', 'ntn1b', 'pik3r6b', 'tom1', 'tbc1d10b', 'hook2', 'junbb', 'ubn2b', 'myh9b', 'pkn1b', 'ENSGWIG00000018069', 'WIZ', 'ilf3b', 'sox10', 'ENSGWIG00000018440', 'GGA1', 'CARD10', 'cdc42ep1b', 'LGALS2', 'plbd1', 'col1a1a', 'ENSGWIG00000021139', 'ENSGWIG00000021141', 'ENSGWIG00000021142', 'ENSGWIG00000021151', 'NSF', 'ENSGWIG00000021273', 'ENSGWIG00000022566', 'ENSGWIG00000022650', 'carhsp1', 'nat15', 'prpsap2', 'caskin1', 'ENSGWIG00000023193', 'ENSGWIG00000023196', 'ENSGWIG00000023199', 'map2k4a', 'ENSGWIG00000023904', 'rcvrna', 'ENSGWIG00000023973', 'ENSGWIG00000023976', 'si:ch73-281f12.4', 'ENSGWIG00000024442', 'CACNG2', 'kcna7', 'slc1a9', 'tbc1d17', 'lrrc4ba', 'syt3', 'si:ch211-51c14.1', 'ENSGWIG00000010049', 'ENSGWIG00000010050', 'ENSGWIG00000010051', 'ppp1caa', 'prf1.9', 'si:ch211-262e15.1', 'ENSGWIG00000011146', 'tsen54', 'rbfox3b', 'ENSGWIG00000011970', 'ENSGWIG00000011971', 'cbx2', 'nog1', 'mmd', 'grid2', 'mier3b', 'plk2b', 'pde4d', 'pgm5', 'ubox5', 'slc35d2', 'znf367', 'fbxl17', 'EFNA5', 'ENSGWIG00000009206', 'fbxo4', 'si:dkey-46a10.3', 'oxct1a', 'plcxd3', 'c7a', 'rgs7bpa', 'HTR1A', 'ipo11', 'ptch1', 'AOPEP', 'dock8', 'COMMD10', 'sema6a', 'DMXL1', 'hsd17b4', 'prr16', 'ostf1', 'csgalnact1b', 'loxhd1b', 'fabp1b.1', 'ENSGWIG00000020870', 'ENSGWIG00000020918', 'SCAMP1', 'OTP', 'rasgrf2b', 'msh3', 'adra2b', 'gpat2', 'KCNIP3', 'rassf2a', 'nsd3', 'zgc:63587', 'cltcl1', 'hic2', 'utp15', 'ENSGWIG00000018690', 'adamts12', 'ENSGWIG00000019648', 'ndufs4', 'arl15a', 'slc20a2', 'mthfd2l', 'epgn', 'mccc2', 'si:dkey-84j12.1', 'tars1', 'gas1a', 'ccdc84', 'fam214b', 'ENSGWIG00000023203', 'si:dkey-174n20.1', 'kremen1', 'ENSGWIG00000004831', 'xbp1', 'hspb8', 'suds3', 'taok3a', 'rfc5', 'ksr2', 'nos1', 'ENSGWIG00000016165', 'nipsnap1', 'fbxw8', 'fbxo21', 'nf2a', 'ENSGWIG00000016856', 'zgc:112294', 'ENSGWIG00000016879', 'ENSGWIG00000016882', 'prodhb', 'rtn4r', 'si:ch211-207k7.4', 'notch1b', 'ENSGWIG00000022298', 'ENSGWIG00000023574', 'mrps30', 'secisbp2', 'ENSGWIG00000024285', 'ENSGWIG00000024397', 'ENSGWIG00000024398', 'gak', 'ENSGWIG00000024562', 'wdfy3', 'ENSGWIG00000025076', 'unc13ba', 'mapk1', 'slc4a4a', 'NPFFR2', 'adamts3', 'ENSGWIG00000018591', 'dmrt1', 'kank1a', 'fbp2', 'ctsla', 'dapk1', 'atp8b5a', 'CNNM4', 'ARVCF', 'ENSGWIG00000021656', 'sema4c', 'ENSGWIG00000021665', 'kat6a', 'ap3m2', 'znf703', 'ENSGWIG00000023165', 'pnpla7b', 'araf', 'traf2a', 'tnc', 'ENSGWIG00000023515', 'nsmfb', 'ENSGWIG00000023537', 'htra4', 'ENSGWIG00000023560', 'unc5db', 'loxl2b', 'zmat4b', 'tcima', 'adam28', 'vps37b', 'zgc:110329', 'tacr1a', 'ncaph', 'bicdl1', 'citb', 'si:dkey-91m11.5', 'ENSGWIG00000011665', 'cux2b', 'MTMR3', 'ENSGWIG00000020695', 'smad2', 'ENSGWIG00000020727', 'ENSGWIG00000020730', 'cacna1ba', 'ehmt1a', 'arrdc1a', 'ENSGWIG00000026305', 'pip5k1bb', 'cert1a', 'sv2ca', 'ENSGWIG00000026551', 'si:ch211-130m23.3', 'ENSGWIG00000026561', 'ENSGWIG00000026566', 'hapln1a', 'edil3a', 'cox7c', 'ccnh', 'mef2cb', 'si:dkey-40c11.2', 'rhobtb4', 'wdr54', 'ENSGWIG00000026710', 'ENSGWIG00000026711', 'fam172a', 'ENSGWIG00000026717', 'slf1', 'mctp1a', 'ENSGWIG00000001466', 'adgrv1', 'si:dkey-40c11.1', 'lzts1', 'gfra2a', 'bco2l', 'ENSGWIG00000001840', 'trpm3', 'cemip2', 'entpd4', 'ENSGWIG00000015498', 'nkx2.7', 'stc1', 'ido1', 'PEBP4', 'bin3', 'ENSGWIG00000021460', 'nfu1', 'antxr1a', 'ENSGWIG00000022947', 'adrb3a', 'got1l1', 'prlhr2a', 'pdlim2', 'inpp5l', 'dbnlb', 'ube2d4', 'tcf7l1a', 'zgc:153044', 'ENSGWIG00000025201', 'mn1b', 'TTC28', 'ENSGWIG00000025254', 'rph3ab', 'adra1aa', 'igsf9b', 'es1', 'rorb', 'elmod3', 'hpdb', 'ENSGWIG00000006838', 'aplnra', 'tbc1d10aa', 'sf3a1', 'DTX1', 'slc20a1a', 'vgll4l', 'ENSGWIG00000007722', 'ENSGWIG00000007750', 'ENSGWIG00000008068', 'lmx1bb', 'zbtb34', 'ANGPTL2', 'ENSGWIG00000008110', 'ENSGWIG00000008117', 'zgc:154046', 'asb6', 'si:dkeyp-14d3.1', 'slc15a4', 'glt1d1', 'TMEM132D', 'fzd10', 'rimbp2', 'ENSGWIG00000009029', 'stx2b', 'adgrd1', 'U6', 'ncor2', 'rflna', 'ccdc92', 'chmp7', 'lgi3', 'fgfr1a', 'ankrd39', 'kctd9a', 'npy8br', 'klhl22', 'tbx16', 'cabin1', 'dock5', 'vsig8b', 'ENSGWIG00000020936', 'ENSGWIG00000001467', 'pxnb', 'ENSGWIG00000001679', 'si:ch73-138n13.1', 'pop5', 'unc119.1', 'prkab1b', 'ENSGWIG00000002185', 'ENSGWIG00000002227', 'golga3', 'trmt1l', 'ENSGWIG00000008342', 'ENSGWIG00000008345', 'ENSGWIG00000008348', 'galnt9', 'ENSGWIG00000008364', 'atad1a', 'ENSGWIG00000008369', 'htr7c', 'tnksb', 'idua', 'ENSGWIG00000009063', 'ENSGWIG00000009069', 'ENSGWIG00000009068', 'zdhhc8b', 'ankle2', 'ENSGWIG00000009211', 'ENSGWIG00000009215', 'med27', 'ENSGWIG00000009228', 'endog', 'ENSGWIG00000009237', 'ENSGWIG00000009240', 'ENSGWIG00000009241', 'adam9', 'ENSGWIG00000009537', 'ENSGWIG00000004849', 'ENSGWIG00000004902', 'kazald3', 'ENSGWIG00000005644', 'ENSGWIG00000011883', 'qsox2', 'ENSGWIG00000011902', 'sh3glb2a', 'ENSGWIG00000013894', 'zfr', 'CRB2', 'lhx2b', 'ENSGWIG00000001573', 'nr5a1a', 'nr6a1a', 'olfml2a', 'ENSGWIG00000001706', 'akr1a1a', 'vps13a', 'rfk', 'pcsk5b', 'ENSGWIG00000007739', 'ENSGWIG00000007862', 'mlxip', 'ENSGWIG00000007872', 'clip1a', 'rsrc2', 'kntc1', 'PITPNM2', 'mthfd2', 'smtnb', 'inpp5jb', 'usp39', 'sftpbb', 'ENSGWIG00000010367', 'ebf2', 'ank1a', 'ENSGWIG00000011485', 'dguok', 'si:ch211-106a19.1', 'rasl10a', 'gas2l1', 'rhbdd3', 'atp6v0a2b', 'si:ch211-225b11.4', 'morc2', 'coro1ca', 'alkbh2', 'ankrd13a', 'git2a', 'fam222a', 'mvk', 'kctd10', 'ENSGWIG00000020594', 'pes', 'si:dkey-98f17.5', 'slc2a11b', 'ENSGWIG00000021134', 'ENSGWIG00000021135', 'ENSGWIG00000021136', 'wscd2', 'ENSGWIG00000022357', 'si:ch211-166a6.5', 'ENSGWIG00000022415', 'ENSGWIG00000022421', 'ccn1l2', 'fgf17', 'ENSGWIG00000022575', 'ENSGWIG00000022576', 'nelfb', 'homer1b', 'cmya5', 'mtx3', 'cdk7', 'setbp1', 'slc14a2', 'lnpep', 'rgmb', 'ENSGWIG00000003349', 'si:ch73-266o15.4', 'ENSGWIG00000003404', 'ENSGWIG00000003417', 'si:ch73-72b7.1', 'macrod1', 'zbtb3', 'ENSGWIG00000008234', 'si:ch73-45o6.2', 'si:dkeyp-115e12.6', 'ENSGWIG00000008768', 'si:zfos-80g12.1', 'ubl3b', 'prss23', 'cfl1', 'fibpa', 'mrpl11', 'tmem265', 'kdrl', 'dlg3', 'ENSGWIG00000017308', 'ENSGWIG00000017318', 'nkx1.2lb', 'sil1', 'csnk1a1', 'ENSGWIG00000017359', 'ENSGWIG00000017369', 'ENSGWIG00000017375', 'ENSGWIG00000017386', 'zgc:153146', 'ENSGWIG00000017397', 'gfra3', 'ENSGWIG00000017798', 'ENSGWIG00000017801', 'si:ch1073-303k11.2', 'SLC16A2', 'f8', 'pmt', 'mid2', 'si:ch211-220e11.3', 'pcdh11', 'ENSGWIG00000018718', 'ENSGWIG00000018719', 'klhl4', 'AHNAK', 'eef1g', 'ENSGWIG00000019316', 'ENSGWIG00000019327', 'chrnb3b', 'CHRNA6', 'chrnb5b', 'ENSGWIG00000019380', 'clcf1', 'slc43a1a', 'si:dkey-6i22.5', 'clp1', 'ENSGWIG00000004998', 'cryba1l2', 'ENSGWIG00000008076', 'ENSGWIG00000008084', 'PPP2R2C', 'wfs1b', 'jakmip1', 'ENSGWIG00000009500', 'DOCK2', 'ENSGWIG00000011586', 'polr1a', 'st3gal5', 'ENSGWIG00000016623', 'hspa9', 'pcdh10b', 'ENSGWIG00000020435', 'ENSGWIG00000020436', 'pcdh18b', 'slc7a11', 'ENSGWIG00000020539', 'enpp6', 'trpc7b', 'DIAPH1', 'ENSGWIG00000020828', 'ENSGWIG00000005093', 'si:ch73-233f7.1', 'fbxw11a', 'etf1a', 'ENSGWIG00000005722', 'spock1', 'bicc2', 'ENSGWIG00000005733', 'APBB2', 'ube2ka', 'ENSGWIG00000005827', 'si:ch211-203d1.3', 'tcirg1b', 'ENSGWIG00000005010', 'ndst1a', 'rbm22', 'myoz3a', 'rgs14a', 'ENSGWIG00000005591', 'adra2da', 'si:ch211-199b20.3', 'cnot6b', 'ENSGWIG00000005765', 'ENSGWIG00000008196', 'KIAA1191', 'gfpt2', 'ENSGWIG00000012705', 'RASGEF1C', 'ENSGWIG00000012782', 'ENSGWIG00000012785', 'FAM53C', 'ctbp1', 'REEP2', 'egr1', 'ENSGWIG00000016652', 'ENSGWIG00000016654', 'adra1bb', 'slu7', 'atp7a', 'gabrb2a', 'gabra6a', 'sowahab', 'kctd16a', 'sh3rf2', 'zdhhc15b', 'abcb7', 'nexmifb', 'ENSGWIG00000004997', 'frmpd3', 'ENSGWIG00000005310', 'rhogd', 'ENSGWIG00000005324', 'ogt.1', 'nlgn3a', 'ENSGWIG00000008714', 'gria3b', 'eda', 'cx31.7', 'vma21', 'mtnr1c', 'neurl1b', 'dusp1', 'flt4', 'trim105', 'npy7r', 'ENSGWIG00000014473', 'ENSGWIG00000014476', 'unc5a', 'hbegfa', 'rufy1', 'ENSGWIG00000018872', 'n4bp3', 'ENSGWIG00000018891', 'gpx3', 'dctn4', 'ENSGWIG00000024467', 'tspan17', 'mchr2', 'ctnna1', 'sncb', 'ENSGWIG00000024713', 'nek12', 'ENSGWIG00000024722', 'ENSGWIG00000024724', 'cpeb4a', 'stc2a', 'nkx2.5', 'ppp2r2ca', 'adam19b', 'il12ba', 'sh3tc2', 'ablim3', 'afap1l1a', 'gabrp', 'foxi3b', 'tmsb2', 'gria1a', 'ENSGWIG00000007607', 'si:dkey-34l15.2', 'eif4ebp3', 'efnb1', 'arhgef9b', 'tenm1', 'ENSGWIG00000020702', 'hdx', 'rps6kal', 'ENSGWIG00000025626', 'ENSGWIG00000025627', 'ENSGWIG00000025630', 'ENSGWIG00000025640', 'ENSGWIG00000025644', 'slc7a2', 'mtmr7a', 'ENSGWIG00000025806', 'fam122b', 'phf6', 'gpc3', 'gpc4', 'brd8', 'ppp3cb', 'ube2d2', 'ENSGWIG00000026187', 'psd2', 'nrg2b', 'si:dkey-160o24.3', 'ENSGWIG00000001641', 'gabra4', 'GABRA2', 'si:ch73-49o8.1', 'gpr137', 'LONRF3', 'ENSGWIG00000002386', 'ENSGWIG00000002388', 'fmr1', 'ENSGWIG00000003479', 'hdac3', 'myot', 'ENSGWIG00000003690', 'il1rapl2', 'ENSGWIG00000007323', 'ENSGWIG00000007330', 'snx25', 'rab41', 'P2RY4', 'ENSGWIG00000008159', 'atp11c', 'mcf2a', 'FGF13', 'zic6', 'ENSGWIG00000014738', 'gpr101', 'tm9sf5', 'arhgef6', 'map7d3', 'fhl1a', 'drp2', 'cstf2', 'bcorl1', 'ENSGWIG00000019910', 'mid1ip1l', 'tspan7', 'ENSGWIG00000021048', 'xpnpep2', 'urp1', 'atp1b4', 'upf3b', 'sept6', 'elf1', 'tnmd', 'pcdh19', 'diaph2', 'glra4a', 'btk', 'mars2', 'med12', 'si:ch211-221f10.2', 'enox2', 'SLC4A11', 'gfra4b', 'atp8a1', 'grxcr1a', 'ENSGWIG00000021625', 'HS3ST1', 'ENSGWIG00000021627', 'ENSGWIG00000021632', 'aurkb', 'slit3', 'fgfrl1a', 'cxcl14', 'ENSGWIG00000022880', 'lingo2a', 'ENSGWIG00000022883', 'ENSGWIG00000022884', 'eif4ea', 'zgc:77938', 'elovl6', 'egf', 'gpm6aa', 'asb5a', 'si:dkey-237h12.3', 'ENSGWIG00000023620', 'aga', 'pdgfc', 'GLRB', 'gask1b', 'tmem144b', 'rxfp1', 'etfdh', 'rapgef2', 'ENSGWIG00000013525', 'fstl5', 'qdpra', 'ldb2a', 'prom1a', 'bbs7', 'tmem33', 'phox2bb', 'limch1b', 'ENSGWIG00000015331', 'ENSGWIG00000015335', 'ENSGWIG00000015339', 'fgf24', 'npm1b', 'tlx2', 'adad1', 'SPATA5', 'ANKRD50', 'ENSGWIG00000015472', 'U3', 'ENSGWIG00000015479', 'sec24b', 'NCBP1', 'ppargc1b', 'arhgef37', 'ENSGWIG00000020637', 'PCGF3', 'ENSGWIG00000020672', 'glra1', 'thoc3', 'emx3', 'si:rp71-1d10.5', 'ENSGWIG00000002676', 'ENSGWIG00000002680', 'pcdh1b', 'ENSGWIG00000002683', 'atp9b', 'ENSGWIG00000020995', 'hivep1', 'ENSGWIG00000005345', 'stpg1', 'fhl3b', 'tfap2e', 'CLSPN', 'cdca8', 'ENSGWIG00000012129', 'ENSGWIG00000012167', 'ENSGWIG00000012867', 'prdm1b', 'ENSGWIG00000013478', 'sdc2', 'angpt1', 'ext1b', 'ENSGWIG00000013668', 'TGFBR2', 'STT3B', 'OSBPL10', 'tmem42b', 'nrm', 'ENSGWIG00000014885', 'csnk2b', 'prrc2a', 'skiv2l', 'ENSGWIG00000019990', 'ENSGWIG00000019991', 'adgrb1b', 'atp5if1a', 'ENSGWIG00000020290', 'dnajc8', 'cers2a', 'ENSGWIG00000001625', 'ENSGWIG00000001629', 'ciarta', 'crabp2b', 'ENSGWIG00000003591', 'efna1a', 'efna3a', 'cks1b', 'pbxip1a', 's100u', 'SHE', 'pygo2', 'PIP5K1A', 'necab1', 'tmem64', 'zgc:111986', 'ENSGWIG00000004584', 'ENSGWIG00000004604', 'zgc:63863', 'DGAT1', 'SLC45A4', 'dennd3a', 'ENSGWIG00000009877', 'rplp1', 'ENSGWIG00000011277', 'trps1', 'csmd3a', 'ENSGWIG00000011602', 'ENSGWIG00000011604', 'ENSGWIG00000011632', 'ENSGWIG00000011649', 'ENSGWIG00000011682', 'sf3b4', 'pabpc1b', 'ywhaz', 'zgc:136493', 'spag1a', 'pard6gb', 'jarid2b', 'dtnbp1b', 'mylipa', 'gmpr', 'atxn1a', 'fam8a1a', 'znf704', 'ENSGWIG00000001971', 'hey1', 'si:dkeyp-120h9.1', 'ripor2', 'AKAP9', 'ENSGWIG00000004703', 'ENSGWIG00000004708', 'ENSGWIG00000004748', 'RPS19', 'cap2', 'mapk15', 'fam83hb', 'ankib1a', 'azin1b', 'ENSGWIG00000005463', 'fam92a1', 'triqk', 'runx1t1', 'ZNF385D', 'TINAGL1', 'nfyc', 'ENSGWIG00000002039', 'rprd1a', 'fkbp9', 'susd5', 'clasp2', 'pdcd6ip', 'cyhr1', 'pum1', 'SDC3', 'ENSGWIG00000019883', 'elmo1', 'si:dkey-184p18.2', 'gabbr2', 'ctnnal1', 'sla1a', 'ccn4a', 'dync1li1', 'cspg5b', 'smarcc1b', 'klhl7', 'si:dkey-240h12.4', 'rapgef5a', 'ENSGWIG00000023277', 'ENSGWIG00000023295', 'snx13', 'ENSGWIG00000024171', 'cdk6', 'hepacam2', 'vps50', 'calcr', 'zgc:85777', 'bet1', 'col1a2', 'sgce', 'ppp1r9a', 'DYNC1I1', 'sem1', 'dlx6a', 'npy', 'kcnh8', 'satb1a', 'oxnad1', 'wipf3', 'ENSGWIG00000007966', 'ENSGWIG00000007967', 'nfe2l3', 'snx10a', 'hoxa1a', 'ENSGWIG00000008000', 'hoxa13a', 'hibadha', 'ENSGWIG00000010818', 'smap2', 'trappc3', 'col8a2', 'ptp4a2b', 'ENSGWIG00000010831', 'ENSGWIG00000010833', 'ENSGWIG00000010834', 'adgrb2', 'col9a2', 'angpt2b', 'ENSGWIG00000011118', 'csmd2', 'ENSGWIG00000011627', 'ENSGWIG00000011629', 'cx39.4', 'ENSGWIG00000011646', 'dlgap3', 'SLC6A19', 'ENSGWIG00000011982', 'mfsd2ab', 'myclb', 'trit1', 'fam49al', 'txnl4a', 'ENSGWIG00000012446', 'ENSGWIG00000012618', 'trim71', 'ENSGWIG00000005157', 'rpl14', 'ENSGWIG00000005441', 'ENSGWIG00000008920', 'ENSGWIG00000009031', 'ENSGWIG00000009033', 'si:dkeyp-46h3.1', 'adcy2b', 'srd5a1', 'irx4b', 'mrpl36', 'nrsn1l', 'sox4a', 'ENSGWIG00000013393', 'ENSGWIG00000013395', 'ENSGWIG00000013396', 'fndc5b', 'ENSGWIG00000013405', 'stmn1b', 'ENSGWIG00000013991', 'ENSGWIG00000014035', 'phactr4a', 'ENSGWIG00000014074', 'oprd1a', 'ENSGWIG00000014084', 'ENSGWIG00000014093', 'tent5bb', 'arid1ab', 'ENSGWIG00000014131', 'tpbga', 'tmem222b', 'ENSGWIG00000014143', 'si:dkey-218f9.10', 'kcnk9', 'ENSGWIG00000014173', 'FAM135B', 'ENSGWIG00000014846', 'ENSGWIG00000014847', 'KHDRBS3', 'ENSGWIG00000001536', 'cpsf1', 'oxsm', 'ENSGWIG00000004384', 'thsd7aa', 'nxph1', 'phf14', 'glcci1a', 'rpa3', 'COL28A1', 'sdhaf3', 'tac1', 'ENSGWIG00000008034', 'ENSGWIG00000008041', 'ENSGWIG00000008050', 'top2b', 'ENSGWIG00000009806', 'ENSGWIG00000009809', 'ENSGWIG00000009811', 'ENSGWIG00000009812', 'ENSGWIG00000009837', 'ENSGWIG00000010275', 'thrb', 'ENSGWIG00000010302', 'nkiras1', 'ENSGWIG00000010309', 'ENSGWIG00000010310', 'lin28a', 'ENSGWIG00000010337', 'foxo6b', 'ENSGWIG00000010340', 'ENSGWIG00000010341', 'cited4a', 'ctps1a', 'rab42a', 'gmeb1', 'ptprua', 'ENSGWIG00000010394', 'ENSGWIG00000010395', 'ENSGWIG00000010397', 'ENSGWIG00000008572', 'si:ch73-234b20.5', 'enoph1', 'sec31a', 'lin54', 'cops4', 'plac8.1', 'cwc27', 'trim23', 'cert1b', 'ENSGWIG00000008165', 'ch25hl2', 'pde8b', 'ap3b1a', 'ARSB', 'JMY', 'ENSGWIG00000010915', 'ENSGWIG00000011153', 'gnsb', 'ENSGWIG00000013173', 'ogdhb', 'ENSGWIG00000013511', 'ENSGWIG00000013565', 'ENSGWIG00000014534', 'ENSGWIG00000014543', 'ppp6c', 'pkn3', 'zdhhc12b', 'ppp1r3b', 'ENSGWIG00000005893', 'gpat3', 'ENSGWIG00000005956', 'hpse', 'COX5B', 'hsdl2', 'fer', 'cdc26', 'pisd', 'RNF224', 'apc', 'ENSGWIG00000008039', 'ENSGWIG00000008043', 'whrna', 'ENSGWIG00000009960', 'ENSGWIG00000010003', 'ehmt1b', 'tprn', 'npr2', 'ccnb1', 'ENSGWIG00000015040', 'ENSGWIG00000015126', 'slc30a5', 'pip5k1ba', 'efna5a', 'ENSGWIG00000018576', 'ENSGWIG00000018581', 'ENSGWIG00000018583', 'gadd45gb.1', 'ENSGWIG00000018599', 'si:ch211-22d5.2', 'ENSGWIG00000018600', 'c7b', 'slc31a1', 'zcchc9', 'ENSGWIG00000007243', 'ENSGWIG00000007250', 'ENSGWIG00000012798', 'scarb1', 'grk5', 'ENSGWIG00000016345', 'ENSGWIG00000016398', 'gas1b', 'dab2', 'ENSGWIG00000016772', 'ENSGWIG00000016778', 'ENSGWIG00000016779', 'unc5da', 'ENSGWIG00000016871', 'erbin', 'MAST4', 'palm2akap2', 'ENSGWIG00000021318', 'si:dkey-247m21.3', 'ENSGWIG00000021373', 'dusp4', 'spra', 'dnajc25', 'ENSGWIG00000021827', 'THAP1', 'ENSGWIG00000005321', 'dcc', 'ENSGWIG00000006518', 'ENSGWIG00000018114', 'txnl1', 'atp8b1', 'TCF4', 'ENSGWIG00000021613', 'rab27b', 'onecut2', 'st8sia3', 'WDR7', 'shroom3', 'parn', 'slc38a9', 'ENSGWIG00000023915', 'pomk', 'pam', 'pdzph1', 'reep5', 'chd1', 'syk', 'vav2', 'thnsl2', 'ror2', 'ENSGWIG00000024664', 'auh', 'TBC1D13', 'ENSGWIG00000024839', 'ENSGWIG00000024850', 'fam102ab', 'fpgs', 'ENSGWIG00000024872', 'ENSGWIG00000024875', 'ENSGWIG00000024876', 'si:dkey-28o19.1', 'ENSGWIG00000024879', 'phf24', 'pigo', 'lpar1', 'MUSK', 'rassf6', 'ENSGWIG00000002313', 'ENSGWIG00000002314', 'ccdc97', 'npdc1a', 'nsmfa', 'si:ch73-62l21.1', 'bcr', 'pus1', 'noc4l', 'si:ch73-287m6.1', 'ENSGWIG00000015593', 'si:dkey-112m2.1', 'HOOK3', 'TRABD2A', 'ENSGWIG00000019006', 'sfswap', 'ENSGWIG00000019015', 'mmp17a', 'ulk1b', 'ENSGWIG00000019037', 'ENSGWIG00000019213', 'ENSGWIG00000019240', 'skor2', 'ENSGWIG00000019482', 'ENSGWIG00000019484', 'cntfr', 'si:ch211-282j22.3', 'exd3', 'nrarpa', 'sh2d3ca', 'sardh', 'strbp', 'GPR21', 'rabgap1', 'jak2a', 'rbm18', 'lhx6', 'ndufa8', 'si:ch211-243g18.2', 'nnt', 'fgf10a', 'ENSGWIG00000018566', 'rai14', 'scarb2c', 'ENSGWIG00000018586', 'slc5a1', 'GNAZ', 'sppl3', 'slc2a11l', 'kcnn2', 'loxa', 'ENSGWIG00000021150', 'pptc7b', 'cabp1b', 'rnf34b', 'ENSGWIG00000022510', 'ENSGWIG00000022514', 'ssh1b', 'SUSD2', 'hspb15', 'tescb', 'uqcr10', 'prodha', 'ENSGWIG00000004594', 'ENSGWIG00000005620', 'ENSGWIG00000005621', 'ENSGWIG00000005635', 'WDR70', 'gdnfa', 'opn4xa', 'ENSGWIG00000008238', 'nup155', 'sdad1', 'ptpn13', 'mapk10', 'arhgap24', 'ENSGWIG00000013015', 'hcn1', 'ccbe1', 'cplx4b', 'agxt2', 'dym', 'smad7', 'ctif', 'zgc:86839', 'add1', 'nop14', 'ENSGWIG00000014299', 'ENSGWIG00000014301', 'ZNF462', 'TMEM38B', 'PRAG1', 'fam78ab', 'ENSGWIG00000014333', 'NACC2', 'TMEM250', 'LHX3', 'gpsm1b', 'ak1', 'ENSGWIG00000014450', 'brinp1', 'ENSGWIG00000014453', 'ENSGWIG00000014454', 'ENSGWIG00000014455', 'ASTN2', 'pappaa', 'znf618', 'ENSGWIG00000001633', 'ENSGWIG00000001640', 'traf2b', 'lman2la', 'MED13L', 'tbx5a', 'abl1', 'RIC1', 'UHRF2', 'ENSGWIG00000010824', 'slc45a2', 'btc', 'csgalnact1a', 'si:dkey-27b3.2', 'itga1', 'ENSGWIG00000012290', 'zgc:73226', 'thbs4b', 'notch1a', 'camsap1a', 'MIR455', 'ENSGWIG00000001970', 'lrrc8aa', 'dnlz', 'rapgef1b', 'ntng2b', 'spout1', 'si:ch211-182d3.1', 'ptger4b', 'lamc3', 'fibcd1', 'dipk1b', 'olfm1b', 'ENSGWIG00000012673', 'fnbp1b', 'tyw1', 'galnt17', 'hif1al', 'sae1', 'zc3h4', 'zgc:154093', 'foxo1a', 'cog6', 'fndc3a', 'mlnr', 'prcp', 'FAM181B', 'tenm4', 'ENSGWIG00000012394', 'pgr', 'arhgap42b', 'cntn5', 'ENSGWIG00000019538', 'nectin1b', 'ENSGWIG00000019545', 'treh', 'ENSGWIG00000019583', 'ENSGWIG00000019586', 'cdon', 'bckdha', 'aplp2', 'ENSGWIG00000021744', 'tmem91', 'ppp1r13l', 'SLC7A1', 'gbe1b', 'cadm2b', 'grik1b', 'tiam1b', 'dchs1a', 'arfip2a', 'trim3a', 'frem2b', 'postnb', 'nbeab', 'stard13b', 'ENSGWIG00000011611', 'ENSGWIG00000011686', 'b3glcta', 'ENSGWIG00000013136', 'ENSGWIG00000013139', 'ENSGWIG00000013140', 'stim1a', 'si:dkey-24p1.6', 'robo2', 'robo1', 'ENSGWIG00000018244', 'apbb1', 'ENSGWIG00000018301', 'ENSGWIG00000018409', 'ENSGWIG00000018410', 'ENSGWIG00000018441', 'zgc:174895', 'traf4a', 'tada2a', 'synrg', 'hnf1ba', 'ENSGWIG00000005980', 'ENSGWIG00000005983', 'si:dkey-243i1.1', 'rpa1', 'rtn4rl1b', 'dph1', 'ENSGWIG00000007678', 'hic1', 'ENSGWIG00000008957', 'lrfn1', 'zgc:162339', 'tmprss5', 'zbtb16b', 'ncam1b', 'cadm1b', 'ENSGWIG00000014992', 'esamb', 'ENSGWIG00000015296', 'ENSGWIG00000019409', 'si:dkey-4p15.5', 'picalmb', 'zpld1b', 'alcamb', 'zgc:112083', 'ENSGWIG00000024473', 'MME', 'MBNL1', 'SUCNR1', 'ENSGWIG00000002620', 'si:dkeyp-97b10.3', 'itpkcb', 'lxn', 'cux1b', 'ENSGWIG00000010991', 'sptssb', 'PPM1L', 'ENSGWIG00000013414', 'clasrp', 'znf296', 'egln2', 'ENSGWIG00000013432', 'npas1', 'arhgap35a', 'ap2s1', 'rab4b', 'rhoub', 'ppm1na', 'ptgir', 'ppp1r37', 'sipa1l3', 'map3k10', 'fkrp', 'prkd2', 'gpr184', 'si:dkey-202l22.3', 'dscaml1', 'dock10', 'shox2', 'ENSGWIG00000005954', 'sik2b', 'si:dkey-103g5.3', 'zgc:162608', 'ENSGWIG00000006118', 'ankk1', 'drd2a', 'scn3b', 'ENSGWIG00000006146', 'ENSGWIG00000006149', 'ENSGWIG00000006156', 'ENSGWIG00000006196', 'oafa', 'tlcd5a', 'ENSGWIG00000006207', 'cbl', 'mcamb', 'rnf26', 'nipsnap2', 'ENSGWIG00000009490', 'limk1a', 'ENSGWIG00000013866', 'mnta', 'mettl16', 'PAFAH1B1', 'cluha', 'zgc:153372', 'unc119a', 'cpda', 'aipl1', 'ENSGWIG00000020227', 'abhd15a', 'pde3a', 'vps53', 'zgc:172282', 'ENSGWIG00000020943', 'zgc:162730', 'si:ch211-136a13.1', 'yif1b', 'ENSGWIG00000020980', 'ARHGEF17', 'P2RY6', 'fam168a', 'cwf19l2', 'aasdhppt', 'gria4a', 'pdgfd', 'ENSGWIG00000012095', 'DYNC2H1', 'fzd4', 'RAB38', 'grm5b', 'ENSGWIG00000013121', 'ENSGWIG00000013326', 'inppl1a', 'stard10', 'ARAP1', 'PDE2A', 'ENSGWIG00000014745', 'ENSGWIG00000014749', 'hip1', 'baz1b', 'fzd9b', 'grk7b', 'RASA2', 'ENSGWIG00000020474', 'dner', 'pid1', 'sphkap', 'daw1', 'zgc:158328', 'ENSGWIG00000024589', 'mecom', 'ENSGWIG00000024624', 'ENSGWIG00000024631', 'slitrk3a', 'ENSGWIG00000024768', 'ENSGWIG00000024803', 'ENSGWIG00000024950', 'ENSGWIG00000024953', 'pdcd10a', 'serpini1', 'ccnl1a', 'ENSGWIG00000024979', 'TIPARP', 'KCNAB1', 'ENSGWIG00000002363', 'atm', 'dscamb', 'ENSGWIG00000002793', 'ENSGWIG00000002797', 'get1', 'mipep', 'sacs', 'ENSGWIG00000003905', 'vaspb', 'snrpd2', 'gramd1bb', 'ENSGWIG00000007701', 'ENSGWIG00000007703', 'barx2', 'KIRREL3', 'ENSGWIG00000009512', 'smtla', 'kcnj5', 'st3gal4', 'srpra', 'fam118b', 'ilvbl', 'ENSGWIG00000011703', 'b3gat1a', 'siae', 'tsku', 'gucy2f', 'gdpd5b', 'samsn1a', 'ENSGWIG00000012675', 'vwa5a', 'rap1gap2a', 'ENSGWIG00000016674', 'ENSGWIG00000016676', 'ENSGWIG00000016677', 'ankrd13b', 'igsf11', 'ENSGWIG00000001514', 'ENSGWIG00000001522', 'ENSGWIG00000001533', 'ENSGWIG00000001541', 'ENSGWIG00000001547', 'ENSGWIG00000001553', 'ENSGWIG00000001657', 'gap43', 'sik3', 'dpf1', 'numbl', 'slc5a2', 'ENSGWIG00000007300', 'psma6l', 'otx5', 'ENSGWIG00000007465', 'ENSGWIG00000007489', 'ENSGWIG00000008214', 'meis3', 'ITM2C', 'her13', 'PER2', 'pik3cb', 'ENSGWIG00000012731', 'mrps22', 'lpar5b', 'si:ch211-276c2.2', 'ENSGWIG00000013036', 'ENSGWIG00000013043', 'ENSGWIG00000013050', 'ENSGWIG00000013056', 'ksr1b', 'wsb1', 'cuedc1b', 'akap1b', 'U2', 'ENSGWIG00000013315', 'SMCO4', 'nlk2', 'nlrx1', 'gkup', 'ENSGWIG00000013366', 'ENSGWIG00000013373', 'ENSGWIG00000013378', 'ENSGWIG00000013384', 'dhrs13b.1', 'ENSGWIG00000006091', 'gemin4', 'ENSGWIG00000006114', 'ngef', 'taar10b', 'phf12b', 'sez6b', 'pipox', 'myo18ab', 'ENSGWIG00000008740', 'si:ch211-255f4.7', 'lhx1a', 'ENSGWIG00000008747', 'ENSGWIG00000008750', 'smtnl', 'myo1cb', 'ENSGWIG00000011099', 'prss16', 'psmd2', 'ENSGWIG00000012823', 'ENSGWIG00000020118', 'alp3', 'sat1a.1', 'lipia', 'phldb2a', 'ENSGWIG00000010371', 'sms', 'cxxc5a', 'zgc:100829', 'nrg2a', 'pygma', 'ENSGWIG00000018883', 'ENSGWIG00000019018', 'ENSGWIG00000021883', 'pcnx3', 'arhgap42a', 'angptl5', 'dynll2b', 'rskrb', 'slc47a1', 'si:dkey-175g6.2', 'tmem135', 'cadm2a', 'paxbp1', 'ENSGWIG00000003836', 'stoml3b', 'si:ch211-287a12.9', 'thsd1', 'ENSGWIG00000006542', 'mtnr1bb', 'grm5a', 'grik1a', 'tiam1a', 'hunk', 'synj1', 'si:dkey-5g14.1', 'ENSGWIG00000018279', 'arfip2b', 'fhdc3', 'trarg1b', 'ENSGWIG00000018625', 'sept4a', 'ENSGWIG00000019559', 'mrps17', 'ENSGWIG00000019573', 'ENSGWIG00000019849', 'ENSGWIG00000001521', 'ENSGWIG00000002103', 'ENSGWIG00000002115', 'caln2', 'ssh2b', 'rtn4rl1a', 'RPA1', 'smyd4', 'ENSGWIG00000006371', 'fryb', 'zar1l', 'kl', 'sparta', 'ccna1', 'trpc4a', 'ufm1', 'ENSGWIG00000019179', 'sh3pxd2b', 'GABRB2', 'gabra1', 'gabrg2', 'ccni2', 'hspa4a', 'ARHGAP26', 'kctd16b', 'GRXCR2', 'ENSGWIG00000021313', 'lars1b', 'rbm27', 'pou4f3', 'rad50', 'drd1b', 'ENSGWIG00000022777', 'nsg2', 'CPEB4', 'slc7a3a', 'si:ch211-200p22.4', 'ENSGWIG00000007612', 'STK32A', 'pdlim4', 'ENSGWIG00000015567', 'slc22a21', 'ENSGWIG00000015586', 'slc22a5', 'ENSGWIG00000021237', 'fstl4', 'ENSGWIG00000022027', 'ENSGWIG00000022028', 'vdac1', 'ENSGWIG00000022039', 'ubtd2', 'IL12B', 'rnf145a', 'EBF1', 'crebrf', 'slc13a2', 'larp1', 'macroh2a1', 'ENSGWIG00000022914', 'ddx46', 'camlg', 'sar1b', 'jade2', 'ube2b', 'tcf7', 'ENSGWIG00000024155', 'ENSGWIG00000024157', 'fgfr4', 'ENSGWIG00000024488', 'rab24', 'pgk1', 'gria4b', 'elmod1', 'capn5b', 'ompb', 'ENSGWIG00000008424', 'ENSGWIG00000008447', 'mcama', 'si:ch211-113j13.2', 'ENSGWIG00000013244', 'psmg1', 'pou2f3', 'ppm1e', 'usp2b', 'tbcelb', 'cadm1a', 'ENSGWIG00000018158', 'ncam1a', 'zbtb16a', 'ENSGWIG00000023116', 'htr3a', 'usp28', 'lrwd1', 'arhgef12b', 'ENSGWIG00000023432', 'ENSGWIG00000023434', 'SPRY3', 'zgc:158291', 'klf5l', 'klf8', 'cldn2', 'ENSGWIG00000023451', 'gtf2ird1', 'dtx2', 'ENSGWIG00000023622', 'tmem132e', 'rad51d', 'tmem248', 'doc2b', 'mtmr2', 'ENSGWIG00000024417', 'gria3a', 'ENSGWIG00000006956', 'epd', 'ENSGWIG00000007379', 'ENSGWIG00000007665', 'gjd1a', 'ucp3', 'ENSGWIG00000009196', 'rtn2b', 'ENSGWIG00000009221', 'ENSGWIG00000009226', 'ENSGWIG00000009231', 'ENSGWIG00000009975', 'ENSGWIG00000009997', 'ophn1', 'AR', 'glra4b', 'msi2a', 'omga', 'myo18aa', 'sez6a', 'acaca', 'ENSGWIG00000023805', 'ENSGWIG00000024770', 'pcdh1a', 'ENSGWIG00000024892', 'PCDHAC2', 'ENSGWIG00000024896', 'ENSGWIG00000024897', 'npm1a', 'cldn7b', 'fgf11b', 'nlgn2b', 'kdm6bb', 'gabrb4', 'gabra3', 'ammecr1', 'kif4', 'adssl', 'KCNK4', 'esrra', 'prdx5', 'banf1', 'slc8a4a', 'cnih2', 'tm7sf2', 'frmd8', 'ENSGWIG00000008361', 'ENSGWIG00000008362', 'si:dkey-35m8.1', 'ABR', 'zswim7', 'ENSGWIG00000016746', 'fam222ba', 'flot2a', 'dhrs11a', 'vkorc1l1', 'gusb', 'cfap58', 'MTMR4', 'ENSGWIG00000021693', 'ENSGWIG00000021697', 'pwwp2a', 'c1qbp', 'mlxipl', 'SRRM3', 'hspb1', 'ywhag1', 'ENSGWIG00000021737', 'ENSGWIG00000021758', 'RIOX2', 'HLCS', 'nova1', 'capns1a', 'ppp1r14aa', 'ENSGWIG00000008661', 'ENSGWIG00000008662', 'mark4a', 'ENSGWIG00000010543', 'ENSGWIG00000010637', 'ENSGWIG00000010638', 'wscd1b', 'ENSGWIG00000010640', 'usp25', 'samsn1b', 'gdpd5a', 'ENSGWIG00000011082', 'ENSGWIG00000011104', 'ENSGWIG00000011650', 'dscama', 'pcp4a', 'slc37a4b', 'ENSGWIG00000014302', 'wnt11', 'uvrag', 'mogat2', 'pdzd3b', 'ENSGWIG00000014393', 'SPNS2', 'spns3', 'ankfy1', 'zzef1', 'auts2a', 'ENSGWIG00000016190', 'ints2', 'med13a', 'ENSGWIG00000018235', 'rnft1', 'rps6kb1a', 'map6a', 'ENSGWIG00000019537', 'ZBTB21', 'cxadr', 'ENSGWIG00000004527', 'ENSGWIG00000004878', 'ENSGWIG00000004921', 'ENSGWIG00000004930', 'ENSGWIG00000005252', 'ap4m1', 'serpinf1', 'med31', 'ENSGWIG00000011015', 'dlg2', 'bbx', 'ENSGWIG00000009722', 'ENSGWIG00000011729', 'c1d', 'ENSGWIG00000013092', 'meis1b', 'ENSGWIG00000013418', 'ENSGWIG00000013419', 'SPRED2', 'nckap1', 'ENSGWIG00000015030', 'zc3h15', 'mcm8', 'ENSGWIG00000018843', 'cdh11', 'ENSGWIG00000023701', 'ENSGWIG00000023702', 'ehbp1', 'b3gnt2b', 'ENSGWIG00000023734', 'kcnh1a', 'fbxo5', 'ENSGWIG00000009273', 'mrps5', 'slc8a1b', 'ENSGWIG00000009371', 'atrnl1a', 'ENSGWIG00000009507', 'ccdc172', 'ENSGWIG00000009511', 'ENSGWIG00000009514', 'cbr1', 'itsn1', 'pdxkb', 'CCDC88A', 'ENSGWIG00000018749', 'si:ch211-117n7.8', 'ENSGWIG00000020389', 'ENSGWIG00000020390', 'si:ch73-54n14.2', 'znhit2', 'LRPPRC', 'dync2li1', 'ENSGWIG00000021365', 'rtkn2a', 'ENSGWIG00000021367', 'ENSGWIG00000021368', 'tmem26a', 'tnfaip3', 'sdccag8', 'ENSGWIG00000023312', 'olig3', 'ENSGWIG00000023315', 'map3k21', 'oprm1', 'phactr2', 'htra1a', 'wapla', 'grid1a', 'ENSGWIG00000023946', 'ccser2a', 'si:ch211-248a14.8', 'ENSGWIG00000005524', 'chrm3a', 'ENSGWIG00000005536', 'rgs7b', 'ccdc6a', 'ank3a', 'ubac2', 'si:dkey-15d12.2', 'agt', 'ikzf5', 'hmx3a', 'hmx2', 'gpr26', 'chst15', 'nkx1.2la', 'FAM53B', 'eef1akmt2', 'zranb1b', 'ENSGWIG00000018695', 'IDO', 'INSYN2A', 'ptpreb', 'gnrh3', 'mgmt', 'bnip3', 'sorcs3b', 'ENSGWIG00000024475', 'SH3PXD2A', 'slka', 'ENSGWIG00000025369', 'ENSGWIG00000025374', 'ENSGWIG00000025376', 'minpp1a', 'zgc:91976', 'prkg1a', 'dkk1a', 'pcdh15a', 'si:dkey-27m7.4', 'comtd1', 'znf503', 'lrmda', 'kcnma1a', 'zcchc24', 'ppifa', 'zmiz1a', 'rps24', 'spock2', 'cuedc2', 'pax2a', 'chst3a', 'ENSGWIG00000022243', 'slc18a3a', 'ogdhl', 'parga', 'MARK1', 'EFEMP1', 'si:ch211-233m11.2', 'tp53bp2a', 'ENSGWIG00000025382', 'tet1', 'march8', 'ENSGWIG00000025397', 'marveld1', 'golga7bb', 'ENSGWIG00000009654', 'ENSGWIG00000009694', 'got1', 'morn4', 'ENSGWIG00000009753', 'slit1a', 'ENSGWIG00000010180', 'KCNIP2', 'HPS6', 'ENSGWIG00000015862', 'ldb1a', 'pprc1', 'oga', 'sufu', 'rrp12', 'antxr1d', 'GDF10', 'ENSGWIG00000018163', 'si:ch211-223a10.1', 'WDFY4', 'lrrc18a', 'vstm4a', 'ENSGWIG00000019839', 'ENSGWIG00000019841', 'fuom', 'ENSGWIG00000020124', 'msx3', 'agpat5', 'zc3h6', 'itga9', 'golga4', 'KCNK12', 'gtf2a1l', 'lhcgr', 'ENSGWIG00000022394', 'ENSGWIG00000010070', 'psme4a', 'fam135a', 'ENSGWIG00000020503', 'ENSGWIG00000020526', 'ENSGWIG00000020527', 'adgrb3', 'ENSGWIG00000021705', 'ENSGWIG00000021708', 'phf3', 'ENSGWIG00000021710', 'si:dkey-103j14.5', 'khdrbs2', 'prim2', 'rab23', 'ENSGWIG00000021756', 'col21a1', 'hmgcll1', 'fam83b', 'ENSGWIG00000022139', 'lrrc1', 'eloal', 'elovl5', 'mcu', 'pla2g12b', 'pde10a', 'tbxtb', 'prr18', 'unc5b', 'ly6pge', 'nsmce4a', 'capn1a', 'ENSGWIG00000025827', 'ENSGWIG00000025900', 'ENSGWIG00000025901', 'tasp1', 'sptlc3', 'btbd3b', 'ENSGWIG00000025933', 'ENSGWIG00000025934', 'jag1b', 'ENSGWIG00000025942', 'spast', 'ENSGWIG00000010075', 'si:ch1073-291c23.1', 'STRN', 'ENSGWIG00000014430', 'ENSGWIG00000014444', 'irf2bp2a', 'ENSGWIG00000014447', 'tomm20a', 'GNG4', 'lyst', 'ENSGWIG00000016378', 'nid1a', 'egln1a', 'ENSGWIG00000016562', 'sipa1l2', 'ENSGWIG00000016632', 'klhdc3', 'ENSGWIG00000016637', 'ENSGWIG00000016638', 'ENSGWIG00000016641', 'ENSGWIG00000016644', 'ENSGWIG00000016664', 'mrpl2', 'ENSGWIG00000016683', 'fam160b1', 'ablim1a', 'ENSGWIG00000020680', 'ENSGWIG00000020685', 'ENSGWIG00000020688', 'ENSGWIG00000020811', 'ENSGWIG00000020968', 'mlh1', 'lrrfip2', 'dact2', 'smoc2', 'eif3s6ip', 'vsir', 'ENSGWIG00000021319', 'ENSGWIG00000021322', 'plpp4', 'wdr11', 'fgfr2', 'ATE1', 'MYPN', 'bag3', 'sgpl1', 'fam204a', 'rab11fip2', 'emx2', 'eif3s10', 'prdx3', 'tm9sf3', 'ttl', 'nanp', 'cyp1b1', 'kmo', 'ide', 'march5', 'cpeb3', 'ENSGWIG00000009452', 'dntt', 'ENSGWIG00000009545', 'ENSGWIG00000009549', 'ENSGWIG00000009550', 'ADGRA1', 'ENSGWIG00000009600', 'ENSGWIG00000009604', 'ENSGWIG00000009614', 'valopa', 'nkx6.2', 'INPP5A', 'ENSGWIG00000010196', 'pwwp2b', 'STK32C', 'galm', 'ltbp1', 'ENSGWIG00000010975', 'ttc27', 'ENSGWIG00000013430', 'rin2', 'slc24a3', 'ENSGWIG00000013715', 'ccdc85a', 'ENSGWIG00000013722', 'vrk2', 'fancl', 'ENSGWIG00000013752', 'bcl11aa', 'rims1a', 'b3gat2', 'smap1', 'xpo1b', 'ENSGWIG00000014012', 'pex13', 'REL', 'haao', 'ENSGWIG00000007414', 'ENSGWIG00000007419', 'ENSGWIG00000007423', 'zfp36l2', 'ENSGWIG00000007462', 'plekhh2', 'epcam', 'ENSGWIG00000007544', 'rhoq', 'socs5b', 'ENSGWIG00000007902', 'cdk1', 'ctu2', 'ncapd3', 'ENSGWIG00000008430', 'rab1ab', 'hells', 'ENSGWIG00000008266', 'phf1', 'syngap1b', 'ENSGWIG00000001615', 'ENSGWIG00000001795', 'cpne4a', 'adam15', 'ENSGWIG00000002058', 'ENSGWIG00000002064', 'ENSGWIG00000006079', 'map7d1a', 'sh3d21', 'csf3r', 'GRIK3', 'ENSGWIG00000008587', 'zc3h12a', 'ENSGWIG00000008590', 'inpp5b', 'yrdc', 'themis2', 'mecr', 'foxo6a', 'ENSGWIG00000012420', 'SCMH1', 'cited4b', 'ctps1b', 'si:ch211-79k12.1', 'cadm4', 'bcam', 'ENSGWIG00000012573', 'ENSGWIG00000012602', 'ENSGWIG00000012604', 'fhl3a', 'ENSGWIG00000014197', 'pou3f1', 'ENSGWIG00000014199', 'rragca', 'ENSGWIG00000010121', 'ENSGWIG00000010123', 'si:ch73-373m9.1', 'ENSGWIG00000012174', 'si:ch211-107n13.1', 'ENSGWIG00000012183', 'dhdds', 'ago3a', 'ENSGWIG00000018852', 'ENSGWIG00000018854', 'zgc:110372', 'ENSGWIG00000018858', 'ENSGWIG00000018861', 'mag', 'ENSGWIG00000018927', 'ndufaf6', 'ENSGWIG00000019067', 'plekhf2', 'ENSGWIG00000019073', 'rhbg', 'ENSGWIG00000019398', 'mef2d', 'ENSGWIG00000019444', 'bmper', 'ENSGWIG00000021161', 'bbs9', 'ctsk', 'onecutl', 'ENSGWIG00000021239', 'tmod4', 'ENSGWIG00000021270', 'ENSGWIG00000010494', 'sema6e', 'ENSGWIG00000010702', 'ptpn23a', 'scap', 'ENSGWIG00000010788', 'cspg5a', 'dpy19l1l', 'tbx20', 'herpud2', 'pkdc', 'ankmy2a', 'crppa', 'ENSGWIG00000010850', 'grik5', 'ENSGWIG00000011570', 'ENSGWIG00000011578', 'znf526', 'ENSGWIG00000011864', 'pou2f2a', 'ENSGWIG00000011869', 'dedd1', 'ENSGWIG00000012103', 'si:ch73-22o12.1', 'ddr1', 'gnl1', 'pex5', 'si:ch211-69g19.2', 'atn1', 'ENSGWIG00000023614', 'wnt4b', 'ptpn6', 'rbp5', 'mrpl51', 'tnfrsf1a', 'gapdh', 'ENSGWIG00000001613', 'nop2', 'fam131bb', 'tfpt', 'si:dkey-40m6.8', 'ENSGWIG00000002270', 'leng8', 'ttyh1', 'ENSGWIG00000002385', 'ENSGWIG00000002387', 'si:dkeyp-69b9.3', 'pitpnc1b', 'grin2db', 'u2af2a', 'ENSGWIG00000004057', 'atg12', 'NECAP1', 'il11a', 'slc2a3b', 'ephb6', 'trpv6', 'ENSGWIG00000007062', 'ENSGWIG00000007087', 'si:dkey-283b15.4', 'rpl18', 'ENSGWIG00000010093', 'ENSGWIG00000013309', 'zbtb7b', 'ENSGWIG00000013313', 'pklr', 'RNaseP_nuc', 'setdb1b', 'celf3b', 'ENSGWIG00000010628', 'ENSGWIG00000010676', 'prpf3', 'dap3', 'ENSGWIG00000013684', 'si:dkey-7f3.9', 'kcnn3', 'adar', 'ENSGWIG00000014395', 'slc50a1', 'efna1b', 'efna3b', 'si:dkey-246i14.3', 'shc1', 'chrnb2', 'cgna', 'tuft1a', 'ENSGWIG00000014775', 'znf687a', 'pip5k1ab', 'ENSGWIG00000015122', 'si:ch1073-70f20.1', 'osgin2', 'gtpbp10', 'abcb4', 'crot', 'ENSGWIG00000025594', 'epb41l4b', 'esrp1', 'gem', 'cdh17', 'si:ch211-197h24.6', 'tex10', 'nr4a3', 'sec61b', 'frrs1l', 'tmem245', 'galnt1', 'ino80c', 'ENSGWIG00000011341', 'dgat1b', 'gpr20', 'tbrg4', 'ENSGWIG00000011458', 'steap4', 'ENSGWIG00000011475', 'si:dkey-12j5.1', 'mindy3', 'itga8', 'rbm12b', 'TPPP', 'smpd5', 'mpp6a', 'skap2', 'tax1bp1b', 'hacl1', 'hdac9b', 'fhod3b', 'AIF1', 'ENSGWIG00000002686', 'nphs1', 'kirrel3l', 'zmp:0000001114', 'aplp1', 'si:dkey-238d18.4', 'smg9', 'si:dkey-79d12.5', 'ENSGWIG00000007350', 'has1', 'ENSGWIG00000007358', 'ENSGWIG00000007359', 'ENSGWIG00000007361', 'ENSGWIG00000007365', 'ENSGWIG00000007416', 'ENSGWIG00000007427', 'ENSGWIG00000007430', 'ptprub', 'paqr7b', 'hivep3b', 'phactr4b', 'tent5ba', 'nr0b2a', 'si:dkey-288i20.2', 'cdcp1a', 'lars2', 'LIMD1', 'ncam3', 'mroh1', 'trak1a', 'ENSGWIG00000002141', 'ctnnb1', 'klhl38b', 'ENSGWIG00000005448', 'rad21a', 'eif3hb', 'ENSGWIG00000005586', 'ankib1b', 'eomesb', 'cmc1', 'ENSGWIG00000005911', 'rbms3', 'ddc', 'grb10b', 'cobl', 'ENSGWIG00000010571', 'adamts16', 'ENSGWIG00000010580', 'irx1a', 'irx2a', 'irx4a', 'LPCAT1', 'nrsn1', 'ENSGWIG00000012087', 'sox4b', 'CDKAL1', 'si:ch211-93i7.4', 'TNFRSF11B', 'sntb1', 'ptk2ab', 'ENSGWIG00000019422', 'CPQ', 'col14a1a', 'deptor', 'ubr5', 'atp6v1c1a', 'spire1b', 'ENSGWIG00000001796', 'ENSGWIG00000002321', 'si:dkey-122a22.2', 'zfpm2a', 'dpys', 'RIMS2', 'cthrc1a', 'klhl32', 'ENSGWIG00000007535', 'ENSGWIG00000007536', 'pou3f2b', 'fbxl4', 'faxcb', 'coq3', 'zgc:172323', 'ENSGWIG00000007674', 'me1', 'prss35', 'ube3d', 'tpbgb', 'ibtk', 'TENT5A', 'ENSGWIG00000008808', 'bckdhb', 'ttk', 'sybu', 'gdf6a', 'eny2', 'ENSGWIG00000008861', 'trhra', 'TMEM74', 'emc2', 'rspo2', 'plecb', 'ENSGWIG00000011958', 'ENSGWIG00000011963', 'sall3a', 'ENSGWIG00000011965', 'ENSGWIG00000012009', 'ENSGWIG00000012010', 'mbpb', 'ENSGWIG00000012020', 'ZNF516', 'TSHZ1', 'ZADH2', 'znf407', 'ENSGWIG00000012025', 'has2', 'ENSGWIG00000012030', 'tbc1d31', 'ENSGWIG00000013609', 'ENSGWIG00000013654', 'hpn', 'ENSGWIG00000015330', 'scxb', 'gapdhs', 'pop1', 'NIPAL2', 'KCNS2', 'VPS13B', 'ENSGWIG00000015492', 'ncalda', 'id4', 'cep76', 'ptpn2a', 'zufsp', 'kpna5', 'gprc6a', 'ENSGWIG00000015967', 'ENSGWIG00000015971', 'ldlrad4b', 'mc5rb', 'ENSGWIG00000001793', 'ENSGWIG00000001794', 'aldh2.2', 'rbm42', 'ENSGWIG00000002122', 'fbl', 'GADL1', 'ENSGWIG00000003812', 'ENSGWIG00000003825', 'tgfb2', 'ENSGWIG00000003843', 'ENSGWIG00000003845', 'vegfaa', 'hace1', 'ENSGWIG00000004828', 'ENSGWIG00000004897', 'ENSGWIG00000006219', 'tcaim', 'ENSGWIG00000007812', 'sacm1lb', 'ENSGWIG00000008814', 'adgrb1a', 'ENSGWIG00000010388', 'si:ch211-208k15.1', 'ENSGWIG00000010392', 'ENSGWIG00000010399', 'ENSGWIG00000010407', 'gask1a', 'snrka', 'ano10a', 'ABHD5', 'ENSGWIG00000011081', 'ENSGWIG00000011086', 'taf2', 'enpp2', 'ZNRF2', 'ENSGWIG00000013439', 'ENSGWIG00000013441', 'ENSGWIG00000016119', 'epha4b', 'ENSGWIG00000019096', 'gpx4b', 'ENSGWIG00000020456', 'sema6ba', 'ENSGWIG00000025874', 'rorcb', 'sh3gl1a', 'sema4e', 'ENSGWIG00000025881', 'sirt6', 'ENSGWIG00000025883', 'ENSGWIG00000025884', 'ENSGWIG00000001859', 'ENSGWIG00000002662', 'rab2a', 'tox', 'ENSGWIG00000002726', 'atp1b3a', 'grk7a', 'agfg1b', 'sept2', 'cdc7', 'rpap2', 'rpl5b', 'dipk1aa', 'mtf2', 'ccdc18', 'ENSGWIG00000012079', 'st3gal3b', 'ENSGWIG00000012378', 'zgc:154006', 'kcnn1a', 'angptl4', 'ENSGWIG00000016658', 'crtc1a', 'pde4ca', 'tbl1xr1b', 'kcnmb2', 'pik3ca', 'ephb3a', 'cpdp', 'ENSGWIG00000014481', 'ENSGWIG00000014631', 'ENSGWIG00000014633', 'penka', 'aqp1a.1', 'ENSGWIG00000023159', 'mib1', 'gata6', 'ENSGWIG00000016560', 'tesk2', 'ranbp9', 'myo10', 'ENSGWIG00000018656', 'ENSGWIG00000020236', 'kcnq3', 'ENSGWIG00000020259', 'si:ch211-235o23.1', 'ENSGWIG00000020289', 'ENSGWIG00000020328', 'proca', 'dusp28', 'ENSGWIG00000020346', 'vcpip1', 'fam49ba', 'PHB2', 'dlg1', 'rasal2', 'trnau1apb', 'klhl20', 'ENSGWIG00000025667', 'ENSGWIG00000025669', 'brinp2', 'astn1', 'pappa2', 'ctnnd2b', 'zswim5', 'fabp6', 'ENSGWIG00000018630', 'rnf220a', 'klf17', 'slc6a9', 'lrrc24', 'abhd4', 'carmil3', 'ENSGWIG00000019937', 'cdh24b', 'psmb5', 'ap1g2', 'mmp14b', 'chd8', 'ENSGWIG00000022155', 'eif2b5', 'ctdsplb', 'plcd1b', 'oxsr1a', 'ENSGWIG00000024377', 'csrnp1a', 'col6a1', 'col6a2', 'mcm6', 'ENSGWIG00000025233', 'ENSGWIG00000025234', 'sugt1', 'ENSGWIG00000025240', 'gad1b', 'APBB1IP', 'abi1b', 'yme1l1b', 'ENSGWIG00000016966', 'tfr1a', 'rgs11', 'ENSGWIG00000020340', 'dab1b', 'ENSGWIG00000020472', 'TADA1', 'pde4bb', 'ENSGWIG00000023907', 'SHISAL2A', 'scp2a', 'prdx1', 'IPP', 'ENSGWIG00000024840', 'ENSGWIG00000025296', 'ENSGWIG00000025299', 'ENSGWIG00000025300', 'ENSGWIG00000025301', 'ENSGWIG00000025304', 'rgs5b', 'brinp3b', 'rgs18', 'zbtb41', 'ENSGWIG00000025338', 'si:rp71-1g18.13', 'ENSGWIG00000025346', 'st6galnac3', 'st6galnac5a', 'ak5', 'zfyve9a', 'gnal2', 'PIEZO2', 'acss2l', 'ankrd12', 'skila', 'prkci', 'ENSGWIG00000022299', 'sppl2', 'arhgef18a', 'insra', 'hiat1a', 'loxl5b', 'ENSGWIG00000023054', 'LARP4B', 'DIP2C', 'esyt2a', 'ENSGWIG00000026221', 'LACTB2', 'pitrm1', 'ENSGWIG00000026275', 'fzd8b', 'CCNY', 'pard3ab', 'NRP1', 'ENSGWIG00000026482', 'ENSGWIG00000026535', 'arhgap12b', 'pold3', 'ENSGWIG00000001852', 'zeb1a', 'SVIL', 'lig1', 'atp1a2a', 'cadm3', 'kirrel1b', 'klf2b', 'ap1m1', 'NRTN', 'ptprfb', 'cdcp2', 'tmem59', 'ndc1', 'dcaf8', 'vav3b', 'zgc:92137', 'col11a1b', 'olfm3b', 'PTPRM', 'zbtb14', 'APCDD1', 'ENSGWIG00000026619', 'myom1a', 'myl12.2', 'ENSGWIG00000026682', 'ENSGWIG00000026691', 'fyco1b', 'adcyap1r1a', 'CRHR2', 'myl13', 'pth1r', 'tmie', 'snap47', 'wnt9a', 'cdk5', 'asic1c', 'asb10', 'agap3', 'ENSGWIG00000014388', 'ENSGWIG00000014389', 'ENSGWIG00000014390', 'ube2v2', 'ENSGWIG00000014504', 'SPIDR', 'dtna', 'klhl14', 'ENSGWIG00000014656', 'mfsd12a', 'ENSGWIG00000014708', 'ENSGWIG00000014956', 'zgc:163121', 'cahz', 'ENSGWIG00000015677', 'ptf1a', 'ENSGWIG00000015690', 'ENSGWIG00000015695', 'fam110b', 'tmem70', 'si:ch211-207d6.2', 'klhl18', 'ENSGWIG00000015785', 'vwa5b2', 'ece2a', 'klhl24b', 'kcnab1b', 'mbnl1', 'ENSGWIG00000006410', 'ENSGWIG00000006417', 'ENSGWIG00000006419', 'ephb1', 'gpc1b', 'gpc5c', 'nyap2b', 'ENSGWIG00000006466', 'fgf12a', 'parla', 'rc3h1b', 'otol1a', 'zp3b', 'irf4a', 'foxq1a', 'ENSGWIG00000006629', 'mylk4a', 'ENSGWIG00000006639', 'wrnip1', 'clcn2a', 'eif4g1a', 'cdc20', 'ELOVL1', 'Metazoa_SRP', 'bokb', 'rab6bb', 'ptch2', 'eif2b3', 'plk3', 'cacybp', 'mrps14', 'tnn', 'tnr', 'ENSGWIG00000008695', 'ENSGWIG00000008707', 'TTC39C', 'OSBPL1A', 'ENSGWIG00000008831', 'ZNF521', 'ENSGWIG00000008839', 'ss18', 'psma8', 'KCTD1', 'ENSGWIG00000008887', 'CDH2', 'ENSGWIG00000015801', 'ENSGWIG00000015838', 'dspa', 'adcy1b', 'lamc1', 'ENSGWIG00000003653', 'ENSGWIG00000003663', 'znf219', 'ENSGWIG00000003699', 'ENSGWIG00000003701', 'ccnb3', 'ENSGWIG00000003763', 'zgc:109889', 'ENSGWIG00000003784', 'ENSGWIG00000005162', 'lpcat4', 'supt16h', 'ENSGWIG00000007644', 'ENSGWIG00000013749', 'ENSGWIG00000016457', 'ENSGWIG00000016458', 'ENSGWIG00000016498', 'ENSGWIG00000016523', 'pnp5a', 'gal3st3', 'f13a1b', 'ENSGWIG00000024994', 'tex261', 'vax2', 'gba2', 'si:dkeyp-75h12.2', 'si:dkeyp-75h12.5', 'ENSGWIG00000025090', 'ENSGWIG00000025092', 'haus3', 'ENSGWIG00000025104', 'ENSGWIG00000025170', 'efs', 'ajuba', 'si:ch211-212k18.5', 'ttc5', 'ENSGWIG00000015565', 'tnfsf12', 'sat2a', 'rbpjb', 'tbc1d19', 'stim2b', 'ENSGWIG00000016963', 'ENSGWIG00000016968', 'RBPMS', 'ENSGWIG00000017250', 'HGFAC', 'DOK7', 'ADRA2C', 'ENSGWIG00000017271', 'CPZ', 'tnk1', 'ENSGWIG00000017333', 'ctdnep1a', 'zbtb4', 'zpax4', 'ENSGWIG00000013979', 'bcl6b', 'acap1', 'si:ch73-335l21.1', 'per1b', 'rell1', 'nwd2', 'ENSGWIG00000014588', 'ENSGWIG00000014626', 'pcdh7b', 'ENSGWIG00000015169', 'CAAP1', 'ENSGWIG00000015547', 'ENSGWIG00000021811', 'MPDZ', 'tyrp1a', 'ENSGWIG00000024601', 'USP53', 'sec24d', 'KCNIP4', 'adgra3', 'gba3', 'ENSGWIG00000026490', 'ppargc1a', 'dhx15', 'SLC24A2', 'etnppl', 'larp7', 'ADAMTSL1', 'plin2', 'dennd4c', 'plaa', 'NFIB', 'ENSGWIG00000026730', 'ENSGWIG00000026769', 'pcxb', 'ENSGWIG00000026776', 'brms1', 'ppp1r14ba', 'plcb3', 'slc8a4b', 'ppp2r5b', 'rcor2', 'mark2b', 'hspa12b', 'paip2b', 'dok1b', 'm1ap', 'ENSGWIG00000020082', 'ENSGWIG00000020111', 'ENSGWIG00000020154', 'ENSGWIG00000021207', 'ENSGWIG00000021218', 'kirrel1a', 'ENSGWIG00000022105', 'ENSGWIG00000022120', 'fxr2', 'kdm6ba', 'ENSGWIG00000022988', 'ENSGWIG00000022994', 'efnb3b', 'wrap53', 'nlgn2a', 'ENSGWIG00000004133', 'tmem102', 'fgf11a', 'chrnb1', 'ENSGWIG00000004395', 'RASGRP2', 'ENSGWIG00000004510', 'ENSGWIG00000004523', 'ENSGWIG00000004558', 'nrxn2b', 'si:cabz01078036.1', 'drap1', 'sorcs2', 'grpel1', 'ENSGWIG00000009608', 'TBC1D14', 'S100P', 'ENSGWIG00000010922', 'SLC29A2', 'ENSGWIG00000010944', 'ENSGWIG00000011004', 'camk2d1', 'arsj', 'pde5ab', 'si:dkey-219e21.4', 'afap1', 'ENSGWIG00000007751', 'ENSGWIG00000007762', 'serpine1', 'ENSGWIG00000007832', 'ENSGWIG00000007840', 'eif4a1a', 'focad', 'mllt3', 'sf1', 'pygmb', 'HTRA3', 'acox3', 'nat16', 'mtm1', 'pelp1', 'trmt10a', 'slc1a1', 'RAP1GDS1', 'taf6l', 'rtn3', 'si:dkey-106g10.7', 'ENSGWIG00000012327', 'mta2', 'men1', 'map4k2', 'ENSGWIG00000024135', 'ehbp1l1a', 'lrrc20', 'polr3a', 'ENSGWIG00000025151', 'ZCCHC24', 'ENSGWIG00000025420', 'ZNF503', 'COMTD1', 'kat6b', 'ADK', 'ENSGWIG00000026665', 'vti1a', 'tcf7l2', 'crygmx', 'PPP1R1B', 'med1', 'STAC2', 'gpam', 'ENSGWIG00000011526', 'capn15', 'MSS51', 'PPFIA3', 'ENSGWIG00000017042', 'ENSGWIG00000017050', 'pigp', 'ENSGWIG00000018278', 'prr12a', 'ENSGWIG00000018851', 'si:ch1073-13h15.3', 'col1a1b', 'ENSGWIG00000019659', 'ENSGWIG00000019660', 'cox10', 'si:ch211-216b21.2', 'tekt3', 'cdc42ep4b', 'sdk2b', 'ENSGWIG00000019754', 'rpl38', 'ttyh2', 'ENSGWIG00000019792', 'rgs9b', 'ENSGWIG00000002814', 'syt15', 'si:dkey-49c17.4', 'si:dkey-49c17.3', 'sfxn2', 'pgs1', 'socs3b', 'eef2k', 'ENSGWIG00000007864', 'sec24c', 'mta3', 'cox7a2l', 'pigf', 'ENSGWIG00000024232', 'fshr', 'ENSGWIG00000024750', 'psme4b', 'ptprea', 'clrn3', 'dock1', 'ADAM12', 'ENSGWIG00000026345', 'fam53b', 'oat', 'CHST15', 'GPR26', 'bub3', 'ENSGWIG00000026447', 'ENSGWIG00000026450', 'ENSGWIG00000003648', 'eif3c', 'TANC2', 'wnk4b', 'U5', 'sema4gb', 'scn4aa', 'asb3', 'ABCA5', 'baiap2b', 'ENSGWIG00000017281', 'ENSGWIG00000017289', 'cep131', 'OXLD1', 'kcnj16', 'trim16', 'ENSGWIG00000018026', 'SRCIN1', 'grnb', 'acta2', 'ENSGWIG00000014470', 'mat2b', 'ENSGWIG00000014477', 'a1cf', 'ENSGWIG00000014513', 'ENSGWIG00000014803', 'ENSGWIG00000014810', 'GRID1', 'acsl5', 'psme3', 'nbr1a', 'dhx8', 'thrab', 'ormdl3', 'tbkbp1', 'skap1', 'chmp2a', 'ENSGWIG00000023783', 'stx4', 'arhgap44', 'ENSGWIG00000023867', 'mapta', 'ENSGWIG00000002950', 'ngfrb', 'fam20cb', 'ENSGWIG00000005181', 'trrap', 'tmem130', 'baiap2l1a', 'bhlha15', 'ENSGWIG00000009011', 'pms2', 'ENSGWIG00000009296', 'bahcc1b', 'tepsin', 'ENSGWIG00000012845', 'AMZ2', 'WIPI1', 'fam20a', 'ENSGWIG00000013086', 'dlx4b', 'itga3b', 'ENSGWIG00000016758', 'raraa', 'ENSGWIG00000018317', 'med24', 'SLC17A7', 'ENSGWIG00000007978', 'rnf40', 'si:dkey-220f10.4', 'ube2ia', 'si:ch211-231f6.6', 'zmp:0000000624', 'zgc:112185', 'ENSGWIG00000011243', 'si:ch211-183d21.1', 'rab11fip3', 'ENSGWIG00000011436', 'metrn', 'NT5C3B', 'ADAM11', 'ENSGWIG00000014778', 'ENSGWIG00000014782', 'dnmbp', 'nhlrc2', 'atrnl1b', 'gfra1b', 'ENSGWIG00000023725', 'ENSGWIG00000023726', 'ENSGWIG00000023727', 'ENSGWIG00000023729', 'cep55l', 'si:ch211-197g18.2', 'plce1', 'ENSGWIG00000014268', 'ENSGWIG00000016717', 'nmt1b', 'svild', 'znf281b', 'sh3bp1', 'ep300a', 'josd1', 'ENSGWIG00000019324', 'antxr1c', 'ENSGWIG00000021570', 'sox9a', 'ENSGWIG00000021580', 'pemt', 'ENSGWIG00000021766', 'NT5M', 'SLC39A11', 'ENSGWIG00000021796', 'cacna1g', 'UTS2R', 'ENSGWIG00000023776', 'hexdc', 'sap30bp', 'tnrc6b', 'csnk1e', 'maff', 'snx29', 'shisa9a', 'cpped1', 'ercc4', 'ENSGWIG00000015564', 'ENSGWIG00000015573', 'foxj1b', 'grid2ipb', 'SPAG9', 'ENSGWIG00000019348', 'tob1b', 'ENSGWIG00000019352', 'abcc3', 'wu:fj29h11', 'ENSGWIG00000014244', 'hs3st3l', 'ENSGWIG00000014259', 'pts', 'hhex', 'exoc6', 'cyp26a1', 'jmjd1cb', 'reep3b', 'bms1', 'mtr', 'ENSGWIG00000017606', 'ATP6V0A1', 'naglu', 'ipmkb', 'tfam', 'rab3gap1', 'fasn', 'ENSGWIG00000021095', 'ENSGWIG00000021096', 'CSNK1D', 'ENSGWIG00000021131', 'ENSGWIG00000021154', 'ENSGWIG00000021178', 'ENSGWIG00000022333', 'ENSGWIG00000022335', 'ENSGWIG00000022337', 'llgl2', 'ENSGWIG00000024499', 'tmem94', 'ENSGWIG00000024603', 'ENSGWIG00000024604', 'ENSGWIG00000024605', 'ca10a', 'ENSGWIG00000024613', 'eme1', 'RAB37', 'vopp1', 'znf438', 'ZEB1', 'ARHGAP12', 'KIF5B', 'si:ch211-276f18.2', 'ENSGWIG00000023259', 'ENSGWIG00000023260', 'adarb2', 'idi1', 'CPNE3', 'wwp1', 'ENSGWIG00000020817', 'serpinb1l3', 'ENSGWIG00000020886', 'ANKS6', 'arhgap29a', 'tecrl2a', 'ENSGWIG00000021123', 'rnpc3', 'ntng1a', 'VAV3', 'ENSGWIG00000021461', 'cngb3.1', 'ENSGWIG00000021690', 'tnk2a', 'tmtopsb', 'and1', 'arhgap21a', 'acp5b', 'f3a', 'ENSGWIG00000021963', 'ENSGWIG00000021965', 'ENSGWIG00000021984', 'zgc:110852', 'ccr12b.2', 'slc12a7b', 'rsu1', 'c1ql3a', 'ENSGWIG00000021325', 'TGFBR1', 'reck', 'ENSGWIG00000022134', 'snx7', 'plppr5b', 'plppr4b', 'ENSGWIG00000022138', 'mastl', 'ENSGWIG00000022235', 'gad2', 'myo3a', 'anxa13', 'gpr158a', 'lztfl1', 'gli3', 'prpf4bb', 'pxdc1b', 'TMX3', 'DSEL', 'ENSGWIG00000025227', 'CDH7', 'ENSGWIG00000025274', 'ENSGWIG00000025278', 'cdk13', 'ENSGWIG00000025285', 'ca2', 'ENSGWIG00000025289', 'ENSGWIG00000025293', 'topbp1', 'ENSGWIG00000025574', 'scrib', 'nrbp2b', 'ENSGWIG00000025785', 'fam49bb', 'myca', 'pou6f2', 'ENSGWIG00000020418', 'sox17', 'sox32', 'RGS20', 'OPRK1', 'myom1b', 'LPIN2', 'tyms', 'SLC39A12', 'PLXDC2', 'malrd1', 'esyt2b', 'ptprn2', 'lmbr1', 'rnf32', 'shha', 'dlgap1b', 'otud1', 'si:ch211-285f17.1', 'si:dkey-96n2.3', 'chst2b', 'ENSGWIG00000023953', 'dipk2ab', 'plod2', 'si:ch73-206p6.1', 'zic1', 'agtr1b', 'cpb1', 'gyg1b', 'slc35g2b', 'csrnp1b', 'ENSGWIG00000025782', 'map3k22', 'ENSGWIG00000006036', 'boc', 'usf3', 'naa50', 'atp8a2', 'gpr12', 'ENSGWIG00000016714', 'spata13', 'ENSGWIG00000016794', 'ythdf3', 'bhlhe22', 'armc1', 'pde7a', 'crhb', 'rrs1', 'sgk3', 'arfgef1', 'prex2', 'zgc:162928', 'sulf1', 'SLCO5A1', 'ncoa2', 'xkr9', 'eya1', 'msc', 'trpa1b', 'si:dkey-192j17.1', 'SBSPON', 'ENSGWIG00000002789', 'emc9', 'ENSGWIG00000002825', 'ENSGWIG00000003053', 'rnf152', 'CDH20', 'ENSGWIG00000009598', 'mc4r', 'CDH6', 'ENSGWIG00000015375', 'ENSGWIG00000015855', 'cdh12a', 'CDH18', 'epc1a', 'ENSGWIG00000016401', 'dpp6a', 'ptchd1', 'phex', 'mbtps2', 'ENSGWIG00000016488', 'ENSGWIG00000016524', 'map3k15', 'ENSGWIG00000016555', 'SLC22A17', 'phldb2b', 'tfap2a', 'ENSGWIG00000016671', 'ENSGWIG00000016673', 'ENSGWIG00000016675', 'slc35b3', 'bloc1s5', 'txndc5', 'bmp6', 'pdcd6', 'rec8b', 'si:ch211-196f5.2', 'ENSGWIG00000016961', 'psme1', 'dcaf11', 'ENSGWIG00000019236', 'dap', 'ENSGWIG00000019279', 'zfhx4', 'ENSGWIG00000019367', 'hnf4g', 'relch', 'tpk1', 'ENSGWIG00000007508', 'ENSGWIG00000011751', 'shrprbck1r', 'myrip', 'ENSGWIG00000012118', 'pola1', 'mtrr', 'ENSGWIG00000015372', 'sema5a', 'cbln2b', 'ENSGWIG00000017865', 'ENSGWIG00000017870', 'socs6a', 'slc39a10', 'dnah7', 'si:ch211-219a4.3', 'faima', 'cx43.4', 'lct', 'hdac4', 'si:dkey-34f9.3', 'adarb1b', 'ENSGWIG00000022089', 'ube2f', 'ccnt2a', 'TMEM163', 'si:ch73-116o1.2', 'ptpn4a', 'ENSGWIG00000022150', 'ENSGWIG00000023683', 'ENSGWIG00000023685', 'cdk5r2b', 'wnt6b', 'wnt10a', 'ENSGWIG00000024056', 'myo1b', 'ENSGWIG00000024260', 'ENSGWIG00000024261', 'lrp1bb', 'ENSGWIG00000024391', 'stxbp5l', 'gtf2e1', 'rabl3', 'spoplb', 'hnmt', 'thsd7ba', 'dars1', 'asic4a', 'ENSGWIG00000007073', 'fzd7b', 'GTF2F2', 'ENSGWIG00000007188', 'tmem223', 'SLC5A7', 'EPHA6', 'ENSGWIG00000007261', 'STX19', 'erbb4b', 'ikzf2', 'vwc2l', 'bard1', 'abca12', 'ptprna', 'pou1f1', 'HTR1F', 'si:ch211-261p9.4', 'cps1', 'myl1', 'ENSGWIG00000025449', 'ENSGWIG00000025450', 'heg1', 'znf148', 'ENSGWIG00000025459', 'asmt', 'sept10', 'ENSGWIG00000025465', 'ENSGWIG00000025466', 'edar', 'bin1a', 'ENSGWIG00000009152', 'arsh', 'ENSGWIG00000009294', 'ENSGWIG00000009298', 'ENSGWIG00000009300', 'ENSGWIG00000009304', 'ENSGWIG00000009309', 'zgc:158316', 'trpm2', 'pfkla', 'tmem169b', 'ENSGWIG00000023278', 'ENSGWIG00000023286', 'SMARCAL1', 'nme8', 'zgc:162707', 'ehhadh', 'tmem41aa', 'glb1l', 'ENSGWIG00000023877', 'galnt13', 'kcnj3a', 'nr4a2a', 'ENSGWIG00000025137', 'ENSGWIG00000025138', 'PKP4', 'dap1b', 'tanc1b', 'ENSGWIG00000025746', 'tank', 'tbr1b', 'KCNH7', 'fign', 'stk39', 'cers6', 'dhrs9', 'ENSGWIG00000026827', 'ccdc173', 'ubr3', 'ssb', 'myo3b', 'sp5a', 'tlk1a', 'dcaf17', 'hat1', 'ENSGWIG00000017167', 'dlx2a', 'itga6a', 'cdca7a', 'SP3', 'atp5mc3b', 'lnpa', 'hoxd4a', 'hoxd3a', 'hnrnpa3', 'agps', 'osbpl6', 'DOCK9', 'FARP1', 'hs6st3b', 'uggt2', 'gpc6b', 'FASTKD5', 'ENSGWIG00000026736', 'ednrbb', 'mid1', 'tlr7', 'ENSGWIG00000014494', 'ofd1', 'ANOS1', 'ENSGWIG00000015264', 'igsf3', 'rpe', 'ackr3a', 'ENSGWIG00000015637', 'morc3a', 'si:dkey-229b18.3', 'ENSGWIG00000015951', 'FAM155A', 'myo16', 'col4a1', 'ankrd10b', 'sik1', 'sytl5', 'gpr161', 'dcaf6', 'nxpe3', 'gdf3', 'klf7b', 'creb1b', 'itprid2', 'itga4', 'rpl8', 'PPP2R3B', 'ENSGWIG00000009586', 'ENSGWIG00000009595', 'asmtl', 'upf3a', 'cdc16', 'ENSGWIG00000010085', 'fgf14', 'itgbl1', 'nalcn', 'ENSGWIG00000010166', 'zic2a', 'zic5', 'clybl', 'si:ch211-184m13.4', 'gpr18', 'dachd', 'klf5b', 'klf12b', 'tbc1d4', 'lmo7a', 'kctd12.1', 'ENSGWIG00000011737', 'slain1a', 'tspan7b', 'ENSGWIG00000011845', 'ENSGWIG00000012601', 'ENSGWIG00000012603', 'sh3bp4', 'lrch1', 'esd', 'htr2aa', 'sucla2', 'ENSGWIG00000012738', 'lcp1', 'RCBTB1', 'spryd7b', 'il1rapl1a', 'ENSGWIG00000015579', 'dcbld2', 'COL8A1', 'ENSGWIG00000015916', 'enox1', 'ATP11A', 'ENSGWIG00000015486', 'sox1a', 'arhgef7a', 'rev1', 'ENSGWIG00000015649', 'si:dkey-4e7.3', 'slc9a2', 'tmem182a', 'pou3f3a', 'fhl2a', 'nck2a', 'ecrg4a', 'uxs1', 'tmtops2b', 'ST6GAL2', 'jam2b', 'zgc:172067', 'tfdp1a', 'dcun1d2a', 'pou2f1b', 'slc49a4', 'sema5ba', 'pdia5', 'sec22a', 'adcy5', 'hacd2', 'ARL5A', 'neb', 'ENSGWIG00000005634', 'si:ch1073-440b2.1', 'ENSGWIG00000005808', 'arhgap20', 'rnf17', 'cenpj', 'ttf2', 'mphosph8', 'zmym2', 'gja3', 'cx30.3', 'cryl1', 'ift88', 'xpo4', 'lats2', 'ildr1b', 'ENSGWIG00000010126', 'hpxa', 'olig1', 'ENSGWIG00000010537', 'fam160a2', 'cnga4', 'ENSGWIG00000010573', 'wdfy2', 'ints6', 'serpine3', 'actr3', 'DPP10', 'ENSGWIG00000006927', 'ENSGWIG00000006943', 'cops8', 'ENSGWIG00000006965', 'mlphb', 'ENSGWIG00000006983', 'ENSGWIG00000006986', 'pgap1', 'hecw2a', 'stk17b', 'zgc:153521', 'ENSGWIG00000007176', 'ENSGWIG00000007183', 'sympk', 'zgc:136439', 'ENSGWIG00000013742', 'pms1', 'mstnb', 'hibch', 'lss', 'mcm3ap', 'cnot9', 'tns1b', 'ENSGWIG00000021806', 'igfbp5b', 'igfbp2b', 'slc4a3', 'gulp1a', 'rftn2', 'satb2', 'crygs1', 'lcmt2', 'kcnh3', 'ENSGWIG00000023866', 'idh1', 'ENSGWIG00000015018', 'akap11', 'ENSGWIG00000015042', 'dgkh', 'vwa8', 'rgcc', 'f5', 'ccdc80', 'kdm6a', 'ENSGWIG00000023878', 'ndp', 'mao', 'dnajc10', 'HYKK', 'IMPG2', 'ENSGWIG00000025864', 'tfg', 'gdap2', 'hcn5', 'pla1a', 'hsd3b1', 'ENSGWIG00000026049', 'ENSGWIG00000026052', 'tbx15', 'wars2', 'wdr4', 'ENSGWIG00000026116', 'ccdc93', 'insig2', 'ENSGWIG00000026203', 'ENSGWIG00000026256', 'ENSGWIG00000026257', 'map4k5', 'sos2', 'cdkn3', 'gmfb', 'ENSGWIG00000009679', 'ENSGWIG00000009705', 'ENSGWIG00000009712', 'dicer1', 'gsc', 'lrit3b', 'gabrr2a', 'rragd', 'ENSGWIG00000009937', 'ENSGWIG00000009963', 'ENSGWIG00000009983', 'sdsl', 'slc8a3', 'actn1', 'dcaf5', 'pacs2', 'bend3', 'pdss2', 'ENSGWIG00000016389', 'sobpa', 'rock2a', 'slc66a3', 'si:dkeyp-47f9.4', 'alk', 'ENSGWIG00000018388', 'ENSGWIG00000018393', 'ENSGWIG00000018397', 'ENSGWIG00000018404', 'commd8', 'golga5', 'trip11', 'si:ch211-10a23.2', 'zfyve26', 'ENSGWIG00000003458', 'smoc1', 'paplna', 'galnt16', 'ENSGWIG00000004900', 'ENSGWIG00000005292', 'sash1b', 'plekhh1', 'zfp36l1a', 'tmem229b', 'crip2', 'elmsan1b', 'pygl', 'tmx1', 'frmd6', 'dusp23a', 'oct2', 'hivep2b', 'TP53I3', 'ENSGWIG00000009216', 'slc25a21', 'pax9', 'rps6ka5', 'dpysl5a', 'ENSGWIG00000022688', 'ENSGWIG00000022743', 'C14orf132', 'ENSGWIG00000022754', 'spred1', 'meis2a', 'zgc:154061', 'dph6', 'GJD2', 'stxbp6', 'fermt2', 'ddhd1a', 'bmp4', 'lbh', 'ENSGWIG00000023227', 'manea', 'fhl5', 'tspeara', 'senp2', 'slc1a8b', 'shda', 'plpp2a', 'gna11b', 'pdcd10b', 'si:dkey-56i24.1', 'slitrk3b', 'nrd1a', 'NPAS3', 'ENSGWIG00000019569', 'akap6', 'ppp2r5ea', 'kcnh5b', 'ENSGWIG00000023139', 'ENSGWIG00000023145', 'ENSGWIG00000023163', 'ENSGWIG00000023257', 'ENSGWIG00000023261', 'btbd6b', 'zfyve21', 'klc1a', 'trmt61a', 'CKB', 'ENSGWIG00000023644', 'exoc3l4', 'letm1', 'nsd2', 'larp1b', 'cdca4', 'ENSGWIG00000024059', 'fgfr3', 'ENSGWIG00000026265', 'ENSGWIG00000026266', 'ENSGWIG00000026269', 'si:ch211-194c3.5', 'ky', 'slc4a11', 'rpia', 'eif2ak3', 'atrn', 'gfra4a', 'ap5s1', 'rab11fip5a', 'ENSGWIG00000015754', 'pank2', 'ENSGWIG00000016390', 'dctn1b', 'cpxm1a', 'hhipl2', 'vrk1', 'BCL11B', 'ENSGWIG00000022049', 'adck1', 'daam2', 'ENSGWIG00000023461', 'kcnk17', 'kcnk5a', 'ENSGWIG00000024403', 'ENSGWIG00000024410', 'pxdn', 'MYT1L', 'ENSGWIG00000024561', 'eipr1', 'trappc12', 'ENSGWIG00000024677', 'bsdc1', 'fndc5a', 'syf2', 'runx3', 'tdh2', 'pabpc4', 'ENSGWIG00000025246', 'gale', 'ralgapa2', 'kiz', 'nkx2.4a', 'pax1a', 'foxa2', 'prkd3', 'SCAF8', 'tiam2a', 'tfb1m', 'col12a1a', 'CD109', 'ENSGWIG00000017552', 'dlgap2a', 'cln8', 'arhgef10', 'kbtbd11', 'ENSGWIG00000017673', 'ZDHHC14', 'nt5c1ab', 'MYCL', 'mfsd2aa', 'ENSGWIG00000018283', 'rbbp4', 'ENSGWIG00000018383', 'ENSGWIG00000019238', 'zfyve19', 'bub1bb', 'ENSGWIG00000019300', 'xgb', 'dnajc17', 'gchfr', 'cyp1c2', 'ENSGWIG00000021349', 'ENSGWIG00000021350', 'ENSGWIG00000021357', 'arhgap11a', 'FMN1', 'ENSGWIG00000021654', 'ENSGWIG00000021676', 'ENSGWIG00000022670', 'ENSGWIG00000022811', 'ENSGWIG00000023488', 'VSX2', 'ENSGWIG00000007064', 'AREL1', 'rps6kl1', 'ESRRB', 'GPATCH2L', 'IFT43', 'si:dkey-1f12.3', 'ngb', 'zdhhc22', 'ap5m1', 'exoc5', 'otx2b', 'peli2', 'ktn1', 'msh4', 'vcpkmt', 'kif26ab', 'ENSGWIG00000013628', 'klf11b', 'grhl1', 'ywhaqb', 'bahd1', 'eml1', 'degs2', 'yy1a', 'slc38a6', 'six4a', 'ppm1aa', 'si:dkey-85n7.6', 'ENSGWIG00000008529', 'btbd7', 'ENSGWIG00000008536', 'itpk1b', 'hspa4l', 'prph2la', 'mboat2a', 'id2a', 'rnf144aa', 'cmpk2', 'sox11a', 'ENSGWIG00000014212', 'ENSGWIG00000014214', 'myo6b', 'impg1b', 'HTR1B', 'znf292a', 'akirin2', 'stac', 'ENSGWIG00000018522', 'rlf', 'rpl13a', 'ENSGWIG00000018572', 'ENSGWIG00000018633', 'DLK1', 'id3', 'si:ch1073-358c10.1', 'vgll2b', 'ctsl.1', 'ENSGWIG00000024120', 'qkia', 'ankrd6b', 'gabrr2b', 'ube2j1', 'ENSGWIG00000026624', 'slc5a6a', 'gopc', 'znf292b', 'ENSGWIG00000012698', 'rmdn3', 'si:ch211-241j12.3', 'ephx1', 'lbr', 'ENSGWIG00000012734', 'mcm9', 'cx43', 'hsf2', 'nus1', 'serinc1', 'psmc6', 'cgrrf1', 'dtnba', 'ENSGWIG00000014049', 'kif3ca', 'rab10', 'lrp11', 'map3k5', 'ENSGWIG00000014405', 'tfap2b', 'tfap2d', 'hadhb', 'ENSGWIG00000018340', 'ENSGWIG00000018342', 'ENSGWIG00000018346', 'jag2b', 'heca', 'reps1', 'ENSGWIG00000020188', 'ENSGWIG00000020249', 'dlgap2b', 'ptprk', 'ENSGWIG00000025350', 'ENSGWIG00000025352', 'lama2', 'rgs6', 'sipa1l1', 'srsf5b', 'susd6', 'ENSGWIG00000025504', 'thbs1b', 'katnbl1', 'aven', 'ENSGWIG00000003254', 'ENSGWIG00000003796', 'grem1a', 'ENSGWIG00000003806', 'serac1', 'tmem181', 'ppil6', 'cep57l1', 'ENSGWIG00000004529', 'tmem121ab', 'extl3', 'nt5c1bb', 'si:dkey-174m14.3', 'rngtt', 'cnr1', 'rars2', 'slc35a1', 'rhag', 'ENSGWIG00000008327', 'nrbp1', 'capn3b', 'ttc32', 'pum2', 'rock2b', 'mrap2b', 'sim1b', 'prep', 'ENSGWIG00000013948', 'mycn', 'smek1', 'fndc1', 'ENSGWIG00000018668', 'foxo3b', 'sesn1', 'nhsl1b', 'ENSGWIG00000021491', 'smyd2b', 'PTPN14', 'prox1a', 'rps6kc1', 'si:ch211-203k16.3', 'prkg3', 'ABHD1', 'brms1lb', 'kcnk13b', 'ches1', 'CEP128', 'nrxn3b', 'snw1', 'ccdc85cb', 'ENSGWIG00000025422', 'clic5b', 'runx2b', 'opn8b', 'mfsd2b', 'atad2b', 'klhl29', 'trib2', 'ENSGWIG00000025709', 'gpcpd1', 'PLCB1', 'PLCB4', 'ENSGWIG00000016212', 'ENSGWIG00000016213', 'ENSGWIG00000016214', 'ENSGWIG00000016216', 'ENSGWIG00000016218', 'entpd5b', 'actr10', 'naa30', 'ENSGWIG00000016241', 'ENSGWIG00000016245', 'arf6a', 'fkbp3', 'MDGA2', 'ENSGWIG00000017895', 'ENSGWIG00000018373', 'RPS29', 'ppp2r5cb', 'slc25a47a', 'vrtn', 'dll4', 'vps18', 'sel1l', 'tmem244', 'tmem200a', 'si:ch73-18b11.1', 'ENSGWIG00000024347', 'nkain2', 'mrps18a', 'vegfab', 'wdr26b', 'si:ch73-21k16.1', 'ENSGWIG00000003767', 'eif2b2', 'jdp2b', 'ENSGWIG00000004100', 'ENSGWIG00000005357', 'ENSGWIG00000005370', 'ENSGWIG00000005378', 'flvcr2b', 'ptk2bb', 'chrna2b', 'ENSGWIG00000009557', 'nme6', 'eif2b4', 'atraid', 'znf513a', 'si:ch211-278j3.3', 'ENSGWIG00000010630', 'sox7', 'mtmr9', 'SGK1', 'nbas', 'ddx1', 'itsn2a', 'si:dkey-119f1.1', 'tmem18', 'fbxo25', 'fut8b', 'gphnb', 'tmem151ba', 'tcte1', 'dusp10', 'hlx1', 'ENSGWIG00000016278', 'EZR', 'ppp2r5a', 'lpgat1', 'ENSGWIG00000023598', 'cnstb', 'ENSGWIG00000023606', 'cdc42bpab', 'fam49a', 'vsnl1a', 'ENSGWIG00000025947', 'flrt2', 'ENSGWIG00000025950', 'ncbp2', 'FIG4', 'cdc40', 'slc22a16', 'sash1a', 'stxbp5a', 'epm2a', 'utrn', 'ENSGWIG00000026238', 'dpysl5b', 'dnmt3aa', 'ENSGWIG00000026484', 'ENSGWIG00000003258', 'emilin1a', 'tab2', 'disp1', 'ENSGWIG00000004332', 'ENSGWIG00000004335', 'ptchd4', 'cnih3']

#pigra = ['cdca8', 'ENSGWIG00000012129', 'grhl2b', 'asic4a', 'ENSGWIG00000000065', 'ENSGWIG00000000100', 'atf4a', 'tab1', 'syngr1b', 'cacng2b', 'ENSGWIG00000000124', 'rac2', 'fam83fb', 'ENSGWIG00000000457', 'ENSGWIG00000000466', 'ENSGWIG00000000498', 'ENSGWIG00000000502', 'ENSGWIG00000000505', 'lgals2b', 'ENSGWIG00000000656', 'ENSGWIG00000000735', 'elfn2a', 'ENSGWIG00000000743', 'mfng', 'trmt1', 'nacc1b', 'ier2b', 'cacna1ab', 'ENSGWIG00000002175', 'c3b.2', 'ENSGWIG00000002240', 'ccdc130', 'NKX2', '3', 'ENSGWIG00000002940', 'ENSGWIG00000002962', 'lcor', 'ENSGWIG00000003015', 'slit1b', 'arhgap19', 'e2f5', 'dla', 'ENSGWIG00000003452', 'ENSGWIG00000003838', 'myom2a', 'ENSGWIG00000015286', 'ptp4a1', '5S_rRNA', 'si:dkey-23f9.4', 'abcc10', 'sp2', 'wipf2a', 'capn9', 'ENSGWIG00000020682', 'ENSGWIG00000020683', 'ENSGWIG00000020686', 'akt3b', 'pus10', 'fam161a', 'USP34', 'ENSGWIG00000023117', 'tbce', 'nid1b', 'ero1b', 'ENSGWIG00000023941', 'vps54', 'ENSGWIG00000023995', 'ENSGWIG00000023997', 'ENSGWIG00000023998', 'akap8l', 'ENSGWIG00000024289', 'lpar2b', 'ENSGWIG00000024303', 'ENSGWIG00000024304', 'soul5l', 'gpr108', 'ENSGWIG00000024313', 'ENSGWIG00000024314', 'uncx4.1', 'ENSGWIG00000024316', 'ENSGWIG00000024322', 'ENSGWIG00000024323', 'ENSGWIG00000024434', 'ENSGWIG00000024438', 'ENSGWIG00000024440', 'ENSGWIG00000024441', 'zgc:153247', 'ENSGWIG00000000120', 'ENSGWIG00000000123', 'ENSGWIG00000000128', 'ENSGWIG00000000222', 'thumpd1', 'ENSGWIG00000000303', 'ENSGWIG00000000319', 'ENSGWIG00000000323', 'mad1l1', 'ENSGWIG00000000557', 'ENSGWIG00000000562', 'ENSGWIG00000000568', 'ENSGWIG00000000752', 'ENSGWIG00000001813', 'smcr8b', 'ENSGWIG00000001987', 'ENSGWIG00000002139', 'zgc:77849', 'arpc1b', 'ENSGWIG00000005848', 'ENSGWIG00000008763', 'sdk1a', 'gna12a', 'ENSGWIG00000009632', 'ern2', 'si:dkey-26i13.7', 'si:dkeyp-9d4.5', 'chtf18', 'prkcba', 'ENSGWIG00000007106', 'ENSGWIG00000007145', 'ENSGWIG00000007152', 'trim2a', 'fhdc1', 'arfip1', 'ENSGWIG00000007598', 'pdlim3a', 'sorbs2a', 'tlr3', 'cyp4v8', 'mtnr1aa', 'fat1a', 'ENSGWIG00000008676', 'ENSGWIG00000008678', 'ENSGWIG00000008685', 'ENSGWIG00000008687', 'frg1', 'asah1b', 'klhl2', 'prom1b', 'ENSGWIG00000021108', 'ENSGWIG00000021111', 'ENSGWIG00000000129', 'dctpp1', 'fam160a1a', 'sh3d19', 'rps3a', 'lrba', 'DCLK2', 'nr3c2', 'arhgap10', 'ednraa', 'pou4f2', 'ENSGWIG00000004272', 'znf827', 'smad1', 'hhip', 'gab1', 'inpp4b', 'ENSGWIG00000009815', 'fam13a', 'uso1', 'ppm1k', 'ENSGWIG00000012342', 'coro2a', 'TRMO', 'foxe1', 'sh3gl2a', 'cntln', 'bnc2', 'ENSGWIG00000012921', 'TTC39B', 'dnajb14', 'MCUB', 'cyp2u1', 'papss1', 'dkk2', 'aimp1a', 'tbck', 'npnta', 'gstcd', 'tet2', 'cxxc4', 'ENSGWIG00000000135', 'ENSGWIG00000000369', 'fbxo41', 'smox', 'ENSGWIG00000012210', 'ENSGWIG00000012969', 'slc22a7b.1', 'sod3b', 'lgi2b', 'mttp', 'acer2', 'ENSGWIG00000015969', 'SMURF2', 'SPHK1', 'myh10', 'slc25a38b', 'myh9a', 'map2k7', 'pkn1a', 'si:ch211-214c7.5', 'ENSGWIG00000025534', 'ENSGWIG00000025535', 'ENSGWIG00000025915', 'ENSGWIG00000026099', 'ENSGWIG00000002933', 'ENSGWIG00000002960', 'rasd4', 'NTN1', 'ENSGWIG00000004085', 'ENSGWIG00000004090', 'cfap52', 'usp43a', 'ENSGWIG00000004158', 'jpt1a', 'sept9b', 'ENSGWIG00000004234', 'rhbdl3', 'ENSGWIG00000011455', 'rab11fip4b', 'ENSGWIG00000011548', 'ENSGWIG00000011550', 'ENSGWIG00000011551', 'ENSGWIG00000011556', 'ENSGWIG00000011564', 'NFIX', 'ENSGWIG00000011749', 'ENSGWIG00000011757', 'ENSGWIG00000011759', 'si:zfos-323e3.4', 'SMARCA4', 'ENSGWIG00000021086', 'serhl', 'sept3', 'grin2bb', 'zgc:101679', 'ENSGWIG00000021712', 'rbfox2', 'ENSGWIG00000021717', 'clockb', 'si:dkey-245p14.4', 'kdm4c', 'ENSGWIG00000000992', 'PTPRD', 'ENSGWIG00000023376', 'ENSGWIG00000023375', 'ENSGWIG00000023378', 'tyrp1b', 'ENSGWIG00000023380', 'adgrl3.1', 'gtf2e2', 'ENSGWIG00000024571', 'ENSGWIG00000024646', 'pcdh7a', 'ank2a', 'camk2d2', 'fbxo10', 'ENSGWIG00000026653', 'ptpn9b', 'pax5', 'zcchc7', 'pgm2', 'tbc1d1', 'klf3', 'klhl5', 'wdr19', 'ugdh', 'ENSGWIG00000026757', 'slit2', 'ENSGWIG00000026772', 'ENSGWIG00000026784', 'apbb2b', 'ENSGWIG00000026787', 'ENSGWIG00000026788', 'rbm47', 'si:ch211-113e8.10', 'ENSGWIG00000026792', 'ENSGWIG00000026793', 'ankrd49', 'GPR83', 'doc2d', 'CABP2', 'dlc1', 'slc44a1a', 'ugt8', 'ndst3', 'adra1d', 'ctnna2', 'ENSGWIG00000000861', 'suclg1', 'bdh2', 'TACR3', 'ENSGWIG00000000945', 'ENSGWIG00000000946', 'prkacab', 'zgc:136908', 'ndufb7', 'dnm2b', 'ENSGWIG00000002495', 'si:ch211-114l13.7', 'slc27a1b', 'adgre5b.3', 'ENSGWIG00000019350', 'TFAP4', 'SRL', 'crebbpa', 'cxcl12b', 'ENSGWIG00000021771', 'add3a', 'mxi1', 'dusp5', 'smc3', 'RBM20', 'pdcd4b', 'atp5pd', 'ENSGWIG00000022530', 'caly', 'micu3a', 'slc24a2', 'ENSGWIG00000004055', 'clta', 'tmod1', 'abca1a', 'SGCZ', 'tusc3', 'ENSGWIG00000021035', 'PITPNC1', 'psmd12', 'PRKCA', 'slc25a10', 'ENSGWIG00000000276', 'metrnlb', 'tekt4', 'rln3b', 'zgc:136864', 'mri1', 'CC2D1A', 'ENSGWIG00000010130', 'ENSGWIG00000010140', 'aspscr1', 'TMC6', 'tk1', 'cant1b', 'ldb1b', 'armh3', 'jupa', 'krt98', 'SMAP1', 'lbx1b', 'slc2a15b', 'fgf8b', 'wbp1lb', 'ENSGWIG00000016334', 'ccdc40', 'ENSGWIG00000016352', 'ENSGWIG00000016354', 'si:dkey-237i9.1', 'ENDOV', 'NPTX1', 'rptor', 'p4hb', 'ppp1r27b', 'ENSGWIG00000000864', 'ARHGDIA', 'ENSGWIG00000000898', 'sp6', 'ENSGWIG00000001091', 'ENSGWIG00000001101', 'ENSGWIG00000001120', 'krt97', 'pcdh18a', 'ENSGWIG00000001127', 'pcdh10a', 'ENSGWIG00000001131', 'zgc:77739', 'chrna6', 'rnf38', 'zgc:195282', 'pi4k2b', 'htra3a', 'gpr78a', 'cpz', 'hmx4', 'ENSGWIG00000001929', 'adra2c', 'lrpap1', 'rgs12b', 'ENSGWIG00000002056', 'grk4', 'ENSGWIG00000003004', 'ENSGWIG00000003051', 'ENSGWIG00000003063', '', 'si:dkey-217d24.6', 'slc44a2', 'rgl3a', 'ENSGWIG00000005678', 'ENSGWIG00000005681', 'ENSGWIG00000010077', 'abcc6a', 'rps15a', 'xylt1', 'nomo', 'ENSGWIG00000021987', 'cep170ab', 'wfikkn1', 'ENSGWIG00000022054', 'ENSGWIG00000022055', 'gli2b', 'GABRG3', 'en1b', 'ENSGWIG00000004740', 'ENSGWIG00000004780', 'si:dkey-1h24.2', 'irs2a', 'LYPD6', 'rnd3a', 'ENSGWIG00000007266', 'ENSGWIG00000009117', 'ENSGWIG00000009127', 'ENSGWIG00000009151', 'arhgap1', 'chrm4a', 'ENSGWIG00000010163', 'ENSGWIG00000006284', 'ENSGWIG00000006303', 'pudp', 'nlgn4xa', 'prkx', 'mxra5a', 'ENSGWIG00000006322', 'ENSGWIG00000006327', 'ENSGWIG00000007294', 'zgc:152904', 'ENSGWIG00000011144', 'ENSGWIG00000011182', 'dock9b', 'slc4a10a', 'cldn10a', 'APP', 'cyyr1', 'ADAMTS5', 'ENSGWIG00000026227', 'ENSGWIG00000026230', 'fundc1', 'caskb', 'ENSGWIG00000003616', 'ENSGWIG00000003631', 'ENSGWIG00000003650', 'gpd1c', 'ENSGWIG00000003928', 'ENSGWIG00000003952', 'ENSGWIG00000003957', 'gpd2', 'pwp2h', 'atp7b', 'DMD', 'ENSGWIG00000012104', 'hist1h2a2', 'ENSGWIG00000012343', 'ENSGWIG00000016131', 'ENSGWIG00000016162', 'ENSGWIG00000016169', 'mbd5', 'orc4', 'acvr2ab', 'bzw1b', 'si:ch211-246m6.4', 'si:ch211-246m6.5', 'ENSGWIG00000020880', 'mrpl39', 'rap2ab', 'ENSGWIG00000020935', 'nrp2a', 'mdh1b', 'raph1b', 'cpb2', 'ENSGWIG00000023509', 'ENSGWIG00000023511', 'atp11a', 'ENSGWIG00000024127', 'cdadc1', 'si:ch211-132g1.7', 'zgc:85722', 'ENSGWIG00000007783', 'arhgef7b', 'ENSGWIG00000010286', 'ENSGWIG00000010471', 'ENSGWIG00000010766', 'ENSGWIG00000010774', 'HDAC4', 'ENSGWIG00000013842', 'ENSGWIG00000013896', 'TCEANC', 'ENSGWIG00000013907', 'ENSGWIG00000014791', 'ENSGWIG00000014896', 'ENSGWIG00000014908', 'ENSGWIG00000014913', 'ENSGWIG00000014931', 'FILIP1L', 'ccdc28a', 'ENSGWIG00000015013', 'ENSGWIG00000015093', 'atp1a1a.4', 'CTDSP1', 'ENSGWIG00000021651', 'igfbp2a', 'ENSGWIG00000021677', 'CREB1', 'klf7a', 'ENSGWIG00000021835', 'erbb4a', 'ENSGWIG00000024821', 'lancl1', 'mylz3', 'gart', 'kcnh7', 'ENSGWIG00000000472', 'cobll1a', 'ENSGWIG00000001172', 'ENSGWIG00000001175', 'xirp2a', 'b3galt1a', 'ENSGWIG00000001178', 'ndfip2', 'ENSGWIG00000001223', 'ENSGWIG00000001228', 'slitrk6', 'ENSGWIG00000001235', 'ENSGWIG00000001237', 'MID1', 'ARHGAP6', 'FRMPD4', 'ENSGWIG00000001324', 'ENSGWIG00000001332', 'ENSGWIG00000001336', 'ENSGWIG00000001338', 'ENSGWIG00000001340', 'zeb2b', 'ENSGWIG00000001348', 'ENSGWIG00000019257', 'si:ch211-186j3.6', 'dpy19l3', 'ENSGWIG00000019293', 'ENSGWIG00000019294', 'tshz3b', 'ENSGWIG00000019298', 'ENSGWIG00000019301', 'ENSGWIG00000019303', 'ccne1', 'ENSGWIG00000019341', 'cd81a', 'ENSGWIG00000019709', 'ENSGWIG00000020057', 'celf1', 'ENSGWIG00000020512', 'ENSGWIG00000020513', 'chsy1', 'lrrk1', 'cep89', 'zgc:165481', 'ENSGWIG00000021891', 'mctp2b', 'ENSGWIG00000021896', 'SLCO3A1', 'ENSGWIG00000000451', 'GLO1', 'ENSGWIG00000000540', 'fam219b', 'igf1rb', 'ENSGWIG00000000995', 'ENSGWIG00000001063', 'ENSGWIG00000001078', 'sv2ba', 'si:dkey-172h23.2', 'ntrk3b', 'ENSGWIG00000001155', 'ENSGWIG00000001158', 'ENSGWIG00000001160', 'mrpl46', 'mrps11', 'si:cabz01113374.3', 'pnoca', 'kif13ba', 'ENSGWIG00000010653', 'tdp1', 'ENSGWIG00000010657', 'eprs1', 'ENSGWIG00000011285', 'ENSGWIG00000011306', 'ENSGWIG00000011329', 'ENSGWIG00000011339', 'cemip', 'abhd17c', 'arnt2', 'mthfs', 'si:cabz01029535.1', 'si:cabz01029535.1-adamts17', 'adamts17', 'lins1', 'asb7', 'aldh1a3', 'ENSGWIG00000017036', 'ENSGWIG00000017049', 'cyp1a', 'ENSGWIG00000017054', 'CSK', 'cplx3b', 'ulk3', 'larp6a', 'ENSGWIG00000020999', 'THSD4', 'pepd', 'ENSGWIG00000021087', 'si:ch211-269c21.2', 'ENSGWIG00000003030', 'si:ch211-79l17.1', 'gpib', 'wtip', 'heatr3', 'tent4b', 'adcy7', 'brd7', 'nkd1', 'nod2', 'cylda', 'sall1a', 'tox3', 'ENSGWIG00000007870', 'ENSGWIG00000007873', 'chd9', 'fto', 'irx3a', 'ENSGWIG00000012637', 'irx5a', 'ENSGWIG00000012647', 'irx6a', 'mmp2', 'slc6a2', 'slc12a4', 'rpp25b', 'ENSGWIG00000020407', 'ENSGWIG00000020499', 'hcn4', 'nptnb', 'ENSGWIG00000022198', 'ube2q1', 'SHF', 'cd276', 'ENSGWIG00000022257', 'insyn1', 'loxl1', 'scamp5b', 'igdcc4', 'UNC13C', 'onecut1', 'fam214a', 'myo5aa', 'gnb5b', 'ENSGWIG00000026142', 'trpm7', 'ENSGWIG00000026633', 'ENSGWIG00000026637', 'ENSGWIG00000026643', 'ENSGWIG00000026646', 'znf710b', 'si:ch211-107o10.3', 'ENSGWIG00000026687', 'frmd5', 'pstpip1b', 'ISL2', 'ENSGWIG00000000305', 'ENSGWIG00000000432', 'tln2a', 'nfatc3a', 'styx', 'ENSGWIG00000010283', 'ENSGWIG00000010284', 'mdga1', 'ENSGWIG00000010428', 'slc7a9', 'nip7', 'greb1', 'spata17', 'ENSGWIG00000011713', 'esrrga', 'ENSGWIG00000011725', 'FZD3', 'kif26ba', 'SMYD3', 'SCCPDH', 'gcfc2', 'efcab2', 'ENSGWIG00000018529', 'smyd2a', 'galnt2', 'urb2', 'abcb10', 'ENSGWIG00000020102', 'tspan15', 'ENSGWIG00000020107', 'vps26a', 'ddx21', 'chs1', 'ENSGWIG00000000552', 'ENSGWIG00000000887', 'rps6ka2', 'immp1l', 'elp4', 'depdc7a', 'kiaa1549lb', 'spon1a', 'syt9a', 'ppfibp2a', 'MAF', 'ENSGWIG00000006287', 'chchd2', 'CDYL2', 'gse1', 'dusp22a', 'irf8', 'ENSGWIG00000006319', 'bdnf', 'KIF18A', 'mettl15', 'lgr4', 'mybpc3', 'spi1b', 'slc39a13', 'gas2b', 'ano5b', 'igdcc3', 'ENSGWIG00000017487', 'FAM189A1', 'tjp1a', 'terb2', 'apba2b', 'syt7b', 'plekha7b', 'sox6', 'ENSGWIG00000024774', 'rras2', 'ENSGWIG00000024802', 'lmo1', 'stk33', 'dennd2b', 'tmem9b', 'scube2', 'ipo7', 'sbf2', 'ctr9', 'galnt18b', 'ENSGWIG00000018698', 'mical2a', 'ENSGWIG00000019008', 'usp47', 'bola3', 'nfat5b', 'rpl13', 'cpne7', 'zgc:162592', 'ENSGWIG00000023881', 'zfhx3', 'crispld2', 'usp10', 'marveld3', 'PHLPP2', 'ap1g1', 'myo1ea', 'rnf111', 'sltm', 'aldh1a2', 'polr2m', 'tcf12', 'znf280d', 'nedd4a', 'tradd', 'cog4', 'gfod2', 'calb2b', 'ENSGWIG00000026150', 'ENSGWIG00000026158', 'furina', 'rhcgl1', 'kif7', 'ENSGWIG00000003181', 'ENSGWIG00000003190', 'mvda', 'dpep1', 'ankrd11', 'cdh15', 'ACSF3', 'cbfa2t3', 'galns', 'cmtm3', 'nae1', 'nsun3', 'epha6', 'kctd12.2', 'gpc6a', 'gpc5a', 'rasa3', 'pros1', 'ENSGWIG00000015546', 'ENSGWIG00000015551', 'adprhl1', 'lamp1a', 'tgfbrap1', 'nkpd1', 'ENSGWIG00000018771', 'ENSGWIG00000018774', 'pcdh17', 'diaph3', 'tdrd3', 'PCDH20', 'pcdh9', 'pou4f1', 'cyp27c1', 'gtf2f2a', 'ENSGWIG00000019046', 'baz2ba', 'march7', 'pla2r1', 'itgb6', 'rbms1a', 'atic', 'stk16', 'RS1', 'cdkl5', 'scml2', 'nhsb', 'ENSGWIG00000002084', 'dtwd1', 'fgf7', 'cops2', 'SECISBP2L', 'ENSGWIG00000006295', 'slc12a1', 'si:dkeyp-86f7.4', 'sema6dl', 'dmxl2', 'scg3', 'ENSGWIG00000022524', 'pde8a', 'ENSGWIG00000023981', 'ENSGWIG00000023996', 'TM6SF1', 'zrsr2', 'ENSGWIG00000024306', 'ENSGWIG00000024354', 'rxfp3.3a2', 'itga11a', 'KIF23', 'ENSGWIG00000024494', 'TLE3', 'uacab', 'map2k5', 'SMAD3', 'slc39a1', 'abhd2a', 'dapk2a', 'rab11a', 'cd82a', 'kcng4b', 'ENSGWIG00000006930', 'ENSGWIG00000007011', 'ENSGWIG00000007025', 'cngb1a', 'kifc3', 'cebpa', 'ENSGWIG00000010036', 'fhod1', 'plekhg4', 'ENSGWIG00000011057', 'ENSGWIG00000011312', 'mpped2a', 'nav2a', 'dbx1a', 'slc6a5', 'ENSGWIG00000012438', 'NELL1', 'ppfia1', 'rbpms2a', 'psma4', 'slc25a44b', 'ENSGWIG00000023328', 'brsk2b', 'ENSGWIG00000000463', 'shank2a', 'ENSGWIG00000000713', 'ttc17', 'kcnq1.1', 'luzp2', 'ENSGWIG00000000858', 'FIBIN', 'bbox1', 'madd', 'kcna4', 'tspan4a', 'large2', 'ppip5k1a', 'cart3', 'ENSGWIG00000011404', 'ENSGWIG00000011425', 'eef1db', 'ENSGWIG00000007221', 'gmds', 'foxq1b', 'exoc2', 'SLC22A23', 'dspb', 'ENSGWIG00000007341', 'ENSGWIG00000007345', 'ENSGWIG00000007346', 'ENSGWIG00000007371', 'ENSGWIG00000007719', 'rgs16', 'rgs8', 'brinp3a.1', 'ENSGWIG00000007754', 'brinp3a.2', 'ENSGWIG00000007798', 'ENSGWIG00000007815', 'RGS1', 'si:ch211-117l17.5', 'ENSGWIG00000007901', 'b3galt2', 'ENSGWIG00000007906', 'ENSGWIG00000007910', 'depdc1a', 'fnbp1l', 'ttc14', 'fxr1', 'ENSGWIG00000005481', 'si:dkey-211e20.10', 'mccc1', 'ENSGWIG00000013266', 'ENSGWIG00000013283', 'FNDC7', 'fam183a', 'ENSGWIG00000013835', 'ENSGWIG00000013861', 'lrp8', 'ENSGWIG00000013932', 'ebna1bp2', 'tie1', 'DDAH1', 'ENSGWIG00000016363', 'ENSGWIG00000016365', 'SYDE2', 'MCOLN3', 'ENSGWIG00000016459', 'ENSGWIG00000016462', 'si:ch211-262h13.5', 'gpc1a', 'ENSGWIG00000017075', 'ENSGWIG00000017092', 'ENSGWIG00000017096', 'ENSGWIG00000017636', 'meltf', 'ENSGWIG00000024449', 'ENSGWIG00000024483', 'cp', 'ENSGWIG00000004415', 'lepr', 'dnajc6', 'jak1', 'raver2', 'cachd1', 'ror1', 'pgm1', 'foxd3', 'atg4c', 'dock7', 'sgip1a', 'pde4ba', 'ENSGWIG00000023956', 'ano8b', 'gtpbp3', 'babam1', 'ENSGWIG00000023986', 'nr2f6b', 'si:ch73-61d6.3', 'csnk1g2b', 'zap70', 'adamts10', 'ENSGWIG00000025111', 'fcho1', 'tprb', 'arpc5b', 'ENSGWIG00000025735', 'ivns1abpa', 'swt1', 'ENSGWIG00000025740', 'ptprc', 'nek7', 'crb1', 'ENSGWIG00000025959', 'RABGAP1L', 'homer3b', 'PGPEP1', 'ENSGWIG00000026146', 'kank3', 'arid3a', 'r3hdm4', 'zgc:172302', 'ENSGWIG00000001303', 'smim7', 'CRTC1', 'ENSGWIG00000002093', 'mast3b', 'slc5a5', 'cluap1', 'ccdc124', 'KCNN1', 'uhrf1', 'kdm4b', 'ptprsa', 'ENSGWIG00000005502', 'si:zfos-943e10.1', 'kif14', 'ENSGWIG00000005770', 'nr5a2', 'ENSGWIG00000005850', 'ENSGWIG00000006451', 'ENSGWIG00000006452', 'nppc', 'b3gnt7', 'armc9', 'ENSGWIG00000006753', 'ENSGWIG00000006862', 'ENSGWIG00000010056', 'ENSGWIG00000010071', 'ENSGWIG00000010072', 'ENSGWIG00000010080', 'ENSGWIG00000010081', 'PFN2', 'her6', 'opa1', 'atp13a3', 'ENSGWIG00000017304', 'xxylt1', 'srsf11', 'ankrd13c', 'cthl', 'zranb2', 'negr1', 'adgrl4', 'adgrl2a', 'ttll7', 'GNG5', 'ENSGWIG00000026088', 'plaat1', 'gnb4b', 'actl6a', 'pex5la', 'ENSGWIG00000026403', 'ENSGWIG00000026408', 'TMEM47', 'tab3', 'si:dkey-30h22.11', 'ENSGWIG00000026429', 'guk1b', 'rnf220b', 'dmap1', 'hnrnpm', 'timm44', 'ctxn1', 'fbn2b', 'lsm7', 'ENSGWIG00000026467', 'ENSGWIG00000026469', 'arhgef18b', 'insrb', 'ENSGWIG00000026491', 's1pr4', 'ncln', 'cers1', 'cope', 'tnfaip8l1', 'ENSGWIG00000026516', 'haus5', 'sin3b', 'ENSGWIG00000000375', 'xpr1a', 'acbd6', 'cep350', 'cacna1ea', 'ENSGWIG00000008436', 'osbpl9', 'ENSGWIG00000010909', 'faf1', 'dmrta2', 'elavl4', 'agbl4', 'ENSGWIG00000011350', 'gclm', 'ccl25b', 'pole4', 'tpm4a', 'si:ch1073-396h14.1', 'safb', 'hdgfl2', 'ap3d1', 'C2CD4C', 'gna15.4', 'ENSGWIG00000013242', 'tle5', 'tle2b', 'mier2', 'ENSGWIG00000014326', 'shc2', 'trabd2b', 'ENSGWIG00000014368', 'ENSGWIG00000000474', 'fyco1a', 'GHRHR', 'faah', 'elovl1b', 'slc44a5b', 'lhx8b', 'erich3', 'ssbp3b', 'glis1b', 'elovl8b', 'rorca', 'sema6bb', 'zgc:77486', 'abhd17ab', 'rgl1', 'scfd2', 'lnx1', 'gsx2', 'pdgfra', 'kita', 'ENSGWIG00000007773', 'kdr', 'tmem165', 'CLOCK', 'exoc1', 'chst14', 'ENSGWIG00000012507', 'paics', 'tgm1l2', 'ENSGWIG00000015324', 'ENSGWIG00000018962', 'usp24', 'bloc1s2', 'dab1a', 'ENSGWIG00000000364', 'oma1', 'suco', 'ENSGWIG00000000631', 'rangrf', 'myoc', 'prrc2c', 'prdx6', 'caiap', 'faslg', 'ENSGWIG00000001082', 'angptl1a', 'ralgps2', 'echdc2', 'si:dkey-52j6.3', 'ENSGWIG00000001733', 'pfas', 'ENSGWIG00000005188', 'ENSGWIG00000005583', 'il12rb2', 'serbp1a', 'ddr2a', 'uap1', 'ENSGWIG00000007555', 'ENSGWIG00000007661', 'tjp3', 'si:ch73-63e15.2', 'hapln4', 'ENSGWIG00000008514', 'ENSGWIG00000008538', 'ENSGWIG00000008539', 'nr2c2ap', 'ENSGWIG00000008568', 'gatad2ab', 'cilp2', 'si:ch211-212d10.2', 'ENSGWIG00000008632', 'ENSGWIG00000000162', 'midn', 'atp5f1d', 'cbarpb', 'RNF126', 'bsg', 'ENSGWIG00000000249', 'polrmt', 'ENSGWIG00000000462', 'hsd11b1la', 'rgmd', 'celf5b', 'creb3l3a', 'ENSGWIG00000000527', 'EEF2', 'unc13a', 'mbd3b', 'mbd3a', 'tcf3b', 'zbtb7a', 'map2k2a', 'lingo3a', 'ENSGWIG00000019368', 'org', 'ell', 'SSBP4', 'tmem59l', 'crlf1b', 'notch2', 'sec22bb', 'ENSGWIG00000020059', 'npl', 'edem3', 'zgc:92140', 'ENSGWIG00000020084', 'ociad2', 'ENSGWIG00000020098', 'fryl', 'palld', 'cpeb2', 'prdm5', 'ENSGWIG00000020613', 'ENSGWIG00000020615', 'slc34a2b', 'ENSGWIG00000020676', 'cdh24a', 'ENSGWIG00000004586', 'nhsa', 'RAI2', 'CDKL5', 'SHROOM2', 'zgc:153311', 'ENSGWIG00000006245', 'chd1l', 'nit2', 'tomm70a', 'jade3', 'rp2', 'slc9a7', 'tubgcp5', 'ENSGWIG00000013341', 'oca2', 'gabrb3', 'atp10a', 'tmem131', 'si:ch211-201h21.5', 'bivm', 'zgc:172121', 'atp1b1a', 'tp63', 'tprg1', 'ENSGWIG00000015237', 'lpp', 'magoh', 'ENSGWIG00000004935', 'RPL5', 'ephx4', 'tgfbr3', 'barhl2', 'ENSGWIG00000009438', 'ENSGWIG00000009439', 'lrrc8db', 'lrrc8c', 'kyat3', 'ENSGWIG00000010721', 'lmo4b', 'hs2st1b', 'SOX14', 'hdlbpa', 'zgc:103559', 'TTC39C', 'ENSGWIG00000016759', 'klhl30', 'sned1', 'ryk', 'ENSGWIG00000018800', 'ENSGWIG00000018802', 'ENSGWIG00000018808', 'hyi', 'ptprfa', 'kdm4aa', 'artna', 'ENSGWIG00000025214', 'ENSGWIG00000025216', 'ENSGWIG00000000187', 'inpp5d', 'ENSGWIG00000000442', 'sap130a', 'amotl2a', 'cep63', 'arhgap39', 'EPS15L1', 'axdnd1', 'ENSGWIG00000001008', 'ENSGWIG00000001048', 'NAALADL2', 'ENSGWIG00000001057', 'nlgn1', 'ENSGWIG00000001097', 'ENSGWIG00000001098', 'ect2', 'mcf2l2', 'si:ch73-74h11.1', 'ENSGWIG00000008413', 'abraa', 'zgc:171566', 'PTPRT', 'ENSGWIG00000021445', 'chd6', 'ENSGWIG00000000153', 'mych', 'mtss1', 'ENSGWIG00000000388', 'ENSGWIG00000000708', 'ahcy', 'ENSGWIG00000000827', 'ENSGWIG00000001134', 'si:dkeyp-87e7.4', 'stab1', 'ENSGWIG00000001196', 'aspn', 'gpr37l1a', 'atp2b4', 'zgc:162255', 'cpne1', 'ENSGWIG00000007545', 'ENSGWIG00000007565', 'pnp4a', 'trib3', 'ENSGWIG00000000134', 'ENSGWIG00000000203', 'pank4', 'draxin', 'rpl22', 'chd5', 'arhgef10la', 'ENSGWIG00000001413', 'igsf21a', 'ephb2b', 'ENSGWIG00000005019', 'alpl', 'ece1', 'eif4g3a', 'sh2d5', 'PLXNA2', 'si:ch211-286o17.1', 'ENSGWIG00000022343', 'ENSGWIG00000022344', 'ENSGWIG00000022350', 'magi3b', 'lrig2', 'padi2', 'si:dkey-166d12.2', 'anks1ab', 'snrpc', 'pacsin1b', 'nudt3b', 'grm4', 'ENSGWIG00000023406', 'csrp1b', 'tnni1b', 'tnnt2b', 'pkp1b', 'rplp2', 'KCNA3', 'ngfb', 'tspan2a', 'slc16a1a', 'si:dkey-238f9.1', 'ENSGWIG00000005008', 'ncoa5', 'ENSGWIG00000007499', 'si:ch211-81n22.1', 'ENSGWIG00000017842', 'cers5', 'smarcd1', 'ENSGWIG00000018806', 'eya2', 'rtf2', 'tfap2c', 'cdh22', 'znfx1', 'cbfa2t2', 'ENSGWIG00000025447', 'tox2', 'st7l', 'cttnbp2nla', 'KCND3', 'INKA2', 'ENSGWIG00000025555', 'timm17a', 'lmod1b', 'ENSGWIG00000025685', 'ppp1r16b', 'NXPH4', 'STAC3', 'INHBC', 'gli1', 'ENSGWIG00000025812', 'dctn2', 'KIF5A', 'soat2', 'igsf8', 'sp7', 'ENSGWIG00000025847', 'ENSGWIG00000025848', 'PCBP2', 'MAP3K12', 'myg1', 'ENSGWIG00000025870', 'ENSGWIG00000025875', 'ENSGWIG00000005945', 'alas1', 'dusp7', 'rpl29', 'iars1', 'atp2b2', 'slc6a11b', 'slc6a1b', 'ENSGWIG00000021449', 'PPARG', 'ENSGWIG00000022362', 'megf6b', 'kcnab2a', 'park7', 'pax7b', 'mrps16', 'errfi1a', 'tas1r3', 'mmel1', 'ENSGWIG00000022736', 'si:dkey-264d12.4', 'FOXJ3', 'ENSGWIG00000022758', 'kaznb', 'skib', 'si:dkey-183j2.10', 'ENSGWIG00000022770', 'cacna2d2a', 'ENSGWIG00000023170', 'ddi2', 'sema3fa', 'rbm6', 'camkva', 'slc38a3b', 'rbm5', 'rrp9', 'pbrm1l', 'ENSGWIG00000020816', 'ca16b', 'cadpsb', 'slc6a22.1', 'trub2', 'hdhd3', 'EIF4ENIF1', 'zgc:172271', 'frs3', 'kbtbd12', 'slc26a6l', 'myhb', 'rad54l2', 'bsnb', 'cacna1da', 'cacna2d3', 'ENSGWIG00000025975', 'rpusd4', 'ENSGWIG00000025981', 'ENSGWIG00000005003', 'si:ch1073-335m2.2', 'ENSGWIG00000005892', 'ora3', 'ENSGWIG00000005951', 'ENSGWIG00000005952', 'capzb', 'celsr2', 'dstyk', 'ccnd3', 'usp49', 'SLC6A17', 'ENSGWIG00000010216', 'WDR77', 'GPR61', 'ENSGWIG00000012991', 'srgap3', 'cav3', 'gpr61l', 'tex264a', 'slc38a3a', 'gnai2a', 'inka1a', 'ENSGWIG00000020421', 'ENSGWIG00000020427', 'ENSGWIG00000020446', 'eefsec', 'ENSGWIG00000000717', 'tatdn2', 'rps26', 'prkag1', 'ENSGWIG00000012941', 'ENSGWIG00000012987', 'c1ql4b', 'ENSGWIG00000014715', 'dip2bb', 'scn8ab', 'slc4a8', 'itpr1a', 'sumf1', 'ENSGWIG00000022032', 'si:dkey-28n18.9', 'ENSGWIG00000022231', 'rbms2a', 'rargb', 'ENSGWIG00000022400', 'ENSGWIG00000022455', 'osgn1', 'ENSGWIG00000022498', 'mettl1', 'suox', 'ENSGWIG00000005534', 'zgc:113307', 'mapkapk2b', 'si:dkey-85p17.3', 'ikbke', 'srgap2', 'zgc:101564', 'foxp4', 'si:dkey-91m3.1', 'ENSGWIG00000009578', 'tfeb', 'ppfia4', 'myog', 'ENSGWIG00000012912', 'psma5', 'nab2', 'dgkab', 'kdm5ba', 'inavab', 'gpr25', 'gdf11', 'ENSGWIG00000025510', 'pcsk1', 'SNORD57', 'tpgs2', 'celf4', 'myorg', 'fam219aa', 'ENSGWIG00000005517', 'crybb3', 'cita', 'MYL2', 'pxna', 'loxl2a', 'tacc1', 'stx18', 'msx1a', 'ENSGWIG00000009977', 'ENSGWIG00000009988', 'ENSGWIG00000009989', 'ENSGWIG00000010154', 'ENSGWIG00000016858', 'chchd6b', 'ENSGWIG00000016920', 'ENSGWIG00000016927', 'quo', 'ENSGWIG00000018578', 'eif4bb', 'vdrb', 'twist3', 'dnajc11b', 'ENSGWIG00000018772', 'si:ch211-160o17.4', 'ddx19', 'si:dkey-33m11.6', 'mapk14b', 'kif21b', 'cpne5b', 'ENSGWIG00000026097', 'ENSGWIG00000026100', 'ENSGWIG00000026102', 'camk1b', 'ENSGWIG00000007422', 'ENSGWIG00000014029', 'WDR1', 'setd5', 'mtmr14', 'ENSGWIG00000015230', 'klhdc8a', 'slc41a1', 'sox13', 'snrpe', 'ENSGWIG00000015807', 'wnt4', 'ENSGWIG00000015815', 'ENSGWIG00000015821', 'ENSGWIG00000015836', 'txnrd3', 'CHST13', 'ENSGWIG00000016035', 'phactr3b', 'rpn2', 'ENSGWIG00000018882', 'ENSGWIG00000018997', 'cntn4', 'ENSGWIG00000021155', 'ENSGWIG00000021257', 'chl1b', 'ENSGWIG00000024538', 'ENSGWIG00000024539', 'ENSGWIG00000024541', 'cntn3b', 'pdzrn3b', 'ppp4r2b', 'gxylt2', 'shq1', 'rybpb', 'ENSGWIG00000024745', 'ENSGWIG00000024786', 'foxp1b', 'ENSGWIG00000024794', 'ENSGWIG00000024795', 'mitfa', 'frmd4ba', 'setmar', 'ENSGWIG00000024820', 'NAAA', 'prickle2b', 'adamts9', 'magi1b', 'ENSGWIG00000001164', 'ENSGWIG00000001166', 'lrig1', 'ENSGWIG00000001396', 'ENSGWIG00000001414', 'kbtbd8', 'ENSGWIG00000001421', 'suclg2', 'ENSGWIG00000001681', 'ENSGWIG00000001685', 'TAFA1', 'tafa4b', 'eogt', 'tmf1', 'fgd5a', 'SFMBT1', 'ENSGWIG00000007388', 'mustn1b', 'ENSGWIG00000007507', 'gata2a', 'rpn1', 'ENSGWIG00000009779', 'cidec', 'ENSGWIG00000009786', 'sec61a1l', 'si:dkey-202e22.2', 'pfkfb4a', 'tada3l', 'ENSGWIG00000010530', 'RBM38', 'ENSGWIG00000011704', 'ENSGWIG00000011705', 'cdh4', 'ENSGWIG00000011987', 'ENSGWIG00000011988', 'ENSGWIG00000011990', 'ENSGWIG00000011991', 'ncoa3', 'id1', 'sulf2a', 'zgc:92107', 'phf20a', 'trpc4apa', 'ENSGWIG00000016048', 'fam83d', 'ENSGWIG00000000784', 'ENSGWIG00000000799', 'hcfc1a', 'opn1lw1', 'ENSGWIG00000001125', 'ccdc120', 'grm6b', 'chrm2a', 'mtpn', 'ENSGWIG00000005265', 'si:dkey-97m3.1', 'ccnd2b', 'FGF6', 'ada2b', 'ENSGWIG00000006538', 'def6c', 'ENSGWIG00000007599', 'sfmbt2', 'tmem110l', 'atp5f1c', 'gata3', 'ENSGWIG00000010793', 'CELF2', 'ENSGWIG00000010976', 'usp6nl', 'ENSGWIG00000011036', 'plxnb2a', 'slc35e3', 'ENSGWIG00000012261', 'ssbp1', 'ENSGWIG00000012678', 'rassf3', 'anks1b', 'si:ch211-244b2.1', 'epyc', 'kera', 'lum', 'dcn', 'ENSGWIG00000019193', 'btg1', 'fam3c', 'si:dkey-159a18.1', 'ENSGWIG00000019457', 'snd1', 'hgfa', 'cacna2d1a', 'ENSGWIG00000024424', 'sema3aa', 'tspan11', 'prmt8b', 'cracr2ab', 'ccdc59', 'mettl25', 'tmtc2a', 'ENSGWIG00000009165', 'ENSGWIG00000009168', 'iqsec3a', 'st8sia1', 'ENSGWIG00000009387', 'ENSGWIG00000009481', 'lrguk', 'exoc4', 'chchd3a', 'plxna4', 'nrf1', 'ube2h', 'smo', 'cacna1c', 'ENSGWIG00000022294', 'cacna2d4a', 'ENSGWIG00000022322', 'ninj2', 'ERC1', 'wnt5b', 'adipor2', 'ENSGWIG00000022948', 'ENSGWIG00000022951', 'LOC114465680', 'ENSGWIG00000022959', 'RASSF8', 'pkp2', 'nudt4b', 'ube2na', 'mrpl42', 'ints13', 'ENSGWIG00000023459', 'ENSGWIG00000023464', 'TMEM229A', 'gpr37b', 'ENSGWIG00000023487', 'ENSGWIG00000023489', 'ENSGWIG00000023491', 'grm8a', 'ENSGWIG00000002605', 'ZNF800', 'ATXN7L1', 'gsap', 'ENSGWIG00000002637', 'lrrc17', 'pus7l', 'ENSGWIG00000002659', 'gxylt1b', 'pdzrn4', 'dnajb9b', 'akr1b1.1', 'tfec', 'si:ch211-262n1.4', 'foxp2', 'ENSGWIG00000017781', 'ENSGWIG00000017786', 'tmem168a', 'ifrd1', 'dock4b', 'ENSGWIG00000021023', 'lrrn3b', 'ENSGWIG00000021027', 'strip2', 'ENSGWIG00000024627', 'cdkn1d', 'ENSGWIG00000024630', 'ENSGWIG00000024632', 'ENSGWIG00000024636', 'pparaa', 'wnt7ba', 'kdm5a', 'USP15', 'ENSGWIG00000025161', 'pim3', 'creld2', 'brd1a', 'ENSGWIG00000025180', 'ENSGWIG00000025181', 'TAFA5', 'tbc1d22a', 'cerk', 'gramd4a', 'celsr1a', 'trmu', 'alg10', 'CPNE8', 'ENSGWIG00000025598', 'SLC2A13', 'ENSGWIG00000001346', 'sox5', 'ENSGWIG00000007382', 'ENSGWIG00000007384', 'ENSGWIG00000007476', 'arl1', 'atp23', 'lrig3', 'zgc:165507', 'ENSGWIG00000010012', 'ETV6', 'SYN3', 'si:cabz01085950.1', 'elk3', 'ENSGWIG00000011303', 'phyh', 'mcm10', 'ccdc3a', 'camk1da', 'cdc123', 'fbxo18', 'ENSGWIG00000014410', 'slc38a4', 'slc38a2', 'arid2', 'gas2l3', 'stk38l', 'ENSGWIG00000021240', 'zgc:162331', 'ENSGWIG00000021419', 'ENSGWIG00000021420', 'pik3cg', 'ano2b', 'ENSGWIG00000023819', 'osgep', 'nav3', 'e2f7', 'OSBPL8', 'phlda1', 'slc26a3.2', 'cbll1', 'ppp6r2a', 'ENSGWIG00000013312', 'gdi2', 'ENSGWIG00000014375', 'net1', 'mkln1', 'chst11', 'slc41a2b', 'aldh1l2', 'samm50l', 'scube1', 'mpped1', 'sult4a1', 'ENSGWIG00000020263', 'dennd5b', 'ipo8', 'ENSGWIG00000022813', 'si:dkey-8e10.3', 'dhx57', 'crebl2', 'ENSGWIG00000023026', 'cped1', 'ENSGWIG00000023158', 'ENSGWIG00000023162', 'krr1', 'ENSGWIG00000023530', 'ENSGWIG00000023536', 'ENSGWIG00000023790', 'COG5', 'plekha5', 'si:ch73-278m9.1', 'kcnc2', 'ENSGWIG00000004990', 'phax', 'dnajc2', 'slc26a5', 'ENSGWIG00000006723', 'ENSGWIG00000007383', 'ENSGWIG00000007387', 'kcna1a', 'ENSGWIG00000007395', 'pthlhb', 'tmpob', 'actr6', 'sycp3', 'ENSGWIG00000007502', 'ENSGWIG00000007919', 'frs2b', 'ENSGWIG00000008997', 'lamb1a', 'ENSGWIG00000009173', 'ENSGWIG00000011091', 'ENSGWIG00000011102', 'ENSGWIG00000011225', 'ambra1b', 'ENSGWIG00000011278', 'anpepb', 'ENSGWIG00000011394', 'ENSGWIG00000012042', 'tln2b', 'TPM1', 'scaper', 'ENSGWIG00000017824', 'si:ch211-69m14.1', 'si:ch211-234p18.3', 'sema4ba', 'zgc:162879', 'si:ch211-13k12.2', 'pex11a', 'GNRHR', 'bcar1', 'ENSGWIG00000005412', 'ENSGWIG00000010487', 'ripor1', 'ENSGWIG00000010680', 'ENSGWIG00000010681', 'ENSGWIG00000010682', 'ENSGWIG00000010685', 'SLCO1C1', 'gpr22b', 'ENSGWIG00000010700', 'hsp90b1', 'ENSGWIG00000011625', 'ptdss2', 'tmem168b', 'bmt2', 'ENSGWIG00000012198', 'ENSGWIG00000012199', 'si:ch211-272n13.3', 'ENSGWIG00000012209', 'AVPR1A', 'U4', 'ppm1h', 'fbln1', 'wnt7bb', 'PPARA', 'ENSGWIG00000016323', 'mrps35', 'ENSGWIG00000016332', 'DENND2A', 'adck2', 'ENSGWIG00000018807', 'parvg', 'shisal1b', 'plxnb2b', 'ENSGWIG00000000164', 'appl2', 'ENSGWIG00000001325', 'nuak1b', 'ENSGWIG00000001331', 'nell2a', 'ENSGWIG00000007749', 'ENSGWIG00000007792', 'ENSGWIG00000008968', 'ENSGWIG00000008975', 'ENSGWIG00000008979', 'nat10', 'adal', 'caprin1b', 'lmo2', 'tspan18b', 'cd82b', 'tp53i11b', 'si:dkey-29p10.4', 'ENSGWIG00000012340', 'ENSGWIG00000012349', 'RELN', 'PLEKHA5', 'ENSGWIG00000012460', 'ENSGWIG00000012466', 'ENSGWIG00000012467', 'parp16', 'sqor', 'cep152', 'tjp1b', 'ENSGWIG00000013038', 'pard6a', 'ENSGWIG00000013046', 'ENSGWIG00000013049', 'galn', 'ENSGWIG00000013063', 'ENSGWIG00000013068', 'ENSGWIG00000013075', 'slco3a1', 'fam174b', 'ENSGWIG00000005028', 'rgma', 'ENSGWIG00000006097', 'ENSGWIG00000006100', 'lysmd4', 'ENSGWIG00000006104', 'nr2f2', 'mctp2a', 'mef2aa', 'SYNM', 'ENSGWIG00000006275', 'ENSGWIG00000006318', 'ENSGWIG00000006737', 'ENSGWIG00000006803', 'ptprq', 'b4galnt3b', 'ENSGWIG00000007084', 'si:dkey-266m15.5', 'dnaja2b', 'ENSGWIG00000007154', 'fam107b', 'si:ch211-233h19.2', 'grm3', 'ENSGWIG00000007363', 'sema3d', 'sema3ab', 'si:dkey-266j7.2', 'hgfb', 'lrrc4.2', 'ENSGWIG00000019034', 'ENSGWIG00000019373', 'si:ch211-220f12.1', 'ENSGWIG00000020159', 'ENSGWIG00000020161', 'ENSGWIG00000020204', 'odf3b', 'ENSGWIG00000000563', 'ENSGWIG00000000567', 'mapk8ip2', 'cpt1b', 'sephs1', 'wwox', 'mafb', 'ENSGWIG00000002477', 'ENSGWIG00000002511', 'ENSGWIG00000002516', 'fbxl14a', 'wnk1a', 'TMEM60', 'RSBN1L', 'ENSGWIG00000004256', 'ENSGWIG00000004260', 'ENSGWIG00000004396', 'sinhcaf', 'magi2b', 'ENSGWIG00000009093', 'ppfibp1b', 'arntl2', 'si:ch211-219a15.3', 'plekhg7', 'itpr2', 'bhlhe41', 'rassf8a', 'kras', 'ENSGWIG00000025524', 'ENSGWIG00000025525', 'ENSGWIG00000025526', 'banp', 'ENSGWIG00000025529', 'polg', 'rccd1', 'ENSGWIG00000005519', 'man2a2', 'furinb', 'ENSGWIG00000009357', 'rag2', 'ENSGWIG00000009364', 'ENSGWIG00000009367', 'ENSGWIG00000009369', 'lrrc4ca', 'EHF', 'pdhx', 'slc1a2b', 'pamr1', 'fjx1', 'trim44', 'LDLRAD3', 'ENSGWIG00000010933', 'ENSGWIG00000010947', 'hrasb', 'ldha', 'ENSGWIG00000011006', 'abtb2b', 'ENSGWIG00000011064', 'tmem86a', 'spty2d1', 'uevld', 'cpt1ab', 'si:ch211-236l14.4', 'mrpl23', 'ENSGWIG00000011871', 'igf2b', 'nap1l4a', 'phlda2', 'osbpl5', 'ENSGWIG00000014843', 'mob2a', 'abcc8', 'agbl2', 'cnot2', 'slc41a2a', 'btbd11b', 'mapk12a', 'mtss1la', 'ENSGWIG00000000603', 'ENSGWIG00000000612', 'ENSGWIG00000000620', 'uri1', 'rxylt1', 'srgap1a', 'parp12a', 'si:ch211-1e14.1', 'mical3a', 'ttc38', 'ENSGWIG00000007037', 'tspan9a', 'ENSGWIG00000007046', 'CRACR2A', 'tmtc2b', 'ada2a', 'kcng4a', 'si:dkey-246g23.2', 'zgc:173742', 'mlycd', 'cdh13', 'ENSGWIG00000014979', 'ccnd2a', 'zgc:77158', 'zgc:154077', 'elmo3', 'lrrc29', 'mmp15a', 'nup93', 'ENSGWIG00000019410', 'gnao1a', 'si:dkey-102f14.5', 'si:ch211-216l23.1', 'cbln1', 'ENSGWIG00000024329', 'ENSGWIG00000024331', 'ENSGWIG00000024332', 'syt9b', 'spon1b', 'ENSGWIG00000000690', 'hipk3a', 'tcp11l1', 'ENSGWIG00000000878', 'ENSGWIG00000000937', 'snrkb', 'ano10b', 'tmem231', 'wwp2', 'exoc3l1', 'smpd3', 'sept15', 'ENSGWIG00000009957', 'arntl1a', 'galnt18a', 'slc12a3', 'chka', 'anpepa', 'fam169b', 'znrf1', 'dkk3b', 'mical2b', 'parvaa', 'tead1a', 'siah1', 'aars1', 'psme3ip1', 'ENSGWIG00000013649', 'ENSGWIG00000014008', 'ENSGWIG00000014026', 'hydin', 'slc38a8a', 'nrn1la', 'ENSGWIG00000021422', 'ranbp10', 'ENSGWIG00000000245', 'zcchc14', 'jph3', 'slc7a5', 'ENSGWIG00000006315', 'ENSGWIG00000014709', 'ENSGWIG00000015147', 'ENSGWIG00000015148', 'rfx7b', 'ENSGWIG00000015150', 'ano3', 'gas2a', 'fancf', 'ENSGWIG00000016737', 'ENSGWIG00000016790', 'ENSGWIG00000019466', 'ENSGWIG00000019563', 'syt7a', 'EPS8L2', 'myrf', 'rapsn', 'ddb2', 'tle3a', 'morf4l1', 'skor1b', 'rpl4', 'map2k1', 'megf11', 'arih1', 'ENSGWIG00000014594', 'ENSGWIG00000014618', 'ENSGWIG00000014628', 'usp3', 'fbxl22', 'ENSGWIG00000014684', 'dapk2b', 'cib2', 'lrrc61', 'ENSGWIG00000015833', 'taf3', 'ENSGWIG00000015891', 'ENSGWIG00000016802', 'slc7a10b', 'ENSGWIG00000016868', 'roraa', 'ENSGWIG00000018211', 'si:dkey-37f18.2', 'klf13', 'ENSGWIG00000018217', 'ENSGWIG00000020356', 'ELK3', 'met', 'CAPZA2', 'st7', 'wnt2', 'asz1', 'ENSGWIG00000004755', 'cftr', 'cttnbp2', 'ENSGWIG00000012930', 'ENSGWIG00000012931', 'CERK', 'ENSGWIG00000012960', 'ptprz1a', 'aass', 'CADPS2', 'iqub', 'ASB15', 'lmod2a', 'ENSGWIG00000021538', 'grm8b', 'si:ch211-253p14.2', 'deaf1', 'ENSGWIG00000023124', 'ENSGWIG00000023314', 'ENSGWIG00000023340', 'ENSGWIG00000023344', 'ENSGWIG00000023347', 'ENSGWIG00000024002', 'ENSGWIG00000024005', 'glg1a', 'rfwd3', 'ENSGWIG00000024025', 'spg11', 'ciao2a', 'ENSGWIG00000024147', 'ckap5', 'zgc:92873', 'cry2', 'chtf8', 'zc3h18', 'trhr2', 'zfpm1', 'plcg2', 'ENSGWIG00000000863', 'cotl1', 'pdf', 'ENSGWIG00000001381', 'vps9d1', 'ENSGWIG00000001397', 'slc10a3', 'ENSGWIG00000001406', 'acanb', 'mfge8b', 'ENSGWIG00000004167', 'alkbh3', 'hsd17b12a', 'si:ch211-68a17.7', 'tdg.2', 'pik3c2a', 'ppp1r15b', 'PLEKHA7', 'ENSGWIG00000008312', 'INSC', 'ENSGWIG00000009342', 'ENSGWIG00000009345', 'akip1', 'ireb2', 'crabp1a', 'sin3aa', 'ptpn9a', 'snx33', 'cspg4', 'lingo1a', 'ENSGWIG00000015323', 'ENSGWIG00000000671', 'hmg20a', 'zfp64', 'tshz2', 'cbln4', 'ENSGWIG00000023681', 'ENSGWIG00000023691', 'nol9', 'klhl21', 'syt6a', 'acap3a', 'VWA5B1', 'fam43b', 'zmp:0000001088', 'pink1', 'hspg2', 'ENSGWIG00000011718', 'wu:fi36a10', 'ENSGWIG00000011722', 'atp6ap1b', 'gdi1', 'itih6', 'pkig', 'ttpal', 'hnf4a', 'ENSGWIG00000015146', 'sarnp', 'nckap5l', 'ENSGWIG00000015412', 'pabpc1l', 'ENSGWIG00000019897', 'ENSGWIG00000020117', 'prickle2a', 'QRICH1', 'ENSGWIG00000021077', 'ENSGWIG00000021090', 'TMCC1', 'ENSGWIG00000000454', 'IFT122', 'creld1b', 'ENSGWIG00000000805', 'ENSGWIG00000000820', 'ENSGWIG00000000897', 'BICD2', 'FBLN2', 'nup210', 'IQSEC1', 'zgc:77375', 'raf1a', 'slc6a1a', 'ENSGWIG00000001806', 'ENSGWIG00000001875', 'ENSGWIG00000002092', 'cdh26.1', 'mtg2', 'ENSGWIG00000002239', 'kcnk15', 'ENSGWIG00000005099', 'ENSGWIG00000005108', 'zgc:113278', 'ENSGWIG00000008592', 'RND1', 'adcy6a', 'ENSGWIG00000015712', 'scube3', 'taf11', 'pacsin1a', 'ENSGWIG00000026007', 'ENSGWIG00000026008', 'ENSGWIG00000026009', 'ENSGWIG00000026011', 'ENSGWIG00000026012', 'syt2a', 'frmd4bb', 'tafa1a', 'ENSGWIG00000000800', 'ENSGWIG00000000852', 'zgc:92287', 'ENSGWIG00000001289', 'dnajc11a', 'camta1a', 'dvl1a', 'mxra8b', 'rap1gap', 'YTHDF1', 'bhlhe23', 'slc17a9b', 'asb14b', 'ENSGWIG00000001385', 'GRM7', 'DAG1', 'ENSGWIG00000001402', 'grip2a', 'si:ch73-233m11.2', 'kctd6b', 'nol4la', 'ENSGWIG00000012142', 'si:dkey-70p6.1', 'zgc:171501', 'ENSGWIG00000014730', 'ENSGWIG00000014731', 'iqsec2b', 'kdm5c', 'ENSGWIG00000022074', 'zgc:162200', 'nt5dc2', 'nkain4', 'ENSGWIG00000007994', 'zbtb46', 'uckl1b', 'si:dkey-256k13.2', 'samd10b', 'aar2', 'EMILIN3', 'b4galt5', 'slc9a8', 'opn7b', 'spata2', 'MAFB', 'zgc:158828', 'plcg1', 'GGT7', 'si:ch211-117c9.2', 'ENSGWIG00000017301', 'ddx20', 'kcnd3', 'ENSGWIG00000020050', 'cttnbp2nlb', 'ERC2', 'wnt5a', 'CACNA2D3', 'uba1', 'ngfa', 'ENSGWIG00000022977', 'LGR6', 'ENSGWIG00000023010', 'tead3a', 'PKP1', 'tnni1a', 'vwa1', 'kcnq2a', 'chrna4b', 'ENSGWIG00000007637', 'ctnnbl1', 'ENSGWIG00000008608', 'VDR', 'hdac7a', 'chchd6a', 'ENSGWIG00000019458', 'ENSGWIG00000019680', 'ENSGWIG00000019681', 'si:ch211-236h17.3', 'ENSGWIG00000019692', 'klf15', 'cpne9', 'lhfpl4a', 'plxnb1a', 'efhd2', 'ENSGWIG00000021281', 'ENSGWIG00000021321', 'mfn2', 'si:dkey-16n15.6', 'ENSGWIG00000021499', 'ENSGWIG00000021500', 'ndufb11', 'ENSGWIG00000021528', 'rtel1', 'rcc2', 'si:ch211-215j19.12', 'ENSGWIG00000021593', 'pdyn', 'stk35', 'mthfr', 'clcn6', 'ENSGWIG00000008425', 'igsf21b', 'stk38a', 'dennd2da', 'ENSGWIG00000008663', 'ENSGWIG00000008666', 'pim1', 'ENSGWIG00000008729', 'ENSGWIG00000010481', 'sars1', 'tmem106c', 'tns2a', 'ENSGWIG00000013591', 'ENSGWIG00000013596', 'krt8', 'krt5', 'klhdc8b', 'ENSGWIG00000013947', 'ENSGWIG00000013953', 'ENSGWIG00000014060', 'ENSGWIG00000014758', 'kif5aa', 'actr5', 'ENSGWIG00000014816', 'ENSGWIG00000014821', 'ENSGWIG00000014822', 'ENSGWIG00000014873', 'pofut1', 'plagl2', 'tm9sf4', 'hck', 'rbbp8l', 'ENSGWIG00000016712', 'gata5', 'rps21', 'ENSGWIG00000016895', 'ppdpfb', 'PCMTD2', 'myt1b', 'tbc1d22b', 'plpbp', 'tmed4', 'sephs2', 'ap4b1', 'ENSGWIG00000006620', 'atp2b3b', 'ENSGWIG00000006940', 'skia', 'plch2a', 'nsfl1c', 'bcl2l1', 'adnpb', 'dpm1', 'ENSGWIG00000012885', 'lamb2l', 'usp19', 'si:dkey-32e6.3', 'espn', 'acot7', 'gpr153', 'ENSGWIG00000018943', 'tprg1l', 'ENSGWIG00000019554', 'ENSGWIG00000019724', 'slc66a1', 'nr1h5', 'tafa5l', 'ENSGWIG00000020486', 'mtor', 'agrn', 'ptpn11b', 'RNF223', 'dennd2c', 'tmco4', 'ENSGWIG00000001390', 'ENSGWIG00000001403', 'ENSGWIG00000002459', 'zgc:174906', 'znf740b', 'cbx5', 'ENSGWIG00000002574', 'ENSGWIG00000002575', 'hoxc13a', 'calcoco1a', 'RARG', 'rbms2b', 'lrp1ab', 'ENSGWIG00000003950', 'fmnl3', 'ENSGWIG00000010858', 'ENSGWIG00000016820', 'wnt1', 'ikzf4', 'lmbr1l', 'ENSGWIG00000018289', 'ARHGEF25', 'ENSGWIG00000018645', 'znf385a', 'ENSGWIG00000018660', 'ENSGWIG00000018689', 'tmem9', 'l1cama', 'ENSGWIG00000020200', 'MIRLET7F2', 'ENSGWIG00000020544', 'idh3g', 'avpr2ab', 'ssr4', 'zgc:158263', 'galnt6', 'adcy6b', 'cacnb3b', 'rnd1b', 'stat6', 'scn8aa', 'dip2ba', 'atf7a', 'ppardb', 'mkrn4', 'ENSGWIG00000020583', 'srpk1a', 'cpne5a', 'adora1b', 'mmp9', 'slc16a4', 'slc6a17', 'sort1b', 'sypl2b', 'dnase1l1l', 'ampd2b', 'gpr61', 'amigo1', 'spsb1', 'ENSGWIG00000025320', 'gpr157', 'eno1a', 'rerea', 'samd11', 'plekhn1', 'ENSGWIG00000025661', 'her9', 'ctnnbip1', 'clstn1', 'ube4b', 'kif1b', 'tardbpl', 'ENSGWIG00000026182', 'casz1', 'pex14', 'masp2', 'cort', 'exosc10', 'cntn3a.2', 'ENSGWIG00000000896', 'cntn3a.1', 'ENSGWIG00000001766', 'phactr3a', 'pigt', 'ENSGWIG00000002288', 'ENSGWIG00000002290', 'ENSGWIG00000002362', 'ZNF362', 'ccdc30', 'emp3b', 'rimkla', 'ENSGWIG00000002448', 'KAZN', 'ela2', 'ENSGWIG00000003715', 'si:ch211-263k4.2', 'atp13a2', 'szrd1', 'MMP23B', 'clcnk', 'spen', 'gnb1a', 'ENSGWIG00000007373', 'ENSGWIG00000007375', 'ajap1', 'ENSGWIG00000007390', 'ENSGWIG00000007442', 'dnajc16', 'ENSGWIG00000007514', 'chadla', 'ENSGWIG00000010643', 'ZC3H7B', 'tefb', 'plppr2a', 'tlcd4b', 'tmem205', 'epor', 'znf653', 'tspan35', 'gadd45gip1', 'nfixb', 'rad23aa', 'gng13b', 'traf7', 'ENSGWIG00000006177', 'ENSGWIG00000006179', 'DCAF7', 'smarca4a', 'spc24', 's1pr5a', 'ENSGWIG00000006491', 'ENSGWIG00000006492', 'si:dkeyp-69e1.8', 'hmgxb4a', 'ankrd54', 'foxred2', 'hdac5', 'cyb561', 'pvalb6', 'ENSGWIG00000006695', 'adsl', 'ENSGWIG00000007399', 'ENSGWIG00000007432', 'arhgap17b', 'slc5a11', 'grid2ipa', 'ENSGWIG00000014739', 'e4f1', 'fahd1', 'ENSGWIG00000015181', 'ENSGWIG00000015215', 'RPL3L', 'tbl3', 'sept9a', 'jpt1b', 'sumo2b', 'mrps7', 'slc25a19', 'ENSGWIG00000004771', 'srebf1', 'ENSGWIG00000005723', 'ENSGWIG00000005728', 'rhot1a', 'fam20ca', 'nptx2a', 'baiap2l1b', 'tecpr1b', 'cacng4b', 'cacng1b', 'ENSGWIG00000008129', 'ENSGWIG00000008131', 'rfx1a', 'ENSGWIG00000008299', 'TNPO2', 'ENSGWIG00000008316', 'fbxw9', 'dhps', 'ENSGWIG00000009615', 'ENSGWIG00000009616', 'ENSGWIG00000009617', 'spag9a', 'luc7l3', 'cacna1ia', 'ENSGWIG00000010686', 'lgals2a', 'ENSGWIG00000011777', 'ELFN2', 'ENSGWIG00000011810', 'alkbh7', 'cacna1aa', 'cabp5b', 'ENSGWIG00000013474', 'antkmt', 'slc6a16a', 'pold1', 'spsb3a', 'ypel3', 'atp2a1', 'ENSGWIG00000012850', 'ENSGWIG00000012875', 'pecam1', 'ENSGWIG00000012880', 'nptx1l', 'ENSGWIG00000012882', 'MGAT5B', 'ENSGWIG00000012929', 'ENSGWIG00000013116', 'ENSGWIG00000013144', 'ENSGWIG00000013158', 'grapa', 'ENSGWIG00000013170', 'epn2', 'b9d1', 'cacna1ha', 'grin2aa', 'ENSGWIG00000016217', 'emp2', 'PRKCB', 'ENSGWIG00000019486', 'ENSGWIG00000019488', 'tnrc6a', 'C16orf72', 'usp7', 'clcn7', 'vasnb', 'coro7', 'glis2b', 'tfap4', 'prkar1b', 'pdgfab', 'ENSGWIG00000023060', 'gprc5ba', 'GPR139', 'ENSGWIG00000000351', 'kdelr2a', 'sdr42e2', 'MOSMO', 'ENSGWIG00000001107', 'cacng5b', 'prkcab', 'cep112', 'axin2', 'tex2', 'tbc1d16', 'cbx4', 'ENSGWIG00000007697', 'ENSGWIG00000007698', 'ENSGWIG00000017223', 'sox8b', 'ENSGWIG00000017231', 'usp22', 'sstr2b', 'ENSGWIG00000017298', 'sox9b', 'ENSGWIG00000017310', 'ENSGWIG00000017322', 'notum1a', 'cenpx', 'cyth1a', 'socs3a', 'bahcc1a', 'ENSGWIG00000019003', 'ENSGWIG00000019263', 'chmp6a', 'ttyh2l', 'sdk2a', 'hs3st3b1a', 'si:ch211-114m9.1', 'epn3a', 'hgs', 'mrpl12', 'gcgra', 'samd1a', 'ENSGWIG00000013183', 'dnm2a', 'adgrl1a', 'ENSGWIG00000019888', 'ptger1c', 'ENSGWIG00000020319', 'colgalt1', 'atp13a1', 'pbx4', 'brd4', 'pet100', 'notch3', 'col5a3a', 'ENSGWIG00000021530', 'olfm2a', 'trip10a', 'pde4a', 'ENSGWIG00000010745', 'ENSGWIG00000010751', 'mgrn1b', 'unkl', 'ENSGWIG00000011661', 'baiap3', 'ENSGWIG00000011666', 'hs3st2', 'hs3st4', 'ENSGWIG00000011673', 'kdm8', 'zc3h7a', 'axin1', 'galr2a', 'uncx', 'micall2a', 'ENSGWIG00000012276', 'psmg3', 'ELFN1', 'mrm2', 'ENSGWIG00000013093', 'ENSGWIG00000000295', 'c1ql3b', 'mylk5', 'hsd17b1', 'stk17al', 'atp6v0a1a', 'cavin1a', 'stat5a', 'si:ch211-210g13.5', 'rab5c', 'kat2a', 'dhx58', 'znf385c', 'cnp', 'ACLY', 'STAC2', 'fbxl20', 'cdk12', 'zgc:91968', 'tlk2', 'vat1', 'znf652', 'eftud2', 'GJC1', 'smarce1', 'ENSGWIG00000013282', 'ENSGWIG00000014934', 'sept12', 'ARL4D', 'aoc2', 'aldh3b1', 'dus1l', 'rrn3', 'pdxdc1', 'ENSGWIG00000000845', 'prr12b', 'ENSGWIG00000004467', 'rbfox1', 'hoxb13a', 'hoxb9a', 'hoxb4a', 'hoxb3a', 'igf2bp1', 'ENSGWIG00000004977', 'ENSGWIG00000005423', 'abcc1', 'st8sia6', 'cacnb1', 'lasp1', 'arhgap23a', 'nmt1a', 'cdk21', 'rnf113a', 'ENSGWIG00000022805', 'ENSGWIG00000022820', 'nxph3', 'ngfra', 'ENSGWIG00000022828', 'slc4a1a', 'ENSGWIG00000024234', 'ENSGWIG00000024239', 'ENSGWIG00000024241', 'ENSGWIG00000024243', 'ENSGWIG00000024265', 'ENSGWIG00000024267', 'ENSGWIG00000024268', 'zgc:63569', 'ENSGWIG00000024988', 'kdelr3', 'ENSGWIG00000004796', 'ENSGWIG00000004806', 'si:dkey-167k11.5', 'ENSGWIG00000005702', 'ENSGWIG00000005760', 'EMP1', 'ENSGWIG00000006178', 'GPRC5D', 'ENSGWIG00000006185', 'ENSGWIG00000006352', 'evplb', 'smurf2', 'usp43b', 'ntn1b', 'zgc:171844', 'setd1a', 'hook2', 'junbb', 'rpsa', 'myh9b', 'txn2', 'si:ch211-106h11.3', 'ENSGWIG00000011695', 'pkn1b', 'ENSGWIG00000018069', 'WIZ', 'ptger1b', 'ENSGWIG00000000611', 'si:dkey-46g23.1', 'CARD10', 'cdc42ep1b', 'col1a1a', 'ENSGWIG00000021139', 'ENSGWIG00000021151', 'psmd11a', 'NSF', 'ENSGWIG00000021273', 'si:dkey-16p21.7', 'cpt1cb', 'fthl31', 'carhsp1', 'caskin1', 'ENSGWIG00000023193', 'ENSGWIG00000023195', 'ENSGWIG00000023196', 'ENSGWIG00000023199', 'sco1', 'ENSGWIG00000023738', 'gas7a', 'gsg1l2a', 'ENSGWIG00000023973', 'ENSGWIG00000023976', 'si:ch73-281f12.4', 'grap2a', 'fam83fa', 'CACNG2', 'ifi35', 'hsd17b14', 'slc1a9', 'ENSGWIG00000004809', 'zgc:195001', 'mybpc2a', 'ENSGWIG00000010030', 'syt3', 'ENSGWIG00000010049', 'ENSGWIG00000010050', 'ENSGWIG00000010051', 'paqr4a', 'elob', 'si:ch211-262e15.1', 'ENSGWIG00000011146', 'grin2ca', 'rbfox3b', 'ENSGWIG00000011970', 'nog1', 'pctp', 'grid2', 'atoh1a', 'me2', 'mier3b', 'plk2b', 'pde4d', 'fbxl17', 'EFNA5', 'ENSGWIG00000009206', 'gadd45ga', 'ghra', 'oxct1a', 'c7a', 'HTR1A', 'ipo11', 'ptch1', 'dock8', 'COMMD10', 'sema6a', 'DMXL1', 'si:ch211-193c2.2', 'sh2b3', 'atxn2', 'rnf165b', 'mrps27', 'lzts3b', 'rasgrf2b', 'msh3', 'dusp2', 'adra2b', 'gpat2', 'KCNIP3', 'slc23a2', 'nsd3', 'ENSGWIG00000009179', 'plpp5', 'cdc45', 'cldn5a', 'sept5a', 'plat', 'si:dkey-178e17.3', 'pi4kab', 'hic2', 'ENSGWIG00000018690', 'ENSGWIG00000018692', 'myl7', 'agpat9l', 'adamts12', 'ENSGWIG00000019648', 'ndufs4', 'arl15a', 'ENSGWIG00000019671', 'lipg', 'slc20a2', 'epgn', 'CCNO', 'mccc2', 'si:dkey-84j12.1', 'si:dkey-174n20.1', 'kremen1', 'ENSGWIG00000004831', 'xbp1', 'ENSGWIG00000004888', 'hspb8', 'suds3', 'taok3a', 'wsb2', 'ksr2', 'nos1', 'nipsnap1', 'fbxw8', 'tesca', 'nf2a', 'ENSGWIG00000016856', 'ENSGWIG00000016882', 'prodhb', 'rtn4r', 'si:ch211-207k7.4', 'si:dkey-21e5.1', 'camsap1b', 'notch1b', 'ENSGWIG00000022325', 'ptgs1', 'si:ch211-236k19.2', 'scarb2a', 'ENSGWIG00000023574', 'ENSGWIG00000023590', 'malt1', 'ENSGWIG00000024285', 'ENSGWIG00000024397', 'ENSGWIG00000024398', 'aacs', 'gak', 'ENSGWIG00000024562', 'wdfy3', 'rusc2', 'ENSGWIG00000025080', 'unc13ba', 'rnf215', 'slc39a14', 'stambpb', 'ENSGWIG00000002055', 'grsf1', 'rufy3', 'NPFFR2', 'adamts3', 'ENSGWIG00000018591', 'dmrt2a', 'kank1a', 'purbb', 'tango2', 'ARVCF', 'ENSGWIG00000021656', 'sema4c', 'ENSGWIG00000021665', 'kat6a', 'tet3', 'hk2', 'arhgap25', 'ENSGWIG00000023152', 'znf703', 'ENSGWIG00000023165', 'ENSGWIG00000023168', 'pnpla7b', 'zgc:64051', 'traf2a', 'tnc', 'nsmfb', 'htra4', 'ENSGWIG00000023560', 'unc5db', 'cyb561a3a', 'loxl2b', 'zmat4b', 'ppp3cca', 'vps37b', 'si:ch211-48m9.1', 'zgc:110329', 'tacr1a', 'ncaph', 'citb', 'ENSGWIG00000004881', 'si:dkey-91m11.5', 'ift81', 'cux2b', 'MTMR3', 'niban2b', 'stxbp1b', 'noxa1', 'man1b1b', 'ENSGWIG00000022080', 'ENSGWIG00000022081', 'JMY', 'pip5k1bb', 'cert1a', 'polk', 'sv2ca', 'iqgap2', 'f2rl2', 'ENSGWIG00000026550', 'si:ch211-130m23.3', 'ENSGWIG00000026563', 'vcana', 'hapln1a', 'ENSGWIG00000026581', 'ENSGWIG00000026583', 'edil3a', 'cox7c', 'rasa1a', 'ccnh', 'mef2cb', 'si:dkey-40c11.2', 'wdr54', 'nol6', 'si:dkey-245n4.2', 'ENSGWIG00000026710', 'ENSGWIG00000026711', 'fam172a', 'slf1', 'mctp1a', 'ENSGWIG00000026726', 'arrdc3a', 'ENSGWIG00000001466', 'adgrv1', 'mblac2', 'si:dkey-40c11.1', 'lzts1', 'gfra2a', 'ela3l', 'trpm3', 'gda', 'entpd4', 'stc1', 'ido1', 'PEBP4', 'egr3', 'bin3', 'ENSGWIG00000021457', 'gfpt1', 'bmp1a', 'adrb3a', 'GPAT4', 'zgc:77112', 'dbnlb', 'tcf7l1a', 'ENSGWIG00000025201', 'mn1b', 'TTC28', 'SDS', 'ENSGWIG00000025254', 'ENSGWIG00000025288', 'rph3ab', 'DPYSL2', 'adra1aa', 'si:dkey-229d2.4', 'bag4', 'ENSGWIG00000005779', 'si:ch1073-459j12.1', 'ENSGWIG00000006448', 'rnf34a', 'orai1a', 'aplnra', 'tbc1d10aa', 'sf3a1', 'RASAL1', 'DTX1', 'unm_hu7912', 'gapvd1', 'ENSGWIG00000007722', 'ENSGWIG00000007750', 'ENSGWIG00000008068', 'ENSGWIG00000008071', 'lmx1bb', 'zbtb34', 'ENSGWIG00000008117', 'asb6', 'si:dkeyp-14d3.1', 'slc15a4', 'glt1d1', 'TMEM132D', 'fzd10', 'rimbp2', 'ENSGWIG00000009029', 'adgrd1', 'ep400', 'U6', 'ncor2', 'rflna', 'ccdc92', 'fgfr1a', 'npy8br', 'klhl22', 'cabin1', 'neflb', 'dock5', 'lrrtm4l2', 'ENSGWIG00000020936', 'ENSGWIG00000001467', 'tmem150aa', 'rnf181', 'pxnb', 'ENSGWIG00000001692', 'ENSGWIG00000001710', 'si:ch73-138n13.1', 'unc119.1', 'msi1', 'mlec', 'lrrc75ba', 'si:dkey-222h21.12', 'golga3', 'trmt1l', 'fbrsl1', 'ENSGWIG00000008342', 'ENSGWIG00000008345', 'ENSGWIG00000008348', 'galnt9', 'ENSGWIG00000008364', 'atad1a', 'ENSGWIG00000008369', 'htr7c', 'tnksb', 'idua', 'ENSGWIG00000009063', 'zdhhc8b', 'ENSGWIG00000009076', 'ankle2', 'ENSGWIG00000009133', 'ENSGWIG00000009207', 'ENSGWIG00000009210', 'ENSGWIG00000009211', 'med27', 'ENSGWIG00000009228', 'ENSGWIG00000009240', 'ENSGWIG00000009241', 'ENSGWIG00000009456', 'ENSGWIG00000009536', 'ENSGWIG00000009537', 'ENSGWIG00000004849', 'ENSGWIG00000004942', 'ENSGWIG00000005644', 'ENSGWIG00000011902', 'mak16', 'ENSGWIG00000011937', 'ENSGWIG00000011951', 'srp19', 'oclnb', 'rpl35', 'CRB2', 'lhx2b', 'ENSGWIG00000001573', 'ENSGWIG00000001585', 'nr5a1a', 'nr6a1a', 'ENSGWIG00000001676', 'olfml2a', 'ENSGWIG00000001706', 'tfip11', 'thoc5', 'ENSGWIG00000001870', 'fancg', 'vcp', 'ENSGWIG00000002384', 'vps13a', 'ENSGWIG00000002484', 'ENSGWIG00000002491', 'ENSGWIG00000002496', 'pcsk5b', 'ubap2b', 'ENSGWIG00000007739', 'mlxip', 'ENSGWIG00000007872', 'clip1a', 'kntc1', 'hip1rb', 'ENSGWIG00000009097', 'trafd1', 'smtnb', 'inpp5jb', 'si:ch211-283h6.4', 'ebf2', 'ank1a', 'tcn2', 'si:ch211-106a19.1', 'rasl10a', 'rhbdd3', 'atp6v0a2b', 'si:ch211-225b11.4', 'ENSGWIG00000007802', 'si:ch211-191d15.2', 'coro1ca', 'ssh1a', 'aldh3b2', 'ENSGWIG00000020591', 'foxn4', 'ENSGWIG00000020594', 'p2rx2', 'lztr1', 'ENSGWIG00000021134', 'ENSGWIG00000021135', 'ENSGWIG00000021136', 'wscd2', 'limk2', 'si:ch73-199g24.2', 'homer1b', 'cmya5', 'setbp1', 'slc14a2', 'erap2', 'rgmb', 'ENSGWIG00000003349', 'ENSGWIG00000003404', 'ENSGWIG00000003417', 'si:ch73-72b7.1', 'macrod1', 'ENSGWIG00000006877', 'rnaseh2c', 'ENSGWIG00000008279', 'si:ch73-45o6.2', 'tgfb5', 'ENSGWIG00000008520', 'ENSGWIG00000008545', 'mbnl3', 'ENSGWIG00000008758', 'zgc:92907', 'ubl3b', 'prss23', 'atl3', 'si:ch211-145o7.3', 'npas4a', 'mrpl11', 'slc25a43', 'lnx2b', 'cdx4', 'kdrl', 'tex11', 'dlg3', 'gdpd2', 'pdzd11', 'stard14', 'fgl1', 'ENSGWIG00000017318', 'nkx1.2lb', 'uvssa', 'sil1', 'csnk1a1', 'ENSGWIG00000017359', 'ENSGWIG00000017369', 'ENSGWIG00000017375', 'ENSGWIG00000017386', 'ENSGWIG00000017392', 'zgc:153146', 'ENSGWIG00000017397', 'gfra3', 'dnd1', 'zdhhc24', 'SLC16A2', 'ENSGWIG00000018323', 'pmt', 'mid2', 'si:ch211-220e11.3', 'pcdh11', 'ENSGWIG00000018718', 'ENSGWIG00000018719', 'klhl4', 'AHNAK', 'CHRNA6', 'chrnb5b', 'ENSGWIG00000019380', 'slc43a3b', 'ENSGWIG00000023085', 'ENSGWIG00000004941', 'CTNND1', 'ENSGWIG00000008079', 'wfs1b', 'ENSGWIG00000009494', 'ENSGWIG00000009500', 'TLX3', 'hrh2a', 'DOCK2', 'ENSGWIG00000011407', 'ENSGWIG00000011586', 'rab33ba', 'pcdh10b', 'ENSGWIG00000020435', 'ENSGWIG00000020436', 'pcdh18b', 'slc7a11', 'nocta', 'ENSGWIG00000020545', 'trpc7b', 'ENSGWIG00000020590', 'DIAPH1', 'ENSGWIG00000005093', 'si:ch73-233f7.1', 'fbxw11a', 'etf1a', 'ENSGWIG00000005722', 'spock1', 'bicc2', 'APBB2', 'CHRNA9', 'klb', 'ENSGWIG00000001548', 'prpf19', 'rnf20', 'ENSGWIG00000003761', 'ndst1a', 'rbm22', 'ENSGWIG00000005562', 'rgs14a', 'ENSGWIG00000005591', 'adra2da', 'si:ch211-199b20.3', 'cnot6b', 'ENSGWIG00000005765', 'ENSGWIG00000012703', 'ENSGWIG00000012705', 'RASGEF1C', 'ENSGWIG00000012782', 'ctbp1', 'spon2a', 'REEP2', 'egr1', 'ENSGWIG00000016652', 'ENSGWIG00000016654', 'adra1bb', 'ccnjl', 'atp7a', 'nr3c1', 'kctd16a', 'sh3rf2', 'fgf16', 'zdhhc15b', 'abcb7', 'nexmifb', 'irg1l', 'ENSGWIG00000004997', 'frmpd3', 'prps1b', 'ENSGWIG00000005310', 'rhogd', 'ENSGWIG00000005324', 'ogt.1', 'nlgn3a', 'ENSGWIG00000008714', 'gria3b', 'eda', 'cx31.7', 'dusp1', 'ENSGWIG00000014476', 'unc5a', 'hnrnph1', 'hbegfa', 'rufy1', 'ENSGWIG00000018872', 'n4bp3', 'ENSGWIG00000018891', 'tnip1', 'gpx3', 'ENSGWIG00000024467', 'tspan17', 'mchr2', 'ctnna1', 'sncb', 'ENSGWIG00000024713', 'cpeb4a', 'stc2a', 'nkx2.5', 'thg1l', 'clint1a', 'ebf3a', 'il12ba', 'adrb2a', 'sh3tc2', 'ablim3', 'afap1l1a', 'foxi3b', 'tmsb2', 'itk', 'cyfip2', 'ENSGWIG00000007607', 'si:dkey-34l15.2', 'eif4ebp3', 'acsl4a', 'psmd10', 'stag2b', 'efnb1', 'arhgef9b', 'tenm1', 'hdx', 'sh3bgrl', 'ENSGWIG00000025629', 'ENSGWIG00000025630', 'p2ry10', 'lpar4', 'ENSGWIG00000025639', 'ENSGWIG00000025640', 'ENSGWIG00000025648', 'mtmr7a', 'ints6l', 'gpc4', 'brd8', 'ppp3cb', 'ube2d2', 'ENSGWIG00000026186', 'psd2', 'purab', 'nrg2b', 'si:dkey-160o24.3', 'ctsf', 'ENSGWIG00000001641', 'gabra4', 'si:ch73-49o8.1', 'fam199x', 'LONRF3', 'ENSGWIG00000002001', 'rab33a', 'il1rapl2', 'ENSGWIG00000007323', 'ENSGWIG00000007330', 'inppl1b', 'P2RY4', 'ENSGWIG00000008159', 'atp11c', 'FGF13', 'zic6', 'map7d3', 'taf7', 'ENSGWIG00000018469', 'drp2', 'cenpi', 'cstf2', 'bcorl1', 'ENSGWIG00000019910', 'mid1ip1l', 'sytl4', 'ENSGWIG00000021044', 'ENSGWIG00000021045', 'ENSGWIG00000021048', 'xpnpep2', 'zdhhc9', 'sept6', 'tnmd', 'pcdh19', 'diaph2', 'glra4a', 'zgc:101583', 'med12', 'si:ch211-221f10.2', 'enox2', 'SLC4A11', 'gfra4b', 'shisa3', 'atp8a1', 'ENSGWIG00000021625', 'HS3ST1', 'ENSGWIG00000021627', 'ENSGWIG00000021923', 'slit3', 'fgfrl1a', 'rnf212', 'zgc:136929', 'fam13b', 'cxcl14', 'ENSGWIG00000022880', 'lingo2a', 'ENSGWIG00000022883', 'ENSGWIG00000022884', 'eif4ea', 'zgc:77938', 'egf', 'tdo2a', 'si:dkey-237h12.3', 'ENSGWIG00000023620', 'aga', 'neil3', 'pdgfc', 'GLRB', 'gria2b', 'gask1b', 'tmem144b', 'rxfp1', 'etfdh', 'rapgef2', 'ENSGWIG00000013525', 'fstl5', 'qdpra', 'ldb2a', 'tapt1a', 'prom1a', 'anxa5a', 'ccna2', 'limch1b', 'ENSGWIG00000015331', 'ENSGWIG00000015335', 'nanos1', 'ENSGWIG00000015338', 'ENSGWIG00000015339', 'lbx2', 'npm1b', 'tlx2', 'adad1', 'SPATA5', 'ANKRD50', 'ENSGWIG00000015470', 'ENSGWIG00000015472', 'U3', 'sec24b', 'pde6a', 'ppargc1b', 'ENSGWIG00000020637', 'arap3', 'si:rp71-1d10.5', 'ENSGWIG00000002676', 'ENSGWIG00000002680', 'pcdh1b', 'ENSGWIG00000002687', 'nfatc1', 'atp9b', 'SCN1B', 'si:dkey-211g8.9', 'ENSGWIG00000023300', 'hivep1', 'nedd9', 'oscp1a', 'psmb2', 'ENSGWIG00000012167', 'ENSGWIG00000012867', 'prdm1b', 'ENSGWIG00000013478', 'ext1c', 'sdc2', 'angpt1', 'SAMD12', 'ext1b', 'slc4a7', 'eomesa', 'ENSGWIG00000013668', 'STT3B', 'OSBPL10', 'tmem42b', 'ENSGWIG00000014787', 'nrm', 'ENSGWIG00000014977', 'nelfe', 'ehmt2', 'slc44a4', 'clk2a', 'ENSGWIG00000001625', 'scamp3', 'ash1l', 'si:ch211-39a7.1', 'efna3a', 'cks1b', 'pbxip1a', 'SHE', 'cgnb', 'ENSGWIG00000003846', 'ubap2l', 'necab1', 'tmem64', 'ENSGWIG00000004604', 'zgc:63863', 'DGAT1', 'dennd3a', 'rplp1', 'ENSGWIG00000011277', 'trps1', 'csmd3a', 'ENSGWIG00000011602', 'ENSGWIG00000011649', 'SV2A', 'znf706', 'pard6gb', 'jarid2b', 'mylipa', 'atxn1a', 'fam8a1a', 'pag1', 'ENSGWIG00000001971', 'hey1', 'si:dkeyp-120h9.1', 'ripor2', 'MTERF1', 'AKAP9', 'krit1', 'tmem79a', 'ENSGWIG00000004802', 'ENSGWIG00000004804', 'gatad1', 'uqcrb', 'rbm24a', 'grinaa', 'ankib1a', 'ggcta', 'adnp2b', 'lratd2', 'trib1', 'ENSGWIG00000005463', 'fam92a1', 'triqk', 'runx1t1', 'otud6b', 'rab5aa', 'ZNF385D', 'TINAGL1', 'cfap300', 'KCNQ4', 'rprd1a', 'fkbp9', 'ENSGWIG00000003752', 'clasp2', 'pdcd6ip', 'dclk3', 'fabp3', 'snrnp40', 'pum1', 'ENSGWIG00000015477', 'ahdc1', 'elmo1', 'rp9', 'poc1bl', 'galnt12', 'gabbr2', 'ctnnal1', 'ENSGWIG00000022290', 'ENSGWIG00000023285', 'itgb8', 'ENSGWIG00000023295', 'twistnb', 'ferd3l', 'cdk6', 'hepacam2', 'vps50', 'calcr', 'zgc:85777', 'col1a2', 'casd1', 'sgce', 'DYNC1I1', 'sem1', 'igf2bp3', 'ENSGWIG00000007940', 'ENSGWIG00000007946', 'wipf3', 'ENSGWIG00000007959', 'ENSGWIG00000007967', 'nfe2l3', 'snx10a', 'hoxa1a', 'hoxa4a', 'hibadha', 'jazf1a', 'tril', 'nup42', 'trappc3', 'col8a2', 'ptp4a2b', 'ENSGWIG00000010831', 'ENSGWIG00000010833', 'ENSGWIG00000010834', 'adgrb2', 'col9a2', 'angpt2b', 'ENSGWIG00000011118', 'csmd2', 'ENSGWIG00000011627', 'cx39.4', 'ENSGWIG00000011646', 'fhod3a', 'tert', 'yars1', 'sh3bp5a', 'si:dkey-166k12.1', 'zgc:103438', 'trim71', 'ENSGWIG00000005157', 'ENSGWIG00000005174', 'rpl14', 'ENSGWIG00000008920', 'srd5a1', 'nsun2', 'ENSGWIG00000013359', 'ube2ql1', 'med10', 'ENSGWIG00000013365', 'irx4b', 'lpcat1', 'nrsn1l', 'sox4a', 'ENSGWIG00000013393', 'ENSGWIG00000013395', 'ENSGWIG00000013396', 'ldlrap1a', 'hdac1', 'eif3i', 'stmn1b', 'ENSGWIG00000013991', 'phactr4a', 'ENSGWIG00000014093', 'tent5bb', 'ENSGWIG00000014096', 'zdhhc18b', 'tbxta', 'arid1ab', 'ENSGWIG00000014131', 'tpbga', 'tmem222b', 'paqr7a', 'si:dkey-218f9.10', 'kcnk9', 'ENSGWIG00000014170', 'ENSGWIG00000014173', 'FAM135B', 'ENSGWIG00000014846', 'ENSGWIG00000014847', 'KHDRBS3', 'ENSGWIG00000001699', 'oxsm', 'ENSGWIG00000004384', 'thsd7aa', 'nxph1', 'phf14', 'glcci1a', 'umad1', 'rpa3', 'COL28A1', 'sdhaf3', 'tac1', 'ENSGWIG00000008034', 'ENSGWIG00000008050', 'top2b', 'ENSGWIG00000009806', 'ENSGWIG00000009809', 'ENSGWIG00000009812', 'ENSGWIG00000009837', 'thrb', 'ENSGWIG00000010310', 'lin28a', 'ENSGWIG00000010340', 'ENSGWIG00000010341', 'cited4a', 'ctps1a', 'ENSGWIG00000010347', 'rab42a', 'gmeb1', 'ptprua', 'ENSGWIG00000010394', 'ENSGWIG00000010395', 'ENSGWIG00000010397', 'ENSGWIG00000007878', 'sec31a', 'plac8.1', 'adamts6', 'cert1b', 'ENSGWIG00000008165', 'si:dkey-163m14.2', 'crhbp', 'pde8b', 'ap3b1a', 'lhfpl2a', 'dmgdh', 'mrps24', 'ENSGWIG00000013517', 'ENSGWIG00000013535', 'hmgcs1', 'ENSGWIG00000014534', 'odf2a', 'ENSGWIG00000014597', 'pkn3', 'zdhhc12b', 'lmbrd2b', 'mfhas1', 'nkx6.1', 'ENSGWIG00000005893', 'ENSGWIG00000007813', 'apc', 'ENSGWIG00000008039', 'ENSGWIG00000008106', 'whrna', 'ENSGWIG00000009960', 'ENSGWIG00000010003', 'ENSGWIG00000018599', 'ENSGWIG00000007243', 'hapln1b', 'grk3', 'ENSGWIG00000012798', 'scarb1', 'grk5', 'ddx56', 'ENSGWIG00000016398', 'gas1b', 'ENSGWIG00000016778', 'ENSGWIG00000016779', 'unc5da', 'erbin', 'MAST4', 'pik3r1', 'palm2akap2', 'slc46a2', 'ENSGWIG00000021318', 'paqr3b', 'ENSGWIG00000021414', 'ntmt1', 'ENSGWIG00000021747', 'ptpa', 'wdr91', 'ENSGWIG00000005293', 'LYZ', 'ENSGWIG00000005321', 'dcc', 'ENSGWIG00000006518', 'vldlr', 'atp8b1', 'TCF4', 'ENSGWIG00000021613', 'rab27b', 'nars1', 'st8sia3', 'WDR7', 'ENSGWIG00000023661', 'wu:fj64h06', 'ENSGWIG00000023672', 'shroom3', 'pam', 'syk', 'vav2', 'ENSGWIG00000024657', 'thnsl2', 'sptlc1', 'ror2', 'zer1', 'ENSGWIG00000024850', 'fam102ab', 'fpgs', 'ENSGWIG00000024875', 'ENSGWIG00000024876', 'si:dkey-28o19.1', 'ENSGWIG00000024879', 'prkab1a', 'phf24', 'pigo', 'stoml2', 'ENSGWIG00000024909', 'ENSGWIG00000025000', 'ENSGWIG00000002300', 'lpar1', 'ENSGWIG00000002313', 'ENSGWIG00000002314', 'ccdc97', 'ENSGWIG00000002371', 'ENSGWIG00000002456', 'nsmfa', 'si:ch73-62l21.1', 'ENSGWIG00000006730', 'bcr', 'noc4l', 'si:ch73-287m6.1', 'ENSGWIG00000015593', 'si:dkey-112m2.1', 'arpc3', 'ENSGWIG00000019006', 'sfswap', 'mmp17a', 'ulk1b', 'crkl', 'ENSGWIG00000019037', 'ENSGWIG00000019213', 'ENSGWIG00000019482', 'ENSGWIG00000019484', 'cntfr', 'sh2d3ca', 'ENSGWIG00000006256', 'ambp', 'strbp', 'rc3h2', 'rbm18', 'ndufa8', 'si:ch211-243g18.2', 'pde6b', 'nnt', 'fgf10a', 'ENSGWIG00000018566', 'rai14', 'scarb2c', 'ENSGWIG00000018587', 'slc5a1', 'GNAZ', 'sppl3', 'drg1', 'stard7', 'ENSGWIG00000019598', 'gmcl1', 'smtna', 'pptc7b', 'anxa4', 'brf2', 'cabp1b', 'lrrc75bb', 'orai1b', 'rhof', 'dgcr2', 'cabp7b', 'ENSGWIG00000001837', 'uqcr10', 'prodha', 'ENSGWIG00000004594', 'ENSGWIG00000005621', 'ENSGWIG00000005635', 'gdnfa', 'ENSGWIG00000007908', 'bmpr1bb', 'ENSGWIG00000008238', 'glis3', 'nup155', 'mapk10', 'arhgap24', 'ENSGWIG00000013015', 'hcn1', 'ccbe1', 'dym', 'smad7', 'ctif', 'zbtb7c', 'SHC3', 'ENSGWIG00000013929', 'ENSGWIG00000014299', 'ENSGWIG00000014301', 'ZNF462', 'TMEM38B', 'fam78ab', 'NACC2', 'TMEM250', 'ENSGWIG00000014450', 'brinp1', 'ENSGWIG00000014453', 'ENSGWIG00000014454', 'ENSGWIG00000014455', 'ASTN2', 'trim32', 'pappaa', 'rabepk', 'golga2', 'dnm1a', 'ciz1a', 'ASS1', 'fubp3', 'fbxw5', 'ENSGWIG00000002640', 'pax8', 'ENSGWIG00000002691', 'lman2la', 'MED13L', 'ENSGWIG00000003630', 'tbx3a', 'tbx5a', 'ddx54', 'abl1', 'ERMP1', 'UHRF2', 'ENSGWIG00000010798', 'ENSGWIG00000010811', 'ENSGWIG00000010824', 'btc', 'ENSGWIG00000010868', 'pstpip2', 'itga1', 'hnrpkl', 'pcsk5a', 'ENSGWIG00000012290', 'zgc:73226', 'zgc:77838', 'ENSGWIG00000019629', 'camsap1a', 'ENSGWIG00000001863', 'si:ch73-29c22.2', 'nup188', 'dnlz', 'rapgef1b', 'ntng2b', 'surf1', 'prkaa1', 'aif1l', 'lamc3', 'fibcd1', 'zgc:63972', 'dipk1b', 'olfm1b', 'agpat2', 'ENSGWIG00000015539', 'tyw1', 'caln1', 'ccdc9', 'zc3h4', 'zgc:154093', 'foxo1a', 'cog6', 'LHFPL6', 'prcp', 'FAM181B', 'tenm4', 'gab2', 'ENSGWIG00000012394', 'arhgap42b', 'cntn5', 'ENSGWIG00000019538', 'FBXO40', 'nectin1b', 'ENSGWIG00000019545', 'treh', 'ddx6', 'ENSGWIG00000019583', 'ENSGWIG00000019586', 'cdon', 'bckdha', 'bicra', 'nup88', 'psmd8', 'aplp2', 'tmem91', 'gbe1b', 'cadm2b', 'usp16', 'zgc:110333', 'tiam1b', 'ENSGWIG00000009469', 'dchs1a', 'trim3a', 'postnb', 'dclk1b', 'nbeab', 'stard13b', 'ENSGWIG00000011686', 'b3glcta', 'ENSGWIG00000013139', 'si:dkey-24p1.6', 'robo2', 'robo1', 'ENSGWIG00000018244', 'rrp8', 'ENSGWIG00000018301', 'ENSGWIG00000018409', 'ENSGWIG00000018423', 'camkk1b', 'ENSGWIG00000018462', 'traf4a', 'tada2a', 'synrg', 'ENSGWIG00000005980', 'ENSGWIG00000005983', 'aldoca', 'si:dkey-243i1.1', 'wdr81', 'rpa1', 'rtn4rl1b', 'dph1', 'ENSGWIG00000007678', 'hic1', 'taok1a', 'si:ch1073-111c8.3', 'lrfn1', 'zgc:162339', 'dixdc1a', 'tmprss5', 'zbtb16b', 'ncam1b', 'cadm1b', 'ENSGWIG00000014952', 'ENSGWIG00000014992', 'vps26b', 'sytl2b', 'picalmb', 'zpld1b', 'alcamb', 'dyrk1ab', 'zgc:112083', 'med13b', 'plch1', 'ENSGWIG00000023617', 'ENSGWIG00000024473', 'MME', 'ARHGEF26', 'ENSGWIG00000025040', 'RAP2B', 'p2ry1', 'MBNL1', 'SUCNR1', 'ENSGWIG00000002620', 'ift80', 'smc4', 'sptssb', 'ENSGWIG00000013414', 'clasrp', 'znf296', 'npas1', 'arhgap35a', 'ap2s1', 'rab4b', 'rhoub', 'rtn2a', 'ENSGWIG00000013664', 'ptgir', 'ENSGWIG00000013678', 'ENSGWIG00000013994', 'gemin7', 'ENSGWIG00000013998', 'ppp1r37', 'six5', 'sipa1l3', 'ENSGWIG00000015850', 'dock10', 'ENSGWIG00000005954', 'ENSGWIG00000005986', 'sik2b', 'laynb', 'zgc:162608', 'ENSGWIG00000006118', 'ankk1', 'drd2a', 'scn3b', 'ENSGWIG00000006136', 'ENSGWIG00000006146', 'ENSGWIG00000006156', 'NRGN', 'pde9al', 'tlcd5a', 'ENSGWIG00000006207', 'cbl', 'mcamb', 'kmt2a', 'limk1a', 'ENSGWIG00000013866', 'mnta', 'PAFAH1B1', 'cluha', 'foxn1', 'unc119a', 'aipl1', 'ENSGWIG00000020227', 'abhd15a', 'pde3a', 'vps53', 'zgc:172282', 'ENSGWIG00000020943', 'zgc:162730', 'zgc:114130', 'bloc1s3', 'si:ch211-136a13.1', 'ENSGWIG00000020957', 'ENSGWIG00000020980', 'relt', 'ARHGEF17', 'fam168a', 'cwf19l2', 'GUCY1A2', 'gria4a', 'pdgfd', 'ENSGWIG00000012095', 'DYNC2H1', 'fzd4', 'ctsc', 'grm5b', 'phox2a', 'CLPB', 'stard10', 'ARAP1', 'PDE2A', 'ENSGWIG00000014749', 'porb', 'zgc:103681', 'ypel2a', 'hip1', 'bcl7bb', 'baz1b', 'fzd9b', 'RASA2', 'trip12', 'dner', 'pid1', 'sphkap', 'agfg1a', 'ENSGWIG00000024586', 'zgc:158328', 'mecom', 'scn4ba', 'ENSGWIG00000024624', 'ENSGWIG00000024631', 'slitrk3a', 'ENSGWIG00000024768', 'ENSGWIG00000024803', 'ENSGWIG00000024950', 'ENSGWIG00000024953', 'pdcd10a', 'ENSGWIG00000024958', 'veph1', 'ccnl1a', 'ENSGWIG00000024979', 'TIPARP', 'ssr3', 'KCNAB1', 'rps6kb1b', 'atm', 'dscamb', 'ENSGWIG00000002793', 'ENSGWIG00000002797', 'sh3bgr', 'mipep', 'tnfrsf19', 'sacs', 'sgcg', 'ENSGWIG00000003905', 'vaspb', 'gramd1bb', 'ENSGWIG00000007701', 'ENSGWIG00000007703', 'barx2', 'KIRREL3', 'ENSGWIG00000009512', 'smtla', 'ets1', 'ENSGWIG00000009689', 'kcnj5', 'st3gal4', 'srpra', 'ENSGWIG00000011703', 'b3gat1a', 'siae', 'tsku', 'serpinh1b', 'gdpd5b', 'hspa13', 'p2rx8', 'samsn1a', 'ENSGWIG00000012676', 'dhx40', 'rap1gap2a', 'blmh', 'ENSGWIG00000016677', 'ankrd13b', 'git1', 'igsf11', 'ENSGWIG00000001514', 'ENSGWIG00000001522', 'ENSGWIG00000001533', 'ENSGWIG00000001541', 'ENSGWIG00000001547', 'ENSGWIG00000001553', 'ENSGWIG00000001657', 'ENSGWIG00000001811', 'ENSGWIG00000001815', 'sidt2', 'pcsk7', 'bace1', 'ENSGWIG00000005368', 'pnkp', 'ENSGWIG00000005461', 'dpf1', 'numbl', 'si:dkey-114g7.4', 'slc5a2', 'si:dkey-260j18.2', 'psma6l', 'nop53', 'ENSGWIG00000007489', 'si:ch1073-228b5.2', 'sirt2', 'ITM2C', 'pik3cb', 'ENSGWIG00000012731', 'mrps22', 'COPB2', 'lpar5b', 'si:ch211-276c2.2', 'ENSGWIG00000013030', 'ENSGWIG00000013036', 'ENSGWIG00000013043', 'ENSGWIG00000013050', 'ENSGWIG00000013056', 'ksr1b', 'wsb1', 'cuedc1b', 'akap1b', 'U2', 'ENSGWIG00000013315', 'ENSGWIG00000013317', 'SMCO4', 'gkup', 'ENSGWIG00000013366', 'ENSGWIG00000013373', 'ENSGWIG00000013378', 'dhrs13b.1', 'ngef', 'taar10b', 'phf12b', 'sez6b', 'pipox', 'myo18ab', 'ENSGWIG00000008740', 'lhx1a', 'ENSGWIG00000008750', 'smtnl', 'tmem199', 'si:dkey-151p21.7', 'ywhae1', 'myo1cb', 'slc43a2a', 'ENSGWIG00000010314', 'ENSGWIG00000011097', 'wdfy1', 'psmd2', 'ENSGWIG00000012823', 'pcyt1bb', 'ENSGWIG00000010343', 'ENSGWIG00000010504', 'tmem45b', 'cxxc5a', 'zgc:100829', 'nrg2a', 'pygma', 'ENSGWIG00000019018', 'pcnx3', 'alg9', 'chek1', 'ENSGWIG00000001982', 'birc2', 'heatr6', 'pigs', 'cadm2a', 'paxbp1', 'ENSGWIG00000006542', 'mtnr1bb', 'ENSGWIG00000006630', 'grm5a', 'grik1a', 'ENSGWIG00000017020', 'synj1', 'si:dkey-5g14.1', 'ENSGWIG00000018279', 'arfip2b', 'trim3b', 'aifm5', 'ENSGWIG00000018729', 'sept4a', 'ENSGWIG00000019559', 'mrps17', 'ENSGWIG00000002115', 'caln2', 'nsrp1', 'ssh2b', 'ovca2', 'rtn4rl1a', 'RPA1', 'smyd4', 'ENSGWIG00000006361', 'ubl3a', 'hmgb1a', 'ENSGWIG00000006383', 'fryb', 'pds5b', 'kl', 'ENSGWIG00000012525', 'nbeaa', 'sparta', 'exosc8', 'supt20', 'ugt5d1', 'trpc4a', 'ufm1', 'ENSGWIG00000019179', 'sh3pxd2b', 'GABRB2', 'gabra6b', 'gabra1', 'gabrg2', 'ccni2', 'sept8a', 'aff4', 'ndfip1l', 'ARHGAP26', 'kctd16b', 'GRXCR2', 'lars1b', 'drd1b', 'ENSGWIG00000022777', 'nsg2', 'CPEB4', 'adad2', 'slc9a6b', 'ENSGWIG00000007612', 'ENSGWIG00000007613', 'ENSGWIG00000015586', 'slc22a5', 'slc22a4', 'ENSGWIG00000015613', 'ENSGWIG00000021237', 'fstl4', 'ENSGWIG00000022027', 'ENSGWIG00000022028', 'ENSGWIG00000022039', 'ubtd2', 'IL12B', 'rnf145a', 'EBF1', 'htatsf1', 'slc13a2', 'larp1', 'macroh2a1', 'ENSGWIG00000022914', 'PCBD2', 'ddx46', 'sar1b', 'jade2', 'tcf7', 'ENSGWIG00000024155', 'afap1l1b', 'mcf2b', 'znrd2', 'fgfr4', 'ENSGWIG00000024482', 'ENSGWIG00000024488', 'rab24', 'pgk1', 'KCNJ6', 'gria4b', 'alkbh4', 'ENSGWIG00000008447', 'tmem120a', 'ap2b1', 'mybbp1a', 'ENSGWIG00000013244', 'psmg1', 'ppm1e', 'tlcd3a', 'ENSGWIG00000017767', 'tecta', 'tbcelb', 'sc5d', 'cadm1a', 'ENSGWIG00000018158', 'ncam1a', 'zbtb16a', 'htr3b', 'arhgef12b', 'ENSGWIG00000023432', 'ENSGWIG00000023434', 'SPRY3', 'zgc:158291', 'klf5l', 'klf8', 'dtx2', 'ENSGWIG00000023622', 'tmem132e', 'doc2b', 'ENSGWIG00000023864', 'si:ch211-176g13.8', 'ENSGWIG00000024416', 'plp1b', 'gria3a', 'ENSGWIG00000006956', 'ENSGWIG00000007379', 'myo1ca', 'ENSGWIG00000007665', 'pafah1b2', 'ucp3', 'ccdc61', 'ENSGWIG00000009221', 'ENSGWIG00000009226', 'ENSGWIG00000009231', 'npas2', 'AR', 'MSN', 'amer1', 'nufip2', 'msi2a', 'omga', 'vezf1a', 'myo18aa', 'sez6a', 'acaca', 'tbx2a', 'ppm1db', 'ENSGWIG00000023805', 'ENSGWIG00000024770', 'pcdh1a', 'ENSGWIG00000024892', 'ENSGWIG00000024897', 'npm1a', 'fgf11b', 'gabrb4', 'gabra3', 'TMEM164', 'adssl', 'esrra', 'banf1', 'slc8a4a', 'cnih2', 'tm7sf2', 'rom1a', 'ENSGWIG00000008361', 'ENSGWIG00000008362', 'stip1', 'si:dkey-35m8.1', 'ABR', 'specc1', 'ENSGWIG00000016746', 'pigl', 'fam222ba', 'si:ch211-283g2.1', 'dhrs11a', 'myo19', 'cfap58', 'ca4a', 'MTMR4', 'ENSGWIG00000021693', 'ENSGWIG00000021694', 'supt4h1', 'slc23a1', 'SRRM3', 'ENSGWIG00000021737', 'tiprl', 'HLCS', 'nova1', 'zgc:153990', 'ppp1r14aa', 'mark4a', 'ENSGWIG00000010633', 'bhlha9', 'ENSGWIG00000010637', 'ENSGWIG00000010638', 'wscd1b', 'usp25', 'samsn1b', 'dscama', 'ENSGWIG00000014302', 'B3GAT1', 'b3gat1b', 'thap12a', 'wnt11', 'uvrag', 'mogat2', 'pdzd3b', 'ENSGWIG00000014457', 'ruvbl2', 'SPNS2', 'cyb5d2', 'zzef1', 'auts2a', 'ENSGWIG00000016190', 'ints2', 'med13a', 'rnft1', 'ENSGWIG00000019537', 'c2cd2', 'ripk4', 'cxadr', 'atp2a3', 'ENSGWIG00000004527', 'ENSGWIG00000004535', 'p2rx1', 'pfdn1', 'ENSGWIG00000004817', 'ENSGWIG00000004878', 'ENSGWIG00000005144', 'slco2b1', 'zgc:171929', 'amot', 'sytl2a', 'dlg2', 'ENSGWIG00000012152', 'ntm', 'lhb', 'map4k3a', 'epas1b', 'slc17a5', 'SPRED2', 'ENSGWIG00000015030', 'zc3h15', 'ENSGWIG00000015066', 'ENSGWIG00000015370', 'mcm8', 'ENSGWIG00000018843', 'cdh11', 'ENSGWIG00000023519', 'ENSGWIG00000023561', 'ENSGWIG00000023568', 'lgalslb', 'ENSGWIG00000023702', 'ehbp1', 'b3gnt2b', 'ENSGWIG00000023734', 'tdrd12', 'hhat', 'kcnh1a', 'desi2', 'vip', 'ENSGWIG00000009272', 'ENSGWIG00000009273', 'pank1a', 'ENSGWIG00000009343', 'slc8a1b', 'ENSGWIG00000009371', 'atrnl1a', 'ENSGWIG00000009507', 'ccdc172', 'ENSGWIG00000009511', 'ENSGWIG00000009514', 'cbr1', 'pdxkb', 'ENSGWIG00000010447', 'ENSGWIG00000010583', 'ENSGWIG00000018749', 'si:ch211-117n7.8', 'ENSGWIG00000020390', 'si:ch73-54n14.2', 'ppm1ba', 'znhit2', 'LRPPRC', 'dync2li1', 'hint1', 'tmem26a', 'akt3a', 'zbtb18', 'adss2', 'pcnx2', 'rgs17', 'oprm1', 'wapla', 'grid1a', 'fam149b1', 'p4ha1b', 'actn2b', 'ENSGWIG00000005524', 'ENSGWIG00000005527', 'chrm3a', 'rgs7b', 'bicc1a', 'ENSGWIG00000008375', 'slc16a9a', 'ank3a', 'ubac2', 'bora', 'pibf1', 'cog2', 'hmx2', 'gpr26', 'abraxas2', 'eef1akmt2', 'INSYN2A', 'ptpreb', 'mgmt', 'sorcs3b', 'ENSGWIG00000024475', 'SH3PXD2A', 'ENSGWIG00000025369', 'sfr1', 'ENSGWIG00000025376', 'papss2a', 'prkg1a', 'pcdh15a', 'lrmda', 'kcnma1a', 'zmiz1a', 'rps24', 'eif4ebp2', 'sar1ab', 'spock2', 'trmt2b', 'cuedc2', 'pax2a', 'cdhr1a', 'ogdhl', 'MARK1', 'EFEMP1', 'si:ch211-233m11.2', 'tp53bp2a', 'lzts2a', 'ftr14l', 'tet1', 'march8', 'ENSGWIG00000025397', 'ENSGWIG00000025401', 'zgc:171482', 'sfrp5', 'golga7bb', 'crtac1b', 'loxl4', 'zgc:123010', 'ENSGWIG00000009654', 'pyroxd2', 'slc25a28', 'got1', 'hpse2', 'hps1', 'morn4', 'ENSGWIG00000009752', 'slit1a', 'cutc', 'KCNIP2', 'HPS6', 'ENSGWIG00000015862', 'ldb1a', 'pprc1', 'ENSGWIG00000018145', 'rrp12', 'antxr1d', 'si:ch211-223a10.1', 'WDFY4', 'lrrc18a', 'ENSGWIG00000019822', 'ENSGWIG00000020168', 'npy4r', 'golga4', 'KCNK12', 'FOXN2', 'lhcgr', 'ENSGWIG00000022394', 'ENSGWIG00000010070', 'thbs2a', 'map3k4', 'fam135a', 'ENSGWIG00000020526', 'ENSGWIG00000020527', 'lmbrd1', 'adgrb3', 'ENSGWIG00000021705', 'ENSGWIG00000021708', 'khdrbs2', 'prim2', 'znf451', 'ENSGWIG00000021756', 'col21a1', 'lrrc1', 'eloal', 'elovl5', 'fbxo9', 'mrps6', 'micu1', 'mcu', 'pde10a', 'unc5b', 'ly6pge', 'ENSGWIG00000024530', 'nsmce4a', 'capn1a', 'ENSGWIG00000025827', 'ENSGWIG00000025900', 'ENSGWIG00000025901', 'flrt3', 'tasp1', 'ism1', 'sptlc3', 'btbd3b', 'ENSGWIG00000025934', 'jag1b', 'ENSGWIG00000025942', 'si:ch1073-291c23.1', 'STRN', 'ENSGWIG00000014445', 'irf2bp2a', 'tomm20a', 'ENSGWIG00000016378', 'lyst', 'GPR137B', 'egln1a', 'ENSGWIG00000016562', 'sipa1l2', 'ENSGWIG00000016632', 'klhdc3', 'ENSGWIG00000016637', 'ENSGWIG00000016638', 'ENSGWIG00000016641', 'ENSGWIG00000016644', 'ENSGWIG00000016664', 'PTK7', 'ablim1a', 'ENSGWIG00000020689', 'herc4', 'vsir', 'ENSGWIG00000021322', 'plpp4', 'wdr11', 'fgfr2', 'ATE1', 'inpp5f', 'ret', 'si:dkey-192p21.6', 'sgpl1', 'sorbs1', 'pik3ap1', 'fam204a', 'rab11fip2', 'emx2', 'eif3s10', 'si:ch211-117n7.7', 'MOCOS', 'vwc2', 'fignl1', 'ikzf1', 'ttl', 'nanp', 'cyp1b1', 'ENSGWIG00000002553', 'cdc42ep3', 'ENSGWIG00000002555', 'ENSGWIG00000002557', 'opn3', 'march5', 'btaf1', 'ccnj', 'ENSGWIG00000009452', 'dntt', 'ENSGWIG00000009545', 'ENSGWIG00000009550', 'ADGRA1', 'ENSGWIG00000009600', 'ENSGWIG00000009604', 'ENSGWIG00000009607', 'ENSGWIG00000009614', 'valopa', 'nkx6.2', 'INPP5A', 'pwwp2b', 'STK32C', 'galm', 'ltbp1', 'ENSGWIG00000010975', 'ttc27', 'ENSGWIG00000013430', 'rin2', 'slc24a3', 'ccdc85a', 'ENSGWIG00000013722', 'vrk2', 'fancl', 'ENSGWIG00000013752', 'bcl11aa', 'rims1a', 'b3gat2', 'xpo1b', 'ENSGWIG00000014004', 'ENSGWIG00000014012', 'SERTAD2', 'ENSGWIG00000007414', 'ENSGWIG00000007419', 'ENSGWIG00000007423', 'zfp36l2', 'ENSGWIG00000007462', 'ENSGWIG00000007544', 'ENSGWIG00000007547', 'calm2a', 'ENSGWIG00000007894', 'rhoq', 'socs5b', 'ENSGWIG00000007902', 'calhm2.2', 'calhm3', 'sdhaf4', 'hells', 'ENSGWIG00000008264', 'ENSGWIG00000008276', 'nr2f5', 'sv2a', 'pbx2', 'phf1', 'syngap1b', 'ENSGWIG00000001776', 'cpne4a', 'ENSGWIG00000002060', 'ENSGWIG00000006079', 'ENSGWIG00000006083', 'map7d1a', 'sh3d21', 'stk40', 'csf3r', 'GRIK3', 'ENSGWIG00000008587', 'zc3h12a', 'mtf1', 'smpdl3b', 'mecr', 'foxo6a', 'cited4b', 'ctps1b', 'si:ch211-79k12.1', 'cadm4', 'bcam', 'fhl3a', 'ENSGWIG00000014197', 'pou3f1', 'ENSGWIG00000014199', 'rragca', 'akirin1', 'cnr2', 'ENSGWIG00000010121', 'si:ch73-373m9.1', 'ENSGWIG00000012174', 'ago3a', 'ENSGWIG00000018852', 'ENSGWIG00000018854', 'zgc:110372', 'ENSGWIG00000018858', 'etfb', 'ENSGWIG00000018885', 'xrcc1', 'drd2l', 'ENSGWIG00000018903', 'ENSGWIG00000018907', 'mag', 'ndufaf6', 'bcan', 'ENSGWIG00000019398', 'mef2d', 'bmper', 'bbs9', 'atg5', 'prdm1a', 'tlr18', 'mcl1b', 'tmod4', 'ENSGWIG00000021267', 'ENSGWIG00000021270', 'ENSGWIG00000010492', 'ENSGWIG00000010494', 'sema6e', 'ptpn23a', 'cspg5a', 'crppa', 'grik5', 'ENSGWIG00000011570', 'znf574', 'znf526', 'cicb', 'ENSGWIG00000011864', 'ENSGWIG00000011869', 'dedd1', 'rabac1', 'ENSGWIG00000012103', 'si:ch73-22o12.1', 'ddr1', 'COPS7B', 'ENSGWIG00000022331', 'clstn3', 'wnt4b', 'CHD4', 'mrpl51', 'plekhg6', 'gapdh', 'wu:fj39g12', 'ncapd2', 'kel', 'zyx', 'fam131bb', 'clcn1b', 'si:dkey-40m6.8', 'tmc4', 'rps9', 'flcn', 'ENSGWIG00000002312', 'si:dkeyp-69b9.3', 'ENSGWIG00000002394', 'grin2db', 'prkcg', 'cacng8b', 'foxj2', 'epn1', 'NECAP1', 'ephb6', 'ENSGWIG00000007059', 'si:dkey-283b15.2', 'si:dkey-283b15.4', 'sphk2', 'rpl18', 'ENSGWIG00000010093', 'zbtb7b', 'ENSGWIG00000013313', 'pklr', 'arnt', 'celf3b', 'ENSGWIG00000010628', 'crabp2a', 'rnf115', 'ENSGWIG00000013959', 'ENSGWIG00000013966', 'kcnn3', 'adar', 'ENSGWIG00000014395', 'krtcap2', 'efna1b', 'efna3b', 'si:dkey-246i14.3', 'shc1', 'chrnb2', 'tuft1a', 'ENSGWIG00000014777', 'atp8b2', 'znf687a', 'ENSGWIG00000014927', 'psmd4b', 'rorc', 'ENSGWIG00000015122', 'abcb4', 'crot', 'fam171a1', 'epb41l4b', 'esrp1', 'virma', 'cdh17', 'tmem67', 'si:ch211-197h24.6', 'invs', 'nr4a3', 'alg2', 'galnt1', 'ino80c', 'ENSGWIG00000011341', 'dgat1b', 'tbrg4', 'steap4', 'ENSGWIG00000011475', 'itga8', 'fam8a1b', 'nradd', 'nbeal2', 'si:ch211-257p13.3', 'cep72', 'zdhhc11', 'ptdss1a', 'NHLRC1', 'grinab', 'mpp6a', 'gsdmeb', 'osbpl3b', 'pde11al', 'ENSGWIG00000016751', 'tax1bp1b', 'ankrd28b', 'hdac9b', 'macc1', 'sp8b', 'ENSGWIG00000017164', 'fhod3b', 'cyp21a2', 'ENSGWIG00000002596', 'nphs1', 'kirrel3l', 'zmp:0000001114', 'hsc70', 'si:dkey-79d12.5', 'ENSGWIG00000007350', 'ENSGWIG00000007359', 'ENSGWIG00000007361', 'si:ch211-79k12.2', 'ptprub', 'paqr7b', 'ENSGWIG00000011979', 'hivep3b', 'phactr4b', 'rcc1', 'ENSGWIG00000012569', 'tcea3', 'tent5ba', 'zdhhc18a', 'arid1aa', 'gpatch3', 'nr0b2a', 'cdcp1a', 'lars2', 'LIMD1', 'mroh1', 'trak1a', 'ctnnb1', 'cmtm8a', 'trmt11', 'ENSGWIG00000005268', 'ENSGWIG00000005269', 'rspo3', 'si:ch211-151o1.4', 'soga3a', 'ENSGWIG00000005448', 'rad21a', 'eif3hb', 'ENSGWIG00000005586', 'rbms3', 'ENSGWIG00000006082', 'cobl', 'ENSGWIG00000010270', 'dnah5', 'adcy2a', 'ENSGWIG00000010552', 'ENSGWIG00000010559', 'adamts16', 'ENSGWIG00000010580', 'irx1a', 'irx2a', 'irx4a', 'LPCAT1', 'slc6a3', 'nrsn1', 'ENSGWIG00000012087', 'sox4b', 'CDKAL1', 'ENSGWIG00000013779', 'si:ch211-93i7.4', 'CARMIL1', 'sntb1', 'ENSGWIG00000018135', 'ptp4a3b', 'ptk2ab', 'dscc1', 'col14a1a', 'deptor', 'ubr5', 'azin1a', 'ENSGWIG00000002321', 'si:dkey-122a22.2', 'zfpm2a', 'RIMS2', 'slc25a32a', 'klhl32', 'ENSGWIG00000007535', 'ENSGWIG00000007536', 'pou3f2b', 'fbxl4', 'faxcb', 'ENSGWIG00000007624', 'usp45', 'prdm13', 'ENSGWIG00000007674', 'me1', 'ENSGWIG00000008712', 'dop1a', 'ube3d', 'tpbgb', 'TENT5A', 'ENSGWIG00000008808', 'bckdhb', 'gdf6a', 'eny2', 'ENSGWIG00000008861', 'trhra', 'TMEM74', 'emc2', 'rspo2', 'plecb', 'ENSGWIG00000011958', 'zgc:91890', 'ENSGWIG00000011963', 'sall3a', 'ENSGWIG00000011965', 'ENSGWIG00000012009', 'ENSGWIG00000012010', 'mbpb', 'ENSGWIG00000012013', 'znf236', 'ENSGWIG00000012020', 'ZNF516', 'TSHZ1', 'ZADH2', 'znf407', 'ENSGWIG00000012025', 'has2', 'ENSGWIG00000012030', 'tbc1d31', 'ENSGWIG00000013609', 'ZHX1', 'ENSGWIG00000013632', 'psma2', 'ENSGWIG00000013654', 'ENSGWIG00000015318', 'ENSGWIG00000015320', 'hpn', 'scxb', 'gapdhs', 'sult2st3', 'NIPAL2', 'KCNS2', 'ENSGWIG00000015445', 'osr2', 'VPS13B', 'ENSGWIG00000015489', 'FBXO43', 'rnf19a', 'ncalda', 'id4', 'cep76', 'zufsp', 'kpna5', 'ENSGWIG00000015960', 'ENSGWIG00000015971', 'ldlrad4b', 'mc5rb', 'ENSGWIG00000001793', 'aldh2.2', 'rbm42', 'ppox', 'ENSGWIG00000002122', 'fbl', 'lyplal1', 'tgfb2', 'ENSGWIG00000003843', 'ENSGWIG00000003845', 'vegfaa', 'hace1', 'bves', 'ENSGWIG00000004897', 'ENSGWIG00000005016', 'cnot10', 'tcaim', 'ENSGWIG00000007812', 'SLC6A20', 'sacm1lb', 'ENSGWIG00000008812', 'ENSGWIG00000008814', 'adgrb1a', 'ENSGWIG00000010388', 'si:ch211-208k15.1', 'ENSGWIG00000010391', 'ENSGWIG00000010392', 'ENSGWIG00000010399', 'ENSGWIG00000010404', 'ENSGWIG00000010407', 'gask1a', 'pomgnt2', 'snrka', 'ano10a', 'ABHD5', 'ENSGWIG00000011081', 'ENSGWIG00000011086', 'zgc:114181', 'taf2', 'enpp2', 'nod1', 'ZNRF2', 'ENSGWIG00000013439', 'ENSGWIG00000013441', 'ppp1r9alb', 'ENSGWIG00000016119', 'ENSGWIG00000016179', 'epha4b', 'ENSGWIG00000019040', 'ENSGWIG00000019096', 'pip5k1cb', 'gpx4b', 'ENSGWIG00000020456', 'ENSGWIG00000025871', 'rorcb', 'sh3gl1a', 'ENSGWIG00000025881', 'sirt6', 'ENSGWIG00000025886', 'ENSGWIG00000002666', 'ENSGWIG00000002662', 'rab2a', 'ca8', 'pif1', 'ENSGWIG00000002709', 'ENSGWIG00000002726', 'zgc:153115', 'slc35a3b', 'zgc:110269', 'pkn2', 'lrrc8da', 'ENSGWIG00000009651', 'gadd45aa', 'dmbx1a', 'st3gal3b', 'kdm4ab', 'abhd8a', 'ENSGWIG00000012404', 'nr2f6a', 'kcnn1a', 'mast3a', 'pik3r2', 'impad1', 'ENSGWIG00000013633', 'ephb3a', 'ENSGWIG00000014484', 'ENSGWIG00000014532', 'ENSGWIG00000014631', 'ENSGWIG00000014633', 'zgc:85843', 'ENSGWIG00000014641', 'agla', 'aqp1a.1', 'ENSGWIG00000023159', 'cables1', 'ranbp9', 'znf622', 'myo10', 'efr3a', 'ENSGWIG00000020236', 'kcnq3', 'si:ch211-235o23.1', 'tmeff1a', 'slc39a6', 'proca', 'vcpip1', 'PHB2', 'ENSGWIG00000020534', 'ENSGWIG00000025565', 'ENSGWIG00000025647', 'map4l', 'klhl20', 'ENSGWIG00000025669', 'brinp2', 'astn1', 'pappa2', 'neto1', 'ctnnd2b', 'lyn', 'zswim5', 'fabp6', 'tmem53', 'Metazoa_SRP', 'ENSGWIG00000018630', 'rnf220a', 'klf17', 'slc6a9', 'abhd4', 'rem2', 'carmil3', 'cdh24b', 'psmb5', 'slc7a8b', 'ENSGWIG00000021579', 'sall2', 'cbln12', 'tox4a', 'ENSGWIG00000022156', 'casq1a', 'hhatla', 'ENSGWIG00000024377', 'wdr48b', 'col6a1', 'col6a2', 'zgc:162816', 'ugt1b7', 'ENSGWIG00000025233', 'ENSGWIG00000025234', 'sugt1', 'ENSGWIG00000025240', 'anxa13l', 'ENSGWIG00000013605', 'APBB1IP', 'abi1b', 'mpp7b', 'ENSGWIG00000016962', 'ENSGWIG00000016966', 'tfr1a', 'zgc:56409', 'dab1b', 'ENSGWIG00000020472', 'ENSGWIG00000020491', 'pde4bb', 'ENSGWIG00000023907', 'slc1a7a', 'scp2a', 'ENSGWIG00000024840', 'ENSGWIG00000025294', 'brinp3b', 'rgs18', 'si:rp71-1g18.13', 'ENSGWIG00000025341', 'ENSGWIG00000025342', 'best4', 'acadm', 'st6galnac3', 'st6galnac5a', 'pigk', 'zzz3', 'usp33', 'miga1', 'fubp1', 'ENSGWIG00000017890', 'gnal2', 'PIEZO2', 'ENSGWIG00000018775', 'ralbp1', 'ENSGWIG00000019827', 'apoda.2', 'sec62', 'prkci', 'sppl2', 'arhgef18a', 'insra', 'hiat1a', 'ENSGWIG00000023034', 'stard3nl', 'ENSGWIG00000023054', 'ENSGWIG00000023055', 'LARP4B', 'DIP2C', 'zmynd11', 'esyt2a', 'ENSGWIG00000026221', 'ENSGWIG00000026226', 'pfkpb', 'pitrm1', 'ENSGWIG00000026275', 'fzd8b', 'GJD4', 'pard3ab', 'NRP1', 'itgb1b', 'ENSGWIG00000026479', 'ENSGWIG00000026482', 'arhgap12b', 'pold3', 'ENSGWIG00000001852', 'zeb1a', 'SVIL', 'ENSGWIG00000008697', 'kcnj9', 'lig1', 'atp1a2a', 'sdhc', 'cadm3', 'kirrel1b', 'ENSGWIG00000013250', 'ENSGWIG00000013322', 'klf2b', 'ap1m1', 'ranbp3b', 'NRTN', 'ptprfb', 'cdcp2', 'lrrc42', 'dcaf8', 'vav3b', 'ENSGWIG00000026173', 'zgc:92137', 'col11a1b', 'PTPRM', 'epb41l3b', 'zbtb14', 'APCDD1', 'si:dkey-154p10.3', 'myom1a', 'ENSGWIG00000026682', 'ENSGWIG00000026691', 'neurod6b', 'ENSGWIG00000026694', 'fyco1b', 'adcyap1r1a', 'CRHR2', 'myl13', 'pth1r', 'tmie', 'ENSGWIG00000013967', 'cx47.1', 'asic1c', 'asb10', 'ENSGWIG00000014389', 'ENSGWIG00000014390', 'pcmtd1', 'ube2v2', 'SPIDR', 'mapre2', 'dtna', 'klhl14', 'ENSGWIG00000014627', 'ENSGWIG00000014656', 'mfsd12a', 'zgc:123035', 'ENSGWIG00000015233', 'ENSGWIG00000015634', 'cahz', 'ENSGWIG00000015677', 'ptf1a', 'tmem70', 'si:ch211-207d6.2', 'paxip1', 'ENSGWIG00000015729', 'ENSGWIG00000015739', 'klhl18', 'ENSGWIG00000015785', 'alg3', 'ece2a', 'mbnl1', 'ENSGWIG00000002537', 'ENSGWIG00000006417', 'ENSGWIG00000006419', 'ephb1', 'gpc1b', 'gpc5c', 'nyap2b', 'ENSGWIG00000006468', 'ENSGWIG00000006470', 'fgf12a', 'parla', 'rc3h1b', 'ppm1la', 'irf4a', 'ENSGWIG00000006629', 'mylk4a', 'wrnip1', 'ENSGWIG00000006688', 'zp2l2', 'eif4g1a', 'ENSGWIG00000011482', 'ppp1r7', 'ELOVL1', 'kif1ab', 'rab6bb', 'ENSGWIG00000013585', 'ENSGWIG00000013586', 'amotl2b', 'ENSGWIG00000001985', 'ptch2', 'eif2b3', 'mrps14', 'cacybp', 'tnn', 'tnr', 'ENSGWIG00000008695', 'ENSGWIG00000008707', 'OSBPL1A', 'ENSGWIG00000008831', 'ZNF521', 'ss18', 'psma8', 'KCTD1', 'AQP4', 'ENSGWIG00000008887', 'CDH2', 'ENSGWIG00000015801', 'ENSGWIG00000015802', 'dsc2l', 'ENSGWIG00000015838', 'TNS3', 'adcy1b', 'lamc1', 'ndrg2', 'ENSGWIG00000003653', 'ENSGWIG00000003663', 'znf219', 'ENSGWIG00000003699', 'si:dkey-46i9.6', 'mrpl48', 'ccnb3', 'ENSGWIG00000003763', 'htr2cl2', 'gpr185b', 'ENSGWIG00000003784', 'si:ch211-200p22.4', 'rab39bb', 'khnyn', 'ENSGWIG00000005286', 'lpcat4', 'nop10', 'SLC12A6', 'clcn5b', 'ENSGWIG00000014428', 'bmp15', 'ENSGWIG00000013652', 'ENSGWIG00000013749', 'ENSGWIG00000016457', 'ENSGWIG00000016458', 'ENSGWIG00000016460', 'ENSGWIG00000016523', 'pnp5a', 'PACS1', 'actn3b', 'ccs', 'gal3st3', 'si:ch211-168d1.3', 'aadat', 'f13a1b', 'ENSGWIG00000024995', 'gba2', 'pik3c3', 'ENSGWIG00000025100', 'ENSGWIG00000025104', 'ENSGWIG00000025126', 'slc7a8a', 'ENSGWIG00000025170', 'efs', 'tm4sf5', 'ENSGWIG00000015565', 'rbpjb', 'ENSGWIG00000016963', 'DMRTA1', 'ENSGWIG00000016968', 'RBPMS', 'ENSGWIG00000016983', 'ENSGWIG00000016996', 'DOK7', 'tnk1', 'YBX2', 'capgb', 'polr2a', 'zbtb4', 'neurl4', 'si:ch73-335l21.1', 'ENSGWIG00000014015', 'dvl2', 'per1b', 'rell1', 'nwd2', 'ENSGWIG00000014588', 'dthd1', 'ENSGWIG00000014626', 'pcdh7b', 'ENSGWIG00000015169', 'CAAP1', 'ENSGWIG00000015547', 'ENSGWIG00000021811', 'MPDZ', 'LURAP1L', 'tyrp1a', 'ENSGWIG00000024601', 'USP53', 'myoz2a', 'sec24d', 'KCNIP4', 'adgra3', 'gba3', 'ENSGWIG00000026490', 'ppargc1a', 'dhx15', 'ccdc149a', 'lgi2a', 'sepsecs', 'etnppl', 'larp7', 'ADAMTSL1', 'plin2', 'plaa', 'NFIB', 'ENSGWIG00000026730', 'DDIT4L', 'ENSGWIG00000026769', 'pcxb', 'ppp1r14ba', 'slc8a4b', 'badb', 'si:dkey-23i12.9', 'rcor2', 'mark2b', 'hspa12b', 'nagk', 'm1ap', 'ENSGWIG00000020082', 'ENSGWIG00000020110', 'tmem88b', 'ENSGWIG00000021212', 'kirrel1a', 'ENSGWIG00000022105', 'rbm4.3', 'kdm6ba', 'ENSGWIG00000022988', 'ENSGWIG00000022990', 'ENSGWIG00000022994', 'ENSGWIG00000022996', 'efnb3b', 'nlgn2a', 'ENSGWIG00000004133', 'fgf11a', 'RASGRP2', 'ENSGWIG00000004510', 'ENSGWIG00000004523', 'ENSGWIG00000004558', 'nrxn2b', 'ENSGWIG00000004681', 'si:cabz01078036.1', 'drap1', 'rela', 'sorcs2', 'grpel1', 'zgc:158564', 'ENSGWIG00000010944', 'ENSGWIG00000011004', 'camk2d1', 'afap1', 'ache', 'ENSGWIG00000007762', 'ENSGWIG00000007768', 'ENSGWIG00000007778', 'ENSGWIG00000007832', 'gal3st4', 'eif4a1a', 'senp3b', 'focad', 'mllt3', 'sf1', 'pygmb', 'HTRA3', 'nat16', 'mtmr1a', 'slc1a1', 'taf6l', 'eml3', 'mta2', 'ENSGWIG00000024109', 'ENSGWIG00000024135', 'ehbp1l1a', 'ppa1a', 'pald1a', 'polr3a', 'ENSGWIG00000025151', 'ENSGWIG00000025420', 'ZNF503', 'DUPD1', 'vti1a', 'tcf7l2', 'ENSGWIG00000008721', 'crygmxl2', 'med1', 'gpam', 'ENSGWIG00000009396', 'ENSGWIG00000009459', 'DNAJC7', 'tbc1d24', 'ENSGWIG00000011526', 'capn15', 'MSS51', 'ENSGWIG00000011553', 'ENSGWIG00000011555', 'lrrc4bb', 'ENSGWIG00000011558', 'ENSGWIG00000011800', 'ENSGWIG00000017040', 'ENSGWIG00000017050', 'rexo4', 'ptprh', 'MPRIP', 'ENSGWIG00000018278', 'prr12a', 'ENSGWIG00000019659', 'ENSGWIG00000019660', 'cox10', 'si:ch211-216b21.2', 'cdc42ep4b', 'sdk2b', 'ENSGWIG00000019754', 'kif19', 'ENSGWIG00000019787', 'ENSGWIG00000019792', 'bicral', 'tbcc', 'QKI', 'si:dkey-49c17.3', 'timp2a', 'pgs1', 'socs3b', 'ENSGWIG00000007485', 'si:ch73-127m5.1', 'sec24c', 'SYNPO2L', 'ldb3b', 'mta3', 'ENSGWIG00000024225', 'cox7a2l', 'ENSGWIG00000024750', 'clrn3', 'foxi1', 'dock1', 'ADAM12', 'ENSGWIG00000026345', 'hpdl', 'fam53b', 'CHST15', 'CPXM2', 'GPR26', 'bub3', 'ENSGWIG00000026450', 'si:ch211-166g5.4', 'ENSGWIG00000003648', 'ENSGWIG00000009887', 'itgam', 'TANC2', 'wnk4b', 'ENSGWIG00000014834', 'U5', 'mfsd13a', 'sema4gb', 'twnk', 'ENSGWIG00000017117', 'plekhm1', 'ENSGWIG00000017137', 'ABCA5', 'baiap2b', 'aatkb', 'ENSGWIG00000017274', 'ENSGWIG00000017290', 'cep131', 'gprc5c', 'map2k6', 'ENSGWIG00000018006', 'ENSGWIG00000018026', 'armc7', 'GRID1', 'waplb', 'ENSGWIG00000019521', 'thrab', 'dhx8', 'MYO1D', 'tbkbp1', 'cdk5rap3', 'skap1', 'fbrs', 'ENSGWIG00000023783', 'stx4', 'ENSGWIG00000023867', 'rundc3ab', 'SPOP', 'ENSGWIG00000002950', 'ngfrb', 'smarcd2', 'tom1l2', 'trrap', 'bhlha15', 'ENSGWIG00000009014', 'ENSGWIG00000009101', 'ENSGWIG00000009105', 'usp42', 'tmem235b', 'ENSGWIG00000012821', 'bahcc1b', 'slc38a10', 'AMZ2', 'ENSGWIG00000014411', 'dlx4b', 'itga3b', 'ace', 'tut1', 'raraa', 'med24', 'ENSGWIG00000002713', 'ENSGWIG00000007978', 'znf646', 'ENSGWIG00000008008', 'si:ch73-81k8.2', 'zmp:0000000624', 'ENSGWIG00000011027', 'ift140', 'ENSGWIG00000014768', 'ENSGWIG00000014770', 'VAT1', 'ENSGWIG00000014778', 'ENSGWIG00000014782', 'becn1', 'dnmbp', 'nhlrc2', 'afap1l2', 'atrnl1b', 'ENSGWIG00000023726', 'ENSGWIG00000023727', 'ENSGWIG00000023729', 'cep55l', 'si:ch211-197g18.2', 'plce1', 'ENSGWIG00000014268', 'nmt1b', 'svild', 'kcnj19b', 'msrb1b', 'cdc42ep1a', 'si:ch73-139e5.4', 'ep300a', 'josd1', 'mrtfab', 'mms19', 'ubtd1b', 'antxr1c', 'ENSGWIG00000019447', 'ENSGWIG00000019448', 'si:dkey-191m6.4', 'ENSGWIG00000021570', 'sox9a', 'ENSGWIG00000021580', 'ADAP1', 'RAB26', 'pemt', 'ENSGWIG00000021766', 'NT5M', 'cacna1g', 'UTS2R', 'cybc1', 'ENSGWIG00000014257', 'itgb4', 'sap30bp', 'csnk1e', 'ENSGWIG00000015321', 'snx29', 'shisa9a', 'cpped1', 'mrtfba', 'ENSGWIG00000015564', 'ENSGWIG00000015573', 'foxj1b', 'lcmt1', 'grid2ipb', 'ENSGWIG00000019348', 'tob1b', 'ENSGWIG00000019352', 'abcc3', 'sult2st2', 'wu:fj29h11', 'dhrs7cb', 'hs3st3l', 'ENSGWIG00000014259', 'ENSGWIG00000014265', 'ENSGWIG00000014293', 'exoc6', 'jmjd1cb', 'reep3b', 'egr2b', 'mtr', 'ENSGWIG00000017606', 'ENSGWIG00000017644', 'FMN2', 'cavin1b', 'ENSGWIG00000017951', 'naglu', 'ipmkb', 'tfam', 'zgc:171971', 'ENSGWIG00000017971', 'si:ch211-214e3.5', 'rab3gap1', 'fasn', 'CSNK1D', 'ENSGWIG00000021131', 'ENSGWIG00000021154', 'ENSGWIG00000021178', 'ENSGWIG00000021279', 'llgl2', 'ENSGWIG00000024497', 'mbtd1', 'ENSGWIG00000024603', 'ENSGWIG00000024604', 'ENSGWIG00000024605', 'ca10a', 'ENSGWIG00000024609', 'ENSGWIG00000024610', 'ENSGWIG00000024611', 'ENSGWIG00000024613', 'eme1', 'ENSGWIG00000024616', 'vopp1', 'lancl2', 'znf438', 'ZEB1', 'ARHGAP12', 'si:ch211-276f18.2', 'ENSGWIG00000023253', 'ENSGWIG00000023256', 'ENSGWIG00000023259', 'ENSGWIG00000023260', 'ENSGWIG00000023264', 'ENSGWIG00000002716', 'CPNE3', 'snx16', 'ENSGWIG00000020886', 'ANKS6', 'SNORD54', 'zgc:56231', 'rgs9bp', 'dph2', 'ncf2', 'ENSGWIG00000020953', 'olfm3a', 'col11a1a', 'rnpc3', 'ntng1a', 'VAV3', 'fam168b', 'Y_RNA', 'arhgap21a', 'ENSGWIG00000021963', 'ENSGWIG00000021965', 'ENSGWIG00000021984', 'ENSGWIG00000014258', 'zgc:110852', 'ccr12b.2', 'si:dkey-225n22.4', 'slc12a7b', 'cubn', 'rsu1', 'c1ql3a', 'ENSGWIG00000021325', 'ENSGWIG00000021343', 'ENSGWIG00000021682', 'TGFBR1', 'hccsb', 'ENSGWIG00000022134', 'snx7', 'plppr5b', 'ENSGWIG00000022138', 'ABI1', 'ENSGWIG00000022230', 'apbb1ip', 'myo3a', 'anxa13', 'gpr158a', 'PTCHD3', 'ENSGWIG00000025045', 'ENSGWIG00000025047', 'lztfl1', 'gli3', 'TMX3', 'DSEL', 'ENSGWIG00000025227', 'CDH7', 'ENSGWIG00000025274', 'ENSGWIG00000025277', 'cdk13', 'ENSGWIG00000025285', 'ca2', 'ENSGWIG00000025289', 'si:dkey-61p9.11', 'topbp1', 'ENSGWIG00000025574', 'scrib', 'slc4a2b', 'smarcd3b', 'asap1b', 'fam49bb', 'myca', 'ankha', 'pou6f2', 'ENSGWIG00000020416', 'OPRK1', 'myom1b', 'emilin2b', 'mettl4', 'ADCYAP1', 'enosf1', 'hacd1', 'cacnb2a', 'zgc:112356', 'PLXDC2', 'esyt2b', 'ncapg2', 'ptprn2', 'dnajb6b', 'nom1', 'rnf32', 'shha', 'rbm33a', 'dlgap1b', 'tgif1', 'si:ch211-285f17.1', 'uba5', 'chst2b', 'ENSGWIG00000023953', 'dipk2ab', 'plod2', 'si:ch73-206p6.1', 'zic1', 'agtr1b', 'cpb1', 'gyg1b', 'oxsr1b', 'plcd1a', 'vill', 'ctdspla', 'spice1', 'naa50', 'qtrt2', 'drd3', 'fam162a', 'ccdc58', 'kpna1', 'vipr1b', 'atp8a2', 'cdk8', 'gpr12', 'gtf3ab', 'pan3', 'tagln3a', 'ythdf3', 'bhlhe22', 'armc1', 'crhb', 'ppp1r42', 'prex2', 'zgc:162928', 'sulf1', 'SLCO5A1', 'ncoa2', 'xkr9', 'eya1', 'msc', 'trpa1b', 'si:dkey-192j17.1', 'SBSPON', 'ENSGWIG00000002737', 'ENSGWIG00000002789', 'emc9', 'ENSGWIG00000002825', 'ENSGWIG00000003053', 'rnf152', 'CDH20', 'ENSGWIG00000009598', 'drosha', 'CDH6', 'ENSGWIG00000015375', 'ENSGWIG00000015855', 'cdh12a', 'CDH18', 'epc1a', 'ENSGWIG00000016401', 'dpp6a', 'adgrg2a', 'smpx', 'ENSGWIG00000016488', 'map3k15', 'RAMP3', 'phldb2b', 'tfap2a', 'ENSGWIG00000016671', 'ENSGWIG00000016673', 'ENSGWIG00000016675', 'slc35b3', 'eef1e1', 'bloc1s5', 'txndc5', 'pdcd6', 'dcaf11', 'ENSGWIG00000019206', 'ENSGWIG00000019232', 'ENSGWIG00000019236', 'ankrd33ba', 'nup58', 'ENSGWIG00000019276', 'ENSGWIG00000019279', 'zfhx4', 'pex2', 'ENSGWIG00000019367', 'tpk1', 'ENSGWIG00000007508', 'bmi1a', 'mllt10', 'ENSGWIG00000011751', 'oplah', 'myrip', 'ENSGWIG00000012118', 'ENSGWIG00000012119', 'pdk3a', 'cct5', 'mtrr', 'ENSGWIG00000015372', 'sema5a', 'cbln2b', 'ENSGWIG00000017865', 'ENSGWIG00000017870', 'socs6a', 'rttn', 'dok6', 'ENSGWIG00000020638', 'slc39a10', 'dnah7', 'cx43.4', 'si:dkey-34f9.3', 'traf3ip1', 'adarb1b', 'ENSGWIG00000022089', 'ENSGWIG00000022101', 'ENSGWIG00000022103', 'ccnt2a', 'TMEM163', 'ENSGWIG00000022113', 'tmem177', 'epb41l5', 'ENSGWIG00000022150', 'ENSGWIG00000023683', 'ENSGWIG00000023685', 'wnt6b', 'ENSGWIG00000024056', 'myo1b', 'ENSGWIG00000024260', 'ENSGWIG00000024261', 'lrp1bb', 'ENSGWIG00000024391', 'stxbp5l', 'nxph2a', 'thsd7ba', 'ENSGWIG00000003482', 'cxcr4a', 'dars1', 'gmppaa', 'ENSGWIG00000006876', 'ENSGWIG00000007070', 'ENSGWIG00000007097', 'fzd7b', 'ENSGWIG00000007126', 'tsc22d1', 'kctd4', 'CLDN14', 'EPHA6', 'ENSGWIG00000007259', 'ENSGWIG00000007261', 'STX19', 'epha3', 'erbb4b', 'ikzf2', 'vwc2l', 'bard1', 'abca12', 'ptprna', 'vgll3', 'pou1f1', 'HTR1F', 'zgc:152951', 'si:ch211-261p9.4', 'cps1', 'myl1', 'ENSGWIG00000025449', 'ENSGWIG00000025450', 'ENSGWIG00000025459', 'asmt', 'akap17a', 'ENSGWIG00000025465', 'ENSGWIG00000025466', 'edar', 'desma', 'ENSGWIG00000009152', 'arsh', 'ENSGWIG00000009294', 'ENSGWIG00000009300', 'ENSGWIG00000009304', 'dip2a', 'ENSGWIG00000010960', 'zgc:158316', 'ENSGWIG00000023286', 'SMARCAL1', 'igf2bp2a', 'ENSGWIG00000023877', 'galnt13', 'kcnj3a', 'nr4a2a', 'ENSGWIG00000025137', 'ACVR1C', 'upp2', 'ENSGWIG00000025194', 'PKP4', 'tanc1b', 'KCNH7', 'fign', 'ENSGWIG00000026541', 'ENSGWIG00000026639', 'xirp2b', 'ENSGWIG00000026812', 'B3GALT1', 'dhrs9', 'ENSGWIG00000026827', 'fastkd1', 'myo3b', 'SLC25A12', 'ENSGWIG00000017167', 'ppp1r9ala', 'rapgef4', 'si:ch73-167c12.2', 'chn1', 'atp5mc3b', 'lnpa', 'hoxd4a', 'hoxd3a', 'hnrnpa3', 'agps', 'DOCK9', 'FARP1', 'MBNL2', 'RAP2A', 'hs6st3b', 'gpc6b', 'slitrk5b', 'ENSGWIG00000026736', 'ENSGWIG00000026737', 'pcdh8', 'si:ch211-199f5.1', 'ednrbb', 'ENSGWIG00000014494', 'glra2', 'mospd2', 'tbl1x', 'ANOS1', 'ENSGWIG00000015271', 'igsf3', 'rpe', 'ackr3a', 'ENSGWIG00000015307', 'ENSGWIG00000015636', 'chaf1b', 'bcl9', 'si:ch73-390b10.2', 'FAM155A', 'myo16', 'col4a1', 'ENSGWIG00000015988', 'ankrd10b', 'sik1', 'u2af1', 'asb11', 'sytl5', 'gpr161', 'me3', 'gdf3', 'klf7b', 'creb1b', 'ENSGWIG00000022618', 'itprid2', 'ENSGWIG00000009586', 'shox', 'ENSGWIG00000009595', 'asmtl', 'cdc16', 'ENSGWIG00000010085', 'fgf14', 'nalcn', 'tmtc4', 'ENSGWIG00000010166', 'zic2a', 'clybl', 'si:ch211-184m13.4', 'dachd', 'klf12b', 'tbc1d4', 'lmo7a', 'kctd12.1', 'ENSGWIG00000011737', 'tspan7b', 'mid1ip1b', 'ENSGWIG00000011845', 'bcor', 'ENSGWIG00000012603', 'sh3bp4', 'lrch1', 'esd', 'htr2aa', 'med4', 'itm2bb', 'ENSGWIG00000013398', 'kpna3', 'spryd7b', 'rnaseh2b', 'ENSGWIG00000015383', 'NR0B1', 'il1rapl1a', 'ENSGWIG00000015579', 'dcbld2', 'COL8A1', 'enox1', 'ENSGWIG00000014591', 'sox1a', 'arhgef7a', 'aff3', 'txndc9', 'ENSGWIG00000015649', 'inpp4aa', 'tmem182a', 'pou3f3a', 'nck2a', 'tmtops2b', 'ST6GAL2', 'jam2b', 'grtp1b', 'pou2f1b', 'ENSGWIG00000016957', 'ildr2', 'slc15a2', 'hspbap1', 'slc49a4', 'sema5ba', 'pdia5', 'sec22a', 'adcy5', 'ENSGWIG00000020254', 'ARL5A', 'neb', 'PDCL3', 'si:ch1073-440b2.1', 'ENSGWIG00000005808', 'ENSGWIG00000007673', 'hacd2', 'ENSGWIG00000007680', 'ENSGWIG00000007710', 'arhgap20', 'rnf17', 'mphosph8', 'gja3', 'xpo4', 'lats2', 'ska3', 'ildr1b', 'ENSGWIG00000010126', 'ENSGWIG00000010128', 'ankrd50l', 'hpxa', 'olig1', 'ENSGWIG00000010537', 'fam160a2', 'cnga4', 'si:dkey-91i10.3', 'ENSGWIG00000010563', 'dhrs12', 'wdfy2', 'actr3', 'DPP10', 'ENSGWIG00000006943', 'cops8', 'ENSGWIG00000006965', 'RAB17', 'ENSGWIG00000006983', 'hecw2a', 'stk17b', 'zgc:153521', 'zgc:153901', 'ENSGWIG00000007172', 'ENSGWIG00000007174', 'ENSGWIG00000007176', 'ENSGWIG00000007183', 'SLC35F5', 'ENSGWIG00000013640', 'pms1', 'mstnb', 'ENSGWIG00000016268', 'hibch', 'tns1b', 'ENSGWIG00000021806', 'slc4a3', 'col5a2a', 'gulp1a', 'rftn2', 'satb2', 'kcnh3', 'ENSGWIG00000023866', 'idh1', 'akap11', 'dgkh', 'vwa8', 'rgcc', 'f5', 'ccdc80', 'kdm6a', 'ENSGWIG00000023878', 'ndp', 'ddx3xa', 'ENSGWIG00000025541', 'ENSGWIG00000025560', 'IMPG2', 'ENSGWIG00000025864', 'man1a2', 'hcn5', 'pla1a', 'tbx15', 'wars2', 'slc37a1', 'pde9a', 'pknox1.2', 'insig2', 'ENSGWIG00000026256', 'ENSGWIG00000026257', 'itgb2', 'map4k5', 'l2hgdh', 'sos2', 'ENSGWIG00000009640', 'cnih1', 'adprhl2', 'ENSGWIG00000009705', 'ENSGWIG00000009712', 'nrde2', 'ENSGWIG00000009721', 'dicer1', 'gsc', 'enpep', 'lrit3b', 'clmn', 'ENSGWIG00000009844', 'ENSGWIG00000009937', 'ENSGWIG00000009939', 'ENSGWIG00000009946', 'ENSGWIG00000009950', 'ENSGWIG00000009963', 'ilkap', 'ENSGWIG00000009976', 'ptp4a2a', 'ENSGWIG00000009983', 'syt14a', 'slc8a3', 'actn1', 'dcaf5', 'pacs2', 'tmem151bb', 'bend3', 'pdss2', 'sobpa', 'slc66a3', 'si:dkeyp-47f9.4', 'alk', 'ENSGWIG00000018388', 'ENSGWIG00000018393', 'ENSGWIG00000018397', 'ENSGWIG00000018404', 'commd8', 'golga5', 'fbln5', 'plekhd1', 'smoc1', 'paplna', 'psen1', 'vps39', 'galnt16', 'ENSGWIG00000004982', 'ENSGWIG00000005292', 'ENSGWIG00000006049', 'shprh', 'stxbp5b', 'sash1b', 'plekhh1', 'zfp36l1a', 'tmem229b', 'mta1', 'crip2', 'trim9', 'tmx1', 'coq8ab', 'ENSGWIG00000009214', 'ENSGWIG00000009220', 'ENSGWIG00000009222', 'ENSGWIG00000009225', 'slc25a21', 'mbip', 'dpysl5a', 'ENSGWIG00000022696', 'ENSGWIG00000022743', 'C14orf132', 'spred1', 'meis2a', 'zgc:154061', 'dph6', 'GJD2', 'stxbp6', 'fermt2', 'ddhd1a', 'bmp4', 'lbh', 'ENSGWIG00000023227', 'manea', 'si:ch211-266g18.9', 'col10a1a', 'fyna', 'ENSGWIG00000024911', 'usp40', 'ENSGWIG00000024915', 'ENSGWIG00000024919', 'ENSGWIG00000024926', 'elavl1a', 'slc1a8b', 'shda', 'si:dkey-56i24.1', 'slitrk3b', 'nrd1a', 'NPAS3', 'akap6', 'arhgap5', 'ENSGWIG00000019728', 'ENSGWIG00000019729', 'kcnh5b', 'ENSGWIG00000023141', 'ENSGWIG00000023143', 'ENSGWIG00000023145', 'ENSGWIG00000023163', 'ENSGWIG00000023257', 'ENSGWIG00000023261', 'klc1a', 'bag5', 'trmt61a', 'mark3a', 'ENSGWIG00000023644', 'letm1', 'larp1b', 'ahsa1b', 'ENSGWIG00000024049', 'cdca4', 'si:dkey-177p2.6', 'ENSGWIG00000024059', 'fgfr3', 'ENSGWIG00000026265', 'ENSGWIG00000026266', 'foxi2', 'ENSGWIG00000026269', 'ky', 'slc4a11', 'eif2ak3', 'atrn', 'gfra4a', 'ap5s1', 'sfxn5a', 'rab11fip5a', 'ENSGWIG00000015754', 'ganc', 'ENSGWIG00000016390', 'kcnk3b', 'ENSGWIG00000016421', 'dctn1b', 'si:ch211-195e19.1', 'slc39a8', 'cpxm1a', 'hhipl2', 'vrk1', 'BCL11B', 'setd3', 'dio2', 'ENSGWIG00000022049', 'adck1', 'daam2', 'ENSGWIG00000023461', 'kif6', 'kcnk5a', 'iars2', 'rab3gap2', 'ENSGWIG00000024410', 'ccr6b', 'pxdn', 'MYT1L', 'ENSGWIG00000024561', 'eipr1', 'trappc12', 'marcksl1a', 'ENSGWIG00000024677', 'bsdc1', 'olig4', 'fndc5a', 'syf2', 'runx3', 'clic4', 'SRRM1', 'lck', 'pabpc4', 'ralgapa2', 'kiz', 'nkx2.4a', 'pax1a', 'foxa2', 'prkd3', 'ENSGWIG00000016720', 'SCAF8', 'tiam2a', 'col12a1a', 'CD109', 'ENSGWIG00000017552', 'dlgap2a', 'arhgef10', 'kbtbd11', 'ENSGWIG00000017673', 'ZDHHC14', 'nt5c1ab', 'MYCL', 'zbtb8os', 'PLCB2', 'ANKRD63', 'pak6a', 'spint1a', 'akt1', 'INF2', 'gchfr', 'cyp1c2', 'ENSGWIG00000021349', 'ENSGWIG00000021350', 'ENSGWIG00000021351', 'FMN1', 'ENSGWIG00000021676', 'ENSGWIG00000021722', 'SIPA1L1', 'ENSGWIG00000022811', 'ZFYVE1', 'ENSGWIG00000002740', 'ENSGWIG00000002774', 'VSX2', 'ENSGWIG00000007064', 'AREL1', 'rps6kl1', 'GPATCH2L', 'IFT43', 'flvcr2a', 'ACYP1', 'ENSGWIG00000009348', 'gskip', 'ATG2B', 'si:ch211-243o19.4', 'tmem63c', 'pgfb', 'ap5m1', 'exoc5', 'otx2b', 'ENSGWIG00000012980', 'peli2', 'aspg', 'kif26ab', 'ENSGWIG00000013628', 'grhl1', 'ywhaqb', 'rpap1', 'ivd', 'bahd1', 'eml1', 'evla', 'prkcha', 'slc38a6', 'mnat1', 'LRRC9', 'FGFRL1', 'si:dkey-85n7.6', 'btbd7', 'itpk1b', 'mrpl35', 'ENSGWIG00000009477', 'UBR1', 'ENSGWIG00000010416', 'id2a', 'rnf144aa', 'cmpk2', 'sox11a', 'ENSGWIG00000014212', 'ENSGWIG00000014214', 'impg1b', 'HTR1B', 'SNX14', 'znf292a', 'orc3', 'CNR1', 'GABRR1', 'stac', 'znf593', 'rpl13a', 'ENSGWIG00000018572', 'DLK1', 'ENSGWIG00000019605', 'WDR25', 'SLC25A29', 'ENSGWIG00000016019', 'tmem234', 'grhl3', 'si:ch1073-358c10.1', 'qkia', 'ENSGWIG00000024124', 'ptafr', 'ENSGWIG00000024166', 'ankrd6b', 'gabrr2b', 'ube2j1', 'ENSGWIG00000026626', 'slc5a6a', 'gopc', 'ENSGWIG00000012698', 'rad51', 'si:ch211-241j12.3', 'ephx1', 'enah', 'ENSGWIG00000012733', 'ENSGWIG00000012734', 'mcm9', 'cx43', 'hsf2', 'serinc1', 'ENSGWIG00000012831', 'katna1', 'kif3ca', 'rab10', 'lats1', 'map3k5', 'ENSGWIG00000014405', 'tfap2b', 'tfap2d', 'hadhb', 'ENSGWIG00000018342', 'ENSGWIG00000018346', 'jag2b', 'ENSGWIG00000020134', 'reps1', 'ENSGWIG00000020188', 'scara3', 'ak7a', 'ENSGWIG00000020212', 'ppp1cb', 'bpnt1', 'ENSGWIG00000020249', 'dlgap2b', 'ptprk', 'ENSGWIG00000025350', 'ENSGWIG00000025352', 'lama2', 'ENSGWIG00000025481', 'rgs6', 'sipa1l1', 'rab15', 'fntb', 'srsf5b', 'susd6', 'gpr176', 'ENSGWIG00000025504', 'thbs1b', 'katnbl1', 'ENSGWIG00000003254', 'serac1', 'gtf2h5', 'tulp4a', 'cep57l1', 'ENSGWIG00000004529', 'slc24a4b', 'cpsf2', 'extl3', 'nt5c1bb', 'si:dkey-174m14.3', 'rngtt', 'slc35a1', 'rhag', 'ENSGWIG00000008327', 'ENSGWIG00000008341', 'capn3b', 'churc1', 'laptm4a', 'pum2', 'rock2b', 'ENSGWIG00000012950', 'mrap2b', 'sim1b', 'ENSGWIG00000012962', 'ENSGWIG00000012964', 'ENSGWIG00000012981', 'lin28b', 'xkr5a', 'ENSGWIG00000013948', 'mycn', 'smek1', 'otofa', 'armc2', 'acbd3', 'si:dkey-206f10.1', 'smyd2b', 'rps6kc1', 'moxd1l', 'snx9b', 'ENSGWIG00000022540', 'calm1b', 'kcnk13b', 'ches1', 'CEP128', 'nrxn3b', 'sptlc2b', 'ccdc85cb', 'si:ch211-197l9.5', 'ENSGWIG00000025423', 'ENSGWIG00000025652', 'runx2b', 'opn8b', 'mfsd2b', 'atad2b', 'klhl29', 'gpcpd1', 'ENSGWIG00000025715', 'PLCB1', 'ENSGWIG00000016212', 'ENSGWIG00000016216', 'ENSGWIG00000016218', 'evlb', 'entpd5b', 'znf410', 'cipcb', 'fkbp3', 'MDGA2', 'ENSGWIG00000017895', 'ENSGWIG00000018373', 'RPS29', 'mgat2', 'ENSGWIG00000022244', 'slc25a47a', 'vrtn', 'dll4', 'ino80', 'ccn2a', 'fkbp6', 'tmem200a', 'ENSGWIG00000022610', 'coq8aa', 'ENSGWIG00000024308', 'ehd3', 'med23', 'ENSGWIG00000024347', 'nkain2', 'elovl4b', 'tent5ab', 'snap91b', 'ENSGWIG00000024492', 'tmem63a', 'mrps18a', 'vegfab', 'naga', 'ENSGWIG00000024525', 'si:ch211-57i17.1', 'si:ch73-21k16.1', 'si:ch73-21k16.5', 'daam1b', 'ENSGWIG00000005357', 'ENSGWIG00000005370', 'ENSGWIG00000005378', 'zc3h14', 'ENSGWIG00000005419', 'ptk2bb', 'chrna2b', 'ENSGWIG00000009557', 'nme6', 'snx17', 'si:ch211-278j3.3', 'ENSGWIG00000010630', 'sox7', 'pinx1', 'tdh', 'ccm2', 'eya4', 'SGK1', 'aldh8a1', 'ENSGWIG00000015516', 'ENSGWIG00000015528', 'ppp1r14c', 'MTLN', 'lratd1', 'nbas', 'fut8b', 'gphnb', 'ENSGWIG00000016278', 'EZR', 'opn8c', 'ppp2r5a', 'myct1a', 'ENSGWIG00000023609', 'cdc42bpab', 'fam49a', 'vsnl1a', 'ENSGWIG00000025947', 'flrt2', 'ENSGWIG00000025950', 'psma6b', 'gpr6', 'wasf1', 'usta', 'sash1a', 'ENSGWIG00000026195', 'utrn', 'tmem214', 'ENSGWIG00000026536', 'gpr63', 'disp1', 'ENSGWIG00000004351', 'tnfrsf21', 'opn5', 'cnih3', 'si:dkey-1j5.4']

#orientalis = ['socs7', 'sdk1a', 'gna12a', 'cyp4v8', 'mtnr1aa', 'stox2a', 'nr3c2', 'inpp4b', 'g3bp2', 'ppm1k', 'usp53b', '5S_rRNA', 'sept9b', 'ENSGWIG00000004234', 'suz12b', 'ldlrb', 'sept3', 'atf7ip', 'hgsnat', 'ENSGWIG00000024646', 'pcdh7a', 'klhl5', 'slit2', 'ENSGWIG00000026772', 'dlc1', 'ugt8', 'zgc:136908', 'SRL', 'adcy9', 'mxi1', 'dusp5', 'stra6l', 'ENSGWIG00000022112', 'ENSGWIG00000009748', 'ENSGWIG00000010140', 'aspscr1', 'ENSGWIG00000022003', 'ENSGWIG00000004740', 'zranb3', 'ENSGWIG00000010163', 'prkx', 'cep70', 'zgc:152904', 'farp1', 'ENSGWIG00000026227', 'ENSGWIG00000026230', 'LIPT1', 'ENSGWIG00000003631', 'pcbp3', 'ENSGWIG00000012343', 'mbnl2', 'ENSGWIG00000020935', 'ENSGWIG00000021677', 'erbb4a', 'cobll1a', 'spry2', 'ENSGWIG00000001228', 'slitrk6', 'ENSGWIG00000001235', 'ARHGAP6', 'ENSGWIG00000019301', 'chsy1', 'zgc:165481', 'abcc12', 'ntrk3b', 'ENSGWIG00000001160', 'mrpl46', 'tm2d3', 'ENSGWIG00000007870', 'ENSGWIG00000007873', 'irx3a', 'ENSGWIG00000012637', 'UNC13C', 'onecut1', 'ENSGWIG00000000181', 'nfatc3a', 'zdhhc7', 'ENSGWIG00000011608', 'galnt2', 'immp1l', 'MAF', 'ENSGWIG00000006287', 'chchd2', 'CDYL2', 'mettl15', 'lgr4', 'mybpc3', 'si:ch211-150g13.3', 'zfhx3', 'crispld2', 'ENSGWIG00000024682', 'mtmr10', 'rnf111', 'slc22a31', 'ankrd11', 'cbfa2t3', 'ENSGWIG00000018771', 'ENSGWIG00000018774', 'pla2r1', 'nhsb', 'cyp19a1a', 'mapk6', 'pde8a', 'ENSGWIG00000024354', 'itga11a', 'coro2ba', 'TLE3', 'uacab', 'mibp2', 'slc30a4', 'MEGF11', 'cebpa', 'slc9a5', 'NELL1', 'ENSGWIG00000000463', 'luzp2', 'ENSGWIG00000000858', 'large2', 'ENSGWIG00000004595', 'ENSGWIG00000011404', 'ENSGWIG00000011425', 'ENSGWIG00000013277', '', 'si:ch211-262h13.5', 'ENSGWIG00000017092', 'ENSGWIG00000017096', 'ENSGWIG00000024483', 'cp', 'cachd1', 'ror1', 'pgm1', 'pde4ba', 'ENSGWIG00000025735', 'kdm4b', 'ptprsa', 'ENSGWIG00000006753', 'ENSGWIG00000006862', 'ENSGWIG00000010056', 'negr1', 'ENSGWIG00000022806', 'actl6a', 'insrb', 'colgalt2', 'pdgfra', 'CLOCK', 'exoc1', 'ENSGWIG00000018962', 'usp24', 'dab1a', 'prrc2c', 'si:dkey-239i20.2', 'ralgps2', 'coa7', 'ENSGWIG00000008539', 'nr2c2ap', 'atp5f1d', 'polrmt', 'rgmd', 'palld', 'CDKL5', 'nme7', 'tp63', 'tgfbr3', 'barhl2', 'lmo4b', 'hs2st1b', 'farp2', 'ENSGWIG00000015078', 'espnla', 'ryk', 'ENSGWIG00000018765', 'ENSGWIG00000018800', 'ENSGWIG00000018802', 'ENSGWIG00000018808', 'ptprfa', 'ENSGWIG00000000187', 'arhgap39', 'EPS15L1', 'ENSGWIG00000001048', 'NAALADL2', 'ect2', 'mcf2l2', 'chd6', 'ENSGWIG00000000153', 'raly', 'ENSGWIG00000000830', 'hgh1', 'si:dkey-238f9.1', 'ENSGWIG00000007499', 'cbfa2t2', 'tox2', 'iars1', 'atp2b2', 'vgll4b', 'tamm41', 'ENSGWIG00000021449', 'PPARG', 'kcnab2a', 'mmel1', 'trim62.1', 'A3GALT2', 'hemk1', 'rrp9', 'pbrm1l', 'camk1a', 'ENSGWIG00000020816', 'ca16b', 'bsnb', 'fbxo42', 'tmcc2', 'scn8ab', 'ttll3', 'rargb', 'mettl1', 'tfeb', 'tmem183a', 'celf4', 'myorg', 'cita', 'ENSGWIG00000008154', 'vdrb', 'ENSGWIG00000018772', 'si:ch211-160o17.4', 'pusl1', 'ENSGWIG00000024272', 'ddx19', 'si:dkey-33m11.6', 'dnajc16l', 'b3galt6', 'srpk1b', 'lhfpl5a', 'kif21b', 'ENSGWIG00000026097', 'ENSGWIG00000007422', 'ENSGWIG00000014029', 'WDR1', 'setd5', 'mtmr14', 'ENSGWIG00000015230', 'ENSGWIG00000015305', 'sox13', 'ENSGWIG00000015808', 'wnt4', 'rybpb', 'adamts9', 'ENSGWIG00000001414', 'si:dkey-13a21.4', 'hcfc1a', 'kmt2e', 'ENSGWIG00000010781', 'CELF2', 'ENSGWIG00000011036', 'ENSGWIG00000012296', 'snd1', 'cacna2d1a', 'iqsec3a', 'chchd3a', 'MIR183', 'zc3hc1', 'ENSGWIG00000022322', 'ENSGWIG00000022957', 'LOC114465680', 'RASSF8', 'ENSGWIG00000023460', 'TMEM229A', 'gpr37b', 'ENSGWIG00000023486', 'ENSGWIG00000023487', 'grm8a', 'nrcama', 'si:ch211-262n1.4', 'dock4b', 'ENSGWIG00000024632', 'ENSGWIG00000024636', 'brd1a', 'ENSGWIG00000025180', 'cerk', 'gramd4a', 'ENSGWIG00000025598', 'mapk11', 'atp23', 'lrig3', 'SYN3', 'TIMP3', 'camk1da', 'ENSGWIG00000014410', 'slc38a4', 'ENSGWIG00000013312', 'wnt16', 'ENSGWIG00000023801', 'ENSGWIG00000006723', 'ENSGWIG00000007387', 'ENSGWIG00000007395', 'pthlhb', 'si:ch211-234p18.3', 'sema4ba', 'gpr22b', 'ENSGWIG00000010700', 'AVPR1A', 'U4', 'fbln1', 'ENSGWIG00000016320', 'ENSGWIG00000016323', 'DENND2A', 'plxnb2b', 'ENSGWIG00000001331', 'szl', 'ENSGWIG00000012349', 'RELN', 'tjp1b', 'ENSGWIG00000013038', 'ENSGWIG00000013063', 'ENSGWIG00000013068', 'ENSGWIG00000013075', 'slco3a1', 'fam174b', 'ENSGWIG00000005028', 'rgma', 'ENSGWIG00000006097', 'fkbp4', 'ppp6r2b', 'dnaja2b', 'ENSGWIG00000007154', 'PCLO', 'ENSGWIG00000000554', 'RSBN1L', 'LRRC17', 'armc10', 'ENSGWIG00000025523', 'lrrc4ca', 'EHF', 'trim44', 'ENSGWIG00000011006', 'ENSGWIG00000011064', 'ENSGWIG00000000612', 'ENSGWIG00000000620', 'kcng4a', 'si:ch211-216l23.2', 'si:ch211-216l23.1', 'cbln1', 'ENSGWIG00000024329', 'spon1b', 'anpepa', 'spire2', 'CADPS2', 'grm8b', 'ENSGWIG00000022722', 'myod1', 'ENSGWIG00000023340', 'trhr2', 'mfge8b', 'ENSGWIG00000008312', 'ireb2', 'cspg4', 'tshz2', 'ENSGWIG00000023668', 'ENSGWIG00000009973', 'gdi1', 'gdap1l1', 'tafa1a', 'grip2a', 'ENSGWIG00000007994', 'camkvl', 'lhfpl4a', 'plxnb1a', 'ENSGWIG00000021290', 'ENSGWIG00000021321', 'krt8', 'krt5', 'HES2', 'tmem240a', 'nr1h5', 'ENSGWIG00000020486', 'mtor', 'agrn', 'wnt10b', 'FIGNL2', 'larp4aa', 'ENSGWIG00000024559', 'samd11', 'tardbpl', 'ENSGWIG00000026182', 'pex14', 'cntn3a.2', 'ppp1r3da', 'pigt', 'ENSGWIG00000002446', 'mib2', 'ttll10', 'tlcd4b', 'nfixb', 'ENSGWIG00000006492', 'si:dkeyp-69e1.8', 'jpt1b', 'nup85', 'slc25a19', 'ENSGWIG00000005723', 'ENSGWIG00000005728', 'fam20ca', 'nptx2a', 'ENSGWIG00000009617', 'ENSGWIG00000012904', 'cacna1ha', 'ENSGWIG00000016217', 'emp2', 'ENSGWIG00000019485', 'GPR139', 'ENSGWIG00000017223', 'sox8b', 'usp22', 'aatka', 'baiap2a', 'mgrn1b', 'si:ch211-210g13.5', 'hcrt', 'znf652', 'eftud2', 'GJC1', 'nbr1b', 'snrnp70', 'ENSGWIG00000004915', 'ENSGWIG00000022820', 'nxph3', 'ENSGWIG00000024988', 'ENSGWIG00000004799', 'evplb', 'smurf2', 'ENSGWIG00000023189', 'ENSGWIG00000010050', 'ENSGWIG00000010051', 'ENSGWIG00000016201', 'ENSGWIG00000016200', 'DMXL1', 'znf366', 'msh3', 'adra2b', 'gpat2', 'camk2b1', 'ENSGWIG00000004831', 'xbp1', 'ksr2', 'nos1', 'scarb2a', 'rusc2', 'adamts3', 'kank1a', 'ENSGWIG00000021656', 'sema4c', 'tm2d2', 'unc5db', 'loxl2b', 'bicdl1', 'citb', 'cacna1ba', 'hapln1a', 'ENSGWIG00000026581', 'ttc37', 'arrdc3a', 'ENSGWIG00000001466', 'ENSGWIG00000001840', 'PEBP4', 'ENSGWIG00000008110', 'stx2b', 'adgrd1', 'ncor2', 'fgfr1a', 'npy8br', 'klhl22', 'vsig8b', 'ENSGWIG00000020930', 'ENSGWIG00000020936', 'ENSGWIG00000001467', 'pxnb', 'anapc5', 'trmt1l', 'fbrsl1', 'ENSGWIG00000008342', 'ankle2', 'lhx2b', 'ENSGWIG00000001573', 'ENSGWIG00000001676', 'rgmb', 'ENSGWIG00000003404', 'ENSGWIG00000003417', 'slc43a1a', 'polr1a', 'ENSGWIG00000020436', 'ENSGWIG00000005722', 'spock1', 'ccnjl', 'atrx', 'ENSGWIG00000005310', 'brd8', 'GABRA2', 'ENSGWIG00000007330', 'snx25', 'ENSGWIG00000021048', 'xpnpep2', 'diaph2', 'enox2', 'SLC4A11', 'slit3', 'gpm6aa', 'fstl5', 'prom1a', 'ENSGWIG00000015331', 'ENSGWIG00000015335', 'pcdh1b', 'ENSGWIG00000012167', 'SAMD12', 'ext1b', 'crabp2b', 'efna3a', 'cks1b', 'csmd3a', 'ENSGWIG00000011602', 'pabpc1b', 'laptm4b', 'pard6gb', 'triqk', 'fbxl2', 'wasf2', 'trappc3', 'col8a2', 'adgrb2', 'col9a2', 'csmd2', 'adcy2b', 'med10', 'ENSGWIG00000013365', 'nrsn1l', 'sox4a', 'ENSGWIG00000013393', 'ENSGWIG00000013395', 'oprd1a', 'ENSGWIG00000014084', 'ENSGWIG00000014093', 'si:dkey-218f9.10', 'KHDRBS3', 'cpsf1', 'thrb', 'ENSGWIG00000010341', 'cited4a', 'mzt2b', 'lhx5', 'zfyve16', 'xpo7', 'ENSGWIG00000007932', 'ENSGWIG00000022485', 'THAP1', 'vav2', 'si:dkey-28o19.1', 'ENSGWIG00000019037', 'ENSGWIG00000019213', 'ENSGWIG00000019482', 'ENSGWIG00000019484', 'cntfr', 'ENSGWIG00000019495', 'rcl1', 'pptc7b', 'ENSGWIG00000004594', 'dym', 'ctif', 'zbtb7c', 'ZNF462', 'TMEM38B', 'ENSGWIG00000014333', 'pappaa', 'pax8', 'si:dkey-34e4.1', 'zc3h4', 'ENSGWIG00000019545', 'treh', 'cadm2b', 'ENSGWIG00000013136', 'dph1', 'ENSGWIG00000007678', 'hic1', 'si:ch1073-111c8.3', 'alcamb', 'ENSGWIG00000023902', 'aadac', 'ENSGWIG00000025054', 'ENSGWIG00000002620', 'strn4', 'ENSGWIG00000006207', 'kmt2a', 'ENSGWIG00000020943', 'zgc:162730', 'tyr', 'trip12', 'pid1', 'sphkap', 'ENSGWIG00000024624', 'ENSGWIG00000024631', 'slitrk3a', 'ENSGWIG00000024950', 'ENSGWIG00000024969', 'ccnl1a', 'KIRREL3', 'ENSGWIG00000009512', 'fli1a', 'blmh', 'igsf11', 'ENSGWIG00000001514', 'ENSGWIG00000007489', 'ENSGWIG00000013036', 'myo1cb', 'wdfy1', 'nrg2a', 'cadm2a', 'paxbp1', 'slc7a1a', 'ENSGWIG00000018279', 'mrps17', 'gabra1', 'elmod1', 'cadm1a', 'SPRY3', 'zgc:158291', 'ENSGWIG00000023622', 'tmem132e', 'aifm1', 'omga', 'ksr1a', 'ENSGWIG00000024770', 'pcdh1a', 'ENSGWIG00000024892', 'nlgn2b', 'adssl', 'esrra', 'ENSGWIG00000021693', 'HLCS', 'nova1', 'capns1a', 'ENSGWIG00000010633', 'bhlha9', 'usp25', 'samsn1b', 'dscama', 'ENSGWIG00000014393', 'ENSGWIG00000001608', 'ppm1ba', 'tnfaip3', 'ENSGWIG00000023312', 'olig3', 'map3k21', 'rgs17', 'oprm1', 'htra1a', 'mgmt', 'slka', 'dlg5a', 'zmiz1a', 'rps24', 'zgc:171482', 'ENSGWIG00000022394', 'ENSGWIG00000021708', 'ENSGWIG00000025901', 'irf2bp2a', 'ENSGWIG00000014447', 'sipa1l2', 'ENSGWIG00000016637', 'ENSGWIG00000016638', 'ENSGWIG00000016641', 'ENSGWIG00000016644', 'ENSGWIG00000020968', 'mlh1', 'smoc2', 'eif3s10', 'dntt', 'ADGRA1', 'ENSGWIG00000009600', 'ENSGWIG00000009604', 'valopa', 'nkx6.2', 'STK32C', 'ltbp1', 'ttc27', 'ENSGWIG00000013430', 'slc24a3', 'ccdc85a', 'REL', 'ENSGWIG00000007423', 'zfp36l2', 'csf3r', 'GRIK3', 'themis2', 'SCMH1', 'pou3f1', 'ENSGWIG00000014199', 'dedd1', 'ENSGWIG00000025728', 'si:dkey-40m6.8', 'grin2db', 'gba', 'ENSGWIG00000013684', 'adam22', 'mpp6a', 'gsdmeb', 'RAPGEF5', 'ptprub', 'washc5', 'ankib1b', 'adcy2a', 'irx1a', 'irx2a', 'nrsn1', 'ENSGWIG00000012087', 'ENSGWIG00000002321', 'si:dkey-122a22.2', 'RIMS2', 'cthrc1a', 'gdf6a', 'eny2', 'TMEM74', 'emc2', 'sall3a', 'ENSGWIG00000011965', 'ENSGWIG00000010407', 'ENSGWIG00000002662', 'rab2a', 'astn1', 'pappa2', 'ankrd33bb', 'ugt1b7', 'mcm6', 'ENSGWIG00000025223', 'ENSGWIG00000025233', 'gnal2', 'prkci', 'ENSGWIG00000022299', 'arhgef18a', 'ENSGWIG00000026275', 'fzd8b', 'pard3ab', 'NRP1', 'ENSGWIG00000026482', 'ENSGWIG00000001852', 'klf2b', 'ap1m1', 'col11a1b', 'sqstm1', 'asb10', 'ENSGWIG00000014956', 'xpr1b', 'ENSGWIG00000006417', 'ENSGWIG00000006419', 'ephb1', 'fam131a', 'ENSGWIG00000011482', 'ppp1r7', 'tnr', 'ENSGWIG00000008695', 'ENSGWIG00000016460', 'ENSGWIG00000024994', 'RBPMS', 'CPZ', 'si:dkey-38p12.3', 'pcdh7b', 'ENSGWIG00000015547', 'ENSGWIG00000021811', 'ENSGWIG00000026730', 'ENSGWIG00000022120', 'nrxn2b', 'drap1', 'afap1', 'ablim2', 'senp3b', 'polr3a', 'ENSGWIG00000007864', 'ENSGWIG00000022443', 'fam53b', 'mto1', 'ABCA5', 'ENSGWIG00000017226', 'waplb', 'thrab', 'CBX1', 'fam20cb', 'ENSGWIG00000005181', 'trrap', 'mki67', 'zgc:91999', 'casp7', 'adrb1', 'ENSGWIG00000023729', 'cep55l', 'mrtfab', 'mms19', 'vstm4b', 'cpped1', 'abcc3', 'jmjd1cb', 'zgc:171971', 'ENSGWIG00000017974', 'znf438', 'ARHGAP12', 'KIF5B', 'ENSGWIG00000020886', 'ANKS6', 'ENSGWIG00000021123', 'olfm3a', 'tfr1b', 'tnk2a', 'arhgap21a', 'TGFBR1', 'acbd5a', 'cdk13', 'ENSGWIG00000025285', 'ENSGWIG00000025574', 'mrc1a', 'ENSGWIG00000022716', 'ptprn2', 'dnajb6b', 'wasf3b', 'prex2', 'zgc:162928', 'sulf1', 'CDH20', 'ENSGWIG00000009598', 'CDH6', 'ENSGWIG00000015375', 'epha3', 'erbb4b', 'arsh', 'tanc1b', 'NUP62', 'FARP1', 'hs6st3b', 'cldn10l2', 'FAM155A', 'ccnyl1', 'tmtc4', 'clybl', 'dachd', 'ENSGWIG00000011845', 'enox1', 'rev1', 'grtp1b', 'neb', 'TBC1D8', 'ENSGWIG00000005808', 'ENSGWIG00000007710', 'tns1b', 'ENSGWIG00000021806', 'satb2', 'ENSGWIG00000023866', 'idh1', 'slc8a3', 'ENSGWIG00000018397', 'ENSGWIG00000018404', 'ENSGWIG00000005292', 'spred1', 'dph6', 'ENSGWIG00000023227', 'fut9a', 'ENSGWIG00000015638', 'akap6', 'ENSGWIG00000019728', 'ppp2r5ea', 'ENSGWIG00000023139', 'fgfr3', 'ENSGWIG00000026265', 'ENSGWIG00000026266', 'slc4a11', 'ENSGWIG00000022049', 'pxdn', 'MYT1L', 'ENSGWIG00000024561', 'eipr1', 'col12a1a', 'cln8', 'ENSGWIG00000017673', 'nt5c1ab', 'ENSGWIG00000023488', 'iah1', 'adam17a', 'bahd1', 'tdrd9', 'si:dkey-85n7.6', 'mcm9', 'kif3ca', 'map3k5', 'tfap2d', 'hadhb', 'rgs6', 'sipa1l1', 'thbs1b', 'katnbl1', 'aven', 'nrxn3b', 'im:7142702', 'ENSGWIG00000025413', 'runx2b', 'opn8b', 'fermt1', 'bmp2b', 'PLCB1', 'ENSGWIG00000016214', 'ENSGWIG00000016216', 'ENSGWIG00000016218', 'ENSGWIG00000016241', 'ino80', 'batf', 'hbs1l', 'vsnl1a', 'ENSGWIG00000017636', 'abtb2b', 'noxa1', 'sem1', 'ENSGWIG00000010915', 'aipl1', 'auts2a', 'atp8a2', 'dlx2a', 'tmem200a', 'ENSGWIG00000022610']

#hofrichteri = set(['ENSGWIG00000011609', 'ENSGWIG00000007448', 'lrrc4.2', 'ENSGWIG00000019034', 'ENSGWIG00000017397', 'gfra3', 'gria3b', 'tbcelb', 'camk2a', 'ttyh1', 'kcnn1a', 'tbc1d19', 'ENSGWIG00000022851', 'ENSGWIG00000022855', 'ENSGWIG00000022859', 'pax1a', 'foxa2', 'ENSGWIG00000000061', 'ENSGWIG00000000062', 'btr01', 'ENSGWIG00000000065', 'p2rx3b', 'ENSGWIG00000000067', 'ENSGWIG00000000089', 'dnal4b', 'ENSGWIG00000000091', 'ENSGWIG00000000092', 'pvalb7', 'ncf4', '5S_rRNA', 'pdgfba', 'ENSGWIG00000000099', 'ENSGWIG00000000100', 'ENSGWIG00000000101', 'ENSGWIG00000000103', 'ENSGWIG00000000104', 'atf4a', 'ENSGWIG00000000110', 'ENSGWIG00000000113', 'c1qtnf6b', 'mgat3b', 'tab1', 'syngr1b', 'ift27', 'cacng2b', 'ENSGWIG00000000124', 'si:zfos-169g10.2', '', 'ENSGWIG00000000244', 'ENSGWIG00000000252', 'rac2', 'cyth4b', 'fam83fb', 'ENSGWIG00000000457', 'ENSGWIG00000000466', 'ENSGWIG00000000498', 'ENSGWIG00000000502', 'ENSGWIG00000000505', 'ENSGWIG00000000549', 'ENSGWIG00000000656', 'ENSGWIG00000000731', 'ENSGWIG00000000735', 'elfn2a', 'ENSGWIG00000000743', 'mfng', 'ENSGWIG00000000748', 'trmt1', 'nacc1b', 'stx10', 'ier2b', 'cacna1ab', 'ENSGWIG00000002175', 'wdr83os', 'wdr83', 'ENSGWIG00000002202', 'gtf3aa', 'ccdc130', 'c3b.2', 'ENSGWIG00000002240', 'atpv0e2', 'ENSGWIG00000002246', 'ENSGWIG00000002299', 'ENSGWIG00000002847', 'SLC25A28', 'NKX2', '3', 'GOT1', 'CNNM1', 'hoga1', 'ENSGWIG00000002940', 'ENSGWIG00000002962', 'lcor', 'ENSGWIG00000003015', 'slit1b', 'ENSGWIG00000003197', 'arhgap19', 'e2f5', 'si:ch211-133l11.10', 'dla', 'ENSGWIG00000003452', 'ENSGWIG00000003838', 'myom2a', 'ENSGWIG00000015286', 'lgsn', 'ptp4a1', 'si:dkey-23f9.4', 'ENSGWIG00000015303', 'mrpl4', 'abcc10', 'sp2', 'socs7', 'ENSGWIG00000016912', 'mrpl45', 'npepps', 'wipf2a', 'mrpl21', 'ccsapa', 'nup133', 'taf5l', 'capn9', 'ENSGWIG00000020682', 'ltv1', 'ENSGWIG00000020683', 'ENSGWIG00000020686', 'akt3b', 'pus10', 'cct4', 'ENSGWIG00000023017', 'fam161a', 'USP34', 'ENSGWIG00000023117', 'ENSGWIG00000023119', 'ENSGWIG00000023121', 'tbce', 'rbm34', 'nid1b', 'ero1b', 'ENSGWIG00000023941', 'vps54', 'ENSGWIG00000023995', 'ENSGWIG00000024021', 'akap8l', 'ENSGWIG00000024289', 'lpar2b', 'ENSGWIG00000024303', 'ENSGWIG00000024304', 'soul5l', 'gpr108', 'ENSGWIG00000024313', 'ENSGWIG00000024314', 'uncx4.1', 'ENSGWIG00000024316', 'ENSGWIG00000024318', 'ENSGWIG00000024319', 'ENSGWIG00000024322', 'ENSGWIG00000024323', 'ENSGWIG00000024324', 'si:ch73-12o23.1', 'ENSGWIG00000024430', 'ENSGWIG00000024433', 'ENSGWIG00000024432', 'ENSGWIG00000024438', 'ENSGWIG00000024439', 'ENSGWIG00000024440', 'ENSGWIG00000024441', 'zgc:153247', 'ENSGWIG00000024447', 'ENSGWIG00000024452', 'ENSGWIG00000024490', 'ENSGWIG00000025389', 'ENSGWIG00000000120', 'ENSGWIG00000000123', 'ENSGWIG00000000128', 'ENSGWIG00000000222', 'ENSGWIG00000000237', 'ENSGWIG00000000242', 'thumpd1', 'ENSGWIG00000000286', 'ENSGWIG00000000303', 'ENSGWIG00000000319', 'ENSGWIG00000000323', 'ENSGWIG00000000345', 'ENSGWIG00000000347', 'ENSGWIG00000000355', 'mad1l1', 'ENSGWIG00000000557', 'ENSGWIG00000000562', 'ENSGWIG00000000568', 'ttyh3b', 'ENSGWIG00000000666', 'ENSGWIG00000000676', 'ENSGWIG00000000700', 'RDH8', 'ENSGWIG00000000738', 'ENSGWIG00000000752', 'TRIP10', 'ENSGWIG00000001813', 'VAV1', 'UBL5', 'ENSGWIG00000001910', 'ptcd1', 'zgc:112980', 'smcr8b', 'mief2', 'alkbh5', 'ENSGWIG00000001987', 'ENSGWIG00000002139', 'zgc:77849', 'abcg2b', 'ENSGWIG00000005783', 'arpc1a', 'radil', 'foxk1', 'ENSGWIG00000005848', 'ENSGWIG00000006025', 'cyp3a65', 'ENSGWIG00000008763', 'ENSGWIG00000009070', 'sdk1a', 'gna12a', 'ENSGWIG00000009632', 'ern2', 'si:dkey-26i13.7', 'eif3bb', 'ENSGWIG00000009827', 'ENSGWIG00000002870', 'si:dkeyp-9d4.5', 'si:dkeyp-9d4.4', 'gng13a', 'ENSGWIG00000003198', 'ENSGWIG00000003215', 'ENSGWIG00000003229', 'prkcba', 'ENSGWIG00000007106', 'ENSGWIG00000007139', 'ENSGWIG00000007152', 'ENSGWIG00000007160', 'pdia2', 'mnd1', 'trim2a', 'fhdc1', 'arfip1', 'fam193a', 'tnip2', 'ENSGWIG00000007598', 'lratb.2', 'npy2r', 'pdlim3a', 'sorbs2a', 'tlr3', 'cyp4v8', 'mtnr1aa', 'fat1a', 'ENSGWIG00000008671', 'ENSGWIG00000008673', 'ENSGWIG00000008676', 'ENSGWIG00000008678', 'ENSGWIG00000008685', 'ENSGWIG00000008687', 'frg1', 'ENSGWIG00000008904', 'osbp', 'casp3a', 'stox2a', 'tll1', 'cpe', 'msmo1', 'klhl2', 'ENSGWIG00000018870', 'clrn2', 'prom1b', 'fgfbp1b', 'anxa5b', 'ENSGWIG00000021108', 'tmem192', 'tma16', 'ENSGWIG00000021117', 'fbxw7', 'ENSGWIG00000021370', 'ENSGWIG00000000129', 'dctpp1', 'gatb', 'fam160a1a', 'sh3d19', 'rps3a', 'SNORD73', 'lrba', 'DCLK2', 'nr3c2', 'arhgap10', 'prmt9', 'tmem184c', 'ednraa', 'pou4f2', 'ENSGWIG00000004272', 'ENSGWIG00000004286', 'ENSGWIG00000004291', 'lsm6', 'znf827', 'mmaa', 'smad1', 'hhip', 'ENSGWIG00000004411', 'frem3', 'usp38', 'gab1', 'inpp4b', 'ENSGWIG00000009815', 'ENSGWIG00000009816', 'rnf150a', 'tbc1d9', 'ucp1', 'elmod2', 'si:dkeyp-123h10.2', 'fam13a', 'herc3', 'uso1', 'g3bp2', 'abcg2d', 'ppm1k', 'ENSGWIG00000012342', 'coro2a', 'anp32b', 'TRMO', 'ENSGWIG00000012586', 'foxe1', 'sh3gl2a', 'cntln', 'bnc2', 'ENSGWIG00000012921', 'ENSGWIG00000012922', 'snapc3', 'pla2g12a', 'MCUB', 'lef1', 'sgms2a', 'papss1', 'dkk2', 'aimp1a', 'tbck', 'npnta', 'gstcd', 'ints12', 'arhgef38', 'tet2', 'cxxc4', 'ENSGWIG00000000135', 'ENSGWIG00000000232', 'pdcd4a', 'mogs', 'ENSGWIG00000000369', 'fbxo41', 'smox', 'rnf24', 'cldnd1b', 'glb1', 'ENSGWIG00000002689', 'ENSGWIG00000002690', 'primpol', 'acsl1a', 'helt', 'slc25a4', 'ENSGWIG00000012210', 'ufsp2', 'ENSGWIG00000012212', 'gucy1a1', 'gucy1b1', 'fabp2', 'ENSGWIG00000012940', 'usp53b', 'ENSGWIG00000012969', 'slc22a7b.1', 'slc22a7b.3', 'DHX15', 'sod3b', 'ENSGWIG00000013200', 'mttp', 'ENSGWIG00000015923', 'acer2', 'ENSGWIG00000015969', 'BPTF', 'C17orf58', 'KPNA2', 'SMURF2', 'U1', 'CEP95', 'srp68', 'evpla', 'ten1', 'acox1', 'RHBDF2', 'UBE2O', 'AANAT', 'SPHK1', 'cygb2', 'myh10', 'ubn2a', 'ttc26', 'slc25a38b', 'myh9a', 'map2k7', 'pkn1a', 'si:ch211-214c7.5', 'ENSGWIG00000025534', 'ENSGWIG00000025535', 'mcoln1a', 'pglyrp2', 'si:ch211-239f4.1', 'ENSGWIG00000026099', 'ENSGWIG00000002879', 'ENSGWIG00000002933', 'ENSGWIG00000002960', 'rasd4', 'ENSGWIG00000004054', 'NTN1', 'ENSGWIG00000004085', 'ENSGWIG00000004090', 'ENSGWIG00000004093', 'cfap52', 'usp43a', 'mchr1a', 'ENSGWIG00000004158', 'jpt1a', 'ENSGWIG00000004163', 'ENSGWIG00000004166', 'sept9b', 'ENSGWIG00000004234', 'znf207a', 'rhbdl3', 'rhot1b', 'ENSGWIG00000011455', 'tefm', 'suz12b', 'utp6', 'ENSGWIG00000011544', 'ENSGWIG00000011545', 'rab11fip4b', 'ENSGWIG00000011548', 'ENSGWIG00000011550', 'ENSGWIG00000011551', 'ENSGWIG00000011552', 'ENSGWIG00000011559', 'ENSGWIG00000011560', 'ENSGWIG00000011561', 'dand5', 'NFIX', 'ENSGWIG00000011749', 'ENSGWIG00000011753', 'ENSGWIG00000011761', 'fdx2', 'ENSGWIG00000011766', 's1pr5b', 'keap1b', 'si:zfos-323e3.4', 'ap1m2', 'ldlrb', 'SMARCA4', 'ENSGWIG00000021086', 'rnaseh2a', 'ENSGWIG00000021328', 'serhl', 'ENSGWIG00000021340', 'sept3', 'atf7ip', 'si:ch211-243a20.3', 'grin2bb', 'ENSGWIG00000021702', 'ENSGWIG00000021711', 'ENSGWIG00000021712', 'rbfox2', 'ENSGWIG00000021717', 'ENSGWIG00000021718', 'ENSGWIG00000021720', 'slc34a2a', 'rhbdd2', 'ENSGWIG00000000168', 'fip1l1b', 'ENSGWIG00000000374', 'clockb', 'si:dkey-245p14.4', 'smc2', 'ecpas', 'kdm4c', 'ENSGWIG00000000992', 'PTPRD', 'ENSGWIG00000023375', 'ENSGWIG00000023378', 'ENSGWIG00000023380', 'adgrl3.1', 'ENSGWIG00000024554', 'gtf2e2', 'rbpja', 'ENSGWIG00000024571', 'ENSGWIG00000024646', 'pcdh7a', 'crmp1', 'wfs1a', 'ppp2r2ca', 'ENSGWIG00000025703', 'ank2a', 'camk2d2', 'ENSGWIG00000026605', 'ZBTB5', 'si:dkey-82j4.2', 'ENSGWIG00000026653', 'ptpn9b', 'melk', 'pax5', 'zcchc7', 'pgm2', 'tbc1d1', 'klf3', 'fam114a1', 'klhl5', 'wdr19', 'rfc1', 'ENSGWIG00000026756', 'ENSGWIG00000026757', 'smim14', 'ENSGWIG00000026758', 'pds5a', 'ENSGWIG00000026762', 'n4bp2', 'slit2', 'ENSGWIG00000026772', 'fam184b', 'ENSGWIG00000026784', 'apbb2b', 'ENSGWIG00000026786', 'ENSGWIG00000026788', 'rbm47', 'ENSGWIG00000026792', 'ENSGWIG00000026793', 'ankrd49', 'GPR83', 'cdk2ap2', 'aip', 'tmem134', 'doc2d', 'CABP2', 'lonrf1l', 'dlc1', 'slc44a1a', 'spock3', 'ugt8', 'ndst3', 'prss12', 'ENSGWIG00000000175', 'adra1d', 'ctnna2', 'lrrtm1', 'ENSGWIG00000000861', 'suclg1', 'si:dkey-162b23.4', 'bdh2', 'TACR3', 'cnga1a', 'odam', 'sparcl1', 'dnaja1', 'prkacab', 'ENSGWIG00000001452', 'zgc:136908', 'ENSGWIG00000002293', 'ENSGWIG00000002294', 'dnm2b', 'tmed1b', 'ENSGWIG00000002493', 'ENSGWIG00000002495', 'ENSGWIG00000002499', 'ENSGWIG00000003619', 'nxnl1', 'si:ch211-114l13.7', 'adgre5b.3', 'ENSGWIG00000019353', 'adcy9', 'crebbpa', 'ENSGWIG00000021759', 'ENSGWIG00000021760', 'tmem186', 'hmox2a', 'cxcl12b', 'ENSGWIG00000021771', 'xpnpep1', 'ENSGWIG00000021804', 'add3a', 'mxi1', 'dusp5', 'smc3', 'RBM20', 'pdcd4b', 'bbip1', 'shoc2', 'adra2a', 'ENSGWIG00000022221', 'ENSGWIG00000022224', 'pnpla6', 'atp5pd', 'ENSGWIG00000022530', 'caly', 'vps37a', 'micu3a', 'fgf20a', 'ENSGWIG00000003494', 'slc24a2', 'nansa', 'clta', 'gne', 'cplx2l', 'tmod1', 'ENSGWIG00000005881', 'abca1a', 'SGCZ', 'tusc3', 'tppp2', 'ENSGWIG00000021035', 'ENSGWIG00000021041', 'PITPNC1', 'psmd12', 'helz', 'cacng1a', 'CACNG4', 'PRKCA', 'ENSGWIG00000022112', 'ENSGWIG00000023369', 'slc25a10', 'ENSGWIG00000023372', 'si:dkey-27j5.5', 'mycbpap', 'b3gntl1', 'metrnlb', 'ENSGWIG00000000339', 'si:cabz01100188.1', 'crtac1a', 'pgam1b', 'exosc1', 'si:ch211-202h22.8', 'ENSGWIG00000001024', 'pkd1a', 'ENSGWIG00000001041', 'TNPO2', 'rln3b', 'rfx1b', 'dcaf15', 'khsrp', 'zgc:136864', 'mri1', 'CC2D1A', 'nanos3', 'ENSGWIG00000009734', 'ENSGWIG00000009748', 'slc26a11', 'sgsh', 'gnav1', 'npb', 'pcyt2', 'ENSGWIG00000010130', 'MYADML2', 'TMC6', 'syngr2b', 'cant1b', 'ENSGWIG00000010181', 'armh3', 'KCNIP2', 'jupa', 'si:dkey-28e7.3', 'ENSGWIG00000012748', 'krt98', 'si:ch211-156l18.7', 'SMAP1', 'cenpk', 'lbx1b', 'slc2a15b', 'fgf8b', 'poll', 'wbp1lb', 'cnnm2b', 'nt5c2b', 'inab', 'ENSGWIG00000016330', 'ENSGWIG00000016334', 'ccdc40', 'card14', 'ENSGWIG00000016352', 'ENSGWIG00000016354', 'cbx8b', 'si:dkey-31g6.6', 'dgke', 'trim25', 'rasal3', 'si:dkey-237i9.1', 'ENDOV', 'NPTX1', 'ENSGWIG00000000297', 'rptor', 'ENSGWIG00000000541', 'p4hb', 'ENSGWIG00000000864', 'ARHGDIA', 'ppp1r12c', 'QPRT', 'ENSGWIG00000000898', 'PLA2G12B', 'ENSGWIG00000000909', 'scrn2', 'sp6', 'psmb3', 'PIP4K2B', 'ENSGWIG00000001091', 'ENSGWIG00000001105', 'ENSGWIG00000001109', 'IGFBP4', 'SMARCE1', 'ENSGWIG00000001117', 'ENSGWIG00000001120', 'krt97', 'pcdh18a', 'ENSGWIG00000001127', 'pcdh10a', 'ENSGWIG00000001131', 'zgc:77739', 'chrnb3a', 'chrna6', 'chrnb5a', 'rnf38', 'zgc:195282', 'pi4k2b', 'htra3a', 'gpr78a', 'cpz', 'hmx4', 'hmx1', 'ENSGWIG00000001929', 'adra2c', 'lrpap1', 'ENSGWIG00000001944', 'rgs12b', 'ENSGWIG00000002056', 'ENSGWIG00000002353', 'grk4', 'ENSGWIG00000003006', 'ENSGWIG00000003051', 'ENSGWIG00000003063', 'prdx2', 'si:dkey-217d24.6', 'ptger1a', 'ilf3a', 'acp5a', 'syce2', 'slc44a2', 'ENSGWIG00000005241', 'micall1a', 'ENSGWIG00000005678', 'ENSGWIG00000005681', 'BMERB1', 'MARF1', 'NDE1', 'ENSGWIG00000010077', 'fopnl', 'abcc6a', 'notum2', 'rps15a', 'xylt1', 'nomo', 'ENSGWIG00000021901', 'ccdc85al', 'ENSGWIG00000021987', 'cep170ab', 'si:ch211-194e18.2', 'ENSGWIG00000021994', 'ENSGWIG00000022003', 'mettl26', 'ENSGWIG00000022013', 'zp3c', 'si:dkeyp-80c12.5', 'zgc:175214', 'pdcd11', 'taf5', 'elf2b', 'noctb', 'ENSGWIG00000022052', 'ENSGWIG00000022053', 'ENSGWIG00000022054', 'ENSGWIG00000022055', 'gli2b', 'CLASP1', 'vps16', 'arl11', 'abi3bpb', 'f2', 'f10', 'pcid2', 'si:ch211-132f19.7', 'ENSGWIG00000004456', 'pou3f3a', 'GABRB3', 'GABRG3', 'STEAP3', 'en1b', 'ENSGWIG00000004740', 'ENSGWIG00000004780', 'mgat5', 'sap18', 'ENSGWIG00000004892', 'ptger2a', 'txndc16', 'gpr137c', 'nudt15', 'si:dkey-1h24.2', 'ENSGWIG00000005039', 'fam155a', 'lig4', 'irs2a', 'ACP2', 'ENSGWIG00000006052', 'ENSGWIG00000006055', 'ENSGWIG00000006139', 'STRADB', 'trak2', 'zgc:92335', 'rnd3a', 'ENSGWIG00000006837', 'zranb3', 'tmem163b', 'ENSGWIG00000007194', 'ENSGWIG00000007209', 'ENSGWIG00000007253', 'ENSGWIG00000007269', 'ENSGWIG00000007494', 'ENSGWIG00000007505', 'bin1b', 'ENSGWIG00000009132', 'cars2', 'DDRGK1', 'chrm4a', 'mdka', 'ENSGWIG00000010163', 'U2', 'cryaa', 'ankrd10a', 'ddx4', 'UBE3A', 'anos1a', 'sts', 'ENSGWIG00000006303', 'pudp', 'nlgn4xa', 'prkx', 'mxra5a', 'cep70', 'ENSGWIG00000006322', 'ENSGWIG00000006325', 'plcd4a', 'fam207a', 'ADARB1', 'ENSGWIG00000007294', 'zgc:152904', 'ENSGWIG00000010099', 'jagn1b', 'zgc:172295', 'ENSGWIG00000011144', 'snx21', 'dock9b', 'slc4a10a', 'GCG', 'dpp4', 'col28a2b', 'hs6st3a', 'si:busm1-52i16.2', 'APP', 'adamts1', 'ADAMTS5', 'ENSGWIG00000026227', 'ENSGWIG00000026230', 'fundc1', 'caskb', 'GPR34', 'ENSGWIG00000002963', 'ENSGWIG00000003135', 'bco1l', 'LIPT1', 'ENSGWIG00000003631', 'gpd1c', 'ENSGWIG00000003671', 'ENSGWIG00000003764', 'ENSGWIG00000003918', 'ENSGWIG00000003921', 'ENSGWIG00000003928', 'ENSGWIG00000003952', 'pwp2h', 'atp7b', 'DMD', 'ENSGWIG00000012104', 'abi2b', 'tmem237b', 'mpp4l', 'ENSGWIG00000016131', 'mbd5', 'acvr2ab', 'bzw1b', 'si:ch211-246m6.4', 'si:ch211-246m6.5', 'mrpl39', 'rap2ab', 'mbnl2', 'ENSGWIG00000020921', 'ENSGWIG00000020935', 'nrp2a', 'abcb6a', 'raph1b', 'ENSGWIG00000023509', 'ENSGWIG00000023511', 'ENSGWIG00000023512', 'atp11a', 'tubgcp3', 'ENSGWIG00000024127', 'cab39l', 'mcf2la', 'ENSGWIG00000024500', 'ENSGWIG00000025058', 'GPR180', 'ENSGWIG00000002400', 'ENSGWIG00000002476', 'ENSGWIG00000007734', 'ENSGWIG00000007747', 'si:ch211-132g1.7', 'ENSGWIG00000007760', 'zgc:85722', 'ENSGWIG00000007783', 'ENSGWIG00000007787', 'etv5b', 'map4k4', 'chst10', 'rpl31', 'zc3h13', 'arhgef7b', 'ENSGWIG00000010286', 'ENSGWIG00000010319', 'sema5bb', 'kpnb3', 'atg13', 'ENSGWIG00000010766', 'slc19a1', 'ENSGWIG00000010772', 'gpbar1', 'aamp', 'HDAC4', 'ENSGWIG00000013842', 'ENSGWIG00000013896', 'TCEANC', 'ENSGWIG00000013907', 'ENSGWIG00000014774', 'ENSGWIG00000014784', 'ENSGWIG00000014791', 'ENSGWIG00000014910', 'ENSGWIG00000014931', 'FILIP1L', 'ENSGWIG00000015009', 'ccdc28a', 'ENSGWIG00000015013', 'ENSGWIG00000015093', 'atp1a1a.4', 'ENSGWIG00000021643', 'obsl1a', 'CTDSP1', 'ENSGWIG00000021651', 'ENSGWIG00000021653', 'stk11ip', 'igfbp2a', 'igfbp5a', 'ENSGWIG00000021677', 'ENSGWIG00000021829', 'ENSGWIG00000021830', 'mettl21a', 'CREB1', 'klf7a', 'ENSGWIG00000021835', 'erbb4a', 'ENSGWIG00000024821', 'lancl1', 'mylz3', 'acadl', 'ENSGWIG00000000151', 'ENSGWIG00000000243', 'ENSGWIG00000000255', 'gart', 'kcnh7', 'ENSGWIG00000000472', 'cobll1a', 'scn1lab', 'ENSGWIG00000001168', 'ENSGWIG00000001172', 'ENSGWIG00000001175', 'xirp2a', 'b3galt1a', 'ENSGWIG00000001178', 'abcb11b', 'atp5f1b', 'ENSGWIG00000001208', 'ENSGWIG00000001211', 'ndfip2', 'spry2', 'SLITRK1', 'ENSGWIG00000001223', 'ENSGWIG00000001228', 'slitrk6', 'ENSGWIG00000001235', 'ENSGWIG00000001237', 'MID1', 'ARHGAP6', 'FRMPD4', 'MSL3', 'ENSGWIG00000001324', 'ENSGWIG00000001332', 'ENSGWIG00000001336', 'ENSGWIG00000001338', 'ENSGWIG00000001340', 'ENSGWIG00000001342', 'zeb2b', 'ENSGWIG00000001348', 'ENSGWIG00000019257', 'si:ch211-186j3.6', 'dpy19l3', 'znf507', 'ENSGWIG00000019291', 'ENSGWIG00000019293', 'ENSGWIG00000019294', 'tshz3b', 'ENSGWIG00000019298', 'ENSGWIG00000019301', 'znf536', 'ENSGWIG00000019303', 'ccne1', 'ENSGWIG00000019341', 'cd81a', 'ENSGWIG00000019709', 'ENSGWIG00000020066', 'ENSGWIG00000020057', 'celf1', 'tollip', 'ENSGWIG00000020512', 'ENSGWIG00000020513', 'selenos', 'chsy1', 'lrrk1', 'ENSGWIG00000020807', 'slc7a10a', 'rhpn2', 'cep89', 'zgc:165481', 'abcc12', 'ENSGWIG00000021891', 'mctp2b', 'ENSGWIG00000021896', 'SLCO3A1', 'ENSGWIG00000000227', 'ENSGWIG00000000451', 'GLO1', 'ENSGWIG00000000524', 'ENSGWIG00000000532', 'ENSGWIG00000000540', 'COX5A', 'igf1rb', 'ap3s2', 'si:ch211-106j24.1', 'ENSGWIG00000000995', 'mespba', 'sv2ba', 'si:dkey-172h23.2', 'ntrk3b', 'ENSGWIG00000001155', 'ENSGWIG00000001158', 'ENSGWIG00000001160', 'mrpl46', 'si:cabz01113374.3', 'pnoca', 'kif13ba', 'ENSGWIG00000010653', 'tdp1', 'ENSGWIG00000010657', 'eprs1', 'ENSGWIG00000011283', 'ENSGWIG00000011306', 'ENSGWIG00000011329', 'ENSGWIG00000011330', 'vps35', 'ENSGWIG00000011339', 'ENSGWIG00000011340', 'ENSGWIG00000011342', 'ENSGWIG00000011344', 'cemip', 'abhd17c', 'arnt2', 'fah', 'ENSGWIG00000016848', 'zfand6', 'mthfs', 'si:cabz01029535.1', 'adamts17', 'cers3a', 'lins1', 'asb7', 'aldh1a3', 'ENSGWIG00000017036', 'mesd', 'ENSGWIG00000017049', 'cyp1a', 'ENSGWIG00000017054', 'CSK', 'cplx3b', 'ENSGWIG00000017213', 'ulk3', 'tars3', 'tm2d3', 'larp6a', 'ENSGWIG00000020999', 'THSD4', 'pepd', 'ENSGWIG00000021087', 'si:ch211-269c21.2', 'KCTD15', 'ENSGWIG00000003030', 'ss18l2', 'lsm14ab', 'si:ch211-79l17.1', 'gpib', 'wtip', 'cep83', 'cnep1r1', 'heatr3', 'tent4b', 'adcy7', 'brd7', 'nkd1', 'nod2', 'cylda', 'sall1a', 'tox3', 'ENSGWIG00000007870', 'ENSGWIG00000007873', 'chd9', 'aktip', 'ENSGWIG00000012624', 'fto', 'irx3a', 'ENSGWIG00000012637', 'irx5a', 'ENSGWIG00000012647', 'irx6a', 'mmp2', 'lpcat2', 'slc6a2', 'slc12a4', 'rpp25b', 'ENSGWIG00000020407', 'ENSGWIG00000020442', 'ENSGWIG00000020444', 'ENSGWIG00000020499', 'pkma', 'ENSGWIG00000022160', 'dhdhl', 'ENSGWIG00000022164', 'hcn4', 'rec114', 'nptnb', 'ENSGWIG00000022174', 'chrna5', 'ENSGWIG00000022198', 'ube2q1', 'SHF', 'cd276', 'ENSGWIG00000022257', 'insyn1', 'ENSGWIG00000022259', 'loxl1', 'ENSGWIG00000022261', 'scamp5b', 'igdcc4', 'UNC13C', 'onecut1', 'fam214a', 'myo5aa', 'gnb5b', 'trpm7', 'ENSGWIG00000026633', 'ENSGWIG00000026636', 'ENSGWIG00000026637', 'si:ch211-39k3.2', 'ENSGWIG00000026643', 'ENSGWIG00000026646', 'znf710b', 'si:ch211-107o10.3', 'ENSGWIG00000026687', 'wdr76', 'frmd5', 'cry5', 'ENSGWIG00000000177', 'ENSGWIG00000000181', 'nmbb', 'tspan3b', 'pstpip1b', 'ISL2', 'ENSGWIG00000000305', 'ENSGWIG00000000432', 'tln2a', 'ddx28', 'nfatc3a', 'esrp2', 'styx', 'ENSGWIG00000010283', 'ENSGWIG00000010284', 'mdga1', 'ENSGWIG00000010428', 'nudt19', 'slc7a9', 'si:ch73-167i17.6', 'zdhhc7', 'nip7', 'cdc42se2', 'ENSGWIG00000011610', 'ENSGWIG00000011612', 'ENSGWIG00000011615', 'lpin1', 'greb1', 'spata17', 'ENSGWIG00000011713', 'esrrga', 'ENSGWIG00000011725', 'kctd3', 'ENSGWIG00000013957', 'FZD3', 'si:dkey-57k2.7', 'ahctf1', 'kif28', 'kif26ba', 'SMYD3', 'ENSGWIG00000016945', 'tfb2m', 'itsn2b', 'ENSGWIG00000018495', 'ENSGWIG00000018497', 'adpgk2', 'ENSGWIG00000018529', 'ENSGWIG00000018532', 'galnt2', 'urb2', 'abcb10', 'rab4a', 'psmc3', 'rhoua', 'ppm1g', 'mis12', 'ENSGWIG00000019959', 'fam120b', 'ENSGWIG00000019988', 'DNA2', 'ENSGWIG00000020102', 'tspan15', 'tacr2', 'ENSGWIG00000020107', 'vps26a', 'ddx21', 'chs1', 'ENSGWIG00000000552', 'ENSGWIG00000000586', 'ENSGWIG00000000887', 'rnaset2', 'rps6ka2', 'immp1l', 'elp4', 'pax6b', 'rcn1', 'wt1b', 'prrg4', 'qser1', 'depdc7a', 'cstf3', 'hipk3b', 'kiaa1549lb', 'spon1a', 'syt9a', 'ppfibp2a', 'MAF', 'ENSGWIG00000006287', 'chchd2', 'CDYL2', 'cenpn', 'cmc2', 'atmin', 'ENSGWIG00000006300', 'gse1', 'dusp22a', 'irf8', 'ENSGWIG00000006319', 'foxf1', 'ENSGWIG00000006407', 'mphosph10', 'rplp2l', 'bdnf', 'KIF18A', 'ENSGWIG00000009510', 'mettl15', 'lgr4', 'mybpc3', 'spi1b', 'slc39a13', 'si:ch211-150g13.3', 'gas2b', 'SLC17A6', 'ano5b', 'igdcc3', 'ENSGWIG00000017487', 'cilp', 'strc1', 'ctdspl2b', 'casc4', 'FAM189A1', 'tjp1a', 'terb2', 'apba2b', 'syt7b', 'polr2l', 'dagla', 'ext2', 'alx4a', 'caprin1a', 'nucb2a', 'plekha7b', 'sox6', 'ENSGWIG00000024774', 'cyp2r1', 'pde3b', 'copb1', 'rras2', 'tub', 'ENSGWIG00000024802', 'lmo1', 'stk33', 'trim66', 'dennd2b', 'tmem9b', 'scube2', 'tmem41b', 'sbf2', 'ampd3a', 'lyve1a', 'mrvi1', 'ctr9', 'eif4g2a', 'galnt18b', 'ENSGWIG00000018698', 'ENSGWIG00000018706', 'TEAD1', 'mical2a', 'ENSGWIG00000019008', 'usp47', 'vat1l', 'bola3', 'nfat5b', 'kars1', 'gcshb', 'rpl13', 'SNORD68', 'cpne7', 'zgc:162592', 'cyb5b', 'ENSGWIG00000023880', 'dhx38', 'ENSGWIG00000023881', 'zfhx3', 'crispld2', 'marveld3', 'usp10', 'PHLPP2', 'ap1g1', 'ENSGWIG00000024682', 'atxn1l', 'myo1ea', 'rnf111', 'sltm', 'mindy2', 'ENSGWIG00000025615', 'aqp9b', 'aldh1a2', 'polr2m', 'myzap', 'cgnl1', 'tcf12', 'znf280d', 'mns1', 'nedd4a', 'ENSGWIG00000026030', 'PRTG', 'ENSGWIG00000026033', 'pigb', 'rab27a', 'ENSGWIG00000026037', 'ENSGWIG00000026038', 'tradd', 'gnao1b', 'cog4', 'slc38a8b', 'cmtr2', 'calb2b', 'ENSGWIG00000026154', 'ENSGWIG00000026156', 'furina', 'fes', 'rhcgl1', 'kif7', 'ENSGWIG00000026288', 'ENSGWIG00000003181', 'ENSGWIG00000003190', 'ENSGWIG00000003199', 'tcf25', 'mc1r', 'ist1', 'dhodh', 'dpep1', 'ankrd11', 'spg7', 'cdh15', 'ACSF3', 'cbfa2t3', 'trappc2l', 'cmtm3', 'nae1', 'ENSGWIG00000010908', 'dync1li2', 'cmtm4', 'ca7', 'rrad', 'nsun3', 'epha6', 'kctd12.2', 'gpc6a', 'gpc5a', 'ENSGWIG00000014031', 'ENSGWIG00000014033', 'ercc5', 'blzf1', 'ENSGWIG00000014069', 'tpp2', 'rasa3', 'pros1', 'ENSGWIG00000015546', 'ENSGWIG00000015551', 'tmco3', 'dcun1d2b', 'adprhl1', 'GRTP1', 'cul4a', 'lims1', 'nkpd1', 'ENSGWIG00000018771', 'ENSGWIG00000018774', 'pcdh17', 'diaph3', 'tdrd3', 'PCDH20', 'pcdh9', 'pou4f1', 'ENSGWIG00000018942', 'rbm26', 'cyp27c1', 'gtf2f2a', 'ENSGWIG00000019046', 'baz2ba', 'march7', 'ly75', 'pla2r1', 'itgb6', 'rbms1a', 'ENSGWIG00000001069', 'fn1b', 'atic', 'mnx2b', 'stk16', 'pcnp', 'RS1', 'cdkl5', 'nhsb', 'ENSGWIG00000002084', 'ace2', 'pir', 'vegfd', 'grpr', 'spata5l1', 'nnt2', 'fgf7', 'cops2', 'SECISBP2L', 'SHC4', 'ENSGWIG00000006295', 'slc12a1', 'myef2', 'si:dkeyp-86f7.4', 'sema6dl', 'dmxl2', 'gldn', 'cyp19a1a', 'tnfaip8l3', 'scg3', 'lysmd2', 'leo1', 'mapk6', 'USP50', 'ENSGWIG00000022523', 'hdc', 'ENSGWIG00000022690', 'gatm', 'pde8a', 'ENSGWIG00000023981', 'HOMER2', 'ENSGWIG00000023996', 'BTBD1', 'TM6SF1', 'tmem39a', 'ENSGWIG00000024299', 'ENSGWIG00000024354', 'rxfp3.3a2', 'itga11a', 'coro2ba', 'anp32a', 'glceb', 'KIF23', 'ENSGWIG00000024494', 'TLE3', 'uacab', 'ENSGWIG00000025736', 'pias1a', 'skor1a', 'map2k5', 'SMAD3', 'smad6a', 'tmem208', 'abhd2a', 'mibp2', 'bloc1s6', 'slc30a4', 'zgc:153317', 'arfgap2', 'lctla', 'dis3l', 'MEGF11', 'rab11a', 'dapk2a', 'ENSGWIG00000006340', 'ccnb2', 'pole3', 'rpl12', 'kcng4b', 'ENSGWIG00000006932', 'ENSGWIG00000006937', 'ENSGWIG00000006999', 'gtf2a2', 'ENSGWIG00000007011', 'ENSGWIG00000007025', 'ENSGWIG00000007029', 'kifc3', 'ENSGWIG00000009875', 'katnb1', 'gpatch1', 'cebpa', 'cebpg', 'ENSGWIG00000010036', 'fhod1', 'slc9a5', 'plekhg4', 'CIB2', 'hsd11b2', 'ENSGWIG00000011057', 'NR2E3', 'ENSGWIG00000011312', 'mpped2a', 'nav2a', 'dbx1a', 'slc6a5', 'ENSGWIG00000012438', 'NELL1', 'ENSGWIG00000013125', 'ENSGWIG00000013126', 'ENSGWIG00000013128', 'ENSGWIG00000013138', 'ENSGWIG00000013148', 'ppfia1', 'cttn', 'rbpms2a', 'psma4', 'crabp1a', 'ENSGWIG00000023328', 'si:ch211-288d18.1', 'brsk2b', 'ENSGWIG00000000463', 'shank2a', 'ENSGWIG00000000581', 'crtc3', 'chst1', 'apip', 'ENSGWIG00000000713', 'slc1a2a', 'lrrc4cb', 'ENSGWIG00000000721', 'ttc17', 'kcnq1.1', 'lingo1b', 'luzp2', 'ENSGWIG00000000858', 'FIBIN', 'bbox1', 'madd', 'nr1h3', 'arl14ep', 'kcna4', 'cd151l', 'ddb1', 'tspan4a', 'si:ch211-71n6.4', 'large2', 'ENSGWIG00000004595', 'ZNF319', 'si:ch211-146m13.3', 'sirt3', 'ENSGWIG00000006731', 'ctsd', 'ENSGWIG00000007128', 'ENSGWIG00000007133', 'ENSGWIG00000007140', 'ppip5k1a', 'map1aa', 'ENSGWIG00000011101', 'tubgcp4', 'cdkn2aip', 'MAPK8IP1', 'ENSGWIG00000011404', 'ENSGWIG00000011425', 'ENSGWIG00000007219', 'ENSGWIG00000007221', 'gmds', 'foxq1b', 'exoc2', 'ENSGWIG00000007295', 'bphl', 'SLC22A23', 'dspb', 'tbc1d7', 'phactr1', 'sirt5', 'adcy1a', 'ENSGWIG00000007341', 'ENSGWIG00000007342', 'igfbp1a', 'igfbp3', 'ENSGWIG00000007345', 'ENSGWIG00000007346', 'ENSGWIG00000007371', 'C8orf82', 'ENSGWIG00000007718', 'ENSGWIG00000007719', 'fetub', 'ENSGWIG00000007721', 'rgs5a', 'rgs16', 'ENSGWIG00000007743', 'rgs8', 'brinp3a.1', 'ENSGWIG00000007754', 'brinp3a.2', 'ENSGWIG00000007798', 'ENSGWIG00000007815', 'RGS1', 'si:ch211-117l17.5', 'si:ch211-117l17.6', 'rgs2', 'ENSGWIG00000007901', 'b3galt2', 'ENSGWIG00000007906', 'ENSGWIG00000007910', 'ENSGWIG00000007970', 'bcar3', 'hps3', 'ENSGWIG00000004600', 'ENSGWIG00000004617', 'ENSGWIG00000004620', 'ENSGWIG00000004626', 'ENSGWIG00000005481', 'si:dkey-211e20.10', 'mccc1', 'ENSGWIG00000013266', 'zmat3', 'ENSGWIG00000013269', 'fam102bb', 'ENSGWIG00000013276', 'ENSGWIG00000013277', 'ENSGWIG00000013279', 'ENSGWIG00000013281', 'ENSGWIG00000013283', 'fam183a', 'ENSGWIG00000013835', 'ENSGWIG00000013861', 'lrp8', 'ENSGWIG00000013931', 'ENSGWIG00000013932', 'ENSGWIG00000013935', 'ebna1bp2', 'TMEM125', 'ap1m3', 'ENSGWIG00000014403', 'tie1', 'DDAH1', 'ccn1', 'ZNHIT6', 'ENSGWIG00000016363', 'ENSGWIG00000016365', 'zgc:65873', 'SYDE2', 'ENSGWIG00000016370', 'si:ch211-262h13.5', 'ENSGWIG00000016505', 'gpc1a', 'ENSGWIG00000017075', 'ENSGWIG00000017092', 'ENSGWIG00000017096', 'ENSGWIG00000017636', 'meltf', 'si:ch211-162e15.3', 'ENSGWIG00000024449', 'ENSGWIG00000024451', 'ENSGWIG00000024483', 'cp', 'ENSGWIG00000004292', 'ENSGWIG00000004415', 'ENSGWIG00000004416', 'ENSGWIG00000004418', 'lepr', 'ak4', 'MIR101', '1', 'jak1', 'cachd1', 'ENSGWIG00000017728', 'ror1', 'pgm1', 'alg6', 'foxd3', 'atg4c', 'angptl3', 'dock7', 'usp1', 'sgip1a', 'pde4ba', 'ENSGWIG00000023956', 'dda1', 'ano8b', 'gtpbp3', 'plvapb', 'babam1', 'ENSGWIG00000023986', 'nr2f6b', 'si:ch73-61d6.3', 'csnk1g2b', 'rexo1', 'mknk2b', 'zap70', 'adamts10', 'ENSGWIG00000025106', 'myo1f', 'acer1', 'ENSGWIG00000025108', 'ENSGWIG00000025111', 'rfx2', 'fcho1', 'prg4b', 'tprb', 'pdcb', 'ptgs2b', 'pla2g4ab', 'arpc5b', 'ENSGWIG00000025735', 'ivns1abpa', 'swt1', 'ENSGWIG00000025740', 'abhd8b', 'ptprc', 'nek7', 'dennd1b', 'crb1', 'ENSGWIG00000025959', 'si:dkey-171o17.8', 'RABGAP1L', 'ENSGWIG00000026054', 'arrdc2', 'homer3b', 'ddx49', 'ENSGWIG00000026089', 'PGPEP1', 'JUND', 'rab3ab', 'mpv17l2', 'ENSGWIG00000026146', 'ENSGWIG00000026168', 'kank3', 'ENSGWIG00000000522', 'dazap1', 'gamt', 'ENSGWIG00000000982', 'ctdspl3', 'arid3a', 'r3hdm4', 'si:ch73-40i7.5', 'ENSGWIG00000001302', 'ENSGWIG00000001303', 'ENSGWIG00000001306', 'slc35e1', 'tmem38a', 'comp', 'CRTC1', 'ENSGWIG00000002093', 'mast3b', 'ENSGWIG00000002127', 'ENSGWIG00000002128', 'ENSGWIG00000002130', 'slc5a5', 'uhrf1', 'kdm4b', 'ptprsa', 'gipc3', 'si:zfos-943e10.1', 'wdr47b', 'camsap2a', 'kif14', 'ENSGWIG00000005767', 'ENSGWIG00000005770', 'ENSGWIG00000005771', 'nr5a2', 'ENSGWIG00000005850', 'zgc:85936', 'alpi.2', 'alpi.1', 'ENSGWIG00000006451', 'ENSGWIG00000006452', 'nppc', 'ENSGWIG00000006458', 'b3gnt7', 'armc9', 'ENSGWIG00000006753', 'ENSGWIG00000006862', 'ENSGWIG00000010056', 'ENSGWIG00000010066', 'ENSGWIG00000010071', 'ENSGWIG00000010072', 'ENSGWIG00000010073', 'rubcn', 'ENSGWIG00000010080', 'ENSGWIG00000010081', 'PFN2', 'rnf13', 'ppp1r2', 'acap2', 'psmd1', 'htr2b', 'ENSGWIG00000017009', 'her6', 'opa1', 'atp13a3', 'ENSGWIG00000017262', 'lsg1', 'ENSGWIG00000017304', 'xxylt1', 'ankrd13c', 'cthl', 'ptger3', 'zranb2', 'negr1', 'vtg3', 'adgrl4', 'adgrl2a', 'ENSGWIG00000022048', 'ttll7', 'ENSGWIG00000022445', 'dnase2b', 'uox', 'GNG5', 'si:dkey-205k8.5', 'ENSGWIG00000022806', 'ENSGWIG00000025904', 'ENSGWIG00000026088', 'mcoln2', 'ftsj3', 'plaat1', 'mfn1b', 'gnb4b', 'actl6a', 'aldh7a1', 'ndufb5', 'usp13', 'pex5la', 'ENSGWIG00000026403', 'ENSGWIG00000026406', 'ENSGWIG00000026408', 'p3h2', 'TMEM47', 'tab3', 'nexn', 'zgc:153738', 'ENSGWIG00000026426', 'ENSGWIG00000026432', 'rnf220b', 'dmap1', 'timm44', 'ctxn1', 'fbn2b', 'cers4a', 'ENSGWIG00000026469', 'enc3', 'arhgef18b', 'insrb', 'use1', 'ENSGWIG00000026491', 's1pr4', 'ncln', 'cers1', 'tnfaip8l1', 'ENSGWIG00000026516', 'ENSGWIG00000026526', 'zfr2', 'rx1', 'sin3b', 'ENSGWIG00000000375', 'xpr1a', 'acbd6', 'lhx4', 'qsox1', 'cep350', 'cacna1ea', 'ENSGWIG00000008407', 'ENSGWIG00000008408', 'soat1', 'ENSGWIG00000008436', 'abl2', 'osbpl9', 'ENSGWIG00000008593', 'calr', 'tor3a', 'eps15', 'ttc39a', 'ENSGWIG00000010909', 'faf1', 'dmrta2', 'elavl4', 'agbl4', 'bend5', 'gclm', 'ccl25b', 'ENSGWIG00000011366', 'slc1a6', 'tax1bp3', 'RAB8A', 'ENSGWIG00000011549', 'tpm4a', 'si:ch1073-396h14.1', 'ENSGWIG00000011678', 'lpl', 'safb', 'hdgfl2', 'ap3d1', 'cdc34a', 'C2CD4C', 'gna15.4', 'ENSGWIG00000013242', 'tle5', 'tle2b', 'tle2a', 'plpp2b', 'mier2', 'ENSGWIG00000014326', 's1pr3a', 'shdb', 'ankrd24', 'odf3l2b', 'ENSGWIG00000014353', 'ENSGWIG00000014360', 'foxe3', 'FOXD2', 'trabd2b', 'ENSGWIG00000014368', 'ENSGWIG00000014378', 'ENSGWIG00000000474', 'atf6', 'ENSGWIG00000000710', 'fyco1a', 'GHRHR', 'ENSGWIG00000000749', 'ENSGWIG00000000765', 'faah', 'mknk1', 'mob3c', 'elovl1b', 'zfyve9b', 'st6galnac5b', 'slc44a5b', 'lhx8b', 'erich3', 'ENSGWIG00000001106', 'tnni3k', 'acot11b', 'ssbp3b', 'dio1', 'glis1b', 'elovl8b', 'rorca', 'sema6bb', 'zgc:77486', 'abhd17ab', 'rgl1', 'colgalt2', 'usp46', 'rasl11b', 'scfd2', 'lnx1', 'gsx2', 'pdgfra', 'kita', 'ENSGWIG00000007773', 'kdr', 'CLOCK', 'ENSGWIG00000012358', 'exoc1', 'ENSGWIG00000012498', 'cep135', 'chst14', 'ENSGWIG00000012507', 'si:dkeyp-117h8.2', 'paics', 'ppat', 'nrl', 'ENSGWIG00000013122', 'rabggta', 'tgm1l2', 'ENSGWIG00000015324', 'osbpl1a', 'VPS4B', 'plekhb2', 'pigx', 'ENSGWIG00000018960', 'ENSGWIG00000018961', 'ENSGWIG00000018962', 'usp24', 'bloc1s2', 'plpp3', 'ENSGWIG00000000220', 'dab1a', 'ENSGWIG00000000364', 'oma1', 'mysm1', 'suco', 'ENSGWIG00000000631', 'ENSGWIG00000000657', 'rangrf', 'vamp4', 'myoc', 'prrc2c', 'si:dkey-239i20.2', 'prdx6', 'caiap', 'faslg', 'fam20b', 'ralgps2', 'angptl1a', 'coa7', 'zyg11', 'si:dkey-52j6.3', 'ENSGWIG00000001733', 'si:ch73-130a3.4', 'ENSGWIG00000001735', 'ipo13', 'pfas', 'cyp2ad2', 'ENSGWIG00000005188', 'ENSGWIG00000005583', 'il12rb2', 'serbp1a', 'ddr2a', 'uap1', 'ENSGWIG00000007555', 'ubxn6', 'chaf1a', 'scamp4', 'si:ch211-129c21.1', 'ENSGWIG00000007575', 'ENSGWIG00000007663', 'rab11ba', 'pip5k1ca', 'gpx4a', 'arhgap45b', 'tjp3', 'si:ch73-63e15.2', 'tm6sf2', 'hapln4', 'ENSGWIG00000008539', 'nr2c2ap', 'borcs8', 'si:ch211-125m10.6', 'mef2b', 'tmem161a', 'ENSGWIG00000008568', 'slc25a42', 'mau2', 'gatad2ab', 'yjefn3', 'cilp2', 'si:ch211-212d10.2', 'ENSGWIG00000008632', 'ENSGWIG00000000162', 'midn', 'cbarpb', 'fstl3', 'RNF126', 'bsg', 'ENSGWIG00000000249', 'polrmt', 'ENSGWIG00000000443', 'ENSGWIG00000000462', 'ENSGWIG00000000484', 'hsd11b1la', 'rgmd', 'celf5b', 'ENSGWIG00000000510', 'creb3l3a', 'ENSGWIG00000000527', 'EEF2', 'ENSGWIG00000000668', 'hmg20b', 'unc13a', 'mbd3a', 'tcf3b', 'zbtb7a', 'pias4a', 'foxq2', 'onecut3a', 'atp8b3', 'lingo3a', 'ENSGWIG00000019368', 'ENSGWIG00000019371', 'org', 'ENSGWIG00000019375', 'oaz1a', 'amh', 'dot1l', 'ell', 'fkbp8', 'SSBP4', 'tmem59l', 'ENSGWIG00000019832', 'rex1bd', 'fam78bb', 'slc35a3a', 'notch2', 'sec22bb', 'npl', 'edem3', 'zgc:92140', 'ENSGWIG00000020084', 'dcun1d4', 'ENSGWIG00000020093', 'ociad2', 'ENSGWIG00000020098', 'fryl', 'slc10a4', 'slain2', 'palld', 'sh3rf1', 'nek1', 'fbxl5', 'ENSGWIG00000020560', 'cpeb2', 'ndnf', 'prdm5', 'ENSGWIG00000020613', 'ENSGWIG00000020615', 'slc34a2b', 'ENSGWIG00000020676', 'psmb11b', 'cdh24a', 'si:dkey-194e6.1', 'reps2', 'ENSGWIG00000004581', 'ENSGWIG00000004586', 'nhsa', 'RAI2', 'ENSGWIG00000004614', 'CDKL5', 'zgc:162858', 'SHROOM2', 'ENSGWIG00000006238', 'zgc:153311', 'wwc3', 'ENSGWIG00000006245', 'chd1l', 'nit2', 'tomm70a', 'tmem45a', 'si:dkey-66a8.7', 'plcxd1', 'jade3', 'rp2', 'slc9a7', 'tubgcp5', 'cyfip1', 'ENSGWIG00000013295', 'nipa2', 'ENSGWIG00000013341', 'nipa1', 'oca2', 'gabra5', 'gabrb3', 'atp10a', 'ube3a', 'cnga3a', 'tmem131', 'si:ch211-201h21.5', 'bivm', 'zgc:172121', 'nme7', 'atp1b1a', 'dpt', 'zbtb11', 'tp63', 'tprg1', 'ENSGWIG00000015237', 'ENSGWIG00000015238', 'lpp', 'ENSGWIG00000015242', 'bcl6aa', 'podn', 'ENSGWIG00000015278', 'tmco1', 'ENSGWIG00000005999', 'ENSGWIG00000006035', 'ENSGWIG00000006058', 'ENSGWIG00000006060', 'ENSGWIG00000006062', 'dr1', 'scinla', 'RPL5', 'evi5b', 'GFI1', 'glmnb', 'si:ch73-389b16.2', 'ephx4', 'tgfbr3', 'barhl2', 'ENSGWIG00000009438', 'ENSGWIG00000009439', 'lrrc8db', 'lrrc8c', 'kyat3', 'ENSGWIG00000010721', 'lmo4b', 'hs2st1b', 'SOX14', 'hdlbpa', 'farp2', 'ENSGWIG00000015078', 'boka', 'dtymk', 'agxta', 'zgc:103559', 'TTC39C', 'ENSGWIG00000016759', 'ENSGWIG00000018465', 'espnla', 'klhl30', 'sned1', 'ryk', 'rab6ba', 'ENSGWIG00000018765', 'ENSGWIG00000018748', 'ENSGWIG00000018802', 'ddx10', 'ENSGWIG00000018808', 'hyi', 'ptprfa', 'kdm4aa', 'artna', 'ENSGWIG00000025213', 'ENSGWIG00000025214', 'ENSGWIG00000025219', 'ENSGWIG00000025220', 'ENSGWIG00000025216', 'ENSGWIG00000025221', 'ENSGWIG00000000183', 'inpp5d', 'gpr17', 'ENSGWIG00000000442', 'sap130a', 'amotl2a', 'cep63', 'im:7152348', 'ENSGWIG00000000975', 'arhgap39', 'EPS15L1', 'ENSGWIG00000001006', 'axdnd1', 'ENSGWIG00000001008', 'cherp', 'ENSGWIG00000001044', 'ENSGWIG00000001045', 'ENSGWIG00000001046', 'ENSGWIG00000001047', 'ENSGWIG00000001048', 'NAALADL2', 'ENSGWIG00000001057', 'nlgn1', 'ENSGWIG00000001097', 'ENSGWIG00000001098', 'ect2', 'mcf2l2', 'b3gnt5a', 'rfc4', 'ENSGWIG00000007472', 'ENSGWIG00000007475', 'ENSGWIG00000008391', 'ccndbp1', 'ENSGWIG00000008400', 'ENSGWIG00000008413', 'rims4', 'abraa', 'lsm3', 'ENSGWIG00000008453', 'grip2b', 'zgc:171566', 'PTPRT', 'ENSGWIG00000021445', 'chd6', 'lmcd1', 'ENSGWIG00000000153', 'blcap', 'ripor3', 'ergic3', 'mych', 'mtss1', 'ENSGWIG00000000388', 'ENSGWIG00000000406', 'ENSGWIG00000000417', 'ENSGWIG00000000423', 'ENSGWIG00000000427', 'ENSGWIG00000000534', 'ENSGWIG00000000560', 'ENSGWIG00000000561', 'ENSGWIG00000000574', 'ENSGWIG00000000595', 'ENSGWIG00000000636', 'ENSGWIG00000000641', 'ENSGWIG00000000644', 'ENSGWIG00000000662', 'ENSGWIG00000000694', 'ENSGWIG00000000688', 'ENSGWIG00000000679', 'ENSGWIG00000000702', 'ENSGWIG00000000708', 'pck1', 'ENSGWIG00000000827', 'raly', 'ENSGWIG00000000830', 'pxk', 'pdhb', 'ENSGWIG00000001134', 'si:dkeyp-87e7.4', 'stab1', 'nisch', 'hgh1', 'ippk', 'aspn', 'ENSGWIG00000001196', 'ogna', 'nol8', 'gpr37l1a', 'atp2b4', 'rbm12', 'cpne1', 'unm_sa1614', 'st3gal8', 'ENSGWIG00000007565', 'pnp4a', 'trib3', 'ENSGWIG00000000134', 'ENSGWIG00000000145', 'ENSGWIG00000000176', 'draxin', 'rpl22', 'chd5', 'arhgef10la', 'ENSGWIG00000001413', 'igsf21a', 'klhdc7a', 'ENSGWIG00000001424', 'dhrs3b', 'ENSGWIG00000001428', 'lrrc38b', 'ephb2b', 'lactbl1b', 'nmur3', 'ENSGWIG00000005013', 'alpl', 'ece1', 'eif4g3a', 'hp1bp3', 'zgc:158258', 'ENSGWIG00000017720', 'PLXNA2', 'si:ch211-286o17.1', 'ENSGWIG00000022343', 'ENSGWIG00000022344', 'ENSGWIG00000022350', 'magi3b', 'padi2', 'si:dkey-166d12.2', 'ENSGWIG00000022680', 'slc2a1b', 'ENSGWIG00000023209', 'ENSGWIG00000023211', 'anks1ab', 'SPDEF', 'pacsin1b', 'rps10', 'nudt3b', 'grm4', 'ENSGWIG00000023400', 'ENSGWIG00000023402', 'igfn1.4', 'ENSGWIG00000023405', 'SYT2', 'mybphb', 'nav1b', 'tnnt2b', 'pkp1b', 'tead3b', 'tulp1b', 'ADORA3', 'rplp2', 'kcna2a', 'KCNA3', 'ngfb', 'sycp1', 'slc16a1a', 'si:dkey-238f9.1', 'ppm1j', 'mov10a', 'ENSGWIG00000005232', 'ncoa5', 'ENSGWIG00000007499', 'ENSGWIG00000017306', 'si:ch211-81n22.1', 'cyp24a1', 'manbal', 'SRC', 'apcdd1l', 'VAPB', 'rab22a', 'gpd1b', 'ENSGWIG00000018806', 'rdh5', 'itcha', 'ENSGWIG00000025002', 'eya2', 'rtf2', 'gcnt7', 'tfap2c', 'ENSGWIG00000025131', 'cdh22', 'znfx1', 'cbfa2t2', 'ENSGWIG00000025447', 'tox2', 'st7l', 'wnt2ba', 'cttnbp2nla', 'KCND3', 'INKA2', 'ENSGWIG00000025555', 'timm17a', 'ipo9', 'ENSGWIG00000025685', 'ppp1r16b', 'NDUFA4L2', 'shmt2', 'NXPH4', 'STAC3', 'r3hdm2', 'INHBC', 'gli1', 'ENSGWIG00000025812', 'ENSGWIG00000025813', 'mars1', 'ddit3', 'dctn2', 'KIF5A', 'ENSGWIG00000025830', 'ENSGWIG00000025837', 'sp7', 'sp1', 'PCBP2', 'MAP3K12', 'myg1', 'zgc:172352', 'ENSGWIG00000005821', 'ITGA7', 'ENSGWIG00000006059', 'ENSGWIG00000006065', 'alas1', 'dusp7', 'rpl29', 'iars1', 'atp2b2', 'slc6a11b', 'slc6a1b', 'HRH1', 'atg7', 'vgll4b', 'tamm41', 'ENSGWIG00000021449', 'ENSGWIG00000022359', 'iffo2b', 'ENSGWIG00000022362', 'megf6b', 'kcnab2a', 'park7', 'pax7b', 'ENSGWIG00000022666', 'errfi1a', 'tas1r3', 'ccdc114', 'ENSGWIG00000022697', 'slc45a1', 'mmel1', 'miip', 'ENSGWIG00000022727', 'trim62.1', 'ENSGWIG00000022730', 'ENSGWIG00000022736', 'si:dkey-264d12.4', 'ppcs', 'FOXJ3', 'ENSGWIG00000022758', 'kaznb', 'tmem51b', 'skib', 'ENSGWIG00000022764', 'si:dkey-183j2.10', 'ENSGWIG00000022770', 'wdr82', 'cish', 'hemk1', 'nprl2', 'ENSGWIG00000022789', 'RASSF1', 'tusc2a', 'hyal1', 'cacna2d2a', 'ENSGWIG00000023061', 'si:ch211-161c3.5', 'ENSGWIG00000023089', 'h6pd', 'epha2a', 'ddi2', 'ENSGWIG00000023175', 'edem1', 'arl8ba', 'gnat1', 'sema3fa', 'rbm6', 'mst1ra', 'uba7', 'gnai2b', 'slc38a3b', 'rbm5', 'TEX264', 'grm2b', 'rrp9', 'zgc:172270', 'oxtrl', 'ENSGWIG00000013292', 'ptgis', 'kcnb1', 'rdh20', 'camk1a', 'si:dkeyp-67f1.2', 'ENSGWIG00000020816', 'ca16b', 'cadpsb', 'ENSGWIG00000022267', 'ENSGWIG00000022269', 'slc6a22.2', 'slc6a22.1', 'fkbp5', 'ENSGWIG00000022719', 'hdhd3', 'zgc:172271', 'zgc:112163', 'tspo', 'frs3', 'ENSGWIG00000023027', 'slc6a11a', 'eno1b', 'rereb', 'ENSGWIG00000025431', 'kbtbd12', 'slc26a6l', 'p4htm', 'myhb', 'ENSGWIG00000025472', 'ENSGWIG00000025474', 'ENSGWIG00000025475', 'cacna2d2b', 'acy1', 'HYAL2', 'tusc2b', 'rad54l2', 'bap1', 'bsnb', 'ENSGWIG00000025753', 'erc2', 'ENSGWIG00000025791', 'wnk2', 'bicd2', 'ENSGWIG00000025797', 'tkta', 'cacna1da', 'parapinopsinb', 'cacna2d3', 'ENSGWIG00000025975', 'tmcc1b', 'ENSGWIG00000025977', 'ENSGWIG00000025978', 'qrich1', 'ENSGWIG00000025981', 'ENSGWIG00000004757', 'nckipsd', 'ENSGWIG00000005003', 'si:ch1073-335m2.2', 'fblim1', 'fbxo42', 'necap2', 'ENSGWIG00000005892', 'ENSGWIG00000005898', 'MFAP2', 'SDHB', 'ENSGWIG00000005940', 'ora3', 'ENSGWIG00000005951', 'ENSGWIG00000005952', 'NBL1', 'ENSGWIG00000005961', 'capzb', 'celsr2', 'dstyk', 'ccnd3', 'usp49', 'tmcc2', 'cdkn1a', 'SRSF3', 'AHCYL1', 'strip1', 'SLC6A17', 'ENSGWIG00000010216', 'ENSGWIG00000010228', 'ENSGWIG00000010257', 'ENSGWIG00000010436', 'si:ch211-103a14.5', 'irf6', 'eps8l3a', 'ampd2a', 'GPR61', 'ENSGWIG00000012975', 'ENSGWIG00000012991', 'thumpd3', 'srgap3', 'rad18', 'oxtr', 'cav3', 'gpr61l', 'grm2a', 'tex264a', 'ENSGWIG00000013672', 'usp4', 'slc38a3a', 'gnai2a', 'inka1a', 'ENSGWIG00000020421', 'ENSGWIG00000020426', 'ENSGWIG00000020427', 'ip6k2b', 'ENSGWIG00000020446', 'MAPKAPK3', 'pfkfb4b', 'arpc4', 'RAB7A', 'GATA2', 'eefsec', 'ruvbl1', 'prkcda', 'ENSGWIG00000000717', 'ccdc174', 'ENSGWIG00000000740', 'crbn', 'erbb3a', 'ENSGWIG00000012836', 'rps26', 'ENSGWIG00000012839', 'rhebl1', 'IKZF4', 'ENSGWIG00000012941', 'b4galnt1b', 'os9', 'apof', 'ENSGWIG00000012987', 'ENSGWIG00000012988', 'endou', 'tsfm', 'smarcc2', 'c1ql4b', 'pfkmb', 'ENSGWIG00000014715', 'espl1', 'atf7b', 'spag6', 'dip2bb', 'scn8ab', 'ENSGWIG00000018918', 'slc4a8', 'ttll3', 'BHLHE40', 'itpr1a', 'sumf1', 'ENSGWIG00000022032', 'si:dkey-28n18.9', 'ENSGWIG00000022231', 'ENSGWIG00000022303', 'rbms2a', 'faim2b', 'nr1d4b', 'rargb', 'calcoco1b', 'ENSGWIG00000022426', 'ENSGWIG00000022432', 'ENSGWIG00000022455', 'osgn1', 'ENSGWIG00000022470', 'mettl1', 'cyp27b1', 'suox', 'mmp19', 'ENSGWIG00000004733', 'ENSGWIG00000004835', 'prph', 'pmela', 'sys1', 'arfgap1', 'ENSGWIG00000008157', 'slc6a14', 'il19l', 'il10', 'mapkapk2b', 'dyrk3', 'si:dkey-85p17.3', 'ikbke', 'srgap2', 'zgc:101564', 'foxp4', 'si:dkey-91m3.1', 'ENSGWIG00000009581', 'ENSGWIG00000009578', 'tfeb', 'ppfia4', 'myog', 'mdm4', 'ENSGWIG00000012912', 'ENSGWIG00000014195', 'psma5', 'per3', 'ENSGWIG00000014224', 'si:ch211-160o17.2', 'dnajc14', 'nab2', 'ENSGWIG00000014238', 'ENSGWIG00000014239', 'NR4A1', 'dgkab', 'ENSGWIG00000015252', 'slc11a2', 'ENSGWIG00000021078', 'npepl1', 'kdm5ba', 'rabif', 'inavab', 'gpr25', 'gdf11', 'avil', 'ENSGWIG00000024639', 'olfml3b', 'idh3b', 'ptpra', 'cd8b', 'ENSGWIG00000025510', 'pcsk1', 'ENSGWIG00000025596', 'ell2', 'si:dkey-6b12.5', 'ube2r2', 'ubap1', 'dcaf12', 'ubap2a', 'ENSGWIG00000025622', 'SNORD57', 'zgc:64106', 'ENSGWIG00000004738', 'aqp7', 'ENSGWIG00000004816', 'celf4', 'myorg', 'zgc:109965', 'fam219aa', 'dnai1.2', 'ENSGWIG00000005204', 'ENSGWIG00000005247', 'ENSGWIG00000005517', 'crybb3', 'cita', 'pxna', 'ENSGWIG00000008142', 'ENSGWIG00000008154', 'loxl2a', 'tacc1', 'aggf1', 'NSG1', 'stx18', 'msx1a', 'CYTL1', 'EVC2', 'ENSGWIG00000009977', 'ENSGWIG00000009988', 'ENSGWIG00000009989', 'ENSGWIG00000010154', 'ENSGWIG00000016857', 'ENSGWIG00000016858', 'chchd6b', 'tpra1', 'trim107', 'zgc:123305', 'myl9a', 'ENSGWIG00000016916', 'ENSGWIG00000016920', 'dlgap4b', 'ENSGWIG00000016927', 'quo', 'tgm2b', 'vstm2l', 'ENSGWIG00000018578', 'eif4bb', 'tns2b', 'vdrb', 'twist3', 'phf13', 'slc48a1b', 'dnajc11b', 'ENSGWIG00000018772', 'si:ch211-160o17.4', 'ube2j2', 'acap3b', 'pusl1', 'ddx19', 'ENSGWIG00000024272', 'si:dkey-33m11.6', 'ENSGWIG00000024274', 'ENSGWIG00000024275', 'dnajc16l', 'b3galt6', 'mapk14b', 'srpk1b', 'lhfpl5a', 'kif21b', 'cpne5b', 'ENSGWIG00000026096', 'ENSGWIG00000026097', 'ENSGWIG00000026100', 'gnl3', 'glt8d1', 'spcs1', 'nek4', 'camk1b', 'ccdc3b', 'si:dkey-156n14.3', 'ENSGWIG00000007080', 'ENSGWIG00000007422', 'ENSGWIG00000014029', 'WDR1', 'setd5', 'mtmr14', 'ENSGWIG00000015230', 'ENSGWIG00000015305', 'ENSGWIG00000015671', 'si:dkey-166c18.1', 'ENSGWIG00000015683', 'klhdc8a', 'slc41a1', 'ENSGWIG00000015796', 'etnk2', 'sox13', 'snrpe', 'ENSGWIG00000015807', 'ENSGWIG00000015808', 'wnt4', 'ENSGWIG00000015810', 'zgc:175088', 'ENSGWIG00000015814', 'ENSGWIG00000015815', 'ENSGWIG00000015821', 'ENSGWIG00000015836', 'bin2b', 'txnrd3', 'CHST13', 'ENSGWIG00000016035', 'phactr3b', 'ENSGWIG00000016634', 'ENSGWIG00000016635', 'mybl2b', 'rpn2', 'ENSGWIG00000018856', 'ENSGWIG00000018882', 'ENSGWIG00000018997', 'cntn4', 'ENSGWIG00000021155', 'ENSGWIG00000021257', 'chl1b', 'ENSGWIG00000024538', 'ENSGWIG00000024539', 'ENSGWIG00000024541', 'cntn3b', 'pdzrn3b', 'ENSGWIG00000024727', 'ppp4r2b', 'gxylt2', 'shq1', 'rybpb', 'ENSGWIG00000024745', 'ENSGWIG00000024749', 'prok2', 'ENSGWIG00000024783', 'ENSGWIG00000024786', 'foxp1b', 'ENSGWIG00000024792', 'ENSGWIG00000024794', 'ENSGWIG00000024795', 'ENSGWIG00000024798', 'mitfa', 'frmd4ba', 'arl6ip5a', 'trnt1', 'lrrn1', 'setmar', 'ENSGWIG00000024820', 'ENSGWIG00000000436', 'ENSGWIG00000000444', 'bhlhe40', 'ENSGWIG00000000573', 'thoc7', 'atxn7', 'psmd6', 'slc2a9l1', 'prickle2b', 'adamts9', 'magi1b', 'ENSGWIG00000001164', 'ENSGWIG00000001166', 'lrig1', 'ENSGWIG00000001396', 'ENSGWIG00000001414', 'kbtbd8', 'ENSGWIG00000001421', 'suclg2', 'ENSGWIG00000001681', 'ENSGWIG00000001685', 'TAFA1', 'tafa4b', 'tmf1', 'fgd5a', 'stimate', 'mustn1b', 'ENSGWIG00000007428', 'ENSGWIG00000007507', 'gata2a', 'rpn1', 'rab7a', 'hmces', 'cidec', 'ENSGWIG00000009786', 'ENSGWIG00000009788', 'atp5f1e', 'sec61a1l', 'ENSGWIG00000009810', 'pfkfb4a', 'tada3l', 'ENSGWIG00000010530', 'RBM38', 'rae1', 'ENSGWIG00000011704', 'ENSGWIG00000011705', 'cdh4', 'ENSGWIG00000011987', 'ENSGWIG00000011988', 'ENSGWIG00000011990', 'ENSGWIG00000011996', 'ENSGWIG00000012000', 'zmynd8', 'ncoa3', 'id1', 'sulf2a', 'zgc:92107', 'phf20a', 'ndrg3a', 'myh7ba', 'snai1a', 'ENSGWIG00000016047', 'ENSGWIG00000016048', 'ENSGWIG00000016050', 'top1l', 'si:ch211-232b12.5', 'fam83d', 'ENSGWIG00000000784', 'ENSGWIG00000000799', 'ENSGWIG00000000808', 'gata1a', 'hcfc1a', 'gnl3l', 'mbd1b', 'ENSGWIG00000001151', 'chrm2a', 'mtpn', 'ENSGWIG00000005265', 'sema3c', 'ENSGWIG00000008823', 'ENSGWIG00000009035', 'ENSGWIG00000009036', 'ENSGWIG00000009041', 'si:dkey-97m3.1', 'kdm7ab', 'kmt2e', 'lhfpl3', 'orc5', 'golgb1', 'ccnd2b', 'fgf23', 'rad51ap1', 'atp6v1e1b', 'spic', 'ENSGWIG00000006538', 'chpt1', 'ENSGWIG00000007577', 'mcat', 'def6c', 'ENSGWIG00000007583', 'washc3', 'ttll1', 'pacsin2', 'arfgap3', 'ENSGWIG00000007626', 'tbc1d30', 'sfmbt2', 'tmem110l', 'itih5', 'ENSGWIG00000010781', 'kin', 'atp5f1c', 'gata3', 'ENSGWIG00000010793', 'CELF2', 'ENSGWIG00000010976', 'usp6nl', 'ENSGWIG00000011036', 'ECHDC3', 'plxnb2a', 'ENSGWIG00000012296', 'creb3l2', 'ENSGWIG00000012678', 'ENSGWIG00000016861', 'rassf3', 'zgc:194209', 'uhrf1bp1l', 'anks1b', 'apaf1', 'si:ch211-244b2.1', 'ENSGWIG00000019149', 'si:ch211-244b2.4', 'epyc', 'kera', 'lum', 'dcn', 'ENSGWIG00000019166', 'ENSGWIG00000019193', 'btg1', 'fam3c', 'si:dkey-159a18.1', 'impdh1b', 'si:dkey-31f5.11', 'ENSGWIG00000019457', 'snd1', 'lrrc4.1', 'pax4', 'hgfa', 'cacna2d1a', 'ENSGWIG00000024424', 'sema3aa', 'tspan11', 'prmt8b', 'ccdc59', 'mettl25', 'tmtc2a', 'PARP11', 'ENSGWIG00000009165', 'ENSGWIG00000009168', 'iqsec3a', 'ENSGWIG00000009384', 'st8sia1', 'ENSGWIG00000009387', 'ENSGWIG00000009481', 'ENSGWIG00000012302', 'lrguk', 'exoc4', 'chchd3a', 'ENSGWIG00000013938', 'plxna4', 'nrf1', 'MIR183', 'ube2h', 'zc3hc1', 'smo', 'ENSGWIG00000022291', 'cacna1c', 'ENSGWIG00000022294', 'ENSGWIG00000022295', 'cacna2d4a', 'lrtm2a', 'cpsf6', 'yeats4', 'ENSGWIG00000022320', 'ENSGWIG00000022321', 'ENSGWIG00000022322', 'ninj2', 'ERC1', 'wnt5b', 'FBXL14', 'adipor2', 'ENSGWIG00000022948', 'ENSGWIG00000022950', 'ENSGWIG00000022951', 'ENSGWIG00000022953', 'ENSGWIG00000022952', 'ENSGWIG00000022956', 'ENSGWIG00000022957', 'LOC114465680', 'ENSGWIG00000022959', 'RASSF8', 'pkp2', 'nudt4b', 'ube2na', 'ano6', 'ENSGWIG00000023456', 'ENSGWIG00000023459', 'ENSGWIG00000023474', 'lmod2b', 'SPAM1', 'TMEM229A', 'gpr37b', 'ENSGWIG00000023486', 'ENSGWIG00000023487', 'ENSGWIG00000023489', 'ENSGWIG00000023491', 'grm8a', 'ENSGWIG00000002606', 'ZNF800', 'ATXN7L1', 'sypl1', 'si:dkey-145c18.3', 'gsap', 'ENSGWIG00000002637', 'fgl2a', 'lrrc17', 'ENSGWIG00000002647', 'ENSGWIG00000002651', 'ENSGWIG00000002653', 'pus7l', 'ENSGWIG00000002659', 'pphln1', 'yaf2', 'gxylt1b', 'pdzrn4', 'cntn1b', 'nrcama', 'pnpla8', 'dnajb9b', 'akr1b1.1', 'tfec', 'si:ch211-262n1.4', 'foxp2', 'ENSGWIG00000017781', 'gpr85', 'ENSGWIG00000017786', 'tmem168a', 'ifrd1', 'dock4b', 'ENSGWIG00000021023', 'lrrn3b', 'ENSGWIG00000021027', 'tnnt2e', 'ENSGWIG00000021070', 'si:dkey-13i19.8', 'strip2', 'ENSGWIG00000023810', 'ahcyl2b', 'ENSGWIG00000024627', 'cdkn1d', 'ENSGWIG00000024630', 'ENSGWIG00000024632', 'ENSGWIG00000024636', 'ENSGWIG00000024638', 'pparaa', 'wnt7ba', 'atxn10', 'ENSGWIG00000024656', 'kdm5a', 'ENSGWIG00000024910', 'USP15', 'ENSGWIG00000025161', 'pim3', 'creld2', 'zbed4', 'brd1a', 'ENSGWIG00000025180', 'ENSGWIG00000025181', 'TAFA5', 'tbc1d22a', 'cerk', 'gramd4a', 'celsr1a', 'trmu', 'alg10', 'CPNE8', 'ENSGWIG00000025598', 'SLC2A13', 'slco1d1', 'ENSGWIG00000003775', 'etnk1', 'sox5', 'ENSGWIG00000007382', 'ENSGWIG00000007384', 'bcat1', 'lrmp', 'casc1', 'lyrm5a', 'ENSGWIG00000007476', 'slc5a8', 'ENSGWIG00000008215', 'mapk11', 'rpap3', 'atp23', 'lrig3', 'zgc:165507', 'ENSGWIG00000010012', 'ETV6', 'si:ch73-252i11.1', 'SYN3', 'TIMP3', 'si:cabz01085950.1', 'hcfc2', 'ENSGWIG00000011303', 'mcm10', 'ccdc3a', 'camk1da', 'cdc123', 'nudt5', 'fbxo18', 'ENSGWIG00000014410', 'slc38a4', 'slc38a2', 'ENSGWIG00000021106', 'arid2', 'gas2l3', 'tm7sf3', 'fgfr1op2', 'ENSGWIG00000021240', 'ARNTL2', 'zgc:162331', 'slc13a4', 'ENSGWIG00000021419', 'ENSGWIG00000021420', 'pik3cg', 'ano2b', 'ENSGWIG00000021812', 'ENSGWIG00000021810', 'ntf3', 'pfkfb3', 'rbm17', 'il15ra', 'ANO4', 'ENSGWIG00000004870', 'ENSGWIG00000004899', 'apex1', 'si:ch211-214j24.10', 'osgep', 'nav3', 'ENSGWIG00000007590', 'e2f7', 'csrp2', 'OSBPL8', 'nap1l1', 'phlda1', 'slc26a3.2', 'ppp6r2a', 'ENSGWIG00000013312', 'ENSGWIG00000014338', 'glt8d2', 'ankrd16', 'gdi2', 'ENSGWIG00000014375', 'net1', 'mkln1', 'ENSGWIG00000015508', 'ENSGWIG00000015510', 'nfyba', 'chst11', 'slc41a2b', 'aldh1l2', 'samm50l', 'scube1', 'mpped1', 'sult4a1', 'pnpla3', 'SNX4', 'ttll12', 'ENSGWIG00000020263', 'dennd5b', 'ipo8', 'ENSGWIG00000022813', 'si:dkey-8e10.3', 'crebl2', 'gpr19', 'ENSGWIG00000022823', 'eps8a', 'ptpro', 'ENSGWIG00000023026', 'sco2', 'wnt16', 'zgc:55888', 'ENSGWIG00000023154', 'svopl', 'krr1', 'tmtc1', 'ENSGWIG00000023536', 'ptprr', 'ENSGWIG00000023790', 'COG5', 'ndufa5', 'plekha5', 'si:ch73-278m9.1', 'kcnc2', 'ENSGWIG00000004990', 'phax', 'dnajc2', 'psmc2', 'slc26a5', 'ENSGWIG00000006699', 'ENSGWIG00000006700', 'ENSGWIG00000006701', 'ENSGWIG00000006723', 'ENSGWIG00000007383', 'ENSGWIG00000007387', 'ENSGWIG00000007395', 'pthlhb', 'tmpob', 'actr6', 'nedd1', 'cnot4a', 'ENSGWIG00000007502', 'ENSGWIG00000007512', 'ENSGWIG00000007918', 'ENSGWIG00000007919', 'ENSGWIG00000007960', 'DNM1L', 'tbc1d15', 'frs2b', 'ENSGWIG00000008994', 'ENSGWIG00000008997', 'nup160', 'ENSGWIG00000009139', 'lamb1a', 'ENSGWIG00000009173', 'ENSGWIG00000011084', 'ENSGWIG00000011091', 'ENSGWIG00000011095', 'bpgm', 'ENSGWIG00000011102', 'tnnt2d', 'ENSGWIG00000011127', 'ENSGWIG00000011142', 'gpia', 'ENSGWIG00000011188', 'ambra1b', 'ENSGWIG00000011225', 'ENSGWIG00000011227', 'kcnj11l', 'ENSGWIG00000011235', 'commd9', 'ENSGWIG00000011278', 'ENSGWIG00000011290', 'ENSGWIG00000011300', 'anpepb', 'dus2', 'ENSGWIG00000011394', 'znf277', 'ENSGWIG00000012039', 'ENSGWIG00000012042', 'c2cd4a', 'tln2b', 'TPM1', 'lactb', 'ENSGWIG00000015874', 'TMEM266', 'etfa', 'scaper', 'rcn2', 'pstpip1a', 'tspan3a', 'ch25hl1.2', 'slc28a1', 'ENSGWIG00000017824', 'malt3', 'si:ch211-69m14.1', 'blm', 'si:ch211-234p18.3', 'ENSGWIG00000005083', 'idh2', 'znf710a', 'sema4ba', 'zgc:162879', 'si:ch211-13k12.2', 'pex11a', 'GNRHR', 'ENSGWIG00000005387', 'bcar1', 'ENSGWIG00000005412', 'lrp5', 'ENSGWIG00000010382', 'tmem170a', 'ppp6r3', 'ric8a', 'ENSGWIG00000010487', 'ripor1', 'AEBP2', 'ENSGWIG00000010680', 'ENSGWIG00000010681', 'ENSGWIG00000010682', 'ENSGWIG00000010683', 'ENSGWIG00000010685', 'SLCO1C1', 'ENSGWIG00000010691', 'gpr22b', 'ENSGWIG00000010700', 'hbp1', 'si:ch211-272n13.7', 'hsp90b1', 'ifitm5', 'ENSGWIG00000011619', 'ENSGWIG00000011621', 'ptdss2', 'bmt2', 'ENSGWIG00000012198', 'ENSGWIG00000012199', 'si:ch211-272n13.3', 'ENSGWIG00000012209', 'AVPR1A', 'U4', 'ppm1h', 'mon2', 'SLC6A13', 'fbln1', 'wnt7bb', 'PPARA', 'yars2', 'cdkn1bb', 'ENSGWIG00000016320', 'ENSGWIG00000016323', 'ENSGWIG00000016325', 'mrps35', 'ENSGWIG00000016327', 'ENSGWIG00000016332', 'DENND2A', 'adck2', 'ENSGWIG00000017110', 'ENSGWIG00000018807', 'hdac10', 'parvb', 'parvg', 'shisal1b', 'plxnb2b', 'ENSGWIG00000022195', 'ENSGWIG00000000164', 'ENSGWIG00000000173', 'appl2', 'ENSGWIG00000001325', 'nuak1b', 'ENSGWIG00000001331', 'szl', 'ENSGWIG00000001343', 'nell2a', 'ENSGWIG00000007749', 'irak4', 'nptna', 'ENSGWIG00000007772', 'ENSGWIG00000007777', 'ENSGWIG00000007792', 'pkmb', 'ENSGWIG00000008968', 'ENSGWIG00000008971', 'arid3b', 'ENSGWIG00000008979', 'edc3', 'adal', 'nat10', 'caprin1b', 'lmo2', 'alx4b', 'tspan18b', 'cd82b', 'tp53i11b', 'si:dkey-29p10.4', 'ENSGWIG00000012340', 'ENSGWIG00000012346', 'ENSGWIG00000012349', 'RELN', 'PLEKHA5', 'ENSGWIG00000012460', 'ENSGWIG00000012466', 'ENSGWIG00000012467', 'ENSGWIG00000012469', 'ENSGWIG00000012538', 'ENSGWIG00000012545', 'scamp5a', 'parp16', 'ENSGWIG00000012555', 'ctdspl2a', 'ENSGWIG00000012591', 'sqor', 'galk2', 'cep152', 'ENSGWIG00000012986', 'ENSGWIG00000012989', 'tjp1b', 'ENSGWIG00000013038', 'pard6a', 'ENSGWIG00000013046', 'ENSGWIG00000013049', 'galn', 'ENSGWIG00000013063', 'ENSGWIG00000013068', 'ENSGWIG00000013075', 'ENSGWIG00000013080', 'slco3a1', 'ENSGWIG00000004754', 'fam174b', 'ENSGWIG00000005028', 'rgma', 'ENSGWIG00000006097', 'ENSGWIG00000006100', 'lysmd4', 'ENSGWIG00000006104', 'nr2f2', 'mctp2a', 'mef2aa', 'SYNM', 'ENSGWIG00000006265', 'igf1ra', 'ENSGWIG00000006275', 'ENSGWIG00000006318', 'ENSGWIG00000006737', 'cd9a', 'ENSGWIG00000006749', 'ENSGWIG00000006803', 'ENSGWIG00000006816', 'ENSGWIG00000006818', 'ptprq', 'iqsec3b', 'ENSGWIG00000006885', 'wash1', 'ddx11', 'fkbp4', 'b4galnt3b', 'c2cd5', 'lmf2a', 'ENSGWIG00000007084', 'si:dkey-266m15.5', 'ppp6r2b', 'dnaja2b', 'ENSGWIG00000007114', 'ENSGWIG00000007132', 'ENSGWIG00000007142', 'calub', 'ENSGWIG00000007154', 'ENSGWIG00000007156', 'si:ch1073-390k14.1', 'si:ch211-233h19.2', 'grm3', 'ENSGWIG00000007363', 'sema3d', 'sema3ab', 'sema3e', 'ENSGWIG00000018601', 'PCLO', 'si:dkey-266j7.2', 'hgfb', 'rbm28', 'ENSGWIG00000019061', 'ENSGWIG00000019373', 'impdh1a', 'si:ch211-220f12.1', 'ENSGWIG00000020161', 'ENSGWIG00000020164', 'neto2b', 'si:dkey-30c15.2', 'ENSGWIG00000020197', 'zgc:77752', 'ENSGWIG00000000539', 'rabl2', 'ENSGWIG00000000554', 'ENSGWIG00000000563', 'ENSGWIG00000000567', 'arsa', 'mapk8ip2', 'ENSGWIG00000000686', 'ENSGWIG00000000949', 'chkb', 'ENSGWIG00000001372', 'cpt1b', 'sephs1', 'ENSGWIG00000002471', 'wwox', 'ENSGWIG00000002474', 'mafb', 'ENSGWIG00000002477', 'prpf18', 'nfat5a', 'nudt7', 'hprt1l', 'ENSGWIG00000002511', 'erc1a', 'wnk1a', 'prr5a', 'TMEM60', 'RSBN1L', 'ENSGWIG00000004256', 'LRRC17', 'armc10', 'ENSGWIG00000004396', 'kxd1', 'magi2b', 'si:ch211-225b10.3', 'ENSGWIG00000007143', 'ENSGWIG00000007230', 'ENSGWIG00000009092', 'ENSGWIG00000009093', 'ENSGWIG00000009129', 'ENSGWIG00000009338', 'tmem263', 'rfx4', 'polr3b', 'ppfibp1b', 'arntl2', 'sass6', 'nr1h4', 'si:ch211-219a15.3', 'nudt4a', 'eea1', 'plekhg7', 'itpr2', 'rassf8a', 'kras', 'ENSGWIG00000025521', 'ENSGWIG00000025522', 'ENSGWIG00000025523', 'ENSGWIG00000025524', 'ENSGWIG00000025525', 'ENSGWIG00000025526', 'banp', 'ENSGWIG00000025528', 'polg', 'ENSGWIG00000005356', 'rhcgb', 'ENSGWIG00000005484', 'unc45a', 'man2a2', 'furinb', 'ENSGWIG00000009315', 'ENSGWIG00000009357', 'rag1', 'rag2', 'ENSGWIG00000009364', 'ENSGWIG00000009367', 'ENSGWIG00000009369', 'lrrc4ca', 'EHF', 'pdhx', 'cd44b', 'slc1a2b', 'pamr1', 'fjx1', 'trim44', 'LDLRAD3', 'ENSGWIG00000010933', 'ascl1b', 'irf7', 'phrf1', 'rassf7a', 'ENSGWIG00000010941', 'hrasb', 'tsg101a', 'ldha', 'ENSGWIG00000011006', 'abtb2b', 'ENSGWIG00000011064', 'tmem86a', 'spty2d1', 'si:dkeyp-19e1.3', 'cpt1ab', 'si:ch211-236l14.4', 'mrpl23', 'ENSGWIG00000011871', 'igf2b', 'nap1l4a', 'phlda2', 'osbpl5', 'ENSGWIG00000014839', 'ENSGWIG00000014843', 'ptpn5', 'dusp8a', 'mob2a', 'kcnj11', 'abcc8', 'ush1c', 'hdhd5', 'RAB19', 'ENSGWIG00000015744', 'fnbp4', 'cnot2', 'ENSGWIG00000017650', 'ENSGWIG00000017652', 'etfbkmt', 'slc41a2a', 'ENSGWIG00000018757', 'si:ch73-62b13.1', 'mterf2', 'btbd11b', 'panx2', 'selenoo1', 'mapk12a', 'mtss1la', 'ENSGWIG00000000603', 'ENSGWIG00000000612', 'ENSGWIG00000000620', 'rxylt1', 'srgap1a', 'parp12a', 'ENSGWIG00000003876', 'tbxas1', 'hipk2', 'si:ch211-1e14.1', 'mical3a', 'ttc38', 'tspan9a', 'ENSGWIG00000007046', 'CRACR2A', 'tmtc2b', 'ENSGWIG00000007281', 'zgc:110789', 'nup205', 'wee2', 'bcl2l13', 'slc25a18', 'cecr2', 'ada2a', 'ribc2', 'GALNT8', 'dyrk4', 'wfdc1', 'si:dkey-246g23.4', 'kcng4a', 'si:dkey-246g23.2', 'hsbp1a', 'zgc:173742', 'mlycd', 'cdh13', 'ENSGWIG00000014979', 'ccnd2a', 'tigara', 'FGF6', 'zgc:77158', 'elmo3', 'lrrc29', 'cngb1b', 'ENSGWIG00000018968', 'mmp15a', 'nup93', 'rspry1', 'ENSGWIG00000019410', 'ENSGWIG00000019459', 'ENSGWIG00000019828', 'ENSGWIG00000019973', 'hsf4', 'gnao1a', 'cbfb', 'si:dkey-102f14.5', 'b3gnt9', 'cnot1', 'si:ch211-216l23.2', 'si:ch211-216l23.1', 'si:ch211-212o1.2', 'cbln1', 'ENSGWIG00000024329', 'ENSGWIG00000024331', 'ENSGWIG00000024332', 'syt9b', 'spon1b', 'ENSGWIG00000000690', 'ENSGWIG00000000691', 'ENSGWIG00000000693', 'hipk3a', 'tcp11l1', 'wt1a', 'ENSGWIG00000000937', 'snrkb', 'fa2h', 'znf319b', 'pla2g15', 'ENSGWIG00000000978', 'chst6', 'tmem231', 'ENSGWIG00000000986', 'wwp2', 'si:dkey-30c15.10', 'smpd3', 'sept15', 'cyb5r2', 'poglut3', 'ENSGWIG00000009957', 'arntl1a', 'btbd10a', 'zgc:162634', 'galnt18a', 'slc12a3', 'herpud1', 'cetp', 'chka', 'si:dkey-12e7.1', 'anpepa', 'arpin', 'ldhd', 'fam169b', 'znrf1', 'ENSGWIG00000011087', 'ndrg4', 'far1', 'dkk3b', 'mical2b', 'parvaa', 'tead1a', 'RASSF10', 'si:ch211-266k8.4', 'ENSGWIG00000013239', 'n4bp1', 'siah1', 'lonp2', 'il34', 'sf3b3', 'ENSGWIG00000013347', 'st3gal2', 'cpne2', 'ENSGWIG00000013649', 'ENSGWIG00000013674', 'ENSGWIG00000014010', 'ENSGWIG00000014026', 'hydin', 'necab2', 'slc38a8a', 'pskh1', 'nrn1la', 'edc4', 'ranbp10', 'ENSGWIG00000000245', 'det1', 'zcchc14', 'jph3', 'klhdc4', 'slc7a5', 'uba2', 'ca5a', 'lcat', 'spire2', 'sntb2', 'cdk10', 'parietopsin', 'ENSGWIG00000006315', 'flnca', 'ENSGWIG00000014679', 'ENSGWIG00000014682', 'ENSGWIG00000014709', 'duox2', 'fan1', 'adam10b', 'ENSGWIG00000015148', 'rfx7b', 'ENSGWIG00000015150', 'ENSGWIG00000015178', 'ENSGWIG00000015461', 'CYP19A1', 'ap4e1', 'ENSGWIG00000015955', 'clpxb', 'PRC1', 'ENSGWIG00000016412', 'fibina', 'ano3', 'gas2a', 'ENSGWIG00000016737', 'ENSGWIG00000016781', 'ENSGWIG00000016790', 'PKP3', 'kcnc1b', 'sergef', 'ENSGWIG00000019466', 'ENSGWIG00000019549', 'ENSGWIG00000019563', 'ENSGWIG00000019570', 'ENSGWIG00000019571', 'ENSGWIG00000019582', 'ENSGWIG00000019658', 'hps5', 'epx', 'rab3il1', 'ENSGWIG00000019909', 'pnpla2', 'pidd1', 'slc25a22a', 'syt7a', 'ENSGWIG00000019998', 'EPS8L2', 'myrf', 'cracr2b', 'ighmbp2', 'ENSGWIG00000000334', 'rapsn', 'kif23', 'tle3a', 'morf4l1', 'pias1b', 'aagab', 'smad3b', 'smad6b', 'ENSGWIG00000005707', 'lctlb', 'zwilch', 'map2k1', 'megf11', 'dennd4a', 'ints14', 'ENSGWIG00000013332', 'arih1', 'hexa', 'nr2e3', 'ENSGWIG00000014594', 'stra6', 'islr2', 'ENSGWIG00000014618', 'ca12', 'ENSGWIG00000014628', 'usp3', 'fbxl22', 'ENSGWIG00000014684', 'dapk2b', 'ENSGWIG00000015263', 'cib2', 'DNAJA4', 'ENSGWIG00000015833', 'taf3', 'ENSGWIG00000015888', 'ENSGWIG00000015939', 'ENSGWIG00000016802', 'slc7a10b', 'ENSGWIG00000016868', 'roraa', 'anxa2a', 'foxb1a', 'bnip2', 'gcnt3', 'ENSGWIG00000018211', 'si:dkey-37f18.2', 'klf13', 'ENSGWIG00000018217', 'dhtkd1', 'ENSGWIG00000020356', 'cdk17', 'met', 'CAPZA2', 'st7', 'wnt2', 'ENSGWIG00000004755', 'asz1', 'cftr', 'cttnbp2', 'ENSGWIG00000012931', 'ENSGWIG00000012932', 'ENSGWIG00000012943', 'CERK', 'ENSGWIG00000012952', 'ptprz1a', 'aass', 'CADPS2', 'iqub', 'ASB15', 'ENSGWIG00000021538', 'hyal6', 'ENSGWIG00000021546', 'ENSGWIG00000021547', 'ENSGWIG00000021557', 'ENSGWIG00000021571', 'grm8b', 'acot19', 'si:ch211-253p14.2', 'ENSGWIG00000022678', 'ftsj1', 'ENSGWIG00000022722', 'deaf1', 'drd4a', 'ENSGWIG00000023124', 'ENSGWIG00000023279', 'dldh', 'ENSGWIG00000023301', 'ENSGWIG00000023303', 'ENSGWIG00000023305', 'tnni2a.2', 'ENSGWIG00000023314', 'lsp1a', 'tnnt3a', 'ENSGWIG00000023344', 'ENSGWIG00000023345', 'ENSGWIG00000023347', 'si:ch1073-174d20.2', 'ENSGWIG00000024002', 'ENSGWIG00000024005', 'glg1a', 'rfwd3', 'ENSGWIG00000024024', 'ENSGWIG00000024025', 'tmed6', 'ENSGWIG00000024029', 'zgc:101566', 'ciao2a', 'ENSGWIG00000024147', 'fem1b', 'ENSGWIG00000024180', 'rxfp3.3b', 'ckap5', 'ENSGWIG00000025909', 'zgc:92873', 'cry2', 'chtf8', 'has3', 'utp4', 'ENSGWIG00000000422', 'zc3h18', 'trhr2', 'zfpm1', 'ces3', 'zgc:91860', 'U6', 'foxl1', 'ENSGWIG00000000584', 'cmip', 'plcg2', 'ENSGWIG00000000863', 'cotl1', 'zgc:56622', 'pdf', 'vps9d1', 'zgc:158366', 'ENSGWIG00000001397', 'slc10a3', 'ENSGWIG00000001406', 'acanb', 'mfge8b', 'ABHD2', 'ENSGWIG00000002199', 'ENSGWIG00000004167', 'C11orf96', 'hsd17b12a', 'alkbh3', 'api5', 'tdg.2', 'ENSGWIG00000005930', 'pik3c2a', 'ppp1r15b', 'ENSGWIG00000008255', 'PLEKHA7', 'ENSGWIG00000008312', 'ENSGWIG00000009265', 'INSC', 'CALCB', 'psma1', 'ENSGWIG00000009342', 'ENSGWIG00000009345', 'ENSGWIG00000009346', 'akip1', 'znf143b', 'ireb2', 'slc25a44a', 'sin3aa', 'ptpn9a', 'snx33', 'cspg4', 'lingo1a', 'ENSGWIG00000015323', 'ENSGWIG00000000671', 'hmg20a', 'nfatc2a', 'ATP9A', 'sall4', 'zfp64', 'tshz2', 'ENSGWIG00000023668', 'ENSGWIG00000023671', 'ENSGWIG00000023674', 'pfdn4', 'ENSGWIG00000023677', 'ENSGWIG00000023681', 'ENSGWIG00000023690', 'ENSGWIG00000023691', 'plekhg5b', 'nol9', 'zbtb48', 'klhl21', 'syt6a', 'olfml3a', 'c1qtnf12', 'sdf4', 'hipk1a', 'acap3a', 'VWA5B1', 'fam43b', 'zmp:0000001088', 'pink1', 'agmat', 'hspg2', 'wu:fi36a10', 'ENSGWIG00000011718', 'RSBN1', 'ENSGWIG00000011742', 'mical1', 'itih6', 'pfkfb1', 'ccn5', 'ada', 'pkig', 'ttpal', 'hnf4a', 'ENSGWIG00000015145', 'ENSGWIG00000015146', 'ENSGWIG00000015173', 'sarnp', 'ccnt1', 'ENSGWIG00000015192', 'tegt', 'ctdsp2', 'pabpc1l', 'YWHAB', 'rbm39b', 'slc12a5b', 'ENSGWIG00000019897', 'eif2d', 'fmodb', 'ENSGWIG00000020117', 'ren', 'magi1a', 'prickle2a', 'ENSGWIG00000020163', 'ENSGWIG00000020167', 'qars1', 'ogg1', 'ENSGWIG00000021077', 'ENSGWIG00000021080', 'ENSGWIG00000021082', 'b3galt4', 'nr2c2', 'ENSGWIG00000000340', 'ENSGWIG00000000346', 'TMCC1', 'ENSGWIG00000000454', 'RHO', 'creld1b', 'ENSGWIG00000000769', 'ENSGWIG00000000805', 'ENSGWIG00000000820', 'ENSGWIG00000000897', 'BICD2', 'ptpdc1b', 'ENSGWIG00000001060', 'FBLN2', 'nup210', 'IQSEC1', 'raf1a', 'ENSGWIG00000001379', 'vgll4a', 'slc6a1a', 'ENSGWIG00000001806', 'ENSGWIG00000001814', 'ENSGWIG00000001875', 'bmp7b', 'ENSGWIG00000002092', 'cdh26.1', 'ENSGWIG00000002102', 'mtg2', 'ENSGWIG00000002239', 'kcnk15', 'ogfr', 'ENSGWIG00000005089', 'ENSGWIG00000005099', 'prkcbp1l', 'zgc:113278', 'ENSGWIG00000008592', 'rbm38', 'ENSGWIG00000008605', 'mipb', 'baz2a', 'ptges3b', 'naca', 'RND1', 'cacnb3a', 'adcy6a', 'g6pd', 'ENSGWIG00000015585', 'asic1a', 'ENSGWIG00000015712', 'ENSGWIG00000019475', 'col2a1a', 'map3k12', 'pcbp2', 'ENSGWIG00000025215', 'pfkfb2a', 'scube3', 'ENSGWIG00000025998', 'ANKS1A', 'taf11', 'uhrf1bp1', 'ILRUN', 'pacsin1a', 'smim29', 'ENSGWIG00000026007', 'ENSGWIG00000026008', 'ENSGWIG00000026009', 'ENSGWIG00000026012', 'syt2a', 'oard1', 'ENSGWIG00000000324', 'MITF', 'frmd4bb', 'LMOD3', 'tafa1a', 'ENSGWIG00000000800', 'zgc:92287', 'mfsd4ab', 'ENSGWIG00000001286', 'CDC42', 'ENSGWIG00000001291', 'dnajc11a', 'camta1a', 'uts2a', 'ccdc187', 'dvl1a', 'mxra8b', 'aurkaip1', 'rap1gap', 'YTHDF1', 'bhlhe23', 'slc17a9b', 'asb14b', 'tasorb', 'ENSGWIG00000001385', 'GRM7', 'DAG1', 'ENSGWIG00000001402', 'DOCK3', 'DCAF1', 'ssuh2rs1', 'grip2a', 'slc6a6b', 'abtb1', 'ENSGWIG00000012089', 'si:ch73-233m11.2', 'slmapb', 'kctd6b', 'nol4la', 'ppdpfa', 'tpd52l2a', 'dnajc5aa', 'ENSGWIG00000012139', 'ENSGWIG00000012142', 'si:dkey-70p6.1', 'UCKL1', 'samd10a', 'emilin3a', 'opn7c', 'rad21l1', 'snpha', 'ENSGWIG00000012245', 'zgc:171501', 'prickle3', 'plp2', 'sypb', 'gpkow', 'ENSGWIG00000014730', 'ENSGWIG00000014731', 'iqsec2b', 'kdm5c', 'si:ch73-162j3.4', 'porcn', 'ENSGWIG00000022017', 'ENSGWIG00000022075', 'tspy', 'sema3ga', 'ENSGWIG00000024034', 'slc38a5a', 'zgc:162200', 'apex2', 'nt5dc2', 'birc7', 'nkain4', 'ENSGWIG00000007976', 'ENSGWIG00000007994', 'zbtb46', 'uckl1b', 'si:dkey-256k13.2', 'samd10b', 'prpf6', 'ENSGWIG00000008094', 'aar2', 'EMILIN3', 'b4galt5', 'slc9a8', 'opn7b', 'spata2', 'rnf114', 'ENSGWIG00000009702', 'MAFB', 'zgc:158828', 'rab5if', 'zhx3', 'plcg1', 'top1', 'ncoa6', 'GGT7', 'ENSGWIG00000012373', 'snai1b', 'samhd1', 'si:ch211-117c9.2', 'si:ch211-117c9.5', 'ENSGWIG00000017118', 'ENSGWIG00000017301', 'parapinopsina', 'ddx20', 'kcnd3', 'ENSGWIG00000020047', 'ENSGWIG00000020048', 'ENSGWIG00000020050', 'cttnbp2nlb', 'capza1b', 'apeh', 'tlr9', 'ERC2', 'wnt5a', 'CACNA2D3', 'lrtm1', 'atp6ap1lb', 'b4galt3', 'bgnb', 'si:dkey-32e6.6', 'lrrc23', 'rbm10', 'usp21', 'arhgef3l', 'uba1', 'camkvl', 'slc25a55b', 'ENSGWIG00000022972', 'ngfa', 'ENSGWIG00000022987', 'LGR6', 'ENSGWIG00000023009', 'ENSGWIG00000023010', 'tead3a', 'PKP1', 'tnnt2a', 'tnni1a', 'phlda3', 'csrp1a', 'ENSGWIG00000004759', 'NAV1', 'ENSGWIG00000004789', 'ENSGWIG00000004858', 'vwa1', 'ralgapb', 'ENSGWIG00000005113', 'prex1', 'kcnq2a', 'chrna4b', 'zgpat', 'tti1', 'ENSGWIG00000007637', 'ctnnbl1', 'ENSGWIG00000008608', 'e2f1', 'znf341', 'itchb', 'ENSGWIG00000019384', 'VDR', 'hdac7a', 'ENSGWIG00000019403', 'ENSGWIG00000019407', 'chchd6a', 'ENSGWIG00000019458', 'ENSGWIG00000019679', 'ENSGWIG00000019680', 'ENSGWIG00000019681', 'si:ch211-236h17.3', 'ENSGWIG00000019692', 'klf15', 'cpne9', 'lhfpl4a', 'plxnb1a', 'efhd2', 'ENSGWIG00000021281', 'tnnc1b', 'ENSGWIG00000021290', 'ENSGWIG00000021321', 'mfn2', 'ENSGWIG00000021474', 'asxl1', 'myt1a', 'suv39h1b', 'gpr173', 'ENSGWIG00000021499', 'ENSGWIG00000021500', 'ENSGWIG00000021528', 'ndufb11', 'rtel1', 'icmt', 'rcc2', 'ENSGWIG00000021593', 'plod1a', 'tpx2', 'si:ch73-302a13.2', 'pdyn', 'stk35', 'MTCH1', 'renbp', 'gss', 'mthfr', 'clcn6', 'ENSGWIG00000008429', 'ENSGWIG00000008443', 'igsf21b', 'vps13d', 'stk38a', 'dennd2da', 'ENSGWIG00000008663', 'ENSGWIG00000008666', 'pim1', 'ENSGWIG00000008729', 'ENSGWIG00000010481', 'fam50a', 'si:ch211-207e14.4', 'sars1', 'tmem106c', 'tns2a', 'ENSGWIG00000013591', 'krt18a.2', 'krt18a.1', 'ENSGWIG00000013596', 'krt8', 'krt5', 'klhdc8b', 'ENSGWIG00000013947', 'ENSGWIG00000013953', 'ENSGWIG00000014060', 'her4.2', 'her12', 'tas1r1', 'rprd1b', 'ENSGWIG00000014132', 'ENSGWIG00000014754', 'ENSGWIG00000014758', 'kif5aa', 'ENSGWIG00000014801', 'stac3', 'ndufa4l2a', 'actr5', 'ENSGWIG00000014814', 'ENSGWIG00000014816', 'ENSGWIG00000014817', 'ENSGWIG00000014821', 'ENSGWIG00000014822', 'ENSGWIG00000014873', 'pofut1', 'plagl2', 'tm9sf4', 'hck', 'rbbp8l', 'ENSGWIG00000016710', 'ENSGWIG00000016712', 'gata5', 'ENSGWIG00000016715', 'ENSGWIG00000016742', 'rps21', 'ENSGWIG00000016895', 'ppdpfb', 'ENSGWIG00000016900', 'ENSGWIG00000004779', 'PCMTD2', 'myt1b', 'tbc1d22b', 'snrpg', 'plpbp', 'tmed4', 'ikbkg', 'sephs2', 'ap4b1', 'ENSGWIG00000006620', 'ENSGWIG00000006623', 'ccnq', 'atp2b3b', 'crebzf', 'ENSGWIG00000006833', 'ENSGWIG00000006835', 'ENSGWIG00000006940', 'SNORA70', 'rpl10', 'ENSGWIG00000010297', 'prkcz', 'skia', 'ENSGWIG00000012684', 'ENSGWIG00000012688', 'plch2a', 'snphb', 'sdcbp2', 'nsfl1c', 'FAM110A', 'cox4i2', 'bcl2l1', 'pard6b', 'dedd', 'adnpb', 'dpm1', 'ENSGWIG00000012885', 'slc35c2', 'si:rp71-17i16.4', 'si:rp71-17i16.5', 'lamb2l', 'lamb2', 'usp19', 'si:dkey-32e6.3', 'zgc:154075', 'ppih', 'arhgef16', 'espn', 'HES2', 'acot7', 'gpr153', 'ENSGWIG00000018943', 'ENSGWIG00000018946', 'ENSGWIG00000019299', 'rer1', 'tp73', 'wrap73', 'ENSGWIG00000019532', 'ENSGWIG00000019724', 'tmem240a', 'otud3', 'bcas2', 'trim33', 'nr1h5', 'brpf3b', 'mapk13', 'tafa5l', 'ENSGWIG00000020486', 'mtor', 'angptl7', 'agrn', 'ptpn11b', 'RNF223', 'dennd2c', 'ampd1', 'CSDE1', 'ENSGWIG00000001353', 'im:6912630', 'ENSGWIG00000001358', 'tmco4', 'mybpha', 'ccdc115', 'fkbpl', 'ENSGWIG00000001403', 'MOV10', 'ENSGWIG00000002458', 'ENSGWIG00000002459', 'zgc:174906', 'znf740b', 'nfe2', 'cbx5', 'hnrnpa1a', 'ENSGWIG00000002574', 'ENSGWIG00000002575', 'hoxc4a', 'hoxc6a', 'hoxc12a', 'hoxc13a', 'calcoco1a', 'RARG', 'nr1d4a', 'mfsd5', 'si:dkey-21c19.3', 'rbms2b', 'lrp1ab', 'ENSGWIG00000003950', 'fmnl3', 'ENSGWIG00000010858', 'erbb3b', 'ENSGWIG00000016820', 'wnt10b', 'wnt1', 'ikzf4', 'lmbr1l', 'dhh', 'ENSGWIG00000018289', 'ACVR1B', 'acvrl1', 'ankrd33aa', 'ARHGEF25', 'sp5l', 'slc26a10', 'znf385a', 'ENSGWIG00000018651', 'ENSGWIG00000018660', 'ENSGWIG00000018689', 'tmem9', 'cacna1sb', 'l1cama', 'MIRLET7F2', 'ENSGWIG00000020200', 'phf8', 'ENSGWIG00000020544', 'idh3g', 'avpr2ab', 'ssr4', 'arhgap4a', 'zgc:158263', 'ENSGWIG00000020940', 'galnt6', 'adcy6b', 'rnd1b', 'ptges3a', 'mipa', 'stat6', 'FIGNL2', 'ENSGWIG00000000333', 'scn8aa', 'dip2ba', 'atf7a', 'larp4aa', 'pfkma', 'ENSGWIG00000020573', 'ppardb', 'fance', 'inavaa', 'smc5', 'ENSGWIG00000022274', 'si:ch211-163l21.8', 'mapk14a', 'srpk1a', 'lhfpl5b', 'cpne5a', 'dclre1b', 'adora1b', 'mmp9', 'si:ch211-269m15.3', 'ENSGWIG00000024550', 'slc16a4', 'rbm15', 'ENSGWIG00000024559', 'slc6a17', 'sort1b', 'sypl2b', 'dnase1l1l', 'ENSGWIG00000024734', 'gstm.1', 'eps8l3b', 'gnat2', 'gnai3', 'gpr61', 'amigo1', 'slc25a33', 'spsb1', 'ENSGWIG00000025320', 'slc2a5', 'ca6', 'eno1a', 'rerea', 'samd11', 'noc2l', 'plekhn1', 'ENSGWIG00000025661', 'her9', 'zgc:194189', 'pik3cd', 'ctnnbip1', 'clstn1', 'ube4b', 'kif1b', 'tardbpl', 'ENSGWIG00000026182', 'casz1', 'pex14', 'masp2', 'cort', 'srm', 'cntn3a.2', 'ENSGWIG00000000896', 'cntn3a.1', 'ENSGWIG00000001766', 'ttll9', 'ENSGWIG00000001775', 'ppp1r3da', 'ENSGWIG00000001782', 'ENSGWIG00000001784', 'phactr3a', 'pigt', 'ENSGWIG00000002160', 'ENSGWIG00000002186', 'ENSGWIG00000002201', 'ENSGWIG00000002207', 'ENSGWIG00000002213', 'ENSGWIG00000002217', 'wfdc2', 'ENSGWIG00000002288', 'ENSGWIG00000002290', 'ENSGWIG00000002316', 'alas2', 'ENSGWIG00000002429', 'ZNF362', 'ccdc30', 'emp3b', 'rimkla', 'ENSGWIG00000002446', 'ENSGWIG00000002447', 'ENSGWIG00000002448', 'KAZN', 'ela2', 'ENSGWIG00000003715', 'si:ch211-263k4.2', 'atp13a2', 'ENSGWIG00000010540', 'MMP23B', 'mib2', 'ttll10', 'arhgef19', 'ENSGWIG00000013131', 'zbtb17', 'clcnk', 'fam131c', 'epha2b', 'spen', 'ENSGWIG00000006332', 'ENSGWIG00000006342', 'ENSGWIG00000006344', 'nadka', 'gnb1a', 'CFAP74', 'gabrd', 'ENSGWIG00000007373', 'ENSGWIG00000007375', 'ajap1', 'ENSGWIG00000007390', 'ENSGWIG00000007397', 'dffb', 'b3gnt7l', 'lrrc47', 'ENSGWIG00000007442', 'ENSGWIG00000007514', 'ep300b', 'xpnpep3', 'st13', 'chadla', 'ZC3H7B', 'tefb', 'TOB2', 'CSDC2', 'esf1', 'plppr2a', 'tlcd4b', 'cnn1b', 'ELOF1', 'tmem205', 'rab3db', 'epor', 'zgc:158403', 'gadd45gip1', 'nfixb', 'ENSGWIG00000006129', 'rad23aa', 'gng13b', 'traf7', 'gucy2c', 'ENSGWIG00000006176', 'ENSGWIG00000006177', 'ENSGWIG00000006179', 'DCAF7', 'smarca4a', 'ldlra', 'ENSGWIG00000006479', 'kri1', 'spc24', 's1pr5a', 'ENSGWIG00000006491', 'ENSGWIG00000006492', 'si:dkeyp-69e1.8', 'hmgxb4a', 'ankrd54', 'foxred2', 'si:ch211-237i5.4', 'pex12', 'ENSGWIG00000006508', 'hdac5', 'cyb561', 'ENSGWIG00000006578', 'pvalb6', 'ENSGWIG00000006617', 'nptxra', 'ENSGWIG00000006695', 'adsl', 'ENSGWIG00000007380', 'ENSGWIG00000007399', 'mief1', 'ENSGWIG00000007412', 'ENSGWIG00000007417', 'zgc:162544', 'ENSGWIG00000007432', 'ENSGWIG00000007439', 'ENSGWIG00000007451', 'arhgap17b', 'ENSGWIG00000007469', 'slc5a11', 'grid2ipa', 'ENSGWIG00000014739', 'e4f1', 'narfl', 'fahd1', 'ENSGWIG00000015181', 'ENSGWIG00000015215', 'ndufb10', 'tbl3', 'ENSGWIG00000017554', 'sept9a', 'ENSGWIG00000017690', 'jpt1b', 'sumo2b', 'nup85', 'gga3b', 'slc25a19', 'ENSGWIG00000004771', 'rai1', 'srebf1', 'TOM1L2', 'drc3', 'ENSGWIG00000005723', 'ENSGWIG00000005728', 'ENSGWIG00000005735', 'zgc:153240', 'rhot1a', 'fam20ca', 'nptx2a', 'baiap2l1b', 'tecpr1b', 'lmtk2', 'pvalb8', 'ENSGWIG00000008124', 'cacng4b', 'cacng1b', 'ENSGWIG00000008129', 'ENSGWIG00000008131', 'rfx1a', 'ENSGWIG00000008284', 'ENSGWIG00000008299', 'ENSGWIG00000008316', 'fbxw9', 'si:dkey-117i10.1', 'dhps', 'ENSGWIG00000009616', 'ENSGWIG00000009617', 'sstr5', 'ENSGWIG00000009624', 'spag9a', 'tob1a', 'wfikkn2a', 'ankrd40', 'luc7l3', 'cacna1ia', 'ENSGWIG00000010686', 'lgals2a', 'sgsm3', 'ENSGWIG00000011777', 'ELFN2', 'ENSGWIG00000011810', 'ENSGWIG00000011811', 'ENSGWIG00000011817', 'ENSGWIG00000011821', 'ENSGWIG00000011820', 'gtf2f1', 'alkbh7', 'cacna1aa', 'ENSGWIG00000012264', 'nacc1a', 'ENSGWIG00000012268', 'ENSGWIG00000012275', 'sez6l2', 'xpo6', 'eps8l1a', 'slc6a16a', 'slc17a7a', 'ENSGWIG00000005565', 'tmem11', 'cramp1', 'mapk8ip3', 'spsb3a', 'atp6v0ca', 'ypel3', 'mapk3', 'atp2a1', 'slc9a3r1b', 'ENSGWIG00000012857', 'arhgdia', 'alyref', 'mcrip1', 'pecam1', 'ENSGWIG00000012880', 'nptx1l', 'ENSGWIG00000012882', 'MGAT5B', 'ENSGWIG00000012901', 'ENSGWIG00000012904', 'ENSGWIG00000012929', 'ENSGWIG00000013116', 'ENSGWIG00000013117', 'ENSGWIG00000013158', 'epn2', 'b9d1', 'cacna1ha', 'ENSGWIG00000014863', 'grin2aa', 'ENSGWIG00000016217', 'emp2', 'clec16a', 'socs1a', 'PRKCB', 'cacng3b', 'ENSGWIG00000019485', 'ENSGWIG00000019486', 'ENSGWIG00000019488', 'tnrc6a', 'ndufab1b', 'dctn5', 'usp7', 'slc2a6', 'clcn7', 'coro7', 'vasnb', 'glis2b', 'tfap4', 'ADCY9', 'crebbpb', 'ENSGWIG00000022993', 'DNAAF5', 'prkar1b', 'pdgfab', 'ENSGWIG00000023060', 'ENSGWIG00000023074', 'iqck', 'gprc5ba', 'proza', 'GPR139', 'ENSGWIG00000000351', 'kdelr2a', 'zgc:171489', 'cdr2a', 'sdr42e2', 'MOSMO', 'uqcrc2a', 'si:ch211-256e16.3', 'anks4b', 'ENSGWIG00000001093', 'stk19', 'DXO', 'ENSGWIG00000001161', 'prkcab', 'cep112', 'axin2', 'tex2', 'ern1', 'tbc1d16', 'cbx4', 'cbx8a', 'ENSGWIG00000007697', 'ENSGWIG00000007698', 'desi1a', 'flii', 'LLGL1', 'ENSGWIG00000010482', 'drg2', 'SYNGR3', 'znf598', 'hmox2b', 'ENSGWIG00000010522', 'ccnf', 'ENSGWIG00000017166', 'ENSGWIG00000017223', 'sox8b', 'ENSGWIG00000017231', 'si:ch73-105m5.1', 'usp22', 'sstr2b', 'ENSGWIG00000017298', 'sox9b', 'ENSGWIG00000017310', 'ENSGWIG00000017322', 'ENSGWIG00000017327', 'ENSGWIG00000017336', 'mafga', 'pycr1a', 'notum1a', 'cenpx', 'ENSGWIG00000018949', 'zgc:112492', 'timp2b', 'usp36', 'cyth1a', 'socs3a', 'TMEM235', 'fscn2a', 'bahcc1a', 'ENSGWIG00000019003', 'ENSGWIG00000019022', 'amn', 'ENSGWIG00000019263', 'ush1ga', 'otop2', 'ENSGWIG00000019377', 'ENSGWIG00000020851', 'ccdc137', 'pde6ga', 'ENSGWIG00000005817', 'tspan10', 'aatka', 'baiap2a', 'chmp6a', 'ttyh2l', 'sdk2a', 'cdc42ep4a', 'pmp22a', 'hs3st3b1a', 'si:ch211-114m9.1', 'ENSGWIG00000008286', 'metrnla', 'tbcd', 'znf750', 'fn3krp', 'epn3a', 'rasd2', 'hgs', 'mrpl12', 'gcgra', 'ENSGWIG00000012473', 'ENSGWIG00000012477', 'prkacaa', 'tecra', 'dnajb1a', 'gipc1', 'ENSGWIG00000013183', 'dnm2a', 'qtrt1', 'ENSGWIG00000013777', 'ENSGWIG00000013782', 'ENSGWIG00000013787', 'adgrl1a', 'ENSGWIG00000019888', 'ptger1c', 'slc27a1a', 'pgls', 'pbx4', 'col5a3a', 'ENSGWIG00000021530', 'olfm2a', 'trip10a', 'dnmt1', 's1pr2', 'ENSGWIG00000021922', 'ENSGWIG00000021924', 'CDC37', 'tyk2', 'pde4a', 'prkcsh', 'elavl3', 'dock6', 'ENSGWIG00000010747', 'ENSGWIG00000010745', 'nprl3', 'rhbdf1a', 'mgrn1b', 'ENSGWIG00000011641', 'shisa9b', 'unkl', 'ENSGWIG00000011661', 'baiap3', 'ENSGWIG00000011666', 'usp31', 'ENSGWIG00000011669', 'hs3st2', 'hs3st4', 'ENSGWIG00000011673', 'kdm8', 'zc3h7a', 'axin1', 'galr2a', 'uncx', 'micall2a', 'ENSGWIG00000012276', 'mafk', 'tmem184a', 'psmg3', 'ELFN1', 'mrm2', 'snx8a', 'ttyh3a', 'lfng', 'retreg3', 'fmnl1a', 'si:ch211-278a6.1', 'ENSGWIG00000013032', 'wnt9b', 'gpatch8', 'map3k14a', 'baxa', 'si:ch211-18i17.2', 'ENSGWIG00000013093', 'cntnap1', 'EZH1', 'ramp2', 'apnl', 'ENSGWIG00000000295', 'c1ql3b', 'ccdc43', 'ENSGWIG00000000328', 'ENSGWIG00000000352', 'mylk5', 'si:ch73-141c7.1', 'hsd17b1', 'stk17al', 'cavin1a', 'stat5a', 'si:ch211-210g13.5', 'hcrt', 'kcnh4a', 'rab5c', 'kat2a', 'dhx58', 'znf385c', 'dnajc7', 'cnp', 'ACLY', 'ENSGWIG00000000911', 'ENSGWIG00000001090', 'STAC2', 'fbxl20', 'cdk12', 'NEUROD2', 'ppp1r1b', 'stard3', 'tcap', 'rprml', 'mettl2a', 'mrc2', 'rnd2', 'vat1', 'phb', 'znf652', 'phospho1', 'eftud2', 'GJC1', 'smarce1', 'ENSGWIG00000013282', 'tspan4b', 'si:ch73-248e21.1', 'nosip', 'abat', 'mettl22', 'hao1', 'ENSGWIG00000014934', 'ENSGWIG00000015167', 'fbxl16', 'sept12', 'ARL4D', 'nbr1b', 'ENSGWIG00000016002', 'aoc2', 'ENSGWIG00000016006', 'timm29', 'yipf2', 'carm1', 'aldh3b1', 'ppp4cb', 'CSNK1D', 'ENSGWIG00000017095', 'dus1l', 'scpep1', 'ENSGWIG00000017432', 'rab11fip4a', 'cog1', 'rogdi', 'ENSGWIG00000017481', 'glyr1', 'ENSGWIG00000017515', 'si:dkeyp-72e1.9', 'gspt1l', 'ENSGWIG00000018594', 'pdxdc1', 'slc25a23b', 'gps1', 'ENSGWIG00000000845', 'prr12b', 'prrg2', 'ap2a1', 'prmt1', 'ENSGWIG00000004420', 'LIN7B', 'snrnp70', 'scaf1', 'ENSGWIG00000004467', 'rbfox1', 'ENSGWIG00000004539', 'ube2z', 'eve1', 'hoxb13a', 'hoxb9a', 'hoxb8a', 'ENSGWIG00000004765', 'hoxb3a', 'hoxb2a', 'ENSGWIG00000004915', 'igf2bp1', 'gna13b', 'ENSGWIG00000005390', 'zgc:86896', 'ENSGWIG00000005423', 'ENSGWIG00000005433', 'abcc1', 'st8sia6', 'cacnb1', 'arl5c', 'PLXDC1', 'lasp1', 'arhgap23a', 'hexim1', 'plcd3a', 'nmt1a', 'retsat', 'cdk21', 'svilc', 'tmem98', 'ENSGWIG00000022192', 'kcnj19a', 'znf281a', 'si:ch211-198m17.1', 'kcnh6a', 'gfap', 'map3k3', 'limd2', 'ddx42', 'strada', 'ENSGWIG00000022805', 'ENSGWIG00000022808', 'ENSGWIG00000022820', 'nxph3', 'ngfra', 'ENSGWIG00000022828', 'selenow2a', 'slc4a1a', 'ubtf', 'ENSGWIG00000024234', 'tmub2', 'asb16', 'ENSGWIG00000024239', 'ENSGWIG00000024241', 'ENSGWIG00000024243', 'mpp3a', 'ENSGWIG00000024265', 'nfil3', 'ENSGWIG00000024267', 'ENSGWIG00000024268', 'zgc:63569', 'kdelr3', 'ENSGWIG00000004777', 'ENSGWIG00000004796', 'ENSGWIG00000004806', 'si:dkey-167k11.5', 'srebf2', 'ndufa6', 'poldip3', 'U12', 'rrp7a', 'wbp2nl', 'ENSGWIG00000005702', 'ENSGWIG00000005760', 'tcf20', 'EMP1', 'ENSGWIG00000006173', 'ENSGWIG00000006178', 'GPRC5D', 'ENSGWIG00000006185', 'ENSGWIG00000006261', 'evplb', 'smurf2', 'kpna2', 'ENSGWIG00000008340', 'grb2b', 'mchr1b', 'usp43b', 'ntn1b', 'hmox1a', 'zgc:153733', 'tbc1d10b', 'hsd3b7', 'setd1a', 'hook2', 'junbb', 'ubn2b', 'myh9b', 'si:ch211-106h11.3', 'DDX39A', 'pkn1b', 'ENSGWIG00000018069', 'ENSGWIG00000018070', 'WIZ', 'ilf3b', 'farsa', 'ENSGWIG00000018299', 'ENSGWIG00000018328', 'sox10', 'ENSGWIG00000018640', 'tmem184bb', 'ENSGWIG00000000611', 'si:dkey-46g23.1', 'CARD10', 'cdc42ep1b', 'col1a1a', 'ENSGWIG00000021139', 'ENSGWIG00000021146', 'nags', 'ENSGWIG00000021149', 'ENSGWIG00000021151', 'psmd11a', 'NSF', 'ENSGWIG00000021273', 'prl', 'si:dkey-16p21.7', 'cpt1cb', 'ENSGWIG00000022650', 'nat15', 'carhsp1', 'caskin1', 'ENSGWIG00000023190', 'ENSGWIG00000023193', 'ENSGWIG00000023195', 'ENSGWIG00000023196', 'ENSGWIG00000023199', 'map2k4a', 'ENSGWIG00000023738', 'ENSGWIG00000023904', 'ENSGWIG00000023905', 'gas7a', 'rcvrna', 'ENSGWIG00000023914', 'ENSGWIG00000023973', 'si:ch73-281f12.4', 'grap2a', 'fam83fa', 'cyth4a', 'eif3d', 'sult3st1', 'ENSGWIG00000024442', 'CACNG2', 'si:dkey-151m15.5', 'ifi35', 'slc1a9', 'ENSGWIG00000004809', 'ccndx', 'akt1s1', 'zgc:195001', 'mybpc2a', 'lrrc4ba', 'med25', 'syt3', 'si:ch211-51c14.1', 'ENSGWIG00000010049', 'ENSGWIG00000010050', 'ENSGWIG00000010051', 'ppp1caa', 'coro1a', 'prf1.9', 'cox6a2', 'armc5', 'elob', 'ENSGWIG00000011049', 'si:ch211-262e15.1', 'hlfa', 'crlf3', 'ENSGWIG00000011146', 'tsen54', 'ENSGWIG00000011835', 'tmem104', 'grin2ca', 'rbfox3b', 'ENSGWIG00000011970', 'ENSGWIG00000011971', 'cbx2', 'nog1', 'pctp', 'ENSGWIG00000011978', 'mmd', 'ENSGWIG00000012017', 'grid2', 'ENSGWIG00000016036', 'atoh1a', 'me2', 'tspan36', 'mier3b', 'plk2b', 'pde4d', 'elovl7a', 'ercc8', 'ZSWIM6', 'ENSGWIG00000008694', 'ENSGWIG00000009145', 'slc35d2', 'znf367', 'ENSGWIG00000009171', 'ENSGWIG00000009174', 'fbxl17', 'EFNA5', 'ENSGWIG00000009206', 'gadd45ga', 'ghra', 'fbxo4', 'plcxd3', 'c7a', 'rgs7bpa', 'HTR1A', 'ipo11', 'hsd17b3', 'ercc6l2', 'ptch1', 'ENSGWIG00000016200', 'AOPEP', 'dock8', 'ENSGWIG00000016878', 'cdo1', 'AP3S1', 'COMMD10', 'sema6a', 'DMXL1', 'hsd17b4', 'prr16', 'si:ch211-193c2.2', 'sh2b3', 'atxn2', 'ostf1', 'nmrk1', 'ENSGWIG00000020458', 'si:ch1073-392o20.1', 'csgalnact1b', 'rnf165b', 'loxhd1b', 'znf366', 'oxt', 'spef2', 'ENSGWIG00000020888', 'lhfpl2b', 'ENSGWIG00000020918', 'SCAMP1', 'OTP', 'ckmt2a', 'rasgrf2b', 'msh3', 'ENSGWIG00000021783', 'ENSGWIG00000004890', 'adra2b', 'gpat2', 'pcna', 'KCNIP3', 'slc23a2', 'rassf2a', 'ENSGWIG00000005393', 'ENSGWIG00000005399', 'ENSGWIG00000009179', 'plpp5', 'cdc45', 'cldn5a', 'zgc:63587', 'sept5a', 'si:ch211-220d9.3', 'ikbkb', 'plat', 'ENSGWIG00000013108', 'cltcl1', 'si:dkey-178e17.1', 'pi4kab', 'serpind1', 'ENSGWIG00000018671', 'hic2', 'PRADC1', 'ENSGWIG00000018681', 'utp15', 'ENSGWIG00000018690', 'ENSGWIG00000018692', 'ykt6', 'myl7', 'agpat9l', 'adamts12', 'fgf10b', 'paip1', 'ndufs4', 'arl15a', 'SNX18', 'si:dkey-193c22.2', 'ENSGWIG00000019671', 'lipg', 'ENSGWIG00000019673', 'slc20a2', 'ENSGWIG00000019687', 'areg', 'ENSGWIG00000019810', 'mccc2', 'si:dkey-84j12.1', 'tars1', 'gas1a', 'ENSGWIG00000023187', 'ccdc84', 'fam214b', 'ENSGWIG00000023203', 'camk2b1', 'si:dkey-174n20.1', 'kremen1', 'ENSGWIG00000004831', 'xbp1', 'ENSGWIG00000004888', 'hspb8', 'suds3', 'taok3a', 'wsb2', 'ksr2', 'nos1', 'nipsnap1', 'ENSGWIG00000016765', 'fbxw8', 'nf2a', 'ENSGWIG00000016856', 'ENSGWIG00000016867', 'ENSGWIG00000016882', 'prodhb', 'rtn4r', 'si:ch211-207k7.4', 'ppil3', 'ENSGWIG00000017533', 'zgc:101858', 'ENSGWIG00000017538', 'si:dkey-21e5.1', 'camsap1b', 'notch1b', 'ENSGWIG00000022298', 'inpp5e', 'pmpca', 'ENSGWIG00000022325', 'ptgs1', 'si:ch211-236k19.2', 'tal2', 'zgc:172315', 'ENSGWIG00000023563', 'scarb2a', 'ENSGWIG00000023574', 'ENSGWIG00000023590', 'hint2', 'ENSGWIG00000024097', 'mfsd10', 'ENSGWIG00000024140', 'gldc', 'ENSGWIG00000024285', 'ENSGWIG00000024397', 'ENSGWIG00000024398', 'aacs', 'gak', 'ENSGWIG00000024562', 'erap1b', 'ENSGWIG00000024625', 'cds1', 'wdfy3', 'ENSGWIG00000025076', 'ENSGWIG00000025077', 'rusc2', 'ENSGWIG00000025080', 'unc13ba', 'sec14l8', 'rnf215', 'slc39a14', 'stambpb', 'ascc2', 'ENSGWIG00000002055', 'mapk1', 'grsf1', 'rufy3', 'slc4a4a', 'NPFFR2', 'adamts3', 'ENSGWIG00000018591', 'dmrt2a', 'dmrt3a', 'dmrt1', 'kank1a', 'fbp2', 'ctsla', 'dapk1', 'atp8b5a', 'CNNM4', 'ENSGWIG00000019979', 'purbb', 'dgcr8', 'tango2', 'ARVCF', 'txnrd2.1', 'gnb1l', 'tbx1', 'ENSGWIG00000021656', 'sema4c', 'ENSGWIG00000021659', 'ENSGWIG00000021665', 'kat6a', 'ap3m2', 'ENSGWIG00000021686', 'tet3', 'npffr1l2', 'hk2', 'mxd1', 'zgc:65851', 'cds2', 'arhgap25', 'ENSGWIG00000023152', 'znf703', 'ENSGWIG00000023165', 'ENSGWIG00000023168', 'pnpla7b', 'zgc:64051', 'araf', 'rabl6b', 'imp4', 'traf2a', 'tnc', 'ENSGWIG00000023515', 'nsmfb', 'ENSGWIG00000023537', 'lman2lb', 'tbx3b', 'htra4', 'ENSGWIG00000023560', 'unc5db', 'ENSGWIG00000024383', 'cyb561a3a', 'loxl2b', 'zmat4b', 'tcima', 'adam28', 'ppp3cca', 'vps37b', 'zgc:113436', 'ogfod2', 'si:ch211-48m9.1', 'zgc:110329', 'tacr1a', 'ncaph', 'vamp8', 'ENSGWIG00000025268', 'gcn1', 'rab35b', 'bicdl1', 'citb', 'adora2aa', 'specc1la', 'si:dkey-91m11.5', 'anapc7', 'ENSGWIG00000011665', 'ENSGWIG00000014832', 'rad9b', 'PPTC7', 'ENSGWIG00000014867', 'cux2b', 'MTMR3', 'si:ch211-236k19.4', 'ENSGWIG00000020693', 'ENSGWIG00000020730', 'niban2b', 'stxbp1b', 'noxa1', 'dpp7', 'man1b1b', 'ENSGWIG00000022080', 'ENSGWIG00000022081', 'ENSGWIG00000022082', 'cacna1ba', 'ehmt1a', 'arrdc1a', 'JMY', 'zgc:103670', 'unc45b', 'cert1a', 'polk', 'poc5', 'ankdd1b', 'sv2ca', 'iqgap2', 'f2rl2', 'ENSGWIG00000026551', 'si:ch211-130m23.3', 'ENSGWIG00000026559', 'ENSGWIG00000026566', 'si:ch211-130m23.5', 'tmem167a', 'ENSGWIG00000026572', 'vcana', 'hapln1a', 'ENSGWIG00000026581', 'ENSGWIG00000026582', 'ENSGWIG00000026583', 'edil3a', 'cox7c', 'rasa1a', 'ccnh', 'mef2cb', 'si:dkey-40c11.2', 'rhobtb4', 'wdr54', 'nol6', 'aqp3a', 'si:dkey-245n4.2', 'ENSGWIG00000026710', 'ENSGWIG00000026711', 'ENSGWIG00000026712', 'nr2f1a', 'fam172a', 'ENSGWIG00000026717', 'slf1', 'mctp1a', 'gig2p', 'ttc37', 'arsk', 'arrdc3a', 'ENSGWIG00000001466', 'adgrv1', 'ENSGWIG00000001474', 'si:dkey-40c11.1', 'lzts1', 'gfra2a', 'bco2l', 'ENSGWIG00000001840', 'fam219ab', 'klf9', 'trpm3', 'ENSGWIG00000012076', 'cemip2', 'abhd17b', 'zgc:101016', 'gda', 'entpd4', 'slc25a37', 'ENSGWIG00000015498', 'nkx2.7', 'stc1', 'ido1', 'rhobtb2a', 'PEBP4', 'bin3', 'acsl2', 'ENSGWIG00000021457', 'ENSGWIG00000021460', 'si:dkey-32e23.4', 'si:dkey-32e23.6', 'aak1a', 'gfpt1', 'ENSGWIG00000022947', 'kansl3', 'bmp1a', 'ENSGWIG00000025063', 'adrb3a', 'got1l1', 'ENSGWIG00000025068', 'adgra2', 'pdlim2', 'sorbs3', 'ENSGWIG00000025166', 'nkx6.3', 'inpp5l', 'GPAT4', 'ENSGWIG00000025182', 'zgc:77112', 'arid5a', 'dbnlb', 'ube2d4', 'tcf7l1a', 'zgc:153044', 'ENSGWIG00000025201', 'mn1b', 'ENSGWIG00000025206', 'TTC28', 'ENSGWIG00000025254', 'ENSGWIG00000025288', 'rph3ab', 'ENSGWIG00000025329', 'vps33a', 'ppp2r2aa', 'bnip3la', 'DPYSL2', 'adra1aa', 'tmem230b', 'igsf9b', 'zfand5a', 'TMC1', 'tmc2b', 'ALDH1A1', 'anxa1a', 'si:dkey-229d2.4', 'rorb', 'serpina7', 'bag4', 'ENSGWIG00000005800', 'lsm1', 'elmod3', 'si:ch1073-459j12.1', 'ENSGWIG00000005974', 'ENSGWIG00000005978', 'NPC1L1', 'ENSGWIG00000006123', 'kdm2ba', 'orai1a', 'ENSGWIG00000006469', 'ENSGWIG00000006471', 'hpdb', 'ENSGWIG00000006838', 'aplnra', 'tbc1d10aa', 'slc7a4', 'RASAL1', 'DTX1', 'slc20a1a', 'vgll4l', 'syn1', 'unm_hu7912', 'gapvd1', 'ENSGWIG00000007722', 'ENSGWIG00000007750', 'ENSGWIG00000008068', 'ENSGWIG00000008071', 'ENSGWIG00000008072', 'lmx1bb', 'zbtb34', 'ANGPTL2', 'ENSGWIG00000008110', 'ENSGWIG00000008117', 'ENSGWIG00000008125', 'zgc:154046', 'IER5L', 'asb6', 'si:dkeyp-14d3.1', 'slc15a4', 'glt1d1', 'TMEM132D', 'fzd10', 'rimbp2', 'ENSGWIG00000009029', 'stx2b', 'adgrd1', 'ulk1a', 'ep400', 'ncor2', 'rflna', 'ccdc92', 'chmp7', 'lgi3', 'fgfr1a', 'ankrd39', 'kctd9a', 'ENSGWIG00000019938', 'ENSGWIG00000019941', 'npy8br', 'klhl22', 'tbx16', 'cabin1', 'neflb', 'dock5', 'ENSGWIG00000020925', 'vsig8b', 'ENSGWIG00000020930', 'lrrtm4l2', 'ENSGWIG00000020933', 'ENSGWIG00000020936', 'ENSGWIG00000001467', 'rnf181', 'ENSGWIG00000001486', 'pxnb', 'ctu1', 'crybb2', 'ENSGWIG00000001692', 'ENSGWIG00000001695', 'si:ch73-138n13.1', 'rnf10', 'pop5', 'unc119.1', 'ENSGWIG00000002134', 'zgc:162025', 'hnf1a', 'msi1', 'prkab1b', 'ENSGWIG00000002179', 'ENSGWIG00000002180', 'ENSGWIG00000002185', 'hps4', 'ENSGWIG00000002198', 'mlec', 'cabp1a', 'asphd2', 'ggt5a', 'ENSGWIG00000002227', 'lrrc75ba', 'si:dkey-222h21.12', 'anapc5', 'golga3', 'trmt1l', 'fbrsl1', 'ENSGWIG00000008342', 'ENSGWIG00000008345', 'ENSGWIG00000008348', 'galnt9', 'ENSGWIG00000008364', 'atad1a', 'ENSGWIG00000008369', 'htr7c', 'tnksb', 'ENSGWIG00000009063', 'ENSGWIG00000009068', 'zdhhc8b', 'pgam5', 'ENSGWIG00000009076', 'ankle2', 'ENSGWIG00000009133', 'ENSGWIG00000009137', 'ENSGWIG00000009191', 'ENSGWIG00000009207', 'ENSGWIG00000009210', 'ENSGWIG00000009211', 'ENSGWIG00000009215', 'ENSGWIG00000009228', 'endog', 'ENSGWIG00000009237', 'cmlc1', 'ENSGWIG00000009240', 'ENSGWIG00000009241', 'ENSGWIG00000009247', 'ENSGWIG00000009249', 'gstt2', 'ENSGWIG00000009254', 'ENSGWIG00000009255', 'OGDH', 'ENSGWIG00000009456', 'adam9', 'ercc6l', 'ENSGWIG00000009536', 'ENSGWIG00000009537', 'ENSGWIG00000004849', 'ENSGWIG00000004883', 'ENSGWIG00000004902', 'kazald3', 'ENSGWIG00000004942', 'ENSGWIG00000005644', 'ENSGWIG00000011883', 'ENSGWIG00000011884', 'ENSGWIG00000011885', 'ENSGWIG00000011887', 'ENSGWIG00000011889', 'qsox2', 'ENSGWIG00000011900', 'ENSGWIG00000011902', 'nudt18', 'ENSGWIG00000011907', 'ENSGWIG00000011909', 'ENSGWIG00000011934', 'ENSGWIG00000011943', 'ENSGWIG00000011951', 'srp19', 'ENSGWIG00000011993', 'ENSGWIG00000013698', 'ENSGWIG00000013702', 'ENSGWIG00000013705', 'PIP5KL1', 'dhfr', 'gtf2h2', 'ENSGWIG00000014180', 'ENSGWIG00000014181', 'marveld2b', 'ENSGWIG00000001499', 'ENSGWIG00000001491', 'CRB2', 'lhx2b', 'ENSGWIG00000001573', 'nr5a1a', 'nr6a1a', 'ENSGWIG00000001676', 'olfml2a', 'ENSGWIG00000001706', 'im:7150988', 'tubb2b', 'tfip11', 'thoc5', 'fancg', 'vcp', 'akr1a1a', 'dnajb5', 'ENSGWIG00000002396', 'ENSGWIG00000002384', 'vps13a', 'ENSGWIG00000002484', 'ENSGWIG00000002491', 'rfk', 'pcsk5b', 'ubap2b', 'ENSGWIG00000007739', 'ENSGWIG00000007851', 'ENSGWIG00000007855', 'mlxip', 'clip1a', 'zcchc8', 'rsrc2', 'kntc1', 'hip1rb', 'ENSGWIG00000009097', 'rilpl2', 'si:ch211-275j6.5', 'ENSGWIG00000009107', 'PITPNM2', 'arl6ip4', 'mthfd2', 'mob1a', 'trafd1', 'smtnb', 'inpp5jb', 'si:ch211-283h6.4', 'usp39', 'sftpbb', 'ENSGWIG00000010367', 'ebf2', 'ank1a', 'dguok', 'si:ch211-106a19.1', 'rasl10a', 'ap1b1', 'gas2l1', 'rhbdd3', 'ENSGWIG00000013115', 'atp6v0a2b', 'si:ch211-225b11.4', 'tpcn1', 'rbm19', 'ENSGWIG00000007801', 'sart3', 'coro1ca', 'ssh1a', 'svopa', 'ENSGWIG00000013492', 'ENSGWIG00000013496', 'ankrd13a', 'git2a', 'trpv4', 'fam222a', 'mvk', 'aldh3b2', 'ube3b', 'foxn4', 'ENSGWIG00000020594', 'pole', 'p2rx2', 'ENSGWIG00000021104', 'si:dkey-98f17.5', 'slc2a11b', 'ENSGWIG00000021129', 'ENSGWIG00000021124', 'ENSGWIG00000021134', 'ENSGWIG00000021135', 'ENSGWIG00000021136', 'cmklrl2', 'wscd2', 'si:ch211-225b11.1', 'limk2', 'pik3ip1', 'ENSGWIG00000022346', 'ENSGWIG00000022348', 'ENSGWIG00000022351', 'si:ch211-204d2.4', 'ddhd2', 'si:ch211-166a6.5', 'ENSGWIG00000022415', 'ENSGWIG00000022421', 'ENSGWIG00000022423', 'ccn1l2', 'si:ch73-199g24.2', 'fgf17', 'ENSGWIG00000022575', 'ENSGWIG00000022576', 'nelfb', 'ENSGWIG00000001497', 'homer1b', 'tent2', 'cmya5', 'thbs4a', 'cdk7', 'setbp1', 'slc14a2', 'lnpep', 'ENSGWIG00000003319', 'erap2', 'rgmb', 'ENSGWIG00000003349', 'bri3bp', 'ENSGWIG00000003404', 'ENSGWIG00000003417', 'si:ch73-72b7.1', 'ccdc88b', 'macrod1', 'flrt1b', 'ENSGWIG00000006877', 'fermt3b', 'zbtb3', 'ENSGWIG00000008234', 'ENSGWIG00000008279', 'si:ch73-45o6.2', 'tgfb5', 'kcnk4a', 'si:dkeyp-115e12.6', 'stx5a', 'ENSGWIG00000008545', 'mbnl3', 'rap2c', 'zgc:92907', 'ENSGWIG00000008768', 'si:zfos-80g12.1', 'foxo4', 'ubl3b', 'prss23', 'cfl1', 'atl3', 'si:ch211-145o7.3', 'yif1a', 'ENSGWIG00000012905', 'fosl1a', 'fibpa', 'efemp2a', 'npas4a', 'mrpl11', 'tmem265', 'ENSGWIG00000013167', 'slc25a43', 'lnx2b', 'chic1', 'cdx4', 'kdrl', 'tex11', 'ENSGWIG00000001630', 'dlg3', 'gdpd2', 'pdzd11', 'stard14', 'fgl1', 'ENSGWIG00000017308', 'ENSGWIG00000017318', 'nkx1.2lb', 'uvssa', 'sil1', 'csnk1a1', 'ENSGWIG00000017359', 'ENSGWIG00000017369', 'ENSGWIG00000017375', 'ENSGWIG00000017386', 'ENSGWIG00000017392', 'zgc:153146', 'dnd1', 'ENSGWIG00000017790', 'si:ch211-168f7.5', 'ttc9c', 'ENSGWIG00000017795', 'ENSGWIG00000017798', 'ENSGWIG00000017801', 'ENSGWIG00000017809', 'pola2', 'ENSGWIG00000018303', 'si:dkey-34l15.1', 'si:ch1073-303k11.2', 'SLC16A2', 'f8', 'ENSGWIG00000018323', 'pmt', 'mid2', 'si:ch211-220e11.3', 'pcdh11', 'ENSGWIG00000018718', 'ENSGWIG00000018719', 'klhl4', 'AHNAK', 'eef1g', 'si:ch211-175m2.5', 'ENSGWIG00000019316', 'ENSGWIG00000019322', 'ENSGWIG00000019327', 'chrnb3b', 'CHRNA6', 'chrnb5b', 'ENSGWIG00000019380', 'ENSGWIG00000019452', 'stx3a', 'rad9a', 'ENSGWIG00000022480', 'p2rx3a', 'ssrp1a', 'clcf1', 'slc43a1a', 'zdhhc5b', 'tmx2a', 'clp1', 'rtn4rl2b', 'slc43a3b', 'ENSGWIG00000023085', 'ENSGWIG00000023086', 'ENSGWIG00000004941', 'CTNND1', 'ENSGWIG00000008061', 'cabp2b', 'ENSGWIG00000008076', 'ENSGWIG00000008084', 'PPP2R2C', 'wfs1b', 'jakmip1', 'ENSGWIG00000009494', 'ENSGWIG00000009495', 'ENSGWIG00000009500', 'TLX3', 'hrh2a', 'DOCK2', 'ENSGWIG00000011384', 'ENSGWIG00000011586', 'ENSGWIG00000016616', 'ENSGWIG00000016618', 'st3gal5', 'ENSGWIG00000016623', 'setd7', 'igbp1', 'pcdh10b', 'ENSGWIG00000020435', 'ENSGWIG00000020436', 'pcdh18b', 'slc7a11', 'nocta', 'ENSGWIG00000020533', 'rnf103', 'ENSGWIG00000020539', 'immt', 'enpp6', 'trpc7b', 'ENSGWIG00000020590', 'DIAPH1', 'ENSGWIG00000020828', 'pcdh2ac', 'ENSGWIG00000005093', 'si:ch73-233f7.1', 'fgf18a', 'fbxw11a', 'ENSGWIG00000005716', 'etf1a', 'ENSGWIG00000005722', 'spock1', 'bicc2', 'APBB2', 'ENSGWIG00000005748', 'CHRNA9', 'ube2ka', 'ints10', 'ENSGWIG00000005811', 'ENSGWIG00000005819', 'slc25a51b', 'ENSGWIG00000005827', 'si:ch211-203d1.3', 'tcirg1b', 'prpf19', 'ENSGWIG00000001755', 'ENSGWIG00000001758', 'rnf20', 'si:dkey-74k8.3', 'tmco6', 'ENSGWIG00000003755', 'ENSGWIG00000003776', 'pdgfrb', 'csf1ra', 'hmgxb3', 'ENSGWIG00000005010', 'ndst1a', 'rbm22', 'ENSGWIG00000005562', 'arl3l2', 'rgs14a', 'ENSGWIG00000005591', 'adra2da', 'si:ch211-199b20.3', 'cnot6b', 'ENSGWIG00000005765', 'ENSGWIG00000008196', 'KIAA1191', 'gfpt2', 'mapk9', 'ENSGWIG00000012703', 'ENSGWIG00000012705', 'RASGEF1C', 'clk4a', 'ENSGWIG00000012782', 'ENSGWIG00000012785', 'hnrnpaba', 'FAM53C', 'phykpl', 'ctbp1', 'spon2a', 'kdm3b', 'REEP2', 'egr1', 'ENSGWIG00000016652', 'ENSGWIG00000016654', 'adra1bb', 'FABP6', 'ccnjl', 'c1qtnf2', 'slu7', 'ENSGWIG00000016774', 'sfxn1', 'atp7a', 'fbxl3l', 'ENSGWIG00000019946', 'gabrb2a', 'gabra6a', 'nudcd2', 'gdf9', 'hspa4b', 'fgf1a', 'nr3c1', 'kctd16a', 'sh3rf2', 'ENSGWIG00000021272', 'atrx', 'fgf16', 'zdhhc15b', 'abcb7', 'nexmifb', 'glod5', 'ENSGWIG00000023887', 'irg1l', 'si:ch211-153b23.4', 'ENSGWIG00000004973', 'ENSGWIG00000004997', 'frmpd3', 'ENSGWIG00000005273', 'zgc:66447', 'znf711', 'kctd12b', 'ENSGWIG00000005302', 'ENSGWIG00000005310', 'rhogd', 'ENSGWIG00000005324', 'ogt.1', 'gcna', 'cetn2', 'nsdhl', 'ENSGWIG00000008669', 'rab9b', 'plp1a', 'pcdh20', 'nlgn3a', 'ENSGWIG00000008714', 'eda', 'ENSGWIG00000013963', 'ENSGWIG00000013968', 'cx31.7', 'mtnr1c', 'neurl1b', 'dusp1', 'flt4', 'trim105', 'prelid1a', 'fam193b', 'mxd3', 'DDX41', 'ENSGWIG00000014476', 'unc5a', 'pdlim7', 'cltb', 'higd2a', 'ankhd1', 'hnrnph1', 'hbegfa', 'rufy1', 'ENSGWIG00000018872', 'n4bp3', 'ENSGWIG00000018891', 'anxa6', 'tnip1', 'gpx3', 'dctn4', 'ENSGWIG00000024466', 'ENSGWIG00000024467', 'tspan17', 'mchr2', 'ctnna1', 'lrrtm2', 'sncb', 'ENSGWIG00000024712', 'ENSGWIG00000024713', 'ENSGWIG00000024715', 'rnf44', 'pin4', 'nek12', 'ENSGWIG00000024722', 'ENSGWIG00000024724', 'cpeb4a', 'stc2a', 'nkx2.5', 'rpl26', 'GPR151', 'adam19b', 'clint1a', 'ebf3a', 'il12ba', 'adrb2a', 'sh3tc2', 'ablim3', 'afap1l1a', 'gabrp', 'foxi3b', 'tmsb2', 'gria1a', 'cyfip2', 'hmmr', 'ENSGWIG00000007607', 'si:dkey-34l15.2', 'ENSGWIG00000009195', 'eif4ebp3', 'acsl4a', 'nxt2', 'xiap', 'stag2b', 'efnb1', 'ENSGWIG00000011862', 'msnb', 'asb12b', 'arhgef9b', 'tenm1', 'agtr2', 'ENSGWIG00000020702', 'si:ch211-26b3.4', 'rps6kal', 'POU3F4', 'ENSGWIG00000025625', 'ENSGWIG00000025626', 'sh3bgrl', 'ENSGWIG00000025630', 'TBX22', 'ENSGWIG00000025633', 'ITM2A', 'p2ry10', 'ENSGWIG00000025639', 'ENSGWIG00000025640', 'ENSGWIG00000025644', 'slc7a2', 'mtmr7a', 'cnot7', 'fgf20b', 'ENSGWIG00000025806', 'her5', 'pfdn6', 'mmgt1', 'ints6l', 'mospd1', 'fam122b', 'hprt1', 'phf6', 'ENSGWIG00000025828', 'gpc3', 'gpc4', 'brd8', 'sting1', 'ppp3cb', 'ube2d2', 'cxxc5b', 'ENSGWIG00000026187', 'psd2', 'nrg2b', 'si:dkey-160o24.3', 'ap1ar', 'ENSGWIG00000001641', 'gabra4', 'GABRA2', 'si:ch73-49o8.1', 'ENSGWIG00000001950', 'LONRF3', 'ENSGWIG00000002001', 'ENSGWIG00000002005', 'dcps', 'ids', 'ENSGWIG00000002386', 'ENSGWIG00000002388', 'fmr1', 'ENSGWIG00000003474', 'ENSGWIG00000003479', 'hdac3', 'myot', 'ENSGWIG00000003690', 'il1rapl2', 'ENSGWIG00000007323', 'ENSGWIG00000007330', 'snx25', 'inppl1b', 'P2RY4', 'ENSGWIG00000008159', 'atp11c', 'f9a', 'FGF13', 'zic6', 'ENSGWIG00000014738', 'gpr101', 'tm9sf5', 'rbmx', 'arhgef6', 'ENSGWIG00000016199', 'map7d3', 'fhl1a', 'slc9a6a', 'aipl2', 'drp2', 'cenpi', 'cstf2', 'bcorl1', 'ENSGWIG00000019910', 'mid1ip1l', 'ENSGWIG00000019918', 'tspan7', 'srpx2', 'sytl4', 'trmt12', 'ocrl', 'ENSGWIG00000021044', 'ENSGWIG00000021045', 'ENSGWIG00000021048', 'xpnpep2', 'sash3', 'zdhhc9', 'si:ch73-335m24.2', 'clic2', 'lamp2', 'tmem255a', 'upf3b', 'ENSGWIG00000021443', 'sept6', 'nkrf', 'ube2a', 'elf1', 'tnmd', 'pcdh19', 'diaph2', 'glra4a', 'gla', 'rpl36a', 'btk', 'timm8a', 'zgc:101583', 'mars2', 'si:ch211-221f10.2', 'enox2', 'SLC4A11', 'gfra4b', 'smyd5', 'sfxn5b', 'ENSGWIG00000020090', 'BEND4', 'shisa3', 'atp8a1', 'grxcr1a', 'slc30a9', 'gsr', 'ENSGWIG00000021620', 'ENSGWIG00000021621', 'ENSGWIG00000021625', 'HS3ST1', 'ENSGWIG00000021627', 'ENSGWIG00000021632', 'ENSGWIG00000021638', 'COPS6', 'ENSGWIG00000021774', 'mcm7', 'ENSGWIG00000021923', 'gpc2', 'slit3', 'fgfrl1a', 'rnf212', 'si:ch211-255i20.3', 'zgc:136929', 'maea', 'ENSGWIG00000022853', 'ENSGWIG00000022854', 'klhl3', 'fam13b', 'cxcl14', 'ENSGWIG00000022880', 'lingo2a', 'ENSGWIG00000022883', 'ENSGWIG00000022884', 'eif4ea', 'zgc:77938', 'pitx2', 'elovl6', 'egf', 'lrit3a', 'gpm6aa', 'asb5a', 'si:dkey-237h12.3', 'ENSGWIG00000023620', 'aga', 'neil3', 'pdgfc', 'GLRB', 'gria2b', 'gask1b', 'tmem144b', 'rxfp1', 'etfdh', 'rapgef2', 'ENSGWIG00000013525', 'fstl5', 'qdpra', 'ldb2a', 'tapt1a', 'prom1a', 'ccna2', 'bbs7', 'tmem33', 'phox2bb', 'limch1b', 'ENSGWIG00000015331', 'ENSGWIG00000015335', 'ENSGWIG00000015338', 'ENSGWIG00000015339', 'lbx2', 'pcgf1', 'fgf24', 'npm1b', 'tlx2', 'adad1', 'ENSGWIG00000015364', 'bbs12', 'fgf2', 'SPATA5', 'spry1', 'ANKRD50', 'ENSGWIG00000015470', 'ENSGWIG00000015472', 'U3', 'ENSGWIG00000015479', 'zgc:92045', 'ENSGWIG00000015485', 'ENSGWIG00000016281', 'ENSGWIG00000016283', 'ENSGWIG00000016287', 'sec24b', 'NCBP1', 'pde6a', 'ppargc1b', 'arhgef37', 'ENSGWIG00000020635', 'ENSGWIG00000020637', 'PCGF3', 'SLC49A3', 'arap3', 'polr2b', 'ENSGWIG00000020652', 'noa1', 'rest', 'atox1', 'slc36a1', 'ENSGWIG00000020670', 'ENSGWIG00000020672', 'tbc1d2', 'slc44a1b', 'ENSGWIG00000020812', 'tgfbi', 'B4GALT1', 'glra1', 'ENSGWIG00000002623', 'slc25a48', 'thoc3', 'gm2a', 'emx3', 'b4galt7', 'si:rp71-1d10.5', 'ENSGWIG00000002676', 'ENSGWIG00000002680', 'pcdh1b', 'ENSGWIG00000002683', 'ENSGWIG00000002687', 'nfatc1', 'atp9b', 'sall3b', 'ENSGWIG00000020991', 'galr1a', 'ENSGWIG00000020995', 'ENSGWIG00000021019', 'ENSGWIG00000021400', 'tekt2', 'slc9a3.2', 'ENSGWIG00000023302', 'tomm40l', 'xkr8.3', 'si:ch211-195b13.1', 'myo1g', 'ENSGWIG00000005335', 'ENSGWIG00000005336', 'hivep1', 'ENSGWIG00000005345', 'tmem170b', 'nedd9', 'fhl3b', 'si:ch211-193k19.1', 'tfap2e', 'psmb2', 'ENSGWIG00000007042', 'CLSPN', 'ago4', 'ENSGWIG00000012125', 'c19h1orf109', 'cdca8', 'ENSGWIG00000012129', 'ENSGWIG00000012167', 'ENSGWIG00000012867', 'prdm1b', 'ENSGWIG00000013478', 'zgc:101785', 'ext1c', 'ENSGWIG00000013506', 'ENSGWIG00000013513', 'sdc2', 'angpt1', 'ENSGWIG00000013532', 'SAMD12', 'ext1b', 'med30', 'slc4a7', 'eomesa', 'ENSGWIG00000013668', 'TGFBR2', 'STT3B', 'OSBPL10', 'tmem42b', 'clec3bb', 'ENSGWIG00000014787', 'ENSGWIG00000014885', 'rps5', 'ENSGWIG00000014949', 'ENSGWIG00000014951', 'spaca4l', 'csnk2b', 'prrc2a', 'neu1', 'zbtb12.1', 'slc44a4', 'RNF5', 'utp23', 'ENSGWIG00000019990', 'ENSGWIG00000019991', 'adgrb1b', 'hnrnpr', 'atp5if1a', 'clk2a', 'cers2a', 'ENSGWIG00000001625', 'ciarta', 'si:ch211-81a5.1', 'ash1l', 'ENSGWIG00000003596', 'si:ch211-39a7.1', 'efna1a', 'efna3a', 'cks1b', 'pbxip1a', 'ENSGWIG00000003810', 's100u', 's100s', 'cgnb', 'znf687b', 'PIP5K1A', 'hcn3', 'pip4p2', 'tmem64', 'cndp2', 'ENSGWIG00000004584', 'ENSGWIG00000004604', 'zgc:63863', 'DGAT1', 'ENSGWIG00000009796', 'SLC45A4', 'dennd3a', 'ENSGWIG00000009877', 'clptm1l', 'rplp1', 'ENSGWIG00000011277', 'trps1', 'csmd3a', 'ENSGWIG00000011602', 'ENSGWIG00000011604', 'ENSGWIG00000011606', 'ENSGWIG00000011607', 'ENSGWIG00000011632', 'ENSGWIG00000011637', 'vps72a', 'lingo4a', 'ENSGWIG00000011649', 'ENSGWIG00000011682', 'SV2A', 'sf3b4', 'LIX1L', 'pabpc1b', 'ENSGWIG00000013077', 'ywhaz', 'ncaldb', 'zgc:136493', 'ptpn2b', 'psmg2', 'ENSGWIG00000013358', 'ENSGWIG00000001744', 'rpl30', 'laptm4b', 'mtdha', 'rrm2b', 'bloc1s4', 'pard6gb', 'jarid2b', 'dtnbp1b', 'mylipa', 'gmpr', 'atxn1a', 'fam8a1a', 'pag1', 'znf704', 'tpd52', 'ENSGWIG00000001971', 'hey1', 'stmn2b', 'si:dkeyp-120h9.1', 'ripor2', 'MTERF1', 'AKAP9', 'krit1', 'si:dkey-266f7.9', 'ENSGWIG00000004659', 'prcc', 'ENSGWIG00000004703', 'MEF2D', 'ENSGWIG00000004748', 'RPS19', 'ENSGWIG00000004802', 'ENSGWIG00000004804', 'chrac1', 'lrrc14b', 'uqcrb', 'nup153', 'ENSGWIG00000005005', 'cap2', 'ENSGWIG00000005036', 'vps28', 'grinaa', 'ENSGWIG00000005121', 'fam83hb', 'ankib1a', 'snrpd1', 'adnp2b', 'azin1b', 'atp6v1c1b', 'ENSGWIG00000005308', 'fam91a1', 'lratd2', 'trib1', 'ENSGWIG00000005463', 'fam92a1', 'triqk', 'runx1t1', 'otud6b', 'ZNF385D', 'ubqln4', 'TINAGL1', 'cfap300', 'KCNQ4', 'nfyc', 'ENSGWIG00000002039', 'rprd1a', 'fkbp9', 'crtap', 'susd5', 'fbxl2', 'clasp2', 'ubp1', 'pdcd6ip', 'ENSGWIG00000010690', 'cyhr1', 'fabp3', 'snrnp40', 'nkain1', 'pum1', 'SDC3', 'ENSGWIG00000015477', 'sesn2', 'yrk', 'ahdc1', 'ENSGWIG00000019883', 'efcab1', 'sytl1', 'elmo1', 'si:dkey-184p18.2', 'poc1bl', 'galnt12', 'gabbr2', 'elp2', 'ctnnal1', 'sla1a', 'ccn4a', 'ndrg1a', 'ENSGWIG00000022290', 'cpne4b', 'mrpl3', 'si:dkey-240h12.4', 'fam126a', 'ENSGWIG00000023272', 'rapgef5a', 'ENSGWIG00000023277', 'cdca7b', 'sp4', 'ENSGWIG00000023285', 'itgb8', 'ENSGWIG00000023295', 'ferd3l', 'ENSGWIG00000023298', 'snx13', 'ENSGWIG00000024191', 'fam133b', 'cdk6', 'hepacam2', 'vps50', 'calcr', 'tfpi2', 'bet1', 'col1a2', 'ppp1r9a', 'pdk4', 'DYNC1I1', 'sem1', 'dlx6a', 'ENSGWIG00000025573', 'ccdc126', 'fam221a', 'tra2a', 'igf2bp3', 'gpnmb', 'ENSGWIG00000001689', 'kcnh8', 'tbc1d5', 'ENSGWIG00000007940', 'ENSGWIG00000007946', 'rftn1a', 'oxnad1', 'ENSGWIG00000007962', 'ENSGWIG00000007967', 'nfe2l3', 'cbx3a', 'snx10a', 'hoxa1a', 'hoxa5a', 'ENSGWIG00000008000', 'hoxa13a', 'hibadha', 'jazf1a', 'tax1bp1a', 'ENSGWIG00000008058', 'tril', 'osbpl3a', 'ENSGWIG00000010818', 'rims3', 'smap2', 'trappc3', 'col8a2', 'ENSGWIG00000010826', 'ptp4a2b', 'ENSGWIG00000010831', 'ENSGWIG00000010833', 'ENSGWIG00000010834', 'adgrb2', 'col9a2', 'angpt2b', 'ENSGWIG00000011118', 'csmd2', 'ENSGWIG00000011627', 'ENSGWIG00000011629', 'smim12', 'cx39.4', 'ENSGWIG00000011646', 'dlgap3', 'zbtb8b', 'ENSGWIG00000011667', 'gstr', 'fhod3a', 'SLC6A19', 'slc6a18', 'tert', 'ENSGWIG00000012368', 'yars1', 'inip', 'mfsd2ab', 'myclb', 'trit1', 'fam49al', 'txnl4a', 'ENSGWIG00000012446', 'mturn', 'sh3bp5a', 'si:dkey-166k12.1', 'ENSGWIG00000012618', 'cckb', 'trim71', 'ENSGWIG00000005157', 'ENSGWIG00000005174', 'rpl14', 'ENSGWIG00000005441', 'ENSGWIG00000008920', 'ENSGWIG00000009031', 'si:dkeyp-46h3.1', 'adcy2b', 'tent4a', 'srd5a1', 'nsun2', 'ENSGWIG00000013359', 'ube2ql1', 'med10', 'ENSGWIG00000013365', 'ndufs6', 'lpcat1', 'nrsn1l', 'sox4a', 'ENSGWIG00000013393', 'ENSGWIG00000013395', 'ENSGWIG00000013396', 'ldlrap1a', 'fndc5b', 'ENSGWIG00000013405', 'ENSGWIG00000013407', 'ENSGWIG00000013408', 'marcksl1b', 'hdac1', 'txlna', 'stmn1b', 'ENSGWIG00000013991', 'ENSGWIG00000014035', 'ENSGWIG00000014043', 'phactr4a', 'ENSGWIG00000014074', 'oprd1a', 'ENSGWIG00000014077', 'ENSGWIG00000014084', 'ENSGWIG00000014093', 'tent5bb', 'ENSGWIG00000014096', 'zdhhc18b', 'nudc', 'tbxta', 'arid1ab', 'ENSGWIG00000014131', 'tpbga', 'si:dkey-218f9.10', 'kcnk9', 'ENSGWIG00000014159', 'ENSGWIG00000014170', 'ENSGWIG00000014173', 'FAM135B', 'ENSGWIG00000014846', 'ENSGWIG00000014847', 'KHDRBS3', 'ENSGWIG00000001516', 'ENSGWIG00000001536', 'ENSGWIG00000001592', 'ENSGWIG00000001699', 'ENSGWIG00000001718', 'cpsf1', 'oxsm', 'ENSGWIG00000004384', 'lrrc3b', 'thsd7aa', 'nxph1', 'phf14', 'naa20', 'glcci1a', 'umad1', 'rpa3', 'COL28A1', 'ENSGWIG00000008019', 'ENSGWIG00000008022', 'si:dkey-202e17.1', 'sdhaf3', 'tac1', 'ENSGWIG00000008034', 'asns', 'ENSGWIG00000008050', 'eepd1', 'ENSGWIG00000008056', 'ngly1', 'top2b', 'ENSGWIG00000009806', 'ENSGWIG00000009809', 'ENSGWIG00000009811', 'ENSGWIG00000009812', 'ENSGWIG00000009837', 'ENSGWIG00000010275', 'thrb', 'nr1d2b', 'ENSGWIG00000010302', 'nkiras1', 'ENSGWIG00000010309', 'ENSGWIG00000010310', 'sf3a3', 'si:ch211-30b16.2', 'lin28a', 'pdik1l', 'ENSGWIG00000010337', 'foxo6b', 'ENSGWIG00000010340', 'ENSGWIG00000010341', 'cited4a', 'ctps1a', 'rab42a', 'gmeb1', 'ptprua', 'ENSGWIG00000010394', 'ENSGWIG00000010395', 'ENSGWIG00000010397', 'ENSGWIG00000007878', 'ENSGWIG00000008569', 'zswim6', 'mat2al', 'si:ch73-234b20.5', 'hnrnpd', 'sec31a', 'lin54', 'cops4', 'plac8.1', 'adamts6', 'cwc27', 'mzt2b', 'lhx5', 'ENSGWIG00000002253', 'cert1b', 'ENSGWIG00000008165', 'ch25hl2', 'crhbp', 'tbca', 'ap3b1a', 'bhmt', 'HOMER1', 'ENSGWIG00000010915', 'si:ch211-127d4.3', 'ENSGWIG00000010962', 'unm_sa808', 'zfyve16', 'PURB', 'gnsb', 'mrps24', 'ogdhb', 'zmiz2', 'ENSGWIG00000013511', 'psmb10', 'ENSGWIG00000013517', 'ENSGWIG00000013535', 'selenoh', 'hmgcs1', 'ENSGWIG00000013565', 'ENSGWIG00000013954', 'scai', 'odf2a', 'ENSGWIG00000014597', 'setb', 'pkn3', 'zdhhc12b', 'uap1l1', 'slc25a25b', 'lmbrd2b', 'ppp1r3b', 'mfhas1', 'nkx6.1', 'ENSGWIG00000005893', 'xpo7', 'COX5B', 'ENSGWIG00000007813', 'actr1', 'ugcg', 'hsdl2', 'ENSGWIG00000007932', 'ENSGWIG00000007933', 'capslb', 'prpf4', 'ENSGWIG00000008017', 'ufc1', 'ythdc1', 'pisd', 'apc', 'ENSGWIG00000008043', 'ENSGWIG00000008106', 'whrna', 'ENSGWIG00000009960', 'ENSGWIG00000010003', 'ehmt1b', 'ssna1', 'npr2', 'ENSGWIG00000014983', 'TMC2', 'ENSGWIG00000015003', 'ENSGWIG00000015040', 'ENSGWIG00000015126', 'apba1b', 'tjp2a', 'pip5k1ba', 'efna5a', 'ENSGWIG00000018576', 'ENSGWIG00000018599', 'ENSGWIG00000018600', 'c6', 'htr1ab', 'ENSGWIG00000019397', 'atp6v1g1', 'ENSGWIG00000006959', 'ENSGWIG00000007019', 'ENSGWIG00000007227', 'zcchc9', 'ENSGWIG00000007243', 'hapln1b', 'cetn3', 'ENSGWIG00000007250', 'mctp1b', 'sept5b', 'ENSGWIG00000010838', 'acads', 'grk3', 'ENSGWIG00000012798', 'sez6l', 'scarb1', 'grk5', 'pnx', 'aebp1', 'ddx56', 'rasgrf2a', 'ENSGWIG00000016396', 'gas1b', 'dab2', 'ENSGWIG00000016772', 'rtkn2b', 'ENSGWIG00000016778', 'ENSGWIG00000016779', 'unc5da', 'ENSGWIG00000016871', 'sgtb', 'trappc13', 'nln', 'erbin', 'MAST4', 'pik3r1', 'zmp:0000000846', 'palm2akap2', 'snx30', 'snx18b', 'ENSGWIG00000021318', 'rasgef1bb', 'antxr2b', 'si:dkey-247m21.3', 'ENSGWIG00000021373', 'dusp4', 'dqx1', 'smyd1a', 'spra', 'spina', 'ndor1', 'ENSGWIG00000021747', 'ptpa', 'ENSGWIG00000021827', 'ENSGWIG00000022478', 'ENSGWIG00000022485', 'THAP1', 'wdr91', 'ENSGWIG00000005282', 'ENSGWIG00000005293', 'LYZ', 'ENSGWIG00000005321', 'dcc', 'ENSGWIG00000006518', 'ENSGWIG00000006545', 'ENSGWIG00000006576', 'derl2', 'smarcb1b', 'vldlr', 'pum3', 'ENSGWIG00000018114', 'txnl1', 'atp8b1', 'TCF4', 'ENSGWIG00000021613', 'rab27b', 'nars1', 'fech', 'onecut2', 'ENSGWIG00000023099', 'st8sia3', 'WDR7', 'wu:fj64h06', 'ccni', 'ENSGWIG00000023672', 'shroom3', 'parn', 'slc38a9', 'ENSGWIG00000023915', 'pomk', 'pam', 'pdzph1', 'zgc:194281', 'reep5', 'chd1', 'syk', 'vav2', 'thnsl2', 'sptlc1', 'ror2', 'ENSGWIG00000024664', 'auh', 'ENSGWIG00000024666', 'TBC1D13', 'ENSGWIG00000024839', 'zer1', 'ENSGWIG00000024850', 'fam102ab', 'fpgs', 'ENSGWIG00000024872', 'ENSGWIG00000024875', 'ENSGWIG00000024876', 'si:dkey-28o19.1', 'ENSGWIG00000024879', 'ENSGWIG00000024881', 'prkab1a', 'phf24', 'pigo', 'stoml2', 'ENSGWIG00000024909', 'ENSGWIG00000025000', 'ENSGWIG00000025037', 'ENSGWIG00000002300', 'lpar1', 'MUSK', 'rassf6', 'ENSGWIG00000002313', 'ENSGWIG00000002314', 'npdc1a', 'pou5f3', 'ENSGWIG00000002365', 'ENSGWIG00000002368', 'clic3', 'ENSGWIG00000002456', 'nsmfa', 'si:ch73-62l21.1', 'bcr', 'atp2a2b', 'pus1', 'noc4l', 'si:ch73-287m6.1', 'ENSGWIG00000015593', 'si:dkey-112m2.1', 'gpn3', 'HOOK3', 'rnf170', 'TRABD2A', 'ENSGWIG00000019006', 'sfswap', 'ENSGWIG00000019015', 'mmp17a', 'ulk1b', 'crkl', 'ENSGWIG00000019037', 'ENSGWIG00000019213', 'pgam2', 'si:dkey-175m17.6', 'ENSGWIG00000019240', 'skor2', 'katnal2', 'sigmar1', 'ENSGWIG00000019482', 'ENSGWIG00000019484', 'cntfr', 'ENSGWIG00000019492', 'ENSGWIG00000019495', 'cdc37l1', 'anxa3b', 'exd3', 'nrarpa', 'sh2d3ca', 'cercam', 'sardh', 'fam163ba', 'slc27a4', 'ENSGWIG00000006256', 'ambp', 'strbp', 'rabgap1', 'GPR21', 'rc3h2', 'jak2a', 'mrrf', 'lhx6', 'morn5', 'ndufa8', 'si:ch211-243g18.2', 'rbp4l', 'atp5mea', 'fybb', 'ccdc80l2', 'nnt', 'fgf10a', 'ENSGWIG00000018566', 'rai14', 'scarb2c', 'ENSGWIG00000018587', 'RNF208', 'slc5a1', 'GNAZ', 'sppl3', 'nt5c2l1', 'slc2a11l', 'kcnn2', 'drg1', 'stard7', 'slc20a1b', 'pcyox1', 'fam136a', 'ggcx', 'snx24', 'mpx', 'ENSGWIG00000020566', 'smtna', 'slc35e4', 'osbp2', 'ENSGWIG00000021150', 'atp6v0a2a', 'ENSGWIG00000021403', 'pptc7b', 'prnpb', 'kcnip3b', 'trim69', 'bmp1b', 'antxr1b', 'aak1b', 'ENSGWIG00000021934', 'gpat4', 'ENSGWIG00000021943', 'ggt1b', 'lrrc75bb', 'rnf34b', 'kdm2bb', 'orai1b', 'tmem120b', 'dgcr2', 'ess2', 'ENSGWIG00000022510', 'rnf185', 'ENSGWIG00000022520', 'ENSGWIG00000022522', 'coro1cb', 'ENSGWIG00000022589', 'ssh1b', 'hspb15', 'm17', 'tescb', 'ENSGWIG00000001837', 'cabp7b', 'uqcr10', 'zmat5', 'prodha', 'sgsm1b', 'git2b', 'ENSGWIG00000004594', 'ENSGWIG00000005620', 'ENSGWIG00000005621', 'ENSGWIG00000005624', 'ENSGWIG00000005629', 'WDR70', 'ENSGWIG00000007271', 'gdnfa', 'opn4xa', 'ENSGWIG00000007908', 'bmpr1bb', 'ENSGWIG00000008238', 'glis3', 'rfx3', 'ENSGWIG00000011480', 'nup155', 'ENSGWIG00000011484', 'ptpn13', 'mapk10', 'arhgap24', 'ENSGWIG00000013015', 'hcn1', 'ccbe1', 'znf532', 'oacyl', 'ENSGWIG00000013100', 'nedd4l', 'prlra', 'rad1', 'dym', 'smad7', 'ctif', 'zbtb7c', 'S1PR3', 'zgc:86839', 'SEMA4D', 'ENSGWIG00000013929', 'add1', 'nop14', 'ENSGWIG00000014299', 'ENSGWIG00000014301', 'ZNF462', 'TMEM38B', 'PRAG1', 'PRRC2B', 'fam78ab', 'ENSGWIG00000014333', 'NACC2', 'TMEM250', 'LHX3', 'gpsm1b', 'ak1', 'ENSGWIG00000014450', 'brinp1', 'ENSGWIG00000014453', 'ENSGWIG00000014454', 'ENSGWIG00000014455', 'ASTN2', 'ENSGWIG00000001554', 'pappaa', 'znf618', 'ENSGWIG00000001633', 'ENSGWIG00000001640', 'rabepk', 'hspa5', 'golga2', 'dnm1a', 'ENSGWIG00000001816', 'surf2', 'gb:bc139872', 'surf4', 'ASS1', 'fubp3', 'prdm12b', 'fbxw5', 'traf2b', 'ENSGWIG00000002633', 'zmynd19', 'ENSGWIG00000002655', 'pax8', 'ENSGWIG00000002691', 'wbp1', 'MTFP1', 'MED13L', 'ENSGWIG00000003630', 'tbx3a', 'tbx5a', 'ddx54', 'rab14', 'abl1', 'RIC1', 'oaz1b', 'UHRF2', 'ENSGWIG00000006028', 'ENSGWIG00000006030', 'klhl8', 'ENSGWIG00000010810', 'ENSGWIG00000010812', 'ENSGWIG00000010824', 'amacr', 'slc45a2', 'csgalnact1a', 'PLPP1', 'si:dkey-27b3.2', 'itga1', 'ntrk2b', 'hnrpkl', 'gna14', 'pcsk5a', 'ENSGWIG00000012290', 'zgc:73226', 'thbs4b', 'zgc:122979', 'zgc:77838', 'ENSGWIG00000019502', 'ENSGWIG00000019620', 'ENSGWIG00000019629', 'notch1a', 'camsap1a', 'ENSGWIG00000001863', 'si:ch73-29c22.2', 'cdk5rap2', 'nup188', 'dnlz', 'rapgef1b', 'ntng2b', 'SNORD36', 'usp20', 'ptges', 'si:ch211-182d3.1', 'ENSGWIG00000012321', 'si:dkey-34e4.1', 'si:ch73-209e20.5', 'ptcd2', 'map1b', 'ttc33', 'prkaa1', 'lamc3', 'fibcd1', 'zgc:63972', 'dab2ipb', 'dipk1b', 'olfm1b', 'ENSGWIG00000012673', 'fnbp1b', 'egfl7', 'ENSGWIG00000015506', 'enc1', 'ENSGWIG00000015539', 'tyw1', 'caln1', 'galnt17', 'ENSGWIG00000006898', 'zmp:0000000529', 'hif1al', 'pak4', 'ccdc9', 'ENSGWIG00000007313', 'sae1', 'zc3h4', 'zgc:154093', 'opa3', 'foxo1a', 'cog6', 'LHFPL6', 'NEK3', 'lpar6a', 'ENSGWIG00000009649', 'rcbtb2', 'fndc3a', 'mlnr', 'zgc:153184', 'prcp', 'FAM181B', 'tenm4', 'gab2', 'ENSGWIG00000012394', 'yap1', 'ENSGWIG00000012402', 'TRPC6', 'pgr', 'arhgap42b', 'cntn5', 'ENSGWIG00000019538', 'FBXO40', 'ttc36', 'nectin1b', 'ENSGWIG00000019545', 'treh', 'ddx6', 'ENSGWIG00000019583', 'ENSGWIG00000019586', 'cdon', 'bckdha', 'b3gnt2l', 'napab', 'bicra', 'EHD2', 'nup88', 'spred3', 'dhx34', 'aplp2', 'prdm10', 'ENSGWIG00000021744', 'exosc5', 'tmem91', 'ENSGWIG00000021752', 'ENSGWIG00000005362', 'ENSGWIG00000005366', 'gbe1b', 'cadm2b', 'rwdd2b', 'usp16', 'ENSGWIG00000006281', 'grik1b', 'cldn17', 'CLDN8', 'zgc:110333', 'tiam1b', 'scaf4b', 'ENSGWIG00000009448', 'ENSGWIG00000009469', 'dchs1a', 'arfip2a', 'fhdc4', 'trim3a', 'frem2b', 'postnb', 'spartb', 'nbeab', 'stard13b', 'ENSGWIG00000011611', 'hdac12', 'brca2', 'ENSGWIG00000011686', 'b3glcta', 'hmgb1b', 'ENSGWIG00000013136', 'ENSGWIG00000013140', 'or116', 'ENSGWIG00000013178', 'ENSGWIG00000013195', 'stim1a', 'ENSGWIG00000014712', 'si:dkey-24p1.6', 'robo2', 'robo1', 'ENSGWIG00000018244', 'apbb1', 'ILK', 'rrp8', 'ENSGWIG00000018301', 'diabloa', 'ENSGWIG00000018441', 'c2cd3', 'camkk1b', 'ywhag2', 'ENSGWIG00000018457', 'ENSGWIG00000018458', 'proca1', 'ENSGWIG00000018492', 'ENSGWIG00000018494', 'ENSGWIG00000018500', 'nek8', 'traf4a', 'tada2a', 'synrg', 'hnf1ba', 'ENSGWIG00000005980', 'ENSGWIG00000005983', 'aldoca', 'si:ch211-259g3.4', 'mrm3a', 'ENSGWIG00000006607', 'si:dkey-243i1.1', 'slc13a5b', 'wdr81', 'rpa1', 'rtn4rl1b', 'dph1', 'ENSGWIG00000007678', 'hic1', 'tsr1', 'taok1a', 'ENSGWIG00000008924', 'si:ch1073-111c8.3', 'socs9', 'ENSGWIG00000008957', 'lrfn1', 'zgc:162339', 'pih1d2', 'tmprss5', 'zbtb16b', 'ncam1b', 'cadm1b', 'ENSGWIG00000014952', 'ENSGWIG00000014992', 'jam3a', 'vps26b', 'esamb', 'ENSGWIG00000015296', 'lim2.3', 'si:dkey-4p15.5', 'sytl2b', 'picalmb', 'zpld1b', 'alcamb', 'thap12b', 'zgc:112083', 'med13b', 'plch1', 'olfcr1', 'ENSGWIG00000022375', 'olfcj1', 'olfch1', 'ENSGWIG00000024473', 'MME', 'GPR149', 'dhx36', 'ARHGEF26', 'ENSGWIG00000025040', 'RAP2B', 'p2ry1', 'MBNL1', 'SUCNR1', 'aadac', 'ENSGWIG00000025054', 'ENSGWIG00000001702', 'mfsd1', 'ENSGWIG00000002620', 'ENSGWIG00000002639', 'ift80', 'smc4', 'zgc:153119', 'itpkcb', 'neu4', 'COL26A1', 'ENSGWIG00000010901', 'cux1b', 'ENSGWIG00000010983', 'SH2B2', 'ENSGWIG00000010991', 'bud23', 'rabgef1l', 'OTOL1', 'nmd3', 'PPM1L', 'ENSGWIG00000012090', 'kpna4', 'ENSGWIG00000013414', 'clasrp', 'znf296', 'egln2', 'tmem160', 'npas1', 'arhgap35a', 'ap2s1', 'rab4b', 'rhoub', 'rtn2a', 'ENSGWIG00000013664', 'ptgir', 'ENSGWIG00000013678', 'gemin7', 'ENSGWIG00000013998', 'bmp16', 'ENSGWIG00000014001', 'ppp1r37', 'mrpl28', 'relb', 'clptm1', 'six9', 'qpctla', 'sipa1l3', 'map3k10', 'ttc9b', 'si:ch211-160d20.3', 'fosb', 'capn12', 'slc1a5', 'strn4', 'prkd2', 'gpr184', 'si:dkey-202l22.3', 'dscaml1', 'ENSGWIG00000015827', 'ENSGWIG00000015837', 'ENSGWIG00000015945', 'dock10', 'shox2', 'ENSGWIG00000005954', 'MLF1', 'ENSGWIG00000005986', 'sik2b', 'si:dkey-103g5.3', 'zgc:162608', 'ENSGWIG00000006118', 'ankk1', 'drd2a', 'scn3b', 'ENSGWIG00000006136', 'ENSGWIG00000006144', 'ENSGWIG00000006146', 'ENSGWIG00000006149', 'NRGN', 'pde9al', 'ENSGWIG00000006196', 'oafa', 'tlcd5a', 'ENSGWIG00000006207', 'cbl', 'mcamb', 'c1qtnf5', 'kmt2a', 'ube4a', 'im:7138535', 'nipsnap2', 'ENSGWIG00000009490', 'ENSGWIG00000009493', 'ENSGWIG00000009498', 'limk1a', 'ENSGWIG00000013866', 'mnta', 'mettl16', 'PAFAH1B1', 'cluha', 'si:dkey-54n8.4', 'tlcd2', 'ENSGWIG00000019697', 'zgc:153372', 'prpf8', 'rilp', 'scarf1', 'sarm1', 'slc46a1', 'SLC13A2', 'unc119a', 'cpda', 'gosr1', 'trarg1a', 'aipl1', 'ENSGWIG00000020227', 'abhd15a', 'pde3a', 'bcap29', 'dus4l', 'vps53', 'zgc:172282', 'ENSGWIG00000020943', 'zgc:162730', 'bloc1s3', 'si:ch211-136a13.1', 'ENSGWIG00000020954', 'yif1b', 'ENSGWIG00000020957', 'ENSGWIG00000020980', 'p2ry2.1', 'relt', 'ARHGEF17', 'P2RY6', 'fam168a', 'KBTBD3', 'alkbh8', 'cwf19l2', 'GUCY1A2', 'aasdhppt', 'gria4a', 'pdgfd', 'ENSGWIG00000012095', 'DYNC2H1', 'fzd4', 'RAB38', 'ctsc', 'grm5b', 'ENSGWIG00000013102', 'tyr', 'ENSGWIG00000013121', 'ENSGWIG00000013326', 'lrrc51', 'folr', 'inppl1a', 'phox2a', 'CLPB', 'stard10', 'ARAP1', 'PDE2A', 'ENSGWIG00000014745', 'ENSGWIG00000014749', 'fam124b', 'mmp28', 'porb', 'fus', 'gdpd1', 'ypel2a', 'hip1', 'baz1b', 'fzd9b', 'grk7b', 'RASA2', 'ENSGWIG00000020474', 'trip12', 'pid1', 'sphkap', 'daw1', 'agfg1a', 'ENSGWIG00000024586', 'zgc:158328', 'mecom', 'scn4ba', 'scn2b', 'ENSGWIG00000024624', 'ENSGWIG00000024631', 'slitrk3a', 'ENSGWIG00000024767', 'ENSGWIG00000024768', 'ENSGWIG00000024803', 'ENSGWIG00000024950', 'ENSGWIG00000024953', 'pdcd10a', 'ENSGWIG00000024957', 'ENSGWIG00000024958', 'veph1', 'ENSGWIG00000024959', 'ENSGWIG00000024969', 'ccnl1a', 'ENSGWIG00000024979', 'TIPARP', 'ssr3', 'KCNAB1', 'vmn2r1', 'gmps', 'rps6kb1b', 'tubd1', 'vmp1', 'ENSGWIG00000002363', 'atm', 'bace2', 'dscamb', 'ENSGWIG00000002793', 'ENSGWIG00000002797', 'ENSGWIG00000002811', 'sh3bgr', 'ENSGWIG00000002904', 'ENSGWIG00000002911', 'mipep', 'tnfrsf19', 'sacs', 'sgcg', 'slc37a4a', 'ENSGWIG00000003905', 'vaspb', 'snrpd2', 'pglyrp5', 'ENSGWIG00000007455', 'ENSGWIG00000007463', 'gramd1bb', 'ENSGWIG00000007701', 'ENSGWIG00000007703', 'barx2', 'KIRREL3', 'ENSGWIG00000009512', 'smtla', 'ets1', 'ENSGWIG00000009689', 'fli1a', 'ENSGWIG00000009757', 'kcnj5', 'st3gal4', 'srpra', 'fam118b', 'ilvbl', 'ENSGWIG00000011703', 'ENSGWIG00000011744', 'b3gat1a', 'siae', 'mmp20a', 'mmp13b', 'tsku', 'gucy2f', 'serpinh1b', 'gdpd5b', 'hspa13', 'p2rx8', 'samsn1a', 'ENSGWIG00000012668', 'ENSGWIG00000012675', 'ENSGWIG00000012676', 'si:ch211-132b12.2', 'akt2', 'dhx40', 'vwa5a', 'rap1gap2a', 'blmh', 'slc6a4a', 'ENSGWIG00000016674', 'ENSGWIG00000016676', 'ENSGWIG00000016677', 'ankrd13b', 'git1', 'ENSGWIG00000021635', 'igsf11', 'ENSGWIG00000001514', 'ENSGWIG00000001522', 'ENSGWIG00000001533', 'ENSGWIG00000001541', 'ENSGWIG00000001547', 'ENSGWIG00000001553', 'ENSGWIG00000001657', 'gap43', 'ENSGWIG00000001811', 'MPZL2', 'ENSGWIG00000001818', 'TMEM25', 'sik3', 'sidt2', 'tagln', 'apoa1b', 'ENSGWIG00000005355', 'ENSGWIG00000005368', 'nudt8', 'mfrp', 'pnkp', 'ENSGWIG00000005461', 'ENSGWIG00000005465', 'ENSGWIG00000005526', 'si:dkey-243k1.3', 'dpf1', 'si:ch211-11n16.2', 'numbl', 'si:dkey-114g7.4', 'slc5a2', 'si:dkey-260j18.2', 'sars2', 'ENSGWIG00000007300', 'nop53', 'ENSGWIG00000007489', 'si:ch1073-228b5.2', 'sirt2', 'ENSGWIG00000008212', 'ENSGWIG00000008214', 'meis3', 'ITM2C', 'PER2', 'pik3cb', 'ENSGWIG00000012731', 'mrps22', 'COPB2', 'rbp2a', 'lpar5b', 'si:ch211-276c2.2', 'ENSGWIG00000013030', 'ENSGWIG00000013031', 'ENSGWIG00000013036', 'ENSGWIG00000013040', 'ENSGWIG00000013043', 'ENSGWIG00000013050', 'ENSGWIG00000013056', 'ksr1b', 'wsb1', 'cuedc1b', 'akap1b', 'ENSGWIG00000013315', 'ENSGWIG00000013317', 'SMCO4', 'nlk2', 'nlrx1', 'ENSGWIG00000013349', 'gkup', 'ENSGWIG00000013366', 'ENSGWIG00000013373', 'ENSGWIG00000013378', 'ENSGWIG00000013384', 'ENSGWIG00000013387', 'dhrs13b.1', 'ENSGWIG00000006091', 'gemin4', 'ENSGWIG00000006114', 'mrps23', 'ngef', 'taar10b', 'DHRS13', 'phf12b', 'sez6b', 'pipox', 'myo18ab', 'ENSGWIG00000008740', 'si:ch211-255f4.7', 'lhx1a', 'ENSGWIG00000008747', 'ENSGWIG00000008750', 'smtnl', 'ENSGWIG00000008753', 'tmem199', 'poldip2', 'si:dkey-151p21.7', 'crk', 'ywhae1', 'myo1cb', 'slc43a2a', 'ENSGWIG00000010277', 'ENSGWIG00000010314', 'ENSGWIG00000011097', 'ENSGWIG00000011099', 'ENSGWIG00000011103', 'wdfy1', 'flj13639', 'serpine2', 'prss16', 'psmd2', 'ENSGWIG00000012823', 'ENSGWIG00000020115', 'alp3', 'sat1a.1', 'ENSGWIG00000010329', 'ENSGWIG00000010330', 'ENSGWIG00000010352', 'phldb2a', 'ENSGWIG00000010372', 'ENSGWIG00000010371', 'sms', 'ENSGWIG00000010504', 'si:ch211-59d17.3', 'ppp3ca', 'cxxc5a', 'zgc:100829', 'nrg2a', 'slc22a6l', 'pygma', 'ENSGWIG00000019018', 'pcnx3', 'DIXDC1', 'dlat', 'LAYN', 'ENSGWIG00000023382', 'snrnp35', 'ENSGWIG00000001982', 'heatr6', 'dynll2b', 'pigs', 'aldocb', 'rskrb', 'slc47a1', 'tmigd1', 'si:dkey-175g6.2', 'ENSGWIG00000002589', 'slc25a1a', 'nle1', 'inpp5kb', 'pitpnab', 'hephl1b', 'cadm2a', 'paxbp1', 'slc7a1a', 'ENSGWIG00000003836', 'stoml3b', 'proser1', 'thsd1', 'smpd1', 'ENSGWIG00000006542', 'mtnr1bb', 'ENSGWIG00000006630', 'grm5a', 'ENSGWIG00000012887', 'map3k7cl', 'ENSGWIG00000012889', 'grik1a', 'tiam1a', 'scaf4a', 'hunk', 'mis18a', 'cfap298', 'synj1', 'si:dkey-5g14.1', 'ENSGWIG00000018279', 'fhdc3', 'trim3b', 'ENSGWIG00000018595', 'trarg1b', 'ENSGWIG00000018616', 'ENSGWIG00000018625', 'aifm5', 'fbxo40.1', 'ENSGWIG00000018729', 'si:ch211-274p24.4', 'pafah1b1b', 'mntb', 'limk1b', 'ENSGWIG00000019558', 'ENSGWIG00000019559', 'mrps17', 'ENSGWIG00000019573', 'abhd11', 'cldnc', 'ENSGWIG00000001521', 'ENSGWIG00000001528', 'ENSGWIG00000002115', 'caln2', 'sdf2', 'supt6h', 'ENSGWIG00000004127', 'ssh2b', 'coro6', 'ENSGWIG00000004758', 'smg6', 'ovca2', 'rtn4rl1a', 'RPA1', 'ENSGWIG00000006362', 'ubl3a', 'hmgb1a', 'b3glctb', 'fryb', 'zar1l', 'ENSGWIG00000012525', 'nbeaa', 'SERTM1', 'ccna1', 'sparta', 'dclk1a', 'rfxap', 'supt20', 'ugt5d1', 'postna', 'trpc4a', 'ufm1', 'frem2a', 'ENSGWIG00000019182', 'stk10', 'sh3pxd2b', 'GABRB2', 'gabra6b', 'gabra1', 'gabrg2', 'ccni2', 'sept8a', 'shroom1', 'ENSGWIG00000020656', 'aff4', 'hspa4a', 'endou2', 'ARHGAP26', 'spry4', 'ENSGWIG00000021307', 'kctd16b', 'GRXCR2', 'ENSGWIG00000021313', 'lars1b', 'rad50', 'drd1b', 'ENSGWIG00000022777', 'nsg2', 'CPEB4', 'adad2', 'ENSGWIG00000006754', 'ENSGWIG00000006755', 'slc9a6b', 'snx12', 'slc7a3a', 'si:ch211-200p22.4', 'rbm14a', 'dpp3', 'ENSGWIG00000007609', 'ENSGWIG00000007612', 'ENSGWIG00000007613', 'STK32A', 'dpysl3', 'jakmip2', 'p4ha2', 'pdlim4', 'ENSGWIG00000015558', 'ENSGWIG00000015567', 'ENSGWIG00000015586', 'ENSGWIG00000015592', 'slc22a4', 'IRF1', 'ENSGWIG00000015762', 'fnip1', 'ENSGWIG00000021237', 'fstl4', 'ENSGWIG00000022027', 'ENSGWIG00000022028', 'vdac1', 'ENSGWIG00000022039', 'ubtd2', 'rnf145a', 'EBF1', 'htatsf1', 'stc2b', 'crebrf', 'pcyox1l', 'TCERG1', 'tmtops3b', 'tpst1l', 'aldh3a2b', 'vtnb', 'ENSGWIG00000022905', 'larp1', 'faxdc2', 'macroh2a1', 'ENSGWIG00000022914', 'pitx1', 'PCBD2', 'txndc15', 'zgc:77058', 'ddx46', 'sec24a', 'jade2', 'ube2b', 'ENSGWIG00000024112', 'tcf7', 'ENSGWIG00000024155', 'ENSGWIG00000024157', 'afap1l1b', 'fgf13b', 'f9b', 'mcf2b', 'ENSGWIG00000024230', 'znrd2', 'fgfr4', 'ENSGWIG00000024482', 'ENSGWIG00000024488', 'rab24', 'pgk1', 'ENSGWIG00000007003', 'kctd7', 'KCNJ6', 'rabgef1', 'ttc3', 'vps26c', 'gria4b', 'elmod1', 'slc35f2', 'ompb', 'myo7ab', 'ENSGWIG00000008424', 'ENSGWIG00000008447', 'mcama', 'ENSGWIG00000008515', 'tmem120a', 'HEPACAM', 'ap2b1', 'zw10', 'pom121', 'tbl2', 'mybbp1a', 'ENSGWIG00000013244', 'psmg1', 'ENSGWIG00000013249', 'trim37', 'ppm1e', 'ENSGWIG00000017765', 'tlcd3a', 'ENSGWIG00000017767', 'usp2b', 'tecta', 'cadm1a', 'ENSGWIG00000018158', 'ncam1a', 'zbtb16a', 'ENSGWIG00000023116', 'usp28', 'htr3b', 'bckdk', 'lrwd1', 'arhgef12b', 'ENSGWIG00000023432', 'ENSGWIG00000023434', 'SPRY3', 'zgc:158291', 'klf5l', 'klf8', 'rbm41', 'cldn2', 'ENSGWIG00000023451', 'clip2', 'gtf2ird1', 'dtx2', 'ENSGWIG00000023622', 'tmem132e', 'rad51d', 'doc2b', 'castor2', 'rcc1l', 'ENSGWIG00000023864', 'si:ch211-176g13.8', 'mtmr2', 'cwc15', 'ENSGWIG00000024417', 'plp1b', 'nlgn3b', 'ENSGWIG00000024420', 'gria3a', 'ENSGWIG00000006956', 'epd', 'ENSGWIG00000007379', 'myo1ca', 'arhgap35b', 'ENSGWIG00000007665', 'ENSGWIG00000008501', 'pafah1b2', 'TAGLN', 'gjd1a', 'ENSGWIG00000008525', 'ucp2', 'ucp3', 'ppme1', 'nectin3b', 'rtn2b', 'kcnk12l', 'ccdc61', 'ENSGWIG00000009221', 'ENSGWIG00000009224', 'ENSGWIG00000009226', 'cnga2b', 'rraga', 'chmp1b', 'ENSGWIG00000009231', 'brwd3', 'radx', 'tbc1d8b', 'ENSGWIG00000009995', 'ENSGWIG00000009996', 'cx27.5', 'ophn1', 'AR', 'MSN', 'zc3h12b', 'amer1', 'aifm1', 'glra4b', 'msi2a', 'ksr1a', 'vezf1a', 'myo18aa', 'sez6a', 'lhx1b', 'acaca', 'tbx2a', 'ppm1db', 'ENSGWIG00000023805', 'ENSGWIG00000024770', 'pcdh1a', 'ENSGWIG00000024892', 'PCDHAC2', 'ENSGWIG00000024896', 'ENSGWIG00000024897', 'kcnip1b', 'npm1a', 'cldn7b', 'fgf11b', 'nlgn2b', 'kdm6bb', 'tmem88a', 'pcolcea', 'vamp2', 'mpp1', 'tmlhe', 'ENSGWIG00000024991', 'gabrb4', 'gabra3', 'TMEM164', 'ammecr1', 'kif4', 'adssl', 'KCNK4', 'esrra', 'si:dkey-175m17.7', 'slc8a4a', 'cnih2', 'tm7sf2', 'vps51', 'frmd8', 'rom1a', 'zfpl1', 'ENSGWIG00000008361', 'ENSGWIG00000008362', 'rps6ka4', 'si:dkey-35m8.1', 'ABR', 'specc1', 'adora2b', 'zswim7', 'ENSGWIG00000016746', 'pigl', 'p2rx5', 'emc6', 'shpk', 'ENSGWIG00000016813', 'fam222ba', 'eral1', 'flot2a', 'si:ch211-283g2.1', 'dhrs11a', 'ggnbp2', 'pigw', 'myo19', 'vkorc1l1', 'gusb', 'cfap58', 'ca4a', 'MTMR4', 'hpda', 'ENSGWIG00000021693', 'supt4h1', 'ENSGWIG00000021699', 'ENSGWIG00000021700', 'ENSGWIG00000021703', 'slc23a1', 'pwwp2a', 'ENSGWIG00000021721', 'c1qbp', 'mlxipl', 'SRRM3', 'hspb1', 'ENSGWIG00000021731', 'ywhag1', 'ENSGWIG00000021737', 'ENSGWIG00000021758', 'tiprl', 'RIOX2', 'smfn', 'HLCS', 'nova1', 'ENSGWIG00000022060', 'capns1a', 'zgc:153990', 'ENSGWIG00000008649', 'dlb', 'ppp1r14aa', 'ENSGWIG00000008661', 'ENSGWIG00000008662', 'mark4a', 'ckma', 'nccrp1', 'ENSGWIG00000009986', 'slc8a2a', 'ENSGWIG00000010543', 'ENSGWIG00000010557', 'irf2bp1', 'ENSGWIG00000010633', 'bhlha9', 'ENSGWIG00000010637', 'ENSGWIG00000010638', 'wscd1b', 'ENSGWIG00000010640', 'usp25', 'samsn1b', 'gdpd5a', 'ENSGWIG00000011063', 'serpinh1a', 'acer3', 'ENSGWIG00000011082', 'ENSGWIG00000011104', 'MMP1', 'mmp30', 'acat1', 'ENSGWIG00000011640', 'ENSGWIG00000011643', 'ENSGWIG00000011650', 'dscama', 'pcp4a', 'igsf5a', 'ENSGWIG00000014246', 'slc37a4b', 'ENSGWIG00000014302', 'b3gat1b', 'ei24', 'stt3a', 'ENSGWIG00000014361', 'wnt11', 'uvrag', 'dgat2', 'mogat2', 'pdzd3b', 'ENSGWIG00000014393', 'ENSGWIG00000014457', 'atp5l', 'ftr82', 'ftr83', 'ruvbl2', 'SPNS2', 'spns3', 'ube2g1a', 'ankfy1', 'zzef1', 'auts2a', 'ENSGWIG00000016190', 'ints2', 'med13a', 'ENSGWIG00000018235', 'map6a', 'ENSGWIG00000019537', 'ZBTB21', 'c2cd2', 'PRDM15', 'ripk4', 'zgc:110006', 'ENSGWIG00000001608', 'cxadr', 'atp2a3', 'ENSGWIG00000004527', 'ENSGWIG00000004535', 'oatx', 'pfdn1', 'ENSGWIG00000004817', 'ENSGWIG00000004878', 'ENSGWIG00000004921', 'ENSGWIG00000004930', 'ENSGWIG00000004945', 'ENSGWIG00000004970', 'ENSGWIG00000005025', 'ENSGWIG00000005144', 'ENSGWIG00000005252', 'ndufa2', 'ENSGWIG00000010483', 'ENSGWIG00000010548', 'ENSGWIG00000010549', 'ENSGWIG00000010550', 'or115', '15', 'ENSGWIG00000010943', 'slco2b1', 'serpinf1', 'slc13a5a', 'ENSGWIG00000011008', 'ENSGWIG00000011009', 'ENSGWIG00000011011', 'ENSGWIG00000011015', 'ENSGWIG00000011226', 'amot', 'sytl2a', 'dlg2', 'ENSGWIG00000020405', 'bbx', 'vps11', 'ENSGWIG00000012152', 'ntm', 'ENSGWIG00000009722', 'ENSGWIG00000009755', 'lhb', 'ENSGWIG00000009774', 'map4k3a', 'epas1b', 'ENSGWIG00000011735', 'slc17a5', 'eef1a1a', 'kcnq5a', 'ENSGWIG00000013044', 'ENSGWIG00000013092', 'meis1b', 'ENSGWIG00000013419', 'SPRED2', 'actr2a', 'ENSGWIG00000013653', 'nckap1', 'ENSGWIG00000015030', 'zc3h15', 'ENSGWIG00000015057', 'ENSGWIG00000015066', 'ENSGWIG00000015084', 'ENSGWIG00000015370', 'mcm8', 'ENSGWIG00000018843', 'cdh11', 'ENSGWIG00000023519', 'ENSGWIG00000023521', 'cdh5', 'ENSGWIG00000023561', 'ENSGWIG00000023565', 'ENSGWIG00000023568', 'ENSGWIG00000023573', 'ENSGWIG00000023701', 'ENSGWIG00000023702', 'ehbp1', 'b3gnt2b', 'ENSGWIG00000023734', 'hhat', 'kcnh1a', 'mrpl57', 'desi2', 'fbxo5', 'anapc1', 'ENSGWIG00000009272', 'ENSGWIG00000009273', 'pank1a', 'mrps5', 'ENSGWIG00000009343', 'slc8a1b', 'ENSGWIG00000009371', 'ENSGWIG00000009375', 'atrnl1a', 'ENSGWIG00000009507', 'ccdc172', 'ENSGWIG00000009511', 'ENSGWIG00000009514', 'cbr1', 'ENSGWIG00000009520', 'cryzl1', 'itsn1', 'ENSGWIG00000010426', 'pdxkb', 'ENSGWIG00000010447', 'synpo2la', 'ENSGWIG00000010583', 'PNPT1', 'CFAP36', 'ENSGWIG00000018746', 'ENSGWIG00000018749', 'si:ch211-117n7.8', 'ENSGWIG00000020388', 'ENSGWIG00000020389', 'ENSGWIG00000020390', 'si:ch73-54n14.2', 'slc3a1', 'ppm1ba', 'znhit2', 'LRPPRC', 'abcg8', 'dync2li1', 'hint1', 'rtkn2a', 'ENSGWIG00000021367', 'ENSGWIG00000021368', 'tmem26a', 'perp', 'tnfaip3', 'sdccag8', 'akt3a', 'zbtb18', 'ENSGWIG00000023100', 'ENSGWIG00000023102', 'slc29a1a', 'atl2', 'ENSGWIG00000023312', 'olig3', 'ENSGWIG00000023315', 'ENSGWIG00000023318', 'map3k21', 'pcnx2', 'rgs17', 'oprm1', 'sf3b5', 'phactr2', 'ENSGWIG00000023368', 'htra1a', 'wapla', 'grid1a', 'ENSGWIG00000023942', 'ENSGWIG00000023943', 'ccser2a', 'ndnfl', 'dnajc9', 'si:ch211-248a14.8', 'ENSGWIG00000001835', 'p4ha1b', 'actn2b', 'ENSGWIG00000005508', 'ENSGWIG00000005524', 'ENSGWIG00000005527', 'chrm3a', 'ENSGWIG00000005536', 'ENSGWIG00000005539', 'grem2a', 'rgs7b', 'bicc1a', 'ENSGWIG00000008375', 'ENSGWIG00000008377', 'slc16a9a', 'ccdc6a', 'ank3a', 'ENSGWIG00000016520', 'ubac2', 'ENSGWIG00000016534', 'bora', 'pibf1', 'pgbd5', 'cog2', 'agt', 'ikzf5', 'hmx3a', 'hmx2', 'gpr26', 'chst15', 'nkx1.2la', 'FAM53B', 'eef1akmt2', 'ENSGWIG00000017006', 'IDO', 'INSYN2A', 'ptpreb', 'gnrh3', 'mgmt', 'ENSGWIG00000022452', 'add3b', 'ENSGWIG00000023782', 'sorcs3b', 'ENSGWIG00000024475', 'SH3PXD2A', 'slka', 'sfr1', 'ENSGWIG00000025371', 'ENSGWIG00000025374', 'ENSGWIG00000025376', 'papss2a', 'ENSGWIG00000025577', 'minpp1a', 'zgc:91976', 'prkg1a', 'dkk1a', 'pcdh15a', 'plek', 'prokr1b', 'pcsk2', 'GTPBP2', 'vdac2', 'comtd1', 'znf503', 'lrmda', 'kcnma1a', 'dlg5a', 'anxa11a', 'zcchc24', 'zmiz1a', 'rps24', 'spock2', 'trmt2b', 'wnt8b', 'hif1an', 'cuedc2', 'pax2a', 'chst3a', 'lrit2', 'rgra', 'ENSGWIG00000022243', 'slc18a3a', 'chata', 'ogdhl', 'parga', 'ENSGWIG00000023634', 'PDE6C', 'lgi1a', 'MARK1', 'EFEMP1', 'si:ch211-233m11.2', 'tp53bp2a', 'sema4ga', 'lzts2a', 'zgc:154058', 'si:ch211-198a12.6', 'stox1', 'ENSGWIG00000025382', 'tet1', 'ALOX5', 'march8', 'zfand4', 'ENSGWIG00000025398', 'ENSGWIG00000025401', 'zgc:171482', 'sfrp5', 'golga7bb', 'crtac1b', 'loxl4', 'zgc:123010', 'ENSGWIG00000009654', 'pyroxd2', 'ENSGWIG00000009694', 'got1', 'hpse2', 'hps1', 'st3gal7', 'ankrd2', 'ENSGWIG00000009753', 'slit1a', 'ENSGWIG00000010183', 'ENSGWIG00000010336', 'cox15', 'HPS6', 'ENSGWIG00000015862', 'ldb1a', 'pprc1', 'ENSGWIG00000016040', 'oga', 'ENSGWIG00000017948', 'sufu', 'si:dkey-15f17.8', 'zdhhc16a', 'ENSGWIG00000018145', 'rrp12', 'antxr1d', 'prdm8', 'GDF10', 'ENSGWIG00000018163', 'FRMPD2', 'ENSGWIG00000018169', 'mapk8a', 'arhgap22', 'si:ch211-223a10.1', 'WDFY4', 'lrrc18a', 'vstm4a', 'drgx', 'ercc6', 'cxcl12a', 'ENSGWIG00000019822', 'ENSGWIG00000019839', 'sprn', 'ENSGWIG00000020124', 'msx3', 'znf511', 'adam8a', 'zgc:66426', 'ENSGWIG00000020168', 'plaua', 'vcla', 'ap3m1', 'si:ch211-223p8.8', 'zgc:162183', 'npy4r', 'mcph1', 'agpat5', 'zc3h6', 'ENSGWIG00000021049', 'itga9', 'golga4', 'KCNK12', 'msh6', 'ENSGWIG00000021597', 'FOXN2', 'ppp1r21', 'gtf2a1l', 'lhcgr', 'ENSGWIG00000022394', 'ENSGWIG00000010070', 'psme4a', 'ENSGWIG00000010972', 'ENSGWIG00000010977', 'thbs2a', 'map3k4', 'kcnk1b', 'slc35f3b', 'zgc:172136', 'fam135a', 'ENSGWIG00000020503', 'ENSGWIG00000020526', 'ENSGWIG00000020527', 'lmbrd1', 'adgrb3', 'ENSGWIG00000021705', 'ENSGWIG00000021708', 'phf3', 'ENSGWIG00000021710', 'si:dkey-103j14.5', 'khdrbs2', 'prim2', 'bag2', 'znf451', 'ENSGWIG00000021756', 'zmp:0000000760', 'col21a1', 'bmp5', 'hmgcll1', 'ENSGWIG00000022128', 'hcrtr2', 'fam83b', 'ENSGWIG00000022139', 'ENSGWIG00000022140', 'lrrc1', 'eloal', 'klhl31', 'gclc', 'elovl5', 'fbxo9', 'slc29a3', 'psap', 'ENSGWIG00000022389', 'ENSGWIG00000022390', 'micu1', 'mcu', 'oit3', 'pde10a', 'tbxtb', 'prr18', 'sft2d1', 'mertka', 'fbln7', 'unc5b', 'ly6pge', 'ENSGWIG00000024530', 'nsmce4a', 'ENSGWIG00000024533', 'ENSGWIG00000024532', 'tmem63ba', 'capn1a', 'ENSGWIG00000025827', 'rrbp1a', 'mgme1', 'ENSGWIG00000025900', 'ENSGWIG00000025901', 'flrt3', 'tasp1', 'ism1', 'sptlc3', 'btbd3b', 'ENSGWIG00000025933', 'ENSGWIG00000025934', 'jag1b', 'ENSGWIG00000025942', 'mkks', 'ENSGWIG00000010075', 'si:ch1073-291c23.1', 'STRN', 'heatr5b', 'ENSGWIG00000014444', 'ENSGWIG00000014445', 'irf2bp2a', 'ENSGWIG00000014447', 'tomm20a', 'ggps1', 'b3galnt2', 'GNG4', 'lyst', 'ENSGWIG00000016378', 'nid1a', 'GPR137B', 'egln1a', 'ENSGWIG00000016562', 'sipa1l2', 'si:ch1073-266p11.2', 'ntpcr', 'ENSGWIG00000016632', 'ENSGWIG00000016633', 'klhdc3', 'ENSGWIG00000016637', 'ENSGWIG00000016638', 'ENSGWIG00000016639', 'ENSGWIG00000016640', 'ENSGWIG00000016641', 'ENSGWIG00000016644', 'ENSGWIG00000016664', 'ENSGWIG00000016683', 'PTK7', 'srfb', 'ablim1a', 'ENSGWIG00000020685', 'ENSGWIG00000020680', 'ENSGWIG00000020689', 'ENSGWIG00000020811', 'herc4', 'ENSGWIG00000020967', 'ENSGWIG00000020968', 'lrrfip2', 'dact2', 'smoc2', 'eif3s6ip', 'vsir', 'ENSGWIG00000021322', 'plpp4', 'wdr11', 'fgfr2', 'ATE1', 'ALDH18A1', 'atoh7', 'pkd2l1', 'bag3', 'inpp5f', 'mcmbp', 'sec23ip', 'csgalnact2', 'ret', 'si:dkey-192p21.6', 'pcbd1', 'sgpl1', 'pdlim1', 'sorbs1', 'ENSGWIG00000026298', 'pik3ap1', 'fam204a', 'rab11fip2', 'emx2', 'eif3s10', 'tm9sf3', 'si:ch211-117n7.7', 'MOCOS', 'ENSGWIG00000026337', 'vwc2', 'fignl1', 'ikzf1', 'ttl', 'nanp', 'cyp1b1', 'ENSGWIG00000002553', 'cdc42ep3', 'ENSGWIG00000002555', 'ENSGWIG00000002557', 'opn3', 'kmo', 'ide', 'march5', 'cpeb3', 'btaf1', 'ccnj', 'ENSGWIG00000009452', 'dntt', 'ENSGWIG00000009545', 'vox', 'ENSGWIG00000009549', 'ENSGWIG00000009550', 'ADGRA1', 'ENSGWIG00000009600', 'ENSGWIG00000009604', 'ENSGWIG00000009607', 'ENSGWIG00000009614', 'valopa', 'nkx6.2', 'INPP5A', 'ENSGWIG00000010196', 'pwwp2b', 'ENSGWIG00000010199', 'STK32C', 'ENSGWIG00000010956', 'galm', 'ltbp1', 'ENSGWIG00000010975', 'ttc27', 'ENSGWIG00000013430', 'rin2', 'slc24a3', 'ENSGWIG00000013715', 'ccdc85a', 'ENSGWIG00000013722', 'vrk2', 'fancl', 'ENSGWIG00000013752', 'bcl11aa', 'rims1a', 'ogfrl1', 'b3gat2', 'smap1', 'slc1a4', 'xpo1b', 'pcgf6', 'ENSGWIG00000014012', 'SERTAD2', 'si:ch211-189k9.2', 'pex13', 'REL', 'haao', 'ENSGWIG00000007414', 'ENSGWIG00000007419', 'ENSGWIG00000007423', 'zfp36l2', 'ENSGWIG00000007462', 'plekhh2', 'ENSGWIG00000007539', 'ENSGWIG00000007544', 'ENSGWIG00000007547', 'calm2a', 'ttc7a', 'ENSGWIG00000007894', 'rhoq', 'socs5b', 'ENSGWIG00000007902', 'mcfd2', 'cdk1', 'rnf166', 'borcs7', 'ctu2', 'ncapd3', 'calhm2.2', 'ENSGWIG00000008434', 'sdhaf4', 'hells', 'ENSGWIG00000008266', 'ENSGWIG00000008276', 'nr2f5', 'bola1', 'sv2a', 'mtmr11', 'ENSGWIG00000008847', 'rgl2', 'phf1', 'syngap1b', 'ENSGWIG00000009078', 'paqr6', 'ENSGWIG00000009095', 'dennd4b', 'ENSGWIG00000001615', 'aldh5a1', 'ENSGWIG00000001792', 'cpne4a', 'rpp25l', 'ENSGWIG00000002051', 'adam15', 'ENSGWIG00000002058', 'ENSGWIG00000002060', 'ENSGWIG00000006063', 'ENSGWIG00000002245', 'ENSGWIG00000006066', 'nudt17', 'ENSGWIG00000006079', 'syt11b', 'ENSGWIG00000006106', 'map7d1a', 'thrap3a', 'eva1ba', 'ENSGWIG00000006140', 'stk40', 'OSCP1', 'mrps15', 'csf3r', 'GRIK3', 'ENSGWIG00000008587', 'zc3h12a', 'ENSGWIG00000008590', 'inpp5b', 'mtf1', 'MANEAL', 'themis2', 'rpa2', 'mecr', 'foxo6a', 'ENSGWIG00000012420', 'SCMH1', 'cited4b', 'ctps1b', 'si:dkey-34d22.5', 'si:ch211-79k12.1', 'cadm4', 'bcam', 'ENSGWIG00000012573', 'apoa2', 'ENSGWIG00000012588', 'apoeb', 'ENSGWIG00000012600', 'ENSGWIG00000012611', 'si:ch211-1i11.3', 'gnl2', 'rspo1', 'fhl3a', 'pou3f1', 'ENSGWIG00000014199', 'rragca', 'rhbdl2', 'akirin1', 'cnr2', 'ENSGWIG00000010121', 'ENSGWIG00000010122', 'ENSGWIG00000010123', 'si:ch73-373m9.1', 'ghdc', 'ENSGWIG00000012174', 'si:ch211-107n13.1', 'ENSGWIG00000012183', 'ENSGWIG00000012185', 'dhdds', 'ago1', 'ago3a', 'ENSGWIG00000018852', 'ENSGWIG00000018854', 'zgc:110372', 'ENSGWIG00000018858', 'ENSGWIG00000018861', 'ENSGWIG00000018885', 'ENSGWIG00000018887', 'ENSGWIG00000018889', 'xrcc1', 'drd2l', 'ethe1', 'ENSGWIG00000018925', 'ENSGWIG00000018927', 'tp53inp1', 'C8orf37', 'ENSGWIG00000019073', 'ENSGWIG00000019124', 'isg20l2', 'bcan', 'hapln2', 'rhbg', 'ENSGWIG00000019398', 'mef2d', 'sh3bp5b', 'ENSGWIG00000019444', 'mettl6', 'bmper', 'ENSGWIG00000021161', 'bbs9', 'nt5c3a', 'atg5', 'prdm1a', 's100a1', 'tlr18', 'si:ch211-113g11.6', 'lingo4b', 'ctsk', 'onecutl', 'mcl1b', 'tmod4', 'scnm1', 'lysmd1', 'ENSGWIG00000021267', 'tnfaip8l2b', 'ENSGWIG00000021270', 'ENSGWIG00000010492', 'ENSGWIG00000010494', 'sema6e', 'ENSGWIG00000010702', 'cspg5a', 'ENSGWIG00000010814', 'dpy19l1l', 'tbx20', 'herpud2', 'crppa', 'ENSGWIG00000010850', 'cxcr3.2', 'grik5', 'ENSGWIG00000011570', 'znf574', 'ENSGWIG00000011578', 'znf526', 'cicb', 'ENSGWIG00000011864', 'pou2f2a', 'ENSGWIG00000011869', 'dedd1', 'ENSGWIG00000012103', 'si:ch73-22o12.1', 'si:ch73-54f23.4', 'ENSGWIG00000021975', 'flot1b', 'ddr1', 'gnl1', 'si:ch211-154o6.2', 'si:ch211-154o6.3', 'COPS7B', 'ENSGWIG00000022331', 'ENSGWIG00000022332', 'p3h3', 'gnb3a', 'si:ch211-69g19.2', 'pex5', 'clstn3', 'lpcat3', 'tpi1b', 'atn1', 'ENSGWIG00000023614', 'cdc42l', 'ptpn6', 'tapbpl', 'mrpl51', 'plekhg6', 'ENSGWIG00000025728', 'iffo1b', 'gapdh', 'ENSGWIG00000001611', 'ing4', 'emg1', 'zyx', 'fam131bb', 'clcn1b', 'casp2', 'zgc:158689', 'ndufa3', 'si:dkey-40m6.8', 'tmc4', 'mboat7', 'rbfox1l', 'leng8', 'cdc42ep5', 'grwd1', 'ENSGWIG00000002308', 'flcn', 'josd2', 'ube2s', 'ENSGWIG00000002385', 'ENSGWIG00000002387', 'ntd5', 'ENSGWIG00000002391', 'si:dkeyp-69b9.3', 'ENSGWIG00000002394', 'pitpnc1b', 'grin2db', 'prkcg', 'cacng8b', 'u2af2a', 'si:dkey-17m8.1', 'ENSGWIG00000004066', 'si:ch211-149k23.9', 'atg12', 'sbk3', 'mfap5', 'il11a', 'rasip1', 'slc2a3b', 'ephb6', 'trpv6', 'm6pr', 'styk1b', 'gstk4', 'si:dkey-283b15.2', 'sphk2', 'rpl18', 'ENSGWIG00000010093', 'kcnj14', 'uts2r3', 'khdc4', 'ENSGWIG00000013309', 'zbtb7b', 'ENSGWIG00000013313', 'pklr', 'clk2b', 'ENSGWIG00000015217', 'RNaseP_nuc', 'setdb1b', 'arnt', 'cers2b', 'celf3b', 'ENSGWIG00000010628', 'ENSGWIG00000010676', 'rprd2b', 'prpf3', 'ENSGWIG00000012310', 'ca14', 'ENSGWIG00000012312', 'crabp2a', 'itga10', 'polr3c', 'gba', 'rnf115', 'dap3', 'ENSGWIG00000013684', 'fdps', 'si:dkey-7f3.9', 'ENSGWIG00000013966', 'kcnn3', 'adar', 'SNX27', 'ENSGWIG00000014655', 'ENSGWIG00000014657', 'ENSGWIG00000014659', 'slc50a1', 'efna1b', 'efna3b', 'si:dkey-246i14.3', 'shc1', 'ENSGWIG00000014753', 'ENSGWIG00000014755', 'she', 'cgna', 'tuft1a', 'aqp10a', 'atp8b2', 'ENSGWIG00000014880', 'pip5k1ab', 'ecm1b', 'rorc', 'nbn', 'gtpbp10', 'adam22', 'ENSGWIG00000019788', 'rundc3b', 'crot', 'nmt2', 'fam171a1', 'ENSGWIG00000025568', 'msh5', 'ENSGWIG00000025589', 'ENSGWIG00000025590', 'ENSGWIG00000025593', 'l3mbtl1b', 'epb41l4b', 'esrp1', 'virma', 'rad54b', 'rnf41l', 'fsbp', 'gem', 'cdh17', 'pdp1', 'tmem67', 'si:ch211-197h24.6', 'tex10', 'invs', 'erp44', 'stx17', 'nr4a3', 'sec61b', 'alg2', 'frrs1l', 'tmem245', 'galnt1', 'ino80c', 'ENSGWIG00000011341', 'dgat1b', 'gpr20', 'ENSGWIG00000011443', 'tbrg4', 'ENSGWIG00000011458', 'steap4', 'ENSGWIG00000011475', 'si:dkey-12j5.1', 'mindy3', 'itga8', 'atxn1b', 'ENSGWIG00000011585', 'ENSGWIG00000011587', 'rbm12b', 'ENSGWIG00000011594', 'nradd', 'nbeal2', 'si:ch211-257p13.3', 'cep72', 'zdhhc11', 'ENSGWIG00000011881', 'trip13', 'nkd2a', 'PLEKHG4B', 'NHLRC1', 'kif13a', 'ENSGWIG00000012360', 'smpd5', 'NPY', 'gsdmeb', 'osbpl3b', 'ENSGWIG00000016729', 'pde11al', 'cbx3b', 'ENSGWIG00000016751', 'skap2', 'hoxa9b', 'hoxa13b', 'hibadhb', 'tax1bp1b', 'creb5b', 'chn2', 'plekha8', 'fkbp14', 'hdac9b', 'twist1b', 'macc1', 'sp8b', 'ENSGWIG00000017164', 'RAPGEF5', 'slc6a19b', 'fhod3b', 'rxrbb', 'ENSGWIG00000002591', 'ENSGWIG00000002596', 'cyp21a2', 'ENSGWIG00000002686', 'kcnk5b', 'ENSGWIG00000003606', 'nphs1', 'kirrel3l', 'zmp:0000001114', 'aplp1', 'ENSGWIG00000004660', 'kmt2bb', 'hspb6', 'lin37', 'hsc70', 'si:dkey-238d18.4', 'ENSGWIG00000007107', 'smg9', 'kcnn4', 'si:dkey-79d12.5', 'ENSGWIG00000007352', 'fpr1', 'ENSGWIG00000007356', 'ENSGWIG00000007357', 'ENSGWIG00000007358', 'ENSGWIG00000007359', 'ENSGWIG00000007362', 'ENSGWIG00000007365', 'prss1', 'ENSGWIG00000007406', 'ENSGWIG00000007416', 'ENSGWIG00000007424', 'ENSGWIG00000007427', 'ENSGWIG00000007430', 'si:ch211-79k12.2', 'si:dkey-34d22.1', 'ENSGWIG00000007452', 'ENSGWIG00000007461', 'ptprub', 'paqr7b', 'serinc2l', 'hivep3b', 'phactr4b', 'rcc1', 'oprd1b', 'ENSGWIG00000012569', 'tcea3', 'tent5ba', 'zdhhc18a', 'arid1aa', 'gpn2', 'ENSGWIG00000013545', 'gpatch3', 'tpbgl', 'wdtc1', 'cep85', 'si:dkey-288i20.2', 'ENSGWIG00000013762', 'lars2', 'LIMD1', 'ncam3', 'mroh1', 'ccka', 'trak1a', 'ENSGWIG00000002141', 'ctnnb1', 'cmtm8a', 'ENSGWIG00000004412', 'trmt11', 'ENSGWIG00000005268', 'ENSGWIG00000005269', 'rspo3', 'RNF146', 'si:ch211-151o1.4', 'soga3a', 'klhl38b', 'zgc:110182', 'ENSGWIG00000005386', 'washc5', 'ENSGWIG00000005448', 'rad21a', 'eif3hb', 'ENSGWIG00000005586', 'zgc:110410', 'ankib1b', 'eomesb', 'ENSGWIG00000005911', 'rbms3', 'ENSGWIG00000006082', 'grb10b', 'cobl', 'ENSGWIG00000010270', 'dnah5', 'ENSGWIG00000010559', 'ENSGWIG00000010571', 'adamts16', 'ENSGWIG00000010580', 'irx1a', 'irx2a', 'irx4a', 'LPCAT1', 'slc6a3', 'DCDC2', 'nrsn1', 'ENSGWIG00000012087', 'sox4b', 'CDKAL1', 'ENSGWIG00000013779', 'MTSS1', 'mal2', 'si:ch211-93i7.4', 'TNFRSF11B', 'ENSGWIG00000013821', 'CARMIL1', 'sntb1', 'ENSGWIG00000018135', 'ENSGWIG00000018138', 'ptp4a3b', 'ptk2ab', 'ENSGWIG00000019422', 'mrs2', 'ENSGWIG00000019425', 'ENSGWIG00000019426', 'dscc1', 'mrpl13', 'col14a1a', 'ENSGWIG00000019636', 'deptor', 'ENSGWIG00000019638', 'MATN2', 'ENSGWIG00000019643', 'ubr5', 'ENSGWIG00000001734', 'azin1a', 'spire1b', 'ENSGWIG00000001796', 'sqlea', 'ENSGWIG00000002321', 'si:dkey-122a22.2', 'zfpm2a', 'dpys', 'ENSGWIG00000002348', 'RIMS2', 'dcaf13', 'slc25a32a', 'fzd6', 'cthrc1a', 'klhl32', 'ENSGWIG00000007535', 'ENSGWIG00000007536', 'pou3f2b', 'fbxl4', 'ENSGWIG00000007618', 'faxcb', 'coq3', 'usp45', 'zgc:172323', 'si:dkey-165n16.1', 'prdm13', 'ENSGWIG00000007674', 'me1', 'ENSGWIG00000008712', 'dop1a', 'ube3d', 'tpbgb', 'ibtk', 'TENT5A', 'ENSGWIG00000008808', 'bckdhb', 'ebag9', 'sybu', 'gdf6a', 'eny2', 'ENSGWIG00000008861', 'trhra', 'ENSGWIG00000008866', 'TMEM74', 'emc2', 'eif3ea', 'rspo2', 'ENSGWIG00000010712', 'plecb', 'ENSGWIG00000011958', 'zgc:91890', 'ENSGWIG00000011963', 'sall3a', 'ENSGWIG00000011965', 'ENSGWIG00000011968', 'ENSGWIG00000012007', 'ENSGWIG00000012009', 'ENSGWIG00000012010', 'mbpb', 'ENSGWIG00000012013', 'ENSGWIG00000012014', 'znf236', 'ENSGWIG00000012020', 'ZNF516', 'TSHZ1', 'ZADH2', 'znf407', 'ENSGWIG00000012025', 'has2', 'ENSGWIG00000012030', 'tbc1d31', 'derl1', 'ENSGWIG00000013609', 'ZHX1', 'ENSGWIG00000013632', 'ENSGWIG00000013631', 'psma2', 'ENSGWIG00000013654', 'ENSGWIG00000015318', 'ENSGWIG00000015325', 'ENSGWIG00000015330', 'scxb', 'gapdhs', 'tmem147', 'pop1', 'NIPAL2', 'KCNS2', 'ENSGWIG00000015445', 'osr2', 'VPS13B', 'ENSGWIG00000015489', 'ENSGWIG00000015492', 'rnf19a', 'pabpc1a', 'ENSGWIG00000015920', 'ENSGWIG00000015921', 'ENSGWIG00000015926', 'ncalda', 'id4', 'cep76', 'ptpn2a', 'zufsp', 'kpna5', 'gprc6a', 'ENSGWIG00000015960', 'ENSGWIG00000015967', 'ENSGWIG00000015971', 'seh1l', 'ldlrad4b', 'fam210ab', 'mc5rb', 'mc2r', 'ENSGWIG00000001791', 'ENSGWIG00000001793', 'ENSGWIG00000001794', 'ENSGWIG00000001797', 'aldh2.2', 'rbm42', 'ppox', 'ENSGWIG00000002122', 'fbl', 'ENSGWIG00000003808', 'GADL1', 'ENSGWIG00000003812', 'lyplal1', 'ENSGWIG00000003825', 'tgfb2', 'rrp15', 'ENSGWIG00000003843', 'ENSGWIG00000003845', 'vegfaa', 'hace1', 'bves', 'popdc3', 'ENSGWIG00000004860', 'stx12l', 'ENSGWIG00000004897', 'ENSGWIG00000005016', 'ENSGWIG00000005217', 'rwdd1', 'ENSGWIG00000006219', 'ENSGWIG00000007800', 'cnot10', 'tcaim', 'ENSGWIG00000007810', 'ENSGWIG00000007812', 'SLC6A20', 'sacm1lb', 'ENSGWIG00000008812', 'ENSGWIG00000008814', 'adgrb1a', 'ENSGWIG00000010387', 'ENSGWIG00000010388', 'si:ch211-208k15.1', 'ENSGWIG00000010391', 'ENSGWIG00000010392', 'ENSGWIG00000010399', 'ENSGWIG00000010401', 'ENSGWIG00000010404', 'ENSGWIG00000010407', 'gask1a', 'pomgnt2', 'snrka', 'ano10a', 'ABHD5', 'ENSGWIG00000011077', 'ENSGWIG00000011079', 'ENSGWIG00000011081', 'ENSGWIG00000011086', 'zgc:114181', 'taf2', 'enpp2', 'nod1', 'ZNRF2', 'ENSGWIG00000013439', 'ENSGWIG00000013441', 'ppp1r9alb', 'zgc:171558', 'ENSGWIG00000016070', 'ENSGWIG00000016174', 'ENSGWIG00000016119', 'ENSGWIG00000016179', 'epha4b', 'ENSGWIG00000019096', 'pip5k1cb', 'ENSGWIG00000020456', 'sema6ba', 'ENSGWIG00000025874', 'rorcb', 'sh3gl1a', 'stap2a', 'sema4e', 'cactin', 'ENSGWIG00000025881', 'sirt6', 'ENSGWIG00000025884', 'btbd2b', 'dusp12', 'ENSGWIG00000001859', 'dpydb', 'ENSGWIG00000002657', 'ENSGWIG00000002662', 'rab2a', 'ENSGWIG00000002678', 'dcun1d1', 'ENSGWIG00000002709', 'ripk2', 'ENSGWIG00000002726', 'zgc:153115', 'tsen15', 'slc35a3b', 'cmpk', 'bcl6ab', 'cldn1', 'ENSGWIG00000002852', 'gk5', 'atp1b3a', 'hs2st1a', 'pkn2', 'gtf2b', 'lrrc8da', 'ENSGWIG00000009644', 'glmna', 'rpap2', 'wls', 'gadd45aa', 'dmbx1a', 'ENSGWIG00000012061', 'ptger4c', 'ENSGWIG00000012079', 'st3gal3b', 'kdm4ab', 'abhd8a', 'mrpl34', 'ENSGWIG00000012378', 'si:dkey-208k22.6', 'zgc:154006', 'ENSGWIG00000012423', 'dpp9', 'rps15', 'ENSGWIG00000016658', 'crtc1a', 'ENSGWIG00000016663', 'mast3a', 'pik3r2', 'pde4ca', 'rab3aa', 'ENSGWIG00000011931', 'ENSGWIG00000013572', 'b3gnt5b', 'ENSGWIG00000013581', 'tbl1xr1b', 'kcnmb2', 'pik3ca', 'si:ch211-38m6.6', 'ephb3a', 'ENSGWIG00000014480', 'ENSGWIG00000014481', 'ENSGWIG00000014483', 'ENSGWIG00000014484', 'ENSGWIG00000014496', 'fam102ba', 'ENSGWIG00000014641', 'agla', 'slc44a5a', 'sdr16c5a', 'penka', 'aqp1a.1', 'usp14', 'ENSGWIG00000023159', 'greb1l', 'mib1', 'ENSGWIG00000023250', 'cables1', 'ENSGWIG00000015223', 'tesk2', 'retreg1', 'ENSGWIG00000016643', 'ranbp9', 'nol7', 'znf622', 'myo10', 'ENSGWIG00000018656', 'adcy8', 'efr3a', 'ENSGWIG00000020236', 'kcnq3', 'ENSGWIG00000020242', 'lrrc6', 'si:ch211-235o23.1', 'cavin4a', 'tmeff1a', 'slc39a6', 'ENSGWIG00000020289', 'saga', 'ENSGWIG00000020328', 'ENSGWIG00000020330', 'proca', 'dusp28', 'vcpip1', 'ENSGWIG00000020349', 'fam49ba', 'PHB2', 'ENSGWIG00000020534', 'dlg1', 'rasal2', 'ENSGWIG00000025546', 'trnau1apb', 'ENSGWIG00000025549', 'myh7', 'ENSGWIG00000025565', 'ENSGWIG00000025647', 'ENSGWIG00000025650', 'map4l', 'ENSGWIG00000025654', 'klhl20', 'ENSGWIG00000025667', 'ENSGWIG00000025668', 'ENSGWIG00000025669', 'brinp2', 'astn1', 'pappa2', 'cop1', 'xrn1', 'ENSGWIG00000025859', 'slc12a7a', 'crispld1b', 'ctnnd2b', 'ENSGWIG00000013228', 'urod', 'zswim5', 'fabp6', 'elovl8a', 'tmem53', 'Metazoa_SRP', 'ENSGWIG00000018630', 'rnf220a', 'klf17', 'slc6a9', 'lrrc24', 'pigr', 'tgm1l1', 'ENSGWIG00000019917', 'rem2', 'ENSGWIG00000019921', 'carmil3', 'ENSGWIG00000019937', 'cdh24b', 'psmb5', 'ap1g2', 'ENSGWIG00000021551', 'ENSGWIG00000021574', 'si:ch211-14a17.11', 'ENSGWIG00000021579', 'mmp14b', 'sall2', 'chd8', 'tox4a', 'ENSGWIG00000022155', 'casq1a', 'ctdsplb', 'oxsr1a', 'slc22a13a', 'ENSGWIG00000024377', 'csrnp1a', 'gorasp1a', 'wdr48b', 'scn12aa', 'ENSGWIG00000024407', 'col6a1', 'col6a2', 'zgc:162816', 'orc2', 'cfap410', 'ENSGWIG00000025223', 'ENSGWIG00000025229', 'catip', 'ENSGWIG00000025232', 'ENSGWIG00000025233', 'ENSGWIG00000025234', 'PCDH8', 'CNMD', 'sugt1', 'ENSGWIG00000025238', 'ednrba', 'ENSGWIG00000025240', 'gad1b', 'gorasp2', 'ENSGWIG00000025742', 'gpr158b', 'anxa13l', 'APBB1IP', 'abi1b', 'ENSGWIG00000013676', 'rab18b', 'mpp7b', 'ENSGWIG00000016962', 'ENSGWIG00000016966', 'tfr1a', 'zgc:56409', 'pgap6', 'ENSGWIG00000020336', 'ENSGWIG00000020340', 'C8A', 'c8b', 'dab1b', 'ENSGWIG00000020472', 'ENSGWIG00000020491', 'MYOC', 'ENSGWIG00000020625', 'fggy', 'si:ch73-182a11.2', 'fzr1b', 'pde4bb', 'ENSGWIG00000024829', 'ENSGWIG00000023907', 'slc1a7a', 'scp2a', 'ENSGWIG00000024838', 'aldh9a1b', 'IPP', 'pik3r3a', 'ENSGWIG00000024840', 'ENSGWIG00000025298', 'ENSGWIG00000025300', 'ENSGWIG00000025301', 'ENSGWIG00000025304', 'rgs5b', 'brinp3b', 'rgs18', 'rgs1', 'rgs13', 'ro60', 'B3GALT2', 'cdc73', 'zgc:66427', 'ENSGWIG00000025338', 'si:rp71-1g18.13', 'ENSGWIG00000025340', 'ENSGWIG00000025341', 'ENSGWIG00000025342', 'st6galnac3', 'st6galnac5a', 'pigk', 'ak5', 'zzz3', 'usp33', 'miga1', 'fubp1', 'gipc2', 'zfyve9a', 'gnal2', 'PIEZO2', 'napgb', 'ENSGWIG00000018775', 'ndufv2', 'ankrd12', 'RAB31', 'ENSGWIG00000019827', 'and2', 'apoda.2', 'ENSGWIG00000020178', 'samd7', 'sec62', 'nadkb', 'skila', 'prkci', 'ENSGWIG00000022299', 'sppl2', 'arhgef18a', 'insra', 'hiat1a', 'stard3nl', 'ENSGWIG00000023054', 'ENSGWIG00000023055', 'ENSGWIG00000023056', 'LARP4B', 'DIP2C', 'zmynd11', 'VIPR2', 'esyt2a', 'ENSGWIG00000026221', 'LACTB2', 'ENSGWIG00000026226', 'pfkpb', 'pitrm1', 'ENSGWIG00000026275', 'fzd8b', 'GJD4', 'CCNY', 'CREM', 'pard3ab', 'NRP1', 'itgb1b', 'ENSGWIG00000026479', 'ENSGWIG00000026480', 'ENSGWIG00000026481', 'ENSGWIG00000026482', 'arhgap12b', 'kif5ba', 'pold3', 'ENSGWIG00000001852', 'zeb1a', 'ENSGWIG00000001926', 'ENSGWIG00000001939', 'SVIL', 'ENSGWIG00000008697', 'kcnj9', 'lig1', 'mpz', 'atp1a2a', 'sdhc', 'cadm3', 'kirrel1b', 'ENSGWIG00000013250', 'calr3b', 'klf2b', 'ap1m1', 'rab8a', 'im:7138239', 'ccl25a', 'ranbp3b', 'NRTN', 'ptprfb', 'cdcp2', 'tmem59', 'lrrc42', 'zgc:113691', 'ndc1', 'dcaf8', 'vav3b', 'ENSGWIG00000026173', 'zgc:92137', 'col11a1b', 'olfm3b', 'abca4b', 'arhgap29b', 'rheb', 'ENSGWIG00000026235', 'PTPRM', 'ENSGWIG00000026560', 'TMEM200C', 'epb41l3b', 'APCDD1', 'si:dkey-154p10.3', 'ENSGWIG00000026619', 'myom1a', 'oprk1', 'rgs20', 'tcea1', 'ENSGWIG00000026682', 'ENSGWIG00000026691', 'neurod6b', 'ENSGWIG00000026694', 'ENSGWIG00000026696', 'xcr1b.1', 'fyco1b', 'adcyap1r1a', 'CRHR2', 'myl13', 'pth1r', 'tmie', 'ENSGWIG00000013967', 'cyp8b2', 'cx47.1', 'iba57', 'snap47', 'wnt9a', 'abcb8', 'cdk5', 'asic1c', 'asb10', 'gbx1', 'agap3', 'ENSGWIG00000014388', 'ENSGWIG00000014389', 'ENSGWIG00000014390', 'ENSGWIG00000014391', 'pcmtd1', 'sntg1', 'SNAI2', 'ube2v2', 'mcm4', 'ENSGWIG00000014504', 'SPIDR', 'mapre2', 'dtna', 'klhl14', 'ENSGWIG00000014589', 'ENSGWIG00000014656', 'cyp7a1', 'amph', 'ENSGWIG00000014690', 'ENSGWIG00000014956', 'ENSGWIG00000015233', 'zgc:123035', 'ENSGWIG00000015236', 'ENSGWIG00000015257', 'zgc:163121', 'ENSGWIG00000015634', 'cahz', 'ENSGWIG00000015677', 'ptf1a', 'ENSGWIG00000015690', 'ENSGWIG00000015695', 'fam110b', 'tmem70', 'si:ch211-207d6.2', 'cnpy1', 'ENSGWIG00000015729', 'ENSGWIG00000015731', 'tyw3', 'ENSGWIG00000015751', 'xpr1b', 'ccr9a', 'bfsp2', 'klhl18', 'ENSGWIG00000015785', 'eif4e2', 'ENSGWIG00000015903', 'prss56', 'vwa5b2', 'alg3', 'mul1a', 'camk2n1a', 'ece2a', 'ENSGWIG00000016029', 'map6d1', 'yeats2', 'klhl24b', 'kcnab1b', 'pask', 'mbnl1', 'ENSGWIG00000002537', 'CUL3', 'scg2b', 'acsl3b', 'ENSGWIG00000006401', 'pax3a', 'ENSGWIG00000006410', 'ENSGWIG00000006417', 'ENSGWIG00000006419', 'ephb1', 'gpc1b', 'gpc5c', 'nyap2b', 'ENSGWIG00000006460', 'ENSGWIG00000006457', 'ENSGWIG00000006468', 'ENSGWIG00000006470', 'fgf12a', 'parla', 'zgc:110366', 'tal1', 'otol1a', 'ppm1la', 'ENSGWIG00000006546', 'irf4a', 'foxq1a', 'foxc1a', 'ENSGWIG00000006629', 'mylk4a', 'ENSGWIG00000006688', 'zp2l2', 'clcn2a', 'fam131a', 'eif4g1a', 'ENSGWIG00000011482', 'tfa', 'ubxn7', 'ppp1r7', 'ENSGWIG00000012376', 'cdc20', 'ELOVL1', 'agxtb', 'thap4', 'kif1ab', 'rab6bb', 'ENSGWIG00000013586', 'amotl2b', 'ENSGWIG00000001985', 'ptch2', 'eif2b3', 'plk3', 'cacybp', 'mrps14', 'tnn', 'tnr', 'ENSGWIG00000008695', 'ENSGWIG00000008696', 'ENSGWIG00000008707', 'OSBPL1A', 'ENSGWIG00000008831', 'ZNF521', 'ENSGWIG00000008839', 'ENSGWIG00000008842', 'ss18', 'psma8', 'KCTD1', 'AQP4', 'ENSGWIG00000008887', 'CDH2', 'ENSGWIG00000015801', 'ENSGWIG00000015802', 'dsc2l', 'ENSGWIG00000015826', 'tubb2', 'ENSGWIG00000015838', 'ENSGWIG00000015839', 'dspa', 'TNS3', 'IGFBP3', 'adcy1b', 'ENSGWIG00000018204', 'lamc1', 'dhx9', 'ENSGWIG00000016581', 'ENSGWIG00000016596', 'rnf150b', 'ENSGWIG00000016604', 'ENSGWIG00000016608', 'ENSGWIG00000019780', 'txk', 'nhsl2', 'ENSGWIG00000002461', 'ndrg2', 'si:cabz01007807.1', 'si:cabz01007807.1-ENSGWIG00000003632', 'ENSGWIG00000003632', 'ENSGWIG00000003636', 'ENSGWIG00000003663', 'arhgef40', 'ENSGWIG00000003695', 'znf219', 'ENSGWIG00000003699', 'ENSGWIG00000003701', 'btr12', 'ENSGWIG00000003738', 'pea15', 'ccnb3', 'ENSGWIG00000003763', 'htr2cl2', 'zgc:109889', 'gpr185b', 'zgc:162171', 'rab39bb', 'vbp1', 'ENSGWIG00000005146', 'ENSGWIG00000005162', 'ENSGWIG00000005175', 'si:dkeyp-74b6.2', 'khnyn', 'ENSGWIG00000005286', 'ENSGWIG00000006189', 'SLC12A6', 'nop10', 'tox4b', 'supt16h', 'tnfsf10l', 'clcn5b', 'ENSGWIG00000014428', 'ENSGWIG00000013749', 'ENSGWIG00000016457', 'ENSGWIG00000016458', 'ENSGWIG00000016460', 'ENSGWIG00000016498', 'vangl2', 'ENSGWIG00000016523', 'pnp5a', 'PACS1', 'ENSGWIG00000019704', 'actn3b', 'rbm14b', 'gal3st3', 'si:ch211-168d1.3', 'zgc:101810', 'six7', 'b4gat1', 'nit1', 'clcn3', 'aadat', 'intu', 'cxcl19', 'f13a1b', 'ENSGWIG00000024994', 'ENSGWIG00000024995', 'tex261', 'vax2', 'gba2', 'rgp1', 'pik3c3', 'ENSGWIG00000025090', 'ENSGWIG00000025092', 'ENSGWIG00000025096', 'haus3', 'si:dkey-11k2.7', 'ENSGWIG00000025104', 'slc7a8a', 'ENSGWIG00000025170', 'ENSGWIG00000025171', 'efs', 'ajuba', 'lrp10', 'ENSGWIG00000014112', 'si:ch211-212k18.5', 'ttc5', 'ENSGWIG00000014308', 'tm4sf5', 'ENSGWIG00000014319', 'ENSGWIG00000015565', 'tnfsf12', 'sat2a', 'si:dkey-19b23.8', 'smim20', 'man2b2', 'rbpjb', 'CCKAR', 'stim2b', 'ENSGWIG00000016963', 'DMRTA1', 'ENSGWIG00000016968', 'RBPMS', 'ENSGWIG00000016983', 'ENSGWIG00000016996', 'mfsd8', 'ENSGWIG00000017187', 'ENSGWIG00000017193', 'msantd1', 'ENSGWIG00000017250', 'HGFAC', 'DOK7', 'ADRA2C', 'HMX1', 'ENSGWIG00000017271', 'CPZ', 'gpr78b', 'trmt44', 'tnk1', 'plscr3b', 'phf23b', 'elp5', 'ctdnep1a', 'gabarapb', 'gps2', 'SLC2A4', 'capgb', 'polr2a', 'zbtb4', 'neurl4', 'hdlbpb', 'si:dkey-38p12.3', 'zpax4', 'ENSGWIG00000013979', 'bcl6b', 'slc16a13', 'acap1', 'si:ch73-335l21.1', 'dvl2', 'ENSGWIG00000014547', 'ntn5', 'si:dkey-19b23.13', 'si:dkey-19b23.12', 'pcolceb', 'per1b', 'rell1', 'ENSGWIG00000014581', 'nwd2', 'ENSGWIG00000014588', 'dthd1', 'ENSGWIG00000014626', 'pcdh7b', 'ENSGWIG00000015169', 'CAAP1', 'ENSGWIG00000015547', 'ENSGWIG00000021811', 'MPDZ', 'LURAP1L', 'tyrp1a', 'ENSGWIG00000024601', 'USP53', 'myoz2a', 'SYNPO2', 'sec24d', 'KCNIP4', 'adgra3', 'gba3', 'ENSGWIG00000026490', 'ppargc1a', 'dhx15', 'ccdc149a', 'ENSGWIG00000026588', 'SLC24A2', 'etnppl', 'larp7', 'ENSGWIG00000026629', 'ADAMTSL1', 'plin2', 'ENSGWIG00000026644', 'dennd4c', 'ENSGWIG00000026683', 'NFIB', 'ENSGWIG00000026730', 'DDIT4L', 'ENSGWIG00000026769', 'pcxb', 'lrfn4b', 'ENSGWIG00000026776', 'ENSGWIG00000026777', 'prox3', 'ppp1r14ba', 'plcb3', 'slc8a4b', 'badb', 'gpha2', 'SNORD31', 'ENSGWIG00000007794', 'naa40', 'rcor2', 'mark2b', 'krcp', 'cct7', 'dok1b', 'm1ap', 'ENSGWIG00000020082', 'syt4', 'si:dkey-183c6.7', 'si:dkey-183c6.8', 'ENSGWIG00000020106', 'ENSGWIG00000020110', 'ENSGWIG00000020154', 'tmem88b', 'ENSGWIG00000021212', 'ENSGWIG00000021218', 'ndufs2', 'kirrel1a', 'ENSGWIG00000022104', 'ENSGWIG00000022105', 'ENSGWIG00000022120', 'fxr2', 'rbm4.3', 'ENSGWIG00000022846', 'kdm6ba', 'ENSGWIG00000022988', 'ENSGWIG00000022990', 'ENSGWIG00000022994', 'ENSGWIG00000022996', 'efnb3b', 'wrap53', 'gigyf1a', 'ufsp1', 'epoa', 'nlgn2a', 'ENSGWIG00000004133', 'tmem256', 'ENSGWIG00000004136', 'tmem102', 'fgf11a', 'chrnb1', 'chrnb1l', 'ENSGWIG00000004403', 'RASGRP2', 'ENSGWIG00000004510', 'ENSGWIG00000004523', 'ENSGWIG00000004558', 'nrxn2b', 'ENSGWIG00000004681', 'si:cabz01078036.1', 'zgc:63470', 'rela', 'ENSGWIG00000004723', 'aco1', 'PAQR9', 'sorcs2', 'grpel1', 'ENSGWIG00000009608', 'ENSGWIG00000009611', 'ENSGWIG00000009613', 'TBC1D14', 'zgc:158564', 'ENSGWIG00000010920', 'SLC29A2', 'ENSGWIG00000010944', 'ENSGWIG00000011004', 'camk2d1', 'arsj', 'mad2l1', 'pde5ab', 'ENSGWIG00000013497', 'zcchc4', 'si:dkey-219e21.4', 'afap1', 'ablim2', 'ENSGWIG00000007740', 'ENSGWIG00000007751', 'ache', 'ENSGWIG00000007768', 'ENSGWIG00000007778', 'ap1s1', 'serpine1', 'ENSGWIG00000007821', 'ENSGWIG00000007832', 'ENSGWIG00000007840', 'gal3st4', 'zgc:172079', 'mpdu1b', 'eif4a1a', 'ENSGWIG00000009891', 'senp3b', 'si:ch73-41h24.1', 'focad', 'mllt3', 'rbm4.1', 'sf1', 'pygmb', 'ENSGWIG00000010184', 'HTRA3', 'ENSGWIG00000010189', 'nat16', 'acox3', 'ENSGWIG00000010207', 'mtm1', 'mtmr1a', 'cd99l2', 'pelp1', 'trmt10a', 'slc1a1', 'RAP1GDS1', 'ENSGWIG00000012308', 'taf6l', 'rtn3', 'si:dkey-106g10.7', 'ENSGWIG00000012327', 'ecsit', 'eml3', 'mta2', 'cth1', 'men1', 'map4k2', 'ENSGWIG00000024109', 'ENSGWIG00000024111', 'ENSGWIG00000024114', 'ENSGWIG00000024126', 'ENSGWIG00000024135', 'ehbp1l1a', 'ascc1', 'ENSGWIG00000024858', 'ppa1a', 'ndr2', 'pald1a', 'adamts14', 'polr3a', 'ENSGWIG00000025151', 'ppifb', 'ZCCHC24', 'dlg5b.1', 'ENSGWIG00000025420', 'ZNF503', 'COMTD1', 'DUPD1', 'kat6b', 'ADK', 'vclb', 'dcakd', 'ENSGWIG00000026665', 'ENSGWIG00000026673', 'zdhhc6', 'vti1a', 'tcf7l2', 'ENSGWIG00000008721', 'crygmxl2', 'crygmx', 'aldh18a1', 'med1', 'rpl19', 'ENSGWIG00000009396', 'ENSGWIG00000009459', 'DNAJC7', 'aclyb', 'spsb3b', 'ATP6V0C', 'tbc1d24', 'ENSGWIG00000011521', 'ntn2', 'nog2', 'ENSGWIG00000011526', 'capn15', 'MSS51', 'ENSGWIG00000011553', 'ENSGWIG00000011555', 'lrrc4bb', 'ENSGWIG00000011558', 'ENSGWIG00000011573', 'ENSGWIG00000011795', 'PPFIA3', 'ENSGWIG00000017040', 'ENSGWIG00000017050', 'rexo4', 'pigp', 'ENSGWIG00000017065', 'im:7160594', 'MPRIP', 'si:ch211-248l17.3', 'crp1', 'ENSGWIG00000017498', 'ENSGWIG00000018278', 'prr12a', 'irf3', 'ENSGWIG00000018828', 'ENSGWIG00000018832', 'ENSGWIG00000018834', 'zgc:92313', 'si:ch1073-13h15.3', 'col1a1b', 'sgca', 'ENSGWIG00000019659', 'ppp1r9bb', 'ENSGWIG00000019660', 'cox10', 'si:ch211-216b21.2', 'tekt3', 'cdc42ep4b', 'sdk2b', 'ENSGWIG00000019754', 'rpl38', 'ttyh2', 'kif19', 'ENSGWIG00000019787', 'ENSGWIG00000019792', 'rgs9b', 'bicral', 'QKI', 'zgc:112285', 'timm23a', 'syt15', 'ENSGWIG00000003013', 'si:dkey-49c17.4', 'si:dkey-49c17.3', 'ENSGWIG00000004746', 'st6galnac1.1', 'sfxn2', 'arl3b', 'timp2a', 'pgs1', 'socs3b', 'tha1', 'eef2k', 'polr3e', 'ENSGWIG00000007485', 'si:ch73-127m5.1', 'ENSGWIG00000007864', 'sec24c', 'SYNPO2L', 'myoz1b', 'glud1b', 'ldb3b', 'opn4b', 'ENSGWIG00000022446', 'mta3', 'cox7a2l', 'pigf', 'cript', 'msh2', 'ENSGWIG00000024256', 'ENSGWIG00000024750', 'chac2', 'psme4b', 'entpd1', 'ptprea', 'clrn3', 'foxi1', 'dock1', 'ADAM12', 'mmp21', 'ENSGWIG00000026345', 'fam53b', 'lhpp', 'oat', 'CHST15', 'CPXM2', 'bub3', 'mto1', 'adprh', 'ENSGWIG00000026450', 'kif22', 'tlcd3bb', 'si:ch211-166g5.4', 'ENSGWIG00000003648', 'nif3l1', 'eif3c', 'ENSGWIG00000009886', 'ENSGWIG00000009888', 'itgam', 'tmem101', 'TANC2', 'wnk4b', 'vps25', 'sema4gb', 'twnk', 'lzts2b', 'selenou1b', 'scn4aa', 'ENSGWIG00000017117', 'ENSGWIG00000017124', 'plekhm1', 'ENSGWIG00000017137', 'ABCA5', 'baiap2b', 'aatkb', 'ENSGWIG00000017266', 'ENSGWIG00000017274', 'ENSGWIG00000017289', 'ENSGWIG00000017294', 'ENSGWIG00000017285', 'ENSGWIG00000017295', 'cep131', 'ENSGWIG00000017681', 'pde6gb', 'OXLD1', 'gprc5c', 'gpr142', 'map2k6', 'kcnj2a', 'kcnj16', 'trim16', 'OTOP3', 'hid1b', 'ccr10', 'ENSGWIG00000018026', 'SRCIN1', 'grb7', 'arf2b', 'grnb', 'ENSGWIG00000018058', 'ENSGWIG00000018090', 'ENSGWIG00000018092', 'armc7', 'cox11', 'ENSGWIG00000014470', 'mat2b', 'ENSGWIG00000014500', 'ENSGWIG00000014573', 'ENSGWIG00000014759', 'asah2', 'ENSGWIG00000014803', 'ENSGWIG00000014810', 'GRID1', 'waplb', 'ENSGWIG00000018966', 'htra1b', 'ENSGWIG00000018994', 'acsl5', 'ENSGWIG00000019521', 'g6pca.1', 'g6pca.2', 'nbr1a', 'thrab', 'etv4', 'ormdl3', 'wnt3', 'nsfb', 'MYO1D', 'pdk2a', 'tbkbp1', 'tbx21', 'ENSGWIG00000023697', 'ENSGWIG00000023696', 'skap1', 'hoxb1b', 'ENSGWIG00000023765', 'fbrs', 'ENSGWIG00000023783', 'stx4', 'arhgap44', 'acsf2', 'tvp23b', 'tap2t', 'slc16a5a', 'kctd2', 'ENSGWIG00000023874', 'ENSGWIG00000023921', 'mapta', 'kansl1a', 'crhr1', 'fam171a2b', 'ENSGWIG00000024336', 'taco1', 'ENSGWIG00000002734', 'ENSGWIG00000002921', 'ENSGWIG00000002950', 'ngfrb', 'ENSGWIG00000003730', 'smarcd2', 'psmc5', 'mpp3b', 'ENSGWIG00000004549', 'GID4', 'fam20cb', 'ENSGWIG00000005181', 'trrap', 'tecpr1a', 'bhlha15', 'ENSGWIG00000009011', 'ENSGWIG00000009014', 'ENSGWIG00000009019', 'ccz1', 'ENSGWIG00000009101', 'pms2', 'usp42', 'ENSGWIG00000009296', 'uqcrc2b', 'birc5a', 'tmem235b', 'fscn2b', 'ENSGWIG00000012821', 'bahcc1b', 'slc38a10', 'tepsin', 'AMZ2', 'arsg', 'WIPI1', 'fam20a', 'ENSGWIG00000013086', 'ENSGWIG00000014411', 'dlx4b', 'dlx3b', 'itga3b', 'ace', 'ENSGWIG00000016757', 'mpp2b', 'mki67', 'raraa', 'GJD3', 'top2a', 'lrrc3ca', 'ENSGWIG00000018317', 'med24', 'ENSGWIG00000018325', 'ENSGWIG00000018327', 'U5', 'cox6b1', 'fam234a', 'eps8l1b', 'ENSGWIG00000018401', 'SLC17A7', 'ENSGWIG00000007978', 'ENSGWIG00000007981', 'zgc:91999', 'ENSGWIG00000007992', 'znf668', 'znf646', 'ENSGWIG00000008008', 'rnf40', 'si:ch73-81k8.2', 'ube2ia', 'kctd5a', 'pdpk1b', 'ENSGWIG00000011027', 'zmp:0000000624', 'zgc:112185', 'rhbdl1', 'h1', '0', 'si:ch211-183d21.1', 'si:ch211-183d21.3', 'ENSGWIG00000011436', 'rab11fip3', 'telo2', 'ift140', 'tmem204', 'metrn', 'NT5C3B', 'ADAM11', 'ENSGWIG00000014764', 'VAT1', 'ENSGWIG00000014778', 'ENSGWIG00000014782', 'ENSGWIG00000014865', 'itgb3b', 'cpn1', 'dnmbp', 'ENSGWIG00000022703', 'zgc:158404', 'adrb1', 'ccdc186', 'afap1l2', 'atrnl1b', 'gfra1b', 'ENSGWIG00000023725', 'ENSGWIG00000023726', 'ENSGWIG00000023727', 'ENSGWIG00000023729', 'cep55l', 'pde6c', 'fra10ac1', 'si:ch211-197g18.2', 'plce1', 'ENSGWIG00000014268', 'nmt1b', 'svild', 'kcnj19b', 'znf281b', 'tex2l', 'cdc42ep1a', 'cby1', 'si:ch73-139e5.2', 'si:ch73-139e5.4', 'aco2', 'tefa', 'zc3h7bb', 'rangap1b', 'chadlb', 'ep300a', 'ENSGWIG00000019233', 'ENSGWIG00000019305', 'shisa8b', 'mrtfab', 'vps35l', 'ubtd1b', 'antxr1c', 'rbp3', 'ENSGWIG00000019447', 'ENSGWIG00000019448', 'mapk8b', 'ENSGWIG00000021459', 'si:dkey-191m6.4', 'lrrc18b', 'vstm4b', 'ENSGWIG00000021570', 'sox9a', 'ENSGWIG00000021580', 'ENSGWIG00000021581', 'cox19', 'lmf1', 'RAB26', 'pemt', 'ENSGWIG00000021766', 'cops3', 'ENSGWIG00000021785', 'sstr2a', 'ENSGWIG00000021789', 'SLC39A11', 'spata20', 'cacna1g', 'UTS2R', 'ENSGWIG00000023776', 'uts2r2', 'narf', 'fdxr', 'wbp2', 'ENSGWIG00000014257', 'ENSGWIG00000014263', 'itgb4', 'sap30bp', 'ENSGWIG00000015253', 'si:ch211-141o9.10', 'tnrc6b', 'ENSGWIG00000015321', 'maff', 'ENSGWIG00000015408', 'rsl1d1', 'snx29', 'ENSGWIG00000015434', 'shisa9a', 'cpped1', 'mrtfba', 'ubald1b', 'zgc:163057', 'ENSGWIG00000015573', 'foxj1b', 'ENSGWIG00000015601', 'ENSGWIG00000015602', 'lcmt1', 'aqp8b', 'arhgap17a', 'grid2ipb', 'ENSGWIG00000018862', 'zdhhc4', 'msrb1b', 'SPAG9', 'daglb', 'mrpl58', 'ENSGWIG00000019348', 'tob1b', 'ENSGWIG00000019352', 'abcc3', 'si:ch211-176g6.2', 'ENSGWIG00000020376', 'wu:fj29h11', 'ENSGWIG00000014244', 'dhrs7cb', 'hs3st3l', 'ENSGWIG00000014259', 'ENSGWIG00000014265', 'ENSGWIG00000014286', 'ENSGWIG00000014289', 'ENSGWIG00000014290', 'dcxr', 'ENSGWIG00000014316', 'lrrc45', 'hhex', 'exoc6', 'cyp26a1', 'jmjd1cb', 'reep3b', 'nrbf2b', 'ENSGWIG00000014601', 'egr2b', 'adoa', 'bms1', 'mtr', 'ENSGWIG00000017606', 'ENSGWIG00000017644', 'FMN2', 'ATP6V0A1', 'cavin1b', 'ENSGWIG00000017951', 'naglu', 'ipmkb', 'tfam', 'zgc:171971', 'ENPP7', 'ENSGWIG00000017970', 'ENSGWIG00000017971', 'si:ch211-214e3.5', 'ENSGWIG00000017977', 'rab3gap1', 'fasn', 'ENSGWIG00000021095', 'ENSGWIG00000021096', 'slc16a3', 'ENSGWIG00000021132', 'ENSGWIG00000021154', 'ENSGWIG00000021178', 'ENSGWIG00000021279', 'ENSGWIG00000022333', 'ENSGWIG00000022335', 'galk1', 'ENSGWIG00000022337', 'llgl2', 'ENSGWIG00000024497', 'ENSGWIG00000024498', 'ENSGWIG00000024499', 'tmem94', 'tdrkh', 'mbtd1', 'nme2a', 'ENSGWIG00000024603', 'ENSGWIG00000024604', 'ENSGWIG00000024605', 'ca10a', 'ENSGWIG00000024610', 'ENSGWIG00000024611', 'ENSGWIG00000024613', 'eme1', 'RAB37', 'ENSGWIG00000003602', 'ENSGWIG00000003665', 'si:ch211-119c20.2', 'stk17a', 'vopp1', 'lancl2', 'znf438', 'ZEB1', 'ARHGAP12', 'ENSGWIG00000023132', 'KIF5B', 'si:ch211-276f18.2', 'ENSGWIG00000023251', 'ENSGWIG00000023253', 'ENSGWIG00000023256', 'ENSGWIG00000023259', 'ENSGWIG00000023260', 'adarb2', 'wdr37', 'idi1', 'CPNE3', 'wwp1', 'ENSGWIG00000020817', 'snx16', 'ENSGWIG00000020820', 'ENSGWIG00000020832', 'ENSGWIG00000020886', 'ANKS6', 'rgs9bp', 'dph2', 'ncf2', 'ENSGWIG00000020953', 'arhgap29a', 'ENSGWIG00000021113', 'ENSGWIG00000021118', 'tecrl2a', 'ENSGWIG00000021123', 'olfm3a', 'ENSGWIG00000021153', 'col11a1a', 'rnpc3', 'ENSGWIG00000021184', 'prmt6', 'ntng1a', 'VAV3', 'cngb3.1', 'ENSGWIG00000021690', 'MMP16', 'Y_RNA', 'tfr1b', 'tnk2a', 'tmtopsb', 'and1', 'arhgap21a', 'unm_sa821', 'cnn3b', 'mos', 'plag1', 'ARHGEF4', 'ENSGWIG00000021963', 'ENSGWIG00000021965', 'ENSGWIG00000021984', 'ENSGWIG00000014258', 'ENSGWIG00000014274', 'ENSGWIG00000014281', 'zgc:110852', 'ccr12b.2', 'si:dkey-225n22.4', 'slc12a7b', 'cubn', 'c1ql3a', 'ENSGWIG00000021325', 'ENSGWIG00000021343', 'ENSGWIG00000021680', 'TGFBR1', 'reck', 'ENSGWIG00000022134', 'snx7', 'plppr5b', 'plppr4b', 'ENSGWIG00000022138', 'mastl', 'ABI1', 'pdss1', 'ENSGWIG00000022230', 'apbb1ip', 'ENSGWIG00000022235', 'gad2', 'myo3a', 'anxa13', 'gpr158a', 'PTCHD3', 'abcd3a', 'ENSGWIG00000025045', 'ENSGWIG00000025047', 'lztfl1', 'gli3', 'prpf4bb', 'TMX3', 'DSEL', 'ENSGWIG00000025227', 'CDH7', 'ENSGWIG00000025274', 'ENSGWIG00000025278', 'si:dkey-192l18.9', 'ralaa', 'ENSGWIG00000025281', 'cdk13', 'ENSGWIG00000025285', 'si:ch211-282b22.1', 'ca2', 'ENSGWIG00000025289', 'rdh12l', 'si:dkey-61p9.11', 'ENSGWIG00000025293', 'topbp1', 'ENSGWIG00000025574', 'scrib', 'nrbp2b', 'ENSGWIG00000025785', 'ENSGWIG00000025787', 'slc4a2b', 'si:ch73-173p19.1', 'smarcd3b', 'si:dkey-10f23.2', 'asap1b', 'fam49bb', 'si:dkey-37o8.1', 'myca', 'ankha', 'otulina', 'pou6f2', 'vps41', 'ENSGWIG00000020416', 'mrpl15', 'OPRK1', 'fastkd3', 'myom1b', 'emilin2b', 'mettl4', 'ADCYAP1', 'YES1', 'tyms', 'ENSGWIG00000022643', 'ENSGWIG00000022644', 'hacd1', 'TMEM236', 'mrc1a', 'cacnb2a', 'arl8', 'PLXDC2', 'esyt2b', 'ENSGWIG00000023531', 'ncapg2', 'ptprn2', 'dnajb6b', 'ube3c', 'lmbr1', 'rnf32', 'shha', 'rbm33a', 'en2a', 'HTR5A', 'ENSGWIG00000023654', 'dlgap1b', 'otud1', 'si:ch211-285f17.1', 'si:dkey-96n2.3', 'pcolce2b', 'trpc1', 'chst2b', 'ENSGWIG00000023953', 'dipk2ab', 'plod2', 'si:ch73-206p6.1', 'zic1', 'agtr1b', 'gyg1b', 'nck1b', 'ENSGWIG00000025779', 'csrnp1b', 'ENSGWIG00000025782', 'slc22a13b', 'ENSGWIG00000025784', 'oxsr1b', 'cry', 'dash', 'map3k22', 'ENSGWIG00000005828', 'ENSGWIG00000005832', 'plcd1a', 'vill', 'ctdspla', 'spice1', 'boc', 'usf3', 'naa50', 'atp6v1ab', 'gramd1c', 'zdhhc23b', 'ccdc191', 'qtrt2', 'drd3', 'fam162a', 'ENSGWIG00000010959', 'vipr1b', 'sec22c', 'nktr', 'klhl40b', 'shisa2b', 'atp8a2', 'ENSGWIG00000016510', 'rnf6', 'wasf3b', 'usp12a', 'gtf3ab', 'lnx2a', 'si:ch211-140b10.6', 'urad', 'ENSGWIG00000016714', 'spata13', 'tagln3a', 'ythdf3', 'bhlhe22', 'armc1', 'mtfr1', 'pde7a', 'crhb', 'rrs1', 'sgk3', 'TCF24', 'ppp1r42', 'cops5', 'arfgef1', 'cpa6', 'prex2', 'zgc:162928', 'sulf1', 'SLCO5A1', 'prdm14', 'ncoa2', 'xkr9', 'eya1', 'msc', 'trpa1b', 'si:dkey-192j17.1', 'NCEH1', 'ENSGWIG00000002789', 'emc9', 'ENSGWIG00000002825', 'psme2', 'kbtbd2', 'ENSGWIG00000003053', 'rnf152', 'CDH20', 'ENSGWIG00000009598', 'drosha', 'CDH6', 'ENSGWIG00000015375', 'ENSGWIG00000015855', 'cdh12a', 'CDH18', 'epc1a', 'ENSGWIG00000016401', 'ggh', 'dpp6a', 'adgrg2a', 'ENSGWIG00000016425', 'ptchd1', 'phex', 'mbtps2', 'smpx', 'KLHL34', 'ENSGWIG00000016488', 'ENSGWIG00000016524', 'ccl20b', 'map3k15', 'sh3kbp1', 'pomp', 'RAMP3', 'ENSGWIG00000016555', 'SLC22A17', 'phldb2b', 'mak', 'tfap2a', 'ENSGWIG00000016671', 'ENSGWIG00000016673', 'ENSGWIG00000016675', 'slc35b3', 'eef1e1', 'bloc1s5', 'txndc5', 'bmp6', 'pdcd6', 'rec8b', 'si:ch211-196f5.2', 'ipo4', 'tm9sf1', 'ENSGWIG00000016961', 'psme1', 'ENSGWIG00000017025', 'dcaf11', 'thtpa', 'ENSGWIG00000019206', 'pabpn1', 'si:dkey-10f21.4', 'zc2hc1a', 'ENSGWIG00000019232', 'ENSGWIG00000019236', 'dap', 'ankrd33ba', 'nup58', 'amer2', 'ENSGWIG00000019279', 'zfhx4', 'pex2', 'ENSGWIG00000019367', 'hnf4g', 'pign', 'relch', 'ENSGWIG00000007320', 'sec61g', 'tpk1', 'ENSGWIG00000007508', 'c8g', 'msrb2', 'COMMD3', 'mllt10', 'ENSGWIG00000011738', 'pdia4', 'GPT', 'ppp1r16a', 'myrip', 'ENSGWIG00000012118', 'ENSGWIG00000012119', 'pola1', 'pcyt1ba', 'pdk3a', 'slc51a', 'si:rp71-1f1.4', 'march6', 'mtrr', 'ENSGWIG00000015372', 'sema5a', 'si:dkey-118j18.2', 'neto1l', 'ENSGWIG00000017857', 'cbln2b', 'ENSGWIG00000017865', 'ENSGWIG00000017870', 'socs6a', 'rttn', 'cd226', 'dok6', 'crispld1a', 'ENSGWIG00000020638', 'slc39a10', 'dnah7', 'esyt3', 'si:ch211-219a4.3', 'ENSGWIG00000020711', 'faima', 'cx43.4', 'hdac4', 'si:dkey-34f9.3', 'asb1', 'agr1', 'adarb1b', 'ENSGWIG00000022089', 'zgc:162396', 'ube2f', 'zgc:91818', 'ENSGWIG00000022103', 'TMEM163', 'ENSGWIG00000022113', 'si:ch73-116o1.2', 'ptpn4a', 'epb41l5', 'ENSGWIG00000022150', 'ENSGWIG00000023683', 'ENSGWIG00000023685', 'ENSGWIG00000024048', 'cdk5r2b', 'wnt6b', 'wnt10a', 'ENSGWIG00000024056', 'myo1b', 'ENSGWIG00000024257', 'ENSGWIG00000024260', 'ENSGWIG00000024261', 'lrp1bb', 'ENSGWIG00000024391', 'stxbp5l', 'gtf2e1', 'rabl3', 'ugt1a1', 'nxph2a', 'ENSGWIG00000002895', 'HNMT', 'hnmt', 'thsd7ba', 'ENSGWIG00000003482', 'cxcr4a', 'dars1', 'gmppaa', 'ENSGWIG00000006876', 'asic4a', 'chpfa', 'agpat3', 'ENSGWIG00000007070', 'ENSGWIG00000007073', 'ENSGWIG00000007076', 'fzd7b', 'ENSGWIG00000007115', 'ENSGWIG00000007126', 'tsc22d1', 'kctd4', 'GTF2F2', 'ENSGWIG00000007188', 'lims2', 'GCC2', 'SLC5A7', 'CLDN14', 'EPHA6', 'ENSGWIG00000007259', 'ENSGWIG00000007261', 'STX19', 'epha3', 'erbb4b', 'ikzf2', 'vwc2l', 'bard1', 'abca12', 'ENSGWIG00000023476', 'ptprna', 'vgll3', 'pou1f1', 'HTR1F', 'zgc:152951', 'si:ch211-261p9.4', 'cps1', 'myl1', 'ENSGWIG00000025449', 'ENSGWIG00000025450', 'ENSGWIG00000025451', 'ENSGWIG00000025452', 'heg1', 'znf148', 'ENSGWIG00000025459', 'asmt', 'akap17a', 'sept10', 'ENSGWIG00000025465', 'ENSGWIG00000025466', 'edar', 'desma', 'ENSGWIG00000025490', 'ENSGWIG00000009152', 'snx4', 'ENSGWIG00000009287', 'arsh', 'ENSGWIG00000009294', 'ENSGWIG00000009300', 'ENSGWIG00000009304', 'ENSGWIG00000009309', 'ercc1', 'ENSGWIG00000009316', 'dip2a', 'ENSGWIG00000010960', 'zgc:158316', 'trpm2', 'si:ch211-45c16.2', 'pfkla', 'tmem169b', 'xrcc5', 'ENSGWIG00000023278', 'ENSGWIG00000023286', 'nme8', 'zgc:162707', 'ehhadh', 'tmem41aa', 'igf2bp2a', 'hce2l1', 'ankzf1', 'glb1l', 'ENSGWIG00000023877', 'ENSGWIG00000023938', 'prpf40a', 'rprmb', 'galnt13', 'kcnj3a', 'nr4a2a', 'ENSGWIG00000025137', 'ENSGWIG00000025138', 'ENSGWIG00000025140', 'ENSGWIG00000025145', 'ENSGWIG00000025194', 'PKP4', 'ENSGWIG00000025197', 'dap1b', 'tanc1b', 'rbms1b', 'tank', 'tbr1b', 'slc4a10b', 'fap', 'ifih1', 'KCNH7', 'fign', 'ENSGWIG00000026541', 'ENSGWIG00000026545', 'ENSGWIG00000026544', 'ENSGWIG00000026546', 'ENSGWIG00000026639', 'ENSGWIG00000026804', 'ENSGWIG00000026805', 'GALNT3', 'ENSGWIG00000026809', 'ENSGWIG00000026810', 'xirp2b', 'ENSGWIG00000026812', 'B3GALT1', 'stk39', 'cers6', 'abcb11a', 'dhrs9', 'ENSGWIG00000026827', 'col28a2a', 'bbs5', 'fastkd1', 'ccdc173', 'ssb', 'klhl23', 'ubr3', 'myo3b', 'gad1a', 'tlk1a', 'dync1i2a', 'SLC25A12', 'hat1', 'dlx1a', 'itga6a', 'NUP62', 'pdk1', 'ppp1r9ala', 'rapgef4', 'cdca7a', 'SP3', 'si:ch73-167c12.2', 'sp9', 'CHRNA1', 'chn1', 'atf2', 'atp5mc3b', 'lnpa', 'hoxd10a', 'hoxd3a', 'hnrnpa3', 'nfe2l2a', 'agps', 'pde11a', 'cycsb', 'osbpl6', 'adam23a', 'ino80da', 'DOCK9', 'ENSGWIG00000026483', 'FARP1', 'RAP2A', 'MBNL2', 'hs6st3b', 'uggt2', 'ABCC4', 'dct', 'gpc6b', 'ENSGWIG00000026733', 'FASTKD5', 'slitrk5b', 'ENSGWIG00000026736', 'ENSGWIG00000026737', 'pcdh8', 'si:ch211-199f5.1', 'ednrbb', 'cog3', 'mid1', 'tlr7', 'ENSGWIG00000014494', 'egfl6', 'trappc2', 'ofd1', 'gpm6bb', 'ENSGWIG00000014527', 'glra2', 'ENSGWIG00000014615', 'ENSGWIG00000015212', 'gpr143', 'tbl1x', 'ANOS1', 'ENSGWIG00000015264', 'ENSGWIG00000015271', 'igsf3', 'rpe', 'ackr3a', 'ENSGWIG00000015284', 'ENSGWIG00000015307', 'ENSGWIG00000015309', 'casq2', 'ENSGWIG00000015636', 'ENSGWIG00000015637', 'morc3a', 'chaf1b', 'si:dkey-229b18.3', 'bcl9', 'gja5a', 'ENSGWIG00000015665', 'gja8a', 'ENSGWIG00000015937', 'zgc:113337', 'si:ch73-390b10.2', 'ENSGWIG00000015951', 'FAM155A', 'abhd13', 'myo16', 'ENSGWIG00000015973', 'col4a1', 'ENSGWIG00000015988', 'rab20', 'ankrd10b', 'sik1', 'u2af1', 'gabpa', 'piga', 'XK', 'sytl5', 'srpx', 'me3', 'gdf3', 'klf7b', 'creb1b', 'ccnyl1', 'plekhm3', 'sestd1', 'ENSGWIG00000022618', 'cwc22', 'itprid2', 'ENSGWIG00000023182', 'itga4', 'rpl8', 'usp9', 'med14', 'ENSGWIG00000009333', 'PPP2R3B', 'ENSGWIG00000009586', 'shox', 'ENSGWIG00000009592', 'ENSGWIG00000009595', 'asmtl', 'upf3a', 'cdc16', 'ENSGWIG00000010085', 'fgf14', 'itgbl1', 'nalcn', 'tmtc4', 'ENSGWIG00000010166', 'zic2a', 'zic5', 'clybl', 'si:ch211-184m13.4', 'gpr18', 'dachd', 'klf5b', 'klf12b', 'tbc1d4', 'uchl3', 'lmo7a', 'kctd12.1', 'cln5', 'ENSGWIG00000011737', 'slain1a', 'tspan7b', 'mid1ip1b', 'ENSGWIG00000011845', 'bcor', 'ENSGWIG00000012601', 'ENSGWIG00000012603', 'sh3bp4', 'arl4cb', 'lrch1', 'esd', 'htr2aa', 'ENSGWIG00000012711', 'sucla2', 'med4', 'itm2bb', 'ENSGWIG00000012740', 'alg11', 'lcp1', 'trim13', 'spryd7b', 'rnaseh2b', 'ENSGWIG00000015383', 'NR0B1', 'il1rapl1a', 'ENSGWIG00000015579', 'dcbld2', 'COL8A1', 'mpp4a', 'itgav', 'lmln', 'enox1', 'ENSGWIG00000014591', 'cnot11', 'ENSGWIG00000014693', 'ENSGWIG00000014711', 'ATP11A', 'ENSGWIG00000015486', 'sox1a', 'arhgef7a', 'aff3', 'rev1', 'mitd1', 'ENSGWIG00000015649', 'mgat4a', 'inpp4aa', 'tmem182a', 'fhl2a', 'nck2a', 'ecrg4a', 'uxs1', 'tmtops2b', 'ST6GAL2', 'jam2b', 'zgc:172067', 'grtp1b', 'pou2f1b', 'ENSGWIG00000016957', 'ildr2', 'slc15a2', 'hspbap1', 'slc49a4', 'sema5ba', 'pdia5', 'sec22a', 'adcy5', 'ENSGWIG00000020254', 'stam2', 'ENSGWIG00000004654', 'ARL5A', 'neb', 'TBC1D8', 'PDCL3', 'ENSGWIG00000005634', 'si:ch1073-440b2.1', 'ENSGWIG00000005808', 'ENSGWIG00000007673', 'hacd2', 'ENSGWIG00000007680', 'ENSGWIG00000007686', 'ENSGWIG00000007710', 'arhgap20', 'rnf17', 'cenpj', 'ttf2', 'mphosph8', 'zmym2', 'gja3', 'cx30.3', 'cryl1', 'ift88', 'eef1akmt1', 'xpo4', 'lats2', 'ska3', 'ildr1b', 'ENSGWIG00000010126', 'ENSGWIG00000010125', 'ENSGWIG00000010128', 'ankrd50l', 'hpxa', 'olig1', 'olig2', 'ENSGWIG00000010537', 'fam160a2', 'cnga4', 'si:dkey-91i10.3', 'eaf2', 'si:dkey-91i10.2', 'ENSGWIG00000010562', 'ENSGWIG00000010563', 'ENSGWIG00000010565', 'tsn', 'ENSGWIG00000010573', 'ENSGWIG00000010576', 'ENSGWIG00000010577', 'dhrs12', 'wdfy2', 'ints6', 'serpine3', 'actr3', 'ENSGWIG00000004149', 'DPP10', 'ENSGWIG00000006927', 'ENSGWIG00000006943', 'cops8', 'ENSGWIG00000006965', 'mlphb', 'RAB17', 'ENSGWIG00000006983', 'ENSGWIG00000006986', 'tyw5', 'maip1', 'pgap1', 'hecw2a', 'ENSGWIG00000007164', 'zgc:153521', 'stk17b', 'ENSGWIG00000007172', 'ENSGWIG00000007174', 'ENSGWIG00000007176', 'ENSGWIG00000007183', 'gpr39', 'SLC35F5', 'sympk', 'slc40a1', 'wdr75', 'stat4', 'ENSGWIG00000013640', 'ENSGWIG00000013742', 'osgepl1', 'pms1', 'mstnb', 'ENSGWIG00000016268', 'hibch', 'mcm3ap', 'usp37', 'cnot9', 'plcd4b', 'znf142', 'ENSGWIG00000021786', 'tns1b', 'ENSGWIG00000021806', 'igfbp5b', 'igfbp2b', 'ENSGWIG00000022038', 'slc4a3', 'col5a2a', 'ENSGWIG00000023480', 'gulp1a', 'ENSGWIG00000023492', 'rftn2', 'satb2', 'dnajb11', 'tbccd1', 'lcmt2', 'rab5b', 'dbr1', 'armc8', 'kcnh3', 'ENSGWIG00000023866', 'idh1', 'ENSGWIG00000015000', 'ENSGWIG00000015018', 'ENSGWIG00000015028', 'akap11', 'ENSGWIG00000015042', 'dgkh', 'vwa8', 'rgcc', 'f5', 'ccdc80', 'kdm6a', 'ndp', 'mao', 'CASK', 'ddx3xa', 'nup35', 'frzb', 'ENSGWIG00000025541', 'dusp19a', 'HYKK', 'ENSGWIG00000025560', 'IMPG2', 'ENSGWIG00000025864', 'tfg', 'jagn1a', 'trim45', 'man1a2', 'gdap2', 'wdr3', 'trappc10', 'hcn5', 'pla1a', 'ENSGWIG00000026049', 'tbx15', 'wars2', 'slc37a1', 'pde9a', 'wdr4', 'ENSGWIG00000026112', 'pknox1.2', 'tmprss3a', 'ENSGWIG00000026116', 'ccdc93', 'si:dkey-251i10.3', 'insig2', 'ENSGWIG00000026203', 'ENSGWIG00000026204', 'cntnap5a', 'ENSGWIG00000026256', 'ENSGWIG00000026257', 'NMI', 'rnd3b', 'map4k5', 'l2hgdh', 'sos2', 'cnih1', 'gmfb', 'lgals3a', 'synj2bp', 'ENSGWIG00000009673', 'utp25', 'adprhl2', 'ENSGWIG00000009705', 'ENSGWIG00000009707', 'ENSGWIG00000009712', 'nrde2', 'ENSGWIG00000009721', 'dicer1', 'gsc', 'enpep', 'lrit3b', 'gpr135', 'clmn', 'syne3', 'gabrr2a', 'rragd', 'ENSGWIG00000009844', 'ENSGWIG00000009879', 'ENSGWIG00000009937', 'ENSGWIG00000009939', 'ENSGWIG00000009946', 'ENSGWIG00000009950', 'ENSGWIG00000009963', 'ilkap', 'ENSGWIG00000009976', 'ENSGWIG00000009983', 'sdsl', 'syt14a', 'ENSGWIG00000015121', 'slc8a3', 'ENSGWIG00000015136', 'ENSGWIG00000015137', 'ENSGWIG00000015139', 'cox16', 'actn1', 'dcaf5', 'pacs2', 'srsf5a', 'tmem151bb', 'mtres1', 'bend3', 'pdss2', 'ENSGWIG00000016389', 'sobpa', 'rock2a', 'si:dkeyp-47f9.4', 'alk', 'ENSGWIG00000018388', 'ENSGWIG00000018393', 'ENSGWIG00000018397', 'ENSGWIG00000018404', 'commd8', 'atp10d', 'ENSGWIG00000018549', 'oip5', 'ndufaf1', 'golga5', 'trip11', 'fbln5', 'plek2', 'zfyve26', 'rdh12', 'ENSGWIG00000003458', 'arg2', 'rbm25a', 'plekhd1', 'smoc1', 'ENSGWIG00000003972', 'ENSGWIG00000004262', 'ENSGWIG00000004341', 'paplna', 'psen1', 'ENSGWIG00000004727', 'vps39', 'galnt16', 'ENSGWIG00000004900', 'si:ch211-168d23.3', 'ENSGWIG00000004917', 'tulp4b', 'ENSGWIG00000005292', 'ENSGWIG00000006049', 'shprh', 'fbxo30a', 'ENSGWIG00000007257', 'grm1b', 'stxbp5b', 'sash1b', 'plekhh1', 'zfp36l1a', 'tmem229b', 'zgc:172341', 'mta1', 'crip2', 'ENSGWIG00000018326', 'elmsan1b', 'pygl', 'trim9', 'tmx1', 'frmd6', 'gng2', 'dusp23a', 'oct2', 'txlnbb', 'hivep2b', 'coq8ab', 'dnajc5gb', 'cenpo', 'slc5a6b', 'ENSGWIG00000009222', 'ENSGWIG00000009225', 'SCML4', 'ENSGWIG00000009262', 'zbtb24', 'slc25a21', 'nkx2.1', 'mbip', 'rps6ka5', 'hadhab', 'kif3cb', 'asxl2', 'dtnbb', 'dnmt3ab', 'dpysl5a', 'mapre3a', 'ENSGWIG00000022696', 'ENSGWIG00000022742', 'ENSGWIG00000022743', 'C14orf132', 'ENSGWIG00000022754', 'gpr65', 'kcnk10b', 'ENSGWIG00000022787', 'ptpn21', 'spred1', 'meis2a', 'zgc:154061', 'dph6', 'aqr', 'GJD2', 'stxbp6', 'fermt2', 'ddhd1a', 'bmp4', 'lbh', 'ENSGWIG00000023227', 'manea', 'si:ch211-266g18.9', 'ENSGWIG00000024911', 'fabp7a', 'tspeara', 'ENSGWIG00000024924', 'ENSGWIG00000024925', 'ENSGWIG00000024926', 'elavl1a', 'slc1a8b', 'zgc:101744', 'diras1b', 'gng7', 'GADD45B', 'map1sb', 'shda', 'chico', 'gna11b', 'pdcd10b', 'si:dkey-56i24.1', 'ENSGWIG00000015966', 'slitrk3b', 'nrd1a', 'NPAS3', 'ENSGWIG00000019569', 'akap6', 'arhgap5', 'nubpl', 'ENSGWIG00000019728', 'ENSGWIG00000019729', 'sgpp1', 'wdr89', 'ppp2r5ea', 'kcnh5b', 'ENSGWIG00000023137', 'gphb5', 'ENSGWIG00000023139', 'ENSGWIG00000023140', 'ENSGWIG00000023141', 'ENSGWIG00000023143', 'ENSGWIG00000023145', 'ENSGWIG00000023163', 'ENSGWIG00000023257', 'ENSGWIG00000023261', 'btbd6b', 'klc1a', 'bag5', 'trmt61a', 'mark3a', 'ENSGWIG00000023644', 'exoc3l4', 'letm1', 'nsd2', 'asmt2', 'larp1b', 'ENSGWIG00000024049', 'ahsa1b', 'ENSGWIG00000024053', 'cdca4', 'si:dkey-177p2.6', 'ENSGWIG00000024059', 'fgfr3', 'ENSGWIG00000026264', 'ENSGWIG00000026265', 'ENSGWIG00000026266', 'foxi2', 'ENSGWIG00000026269', 'ENSGWIG00000026271', 'si:ch211-194c3.5', 'ky', 'slc4a11', 'rpia', 'eif2ak3', 'atrn', 'gfra4a', 'ap5s1', 'ENSGWIG00000015430', 'noto', 'emx1', 'rab11fip5a', 'ENSGWIG00000015754', 'ENSGWIG00000015756', 'ENSGWIG00000015757', 'loxl3b', 'ganc', 'znf106a', 'nenf', 'ENSGWIG00000016390', 'kcnk3b', 'MARC2', 'ENSGWIG00000016421', 'dctn1b', 'slc39a8', 'manba', 'zmp:0000000662', 'cpxm1a', 'hhipl2', 'vrk1', 'BCL11B', 'setd3', 'dio2', 'ENSGWIG00000022049', 'adck1', 'daam2', 'ENSGWIG00000023461', 'kif6', 'kcnk17', 'kcnk5a', 'rab3gap2', 'iars2', 'ENSGWIG00000024403', 'rtn4ip1', 'ENSGWIG00000024410', 'ccr6b', 'pxdn', 'MYT1L', 'ENSGWIG00000024561', 'eipr1', 'trappc12', 'adi1', 'marcksl1a', 'ENSGWIG00000024677', 'bsdc1', 'olig4', 'fndc5a', 'ENSGWIG00000024725', 'syf2', 'runx3', 'clic4', 'lck', 'HDAC1', 'tmem54a', 'ppie', 'ENSGWIG00000025246', 'hmgcl', 'gale', 'ENSGWIG00000025251', 'insm1b', 'ralgapa2', 'kiz', 'nkx2.4a', 'prkd3', 'qpct', 'ENSGWIG00000016720', 'SCAF8', 'tiam2a', 'tfb1m', 'CLDN20', 'filip1a', 'tmem30aa', 'cox7a2a', 'col12a1a', 'CD109', 'ENSGWIG00000017552', 'dlgap2a', 'arhgef10', 'kbtbd11', 'ENSGWIG00000017673', 'ZDHHC14', 'nt5c1ab', 'MYCL', 'ENSGWIG00000018283', 'ENSGWIG00000018286', 'khdrbs1a', 'ENSGWIG00000018383', 'KLHL28', 'ENSGWIG00000019238', 'PLCB2', 'ANKRD63', 'pak6a', 'zfyve19', 'bub1bb', 'spint1a', 'ENSGWIG00000019300', 'cep170b', 'akt1', 'zbtb42', 'siva1', 'adss1', 'INF2', 'dnajc17', 'ENSGWIG00000021345', 'gchfr', 'cyp1c2', 'cyp1c1', 'ENSGWIG00000021349', 'ENSGWIG00000021350', 'ENSGWIG00000021351', 'ENSGWIG00000021357', 'ENSGWIG00000021411', 'arhgap11a', 'GREM1', 'FMN1', 'ENSGWIG00000021654', 'ENSGWIG00000021673', 'chrm5a', 'ENSGWIG00000021676', 'acp7', 'ENSGWIG00000021722', 'ENSGWIG00000022667', 'ENSGWIG00000022670', 'ENSGWIG00000022671', 'MAX', 'SIPA1L1', 'ENSGWIG00000022811', 'ENSGWIG00000002740', 'ENSGWIG00000002774', 'RASGRP1', 'SYNDIG1L', 'ENSGWIG00000007064', 'AREL1', 'VASH1', 'rps6kl1', 'dlst', 'prox2', 'ESRRB', 'GPATCH2L', 'IFT43', 'tgfb3', 'ttll5', 'JDP2', 'ENSGWIG00000009350', 'si:dkey-1f12.3', 'isca2', 'aldh6a1', 'si:ch211-243o19.4', 'coq6', 'tmem63c', 'pgfb', 'ap5m1', 'exoc5', 'otx2b', 'ENSGWIG00000012980', 'tmem260', 'peli2', 'ktn1', 'msh4', 'aspg', 'kif26ab', 'ENSGWIG00000013628', 'asap2a', 'ENSGWIG00000015859', 'grhl1', 'ywhaqb', 'adam17a', 'iah1', 'rpap1', 'ltk', 'itpka', 'bahd1', 'rpusd2', 'eml1', 'evla', 'degs2', 'hif1aa', 'prkcha', 'slc38a6', 'mnat1', 'six4a', 'LRRC9', 'rtn1b', 'tdrd9', 'ENSGWIG00000007631', 'FGFRL1', 'ENSGWIG00000007655', 'si:ch1073-416d2.3', 'ddx24', 'UNC79', 'ENSGWIG00000008529', 'btbd7', 'itpk1b', 'plk4', 'hspa4l', 'chmp3', 'reep1', 'UBR1', 'mboat2a', 'ENSGWIG00000010416', 'id2a', 'rnf144aa', 'cmpk2', 'rsad2', 'sox11a', 'ENSGWIG00000014212', 'ENSGWIG00000014214', 'myo6b', 'impg1b', 'SNX14', 'cga', 'syncripl', 'znf292a', 'orc3', 'CNR1', 'GABRR1', 'tonsl', 'mtfr1l', 'ENSGWIG00000018522', 'znf593', 'rlf', 'zmpste24', 'rpl13a', 'ENSGWIG00000018572', 'ENSGWIG00000018633', 'DLK1', 'ENSGWIG00000019605', 'WDR25', 'ENSGWIG00000016019', 'soul3', 'tmem234', 'rcan3', 'grhl3', 'si:dkey-221l4.11', 'cnksr1', 'rps6ka1', 'ENSGWIG00000024080', 'si:ch1073-358c10.1', 'ythdf2', 'vgll2b', 'ctsl.1', 'ENSGWIG00000024120', 'qkia', 'PACRG', 'ENSGWIG00000024124', 'ptafr', 'ENSGWIG00000024145', 'ENSGWIG00000024152', 'ENSGWIG00000024156', 'traf3', 'srp54', 'ENSGWIG00000024163', 'ENSGWIG00000024164', 'ENSGWIG00000024166', 'lrrc57', 'ankrd6b', 'gabrr2b', 'ube2j1', 'ENSGWIG00000026624', 'ENSGWIG00000026626', 'slc5a6a', 'gopc', 'znf292b', 'cx28.8', 'ddx51', 'smim8', 'ENSGWIG00000012698', 'rad51', 'rmdn3', 'si:ch211-241j12.3', 'ephx1', 'srp9', 'enah', 'ENSGWIG00000012721', 'lbr', 'ENSGWIG00000012733', 'ENSGWIG00000012734', 'map1lc3c', 'mcm9', 'cx43', 'hsf2', 'nus1', 'psmc6', 'cgrrf1', 'ENSGWIG00000012831', 'ENSGWIG00000012833', 'katna1', 'dtnba', 'ENSGWIG00000014049', 'kif3ca', 'rab10', 'lats1', 'nup43', 'pcmt', 'lrp11', 'map3k5', 'ENSGWIG00000014405', 'tfap2b', 'tfap2d', 'hadhb', 'ENSGWIG00000018340', 'ENSGWIG00000018342', 'ENSGWIG00000018346', 'ENSGWIG00000018442', 'jag2b', 'ENSGWIG00000020132', 'ENSGWIG00000020134', 'ENSGWIG00000020136', 'heca', 'abracl', 'reps1', 'clu', 'ENSGWIG00000020188', 'scara3', 'ENSGWIG00000020190', 'ak7a', 'ENSGWIG00000020212', 'ENSGWIG00000020249', 'dlgap2b', 'ENSGWIG00000020253', 'ptprk', 'ENSGWIG00000025350', 'ENSGWIG00000025352', 'lama2', 'rgs6', 'ENSGWIG00000025481', 'ENSGWIG00000025484', 'sipa1l1', 'rab15', 'fntb', 'max', 'slc10a1', 'srsf5b', 'susd6', 'gpr176', 'ENSGWIG00000025504', 'thbs1b', 'katnbl1', 'emc7', 'aven', 'ENSGWIG00000003254', 'ENSGWIG00000003796', 'grem1a', 'ENSGWIG00000003806', 'ENSGWIG00000003832', 'ENSGWIG00000003911', 'serac1', 'gtf2h5', 'tulp4a', 'msgn1', 'ppil6', 'ENSGWIG00000004517', 'cep57l1', 'ENSGWIG00000004529', 'tmem121ab', 'slc24a4b', 'extl3', 'nt5c1bb', 'si:dkey-174m14.3', 'ENSGWIG00000005815', 'rngtt', 'cnr1', 'akirin2', 'ypel5', 'rhag', 'ENSGWIG00000008327', 'ENSGWIG00000008341', 'si:dkey-12h9.6', 'nrbp1', 'capn3b', 'ENSGWIG00000009642', 'churc1', 'elmsan1a', 'kcns3b', 'ttc32', 'laptm4a', 'ENSGWIG00000009666', 'pum2', 'ENSGWIG00000009763', 'hpcal1', 'kcnf1a', 'pdia6', 'ENSGWIG00000009777', 'ENSGWIG00000009781', 'rock2b', 'sobpb', 'ENSGWIG00000012935', 'ENSGWIG00000012937', 'ENSGWIG00000012950', 'mrap2b', 'sim1b', 'ENSGWIG00000012962', 'ENSGWIG00000012964', 'ENSGWIG00000012967', 'ENSGWIG00000012974', 'ENSGWIG00000012977', 'ENSGWIG00000012981', 'lin28b', 'prep', 'ENSGWIG00000013869', 'xkr5a', 'efhc1', 'tram2', 'ENSGWIG00000013948', 'mycn', 'smek1', 'erh', 'paplnb', 'PLEKHG1', 'tcp1', 'sod2', 'fndc1', 'otofa', 'ENSGWIG00000018670', 'nr2e1', 'afg1lb', 'foxo3b', 'armc2', 'sesn1', 'cmtr1', 'nhsl1b', 'acbd3', 'parp1', 'lin9', 'si:dkey-206f10.1', 'efr3bb', 'ENSGWIG00000021491', 'smyd2b', 'PTPN14', 'prox1a', 'rps6kc1', 'moxd1l', 'snx9b', 'si:ch211-203k16.3', 'kcnk2b', 'prkg3', 'ABHD1', 'ctsbb', 'fdft1', 'brms1lb', 'calm1b', 'psmc1a', 'kcnk13b', 'ches1', 'gtf2a1', 'tshr', 'CEP128', 'nrxn3b', 'snw1', 'ism2b', 'ccdc85cb', 'CCNK', 'im:7142702', 'ENSGWIG00000025413', 'agbl5', 'si:ch211-197l9.5', 'tbx18', 'ENSGWIG00000025422', 'ENSGWIG00000025423', 'irak1bp1', 'si:dkey-261l7.2', 'phip', 'ENSGWIG00000025652', 'clic5b', 'runx2b', 'opn8b', 'wdcp', 'mfsd2b', 'ubxn2a', 'atad2b', 'klhl29', 'trib2', 'ENSGWIG00000025709', 'napbb', 'gpcpd1', 'ENSGWIG00000025715', 'ENSGWIG00000025716', 'fermt1', 'bmp2b', 'PLCB1', 'PLCB4', 'LAMP5', 'pak7', 'ENSGWIG00000016211', 'ENSGWIG00000016212', 'ENSGWIG00000016213', 'ENSGWIG00000016214', 'ENSGWIG00000016216', 'ENSGWIG00000016218', 'evlb', 'YY1', 'lrfn5b', 'ALDH6A1', 'entpd5b', 'znf410', 'actr10', 'ENSGWIG00000016238', 'naa30', 'ENSGWIG00000016241', 'ENSGWIG00000016245', 'arf6a', 'prpf39', 'faua', 'fkbp3', 'MDGA2', 'ENSGWIG00000017895', 'ENSGWIG00000018373', 'RPS29', 'wdr20b', 'ppp2r5cb', 'hsp90aa1.2', 'ENSGWIG00000022244', 'slc25a47a', 'vrtn', 'dll4', 'chac1', 'ino80', 'rhov', 'zfyve1', 'ENSGWIG00000022574', 'stx7l', 'ccn2a', 'tmem244', 'tmem200a', 'ENSGWIG00000022610', 'si:ch73-18b11.1', 'ENSGWIG00000022795', 'ENSGWIG00000024308', 'ehd3', 'ENSGWIG00000024312', 'med23', 'ENSGWIG00000024346', 'ENSGWIG00000024347', 'nkain2', 'rnf146', 'ENSGWIG00000024369', 'soga3b', 'elovl4b', 'tent5ab', 'ENSGWIG00000024492', 'tmem63a', 'mrps18a', 'vegfab', 'naga', 'wdr26b', 'si:ch211-57i17.1', 'si:ch73-21k16.1', 'si:ch73-21k16.5', 'daam1b', 'ENSGWIG00000003561', 'si:ch211-212g7.6', 'abcd4', 'si:ch211-153j24.3', 'jdp2b', 'ENSGWIG00000005357', 'ENSGWIG00000005370', 'ENSGWIG00000005378', 'batf', 'flvcr2b', 'zc3h14', 'ENSGWIG00000005419', 'ptk2bb', 'chrna2b', 'ENSGWIG00000009557', 'nme6', 'MIR133B', 'snx17', 'znf513a', 'si:ch211-278j3.3', 'fndc4a', 'blk', 'ENSGWIG00000010630', 'sox7', 'pinx1', 'mtmr9', 'xkr6b', 'ccm2', 'lca5', 'fam167ab', 'sh3bgrl2', 'ENSGWIG00000010654', 'rps12', 'eya4', 'tcf21', 'tbpl1', 'SGK1', 'aldh8a1', 'hbs1l', 'myb', 'PDE7B', 'ENSGWIG00000015516', 'ENSGWIG00000015528', 'sec63', 'MTLN', 'lratd1', 'nbas', 'ddx1', 'itsn2a', 'si:dkey-119f1.1', 'tmem18', 'ENSGWIG00000023072', 'fut8b', 'gphnb', 'mpp5b', 'slc35b2', 'aida', 'taf1a', 'dusp10', 'hlx1', 'kcnk3a', 'ENSGWIG00000016278', 'EZR', 'ppp2r5a', 'dtl', 'ints7', 'lpgat1', 'rd3', 'ENSGWIG00000023598', 'slc30a1a', 'nek2', 'ENSGWIG00000023609', 'hnrnpub', 'cdc42bpab', 'fzd3a', 'fam49a', 'vsnl1a', 'sf3b6', 'tdrd6', 'pla2g7', 'ENSGWIG00000025947', 'flrt2', 'ENSGWIG00000025950', 'ENSGWIG00000025951', 'ENSGWIG00000025953', 'psma6b', 'nfkbiaa', 'ENSGWIG00000025957', 'ENSGWIG00000025961', 'ncbp2', 'ppp1r13bb', 'ENSGWIG00000025985', 'ENSGWIG00000025984', 'FIG4', 'gpr6', 'wasf1', 'cdc40', 'slc22a16', 'usta', 'sash1a', 'SAMD5', 'stxbp5a', 'ENSGWIG00000026195', 'grm1a', 'ENSGWIG00000026209', 'utrn', 'stx11b.2', 'tmem214', 'dpysl5b', 'dnmt3aa', 'ENSGWIG00000026484', 'ENSGWIG00000026536', 'fam89a', 'arv1', 'kcnk10a', 'stmn4', 'ENSGWIG00000003258', 'snap25a', 'ENSGWIG00000004182', 'disp1', 'mep1a.2', 'mep1a.1', 'ENSGWIG00000004335', 'ENSGWIG00000004336', 'ENSGWIG00000004351', 'tnfrsf21', 'opn5', 'cd2ap', 'ptchd4', 'cnih3', 'si:dkey-1j5.4'])

#import itertools
#import numpy as np
#from scipy.spatial.distance import jaccard
#import random

# Your 4 gene sets
#gene_sets = {
#    'A': set(hofrichteri),
#    'B': set(pigra),
#    'C': set(willdenowi),
#    'D': set(orientalis)
#}

#experiments = list(gene_sets.keys())

# Compute all pairwise Jaccard overlaps
#def jaccard_index(s1, s2):
#    return len(s1 & s2) / len(s1 | s2)

#pairwise_scores = {}
#for (i, j) in itertools.combinations(experiments, 2):
#    pairwise_scores[(i, j)] = jaccard_index(gene_sets[i], gene_sets[j])

# Define within-group and between-group pairs
#within = [('A', 'B'), ('C', 'D')]
#between = [p for p in pairwise_scores if p not in within]

# Observed test statistic
#within_mean = np.mean([pairwise_scores[p] for p in within])
#between_mean = np.mean([pairwise_scores[p] for p in between])
#T_obs = within_mean - between_mean

# Permutation test
#def permute_test(gene_sets, n_perms=10000):
#    experiments = list(gene_sets.keys())
#    T_null = []
#    for _ in range(n_perms):
#        random.shuffle(experiments)
#        group1 = experiments[:2]
#        group2 = experiments[2:]
#        within = [(group1[0], group1[1]), (group2[0], group2[1])]
#        between = [p for p in itertools.combinations(experiments, 2) if p not in within]
#        within_mean = np.mean([jaccard_index(gene_sets[i], gene_sets[j]) for (i,j) in within])
#        between_mean = np.mean([jaccard_index(gene_sets[i], gene_sets[j]) for (i,j) in between])
#        T_null.append(within_mean - between_mean)
#    return T_null

#T_null = permute_test(gene_sets)
#p_value = np.mean([t >= T_obs for t in T_null])

#p_value

### 2.2.2 GO Enrichments from AF scans

All GO analyses were conducted online at https://biit.cs.ut.ee/gprofiler/gost

##### Create background gene set for GO analysis

In [ ]:
#background_snpeff = extract_deduplicated_gene_from_snpeff_list(AF_df["gene_snpeff"], 
#                                                               "results/GO_analysis/AF_background_snpeff.txt")

<b>Overview GO Terms related to the 1850 genes obtained from fixed parallel differences</b>

In [ ]:
GO_AFs = pd.read_csv("results/GO_analysis/AF_scans/AF_unique_all_in_all_snpeff_two_comp_gProfiler_drerio_03-10-2023_15-49-30__intersections.csv")
GO_AFs_BP = GO_AFs[GO_AFs["source"] == "GO:BP"]
GO_AFs_BP

##### random shuffling of sets 

In order to test if the genes that were obtained from the scans above are simply driven by strong puryfing selection, we randomly subsample gene sets and make seperate GO analysis.

In [ ]:
#random_selections = []

#for i in range(1,10):
#    random_list_fixed_genes = ["fixed_pigra_genes", "fixed_hofrichteri_genes", 
#               "fixed_willdenowi_genes", "fixed_orientalis_genes"]
#    random_selection = random.sample(random_list_fixed_genes, len(random_list_fixed_genes))
#    set1 = random_selection[0]
#    set2 = random_selection[1]
#    set3 = random_selection[2]
#    set4 = random_selection[3]
    
#    run = f"(set({set1}) & set({set2})) - set({set3}) - set({set4})"
#    
#    if run.startswith("(set(fixed_hofrichteri_genes) & set(fixed_pigra_genes))"):
#        pass
#    elif run.startswith("(set(fixed_pigra_genes) & set(fixed_hofrichteri_genes))"):
#        pass
#    else:
#        random_selections.append(run)

#randomization_gene_lists = dict()
#for run in random_selections:
#    #with open(f"results/outliers/AF_outliers/random_two_comp/{run}.txt","w") as outfile:
#        gene_list = eval(run)
#        randomization_gene_lists[run] = gene_list
#        for i in gene_list:
#            outfile.write(i + "\n")

#for k,v in randomization_gene_lists.items():
#    print("{}: {} genes".format(k,len(v)))

<b>Based on the above created lists a GO analyses was conducted to test how many GO terms overlap in the groups.</b>

In [ ]:
random_overview_dict = {}

with open("results/GO_analysis/GO_ultimate_snpeff_random_comparison_two_comp.tsv", "r") as file:
    for line in file:
        if line.startswith("set"):
            pass
        else:
            line = line.rstrip()
            random_overview_dict[line.split("\t")[0]] = line.split("\t")[1:]
#print(random_overview_dict)
ultimate_GO_set = random_overview_dict["AF_unique_all_in_all_snpeff"] # see above AF_unique_all_in_all

random_GO_sets = []

for k,v in random_overview_dict.items():
    if k == "AF_unique_all_in_all_snpeff":
        pass
    else:
        for i in v:
            if i in random_GO_sets:
                pass
            elif i == "No enriched BP sets":
                pass
            else:
                random_GO_sets.append(i)


#random_GO_sets
            
fixed_genes_venn = {'GO terms convergent': set(ultimate_GO_set),
               'GO terms other combinations': set(random_GO_sets)}#

ax = venn(fixed_genes_venn)
ax.set_title('UNIQUE GO TERMS')

plt.savefig('results/GO_random_versus_convergent.svg', format='svg')

unique_GO_terms_ultimate = list(set(ultimate_GO_set) - set(random_GO_sets))


print(f"There are {len(unique_GO_terms_ultimate)} BP:GO terms unique the ultimate dataset.\nThese include:")
for i in unique_GO_terms_ultimate:
    print("\t"+i)

In [ ]:
GO_AFs_BP_subset_unique = GO_AFs_BP[GO_AFs_BP['term_name'].isin(unique_GO_terms_ultimate)]
print(f"From a total of {len(GO_AFs_BP)} GO terms associated with biological processes, after random shuffling {len(GO_AFs_BP_subset_unique)} GO Terms are unique for convergent changes.")
GO_AFs_BP_subset_unique

In [ ]:
# Select the desired columns and rename them
subset_df = GO_AFs_BP_subset_unique[['term_id', 'term_name', 'adjusted_p_value', 'intersections']].copy()
subset_df.rename(columns={
    'term_id': 'GO.ID',
    'term_name': 'Description',
    'adjusted_p_value': 'p.value',
    'intersections': 'Genes'
}, inplace=True)

# Copy the 'p.value' column and rename it to 'FDR'
subset_df['FDR'] = subset_df['p.value']

# Add a new column 'Phenotype' with the value +1 for all rows
subset_df['Phenotype'] = 1
subset_df.to_csv('results/GO_analysis/Enrichment_map/UNIQUE_GENES/GO_AFs_BP_subset_unique_modified_input.tsv', sep = "\t", index=False)

### 2.2.3 MK from AF scans

#### Subset AF dataframe to missense and synonymous sites

In [ ]:
AF_subset = AF_df[(AF_df["type"] == "synonymous_variant") | (AF_df["type"] == "missense_variant")]
AF_subset

#### Extract selection scores for genes

##### extract tables for each species combination

In [ ]:
warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)
willdenowi_hofrichteri_MK_gene, willdenowi_hofrichteri_MK_genome = extract_MK(AF_subset, "AF_willdenowi", "AF_hofrichteri") 
willdenowi_pigra_MK_gene, willdenowi_pigra_MK_genome = extract_MK(AF_subset, "AF_willdenowi", "AF_pigra")
willdenowi_orientalis_MK_gene, willdenowi_orientalis_MK_genome = extract_MK(AF_subset, "AF_willdenowi", "AF_orientalis")
pigra_hofrichteri_MK_gene, pigra_hofrichteri_MK_genome = extract_MK(AF_subset, "AF_pigra","AF_hofrichteri")
pigra_orientalis_MK_gene, pigra_orientalis_MK_genome = extract_MK(AF_subset, "AF_pigra","AF_orientalis")
orientalis_hofrichteri_MK_gene, orientalis_hofrichteri_MK_genome = extract_MK(AF_subset, "AF_orientalis","AF_hofrichteri")

In [ ]:
##### extract and plot genome wide estimates

#whole_genome_selections_slender_vs_stout = [willdenowi_hofrichteri_MK_genome, 
#                                            willdenowi_pigra_MK_genome,
#                                            orientalis_hofrichteri_MK_genome,
#                                            pigra_orientalis_MK_genome]
                           
                           
#whole_genome_selections_same_type = [willdenowi_orientalis_MK_genome,
#                                     pigra_hofrichteri_MK_genome]


# Sample data for list1 and list2
#list1 = whole_genome_selections_slender_vs_stout
#list2 = whole_genome_selections_same_type

# Create lists for x-axis and y-axis data for both lists and both keys
#list1_ni = [d['NI'] for d in list1]
#list1_alpha = [d['alpha'] for d in list1]
#list1_nitg = [d['NITG'] for d in list1]
#list2_ni = [d['NI'] for d in list2]
#list2_nitg = [d['NITG'] for d in list2]
#list2_alpha = [d['alpha'] for d in list2]

# Create jitter function
#def rand_jitter(arr):
#    stdev = .05*(max(arr)-min(arr))
#    return arr + np.random.randn(len(arr)) * stdev

# Create the plots for NI

# Calculate t-test and p-value for alpha
#tstat_NI, pval_NI = ttest_ind(list1_ni, list2_ni)

#fig, ax = plt.subplots()
#ax.scatter(rand_jitter(np.zeros(len(list1_ni))), list1_ni, c='blue', linewidth = 20,
#           s=100, alpha=0.5, edgecolors='none', label='slender vs. stout')
#ax.scatter(rand_jitter(np.ones(len(list2_ni))), list2_ni, c='red', linewidth = 20,
#           s=100, alpha=0.5, edgecolors='none', label='same ecotype')
#ax.set_xticks([0,1])
#ax.set_xticklabels(['slender vs. stout', 'same ecotype'])
#ax.set_xlabel('Category')
#ax.set_ylabel('NI')
#ax.legend()

# Add t-test and p-value to the plot
#ax.text(0, 1.75, "t-test p-value: {:.4f}".format(pval_NI))

#ax.set_xlim(-0.5, 1.5)
#ax.set_ylim(0, 2)

#plt.show()


# Create the plots for NItg

# Calculate t-test and p-value for alpha
#tstat_NITG, pval_NITG = ttest_ind(list1_nitg, list2_nitg)#

#fig, ax = plt.subplots()
#ax.scatter(rand_jitter(np.zeros(len(list1_nitg))), list1_nitg, c='blue', linewidth = 20,
#           s=100, alpha=0.5, edgecolors='none', label='slender vs. stout')
#ax.scatter(rand_jitter(np.ones(len(list2_nitg))), list2_nitg, c='red', linewidth = 20,
#           s=100, alpha=0.5, edgecolors='none', label='same ecotype')
#ax.set_xticks([0,1])
#ax.set_xticklabels(['slender vs. stout', 'same ecotype'])
#ax.set_xlabel('Category')
#ax.set_ylabel('NITG')
#ax.legend()

# Add t-test and p-value to the plot
#ax.text(0, 1.75, "t-test p-value: {:.4f}".format(pval_NITG))

#ax.set_xlim(-0.5, 1.5)
#ax.set_ylim(0, 2)

#plt.savefig('results/NITG_selection_two_comp.pdf')

#plt.show()

# Create the plots for alpha

# Calculate t-test and p-value for alpha
#tstat_alpha, pval_alpha = ttest_ind(list1_alpha, list2_alpha)

#fig, ax = plt.subplots()
#ax.scatter(rand_jitter(np.zeros(len(list1_alpha))), list1_alpha, c='blue', linewidth = 100,
#           s=100, alpha=0.5, edgecolors='none', label='slender vs. stout')
#ax.scatter(rand_jitter(np.ones(len(list2_alpha))), list2_alpha, c='red', linewidth = 100,
#           s=100, alpha=0.5, edgecolors='none', label='same ecotype')
#ax.set_xticks([0,1])
#ax.set_xticklabels(['slender vs. stout', 'same ecotype'])
#ax.set_xlabel('Category')
#ax.set_ylabel('alpha')
#ax.legend()

# Add t-test and p-value to the plot
#ax.text(0, 0.75, "t-test p-value: {:.4f}".format(pval_alpha))

# Adjust the limits of the plot
#ax.set_xlim(-0.5, 1.5)
#ax.set_ylim(-1, 1)

#plt.show()


##### Extract and plot gene wide estimates for the 1850 genes obtained from AF-scans

Excess divergence can come from drift. 
To test this we conducted a selection scan (DoS) and compared the 1850 genes to the background and found no signals of purifying selection [add medians of both DOS and p value and t-test pvalue]

In [ ]:
gene_selections_slender_vs_stout = [willdenowi_hofrichteri_MK_gene, 
                                            willdenowi_pigra_MK_gene,
                                            orientalis_hofrichteri_MK_gene,
                                            pigra_orientalis_MK_gene]
                           
gene_selections_slender_vs_stout_names = ["willdenowi vs. hofrichteri", "willdenowi vs. pigra",
                                            "orientalis vs. hofrichteri", "pigra vs. orientalis"]
                                          
                                    
gene_selections_same_type = [willdenowi_orientalis_MK_gene,
                                     pigra_hofrichteri_MK_gene]

#Extract NI from all genes that are in slender and stout (1850 genes).

print("There are {} genes unique for parallel trajectories.".format(len(AF_unique_all_in_all)))

In [ ]:
def plot_unique_vs_background(input_MK_gene_df, header, ax=None):
    selected_genes =  input_MK_gene_df[input_MK_gene_df['gene_snpeff'].isin(AF_unique_all_in_all)]
    all_genes = input_MK_gene_df
    
    # Perform t-test
    t_stat, p_value = stats.ttest_ind(selected_genes["DoS"], all_genes["DoS"])

    # Plot the histograms
    if ax is None:
        ax = plt.gca()
    selected_genes["DoS"].hist(bins=40, alpha=0.5, label=f'selected {len(AF_unique_all_in_all)} genes', density=True, ax=ax)
    all_genes["DoS"].hist(bins=40, alpha=0.5, label='all Genes', density=True, ax=ax)

    # Add axis labels and title
    ax.set_xlabel('DoS', fontsize=16,fontweight='bold')
    ax.set_ylabel('frequency', fontsize=16,fontweight='bold')
    # ax.set_title(f'DoS focal vs. all genes \n {header}')

    # Add legend
    ax.legend(loc='upper right')

    # Add p-value to the plot
    ax.text(ax.get_xlim()[0] + (ax.get_xlim()[1] - ax.get_xlim()[0]) * 0.01, ax.get_ylim()[1] * 0.97, f'p-value: {p_value:.4f}', ha='left', fontsize=14,va='center', color='black')
    
    ax.tick_params(axis='both', which='major', labelsize=14)
    # Remove gridlines
    ax.grid(False)

# Create a new figure and axis object
num_pairs = len(gene_selections_slender_vs_stout)
num_cols = 2
num_rows = (num_pairs + num_cols - 1) // num_cols  # Ceiling division
fig, axs = plt.subplots(num_rows, num_cols, figsize=(16, num_rows*6))

# Sort the pairs by header alphabetically
sorted_pairs = sorted(zip(gene_selections_slender_vs_stout, gene_selections_slender_vs_stout_names), key=lambda x: x[1])

# Iterate through the sorted list of datasets
for idx, (pair, header) in enumerate(sorted_pairs):
    # Determine the position for this pair of plots
    row = idx // num_cols
    col = idx % num_cols

    # Plot each dataset
    plot_unique_vs_background(pair, header, ax=axs[row, col])

    # Add label to each pair
    axs[row, col].text(0.00, 1.01, f'{chr(97+idx)}', transform=axs[row, col].transAxes, fontweight='bold', fontsize=16, ha='left', va='bottom')
    axs[row, col].text(0.40, 1.01, f'{header}', transform=axs[row, col].transAxes, fontsize=16, ha='left', va='bottom')

# Remove empty subplots
for idx in range(len(sorted_pairs), num_rows * num_cols):
    row = idx // num_cols
    col = idx % num_cols
    fig.delaxes(axs[row, col])

# Adjust layout to prevent overlapping
plt.tight_layout()

# Save the entire figure once
plt.savefig('results/DoSfocalVsall_1850genes_combined.pdf')

# Show the plot
plt.show()


## 2.3 Convergence score

### 2.3.1 Overview Convergence Score

#### load dataframe (already prepared before)

In [ ]:
############## BOTH PAIRS  ###################
#par_score_pair1 = pd.read_csv("parallel_score/AFpigVsAFwilAndAFhofVsAFori_parallel_score.tsv.gz", sep = "\t")
#par_score_pair2 = pd.read_csv("parallel_score/AFpigVsAFadrAndAFhofVsAFori_parallel_score.tsv.gz", sep = "\t")
#par_score_pair1["par_score_pair2"] = par_score_pair2["par_score_pair2"]
#par_score = par_score_pair1
#par_score = par_score.rename(columns = {'CHROM':'chrom', 'POS':'pos', 'type':'cat'})
#par_score['type'] = par_score['ANN'].str.split('|').str[1]
#par_score['gene'] = par_score['ANN'].str.split('|').str[3]
#par_score.to_csv("raw_data/parallel_score_renamed.tsv.gz",
#                 compression='gzip', sep = "\t", index=False,
#                columns = ["chrom", "pos","cat", "type",  "gene", "par_score_pair1", "par_score_pair2"])

In [ ]:
par_score = pd.read_csv("raw_data/parallel_score_renamed.tsv.gz", sep = "\t")
par_score.rename(columns = {'gene':'gene_snpeff'}, inplace = True)
par_score = par_score.sort_values(by=['chrom', 'pos'])

In [ ]:
total_N_variants = len(par_score)
print("There are {} variants in the dataset.".format(total_N_variants))

Without 0 values for plotting.
**Par_score_pair_1 = (AFpig - AFwil) * (AFhof - AFori)**

In [ ]:
par_score_wo0 = par_score[par_score["par_score_pair1"] != 0]
par_score_wo0 = par_score_wo0.sort_values(by=['chrom', 'pos'])
par_score_wo0

In [ ]:
without_0_variants = len(par_score_wo0)
without_0_variants_fraction =  (without_0_variants/total_N_variants)*100
print(f"From all {total_N_variants}, there are {without_0_variants} variants having values that are not 0 ({without_0_variants_fraction}%).")

In [ ]:
par_score_wo0_pos = par_score_wo0[par_score_wo0["par_score_pair1"] > 0]
par_score_wo0_neg = par_score_wo0[par_score_wo0["par_score_pair1"] < 0]

In [ ]:
print(f"There are {len(par_score_wo0_pos)} positive (convergent) and {len(par_score_wo0_neg)} negative (divergent) sites.")

#### Fixed convergent sites (par_score_pair1==1)

In [ ]:
par_score_wo0_fixed = par_score_wo0[par_score_wo0["par_score_pair1"] == 1]
par_score_wo0_fixed

##### binomial test for excess of positive scores (sites)

In [ ]:
result = binomtest(k=len(par_score_wo0_pos), n=len(par_score_wo0_pos)+len(par_score_wo0_neg), p=0.5, alternative='two-sided')
print(f"There is a significant excess of overall positive scores (p={result.pvalue}).")

#### high convergent/divergent scores

In [ ]:
par_score_wo0_pos_sign = par_score_wo0[par_score_wo0["par_score_pair1"] >= 0.25]
par_score_wo0_neg_sign = par_score_wo0[par_score_wo0["par_score_pair1"] <= -0.25]

In [ ]:
print(f"There are {len(par_score_wo0_pos_sign)} positive (convergent) and {len(par_score_wo0_neg_sign)} negative (divergent) sites.")

In [ ]:
result = binomtest(k=len(par_score_wo0_pos_sign), n=len(par_score_wo0_pos_sign)+len(par_score_wo0_neg_sign), p=0.5, alternative='two-sided')
print(f"There is a significant excess of overall positive scores (p={result.pvalue}).")

### 2.3.2 Enrichments of coding changes (Covergence Score)?

In [ ]:
#par_score_wo0_pos_sign_permutation_test = run_permutation_variants_test(par_score_wo0_pos_sign, par_score)
par_score_wo0_pos_sign_permutation_test

In [ ]:
import pandas as pd

# Count frequencies of each 'type' in the whole genome and outliers
all_counts = par_score['type'].value_counts()
pos_counts = par_score_wo0_pos_sign['type'].value_counts()
neg_counts = par_score_wo0_neg_sign['type'].value_counts()

# Get union of all unique types
all_types = set(all_counts.index) | set(pos_counts.index) | set(neg_counts.index)




In [ ]:
pos_results = enrichment_test_variants(pos_counts, all_counts, all_types)
neg_results = enrichment_test_variants(neg_counts, all_counts, all_types)

In [ ]:
pos_results

### 2.3.3 Manhatten plot all chromosomes

In [ ]:
# Get unique chromosomes
chromosomes = par_score_wo0['chrom'].unique()

# Calculate the overall minimum and maximum 'par_score_pair1' values
min_par_score = par_score_wo0['par_score_pair1'].min()
max_par_score = par_score_wo0['par_score_pair1'].max()

# Expand the y-axis limits to include -1 and 1
y_lower_limit = min(min_par_score, -1.1)
y_upper_limit = max(max_par_score, 1.1)

# Function to determine colors based on chromosome values
def determine_color(chrom_values):
    return ['lightgrey' if chrom == 24 else 'lightgrey' if chrom % 2 == 1 else 'grey' for chrom in chrom_values]

# Use GridSpec for the second plot
fig = plt.figure(figsize=(15, 6),facecolor='none')
gs = gridspec.GridSpec(1, len(chromosomes), width_ratios=par_score_wo0.groupby('chrom')['pos'].max(), wspace=0.04)

for i, chrom in enumerate(chromosomes):

    ax = plt.subplot(gs[i])
    chrom_data = par_score_wo0[par_score_wo0['chrom'] == chrom]
    
    # Draw dotted lines at y = -0.25 and y = 0.25
    ax.axhline(y=-0.25, color='black', linestyle=':', linewidth=1,zorder=1)
    ax.axhline(y=0.25, color='black', linestyle=':', linewidth=1,zorder=1)
    
    # Draw dashed lines at y = -1 and y = 1
    ax.axhline(y=-1, color='black', linestyle='--', linewidth=1,zorder=1)
    ax.axhline(y=1, color='black', linestyle='--', linewidth=1,zorder=1)
    
    # Highlight values >= 0.25 in green
    ax.scatter(
        chrom_data[chrom_data['par_score_pair1'] >= 0.25]['pos'],
        chrom_data[chrom_data['par_score_pair1'] >= 0.25]['par_score_pair1'],
        c='#398f55ff',
        edgecolor='#398f55ff',
        s=25,
        alpha=0.8,
        zorder=2
    )
    
    # Highlight values <= -0.25 in orange
    ax.scatter(
        chrom_data[chrom_data['par_score_pair1'] <= -0.25]['pos'],
        chrom_data[chrom_data['par_score_pair1'] <= -0.25]['par_score_pair1'],
        c='#e7ae3dff',
        edgecolor='#e7ae3dff',
        s=25,
        alpha=0.8,
        zorder=2
    )
    
    
    # Highlight values between -0.25 and 0.25 with different colors based on chromosomes
    mask = (chrom_data['par_score_pair1'] > -0.25) & (chrom_data['par_score_pair1'] < 0.25)
    colors_within_range = determine_color(chrom_data[mask]['chrom'])
    
    ax.scatter(
        chrom_data[mask]['pos'],
        chrom_data[mask]['par_score_pair1'],
        c=colors_within_range,
        edgecolor=colors_within_range,
        s=25,
        alpha=0.5,
        zorder=2
    )
    
    ax.set_xlabel(f'{chrom}', fontsize=14, fontweight='bold')
    ax.grid(False)
    ax.set_xlim(0, par_score_wo0[par_score_wo0['chrom'] == chrom]['pos'].max())  # Set x-axis limit
    
    ax.set_ylim(y_lower_limit, y_upper_limit)  # Adjusted y-axis limits
    
    if i == 0:
        ax.set_ylabel('parallel score', fontsize=17, fontweight='bold')
        ax.tick_params(labelsize=14)
    else:
        ax.set_yticks([])
        ax.spines['left'].set_visible(False)
    
    # Adjust margins to ensure data points are fully visible
    ax.margins(3)  # You can adjust the value to leave more or less space around the plot
    
    
    ax.spines['top'].set_visible(False)  # Hide top spine
    ax.spines['right'].set_visible(False)  # Hide right spine
    ax.spines['bottom'].set_visible(False)
    ax.set_xticklabels([])
    ax.set_xticks([])
    

#plt.savefig('parallel_score/par_score_pair1.svg', format='svg')
#plt.show()


plt.savefig('results/par_score_pair1.png', format='png', dpi=300, transparent=True)  # Adjust DPI as needed
plt.show()




#### Manhatten plot chromosomes 15 and details around adam12

In [ ]:
# Filter for chromosome 15 data
chromosome_of_interest = 15
par_score_wo0_chr15 = par_score_wo0[par_score_wo0['chrom'] == chromosome_of_interest]

# Calculate the overall minimum and maximum 'par_score_pair1' values for chromosome 15 only
min_par_score = par_score_wo0_chr15['par_score_pair1'].min()
max_par_score = par_score_wo0_chr15['par_score_pair1'].max()

# Expand the y-axis limits to include -1 and 1
y_lower_limit = min(min_par_score, -1.1)
y_upper_limit = max(max_par_score, 1.1)

# Function to determine colors based on chromosome values
def determine_color(chrom_values):
    return ['lightgrey' if chrom == 24 else 'lightgrey' if chrom % 2 == 1 else 'grey' for chrom in chrom_values]

# Create the plot for chromosome 15
fig, ax = plt.subplots(figsize=(10, 4), facecolor='none')

# Draw dotted lines at y = -0.25 and y = 0.25
ax.axhline(y=-0.25, color='black', linestyle=':', linewidth=1, zorder=1)
ax.axhline(y=0.25, color='black', linestyle=':', linewidth=1, zorder=1)

# Draw dashed lines at y = -1 and y = 1
ax.axhline(y=-1, color='black', linestyle='--', linewidth=1, zorder=1)
ax.axhline(y=1, color='black', linestyle='--', linewidth=1, zorder=1)

# Highlight values >= 0.25 in green
ax.scatter(
    par_score_wo0_chr15[par_score_wo0_chr15['par_score_pair1'] >= 0.25]['pos'],
    par_score_wo0_chr15[par_score_wo0_chr15['par_score_pair1'] >= 0.25]['par_score_pair1'],
    c='#398f55ff',
    edgecolor='#398f55ff',
    s=25,
    alpha=0.8,
    zorder=2
)

# Highlight values <= -0.25 in orange
ax.scatter(
    par_score_wo0_chr15[par_score_wo0_chr15['par_score_pair1'] <= -0.25]['pos'],
    par_score_wo0_chr15[par_score_wo0_chr15['par_score_pair1'] <= -0.25]['par_score_pair1'],
    c='#e7ae3dff',
    edgecolor='#e7ae3dff',
    s=25,
    alpha=0.8,
    zorder=2
)

# Highlight values between -0.25 and 0.25 with different colors based on chromosomes
mask = (par_score_wo0_chr15['par_score_pair1'] > -0.25) & (par_score_wo0_chr15['par_score_pair1'] < 0.25)
colors_within_range = determine_color(par_score_wo0_chr15[mask]['chrom'])

ax.scatter(
    par_score_wo0_chr15[mask]['pos'],
    par_score_wo0_chr15[mask]['par_score_pair1'],
    c=colors_within_range,
    edgecolor=colors_within_range,
    s=25,
    alpha=0.5,
    zorder=2
)

ax.set_xlabel(f'chromosome {chromosome_of_interest}', fontname='Arial', fontsize=12, fontweight='bold')
ax.set_ylabel('parallel score', fontname='Arial', fontsize=12, fontweight='bold')

ax.grid(False)
ax.set_xlim(-100, par_score_wo0_chr15['pos'].max()+100)  # Set x-axis limit
ax.set_ylim(y_lower_limit, y_upper_limit)  # Adjusted y-axis limits
ax.tick_params(labelsize=14)

#ax.spines['top'].set_visible(True)  # Hide top spine
#ax.spines['right'].set_visible(True)  # Hide right spine
#ax.spines['bottom'].set_visible(True)
#ax.spines['left'].set_visible(True)


plt.savefig('results/par_score_pair1_chrom_15.png', format='png', dpi=300, transparent=True)  # Adjust DPI as needed
plt.show()



In [ ]:
# Filter for chromosome 15 data and within specified x-axis limits
chromosome_of_interest = 15
x_min = 10900000
x_max = 11111000
par_score_wo0_chr15_focal = par_score_wo0[(par_score_wo0['chrom'] == chromosome_of_interest) & (par_score_wo0['pos'] >= x_min) & (par_score_wo0['pos'] <= x_max)]

# If there are no data points within the specified x-axis limits, add a data point at x=0
if chrom_data.empty:
    par_score_wo0_chr15_focal = pd.DataFrame({'chrom': [chromosome_of_interest], 'pos': [0], 'par_score_pair1': [0]})

# Calculate the overall minimum and maximum 'par_score_pair1' values for chromosome 15 only within the specified x-axis limits
min_par_score = par_score_wo0_chr15_focal['par_score_pair1'].min()
max_par_score = par_score_wo0_chr15_focal['par_score_pair1'].max()

# Expand the y-axis limits to include -1 and 1
y_lower_limit = min(min_par_score, -1.1)
y_upper_limit = max(max_par_score, 1.1)

# Function to determine colors based on chromosome values
def determine_color(chrom_values):
    return ['lightgrey' if chrom == 24 else 'lightgrey' if chrom % 2 == 1 else 'grey' for chrom in chrom_values]

# Create the plot for chromosome 15
fig, ax = plt.subplots(figsize=(10, 4), facecolor='none')

# Draw dotted lines at y = -0.25 and y = 0.25
ax.axhline(y=-0.25, color='black', linestyle=':', linewidth=1, zorder=1)
ax.axhline(y=0.25, color='black', linestyle=':', linewidth=1, zorder=1)

# Draw dashed lines at y = -1 and y = 1
ax.axhline(y=-1, color='black', linestyle='--', linewidth=1, zorder=1)
ax.axhline(y=1, color='black', linestyle='--', linewidth=1, zorder=1)

# Highlight values >= 0.25 in green
ax.scatter(
    par_score_wo0_chr15_focal[par_score_wo0_chr15_focal['par_score_pair1'] >= 0.25]['pos'],
    par_score_wo0_chr15_focal[par_score_wo0_chr15_focal['par_score_pair1'] >= 0.25]['par_score_pair1'],
    c='#398f55ff',
    edgecolor='#398f55ff',
    s=25,
    alpha=0.8,
    zorder=2
)

# Highlight values <= -0.25 in orange
ax.scatter(
    par_score_wo0_chr15_focal[par_score_wo0_chr15_focal['par_score_pair1'] <= -0.25]['pos'],
    par_score_wo0_chr15_focal[par_score_wo0_chr15_focal['par_score_pair1'] <= -0.25]['par_score_pair1'],
    c='#e7ae3dff',
    edgecolor='#e7ae3dff',
    s=25,
    alpha=0.8,
    zorder=2
)

# Highlight values between -0.25 and 0.25 with different colors based on chromosomes
mask = (par_score_wo0_chr15_focal['par_score_pair1'] > -0.25) & (par_score_wo0_chr15_focal['par_score_pair1'] < 0.25)
colors_within_range = determine_color(par_score_wo0_chr15_focal[mask]['chrom'])

ax.scatter(
    par_score_wo0_chr15_focal[mask]['pos'],
    par_score_wo0_chr15_focal[mask]['par_score_pair1'],
    c=colors_within_range,
    edgecolor=colors_within_range,
    s=25,
    alpha=0.5,
    zorder=2
)

ax.set_xlabel(f'chromosome {chromosome_of_interest}', fontname='Arial', fontsize=12, fontweight='bold')
ax.set_ylabel('parallel score', fontname='Arial', fontsize=12, fontweight='bold')

ax.grid(False)
ax.set_xlim(x_min, x_max)  # Set x-axis limit between specified values
ax.set_ylim(y_lower_limit, y_upper_limit)  # Adjusted y-axis limits
ax.tick_params(labelsize=14)

plt.savefig('results/par_score_pair1_chrom_15_focal.png', format='png', dpi=300, transparent=True)  # Adjust DPI as needed
plt.show()


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

par_score_wo0_chr15_focal_sign = par_score_wo0_chr15_focal[par_score_wo0_chr15_focal["par_score_pair1"] >= 0.25]

# Create a copy to avoid SettingWithCopyWarning
par_score_wo0_chr15_focal_sign = par_score_wo0_chr15_focal_sign.copy()

par_score_wo0_chr15_focal_sign['gene_snpeff'] = par_score_wo0_chr15_focal_sign['gene_snpeff'].apply(lambda x: '-'.join([enseble_ids_to_genes.get(val, val) for val in x.split('-')]))

# Count instances for each unique combination of 'type' and 'gene_snpeff'
par_score_wo0_chr15_focal_sign_counts = par_score_wo0_chr15_focal_sign.groupby(['type', 'gene_snpeff']).size().reset_index(name='count')
par_score_wo0_chr15_focal_sign_counts = par_score_wo0_chr15_focal_sign_counts.sort_values(by="gene_snpeff")

sns.set(style="whitegrid")

plt.figure(figsize=(9, 6))
sns.barplot(x='count', y='gene_snpeff', hue='type', data=par_score_wo0_chr15_focal_sign_counts, dodge=False)

# Rotate x-axis labels for better readability
#plt.xticks(rotation=45, ha='right')
plt.xticks(fontsize=14)
plt.xlabel('count', fontsize=16, fontweight="bold")

# Set labels and title#
#plt.ylabel('gene', rotation=45, fontsize=14)
plt.yticks(rotation=23, fontsize=14)

# Show legend
plt.legend()

plt.savefig('results/ADAM12_variants_counts.svg')#, format='png', dpi=300, transparent=True)  # Adjust DPI as needed


# Show the plot
plt.show()

par_score_wo0_chr15_focal_sign_counts


I checked on ENSEMBL and found out that the missense variant is on chrom 15 <b>exon 3 of adam12 (15:10,988,711–10,988,522; ENSGWIE00000201663)</b>

Extract fasta sequence file from that region.

In [ ]:
#### ADAM12 many missense mutations?
#AF_df
#Adam12_df_AF = AF_df[(AF_df["CHROM"]== 15) & 
#             (AF_df["POS"] >= 10900000) & 
#            (AF_df["POS"] <= 11111000)]
#Adam12_df_AF

#Adam12_df_AF = AF_df[(AF_df["gene_snpeff"] == "adam12") & 
#                     (AF_df["type"].isin(["missense_variant","synonymous_variant"]))] #| 
                     #(AF_df["type"] == "synonymous_variant")]
#Adam12_df_AF

#AF_df_missense = AF_df[AF_df["type"] == "missense_variant"]
#AF_df_missense_counts = AF_df_missense['gene_snpeff'].value_counts()
#AF_df_missense_counts.to_excel("results/missense_counts.xlsx")

In [ ]:
chrom15_vcf = read_vcf("raw_data/all_sites.tif1.sft1.variants.whatshap.shapeit4.annotate.pass.snps.indels.biallelic.chr15.annSNPEFF.vcf.gz")
chrom15_vcf = chrom15_vcf.sort_values(by="POS")

In [ ]:
## EXON3 ADAM 12

end = 10988711
start = 10988522


#ref_chr15_exon3_adam12 = ">ref_chr15_exon3_adam12_10988711_10988522\nAGGCTTCATCTCACTTGAAAACAAAACTTTCATCATTGAACCAGTCCCCAGCAATGACGCTGGTGCTCATTTCATCTACCGAGCAGAAGAACTCCGGCTGACCCCGGGAGCCTGTGGCCATGGCTTCAACATGTCCTCAGTTGCCCCTGAAAGTCACATTAAAAGCCCCTTCCAGTCCTTCCACACAAGG"

chrom15_vcf_exon3_adam12 = chrom15_vcf[(chrom15_vcf["POS"] >= start) & 
                                (chrom15_vcf["POS"] <= end)]


chrom15_vcf_exon3_adam12.to_excel("results/adam12exon3.xlsx")

In [ ]:
chrom15_vcf_exon3_adam12_genotype = chrom15_vcf_exon3_adam12.iloc[:, [0,1,3,4]]

# Split columns 6 to 10 and keep only the first entry
for col_name in chrom15_vcf_exon3_adam12.columns[9:]:
    chrom15_vcf_exon3_adam12_genotype[col_name] = chrom15_vcf_exon3_adam12[col_name].str.split(':', expand=True)[0]


In [ ]:
# Replace values in columns 5 onwards
for col in chrom15_vcf_exon3_adam12_genotype.columns[4:]:
    chrom15_vcf_exon3_adam12_genotype[col] = chrom15_vcf_exon3_adam12_genotype.apply(lambda x: replace_values(x[col], x['REF'], x['ALT']), axis=1)


# Replace values in the DataFrame using the dictionary
for col in chrom15_vcf_exon3_adam12_genotype.columns[4:]:  # Start from the 5th column
    chrom15_vcf_exon3_adam12_genotype[col] = chrom15_vcf_exon3_adam12_genotype[col].apply(lambda x: replace_values_iupac(x))

chrom15_vcf_exon3_adam12_genotype

In [ ]:
ref_chr15_exon3_adam12_r = ">ref_chr15_exon3_adam12_10988711_10988522\nAGGCTTCATCTCACTTGAAAACAAAACTTTCATCATTGAACCAGTCCCCAGCAATGACGCTGGTGCTCATTTCATCTACCGAGCAGAAGAACTCCGGCTGACCCCGGGAGCCTGTGGCCATGGCTTCAACATGTCCTCAGTTGCCCCTGAAAGTCACATTAAAAGCCCCTTCCAGTCCTTCCACACAAGG"
ref_chr15_exon3_adam12_f = ">ref_chr15_exon3_adam12_10988522_10988711\nCCTTGTGTGGAAGGACTGGAAGGGGCTTTTAATGTGACTTTCAGGGGCAACTGAGGACATGTTGAAGCCATGGCCACAGGCTCCCGGGGTCAGCCGGAGTTCTTCTGCTCGGTAGATGAAATGAGCACCAGCGTCATTGCTGGGGACTGGTTCAATGATGAAAGTTTTGTTTTCAAGTGAGATGAAGCCT"
# Initialize the outer dictionary
column_dict = {}

# Iterate over the columns starting from the 5th column
for col in chrom15_vcf_exon3_adam12_genotype.columns[4:]:
    # Initialize the inner dictionary for the current column
    inner_dict = {}
    
    # Iterate over the rows and populate the inner dictionary
    for index, row in chrom15_vcf_exon3_adam12_genotype.iterrows():
        inner_dict[row['POS']] = row[col]
    
    # Add the inner dictionary to the outer dictionary with the column name as the key
    column_dict[col] = inner_dict

    
# Given fasta sequence
fasta_sequence = ref_chr15_exon3_adam12_f


# Create a fasta file
with open("results/chr15_exon3_adam12_for.fasta", "w") as fasta_file:
    # Write the original fasta header and sequence to the file
    fasta_file.write(fasta_sequence + "\n")
    
    # Iterate over the individuals in the column dictionary
    for individual, positions in column_dict.items():
        # Initialize a new sequence for the current individual
        individual_sequence = fasta_sequence[1:].replace("\n", "")
        individual_sequence = fasta_sequence[len(ref_chr15_exon3_adam12_f.split("\n")[0]):].replace("\n", "")
        #print(individual_sequence)
        
        # Iterate over the positions and replace the values in the sequence
        for position, value in positions.items():
            # Calculate the index in the sequence to replace the value
            index = position - 10988522
            
            # Replace the value in the sequence
            individual_sequence = individual_sequence[:index] + value + individual_sequence[index+1:]
        
        # Write the individual's fasta header and sequence to the file
        fasta_file.write(">" + individual + "\n")
        fasta_file.write(individual_sequence + "\n")

I TOOK THE ALIGNMENT AND MADE AA ALIGNEMNTS FOR SINGLE SPECIES AS INPUT FOR https://weblogo.threeplusone.com/ in AliView.

#### Sequencing depth

In [ ]:
#snps_chrom15_vcf = read_vcf("raw_data/all_sites.tif1.sft1.variants.whatshap.shapeit4.annotate.pass.snps.biallelic.chr15.annSNPEFF.vcf.gz")
#indels_chrom15_vcf = read_vcf("raw_data/all_sites.tif1.sft1.pass.indels.biallelic.chr15.annSNPEFf.vcf.gz")
#chrom15_vcf = pd.concat([snps_chrom15_vcf, indels_chrom15_vcf])
#chrom15_vcf = read_vcf("raw_data/all_sites.tif1.sft1.variants.whatshap.shapeit4.annotate.pass.snps.indels.biallelic.chr15.annSNPEFF.vcf.gz")
#chrom15_vcf = chrom15_vcf.sort_values(by="POS")

In [ ]:
chrom15_vcf_depth = pd.DataFrame()

for col_name in chrom15_vcf.columns[9:]:
    chrom15_vcf_depth[col_name] = chrom15_vcf[col_name].apply(extract_depth)

chrom15_vcf_depth = chrom15_vcf_depth.apply(pd.to_numeric, errors='coerce')

chrom15_vcf_depth_genome_depth_dict = {col: chrom15_vcf_depth[col].mean() for col in chrom15_vcf_depth.columns}

In [ ]:
chrom15_vcf_focal = chrom15_vcf[(chrom15_vcf["POS"] >= 10900000) & 
                                (chrom15_vcf["POS"] <= 11111000)]

chrom15_vcf_focal_depth = pd.DataFrame()

for col_name in chrom15_vcf.columns[9:]:
    chrom15_vcf_focal_depth[col_name] = chrom15_vcf_focal[col_name].apply(extract_depth)

chrom15_vcf_focal_depth = chrom15_vcf_focal_depth.apply(pd.to_numeric, errors='coerce')

chrom15_vcf_depth_focal_dict = {col: chrom15_vcf_focal_depth[col].mean() for col in chrom15_vcf_focal_depth.columns}
#chrom15_vcf_depth_focal_dict

In [ ]:


# Combine data into a DataFrame
df1 = pd.DataFrame(list(chrom15_vcf_depth_genome_depth_dict.items()), columns=['sample', 'genome'])
df2 = pd.DataFrame(list(chrom15_vcf_depth_focal_dict.values()), columns=['focal'])
df = pd.concat([df1, df2], axis=1)

t_stat, p_value = ttest_ind(df['genome'], df['focal'])

if p_value > 0.05:
    print(f"There is no significant difference between the mean individual genome and focal depth (p={p_value}, t-statistic={t_stat}).")
else:
    print(f"There is a significant difference between the mean individual genome and focal depth (p={p_value}, t-statistic={t_stat}).")

print("Here's a comparison of the species-specific values...")

# Extract species names from sample names
df['species'] = df['sample'].str.split('_').str[0]

df['genome'] = pd.to_numeric(df['genome'], errors='coerce')
df['focal'] = pd.to_numeric(df['focal'], errors='coerce')

# Melt the DataFrame to long format for easy plotting
df_melted = df.melt(id_vars=['species'], value_vars=['genome', 'focal'], var_name='variable', value_name='value')

# Create side-by-side boxplots with faceting by 'species'
plt.figure(figsize=(6, 2))
#ax = sns.boxplot(x='species', y='value', hue='variable', data=df_melted, palette={'genome': 'skyblue', 'focal': 'lightcoral'})
ax = sns.boxplot(x='species', y='value', hue='variable', data=df_melted, palette={'genome': 'white', 'focal': '#ffa500ff'}, linewidth=0.5)  # Adjust linewidth as needed



# Set labels and title
plt.xlabel('')
plt.ylabel('Depth')

# Show legend
plt.legend(title='Variable', bbox_to_anchor=(1.05, 1), loc='upper left')



# Set y-axis ticks
ax.yaxis.set_ticks(np.arange(0, max(df['genome'].max(), df['focal'].max()) + 1, 5))

# Perform t-test for each species
for species in df['species'].unique():
    genome_values = df[df['species'] == species]['genome']
    focal_values = df[df['species'] == species]['focal']
    t_stat, p_value = ttest_ind(genome_values, focal_values)
    
    # Add p-value annotation to the plot
    ax.annotate(f'p = {p_value:.4f}', xy=(df['species'].unique().tolist().index(species), max(df['genome'].max(), df['focal'].max())), ha='center', va='bottom')

plt.savefig('results/Depth_focalVSgenome.pdf')  # Adjust DPI as needed

# Show the plot
plt.show()


#### Are more heterozygotes in the focal region? 

##### Genotype plots

In [ ]:
replacement_mapping = {
    '0|0': 0.0,
    '0/0': 0.0,
    '1|1': 1.0,
    '1/1':1.0,
    '0/1': 0.5,
    '0|1': 0.5,
    '1/0':0.5,
    '1|0': 0.5,
    './.': np.nan
}

In [ ]:
genotype_df_chr15 = chrom15_vcf.iloc[:, :2]

# Split columns 6 to 10 and keep only the first entry
for col_name in chrom15_vcf.columns[9:]:
    genotype_df_chr15[col_name] = chrom15_vcf[col_name].str.split(':', expand=True)[0]

genotype_df_chr15_floats = genotype_df_chr15.replace(replacement_mapping) # floats for plotting
genotype_df_chr15["POS"] = genotype_df_chr15["POS"].astype(int)
genotype_df_chr15_floats

In [ ]:
# legend

x = 0.1
y1 = 1.049
y2= 1.05
color_list = ["lightblue","tomato","orange"]


# Add custom legend annotations
legend_annotations = [
    dict(
        x=x+0.05,
        y=y1,
        xref='paper',
        yref='paper',
        text='ref_hom',
        showarrow=False,
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color='black'#color_list[0]
        )
    ),
    dict(
        x=x+0.25,
        y=y1,
        xref='paper',
        yref='paper',
        text='het',
        showarrow=False,
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color='black'#color_list[1]
        )
    ),
    dict(
        x=x+0.45,
        y=y1,
        xref='paper',
        yref='paper',
        text='alt_hom',
        showarrow=False,
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color='black' #color_list[2]
        )
    )
]


# Add rectangles to legend
for i, color in enumerate(color_list):
    legend_annotations.append(
        dict(
            x=x + i * 0.18,
            y=y2,
            xref='paper',
            yref='paper',
            xanchor='left',
            yanchor='top',
            text='',
            showarrow=False,
            bgcolor=color,
            width=30,
            height=20,
            bordercolor='black',  # Add border color
            borderwidth=1
        )
    )

    
legend_annotations.append(
        dict(
            x=x * 6.8,
            y=y2,
            xref='paper',
            yref='paper',
            xanchor='left',
            yanchor='top',
            text='',
            showarrow=False,
            bgcolor="white",
            width=30,
            height=20,
            bordercolor='black',  # Add border color
            borderwidth=1
        ))
legend_annotations.append(
        dict(
        x=x+0.71,
        y=y1,
        xref='paper',
        yref='paper',
        text='missing',
        showarrow=False,
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color='black' #color_list[2]
        ))
)


genotype_df_chr15_floats = order_columns_by_prefix(genotype_df_chr15_floats, ['adriatica', 'orientalis', 'willdenowi', 'hofrichteri', 'pigra'])
genotype_df_chr15_floats

In [ ]:
heatmap_data = genotype_df_chr15_floats.iloc[:, 0:58]
y_labels = heatmap_data.columns.to_list()
x_labels = genotype_df_chr15_floats['POS'].to_list()


fig = make_subplots()


fig = px.imshow(heatmap_data.T, # transpose
                labels=dict(x="position", y="individual", color="genotype"),
                x=x_labels,
                y=y_labels,
                color_continuous_scale=color_list
               )


fig.update_layout(height=1000, width=1000)


# Update figure layout with legend annotations
fig.update_layout(annotations=legend_annotations)

# Remove color scale bar
fig.update_layout(coloraxis_showscale=False, plot_bgcolor='white')


fig.update_yaxes(categoryorder="array", categoryarray=y_labels, tickangle=0)


pio.write_image(fig, 'results/genotype_plot_chr15.pdf')


# Show the plot
fig.show()

In [ ]:
color_list = ["lightblue","tomato","orange"]

x_min = 10900000
x_max = 11111000

genotype_df_chr15_floats_focal = genotype_df_chr15_floats[(genotype_df_chr15_floats["POS"] >= x_min) &
                                                         (genotype_df_chr15_floats["POS"] <= x_max)]
#genotype_df_chr15_floats_focal



heatmap_data = genotype_df_chr15_floats_focal.iloc[:, 0:58]
y_labels = heatmap_data.columns.to_list()
x_labels = genotype_df_chr15_floats_focal['POS'].to_list()


fig = make_subplots()

fig = px.imshow(heatmap_data.T, # transpose
                labels=dict(x="position", y="individual", color="genotype"),
                x=x_labels,
                y=y_labels,
                color_continuous_scale=color_list

               )
fig.update_layout(height = 1000, width = 1000)


# Remove color scale bar
fig.update_layout(coloraxis_showscale=False, plot_bgcolor='white')


# Update figure layout with legend annotations
fig.update_layout(annotations=legend_annotations)


fig.update_yaxes(categoryorder="array", categoryarray=y_labels,tickangle=0)

pio.write_image(fig, 'results/genotype_plot_chr15_focal.pdf')

fig.show()




#### only sigificant >= 0.25 genotype plot

In [ ]:
# Merging the data frames on the common column 'POS'
par_score_wo0_chr15_focal_sign = par_score_wo0_chr15_focal_sign.rename(columns={'pos': 'POS'})
genotype_df_chr15_floats_focal_sign = pd.merge(par_score_wo0_chr15_focal_sign, genotype_df_chr15_floats_focal, on='POS', how='inner')
#genotype_df_chr15_floats_focal_sign

In [ ]:
color_list = ["lightblue","tomato","orange"]

heatmap_data = genotype_df_chr15_floats_focal_sign.iloc[:, 7:65]
y_labels = heatmap_data.columns.to_list()
x_labels = genotype_df_chr15_floats_focal_sign['POS'].to_list()

# Create a numerical sequence for x-values
x_values = np.arange(len(x_labels))

fig = make_subplots()

fig = px.imshow(heatmap_data.T,  # transpose
                labels=dict(x="position", y="individual", color="genotype"),
                x=x_values,  # Use numerical sequence for x-values
                y=y_labels,
                color_continuous_scale=color_list
               )

fig.update_layout(height=1000, width=3000)

# Remove color scale bar
fig.update_layout(coloraxis_showscale=False, plot_bgcolor='white')


fig.update_yaxes(categoryorder="array", categoryarray=y_labels, tickangle=0)


# Customize x-axis ticks
fig.update_xaxes(tickvals=x_values, ticktext=x_labels)

# Update figure layout with legend annotations
fig.update_layout(annotations=legend_annotations)

# Save the plot as a PDF
pio.write_image(fig, 'results/genotype_plot_sign_focal_chr15.svg')

# Show the plot
fig.show()

## CODING AND NON-CODING Paralell Evolution

### Tests for positive selection

In [ ]:
AF_subset = AF_df[(AF_df["type"] == "synonymous_variant") | (AF_df["type"] == "missense_variant")]
#AF_subset

#### Extract selection stats for each species combination

In [ ]:
willdenowi_hofrichteri_MK_gene, willdenowi_hofrichteri_MK_genome = extract_MK(AF_subset, "AF_willdenowi", "AF_hofrichteri") 
willdenowi_pigra_MK_gene, willdenowi_pigra_MK_genome = extract_MK(AF_subset, "AF_willdenowi", "AF_pigra")
willdenowi_adriatica_MK_gene, willdenowi_adriatica_MK_genome  = extract_MK(AF_subset, "AF_willdenowi", "AF_adriatica")
willdenowi_orientalis_MK_gene, willdenowi_orientalis_MK_genome = extract_MK(AF_subset, "AF_willdenowi", "AF_orientalis")

pigra_hofrichteri_MK_gene, pigra_hofrichteri_MK_genome = extract_MK(AF_subset, "AF_pigra","AF_hofrichteri")
pigra_adriatica_MK_gene, pigra_adriatica_MK_genome = extract_MK(AF_subset, "AF_pigra","AF_adriatica")
pigra_orientalis_MK_gene, pigra_orientalis_MK_genome = extract_MK(AF_subset, "AF_pigra","AF_orientalis")


orientalis_hofrichteri_MK_gene, orientalis_hofrichteri_MK_genome = extract_MK(AF_subset, "AF_orientalis","AF_hofrichteri")
adriatica_orientalis_MK_gene, adriatica_orientalis_MK_genome = extract_MK(AF_subset, "AF_adriatica","AF_orientalis")

adriatica_hofrichteri_MK_gene, adriatica_hofrichteri_MK_genome  = extract_MK(AF_subset, "AF_adriatica","AF_hofrichteri")

In [ ]:
#for i in whole_genome_selections_same_type:
#    print(i)

In [ ]:
#for i in whole_genome_selections_slender_vs_stout:
#    print(i)

#### Extract and plot GENOME wide estimates

In [ ]:
whole_genome_selections_slender_vs_stout = [willdenowi_hofrichteri_MK_genome, 
                                            willdenowi_pigra_MK_genome,
                                            adriatica_hofrichteri_MK_genome,
                                            pigra_adriatica_MK_genome,
                                            orientalis_hofrichteri_MK_genome,
                                            pigra_orientalis_MK_genome]
                           
                           
whole_genome_selections_same_type = [willdenowi_orientalis_MK_genome,
                                     pigra_hofrichteri_MK_genome,
                                     adriatica_orientalis_MK_genome,
                                     willdenowi_adriatica_MK_genome]


# Sample data for list1 and list2
list1 = whole_genome_selections_slender_vs_stout
list2 = whole_genome_selections_same_type

In [ ]:
# Get parametric NITG p-values
list1_nitg_param_pvals = [d['pvalue_NITG_parametric'] for d in list1]
list2_nitg_param_pvals = [d['pvalue_NITG_parametric'] for d in list2]

# Jittered positions for plotting
list1_x = rand_jitter(np.zeros(len(list1_nitg)))
list2_x = rand_jitter(np.ones(len(list2_nitg)))


# Create lists for x-axis and y-axis data for both lists and both keys
list1_ni = [d['NI'] for d in list1]
list1_alpha = [d['alpha'] for d in list1]
list1_nitg = [d['NITG'] for d in list1]
list2_ni = [d['NI'] for d in list2]
list2_nitg = [d['NITG'] for d in list2]
list2_alpha = [d['alpha'] for d in list2]

# Create jitter function
def rand_jitter(arr):
    stdev = .05*(max(arr)-min(arr))
    return arr + np.random.randn(len(arr)) * stdev

# Create the plots for NI

# Calculate t-test and p-value for alpha
tstat_NI, pval_NI = ttest_ind(list1_ni, list2_ni)

fig, ax = plt.subplots()
ax.scatter(rand_jitter(np.zeros(len(list1_ni))), list1_ni, c='blue', linewidth = 20,
           s=100, alpha=0.5, edgecolors='none', label='slender vs. stout')
ax.scatter(rand_jitter(np.ones(len(list2_ni))), list2_ni, c='red', linewidth = 20,
           s=100, alpha=0.5, edgecolors='none', label='same ecotype')
ax.set_xticks([0,1])
ax.set_xticklabels(['slender vs. stout', 'same ecotype'])
#ax.set_xlabel('Category')
ax.set_ylabel('NI')
ax.legend()

# Add t-test and p-value to the plot
ax.text(0, 1.75, "t-test p-value: {:.4f}".format(pval_NI))

ax.set_xlim(-0.5, 1.5)
ax.set_ylim(0, 2)

plt.show()


# Create the plots for NItg

# Calculate t-test and p-value for alpha
tstat_NITG, pval_NITG = ttest_ind(list1_nitg, list2_nitg)

fig, ax = plt.subplots()
ax.scatter(rand_jitter(np.zeros(len(list1_nitg))), list1_nitg, c='blue', linewidth = 20,
           s=100, alpha=0.5, edgecolors='none', label='slender vs. stout')
ax.scatter(rand_jitter(np.ones(len(list2_nitg))), list2_nitg, c='red', linewidth = 20,
           s=100, alpha=0.5, edgecolors='none', label='same ecotype')
ax.set_xticks([0,1])
ax.set_xticklabels(['slender vs. stout', 'same ecotype'])
#ax.set_xlabel('Category')
ax.set_ylabel('NITG')
ax.legend()

# Add t-test and p-value to the plot
ax.text(0, 1.75, "t-test p-value: {:.4f}".format(pval_NITG))

ax.set_xlim(-0.5, 1.5)
ax.set_ylim(0, 2)

plt.savefig('results/NITG_selection_allcomp.svg')
plt.show()


# second NItG plot with pvalue annotation

fig, ax = plt.subplots()

# Plot slender vs. stout
ax.scatter(list1_x, list1_nitg, c='blue', linewidth=20,
           s=100, alpha=0.5, edgecolors='none', label='slender vs. stout')

# Plot same ecotype
ax.scatter(list2_x, list2_nitg, c='red', linewidth=20,
           s=100, alpha=0.5, edgecolors='none', label='same ecotype')

# Add stars for significant parametric NITG p-values
for x, y, p in zip(list1_x, list1_nitg, list1_nitg_param_pvals):
    if p < 0.05:
        ax.text(x + 0.03, y, '*', ha='left', va='center', fontsize=16, color='black')

for x, y, p in zip(list2_x, list2_nitg, list2_nitg_param_pvals):
    if p < 0.05:
        ax.text(x + 0.03, y, '*', ha='left', va='center', fontsize=16, color='black')

# Axis labels and legend
ax.set_xticks([0, 1])
ax.set_xticklabels(['slender vs. stout', 'same ecotype'])
ax.set_ylabel('NITG')
ax.legend()

# Annotate t-test p-value
ax.text(0, 1.75, "t-test p-value: {:.4f}".format(pval_NITG))

# Axis limits
ax.set_xlim(-0.5, 1.5)
ax.set_ylim(0, 2)

plt.savefig('results/NITG_selection_allcomp_parametric_stars.svg')
plt.show()


# Create the plots for alpha

# Calculate t-test and p-value for alpha
tstat_alpha, pval_alpha = ttest_ind(list1_alpha, list2_alpha)

fig, ax = plt.subplots()
ax.scatter(rand_jitter(np.zeros(len(list1_alpha))), list1_alpha, c='blue', linewidth = 100,
           s=100, alpha=0.5, edgecolors='none', label='slender vs. stout')
ax.scatter(rand_jitter(np.ones(len(list2_alpha))), list2_alpha, c='red', linewidth = 100,
           s=100, alpha=0.5, edgecolors='none', label='same ecotype')
ax.set_xticks([0,1])
ax.set_xticklabels(['slender vs. stout', 'same ecotype'])
#ax.set_xlabel('Category')
ax.set_ylabel('alpha')
ax.legend()

# Add t-test and p-value to the plot
ax.text(0, 0.75, "t-test p-value: {:.4f}".format(pval_alpha))

# Adjust the limits of the plot
ax.set_xlim(-0.5, 1.5)
ax.set_ylim(-1, 1)

plt.show()

#### Extract and plot GENE wide estimates 

In [ ]:
gene_selections_slender_vs_stout = [willdenowi_hofrichteri_MK_gene, 
                                            willdenowi_pigra_MK_gene,
                                            adriatica_hofrichteri_MK_gene,
                                            pigra_adriatica_MK_gene,
                                            orientalis_hofrichteri_MK_gene,
                                            pigra_orientalis_MK_gene]
                           
                           
gene_selections_same_type = [willdenowi_orientalis_MK_gene,
                                     pigra_hofrichteri_MK_gene,
                                     adriatica_orientalis_MK_gene,
                                     willdenowi_adriatica_MK_gene]

In [ ]:
significant_genes_slender_vs_stout_exact = {}
significant_genes_same_ecotype_exact = {}

for i in gene_selections_slender_vs_stout:
    for j in namestr(i, globals()): # this is only to get the key name as a string there are some strange things going on with some combinations
        if  any(spec in j for spec in ["hofrichteri", "willdenowi","adriatica","orientalis", "pigra"]):
            key = str(j)
            significant_genes_slender_vs_stout_exact[key] = pd.DataFrame(extract_significant_genes_exact(i))

for i in gene_selections_same_type:
    for j in namestr(i, globals()): # this is only to get the key name as a string there are some strange things going on with some combinations
        if  any(spec in j for spec in ["hofrichteri", "willdenowi","adriatica","orientalis", "pigra"]):
            key = str(j)
            significant_genes_same_ecotype_exact[key] = pd.DataFrame(extract_significant_genes_exact(i))



##### extract signifcant genes for each pair

In [ ]:
# different morphotype    
willdenowi_hofrichteri_MK_gene = extract_significant_genes_exact(willdenowi_hofrichteri_MK_gene)
#print(len(willdenowi_hofrichteri_MK_gene))
willdenowi_pigra_MK_gene = extract_significant_genes_exact(willdenowi_pigra_MK_gene)
#print(len(willdenowi_pigra_MK_gene))
adriatica_hofrichteri_MK_gene = extract_significant_genes_exact(adriatica_hofrichteri_MK_gene)
#print(len(adriatica_hofrichteri_MK_gene))
pigra_adriatica_MK_gene = extract_significant_genes_exact(pigra_adriatica_MK_gene)
#print(len(pigra_adriatica_MK_gene))
orientalis_hofrichteri_MK_gene = extract_significant_genes_exact(orientalis_hofrichteri_MK_gene)
#print(len(orientalis_hofrichteri_MK_gene))
pigra_orientalis_MK_gene = extract_significant_genes_exact(pigra_orientalis_MK_gene)
#print(len(pigra_orientalis_MK_gene))


# same type                      
willdenowi_orientalis_MK_gene = extract_significant_genes_exact(willdenowi_orientalis_MK_gene)
#print(len(willdenowi_orientalis_MK_gene))
pigra_hofrichteri_MK_gene = extract_significant_genes_exact(pigra_hofrichteri_MK_gene)
#print(len(pigra_hofrichteri_MK_gene))
adriatica_orientalis_MK_gene = extract_significant_genes_exact(adriatica_orientalis_MK_gene)
#print(len(adriatica_orientalis_MK_gene))
willdenowi_adriatica_MK_gene = extract_significant_genes_exact(willdenowi_adriatica_MK_gene)
#print(len(willdenowi_adriatica_MK_gene))


# slender vs. stout combinations
willdenowi_hofrichteri_MK_gene_list = extract_significant_genes_exact_list(willdenowi_hofrichteri_MK_gene)
#print(len(willdenowi_hofrichteri_MK_gene_list))
willdenowi_pigra_MK_gene_list = extract_significant_genes_exact_list(willdenowi_pigra_MK_gene)
#print(len(willdenowi_pigra_MK_gene_list))
adriatica_hofrichteri_MK_gene_list = extract_significant_genes_exact_list(adriatica_hofrichteri_MK_gene)
#print(len(adriatica_hofrichteri_MK_gene_list))
pigra_adriatica_MK_gene_list = extract_significant_genes_exact_list(pigra_adriatica_MK_gene)
#print(len(pigra_adriatica_MK_gene_list))
orientalis_hofrichteri_MK_gene_list = extract_significant_genes_exact_list(orientalis_hofrichteri_MK_gene)#
#print(len(orientalis_hofrichteri_MK_gene_list))
pigra_orientalis_MK_gene_list = extract_significant_genes_exact_list(pigra_orientalis_MK_gene)
#print(len(pigra_orientalis_MK_gene_list))

# same type                      
willdenowi_orientalis_MK_gene_list = extract_significant_genes_exact_list(willdenowi_orientalis_MK_gene)
#print(len(willdenowi_orientalis_MK_gene_list))
pigra_hofrichteri_MK_gene_list = extract_significant_genes_exact_list(pigra_hofrichteri_MK_gene)
#print(len(pigra_hofrichteri_MK_gene_list))
adriatica_orientalis_MK_gene_list = extract_significant_genes_exact_list(adriatica_orientalis_MK_gene)
#print(len(adriatica_orientalis_MK_gene_list))
willdenowi_adriatica_MK_gene_list = extract_significant_genes_exact_list(willdenowi_adriatica_MK_gene)
#print(len(willdenowi_adriatica_MK_gene_list))

In [ ]:
significant_genes_dict = {"willdenowi_hofrichteri": len(willdenowi_hofrichteri_MK_gene_list),
                         "willdenowi_pigra":len(willdenowi_pigra_MK_gene_list),
                         "adriatica_hofrichteri":len(adriatica_hofrichteri_MK_gene_list),
                         "pigra_adriatica":len(pigra_adriatica_MK_gene_list),
                         "orientalis_hofrichteri":len(orientalis_hofrichteri_MK_gene_list),
                         "pigra_orientalis":len(pigra_orientalis_MK_gene_list),
                         "willdenowi_orientalis":len(willdenowi_orientalis_MK_gene_list),
                         "pigra_hofrichteri":len(pigra_hofrichteri_MK_gene_list),
                         "adriatica_orientalis":len(adriatica_orientalis_MK_gene_list),
                         "willdenowi_adriatica":len(willdenowi_adriatica_MK_gene_list) }

# Create a list of keys and a list of values from the dictionary
keys = list(significant_genes_dict.keys())
values = list(significant_genes_dict.values())

# Create a bar chart
plt.bar(keys, values)

# Add axis labels and a title
plt.xlabel('species comparison')
plt.ylabel('number of significant genes')
plt.title('Number of significant genes under positive selection')

x_pos = range(len(keys))
plt.xticks(x_pos, keys, rotation=45, ha='right')

for i, v in enumerate(values):
    plt.text(i, v+1, str(v), ha='center', fontweight='bold')

plt.savefig('results/Fishers_exact_positive_significant_genes.pdf')

# Show the plot
plt.show()




### Transcription Factor Sites related GOs

In [ ]:
GO_AFs = pd.read_csv("results/GO_analysis/AF_scans/AF_unique_all_in_all_snpeff_two_comp_gProfiler_drerio_03-10-2023_15-49-30__intersections.csv")
GO_AFs_TF = GO_AFs[GO_AFs["source"] == "TF"]
print(f"There are {len(GO_AFs_TF)} Transcription Factors associated with all genes associated from the AF-SCANS.")
#GO_AFs_TF
#GO_AFs_TF_top10 = GO_AFs_TF.head(11).iloc[1:] # extract top 10 TF and skip "tranfac" row
#GO_AFs_TF_top10

In [ ]:
GO_par_score_pos = pd.read_csv("results/GO_analysis/Parallel_Score/GO_par_score_outliers_pos_snpeff_UNIQUE_gProfiler_drerio_10-05-2023_13-49-42__intersections.csv")
GO_par_score_pos_TF = GO_par_score_pos[GO_par_score_pos["source"] == "TF"]
print(f"There are {len(GO_par_score_pos_TF)} Transcription Factors associated with all 328 genes associated with significant positive parallel scores.")
#GO_par_score_pos_TF                              

In [ ]:
GO_par_score_neg = pd.read_csv("results/GO_analysis/Parallel_Score/GO_par_score_outliers_neg_snpeff_UNIQUE_gProfiler_drerio_10-05-2023_13-49-42__intersections.csv")
GO_par_score_neg_TF = GO_par_score_neg[GO_par_score_neg["source"] == "GO:TF"]
print(f"There are {len(GO_par_score_neg_TF)} Transcription Factors associated with all 328 genes associated with significant negative parallel scores.")
#GO_par_score_neg_TF

In [ ]:
TF_overlap = {'TFs from AF-scans': set(GO_AFs_TF["term_name"]),
               'TFs from positive parallel scores': set(GO_par_score_pos_TF["term_name"])}#

TF_overlap_df = pd.merge(GO_par_score_pos_TF,GO_AFs_TF, on="term_name", how="inner")


ax = venn(TF_overlap)
ax.set_title('TF overlap')
plt.savefig('results/venn_TF_comparison_parallel_score_AF-scans.svg', format='svg')
TF_overlap_df

In [ ]:
TF_overlap_df = TF_overlap_df[1:]
TF_overlap_df['term_name_new'] = TF_overlap_df.apply(lambda row: row['term_name'].split("Factor:")[1].split(";")[0] +
                                                               " (" + row['term_id_x'] + ")",
                                                               axis=1)
TF_top10 = TF_overlap_df.head(10)
TF_top10

In [ ]:
plt.figure(figsize=(12, 8))

# Create the scatter plot
scatter_plot = plt.scatter(
    x=TF_top10['negative_log10_of_adjusted_p_value_x'],
    y=TF_top10['term_name_new'],
    s=200,  # Size of the dots
    c=TF_top10['intersection_size_x']/TF_top10['term_size_x'],  # Color based on adjusted_p_value
    cmap='plasma',  # You can choose a different colormap
    #alpha=0.7  # Adjust the transparency
)

# Reverse the y-axis
plt.gca().invert_yaxis()

# Add labels and title
plt.xlabel('negative log10 of adjusted p-value')
plt.ylabel('associated transcription factors')  # Updated ylabel

# Add colorbae
cbar = plt.colorbar(scatter_plot)
cbar.set_label('relative intersection size\n(term size - intersection size)')

# Adjust the colorbar to make it more compact
cbar.ax.yaxis.set_tick_params(width=0)  # Remove tick marks
cbar.ax.tick_params(size=0)  # Remove tick lines

plt.savefig('results/TF_top10_overlap.svg', format='svg')

# Show the plot
plt.show()


## Appendix (Supplementary results)

### AF-analysis including G. adriatica

#### AF screens

In [ ]:
AF_df = pd.read_csv("raw_data/AF_Gouania.tsv.gz", sep = "\t")#, compression='gzip')
#AF_df

In [ ]:
#background_snpeff = extract_deduplicated_gene_from_snpeff_list(AF_df["gene_snpeff"], 
#                                                               "outliers/AF_outliers/AF_background_snpeff.txt")

### I GENERAL: Fixed alleles in genes for species and groups (slender, stout) 

##### fixed for both slender (1) and stout (0)

In [ ]:
fixed_slender_1_stout_0 = AF_df[(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 1)]

fixed_slender_1_stout_0_genes = extract_deduplicated_gene_from_snpeff_list(fixed_slender_1_stout_0["gene_snpeff"], "results/outliers/AF_outliers/AF_fixed_slender_1_stout_0_snpeff.txt")
fixed_slender_1_stout_0

##### fixed for both stout (1) and slender (0)

In [ ]:
fixed_stout_1_stout_0 = AF_df[(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 0)]
fixed_stout_1_stout_0_genes = extract_deduplicated_gene_from_snpeff_list(fixed_stout_1_stout_0["gene_snpeff"], "results/outliers/AF_outliers/AF_fixed_stout_1_slender_0_snpeff.txt")
fixed_stout_1_stout_0

##### fixed for pigra 

In [ ]:
fixed_pigra_1 = AF_df[(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 0)]


fixed_pigra_0 = AF_df[(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 1)]

fixed_pigra = pd.concat([fixed_pigra_0, fixed_pigra_1])

fixed_pigra_genes = extract_deduplicated_gene_from_snpeff_list(fixed_pigra["gene_snpeff"], 
                                                               "results/outliers/AF_outliers/AF_pigra_fixed_snpeff.txt")

print("There are {} fixed snps with genotype 1|1 for pigra.".format(len(fixed_pigra_1)))
print("There are {} fixed snps with genotype 0|0 for pigra.".format(len(fixed_pigra_0)))

fixed_pigra

##### fixed for hofrichteri

In [ ]:
fixed_hofrichteri_1 = AF_df[(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 1)]

fixed_hofrichteri_0 = AF_df[(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 0)]

fixed_hofrichteri = pd.concat([fixed_hofrichteri_0, fixed_hofrichteri_1])                            
fixed_hofrichteri_genes = extract_deduplicated_gene_from_snpeff_list(fixed_hofrichteri["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/AF_hofrichteri_fixed_snpeff.txt")

print("There are {} fixed snps with genotype 1|1 for hofrichteri.".format(len(fixed_hofrichteri_1)))
print("There are {} fixed snps with genotype 0|0 for hofrichteri.".format(len(fixed_hofrichteri_0)))
                            
fixed_hofrichteri

##### fixed willdenowi 

In [ ]:
fixed_willdenowi_1 = AF_df[(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 0)]

fixed_willdenowi_0 = AF_df[(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 1)]

fixed_willdenowi = pd.concat([fixed_willdenowi_1, fixed_willdenowi_0])                            
fixed_willdenowi_genes = extract_deduplicated_gene_from_snpeff_list(fixed_willdenowi["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/AF_willdenowi_fixed_snpeff.txt")

print("There are {} fixed snps with genotype 1|1 for willdenowi.".format(len(fixed_willdenowi_1)))
print("There are {} fixed snps with genotype 0|0 for willdenowi.".format(len(fixed_willdenowi_0)))
                            
fixed_willdenowi

##### fixed orientalis 

In [ ]:
fixed_orientalis_1 = AF_df[(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 0)]

fixed_orientalis_0 = AF_df[(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 1)]


fixed_orientalis = pd.concat([fixed_orientalis_1, fixed_orientalis_0])                            
fixed_orientalis_genes = extract_deduplicated_gene_from_snpeff_list(fixed_orientalis["gene_snpeff"], 
                                                                     "results/outliers/AF_outliers/AF_orientalis_fixed_snpeff.txt")

print("There are {} fixed snps with genotype 1|1 for orientalis.".format(len(fixed_orientalis_1)))
print("There are {} fixed snps with genotype 0|0 for orienatlis.".format(len(fixed_orientalis_0)))

fixed_orientalis

##### fixed adriatica

In [ ]:
fixed_adriatica_1 = AF_df[(AF_df['AF_adriatica'] == 1) &
                    (AF_df['AF_orientalis'] == 0) &
                    (AF_df['AF_willdenowi'] == 0) &
                    (AF_df['AF_pigra'] == 0) &
                    (AF_df['AF_hofrichteri'] == 0)]

fixed_adriatica_0 = AF_df[(AF_df['AF_adriatica'] == 0) &
                    (AF_df['AF_orientalis'] == 1) &
                    (AF_df['AF_willdenowi'] == 1) &
                    (AF_df['AF_pigra'] == 1) &
                    (AF_df['AF_hofrichteri'] == 1)]
fixed_adriatica = pd.concat([fixed_adriatica_0,fixed_adriatica_1])

fixed_adriatica_genes = extract_deduplicated_gene_from_snpeff_list(fixed_adriatica["gene_snpeff"], 
                                                                   "results/outliers/AF_outliers/AF_adriatica_fixed_snpeff.txt")

print("There are {} fixed snps with genotype 1|1 for adriatica.".format(len(fixed_adriatica_1)))
print("There are {} fixed snps with genotype 0|0 for adriatica.".format(len(fixed_adriatica_0)))


fixed_adriatica

#### overlapping genes slender (fixed hofrichteri ∩ pigra)

In [ ]:
slender_overlap = list(set(fixed_hofrichteri_genes) & set(fixed_pigra_genes))
print("There are fixed mutations in {} genes that overlap in the two slender species (fixed hofrichteri ∩ pigra).".format(len(slender_overlap)))



with open("results/outliers/AF_outliers/AF_slender_fixed_snpeff.txt", "w") as outfile:
    for i in slender_overlap:
        outfile.write(i + "\n")
        

#### overlapping stout (fixed adriatica ∩ orientalis ∩ willdenowi)

In [ ]:
stout_overlap = list(set(fixed_adriatica_genes) & set(fixed_orientalis_genes) & set(fixed_willdenowi_genes))
print("There are fixed mutations in {} genes that overlap in the three stout species (fixed adriatica ∩ orientalis ∩ willdenowi).".format(len(stout_overlap)))

with open("results/outliers/AF_outliers/AF_stout_fixed_snpeff.txt", "w") as outfile:
    for i in stout_overlap:
        outfile.write(i + "\n")
        

##### overlapping genes (fixed SLENDER)
(fixed_pigra ∩ fixed_hofrichteri) - fixed_adriatica - fixed_orientalis - fixed_willdenowi

In [ ]:
slender_unique = list(set(slender_overlap) - set(fixed_adriatica_genes) - set(fixed_orientalis_genes) - set(fixed_willdenowi_genes))
print("There are {} unique genes shared between the two slender types only ((fixed_pigra ∩ fixed_hofrichteri) - fixed_adriatica - fixed_orientalis - fixed_willdenowi).".format(len(slender_unique)))

with open("results/outliers/AF_outliers/AF_slender_unique_snpeff.txt", "w") as outfile:
    for i in slender_unique:
        outfile.write(i + "\n")

##### overlapping genes (fixed STOUT)
(fixed_adriatica ∩ fixed_orientalis ∩ fixed_willdenowi) - fixed_pigra - fixed_hofrichteri) 

In [ ]:
stout_unique = list(set(stout_overlap) - set(fixed_pigra_genes) - set(fixed_hofrichteri_genes))
print("There are {} unique genes shared between the three stout types only ((fixed_pigra ∩ fixed_hofrichteri) - fixed_adriatica - fixed_orientalis - fixed_willdenowi).".format(len(stout_unique)))

with open("results/outliers/AF_outliers/AF_stout_unique_snpeff.txt", "w") as outfile:
    for i in stout_unique:
        outfile.write(i + "\n")

##### fixed parallel slender and stout all in all

In [ ]:
AF_unique_all_in_all = stout_unique + slender_unique
with open("results/outliers/AF_outliers/AF_unique_all_in_all_snpeff.txt", "w") as outfile:
    for i in AF_unique_all_in_all:
        outfile.write(i + "\n")

##### VENNs

In [ ]:

%matplotlib inline


fixed_genes_slender_venn = {#'fixed adriatica (0)':set(fixed_adriatica_0_genes), 
               #'fixed orientalis (1)': set(fixed_orientalis_genes),
               #'fixed willdenowi (1)': set(fixed_willdenowi_genes),
               'fixed hofrichteri': set(fixed_hofrichteri_genes),
               'fixed pigra': set(fixed_pigra_genes)}

ax1 = venn(fixed_genes_slender_venn)
ax1.set_title('Fixed genes in Slender')

fixed_genes_stout_venn = {'fixed adriatica':set(fixed_adriatica_genes), 
               'fixed orientalis': set(fixed_orientalis_genes),
               'fixed willdenowi': set(fixed_willdenowi_genes),
               #'fixed hofrichteri (1)': set(fixed_hofrichteri_genes),
               #'fixed pigra (1)': set(fixed_pigra_genes)
                    }

ax2 = venn(fixed_genes_stout_venn)
ax2.set_title('Fixed genes in Stout')

fixed_genes_venn = {'fixed adriatica':set(fixed_adriatica_genes), 
               'fixed orientalis': set(fixed_orientalis_genes),
               'fixed willdenowi': set(fixed_willdenowi_genes),
               'fixed hofrichteri': set(fixed_hofrichteri_genes),
               'fixed pigra': set(fixed_pigra_genes)
                    }

ax3 = venn(fixed_genes_venn)
ax3.set_title('Fixed genes in all species')

plt.savefig('results/venn_all-in-all.svg', format='svg')

##### random shuffling of sets 

This is done to test whether the GO terms found are simply driven by genes under strong puryfing selection. 

In [ ]:

#random_selections = []

#for i in range(1,10):
#    random_list_fixed_genes = ["fixed_pigra_genes", "fixed_hofrichteri_genes", 
#               "fixed_willdenowi_genes", "fixed_adriatica_genes", "fixed_orientalis_genes"]
#    random_selection = random.sample(random_list_fixed_genes, len(random_list_fixed_genes))
#    set1 = random_selection[0]
#    set2 = random_selection[1]
#    set3 = random_selection[2]
#    set4 = random_selection[3]
#    set5 = random_selection[4]
    
#    run = f"(set({set1}) & set({set2})) - set({set3}) - set({set4}) - set({set5})"
#    
#    if run.startswith("(set(fixed_hofrichteri_genes) & set(fixed_pigra_genes))"):
#        pass
#    elif run.startswith("(set(fixed_pigra_genes) & set(fixed_hofrichteri_genes))"):
#        pass
#    else:
#        random_selections.append(run)

#randomization_gene_lists = dict()
#for run in random_selections:
#    with open(f"results/outliers/AF_outliers/random/{run}.txt","w") as outfile:
#        gene_list = eval(run)
#        randomization_gene_lists[run] = gene_list
#        for i in gene_list:
#            outfile.write(i + "\n")

#for k,v in randomization_gene_lists.items():
#    print("{}: {} genes".format(k,len(v)))

<b>Based on the above created lists a GO analyses was conducted to test how many GO terms overlap in the groups.</b>

In [ ]:


random_overview_dict = {}

with open("results/GO_analysis/GO_ultimate_snpeff_random_comparison.tsv", "r") as file:
    for line in file:
        if line.startswith("set"):
            pass
        else:
            line = line.rstrip()
            random_overview_dict[line.split("\t")[0]] = line.split("\t")[1:]

ultimate_GO_set = random_overview_dict["AF_unique_ULTIMATE"] # see above AF_unique_all_in_all

random_GO_sets = []

for k,v in random_overview_dict.items():
    if k == "AF_unique_ULTIMATE":
        pass
    else:
        for i in v:
            if i in random_GO_sets:
                pass
            elif i == "No enriched BP sets":
                pass
            else:
                random_GO_sets.append(i)


#random_GO_sets
            
fixed_genes_venn = {'GO terms ultimate': set(ultimate_GO_set),
               'GO terms other combinations': set(random_GO_sets)}#

ax = venn(fixed_genes_venn)
ax.set_title('UNIQUE GO TERMS')

unique_GO_terms_ultimate = list(set(ultimate_GO_set) - set(random_GO_sets))

print(f"There are {len(unique_GO_terms_ultimate)} GO terms unique the ultimate dataset.\nThese include:{unique_GO_terms_ultimate}")

In [ ]:
GO_TERMS_PARALLEL_ALL = pd.read_csv("results/GO_analysis/GO_terms_parallel_all.txt", sep = "\t")
#GO_TERMS_PARALLEL_ALL
count_values = lambda x: len(x.split(';'))

#GO_TERMS_PARALLEL_ALL['gene_counts'] = GO_TERMS_PARALLEL_ALL['intersections'].apply(count_values)

GO_TERMS_PARALLEL_ALL = GO_TERMS_PARALLEL_ALL[GO_TERMS_PARALLEL_ALL['term_name'].isin(unique_GO_terms_ultimate)]
GO_TERMS_PARALLEL_ALL


#### CHECK FOR SELECTION IN 2010 GENES

Excess divergence can come from drift. 
To test this we conducted a selection scan (DoS) and compared the 1850 genes to the background and found no signals of purifying selection [add medians of both DOS and p value and t-test pvalue]

In [ ]:
gene_selections_slender_vs_stout = [willdenowi_hofrichteri_MK_gene, 
                                    willdenowi_pigra_MK_gene,
                                    adriatica_hofrichteri_MK_gene,
                                    pigra_adriatica_MK_gene,
                                    orientalis_hofrichteri_MK_gene,
                                    pigra_orientalis_MK_gene]
                           
gene_selections_slender_vs_stout_names = ["willdenowi vs. hofrichteri", 
                                          "willdenowi vs. pigra",
                                          "adriatica vs. hofrichteri",
                                          "pigra vs. adriatica",
                                          "orientalis vs. hofrichteri", 
                                          "pigra vs. orientalis"]
                                          
        
#gene_selections_same_type = [willdenowi_orientalis_MK_gene,
#                                     pigra_hofrichteri_MK_gene]

#Extract NI from all genes that are in slender and stout (1850 genes).

print("There are {} genes unique for parallel trajectories.".format(len(AF_unique_all_in_all)))

In [ ]:
def plot_unique_vs_background(input_MK_gene_df, header, ax=None):
    selected_genes =  input_MK_gene_df[input_MK_gene_df['gene_snpeff'].isin(AF_unique_all_in_all)]
    all_genes = input_MK_gene_df
    
    # Perform t-test
    t_stat, p_value = stats.ttest_ind(selected_genes["DoS"], all_genes["DoS"])

    # Plot the histograms
    if ax is None:
        ax = plt.gca()
    selected_genes["DoS"].hist(bins=40, alpha=0.5, label=f'selected {len(AF_unique_all_in_all)} genes', density=True, ax=ax)
    all_genes["DoS"].hist(bins=40, alpha=0.5, label='all Genes', density=True, ax=ax)

    # Add axis labels and title
    ax.set_xlabel('DoS', fontsize=16,fontweight='bold')
    ax.set_ylabel('frequency', fontsize=16,fontweight='bold')
    # ax.set_title(f'DoS focal vs. all genes \n {header}')

    # Add legend
    ax.legend(loc='upper right')

    # Add p-value to the plot
    ax.text(ax.get_xlim()[0] + (ax.get_xlim()[1] - ax.get_xlim()[0]) * 0.01, ax.get_ylim()[1] * 0.97, f'p-value: {p_value:.4f}', ha='left', fontsize=14,va='center', color='black')
    
    ax.tick_params(axis='both', which='major', labelsize=14)
    # Remove gridlines
    ax.grid(False)

# Create a new figure and axis object
num_pairs = len(gene_selections_slender_vs_stout)
num_cols = 2
num_rows = (num_pairs + num_cols - 1) // num_cols  # Ceiling division
fig, axs = plt.subplots(num_rows, num_cols, figsize=(16, num_rows*6))

# Sort the pairs by header alphabetically
sorted_pairs = sorted(zip(gene_selections_slender_vs_stout, gene_selections_slender_vs_stout_names), key=lambda x: x[1])

# Iterate through the sorted list of datasets
for idx, (pair, header) in enumerate(sorted_pairs):
    # Determine the position for this pair of plots
    row = idx // num_cols
    col = idx % num_cols

    # Plot each dataset
    plot_unique_vs_background(pair, header, ax=axs[row, col])

    # Add label to each pair
    axs[row, col].text(0.00, 1.01, f'{chr(97+idx)}', transform=axs[row, col].transAxes, fontweight='bold', fontsize=16, ha='left', va='bottom')
    axs[row, col].text(0.40, 1.01, f'{header}', transform=axs[row, col].transAxes, fontsize=16, ha='left', va='bottom')

# Remove empty subplots
for idx in range(len(sorted_pairs), num_rows * num_cols):
    row = idx // num_cols
    col = idx % num_cols
    fig.delaxes(axs[row, col])

# Adjust layout to prevent overlapping
plt.tight_layout()

# Save the entire figure once
plt.savefig('results/DoSfocalVsall_2010genes_combined.pdf')

# Show the plot
plt.show()



### TRASH BUGS

In [ ]:
TF_top10overlap_df['term_name_new'] = TF_top10overlap_df.apply(lambda row: row['term_name'].split("Factor:")[1].split(";")[0] +
                                                               " (" + row['term_id_x'] + ")",
                                                               axis=1)
TF_top10overlap_df

In [ ]:

plt.figure(figsize=(12, 8))

# Create the scatter plot
scatter_plot = plt.scatter(
    x=TF_top10overlap_df['negative_log10_of_adjusted_p_value_x'],
    y=TF_top10overlap_df['term_name_new'],
    s=200,  # Size of the dots
    c=TF_top10overlap_df['intersection_size_x']/TF_top10overlap_df['term_size_x'],  # Color based on adjusted_p_value
    cmap='plasma',  # You can choose a different colormap
    #alpha=0.7  # Adjust the transparency
)

# Reverse the y-axis
plt.gca().invert_yaxis()

# Add labels and title
plt.xlabel('negative log10 of adjusted p-value')
plt.ylabel('associated transcription factors')  # Updated ylabel

# Add colorbae
cbar = plt.colorbar(scatter_plot)
cbar.set_label('relative intersection size\n(term size - intersection size)')

# Adjust the colorbar to make it more compact
cbar.ax.yaxis.set_tick_params(width=0)  # Remove tick marks
cbar.ax.tick_params(size=0)  # Remove tick lines

plt.savefig('results/TF_top10_overlap_unique.svg', format='svg')

# Show the plot
plt.show()


In [ ]:
GO_AFs_TF_top10 = GO_AFs_TF.head(11).iloc[1:] # extract top 10 TF and skip "tranfac" row
#GO_AFs_TF_top10
GO_par_score_pos_TF_top10 = GO_par_score_pos_TF.head(11).iloc[1:] # extract top 10 TF and skip "tranfac" row
#GO_par_score_pos_TF_top10


TF_top10overlap = {'Top 10 TFs from AF-scans': set(GO_AFs_TF_top10["term_name"]),
               'Top 10 TFs from positive parallel scores': set(GO_par_score_pos_TF_top10["term_name"])}#

ax = venn(TF_top10overlap)
ax.set_title('TF overlap top 10')
plt.savefig('results/venn_TF_comparison_parallel_score_AF-scans_TOP10.svg', format='svg')

TF_top10overlap_df = pd.merge(GO_par_score_pos_TF_top10,GO_AFs_TF_top10, on="term_name", how="inner")
TF_top10overlap_df

## ABBA-BABA TESTS - VISUALIZATION

In [ ]:
from statsmodels.stats.multitest import multipletests
Dmin = pd.read_csv("raw_data/Dtrios_Gouania_combined_Dmin.txt", sep = "\t")

# Benjamini-Hochberg correction for hofrichteri_Dmin
p_values = Dmin['p-value']
adjusted_pvals = multipletests(p_values, alpha=0.05, method='fdr_bh')[1]
Dmin['p-value_adj'] = adjusted_pvals
Dmin['Significance'] = Dmin['p-value_adj'].apply(lambda p: 's.' if p < 0.05 else 'n.s.')

Dmin = Dmin.sort_values(by='Dstatistic', ascending=False)

#Dmin.to_excel('results/Dtrios/CreteDtrios_hofrichteri_combined_Dmin.xlsx', index=False)
Dmin

In [ ]:

def plot_heatmaps(data, dataset_name):
    """
    Plot heatmaps of mean D-statistic and f4-ratio for each (P2, P3) pair in the dataset,
    using the orange color scale and colorbar ratio similar to the example.
    Saves the plot as 'results/Dmin.svg'.
    """

    # Aggregate by (P2, P3) pair: compute mean values
    aggregated = (
        data.groupby(['P2', 'P3'])
            .agg({'Dstatistic': 'mean', 'f4-ratio': 'mean'})
            .reset_index()
    )

    # Create pivot tables for heatmaps (rows: P3, columns: P2)
    dstat_pivot = aggregated.pivot(index='P3', columns='P2', values='Dstatistic')
    f4ratio_pivot = aggregated.pivot(index='P3', columns='P2', values='f4-ratio')

    # Create results directory if it doesn't exist
    os.makedirs("results", exist_ok=True)

    # Setup figure with two subplots side-by-side
    plt.figure(figsize=(16, 6))

    # D-statistic heatmap (orange color palette similar to your example)
    plt.subplot(1, 2, 1)
    sns.heatmap(
        dstat_pivot,
        annot=True,
        fmt=".3f",
        cmap="Oranges",
        linewidths=0.5,
        square=True,
        vmin=0,
        vmax=dstat_pivot.max().max(),  # scale max to data max for better contrast
        cbar_kws={"label": "D-statistic", "shrink": 0.4, 'aspect': 5},
        annot_kws={"size": 10}
    )
    plt.title(f'Heatmap of D-statistic ({dataset_name})')
    plt.xlabel('P2')
    plt.ylabel('P3')

    # f4-ratio heatmap (orange color palette with similar scale bar)
    plt.subplot(1, 2, 2)
    sns.heatmap(
        f4ratio_pivot,
        annot=True,
        fmt=".3f",
        cmap="Oranges",
        linewidths=0.5,
        square=True,
        vmin=0,
        vmax=f4ratio_pivot.max().max(),
        cbar_kws={"label": "f4-ratio", "shrink": 0.4, 'aspect': 5},
        annot_kws={"size": 10}
    )
    plt.title(f'Heatmap of f4-ratio ({dataset_name})')
    plt.xlabel('P2')
    plt.ylabel('P3')

    plt.tight_layout()

    # Save the figure as SVG
    plt.savefig("results/Dmin.svg", format="svg")

    plt.show()


In [ ]:
plot_heatmaps(Dmin, "Gouania")

# OTHER NOT USED ANALYSES

#### MK TESTS ETC. BOOTSTRAPPING NITG ETC. FAIl 

###### First version

In [ ]:
def extract_MK(input_df, AF_colname_species1, AF_colname_species2):#, permutations):
    '''
    This function returns a dataframe inlcuding Pn, Ps, Dn, Ds, alpha (MK), NI and p-value 
    (obtained from chi2 and Fisher's exact test) for each gene of a certain species pair combination. 
    It also extracts the mentioned values across the wholes set of genes (genome wide).
    '''

    # subset to the two species
    AF_subset = input_df[
        ["CHROM", "POS", "cat", "type", "gene_snpeff", str(AF_colname_species1), str(AF_colname_species2)]]
    AF_subset[str(AF_colname_species1)] = AF_subset[str(AF_colname_species1)].astype(float)
    AF_subset[str(AF_colname_species2)] = AF_subset[AF_colname_species2].astype(float)

    # First create a datframe that includes nonsyn. and syn. polymorphism for both species (0 < AF < 1)

    polymorphisms_df = AF_subset[((AF_subset[str(AF_colname_species1)] > 0) &
                                  (AF_subset[str(AF_colname_species1)] < 1)) |
                                 ((AF_subset[str(AF_colname_species2)] > 0) &
                                  (AF_subset[str(AF_colname_species2)] < 1))]

    polymorphisms = pd.DataFrame(polymorphisms_df.groupby(["gene_snpeff", "type"]).size()).reset_index()
    polymorphisms.columns = ['gene_snpeff', 'type', 'count']

    # Pivot the dataframe to rearrange the columns
    polymorphisms = polymorphisms.pivot_table(values='count', index='gene_snpeff', columns='type', aggfunc='sum',
                                              fill_value=0).reset_index()
    # rename colnames for later
    polymorphisms.columns = ['gene_snpeff', 'Pn_nonsynonymous_polymorphisms', 'Ps_synonymous_polymorphisms']

    # Second get fixed nonsyn and synonymous subsitutions for both species

    fixed_variants = AF_subset[((AF_subset[str(AF_colname_species1)] == 0) &
                                (AF_subset[str(AF_colname_species2)] == 1)) |
                               ((AF_subset[str(AF_colname_species1)] == 1) &
                                (AF_subset[str(AF_colname_species2)] == 0))]

    fixed_nonsyn_syn = pd.DataFrame(fixed_variants.groupby(["gene_snpeff", "type"]).size()).reset_index()
    fixed_nonsyn_syn.columns = ['gene_snpeff', 'type', 'count']

    # Pivot the dataframe to rearrange the columns
    fixed_nonsyn_syn = fixed_nonsyn_syn.pivot_table(values='count', index='gene_snpeff', columns='type', 
                                                    aggfunc='sum', fill_value=0).reset_index()
    # rename colnames for later
    fixed_nonsyn_syn.columns = ['gene_snpeff', 'Dn_nonsynonymous_fixed', 'Ds_synonymous_fixed']

    # Now merge the two dataframes and calculate all the stats gene-wise

    mk_table_gene_wise = polymorphisms.merge(fixed_nonsyn_syn, on='gene_snpeff')

    mk_table_gene_wise["alpha"] = 1 - (mk_table_gene_wise["Ds_synonymous_fixed"] * mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"]) / \
                                  (mk_table_gene_wise["Dn_nonsynonymous_fixed"] * mk_table_gene_wise["Ps_synonymous_polymorphisms"])

    

    mk_table_gene_wise["NI"] = (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] / mk_table_gene_wise["Ps_synonymous_polymorphisms"]) / \
                               (mk_table_gene_wise["Dn_nonsynonymous_fixed"] / mk_table_gene_wise[
                                   "Ds_synonymous_fixed"])
    
    #DoS = Dn/(Dn + Ds) − Pn/(Pn + Ps)
    
    mk_table_gene_wise["DoS"] = (mk_table_gene_wise["Dn_nonsynonymous_fixed"]/(mk_table_gene_wise["Dn_nonsynonymous_fixed"] + mk_table_gene_wise["Ds_synonymous_fixed"])) - \
                                (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"]/(mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] + mk_table_gene_wise["Ps_synonymous_polymorphisms"]))
    

    # Calculate p-value from permutation
    
    #mk_table_gene_wise["p_value_permut"] = mk_table_gene_wise.apply(calculate_p_value_permutation, axis=1)
    


    # Fishers Exact test
    pvalues_exact = []
    for i in range(len(mk_table_gene_wise)):
        oddsratio, pvalue = fisher_exact(
            [[mk_table_gene_wise.loc[i, 'Dn_nonsynonymous_fixed'], 
              mk_table_gene_wise.loc[i, 'Ds_synonymous_fixed']],
             [mk_table_gene_wise.loc[i, 'Pn_nonsynonymous_polymorphisms'],
              mk_table_gene_wise.loc[i, 'Ps_synonymous_polymorphisms']]])
        pvalues_exact.append(pvalue)

    mk_table_gene_wise['pvalue_exact'] = pvalues_exact
    
    
    # Chi2 test
    pvalues_chi2 = []
    for i in range(len(mk_table_gene_wise)):
        obs = np.array([[mk_table_gene_wise.loc[i,'Dn_nonsynonymous_fixed']+0.1, mk_table_gene_wise.loc[i,'Ds_synonymous_fixed']+0.1],
                [mk_table_gene_wise.loc[i,'Pn_nonsynonymous_polymorphisms']+0.1, mk_table_gene_wise.loc[i,'Ps_synonymous_polymorphisms']+0.1]])
        chi2, p, dof, expected = chi2_contingency(obs)
        pvalues_chi2.append(p)
    
    mk_table_gene_wise['pvalue_chi2'] = pvalues_chi2
    
    
    # NI and alpha whole genome
    
    #mk_table_whole_genome = pd.DataFrame()
    
    alpha_genome = 1 - (mk_table_gene_wise["Ds_synonymous_fixed"].sum() * mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"].sum()) / \
                                  (mk_table_gene_wise["Dn_nonsynonymous_fixed"].sum() * mk_table_gene_wise["Ps_synonymous_polymorphisms"].sum())
    
    NI_genome = (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"].sum() / mk_table_gene_wise["Ps_synonymous_polymorphisms"].sum()) / \
                               (mk_table_gene_wise["Dn_nonsynonymous_fixed"].sum() / mk_table_gene_wise["Ds_synonymous_fixed"].sum())
    
    
    # add bootstrapping
    
    
    
    # before add adding bootstrapping
    NITG_genome = (mk_table_gene_wise["Ds_synonymous_fixed"] * mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] / \
                   (mk_table_gene_wise["Ps_synonymous_polymorphisms"] + mk_table_gene_wise["Ds_synonymous_fixed"])).sum() / \
                  (mk_table_gene_wise["Ps_synonymous_polymorphisms"] * mk_table_gene_wise["Dn_nonsynonymous_fixed"] / \
                   (mk_table_gene_wise["Ps_synonymous_polymorphisms"] + mk_table_gene_wise["Ds_synonymous_fixed"])).sum()
    
    whole_genome_values = {"alpha":alpha_genome,"NI":NI_genome, "NITG":NITG_genome}
    
    
    return mk_table_gene_wise, whole_genome_values

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact, chi2_contingency


def bootstrap_nitg(mk_df, n_iterations=1000, seed=42):
    np.random.seed(seed)
    nitg_values = []

    for _ in range(n_iterations):
        sample_df = mk_df.sample(n=len(mk_df), replace=True)

        num = (sample_df["Ds_synonymous_fixed"] * sample_df["Pn_nonsynonymous_polymorphisms"] /
               (sample_df["Ps_synonymous_polymorphisms"] + sample_df["Ds_synonymous_fixed"])).sum()

        denom = (sample_df["Ps_synonymous_polymorphisms"] * sample_df["Dn_nonsynonymous_fixed"] /
                 (sample_df["Ps_synonymous_polymorphisms"] + sample_df["Ds_synonymous_fixed"])).sum()

        if denom != 0:
            nitg_values.append(num / denom)
        else:
            nitg_values.append(np.nan)

    nitg_array = np.array(nitg_values)
    nitg_array = nitg_array[~np.isnan(nitg_array)]

    ci_lower = np.percentile(nitg_array, 2.5)
    ci_upper = np.percentile(nitg_array, 97.5)

    # Optional p-value under null of NITG = 1
    nitg_genome_est = (mk_df["Ds_synonymous_fixed"] * mk_df["Pn_nonsynonymous_polymorphisms"] /
                      (mk_df["Ps_synonymous_polymorphisms"] + mk_df["Ds_synonymous_fixed"])).sum() / \
                     (mk_df["Ps_synonymous_polymorphisms"] * mk_df["Dn_nonsynonymous_fixed"] /
                      (mk_df["Ps_synonymous_polymorphisms"] + mk_df["Ds_synonymous_fixed"])).sum()

    p_value = np.mean(nitg_array <= 1) if nitg_genome_est < 1 else np.mean(nitg_array >= 1)

    return nitg_array, ci_lower, ci_upper, p_value


def extract_MK(input_df, AF_colname_species1, AF_colname_species2):
    AF_subset = input_df[["CHROM", "POS", "cat", "type", "gene_snpeff",
                          str(AF_colname_species1), str(AF_colname_species2)]].copy()
    AF_subset[str(AF_colname_species1)] = AF_subset[str(AF_colname_species1)].astype(float)
    AF_subset[str(AF_colname_species2)] = AF_subset[str(AF_colname_species2)].astype(float)

    # Polymorphisms
    polymorphisms_df = AF_subset[((AF_subset[str(AF_colname_species1)] > 0) & (AF_subset[str(AF_colname_species1)] < 1)) |
                                 ((AF_subset[str(AF_colname_species2)] > 0) & (AF_subset[str(AF_colname_species2)] < 1))]

    polymorphisms = polymorphisms_df.groupby(["gene_snpeff", "type"]).size().unstack(fill_value=0).reset_index()
    polymorphisms.columns = ['gene_snpeff', 'Pn_nonsynonymous_polymorphisms', 'Ps_synonymous_polymorphisms']

    # Fixed differences
    fixed_variants = AF_subset[((AF_subset[str(AF_colname_species1)] == 0) & (AF_subset[str(AF_colname_species2)] == 1)) |
                               ((AF_subset[str(AF_colname_species1)] == 1) & (AF_subset[str(AF_colname_species2)] == 0))]

    fixed_nonsyn_syn = fixed_variants.groupby(["gene_snpeff", "type"]).size().unstack(fill_value=0).reset_index()
    fixed_nonsyn_syn.columns = ['gene_snpeff', 'Dn_nonsynonymous_fixed', 'Ds_synonymous_fixed']

    # Merge and compute stats
    mk_table_gene_wise = polymorphisms.merge(fixed_nonsyn_syn, on='gene_snpeff')

    mk_table_gene_wise["alpha"] = 1 - (mk_table_gene_wise["Ds_synonymous_fixed"] * mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"]) / \
                                      (mk_table_gene_wise["Dn_nonsynonymous_fixed"] * mk_table_gene_wise["Ps_synonymous_polymorphisms"])

    mk_table_gene_wise["NI"] = (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] / mk_table_gene_wise["Ps_synonymous_polymorphisms"]) / \
                                (mk_table_gene_wise["Dn_nonsynonymous_fixed"] / mk_table_gene_wise["Ds_synonymous_fixed"])

    mk_table_gene_wise["DoS"] = (mk_table_gene_wise["Dn_nonsynonymous_fixed"] /
                                  (mk_table_gene_wise["Dn_nonsynonymous_fixed"] + mk_table_gene_wise["Ds_synonymous_fixed"])) - \
                                 (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] /
                                  (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] + mk_table_gene_wise["Ps_synonymous_polymorphisms"]))

    # Fisher's exact test
    pvalues_exact = []
    for i in range(len(mk_table_gene_wise)):
        oddsratio, pvalue = fisher_exact([
            [mk_table_gene_wise.loc[i, 'Dn_nonsynonymous_fixed'], mk_table_gene_wise.loc[i, 'Ds_synonymous_fixed']],
            [mk_table_gene_wise.loc[i, 'Pn_nonsynonymous_polymorphisms'], mk_table_gene_wise.loc[i, 'Ps_synonymous_polymorphisms']]])
        pvalues_exact.append(pvalue)
    mk_table_gene_wise['pvalue_exact'] = pvalues_exact

    # Chi-square test
    pvalues_chi2 = []
    for i in range(len(mk_table_gene_wise)):
        obs = np.array([
            [mk_table_gene_wise.loc[i,'Dn_nonsynonymous_fixed'] + 0.1, mk_table_gene_wise.loc[i,'Ds_synonymous_fixed'] + 0.1],
            [mk_table_gene_wise.loc[i,'Pn_nonsynonymous_polymorphisms'] + 0.1, mk_table_gene_wise.loc[i,'Ps_synonymous_polymorphisms'] + 0.1]])
        chi2, p, dof, expected = chi2_contingency(obs)
        pvalues_chi2.append(p)
    mk_table_gene_wise['pvalue_chi2'] = pvalues_chi2

    # Genome-wide stats
    alpha_genome = 1 - (mk_table_gene_wise["Ds_synonymous_fixed"].sum() * mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"].sum()) / \
                    (mk_table_gene_wise["Dn_nonsynonymous_fixed"].sum() * mk_table_gene_wise["Ps_synonymous_polymorphisms"].sum())

    NI_genome = (mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"].sum() / mk_table_gene_wise["Ps_synonymous_polymorphisms"].sum()) / \
                (mk_table_gene_wise["Dn_nonsynonymous_fixed"].sum() / mk_table_gene_wise["Ds_synonymous_fixed"].sum())

    NI_genome_log_ratio = np.log2(NI_genome)
    NI_genome_pval = 2 * min(np.mean(NI_genome_log_ratio > 0), np.mean(NI_genome_log_ratio < 0))

    numerator = (mk_table_gene_wise["Ds_synonymous_fixed"] * mk_table_gene_wise["Pn_nonsynonymous_polymorphisms"] /
                 (mk_table_gene_wise["Ps_synonymous_polymorphisms"] + mk_table_gene_wise["Ds_synonymous_fixed"])).sum()

    denominator = (mk_table_gene_wise["Ps_synonymous_polymorphisms"] * mk_table_gene_wise["Dn_nonsynonymous_fixed"] /
                   (mk_table_gene_wise["Ps_synonymous_polymorphisms"] + mk_table_gene_wise["Ds_synonymous_fixed"])).sum()

    NITG_genome = numerator / denominator

    nitg_distribution, nitg_ci_low, nitg_ci_high, nitg_pval = bootstrap_nitg(mk_table_gene_wise)

    whole_genome_values = {
        "alpha": alpha_genome,
        "NI": NI_genome,
        "NI_pval": NI_genome_pval,
        "NITG": NITG_genome,
        "NITG_CI": (nitg_ci_low, nitg_ci_high),
        "NITG_pval": nitg_pval
    }

    return mk_table_gene_wise, whole_genome_values


##### Overall overview

In [ ]:
heterozygotes = ['0/1','0|1','1/0','1|0']

heterozygotes_count_focal = 0
total_count_focal = len(chrom15_vcf_focal)*len(chrom15_vcf_focal.columns[9:])

for col_name in chrom15_vcf_focal.columns[9:]:
    for index, value in chrom15_vcf_focal[col_name].items():
        # Extract the first element of the split result and convert to string
        genotype = str(value).split(':')[0]
        if genotype in heterozygotes:
            heterozygotes_count_focal += 1
    

heterozygotes_count_genome = 0
total_count_genome = len(chrom15_vcf)*len(chrom15_vcf.columns[9:])
    
for col_name in chrom15_vcf.columns[9:]:
    for index, value in chrom15_vcf[col_name].items():
        # Extract the first element of the split result and convert to string
        genotype = str(value).split(':')[0]
        if genotype in heterozygotes:
            heterozygotes_count_genome += 1

print(f"There are a total {heterozygotes_count_genome} heterozygotes sites from {total_count_genome} sites in chromosome 15.")
print(f"There are a total {heterozygotes_count_focal} heterozygotes sites from {total_count_focal} sites in the focal area of chromosome 15.")

# Perform Fisher's exact test

contingency_table = [[heterozygotes_count_genome, total_count_genome - heterozygotes_count_genome],
                     [heterozygotes_count_focal, total_count_focal - heterozygotes_count_focal]]


odds_ratio, p_value = fisher_exact(contingency_table)

if p_value > 0.5:
    print(f"There is no signifanct difference between the number of heterozygote sites in the focal area copared to the genome-wide estimate on chrom 15 (p={p_value}, Odds Ratio={odds_ratio}).")
else:
     print(f"There is a signifanct difference between the number of heterozygote sites in the focal area copared to the genome-wide estimate on chrom 15 (p={p_value}, Odds Ratio={odds_ratio}).")



#### Interpretation of Odds Ratio

In the context of the contingency table:

|                   | Heterozygotes | Non-Heterozygotes |
|-------------------|---------------|-------------------|
| **Genome**        | heterozygotes_count_genome | total_count_genome - heterozygotes_count_genome |
| **Focal**         | heterozygotes_count_focal | total_count_focal - heterozygotes_count_focal |


The odds ratio (OR) would be calculated as:

OR = (heterozygotes_count_genome * (total_count_focal - heterozygotes_count_focal)) / (heterozygotes_count_focal * (total_count_genome - heterozygotes_count_genome))

This odds ratio will tell you the relationship between being a heterozygote and whether the site is in the genome or focal region. The interpretation depends on the specific values, but here are some general cases:

- If OR = 1: No association between being a heterozygote and being in the genome or focal region.
- If OR > 1: Being a heterozygote is associated with a higher likelihood of being in the genome compared to the focal region.
- ***If OR < 1: Being a heterozygote is associated with a lower likelihood of being in the genome compared to the focal region.***

To assess the significance of the association, you typically perform a statistical test, such as the Fisher's exact test, to determine if the observed association is statistically significant.


##### Species wide overview

In [ ]:
def fisher_exact_per_species(species):
    heterozygotes = ['0/1','0|1','1/0','1|0']
    
    individual_count = 0
    
    individuals = []
    
    heterozygotes_count_focal = 0
    
    for col_name in chrom15_vcf_focal.columns[9:]:
        if species in col_name:
            individual_count += 1
            individuals.append(col_name)
            for index, value in chrom15_vcf_focal[col_name].items():
                # Extract the first element of the split result and convert to string
                genotype = str(value).split(':')[0]
                if genotype in heterozygotes:
                    heterozygotes_count_focal += 1
    

    heterozygotes_count_genome = 0
     
    
    for col_name in chrom15_vcf.columns[9:]:
        if species in col_name:
            for index, value in chrom15_vcf[col_name].items():
                # Extract the first element of the split result and convert to string
                genotype = str(value).split(':')[0]
                if genotype in heterozygotes:
                    heterozygotes_count_genome += 1
            
    total_count_focal = len(chrom15_vcf_focal)*individual_count
    total_count_genome = len(chrom15_vcf)*individual_count
    
    print(f"There are a total {heterozygotes_count_genome} heterozygotes sites from {total_count_genome} sites in chromosome 15 for species {species}.")
    print(f"There are a total {heterozygotes_count_focal} heterozygotes sites from {total_count_focal} sites in the focal area of chromosome 15 for species {species}.")     

    # Perform Fisher's exact test

    contingency_table = [[heterozygotes_count_genome, total_count_genome - heterozygotes_count_genome],
                     [heterozygotes_count_focal, total_count_focal - heterozygotes_count_focal]]


    odds_ratio, p_value = fisher_exact(contingency_table)

    #if p_value > 0.5:
    #    print(f"There is no signifanct difference between the number of heterozygote sites in the focal area compared to the genome-wide estimate on chrom 15 for species {species} (p={p_value}, Odds Ratio={odds_ratio}).")
    #else:
    #    print(f"There is a signifanct difference between the number of heterozygote sites in the focal area compared to the genome-wide estimate on chrom 15 for species {species} (p={p_value}, Odds Ratio={odds_ratio}).")

    sp_dict = {"species":species,
               "N":individual_count,
               "IDs": individuals,
               "het_count_genome":heterozygotes_count_genome,
               "total_count_genome":total_count_genome,
               "het_count_focal":heterozygotes_count_focal,
               "total_count_focal":total_count_focal,
               "p_value": p_value,
               "odds_ratio": odds_ratio}
    
    return(sp_dict)


In [ ]:
species_list = ["adriatica","hofrichteri","orientalis","pigra","willdenowi"]
heterozygotes_chrom15_summary_list = [fisher_exact_per_species(species) for species in species_list]
#heterozygotes_chrom15_summary_list

In [ ]:
heterozygotes_chrom15_summary_df = pd.DataFrame(heterozygotes_chrom15_summary_list)

# Plot the dot plot
plt.figure(figsize=(6, 6))
scatter = plt.scatter(heterozygotes_chrom15_summary_df['species'], 
                      heterozygotes_chrom15_summary_df['odds_ratio'], 
                      #s=heterozygotes_chrom15_summary_df['p_value'] * 10, 
                      c=heterozygotes_chrom15_summary_df['p_value'], cmap='viridis', alpha=0.7)

# Add labels and title
plt.xlabel('')
plt.ylabel('dds Ratio')

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('p-value')

# Show the plot
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

heterozygotes_chrom15_summary_df

#### Individual based heterozygosity

In [ ]:
def fisher_exact_per_individual(individual):
    heterozygotes = ['0/1','0|1','1/0','1|0']
    
    heterozygotes_count_focal = 0
    
    for col_name in chrom15_vcf_focal.columns[9:]:
        if individual == col_name:
            for index, value in chrom15_vcf_focal[col_name].items():
                # Extract the first element of the split result and convert to string
                genotype = str(value).split(':')[0]
                if genotype in heterozygotes:
                    heterozygotes_count_focal += 1
    

    heterozygotes_count_genome = 0
     
    
    for col_name in chrom15_vcf.columns[9:]:
        if individual == col_name:
            for index, value in chrom15_vcf[col_name].items():
                # Extract the first element of the split result and convert to string
                genotype = str(value).split(':')[0]
                if genotype in heterozygotes:
                    heterozygotes_count_genome += 1
            
    total_count_focal = len(chrom15_vcf_focal)
    total_count_genome = len(chrom15_vcf)
    
    #print(f"There are a total {heterozygotes_count_genome} heterozygotes sites from {total_count_genome} sites in chromosome 15 for species {individual}.")
    #print(f"There are a total {heterozygotes_count_focal} heterozygotes sites from {total_count_focal} sites in the focal area of chromosome 15 for species {individual}.")     

    # Perform Fisher's exact test

    contingency_table = [[heterozygotes_count_genome, total_count_genome - heterozygotes_count_genome],
                     [heterozygotes_count_focal, total_count_focal - heterozygotes_count_focal]]


    odds_ratio, p_value = fisher_exact(contingency_table)

    #if p_value > 0.5:
    #    print(f"There is no signifanct difference between the number of heterozygote sites in the focal area compared to the genome-wide estimate on chrom 15 for species {species} (p={p_value}, Odds Ratio={odds_ratio}).")
    #else:
    #    print(f"There is a signifanct difference between the number of heterozygote sites in the focal area compared to the genome-wide estimate on chrom 15 for species {species} (p={p_value}, Odds Ratio={odds_ratio}).")

    sp_ID = {"ID":individual,
               "species":individual.split("_")[0],
               "het_count_genome":heterozygotes_count_genome,
               "total_count_genome":total_count_genome,
               "het_count_focal":heterozygotes_count_focal,
               "total_count_focal":total_count_focal,
               "p_value": p_value,
               "odds_ratio": odds_ratio}
    
    return(sp_ID)



In [ ]:
ID_list = chrom15_vcf.columns[9:].to_list()
heterozygotes_ID_chrom15_summary_list = [fisher_exact_per_individual(i) for i in ID_list]
heterozygotes_chrom15_summary_ID_df = pd.DataFrame(heterozygotes_ID_chrom15_summary_list)

In [ ]:
# Add depth info to individuals
# Calculate mean values for each column
mean_values = chrom15_vcf_depth.mean()
# Create a new DataFrame with "ID" as row names and "mean_depth_chrom15" as the column name
chrom15_vcf_mean_depth_ID = pd.DataFrame({
    'ID': mean_values.index,
    'mean_depth_chrom15': mean_values.values
})


heterozygotes_chrom15_summary_ID_df_depth = pd.merge(heterozygotes_chrom15_summary_ID_df, chrom15_vcf_mean_depth_ID, on='ID')
heterozygotes_chrom15_summary_ID_df_depth

In [ ]:
# BEWARE LEGEND SIZES WRONG!!!
plt.figure(figsize=(6, 6))
scatter = plt.scatter(heterozygotes_chrom15_summary_ID_df_depth['species'], 
                      heterozygotes_chrom15_summary_ID_df_depth['odds_ratio'], 
                      s=heterozygotes_chrom15_summary_ID_df_depth['mean_depth_chrom15']*10, 
                      c=heterozygotes_chrom15_summary_ID_df_depth['p_value'], cmap='viridis', alpha=0.7)

# Add labels and title
plt.xlabel('')
plt.ylabel('Odds Ratio')

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('p-value')

# Create a legend for size
legend_sizes = [5, 10, 20, 28]
legend_labels = [f"{size}" for size in legend_sizes]
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=size, alpha=0.7) for size in legend_sizes]

# Use handles and labels directly in legend
legend = plt.legend(handles=legend_handles, labels=legend_labels,
                    title="Mean Depth", bbox_to_anchor=(1.3, 1), loc='upper left')  # Adjust the parameters for placement

# Show the plot
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()

# Adjust the layout to make more room for the legend outside the plot
plt.subplots_adjust(right=1.1)

plt.show()


In [ ]:
# Scatter plot
plt.figure(figsize=(8, 6))
scatter = plt.scatter(heterozygotes_chrom15_summary_ID_df_depth['mean_depth_chrom15'], 
                      heterozygotes_chrom15_summary_ID_df_depth['odds_ratio'], 
                      c=heterozygotes_chrom15_summary_ID_df_depth['p_value'], cmap='viridis', alpha=0.7)

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('p-value')

# Add labels and title
plt.xlabel('mean depth (chrom15)')
plt.ylabel('odds ratio')

# Show the plot
plt.grid(True)
plt.show()


In [ ]:
import seaborn as sns
import numpy as np

# Scatter plot
plt.figure(figsize=(8, 6))
scatter = sns.scatterplot(x='mean_depth_chrom15', y='odds_ratio', hue='species',
                          data=heterozygotes_chrom15_summary_ID_df_depth, palette='viridis', alpha=0.7)



# Add labels and title
plt.xlabel('mean depth (chrom15)')
plt.ylabel('odds ratio')

# Add linear regression lines for each category
for category, data in heterozygotes_chrom15_summary_ID_df_depth.groupby('species'):
    sns.regplot(x='mean_depth_chrom15', y='odds_ratio', data=data, scatter=False, ax=plt.gca())

# Show the plot
plt.grid(True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Species')
plt.show()


In [ ]:
import seaborn as sns
import numpy as np

# Scatter plot
plt.figure(figsize=(8, 6))
scatter = sns.scatterplot(x='mean_depth_chrom15', y='odds_ratio', hue='species',
                          data=heterozygotes_chrom15_summary_ID_df_depth,
                          alpha=0.7)

# Add linear regression lines for each category
for category, data in heterozygotes_chrom15_summary_ID_df_depth.groupby('species'):
    sns.regplot(x='mean_depth_chrom15', y='odds_ratio', data=data, scatter=False, ax=plt.gca())



# Add labels and title
plt.xlabel('mean depth (chrom15)')
plt.ylabel('odds ratio')


# Set the axis limits
plt.ylim(0, 4)

# Show the plot
plt.grid(True)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Species')
plt.show()


In [ ]:
par_score_wo0_chr15_focal_sign